<a href="https://colab.research.google.com/github/Eric-Chung-0511/Learning-Record/blob/main/Data%20Science%20Projects/VisionScout/(GitHub_Refactoring_YOLO_CLIP_Llama_Places365)_Vision_Scout_Model_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")  # 已新增至Secret
login(token=hf_token)

In [ ]:
!pip install opencv-python gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install yt-dlp requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.4 MB/s eta 0:00:00


In [ ]:
!pip install torch clip git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-cuy9m3y5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-cuy9m3y5
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s e

In [ ]:
!pip install ultralytics==8.3.128

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers accelerate bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %%writefile detection_model.py
from ultralytics import YOLO
from typing import Any, List, Dict, Optional
import torch
import numpy as np
import os

class DetectionModel:
    """Core detection model class for object detection using YOLOv8"""

    # Model information dictionary
    MODEL_INFO = {
        "yolov8n.pt": {
            "name": "YOLOv8n (Nano)",
            "description": "Fastest model with smallest size (3.2M parameters). Best for speed-critical applications.",
            "size_mb": 6,
            "inference_speed": "Very Fast"
        },
        "yolov8m.pt": {
            "name": "YOLOv8m (Medium)",
            "description": "Balanced model with good accuracy-speed tradeoff (25.9M parameters). Recommended for general use.",
            "size_mb": 25,
            "inference_speed": "Medium"
        },
        "yolov8x.pt": {
            "name": "YOLOv8x (XLarge)",
            "description": "Most accurate but slower model (68.2M parameters). Best for accuracy-critical applications.",
            "size_mb": 68,
            "inference_speed": "Slower"
        }
    }

    def __init__(self, model_name: str = 'yolov8m.pt', confidence: float = 0.25, iou: float = 0.25):
        """
        Initialize the detection model

        Args:
            model_name: Model name or path, default is yolov8m.pt
            confidence: Confidence threshold, default is 0.25
            iou: IoU threshold for non-maximum suppression, default is 0.45
        """
        self.model_name = model_name
        self.confidence = confidence
        self.iou = iou
        self.model = None
        self.class_names = {}
        self.is_model_loaded = False

        # Load model on initialization
        self._load_model()

    def _load_model(self):
        """Load the YOLO model"""
        try:
            print(f"Loading model: {self.model_name}")
            self.model = YOLO(self.model_name)
            self.class_names = self.model.names
            self.is_model_loaded = True
            print(f"Successfully loaded model: {self.model_name}")
            print(f"Number of classes the model can recognize: {len(self.class_names)}")
        except Exception as e:
            print(f"Error occurred when loading the model: {e}")
            self.is_model_loaded = False

    def change_model(self, new_model_name: str) -> bool:
        """
        Change the currently loaded model

        Args:
            new_model_name: Name of the new model to load

        Returns:
            bool: True if model changed successfully, False otherwise
        """
        if self.model_name == new_model_name and self.is_model_loaded:
            print(f"Model {new_model_name} is already loaded")
            return True

        print(f"Changing model from {self.model_name} to {new_model_name}")

        # Unload current model to free memory
        if self.model is not None:
            del self.model
            self.model = None

            # Clean GPU memory if available
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        # Update model name and load new model
        self.model_name = new_model_name
        self._load_model()

        return self.is_model_loaded

    def reload_model(self):
        """Reload the model (useful for changing model or after error)"""
        if self.model is not None:
            del self.model
            self.model = None

            # Clean GPU memory if available
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        self._load_model()

    def detect(self, image_input: Any) -> Optional[Any]:
        """
        Perform object detection on a single image

        Args:
            image_input: Image path (str), PIL Image, or numpy array

        Returns:
            Detection result object or None if error occurred
        """
        if self.model is None or not self.is_model_loaded:
            print("Model not found or not loaded. Attempting to reload...")
            self._load_model()
            if self.model is None or not self.is_model_loaded:
                print("Failed to load model. Cannot perform detection.")
                return None

        try:
            results = self.model(image_input, conf=self.confidence, iou=self.iou)
            return results[0]
        except Exception as e:
            print(f"Error occurred during detection: {e}")
            return None

    def get_class_names(self, class_id: int) -> str:
        """Get class name for a given class ID"""
        return self.class_names.get(class_id, "Unknown Class")

    def get_supported_classes(self) -> Dict[int, str]:
        """Get all supported classes as a dictionary of {id: class_name}"""
        return self.class_names

    @classmethod
    def get_available_models(cls) -> List[Dict]:
        """
        Get list of available models with their information

        Returns:
            List of dictionaries containing model information
        """
        models = []
        for model_file, info in cls.MODEL_INFO.items():
            models.append({
                "model_file": model_file,
                "name": info["name"],
                "description": info["description"],
                "size_mb": info["size_mb"],
                "inference_speed": info["inference_speed"]
            })
        return models

    @classmethod
    def get_model_description(cls, model_name: str) -> str:
        """Get description for a specific model"""
        if model_name in cls.MODEL_INFO:
            info = cls.MODEL_INFO[model_name]
            return f"{info['name']}: {info['description']} (Size: ~{info['size_mb']}MB, Speed: {info['inference_speed']})"
        return "Model information not available"

Writing detection_model.py


In [ ]:
# %%writefile color_mapper.py
import numpy as np
from typing import Dict, List, Tuple, Union, Any

class ColorMapper:
    """
    A class for consistent color mapping of object detection classes
    Provides color schemes for visualization in both RGB and hex formats
    """

    # Class categories for better organization
    CATEGORIES = {
        "person": [0],
        "vehicles": [1, 2, 3, 4, 5, 6, 7, 8],
        "traffic": [9, 10, 11, 12],
        "animals": [14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
        "outdoor": [13, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
        "sports": [34, 35, 36, 37, 38],
        "kitchen": [39, 40, 41, 42, 43, 44, 45],
        "food": [46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
        "furniture": [56, 57, 58, 59, 60, 61],
        "electronics": [62, 63, 64, 65, 66, 67, 68, 69, 70],
        "household": [71, 72, 73, 74, 75, 76, 77, 78, 79]
    }

    # Base colors for each category (in HSV for easier variation)
    # HSV:  Hue, Saturation, Value
    CATEGORY_COLORS = {
        "person": (0, 0.8, 0.9),       # Red
        "vehicles": (210, 0.8, 0.9),   # Blue
        "traffic": (45, 0.8, 0.9),     # Orange
        "animals": (120, 0.7, 0.8),    # Green
        "outdoor": (180, 0.7, 0.9),    # Cyan
        "sports": (270, 0.7, 0.8),     # Purple
        "kitchen": (30, 0.7, 0.9),     # Light Orange
        "food": (330, 0.7, 0.85),      # Pink
        "furniture": (150, 0.5, 0.85), # Light Green
        "electronics": (240, 0.6, 0.9), # Light Blue
        "household": (60, 0.6, 0.9)    # Yellow
    }

    def __init__(self):
        """Initialize the ColorMapper with COCO class mappings"""
        self.class_names = self._get_coco_classes()
        self.color_map = self._generate_color_map()

    def _get_coco_classes(self) -> Dict[int, str]:
        """Get the standard COCO class names with their IDs"""
        return {
            0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
            5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
            10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
            14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
            20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
            25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
            30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat',
            35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket',
            39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife',
            44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich',
            49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza',
            54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant',
            59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop',
            64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
            69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
            74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier',
            79: 'toothbrush'
        }

    def _hsv_to_rgb(self, h: float, s: float, v: float) -> Tuple[int, int, int]:
        """
        Convert HSV color to RGB

        Args:
            h: Hue (0-360)
            s: Saturation (0-1)
            v: Value (0-1)

        Returns:
            Tuple of (R, G, B) values (0-255)
        """
        h = h / 60
        i = int(h)
        f = h - i
        p = v * (1 - s)
        q = v * (1 - s * f)
        t = v * (1 - s * (1 - f))

        if i == 0:
            r, g, b = v, t, p
        elif i == 1:
            r, g, b = q, v, p
        elif i == 2:
            r, g, b = p, v, t
        elif i == 3:
            r, g, b = p, q, v
        elif i == 4:
            r, g, b = t, p, v
        else:
            r, g, b = v, p, q

        return (int(r * 255), int(g * 255), int(b * 255))

    def _rgb_to_hex(self, rgb: Tuple[int, int, int]) -> str:
        """
        Convert RGB color to hex color code

        Args:
            rgb: Tuple of (R, G, B) values (0-255)

        Returns:
            Hex color code (e.g. '#FF0000')
        """
        return f'#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}'

    def _find_category(self, class_id: int) -> str:
        """
        Find the category for a given class ID

        Args:
            class_id: Class ID (0-79)

        Returns:
            Category name
        """
        for category, ids in self.CATEGORIES.items():
            if class_id in ids:
                return category
        return "other"  # Fallback

    def _generate_color_map(self) -> Dict:
        """
        Generate a color map for all 80 COCO classes

        Returns:
            Dictionary mapping class IDs and names to color values
        """
        color_map = {
            'by_id': {},      # Map class ID to RGB and hex
            'by_name': {},    # Map class name to RGB and hex
            'categories': {}  # Map category to base color
        }

        # Generate colors for categories
        for category, hsv in self.CATEGORY_COLORS.items():
            rgb = self._hsv_to_rgb(hsv[0], hsv[1], hsv[2])
            hex_color = self._rgb_to_hex(rgb)
            color_map['categories'][category] = {
                'rgb': rgb,
                'hex': hex_color
            }

        # Generate variations for each class within a category
        for class_id, class_name in self.class_names.items():
            category = self._find_category(class_id)
            base_hsv = self.CATEGORY_COLORS.get(category, (0, 0, 0.8))  # Default gray

            # Slightly vary the hue and saturation within the category
            ids_in_category = self.CATEGORIES.get(category, [])
            if ids_in_category:
                position = ids_in_category.index(class_id) if class_id in ids_in_category else 0
                variation = position / max(1, len(ids_in_category) - 1)  # 0 to 1

                # Vary hue slightly (±15°) and saturation
                h_offset = 30 * variation - 15  # -15 to +15
                s_offset = 0.2 * variation  # 0 to 0.2

                h = (base_hsv[0] + h_offset) % 360
                s = min(1.0, base_hsv[1] + s_offset)
                v = base_hsv[2]
            else:
                h, s, v = base_hsv

            rgb = self._hsv_to_rgb(h, s, v)
            hex_color = self._rgb_to_hex(rgb)

            # Store in both mappings
            color_map['by_id'][class_id] = {
                'rgb': rgb,
                'hex': hex_color,
                'category': category
            }

            color_map['by_name'][class_name] = {
                'rgb': rgb,
                'hex': hex_color,
                'category': category
            }

        return color_map

    def get_color(self, class_identifier: Union[int, str], format: str = 'hex') -> Any:
        """
        Get color for a specific class

        Args:
            class_identifier: Class ID (int) or name (str)
            format: Color format ('hex', 'rgb', or 'bgr')

        Returns:
            Color in requested format
        """
        # Determine if identifier is an ID or name
        if isinstance(class_identifier, int):
            color_info = self.color_map['by_id'].get(class_identifier)
        else:
            color_info = self.color_map['by_name'].get(class_identifier)

        if not color_info:
            # Fallback color if not found
            return '#CCCCCC' if format == 'hex' else (204, 204, 204)

        if format == 'hex':
            return color_info['hex']
        elif format == 'rgb':
            return color_info['rgb']
        elif format == 'bgr':
            # Convert RGB to BGR for OpenCV
            r, g, b = color_info['rgb']
            return (b, g, r)
        else:
            return color_info['rgb']

    def get_all_colors(self, format: str = 'hex') -> Dict:
        """
        Get all colors in the specified format

        Args:
            format: Color format ('hex', 'rgb', or 'bgr')

        Returns:
            Dictionary mapping class names to colors
        """
        result = {}
        for class_id, class_name in self.class_names.items():
            result[class_name] = self.get_color(class_id, format)
        return result

    def get_category_colors(self, format: str = 'hex') -> Dict:
        """
        Get base colors for each category

        Args:
            format: Color format ('hex', 'rgb', or 'bgr')

        Returns:
            Dictionary mapping categories to colors
        """
        result = {}
        for category, color_info in self.color_map['categories'].items():
            if format == 'hex':
                result[category] = color_info['hex']
            elif format == 'bgr':
                r, g, b = color_info['rgb']
                result[category] = (b, g, r)
            else:
                result[category] = color_info['rgb']
        return result

    def get_category_for_class(self, class_identifier: Union[int, str]) -> str:
        """
        Get the category for a specific class

        Args:
            class_identifier: Class ID (int) or name (str)

        Returns:
            Category name
        """
        if isinstance(class_identifier, int):
            return self.color_map['by_id'].get(class_identifier, {}).get('category', 'other')
        else:
            return self.color_map['by_name'].get(class_identifier, {}).get('category', 'other')

Writing color_mapper.py


In [ ]:
# %%writefile visualization_helper.py
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from typing import Any, List, Dict, Tuple, Optional
import io
from PIL import Image

class VisualizationHelper:
    """Helper class for visualizing detection results"""

    @staticmethod
    def visualize_detection(image: Any, result: Any, color_mapper: Optional[Any] = None,
                            figsize: Tuple[int, int] = (12, 12),
                            return_pil: bool = False,
                            filter_classes: Optional[List[int]] = None) -> Optional[Image.Image]:
        """
        Visualize detection results on a single image
        Args:
            image: Image path or numpy array
            result: Detection result object
            color_mapper: ColorMapper instance for consistent colors
            figsize: Figure size
            return_pil: If True, returns a PIL Image object

        Returns:
            PIL Image if return_pil is True, otherwise displays the plot
        """
        if result is None:
            print('No data for visualization')
            return None

        # Read image if path is provided
        if isinstance(image, str):
            img = cv2.imread(image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            img = image
            if len(img.shape) == 3 and img.shape[2] == 3:
                # Check if BGR format (OpenCV) and convert to RGB if needed
                if isinstance(img, np.ndarray):
                    # Assuming BGR format from OpenCV
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Create figure
        fig, ax = plt.subplots(figsize=figsize)
        ax.imshow(img)

        # Get bounding boxes, classes and confidences
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()

        # Get class names
        names = result.names

        # Create a default color mapper if none is provided
        if color_mapper is None:
            # For backward compatibility, fallback to a simple color function
            from matplotlib import colormaps
            cmap = colormaps['tab10']
            def get_color(class_id):
                return cmap(class_id % 10)
        else:
            # Use the provided color mapper
            def get_color(class_id):
                hex_color = color_mapper.get_color(class_id)
                # Convert hex to RGB float values for matplotlib
                hex_color = hex_color.lstrip('#')
                return tuple(int(hex_color[i:i+2], 16) / 255 for i in (0, 2, 4)) + (1.0,)

        # Draw detection results
        for box, cls, conf in zip(boxes, classes, confs):
            x1, y1, x2, y2 = box
            cls_id = int(cls)

            if filter_classes and cls_id not in filter_classes:
                continue

            cls_name = names[cls_id]

            # Get color for this class
            box_color = get_color(cls_id)

            box_width = x2 - x1
            box_height = y2 - y1
            box_area = box_width * box_height

            # 根據框大小調整字體大小，但有限制
            adaptive_fontsize = max(10, min(14, int(10 + box_area / 10000)))


            ax.text(x1, y1 - 8, f'{cls_name}: {conf:.2f}',
                    color='white', fontsize=adaptive_fontsize, fontweight="bold",
                    bbox=dict(facecolor=box_color[:3], alpha=0.85, pad=3, boxstyle="round,pad=0.3"),
                    path_effects=[path_effects.withStroke(linewidth=1.5, foreground="black")])

            # Add bounding box
            ax.add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                    fill=False, edgecolor=box_color[:3], linewidth=2))

        ax.axis('off')
        # ax.set_title('Detection Result')
        plt.tight_layout()

        if return_pil:
            # Convert plot to PIL Image
            buf = io.BytesIO()
            fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
            buf.seek(0)
            pil_img = Image.open(buf)
            plt.close(fig)
            return pil_img
        else:
            plt.show()
            return None

    @staticmethod
    def create_summary(result: Any) -> Dict:
        """
        Create a summary of detection results

        Args:
            result: Detection result object

        Returns:
            Dictionary with detection summary statistics
        """
        if result is None:
            return {"error": "No detection result provided"}

        # Get classes and confidences
        classes = result.boxes.cls.cpu().numpy().astype(int)
        confidences = result.boxes.conf.cpu().numpy()
        names = result.names

        # Count detections by class
        class_counts = {}
        for cls, conf in zip(classes, confidences):
            cls_name = names[int(cls)]
            if cls_name not in class_counts:
                class_counts[cls_name] = {"count": 0, "confidences": []}

            class_counts[cls_name]["count"] += 1
            class_counts[cls_name]["confidences"].append(float(conf))

        # Calculate average confidence for each class
        for cls_name, stats in class_counts.items():
            if stats["confidences"]:
                stats["average_confidence"] = float(np.mean(stats["confidences"]))
                stats.pop("confidences")  # Remove detailed confidences list to keep summary concise

        # Prepare summary
        summary = {
            "total_objects": len(classes),
            "class_counts": class_counts,
            "unique_classes": len(class_counts)
        }

        return summary

Writing visualization_helper.py


In [ ]:
# %%writefile evaluation_metrics.py
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Any, Optional, Tuple

class EvaluationMetrics:
    """Class for computing detection metrics, generating statistics and visualization data"""

    @staticmethod
    def calculate_basic_stats(result: Any) -> Dict:
        """
        Calculate basic statistics for a single detection result

        Args:
            result: Detection result object

        Returns:
            Dictionary with basic statistics
        """
        if result is None:
            return {"error": "No detection result provided"}

        # Get classes and confidences
        classes = result.boxes.cls.cpu().numpy().astype(int)
        confidences = result.boxes.conf.cpu().numpy()
        names = result.names

        # Count by class
        class_counts = {}
        for cls, conf in zip(classes, confidences):
            cls_name = names[int(cls)]
            if cls_name not in class_counts:
                class_counts[cls_name] = {"count": 0, "total_confidence": 0, "confidences": []}

            class_counts[cls_name]["count"] += 1
            class_counts[cls_name]["total_confidence"] += float(conf)
            class_counts[cls_name]["confidences"].append(float(conf))

        # Calculate average confidence
        for cls_name, stats in class_counts.items():
            if stats["count"] > 0:
                stats["average_confidence"] = stats["total_confidence"] / stats["count"]
                stats["confidence_std"] = float(np.std(stats["confidences"])) if len(stats["confidences"]) > 1 else 0
                stats.pop("total_confidence")  # Remove intermediate calculation

        # Prepare summary
        stats = {
            "total_objects": len(classes),
            "class_statistics": class_counts,
            "average_confidence": float(np.mean(confidences)) if len(confidences) > 0 else 0
        }

        return stats

    @staticmethod
    def generate_visualization_data(result: Any, class_colors: Dict = None) -> Dict:
        """
        Generate structured data suitable for visualization

        Args:
            result: Detection result object
            class_colors: Dictionary mapping class names to color codes (optional)

        Returns:
            Dictionary with visualization-ready data
        """
        if result is None:
            return {"error": "No detection result provided"}

        # Get basic stats first
        stats = EvaluationMetrics.calculate_basic_stats(result)

        # Create visualization-specific data structure
        viz_data = {
            "total_objects": stats["total_objects"],
            "average_confidence": stats["average_confidence"],
            "class_data": []
        }

        # Sort classes by count (descending)
        sorted_classes = sorted(
            stats["class_statistics"].items(),
            key=lambda x: x[1]["count"],
            reverse=True
        )

        # Create class-specific visualization data
        for cls_name, cls_stats in sorted_classes:
            class_id = -1
            # Find the class ID based on the name
            for idx, name in result.names.items():
                if name == cls_name:
                    class_id = idx
                    break

            cls_data = {
                "name": cls_name,
                "class_id": class_id,
                "count": cls_stats["count"],
                "average_confidence": cls_stats.get("average_confidence", 0),
                "confidence_std": cls_stats.get("confidence_std", 0),
                "color": class_colors.get(cls_name, "#CCCCCC") if class_colors else "#CCCCCC"
            }

            viz_data["class_data"].append(cls_data)

        return viz_data

    @staticmethod
    def create_stats_plot(viz_data: Dict, figsize: Tuple[int, int] = (10, 7), max_classes: int = 30) -> plt.Figure:
        """
        Create a horizontal bar chart showing detection statistics

        Args:
            viz_data: Visualization data generated by generate_visualization_data
            figsize: Figure size (width, height) in inches
            max_classes: Maximum number of classes to display

        Returns:
            Matplotlib figure object
        """
        # Use the enhanced version
        return EvaluationMetrics.create_enhanced_stats_plot(viz_data, figsize, max_classes)

    @staticmethod
    def create_enhanced_stats_plot(viz_data: Dict, figsize: Tuple[int, int] = (10, 7), max_classes: int = 30) -> plt.Figure:
        """
        Create an enhanced horizontal bar chart with larger fonts and better styling

        Args:
            viz_data: Visualization data dictionary
            figsize: Figure size (width, height) in inches
            max_classes: Maximum number of classes to display

        Returns:
            Matplotlib figure with enhanced styling
        """
        if "error" in viz_data:
            # Create empty plot if error
            fig, ax = plt.subplots(figsize=figsize)
            ax.text(0.5, 0.5, viz_data["error"],
                    ha='center', va='center', fontsize=14)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
            return fig

        if "class_data" not in viz_data or not viz_data["class_data"]:
            # Create empty plot if no data
            fig, ax = plt.subplots(figsize=figsize)
            ax.text(0.5, 0.5, "No detection data available",
                    ha='center', va='center', fontsize=14)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
            return fig

        # Limit to max_classes
        class_data = viz_data["class_data"][:max_classes]

        # Extract data for plotting
        class_names = [item["name"] for item in class_data]
        counts = [item["count"] for item in class_data]
        colors = [item["color"] for item in class_data]

        # Create figure and horizontal bar chart with improved styling
        fig, ax = plt.subplots(figsize=figsize)

        # Set background color to white
        fig.patch.set_facecolor('white')
        ax.set_facecolor('white')

        y_pos = np.arange(len(class_names))

        # Create horizontal bars with class-specific colors
        bars = ax.barh(y_pos, counts, color=colors, alpha=0.8, height=0.6)

        # Add count values at end of each bar with larger font
        for i, bar in enumerate(bars):
            width = bar.get_width()
            conf = class_data[i]["average_confidence"]
            ax.text(width + 0.3, bar.get_y() + bar.get_height()/2,
                    f"{width:.0f} (conf: {conf:.2f})",
                    va='center', fontsize=12)

        # Customize axis and labels with larger fonts
        ax.set_yticks(y_pos)
        ax.set_yticklabels(class_names, fontsize=14)
        ax.invert_yaxis()  # Labels read top-to-bottom
        ax.set_xlabel('Count', fontsize=14)
        ax.set_title(f'Objects Detected: {viz_data["total_objects"]} Total',
                    fontsize=16, fontweight='bold')

        # Add grid for better readability
        ax.set_axisbelow(True)
        ax.grid(axis='x', linestyle='--', alpha=0.7, color='#E5E7EB')

        # Increase tick label font size
        ax.tick_params(axis='both', which='major', labelsize=12)

        # Add detection summary as a text box with improved styling
        summary_text = (
            f"Total Objects: {viz_data['total_objects']}\n"
            f"Average Confidence: {viz_data['average_confidence']:.2f}\n"
            f"Unique Classes: {len(viz_data['class_data'])}"
        )
        plt.figtext(0.02, 0.02, summary_text, fontsize=12,
                bbox=dict(facecolor='white', alpha=0.9, boxstyle='round,pad=0.5',
                            edgecolor='#E5E7EB'))

        plt.tight_layout()
        return fig

    @staticmethod
    def format_detection_summary(viz_data: Dict) -> str:
        if "error" in viz_data:
            return viz_data["error"]

        if "total_objects" not in viz_data:
            return "No detection data available."

        total_objects = viz_data["total_objects"]
        avg_confidence = viz_data["average_confidence"]

        lines = [
            f"Detected {total_objects} objects.",
            f"Average confidence: {avg_confidence:.2f}",
            "Objects by class:"
        ]

        if "class_data" in viz_data and viz_data["class_data"]:
            for item in viz_data["class_data"]:
                count = item['count']
                item_text = "item" if count == 1 else "items"
                lines.append(f"• {item['name']}: {count} {item_text} (Confidence: {item['average_confidence']:.2f})")
        else:
            lines.append("No class information available.")

        return "\n".join(lines)

    @staticmethod
    def calculate_distance_metrics(result: Any) -> Dict:
        """
        Calculate distance-related metrics for detected objects

        Args:
            result: Detection result object

        Returns:
            Dictionary with distance metrics
        """
        if result is None:
            return {"error": "No detection result provided"}

        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy().astype(int)
        names = result.names

        # Initialize metrics
        metrics = {
            "proximity": {},  # Classes that appear close to each other
            "spatial_distribution": {},  # Distribution across the image
            "size_distribution": {}  # Size distribution of objects
        }

        # Calculate image dimensions (assuming normalized coordinates or extract from result)
        img_width, img_height = 1, 1
        if hasattr(result, "orig_shape"):
            img_height, img_width = result.orig_shape[:2]

        # Calculate bounding box areas and centers
        areas = []
        centers = []
        class_names = []

        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = box
            width, height = x2 - x1, y2 - y1
            area = width * height
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2

            areas.append(area)
            centers.append((center_x, center_y))
            class_names.append(names[int(cls)])

        # Calculate spatial distribution
        if centers:
            x_coords = [c[0] for c in centers]
            y_coords = [c[1] for c in centers]

            metrics["spatial_distribution"] = {
                "x_mean": float(np.mean(x_coords)) / img_width,
                "y_mean": float(np.mean(y_coords)) / img_height,
                "x_std": float(np.std(x_coords)) / img_width,
                "y_std": float(np.std(y_coords)) / img_height
            }

        # Calculate size distribution
        if areas:
            metrics["size_distribution"] = {
                "mean_area": float(np.mean(areas)) / (img_width * img_height),
                "std_area": float(np.std(areas)) / (img_width * img_height),
                "min_area": float(np.min(areas)) / (img_width * img_height),
                "max_area": float(np.max(areas)) / (img_width * img_height)
            }

        # Calculate proximity between different classes
        class_centers = {}
        for cls_name, center in zip(class_names, centers):
            if cls_name not in class_centers:
                class_centers[cls_name] = []
            class_centers[cls_name].append(center)

        # Find classes that appear close to each other
        proximity_pairs = []
        for i, cls1 in enumerate(class_centers.keys()):
            for j, cls2 in enumerate(class_centers.keys()):
                if i >= j:  # Avoid duplicate pairs and self-comparison
                    continue

                # Calculate minimum distance between any two objects of these classes
                min_distance = float('inf')
                for center1 in class_centers[cls1]:
                    for center2 in class_centers[cls2]:
                        dist = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
                        min_distance = min(min_distance, dist)

                # Normalize by image diagonal
                img_diagonal = np.sqrt(img_width**2 + img_height**2)
                norm_distance = min_distance / img_diagonal

                proximity_pairs.append({
                    "class1": cls1,
                    "class2": cls2,
                    "distance": float(norm_distance)
                })

        # Sort by distance and keep the closest pairs
        proximity_pairs.sort(key=lambda x: x["distance"])
        metrics["proximity"] = proximity_pairs[:5]  # Keep top 5 closest pairs

        return metrics

Writing evaluation_metrics.py


In [ ]:
# %%writefile style.py

class Style:

    @staticmethod
    def get_css():

        css = """
        /* Base styles and typography */
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #f0f9ff, #e1f5fe);
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            min-height: 100vh;
        }

        /* Typography improvements */
        h1, h2, h3, h4, h5, h6, p, span, div, label, button {
            font-family: Arial, sans-serif;
        }

        /* Container styling */
        .gradio-container {
            max-width: 1200px !important;
            margin: auto !important;
            padding: 1rem;
            width: 100%;
        }

        /* Header area styling with gradient background */
        .app-header {
            text-align: center;
            margin-bottom: 2rem;
            background: linear-gradient(135deg, #f8f9fa, #e9ecef);
            padding: 1.5rem;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.05);
            width: 100%;
        }

        .app-title {
            color: #2D3748;
            font-size: 2.5rem;
            margin-bottom: 0.5rem;
            background: linear-gradient(90deg, #38b2ac, #4299e1);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            font-weight: bold;
        }

        .app-subtitle {
            color: #4A5568;
            font-size: 1.2rem;
            font-weight: normal;
            margin-top: 0.25rem;
        }

        .app-divider {
            width: 80px;
            height: 3px;
            background: linear-gradient(90deg, #38b2ac, #4299e1);
            margin: 1rem auto;
        }

        /* Panel styling - gradient background */
        .input-panel, .output-panel {
            background: white;
            border-radius: 10px;
            padding: 1.5rem;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.08);
            margin: 0 auto 1rem auto;
        }

        /* 修改輸出面板確保內容能夠完整顯示 */
        .output-panel {
            display: flex;
            flex-direction: column;
            width: 100%;
            padding: 0 !important;
        }

        /* 確保輸出面板內的元素寬度可以適應面板 */
        .output-panel > * {
            width: 100%;
        }

        /* How-to-use section with gradient background */
        .how-to-use {
            background: linear-gradient(135deg, #f8fafc, #e8f4fd);
            border-radius: 10px;
            padding: 1.5rem;
            margin-top: 1rem;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05);
            color: #2d3748;
        }

        /* Detection button styling */
        .detect-btn {
            background: linear-gradient(90deg, #38b2ac, #4299e1) !important;
            color: white !important;
            border: none !important;
            border-radius: 8px !important;
            transition: transform 0.3s, box-shadow 0.3s !important;
            font-weight: bold !important;
            letter-spacing: 0.5px !important;
            padding: 0.75rem 1.5rem !important;
            width: 100%;
            margin: 1rem auto !important;
            font-family: Arial, sans-serif !important;
        }

        .detect-btn:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2) !important;
        }

        .detect-btn:active {
            transform: translateY(1px) !important;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2) !important;
        }

        /* JSON display improvements */
        .json-display {
            width: 98% !important;
            margin: 0.5rem auto 1.5rem auto !important;
            padding: 1rem !important;
            border-radius: 8px !important;
            background-color: white !important;
            border: 1px solid #E2E8F0 !important;
            box-shadow: inset 0 1px 2px rgba(0, 0, 0, 0.05) !important;
        }

        .json-key {
            color: #e53e3e;
        }

        .json-value {
            color: #2b6cb0;
        }

        .json-string {
            color: #38a169;
        }

        /* Chart/plot styling improvements */
        .plot-container {
            background: white;
            border-radius: 8px;
            padding: 0.5rem;
            box-shadow: 0 2px 6px rgba(0, 0, 0, 0.05);
        }

        /* Larger font for plots */
        .plot-container text {
            font-family: Arial, sans-serif !important;
            font-size: 14px !important;
        }

        /* Title styling for charts */
        .plot-title {
            font-family: Arial, sans-serif !important;
            font-size: 16px !important;
            font-weight: bold !important;
        }

        /* Tab styling with subtle gradient */
        .tabs {
            width: 100%;
            display: flex;
            justify-content: center;
        }

        .tabs > div:first-child {
            background: linear-gradient(to right, #f8fafc, #e8f4fd) !important;
            border-radius: 8px 8px 0 0;
        }

        /* Tab content styling - 確保內容區域有足夠寬度 */
        .tab-content {
            width: 100% !important;
            box-sizing: border-box !important;
            padding: 0 !important;
        }

        /* Footer styling with gradient background */
        .footer {
            text-align: center;
            margin-top: 2rem;
            font-size: 0.9rem;
            color: #4A5568;
            padding: 1rem;
            background: linear-gradient(135deg, #f8f9fa, #e1effe);
            border-radius: 10px;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.05);
            width: 100%;
        }

        /* Ensure centering works for all elements */
        .container, .gr-container, .gr-row, .gr-col {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            width: 100%;
        }

        /* 統一文本框樣式，確保寬度一致 */
        .gr-textbox, .gr-textarea, .gr-text-input {
            width: 100% !important;
            max-width: 100% !important;
            min-width: 100% !important;
            box-sizing: border-box !important;
        }

        /* 確保文本區域可以適應容器寬度 */
        textarea.gr-textarea, .gr-textbox textarea, .gr-text-input textarea {
            width: 100% !important;
            max-width: 100% !important;
            min-width: 100% !important;
            box-sizing: border-box !important;
            padding: 16px !important;
            font-family: 'Arial', sans-serif !important;
            font-size: 14px !important;
            line-height: 1.6 !important;
            white-space: pre-wrap !important;
            word-wrap: break-word !important;
            word-break: normal !important;
        }

        /* 特別針對場景描述文本框樣式增強 */
        #scene-description-text, #detection-details {
            width: 100% !important;
            min-width: 100% !important;
            box-sizing: border-box !important;
            padding: 16px !important;
            line-height: 1.8 !important;
            white-space: pre-wrap !important;
            word-wrap: break-word !important;
            border-radius: 8px !important;
            min-height: 250px !important;
            overflow-y: auto !important;
            border: 1px solid #e2e8f0 !important;
            background-color: white !important;
            display: block !important;
            font-family: 'Arial', sans-serif !important;
            font-size: 14px !important;
            margin: 0 !important;
        }

        /* 針對場景描述容器的樣式 */
        .scene-description-container {
            width: 100% !important;
            max-width: 100% !important;
            box-sizing: border-box !important;
            padding: 0 !important;
            margin: 0 !important;
        }

        /* Scene Understanding Tab 特定樣式 */
        .scene-understanding-tab .result-details-box {
            display: flex !important;
            flex-direction: column !important;
            align-items: stretch !important;
            width: 100% !important;
            box-sizing: border-box !important;
            padding: 0 !important;
        }

        /* 場景分析描述區域樣式 */
        .scene-description-box {
            background-color: #f8f9fa !important;
            border: 1px solid #e2e8f0 !important;
            border-radius: 8px !important;
            padding: 15px !important;
            margin: 10px 0 20px 0 !important;
            box-shadow: 0 1px 3px rgba(0,0,0,0.05) !important;
            font-family: Arial, sans-serif !important;
            line-height: 1.7 !important;
            color: #2D3748 !important;
            font-size: 16px !important;
            width: 100% !important;
            box-sizing: border-box !important;
        }

        #scene_analysis_description_text {
            background-color: #f0f0f0 !important; /* 淺灰色背景 */
            padding: 15px !important;             /* 內邊距，讓文字和邊框有點空間 */
            border-radius: 8px !important;        /* 圓角 */
            margin: 10px 0 20px 0 !important;     /* 其他元素的間距，特別是上下的part */
            display: block !important;
            width: 100% !important;
            box-sizing: border-box !important;
        }

        #scene_analysis_description_text p {
            margin: 0 !important;
            color: #2D3748 !important; /* 確保文字顏色 */
            font-family: Arial, sans-serif !important;
            font-size: 16px !important; /* 你可以調整文字大小 */
            line-height: 1.7 !important;
        }

        /* 結果容器樣式 */
        .result-container {
            width: 100% !important;
            padding: 1rem !important;
            border-radius: 8px !important;
            border: 1px solid #E2E8F0 !important;
            margin-bottom: 1.5rem !important;
            background-color: #F8FAFC !important;
            box-sizing: border-box !important;
        }

        /* 結果文本框的樣式 */
        .wide-result-text {
            width: 100% !important;
            min-width: 100% !important;
            box-sizing: border-box !important;
            padding: 0 !important;
            margin: 0 !important;
        }

        /* 片段標題樣式 */
        .section-heading {
            font-size: 1.25rem !important;
            font-weight: 600 !important;
            color: #2D3748 !important;
            margin: 1rem auto !important;
            padding: 0.75rem 1rem !important;
            background: linear-gradient(to right, #e6f3fc, #f0f9ff) !important;
            border-radius: 8px !important;
            width: 98% !important;
            display: inline-block !important;
            box-sizing: border-box !important;
            text-align: center !important;
            overflow: visible !important;
            line-height: 1.5 !important;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1) !important;
        }

        /* JSON 顯示區域樣式 */
        .json-box {
            width: 100% !important;
            min-height: 200px !important;
            overflow-y: auto !important;
            background: white !important;
            padding: 1rem !important;
            border-radius: 8px !important;
            box-shadow: inset 0 0 6px rgba(0, 0, 0, 0.1) !important;
            font-family: monospace !important;
            box-sizing: border-box !important;
        }

        /* 欄佈局調整 */
        .plot-column, .stats-column {
            display: flex;
            flex-direction: column;
            padding: 1rem;
            box-sizing: border-box !important;
            width: 100% !important;
        }

        /* statistics plot */
        .large-plot-container {
            width: 100% !important;
            min-height: 400px !important;
            box-sizing: border-box !important;
        }

        /* 增強 JSON 顯示 */
        .enhanced-json-display {
            background: white !important;
            border-radius: 8px !important;
            padding: 1rem !important;
            box-shadow: inset 0 0 6px rgba(0, 0, 0, 0.1) !important;
            width: 100% !important;
            min-height: 300px !important;
            max-height: 500px !important;
            overflow-y: auto !important;
            font-family: monospace !important;
            box-sizing: border-box !important;
        }

        /* 確保全寬元素真正占滿整個寬度 */
        .full-width-element {
            width: 100% !important;
            max-width: 100% !important;
            box-sizing: border-box !important;
        }

        /* Video summary HTML 容器與內容樣式 */
        #video-summary-html-output {
            width: 100% !important;
            box-sizing: border-box !important;
            padding: 0 !important;
            margin: 0 !important;
        }

        .video-summary-content-wrapper {
            width: 100% !important;
            padding: 16px !important;
            line-height: 1.8 !important;
            white-space: pre-wrap !important;
            word-wrap: break-word !important;
            border-radius: 8px !important;
            min-height: 250px !important;
            max-height: 600px !important;
            overflow-y: auto !important;
            border: 1px solid #e2e8f0 !important;
            background-color: white !important;
            display: block !important;
            font-family: 'Arial', sans-serif !important;
            font-size: 14px !important;
            margin: 0 !important;
        }

        .video-summary-content-wrapper pre {
            white-space: pre-wrap !important;
            word-wrap: break-word !important;
            margin: 0 !important;
            padding: 0 !important;
            font-family: 'Arial', sans-serif !important;
            font-size: 14px !important;
            line-height: 1.8 !important;
            color: #2D3748 !important;
        }

        /* 視頻結果面板相關樣式 */
        .video-result-panel {
            padding: 1rem !important;
            background: white !important;
            border-radius: 10px !important;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.08) !important;
        }

        .video-output-container {
            width: 100% !important;
            margin-bottom: 1.5rem !important;
            border-radius: 8px !important;
            overflow: hidden !important;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1) !important;
        }

        /* 視頻統計資料顯示增強 */
        .video-stats-display {
            background: white !important;
            border-radius: 8px !important;
            padding: 1rem !important;
            box-shadow: inset 0 0 6px rgba(0, 0, 0, 0.1) !important;
            width: 100% !important;
            min-height: 200px !important;
            max-height: 400px !important;
            overflow-y: auto !important;
            font-family: monospace !important;
            box-sizing: border-box !important;
            color: #2D3748 !important;
        }

        .custom-video-url-input {
            width: 100% !important;
        }

        .custom-video-url-input textarea {
            width: 100% !important;
            min-height: 120px !important;
            padding: 15px !important;
            font-size: 16px !important;
            line-height: 1.6 !important;
            background-color: #F7FAFC !important;
            box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1) !important;
            border: 2px solid #CBD5E0 !important;
            border-radius: 8px !important;
        }

        .custom-video-url-input textarea:focus {
            border-color: #4299E1 !important;
            box-shadow: 0 0 0 3px rgba(66, 153, 225, 0.2) !important;
        }

        /* 輸入框容器100%寬度 */
        .custom-video-url-input > div {
            width: 100% !important;
            max-width: 100% !important;
        }

        /* LLM 增強描述樣式 */
        #llm_enhanced_description_text {
            padding: 15px !important;
            background-color: #ffffff !important;
            border-radius: 8px !important;
            border: 1px solid #e2e8f0 !important;
            margin-bottom: 20px !important;
            box-shadow: 0 1px 3px rgba(0,0,0,0.05) !important;
            font-family: Arial, sans-serif !important;
            line-height: 1.7 !important;
            color: #2D3748 !important;
            font-size: 16px !important;
            width: 100% !important;
            box-sizing: border-box !important;
            min-height: 200px !important;
        }

        /* 原始描述折疊區域樣式 */
        #original_scene_analysis_accordion {
            margin-top: 10px !important;
            margin-bottom: 20px !important;
            background-color: #f8f9fa !important;
            border-radius: 8px !important;
            border: 1px solid #e2e8f0 !important;
        }

        /* 確保折疊區域內容與頁面樣式協調 */
        #original_scene_analysis_accordion > div:nth-child(2) {
            padding: 15px !important;
        }

        /* 動畫效果, 增加互動感 */
        @keyframes fadeIn {
            from { opacity: 0; }
            to { opacity: 1; }
        }

        .video-result-panel > * {
            animation: fadeIn 0.5s ease-in-out;
        }

        /* 響應式調整 */
        @media (max-width: 768px) {
            .app-title {
                font-size: 2rem;
            }

            .app-subtitle {
                font-size: 1rem;
            }

            .gradio-container {
                padding: 0.5rem;
            }

            /* 在小螢幕上調整文本區域的高度 */
            #scene-description-text, #detection-details {
                min-height: 150px !important;
            }
        }

        """
        return css

Writing style.py


In [ ]:
# %%writefile scene_type.py

SCENE_TYPES = {
    "living_room": {
        "name": "Living Room",
        "required_objects": [57, 62],  # couch, tv
        "optional_objects": [56, 60, 73, 75],  # chair, dining table, book, vase
        "minimum_required": 2,
        "description": "A living room area with furniture for relaxation and entertainment"
    },
    "bedroom": {
        "name": "Bedroom",
        "required_objects": [59],  # bed
        "optional_objects": [56, 60, 73, 74, 75],  # chair, dining table, book, clock, vase
        "minimum_required": 1,
        "description": "A bedroom with sleeping furniture"
    },
    "dining_area": {
        "name": "Dining Area",
        "required_objects": [60],  # dining table
        "optional_objects": [56, 39, 41, 42, 43, 44, 45],  # chair, bottle, cup, fork, knife, spoon, bowl
        "minimum_required": 1,
        "description": "A dining area for meals"
    },
    "kitchen": {
        "name": "Kitchen",
        "required_objects": [72, 68, 69, 71],  # refrigerator, microwave, oven, sink
        "optional_objects": [39, 41, 42, 43, 44, 45],  # bottle, cup, fork, knife, spoon, bowl
        "minimum_required": 1,
        "description": "A kitchen area for food preparation"
    },
    "office_workspace": {
        "name": "Office Workspace",
        "required_objects": [56, 63, 66, 64, 73],  # chair, laptop, keyboard, mouse, book
        "optional_objects": [60, 74, 75, 67],  # dining table, clock, vase, cell phone
        "minimum_required": 2,
        "description": "A workspace with computer equipment for office work"
    },
    "meeting_room": {
        "name": "Meeting Room",
        "required_objects": [56, 60],  # chair, dining table
        "optional_objects": [63, 62, 67],  # laptop, tv, cell phone
        "minimum_required": 2,
        "description": "A room set up for meetings with multiple seating"
    },
    "city_street": {
        "name": "City Street",
        "required_objects": [0, 1, 2, 3, 5, 7, 9],  # person, bicycle, car, motorcycle, bus, truck, traffic light
        "optional_objects": [10, 11, 12, 24, 25, 26, 28],  # fire hydrant, stop sign, parking meter, backpack, umbrella, handbag, suitcase
        "minimum_required": 2,
        "description": "A city street with traffic and pedestrians"
    },
    "parking_lot": {
        "name": "Parking Lot",
        "required_objects": [2, 3, 5, 7],  # car, motorcycle, bus, truck
        "optional_objects": [0, 11, 12],  # person, stop sign, parking meter
        "minimum_required": 3,
        "description": "A parking area with multiple vehicles"
    },
    "park_area": {
        "name": "Park or Recreation Area",
        "required_objects": [0, 13],  # person, bench
        "optional_objects": [1, 14, 16, 25, 33],  # bicycle, bird, dog, umbrella, kite
        "minimum_required": 2,
        "description": "An outdoor recreational area for leisure activities"
    },
    "retail_store": {
        "name": "Retail Store",
        "required_objects": [0, 24, 26, 28],  # person, backpack, handbag, suitcase
        "optional_objects": [39, 45, 67],  # bottle, bowl, cell phone
        "minimum_required": 2,
        "description": "A retail environment with shoppers and merchandise"
    },
    "supermarket": {
        "name": "Supermarket",
        "required_objects": [0, 24, 39, 46, 47, 49],  # person, backpack, bottle, banana, apple, orange
        "optional_objects": [26, 37, 45, 48, 51, 52, 53, 54, 55],  # handbag, surfboard, bowl, sandwich, carrot, hot dog, pizza, donut, cake
        "minimum_required": 3,
        "description": "A supermarket with food items and shoppers"
    },
    "classroom": {
        "name": "Classroom",
        "required_objects": [56, 60, 73],  # chair, dining table, book
        "optional_objects": [63, 66, 67],  # laptop, keyboard, cell phone
        "minimum_required": 2,
        "description": "A classroom environment set up for educational activities"
    },
    "conference_room": {
        "name": "Conference Room",
        "required_objects": [56, 60, 63],  # chair, dining table, laptop
        "optional_objects": [62, 67, 73],  # tv, cell phone, book
        "minimum_required": 2,
        "description": "A conference room designed for meetings and presentations"
    },
    "cafe": {
        "name": "Cafe",
        "required_objects": [56, 60, 41],  # chair, dining table, cup
        "optional_objects": [39, 40, 63, 67, 73],  # bottle, wine glass, laptop, cell phone, book
        "minimum_required": 2,
        "description": "A cafe setting with seating and beverages"
    },
    "library": {
        "name": "Library",
        "required_objects": [56, 60, 73],  # chair, dining table, book
        "optional_objects": [63, 67, 75],  # laptop, cell phone, vase
        "minimum_required": 2,
        "description": "A library with books and reading areas"
    },
    "gym": {
        "name": "Gym",
        "required_objects": [0, 32],  # person, sports ball
        "optional_objects": [24, 25, 28, 38],  # backpack, umbrella, suitcase, tennis racket
        "minimum_required": 1,
        "description": "A gym or fitness area for physical activities"
    },
    "beach": {
        "name": "Beach",
        "required_objects": [0, 25, 29, 33, 37],  # person, umbrella, frisbee, kite, surfboard
        "optional_objects": [1, 24, 26, 38],  # bicycle, backpack, handbag, tennis racket
        "minimum_required": 2,
        "description": "A beach area with people and recreational items"
    },
    "restaurant": {
        "name": "Restaurant",
        "required_objects": [56, 60, 41, 42, 43, 44, 45],  # chair, dining table, cup, fork, knife, spoon, bowl
        "optional_objects": [39, 40, 48, 49, 50, 51, 52, 53, 54, 55],  # bottle, wine glass, sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake
        "minimum_required": 3,
        "description": "A restaurant setting for dining with tables and eating utensils"
    },
    "train_station": {
        "name": "Train Station",
        "required_objects": [0, 6],  # person, train
        "optional_objects": [1, 2, 24, 28, 67],  # bicycle, car, backpack, suitcase, cell phone
        "minimum_required": 1,
        "description": "A train station with train and passengers"
    },
    "airport": {
        "name": "Airport",
        "required_objects": [0, 4, 28],  # person, airplane, suitcase
        "optional_objects": [24, 25, 26, 67],  # backpack, umbrella, handbag, cell phone
        "minimum_required": 2,
        "description": "An airport with planes and travelers carrying luggage"
    },
      "upscale_dining": {
        "name": "Upscale Dining Area",
        "required_objects": [56, 60, 40, 41],  # chair, dining table, wine glass, cup
        "optional_objects": [39, 42, 43, 44, 45, 62, 75],  # bottle, fork, knife, spoon, bowl, tv, vase
        "minimum_required": 2,
        "description": "An elegantly designed dining space with refined furniture and decorative elements"
    },
    "asian_commercial_street": {
        "name": "Asian Commercial Street",
        "required_objects": [0, 67],  # person, cell phone
        "optional_objects": [1, 2, 3, 24, 25, 26, 28],  # bicycle, car, motorcycle, backpack, umbrella, handbag, suitcase
        "minimum_required": 1,
        "description": "A bustling commercial street with shops, signage, and pedestrians in an Asian urban setting"
    },
    "financial_district": {
        "name": "Financial District",
        "required_objects": [2, 5, 7, 9],  # car, bus, truck, traffic light
        "optional_objects": [0, 1, 3, 8],  # person, bicycle, motorcycle, boat
        "minimum_required": 2,
        "description": "A major thoroughfare in a business district with high-rise buildings and traffic"
    },
    "urban_intersection": {
        "name": "Urban Intersection",
        "required_objects": [0, 9],  # person, traffic light
        "optional_objects": [1, 2, 3, 5, 7],  # bicycle, car, motorcycle, bus, truck
        "minimum_required": 1,
        "description": "A busy urban crossroad with pedestrian crossings and multiple traffic flows"
    },
    "transit_hub": {
        "name": "Transit Hub",
        "required_objects": [0, 5, 6, 7],  # person, bus, train, truck
        "optional_objects": [1, 2, 3, 9, 24, 28],  # bicycle, car, motorcycle, traffic light, backpack, suitcase
        "minimum_required": 2,
        "description": "A transportation center where multiple modes of transit converge"
    },
    "shopping_district": {
        "name": "Shopping District",
        "required_objects": [0, 24, 26],  # person, backpack, handbag
        "optional_objects": [1, 2, 3, 25, 27, 28, 39, 67],  # bicycle, car, motorcycle, umbrella, tie, suitcase, bottle, cell phone
        "minimum_required": 2,
        "description": "A retail-focused area with shops, pedestrians, and commercial activity"
    },
     "bus_stop": {
        "name": "Bus Stop",
        "required_objects": [0, 5],  # person, bus
        "optional_objects": [1, 2, 7, 24],  # bicycle, car, truck, backpack
        "minimum_required": 2,
        "description": "A roadside bus stop with waiting passengers and buses"
    },
    "bus_station": {
        "name": "Bus Station",
        "required_objects": [0, 5, 7],  # person, bus, truck
        "optional_objects": [24, 28, 67],  # backpack, suitcase, cell phone
        "minimum_required": 2,
        "description": "A bus terminal with multiple buses and travelers"
    },
    "zoo": {
        "name": "Zoo",
        "required_objects": [20, 22, 23],  # elephant, zebra, giraffe
        "optional_objects": [0, 14, 16],  # person, bird, dog
        "minimum_required": 2,
        "description": "A zoo environment featuring large animal exhibits and visitors"
    },
    "harbor": {
        "name": "Harbor",
        "required_objects": [8],  # boat
        "optional_objects": [0, 2, 3, 39],  # person, car, motorcycle, bottle
        "minimum_required": 1,
        "description": "A harbor area with boats docked and surrounding traffic"
    },
    "playground": {
        "name": "Playground",
        "required_objects": [0, 32],  # person, sports ball
        "optional_objects": [33, 24, 1],  # kite, backpack, bicycle
        "minimum_required": 1,
        "description": "An outdoor playground with people playing sports and games"
    },
    "sports_field": {
        "name": "Sports Field",
        "required_objects": [32],  # sports ball
        "optional_objects": [38, 34, 35],  # tennis racket, baseball bat, baseball glove
        "minimum_required": 1,
        "description": "A sports field set up for various ball games"
    },
     "narrow_commercial_alley": {
        "name": "Narrow Commercial Alley",
        "required_objects": [0, 3],  # person, motorcycle
        "optional_objects": [2, 7, 24, 26],  # car, truck, backpack, handbag
        "minimum_required": 2,
        "description": "A tight urban alley lined with shops, with pedestrians and light vehicles"
    },
    "daytime_shopping_street": {
        "name": "Daytime Shopping Street",
        "required_objects": [0, 2],  # person, car
        "optional_objects": [1, 3, 24, 26],  # bicycle, motorcycle, backpack, handbag
        "minimum_required": 2,
        "description": "A busy pedestrian street during daytime, featuring shops, vehicles, and shoppers"
    },
    "urban_pedestrian_crossing": {
        "name": "Urban Pedestrian Crossing",
        "required_objects": [0, 9],  # person, traffic light
        "optional_objects": [2, 3, 5],  # car, motorcycle, bus
        "minimum_required": 1,
        "description": "A city street crossing with pedestrians and traffic signals"
    },
    "aerial_view_intersection": {
    "name": "Aerial View Intersection",
    "required_objects": [0, 9],  # person, traffic light
    "optional_objects": [1, 2, 3, 5, 7],  # bicycle, car, motorcycle, bus, truck
    "minimum_required": 1,
    "description": "An intersection viewed from above, showing crossing patterns and pedestrian movement"
    },
    "aerial_view_commercial_area": {
        "name": "Aerial View Commercial Area",
        "required_objects": [0, 2],  # person, car
        "optional_objects": [1, 3, 5, 7, 24, 26],  # bicycle, motorcycle, bus, truck, backpack, handbag
        "minimum_required": 2,
        "description": "A commercial or shopping area viewed from above showing pedestrians and urban layout"
    },
    "aerial_view_plaza": {
        "name": "Aerial View Plaza",
        "required_objects": [0],  # person
        "optional_objects": [1, 2, 24, 25, 26],  # bicycle, car, backpack, umbrella, handbag
        "minimum_required": 1,
        "description": "An urban plaza or public square viewed from above with pedestrian activity"
    },

    # specific cultural item
    "asian_night_market": {
        "name": "Asian Night Market",
        "required_objects": [0, 67],  # person, cell phone
        "optional_objects": [1, 3, 24, 26, 39, 41],  # bicycle, motorcycle, backpack, handbag, bottle, cup
        "minimum_required": 1,
        "description": "A vibrant night market scene typical in Asian cities with food stalls and crowds"
    },
    "asian_temple_area": {
        "name": "Asian Temple Area",
        "required_objects": [0],  # person
        "optional_objects": [24, 25, 26, 67, 75],  # backpack, umbrella, handbag, cell phone, vase
        "minimum_required": 1,
        "description": "A traditional Asian temple complex with visitors and cultural elements"
    },

    # specific time item
    "nighttime_street": {
        "name": "Nighttime Street",
        "required_objects": [0, 9],  # person, traffic light
        "optional_objects": [1, 2, 3, 5, 7, 67],  # bicycle, car, motorcycle, bus, truck, cell phone
        "minimum_required": 1,
        "description": "An urban street at night with artificial lighting and nighttime activity"
    },
    "nighttime_commercial_district": {
        "name": "Nighttime Commercial District",
        "required_objects": [0, 67],  # person, cell phone
        "optional_objects": [1, 2, 3, 24, 26],  # bicycle, car, motorcycle, backpack, handbag
        "minimum_required": 1,
        "description": "A commercial district illuminated at night with neon signs and evening activity"
    },

    # mixture enviroment item
    "indoor_outdoor_cafe": {
        "name": "Indoor-Outdoor Cafe",
        "required_objects": [56, 60, 41],  # chair, dining table, cup
        "optional_objects": [39, 40, 63, 67, 73],  # bottle, wine glass, laptop, cell phone, book
        "minimum_required": 2,
        "description": "A cafe setting with both indoor elements and outdoor patio or sidewalk seating"
    },
    "transit_station_platform": {
        "name": "Transit Station Platform",
        "required_objects": [0],  # person
        "optional_objects": [5, 6, 7, 24, 28, 67],  # bus, train, truck, backpack, suitcase, cell phone
        "minimum_required": 1,
        "description": "A transit platform with waiting passengers and arriving/departing vehicles"
    },
    "sports_stadium": {
        "name": "Sports Stadium",
        "required_objects": [0, 32],  # person, sports ball
        "optional_objects": [24, 38, 39, 41, 67],  # backpack, tennis racket, bottle, cup, cell phone
        "minimum_required": 1,
        "description": "A sports stadium or arena with spectators and athletic activities"
    },
    "construction_site": {
        "name": "Construction Site",
        "required_objects": [0, 7],  # person, truck
        "optional_objects": [2, 3, 11, 76, 77, 78],  # car, motorcycle, fire hydrant, scissors, teddy bear, hair drier
        "minimum_required": 1,
        "description": "A construction site with workers, equipment, and building materials"
    },
    "medical_facility": {
        "name": "Medical Facility",
        "required_objects": [0, 56, 60],  # person, chair, dining table
        "optional_objects": [63, 64, 66, 67, 73],  # laptop, mouse, keyboard, cell phone, book
        "minimum_required": 2,
        "description": "A medical facility such as hospital, clinic or doctor's office with medical staff and patients"
    },
    "educational_setting": {
        "name": "Educational Setting",
        "required_objects": [0, 56, 60, 73],  # person, chair, dining table, book
        "optional_objects": [63, 64, 66, 67, 74],  # laptop, mouse, keyboard, cell phone, clock
        "minimum_required": 2,
        "description": "An educational environment such as classroom, lecture hall or study area"
    },
    "aerial_view_intersection": {
        "name": "Aerial View Intersection",
        "required_objects": [0, 9],  # person, traffic light
        "optional_objects": [1, 2, 3, 5, 7],  # bicycle, car, motorcycle, bus, truck
        "minimum_required": 1,
        "description": "An intersection viewed from above, showing crossing patterns and pedestrian movement",
        "viewpoint_indicator": "aerial", # view side
        "key_features": ["crosswalk_pattern", "pedestrian_flow", "intersection_layout"],  # key feature
        "detection_priority": 10  # priority
    },
    "perpendicular_crosswalk_intersection": {
        "name": "Perpendicular Crosswalk Intersection",
        "required_objects": [0],  # person
        "optional_objects": [1, 2, 3, 5, 7, 9],  # bicycle, car, motorcycle, bus, truck, traffic light
        "minimum_required": 1,
        "description": "An intersection with perpendicular crosswalks where pedestrians cross in multiple directions",
        "viewpoint_indicator": "aerial",
        "key_features": ["perpendicular_crosswalks", "pedestrian_crossing", "multi_directional_movement"],
        "pattern_detection": True, # specific pattern
        "detection_priority": 15  #
    },
    "beach_water_recreation": {
    "name": "Beach/Water Recreation Area",
    "required_objects": [0, 37],  # person, surfboard
    "optional_objects": [25, 33, 1, 8, 29, 24, 26, 39, 41],  # umbrella, kite, bicycle, boat, frisbee, backpack, handbag, bottle, cup
    "minimum_required": 2,
    "description": "A beach or water recreation area with water sports equipment and beach accessories"
    },
    "sports_venue": {
    "name": "Sports Venue",
    "required_objects": [0, 32],  # person, sports ball
    "optional_objects": [34, 35, 38, 25, 24, 26, 39, 41],  # baseball bat, baseball glove, tennis racket, umbrella, backpack, handbag, bottle, cup
    "minimum_required": 2,
    "description": "A professional sports venue with specialized sports equipment and spectator areas"
    },
    "professional_kitchen": {
    "name": "Professional Kitchen",
    "required_objects": [43, 44, 45],  # knife, spoon, bowl
    "optional_objects": [42, 39, 41, 68, 69, 71, 72, 0],  # fork, bottle, cup, microwave, oven, sink, refrigerator, person
    "minimum_required": 3,
    "description": "A commercial kitchen with professional cooking equipment and food preparation areas"
    },
    "tourist_landmark": {
        "name": "Tourist Landmark",
        "required_objects": [0],  # person
        "optional_objects": [24, 26, 67],  # backpack, handbag, cell phone
        "minimum_required": 0,  # 可能沒有人，但仍然是地標
        "description": "A location featuring a famous landmark with tourist activity",
        "priority": 1.2  # 提高優先級
    },
    "natural_landmark": {
        "name": "Natural Landmark",
        "required_objects": [0],  # person
        "optional_objects": [24, 26, 67],  # backpack, handbag, cell phone
        "minimum_required": 0,
        "description": "A natural landmark site with scenic views",
        "priority": 1.2
    },
    "historical_monument": {
        "name": "Historical Monument",
        "required_objects": [0],  # person
        "optional_objects": [24, 26, 67],  # backpack, handbag, cell phone
        "minimum_required": 0,
        "description": "A historical monument or heritage site",
        "priority": 1.2
    },
    "general_indoor_space": {
        "name": "General Indoor Space",
        "required_objects": [], # No strict required objects, depends on combination
        "optional_objects": [
            56, # chair
            57, # couch
            58, # potted plant
            59, # bed
            60, # dining table
            61, # toilet
            62, # tv
            63, # laptop
            66, # keyboard
            67, # cell phone
            73, # book
            74, # clock
            75, # vase
            39, # bottle
            41, # cup
        ],
        "minimum_required": 2, # Needs at least a few common indoor items
        "description": "An indoor area with various common household or functional items.",
        "priority": 0.8 # Lower priority than more specific scenes
    },
    "generic_street_view": {
        "name": "Generic Street View",
        "required_objects": [], # More about the combination
        "optional_objects": [
            0,  # person
            1,  # bicycle
            2,  # car
            3,  # motorcycle
            5,  # bus
            7,  # truck
            9,  # traffic light
            10, # fire hydrant
            11, # stop sign
            13, # bench
            # Consider adding building if YOLO detects it (not a standard COCO class for YOLOv8, but some custom models might)
        ],
        "minimum_required": 2, # e.g., a car and a person, or multiple vehicles
        "description": "An outdoor street view, likely in an urban or suburban setting, with vehicles and/or pedestrians.",
        "priority": 0.85
    },
    "desk_area_workspace": {
        "name": "Desk Area / Workspace",
        "required_objects": [
            63, # laptop or 62 (tv as monitor) or 66 (keyboard)
        ],
        "optional_objects": [
            56, # chair
            60, # dining table (often used as a desk)
            64, # mouse
            66, # keyboard
            73, # book
            41, # cup
            67, # cell phone
            74, # clock
        ],
        "minimum_required": 2, # e.g., laptop and chair, or table and keyboard
        "description": "A workspace or desk area, typically featuring a computer and related accessories.",
        "priority": 0.9
    },
    "outdoor_gathering_spot": {
        "name": "Outdoor Gathering Spot",
        "required_objects": [
            0,  # person
        ],
        "optional_objects": [
            13, # bench
            32, # sports ball
            24, # backpack
            25, # umbrella
            29, # frisbee
            33, # kite
            58, # potted plant (if in a more structured park area)
        ],
        "minimum_required": 2, # e.g., person and bench, or multiple people
        "description": "An outdoor area where people might gather for leisure or activity.",
        "priority": 0.8
    },
    "kitchen_counter_or_utility_area": {
        "name": "Kitchen Counter or Utility Area",
        "required_objects": [],
        "optional_objects": [
            39, # bottle
            41, # cup
            44, # spoon
            45, # bowl
            68, # microwave
            69, # oven
            70, # toaster
            71, # sink
            72, # refrigerator
        ],
        "minimum_required": 2, # e.g., sink and microwave, or refrigerator and bottles
        "description": "An area likely used for food preparation or kitchen utilities.",
        "priority": 0.9
    }
}

Writing scene_type.py


In [ ]:
# %%writefile confidence_templates.py

CONFIDENCE_TEMPLATES = {
    "high": "{description} {details}",
    "medium": "This appears to be {description} {details}",
    "low": "This might be {description}, but the confidence is low. {details}"
}

Writing confidence_templates.py


In [ ]:
# %%writefile scene_detail_templates.py

SCENE_DETAIL_TEMPLATES = {
            "living_room": [
                "The space is arranged for relaxation with {furniture}.",
                "There is {electronics} for entertainment.",
                "The room has a seating area with {seating}."
            ],
            "bedroom": [
                "The room contains {bed_type} in the {bed_location}.",
                "This sleeping area has {bed_description}.",
                "A personal space with {bed_type} and {extras}."
            ],
            "dining_area": [
                "A space set up for meals with {table_setup}.",
                "The dining area contains {table_description}.",
                "A place for eating with {dining_items}."
            ],
            "kitchen": [
                "A food preparation area with {appliances}.",
                "The kitchen contains {kitchen_items}.",
                "A cooking space equipped with {cooking_equipment}."
            ],
            "office_workspace": [
                "A work environment with {office_equipment}.",
                "A space designed for productivity with {desk_setup}.",
                "A workspace containing {computer_equipment}."
            ],
            "city_street": [
                "An urban thoroughfare with {traffic_description}.",
                "A street scene with {people_and_vehicles}.",
                "A city path with {street_elements}."
            ],
            "park_area": [
                "An outdoor recreational space with {park_features}.",
                "A leisure area featuring {outdoor_elements}.",
                "A public outdoor space with {park_description}."
            ],
            "retail_store": [
                "A shopping environment with {store_elements}.",
                "A commercial space where {shopping_activity}.",
                "A retail area containing {store_items}."
            ],
            "upscale_dining": [
            "The space features {furniture} with {design_elements} for an elegant dining experience.",
            "This sophisticated dining area includes {lighting} illuminating {table_setup}.",
            "A stylish dining environment with {seating} arranged around {table_description}."
            ],
            "asian_commercial_street": [
                "A vibrant street lined with {storefront_features} and filled with {pedestrian_flow}.",
                "This urban commercial area displays {asian_elements} with {cultural_elements}.",
                "A lively shopping street characterized by {signage} and busy with {street_activities}."
            ],
            "financial_district": [
                "A canyon of {buildings} with {traffic_elements} moving through the urban landscape.",
                "This business district features {skyscrapers} along {road_features}.",
                "A downtown corridor with {architectural_elements} framing views of {city_landmarks}."
            ],
            "urban_intersection": [
                "A busy crossroad with {crossing_pattern} where {pedestrian_behavior} is observed.",
                "This urban junction features {pedestrian_density} navigating the {traffic_pattern}.",
                "A well-marked intersection designed for {pedestrian_flow} across multiple directions."
            ],
            "transit_hub": [
                "A transportation nexus where {transit_vehicles} arrive and depart amid {passenger_activity}.",
                "This transit center accommodates {transportation_modes} with facilities for {passenger_needs}.",
                "A busy transport hub featuring {transit_infrastructure} and areas for {passenger_movement}."
            ],
            "shopping_district": [
                "A commercial zone filled with {retail_elements} and {shopping_activity}.",
                "This shopping area features {store_types} along {walkway_features}.",
                "A retail district characterized by {commercial_signage} and {consumer_behavior}."
            ],
            "bus_stop": [
                "Passengers waiting at a roadside stop served by {transit_vehicles}.",
                "A designated bus stop with shelters and {passenger_activity}.",
                "Commuters boarding or alighting from {transit_vehicles} at the curb."
            ],
            "bus_station": [
                "Multiple buses parked in a terminal where {passenger_activity}.",
                "A busy station hub featuring {transit_vehicles} and traveler luggage.",
                "A transit center with waiting areas and various {transportation_modes}."
            ],
            "zoo": [
                "Enclosures showcasing elephants, zebras, and giraffes with visitors observing.",
                "A wildlife exhibit area where families watch animal displays.",
                "A recreational space featuring large animal exhibits and strolling guests."
            ],
            "harbor": [
                "Boats docked along the waterfront with nearby vehicular traffic.",
                "A maritime area where vessels anchor beside roads busy with cars and motorcycles.",
                "A coastal dock featuring moored boats and passing traffic elements."
            ],
            "playground": [
                "An open play area equipped with balls and recreational structures.",
                "People engaging in games and sports in a communal space.",
                "A leisure area featuring playground equipment and active participants."
            ],
            "sports_field": [
                "An athletic field marked for various ball games and matches.",
                "Players using equipment like bats, gloves, and rackets on a grassy pitch.",
                "A designated sports area with goalposts or markings for competitive play."
            ],
            "narrow_commercial_alley": [
                "A tight alley lined with {storefront_features} and light vehicles.",
                "Pedestrians navigate a confined lane flanked by shops and {street_activities}.",
                "An urban passage featuring {storefront_features} with {people_and_vehicles}."
            ],
            "daytime_shopping_street": [
                "A bustling street during daytime with {storefront_features} and {pedestrian_flow}.",
                "Shoppers and vehicles move along a retail strip marked by {signage}.",
                "An open commercial avenue filled with {people_and_vehicles} amid shops."
            ],
            "urban_pedestrian_crossing": [
                "A marked crosswalk with {crossing_pattern} under {lighting_modifier} sky.",
                "Pedestrians use designated crossing with {traffic_pattern} at the intersection.",
                "People waiting at a signal-controlled crossing next to {street_elements}."
            ],
            "aerial_view_intersection": [
                "The crossing pattern shows {crossing_pattern} with {pedestrian_flow} across multiple directions.",
                "From above, this intersection reveals {traffic_pattern} with {pedestrian_density} navigating through defined paths.",
                "This bird's-eye view shows {street_elements} converging at a junction where {pedestrian_behavior} is visible."
            ],
            "aerial_view_commercial_area": [
                "From above, this commercial zone shows {storefront_features} with {pedestrian_flow} moving between establishments.",
                "This overhead view reveals {shopping_activity} amid {walkway_features} connecting different businesses.",
                "The aerial perspective captures {retail_elements} organized along {commercial_layout} with visible customer activity."
            ],
            "aerial_view_plaza": [
                "This overhead view of the plaza shows {pedestrian_pattern} across an open public space.",
                "From above, the plaza reveals {gathering_features} where people congregate in {movement_pattern}.",
                "The aerial perspective captures {urban_elements} arranged around a central area where {public_activity} occurs."
            ],
            "asian_night_market": [
                "This bustling night market features {stall_elements} illuminated by {lighting_features} with crowds enjoying {food_elements}.",
                "Rows of {vendor_stalls} line this vibrant market where {nighttime_activity} continues under {cultural_lighting}.",
                "The market atmosphere is created by {asian_elements} and {night_market_sounds} amid {evening_crowd_behavior}."
            ],
            "asian_temple_area": [
                "This sacred space features {architectural_elements} displaying {cultural_symbols} with visitors engaging in {ritual_activities}.",
                "The temple area contains {religious_structures} adorned with {decorative_features} where people practice {cultural_practices}.",
                "Traditional {temple_architecture} creates a spiritual atmosphere enhanced by {sensory_elements} and {visitor_activities}."
            ],
            "european_plaza": [
                "This historic plaza is framed by {architectural_style} surrounding an open space where {public_activities} take place.",
                "The European square features {historic_elements} and {urban_design} creating a space for {social_behaviors}.",
                "Classical {european_features} define this public space where {tourist_activities} blend with {local_customs}."
            ],
            "nighttime_street": [
                "The night transforms this street with {lighting_effects} casting {shadow_patterns} across {urban_features}.",
                "After dark, this urban corridor is defined by {illuminated_elements} with {evening_activities} visible in the artificial light.",
                "The nocturnal street scene captures {light_sources} creating contrast between {lit_areas} and {shadowed_zones}."
            ],
            "nighttime_commercial_district": [
                "After sunset, this commercial area comes alive with {illuminated_signage} and {evening_activities} under {colorful_lighting}.",
                "The district's nighttime character is defined by {neon_elements} highlighting {storefront_features} amid {night_crowd_behavior}.",
                "Evening transforms this zone through {light_displays} that accentuate {building_features} and frame {nightlife_activities}."
            ],
            "indoor_outdoor_cafe": [
                "This cafe blends indoor comfort with outdoor atmosphere through {transitional_elements} connecting {indoor_features} with {outdoor_setting}.",
                "Customers enjoy both {interior_amenities} and {exterior_features} in this space that bridges indoor comfort and outdoor ambiance.",
                "The cafe design creates flow between {inside_elements} and {outside_spaces} allowing patrons to experience {dual_environment_benefits}."
            ],
            "transit_station_platform": [
                "This transit platform combines covered areas with open sections where {passenger_activities} occur while awaiting {transportation_types}.",
                "The station design balances {sheltered_elements} with {exposed_areas} for passengers engaged in {waiting_behaviors}.",
                "Commuters navigate between {indoor_facilities} and {platform_features} while {transit_routines} unfold around arriving vehicles."
            ],
            "sports_stadium": [
                "This athletic venue features {seating_arrangement} surrounding {playing_surface} where {sporting_activities} take place.",
                "The stadium design incorporates {spectator_facilities} overlooking {competition_space} designed for {sports_events}.",
                "Fans occupy {viewing_areas} arranged to maximize visibility of {field_elements} where athletes engage in {game_activities}."
            ],
            "construction_site": [
                "This development area shows {construction_equipment} amid {building_materials} where workers conduct {construction_activities}.",
                "The construction process is visible through {work_elements} positioned around {structural_components} in various stages of completion.",
                "Workers utilize {site_equipment} to transform {raw_materials} following {construction_process} stages."
            ],
            "medical_facility": [
                "This healthcare environment features {medical_elements} arranged to support {clinical_activities} in a {facility_design}.",
                "The medical space incorporates {healthcare_features} where {patient_interactions} occur in a controlled environment.",
                "Professional medical staff utilize {equipment_types} while conducting {care_procedures} in specialized {treatment_spaces}."
            ],
            "educational_setting": [
                "This learning environment contains {educational_furniture} arranged to facilitate {learning_activities} through {instructional_design}.",
                "The educational space features {classroom_elements} organized for {teaching_methods} and {student_engagement}.",
                "Students and educators interact within {learning_spaces} equipped with {educational_tools} supporting {knowledge_transfer}."
            ],
            "beach_water_recreation": [
                "A coastal recreation area with {beach_equipment} and people enjoying {water_activities}.",
                "This shoreline space features {beach_equipment} where visitors engage in {water_activities}.",
                "An outdoor water recreation zone with {beach_equipment} set up for {water_activities}."
            ],
            "sports_venue": [
                "A professional sports facility with {sports_equipment} arranged for {competitive_activities}.",
                "This athletics venue features {sports_equipment} with spaces designated for {competitive_activities}.",
                "A specialized sports arena containing {sports_equipment} designed for {competitive_activities}."
            ],
            "professional_kitchen": [
                "A commercial cooking space with {kitchen_equipment} organized for {food_preparation}.",
                "This professional culinary area contains {kitchen_equipment} arranged in stations for {food_preparation}.",
                "An industrial kitchen featuring {kitchen_equipment} designed for efficient {food_preparation}."
            ],
            "tourist_landmark": [
                "This notable landmark attracts visitors who come to see {landmark_features} and experience {tourist_activities}.",
                "A famous landmark site where tourists can observe {landmark_features} and engage in {tourist_activities}.",
                "This iconic landmark showcases {landmark_features} and is a popular destination for {tourist_activities}."
            ],
            "natural_landmark": [
                "This natural landmark features {landmark_features} and offers opportunities for {outdoor_activities}.",
                "A scenic natural formation with {landmark_features} where visitors enjoy {outdoor_activities}.",
                "This impressive natural landmark displays {landmark_features} and attracts nature enthusiasts for {outdoor_activities}."
            ],
            "historical_monument": [
                "This historical monument exhibits {landmark_features} and has significance related to {historical_elements}.",
                "An important historical site featuring {landmark_features} and representing {historical_elements}.",
                "This heritage monument showcases {landmark_features} and commemorates {historical_elements}."
            ]
        }

Writing scene_detail_templates.py


In [ ]:
# %%writefile object_template_fillers.py

OBJECT_TEMPLATE_FILLERS = {
                "furniture": ["designer chairs", "wooden dining table", "stylish seating", "upholstered armchairs", "elegant dining furniture"],
                "design_elements": ["art pieces", "decorative wreaths", "statement lighting", "seasonal decorations", "sophisticated decor"],
                "lighting": ["pendant lights", "decorative fixtures", "geometric lighting", "modern chandeliers", "ambient illumination"],
                "table_setup": ["elegantly set table", "tabletop decorations", "seasonal centerpieces", "formal place settings", "floral arrangements"],
                "seating": ["upholstered chairs", "accent armchairs", "mixed seating styles", "designer dining chairs", "comfortable dining seats"],
                "table_description": ["solid wood table", "designer dining table", "expansive dining surface", "artisanal table", "statement dining table"],

                "storefront_features": ["multi-story shops", "illuminated signs", "merchandise displays", "compact storefronts", "vertical retail spaces"],
                "pedestrian_flow": ["people walking", "shoppers", "pedestrians", "locals and tourists", "urban foot traffic"],
                "asian_elements": ["Asian language signage", "decorative lanterns", "local storefronts", "character-based text", "regional design elements"],
                "cultural_elements": ["red lanterns", "local typography", "distinctive architecture", "cultural symbols", "traditional decorations"],
                "signage": ["bright store signs", "multilingual text", "vertical signboards", "neon displays", "electronic advertisements"],
                "street_activities": ["shopping", "commuting", "socializing", "vendor transactions", "urban navigation"],

                "buildings": ["high-rise office buildings", "corporate towers", "skyscrapers", "financial institutions", "commercial headquarters"],
                "traffic_elements": ["vehicle lights", "trams/street cars", "lane markers", "traffic signals", "urban transit"],
                "skyscrapers": ["glass and steel buildings", "tall structures", "modern architecture", "office towers", "urban high-rises"],
                "road_features": ["wide avenues", "tram tracks", "traffic lanes", "median dividers", "urban throughways"],
                "architectural_elements": ["contemporary buildings", "urban design", "varied architectural styles", "corporate architecture", "city planning features"],
                "city_landmarks": ["distant bridge", "skyline features", "iconic structures", "urban monuments", "signature buildings"],

                "crossing_pattern": ["zebra crosswalks", "pedestrian walkways", "crosswalk markings", "intersection design", "safety stripes"],
                "pedestrian_density": ["groups of people", "commuters", "diverse pedestrians", "urban crowds", "varying foot traffic"],
                "pedestrian_behavior": ["walking in different directions", "crossing together", "waiting for signals", "navigating intersections", "following traffic rules"],
                "traffic_pattern": ["four-way intersection", "crossroad", "junction", "multi-directional traffic", "regulated crossing"],
                "pedestrian_flow": ["people crossing", "directional movement", "coordinated crossing", "timed pedestrian traffic", "intersection navigation"],

                "transit_vehicles": ["buses", "trams", "trains", "taxis", "shuttles"],
                "passenger_activity": ["boarding", "waiting", "exiting vehicles", "checking schedules", "navigating stations"],
                "transportation_modes": ["public transit", "private vehicles", "ride services", "light rail", "bus systems"],
                "passenger_needs": ["waiting areas", "information displays", "ticketing services", "transit connections", "seating"],
                "transit_infrastructure": ["stations", "platforms", "boarding areas", "transit lanes", "signaling systems"],
                "passenger_movement": ["transfers", "entrances and exits", "queueing", "platform access", "terminal navigation"],

                "retail_elements": ["storefronts", "display windows", "shopping bags", "merchandise", "retail signage"],
                "shopping_activity": ["browsing", "carrying purchases", "window shopping", "social shopping", "consumer activities"],
                "store_types": ["boutiques", "brand stores", "local shops", "chain retailers", "specialty stores"],
                "walkway_features": ["pedestrian paths", "shopping promenades", "retail corridors", "commercial walkways", "shopping streets"],
                "commercial_signage": ["brand logos", "sale announcements", "store names", "advertising displays", "digital signage"],
                "consumer_behavior": ["shopping in groups", "individual browsing", "carrying bags", "examining products", "moving between stores"],

                "beach_equipment": ["beach umbrellas", "surfboards", "beach towels", "sun protection", "recreational equipment"],
                "water_activities": ["water sports", "surfing", "beach recreation", "sun bathing", "coastal leisure"],
                "sports_equipment": ["game balls", "professional equipment", "athletic gear", "sports apparatus", "competition items"],
                "competitive_activities": ["team sports", "athletic contests", "competitive games", "sporting events", "professional matches"],
                "kitchen_equipment": ["professional appliances", "cooking stations", "preparation surfaces", "culinary tools", "industrial equipment"],
                "food_preparation": ["meal production", "culinary operations", "food service preparation", "commercial cooking", "kitchen workflow"],

                "crossing_pattern": ["grid-like pedestrian crossings", "multi-directional crosswalks", "cross-shaped intersection design", "perpendicular crossing lanes", "zebra-striped crosswalks viewed from above"],
                "pedestrian_pattern": ["scattered distribution of people", "organized flow of pedestrians", "clustered gatherings", "radial movement patterns", "linear procession of individuals"],
                "commercial_layout": ["parallel shopping streets", "interconnected shopping blocks", "radial marketplace design", "grid-like retail arrangement", "meandering commercial pathways"],
                "movement_pattern": ["circular crowd motion", "directional pedestrian flow", "scattered individual movement", "converging foot traffic", "diverging pedestrian patterns"],

                "stall_elements": ["food vendors with steaming woks", "trinket sellers with colorful displays", "lantern-lit stalls", "bamboo-framed shops", "canvas-covered market stands"],
                "asian_elements": ["hanging red lanterns", "character-based signage", "ornate temple decorations", "traditional paper decorations", "stylized gateway arches"],
                "cultural_lighting": ["paper lantern illumination", "neon character signs", "strung festival lights", "hanging light chains", "colorful shop front lighting"],
                "architectural_elements": ["tiered pagoda roofs", "ornate dragon sculptures", "stone guardian statues", "intricately carved railings", "traditional wooden beams"],
                "cultural_symbols": ["dharma wheels", "lotus motifs", "yin-yang symbols", "zodiac animal representations", "traditional calligraphy"],
                "architectural_style": ["Baroque facades", "Gothic spires", "Renaissance colonnades", "Neoclassical pediments", "Medieval archways"],
                "european_features": ["cobblestone paving", "ornate fountains", "bronze statuary", "wrought iron lampposts", "cafe terraces"],

                "lighting_effects": ["streetlamp pools of light", "neon sign glow", "illuminated window squares", "headlight streams", "traffic signal flashes"],
                "illuminated_elements": ["lit storefront windows", "glowing traffic signals", "illuminated advertising", "headlight-lit streets", "backlit silhouettes"],
                "neon_elements": ["colorful shop signs", "animated light displays", "illuminated brand logos", "glowing storefront outlines", "digital advertising screens"],
                "illuminated_signage": ["bright LED displays", "glowing brand names", "projected light advertisements", "illuminated menu boards", "digital information screens"],
                "colorful_lighting": ["multi-colored neon", "warm ambient illumination", "cool blue accent lights", "festive string lighting", "dynamic color-changing displays"],

                "transitional_elements": ["retractable glass walls", "indoor-outdoor bar counters", "terraced seating areas", "threshold planters", "partial canopy coverage"],
                "indoor_features": ["climate-controlled spaces", "soft seating arrangements", "interior decor accents", "mood lighting fixtures", "sound-dampened areas"],
                "outdoor_setting": ["sidewalk tables", "patio seating", "garden furniture", "open-air counters", "courtyard arrangements"],
                "seating_arrangement": ["tiered spectator stands", "premium viewing boxes", "courtside seating", "general admission benches", "stadium chair rows"],
                "playing_surface": ["marked court boundaries", "manicured field turf", "running tracks", "competition equipment", "sports field markers"],
                "construction_equipment": ["tower cranes", "excavators", "cement mixers", "scaffolding structures", "construction barriers"],
                "medical_elements": ["examination furniture", "monitoring equipment", "sanitation stations", "privacy screens", "medical supply carts"],
                "educational_furniture": ["student desks", "lecture podiums", "laboratory benches", "learning stations", "collaborative workspace tables"],

                "landmark_features": ["distinctive architecture", "iconic structural elements", "famous design features", "recognized silhouette", "impressive proportions"],
                "tourist_activities": ["sightseeing", "guided tours", "photography",  "cultural exploration", "souvenir shopping"],
                "outdoor_activities": ["nature photography", "hiking",  "scenic viewing", "wildlife observation", "outdoor exploration"],
                "historical_elements": ["cultural heritage", "historical events", "architectural periods", "traditional craftsmanship", "significant achievements"]
                }

Writing object_template_fillers.py


In [ ]:
# %%writefile safety_templates.py
SAFETY_TEMPLATES = {
    "general": "Pay attention to {safety_element}.",
    "warning": "Be cautious of {hazard} in this environment.",
    "notice": "Note the presence of {element_of_interest}."
}

Writing safety_templates.py


In [ ]:
# %%writefile activity_templates.py

ACTIVITY_TEMPLATES = {
            "living_room": [
                "Watching TV",
                "Relaxing on the sofa",
                "Reading",
                "Socializing"
            ],
            "bedroom": [
                "Sleeping",
                "Resting",
                "Getting dressed",
                "Reading in bed"
            ],
            "dining_area": [
                "Eating a meal",
                "Having a conversation",
                "Working at table"
            ],
            "kitchen": [
                "Cooking",
                "Food preparation",
                "Cleaning dishes"
            ],
            "office_workspace": [
                "Working on computer",
                "Office work",
                "Virtual meetings",
                "Reading documents"
            ],
            "meeting_room": [
                "Group meeting",
                "Presentation",
                "Team discussion",
                "Collaboration"
            ],
            "city_street": [
                "Walking",
                "Commuting",
                "Shopping",
                "Waiting for transportation"
            ],
            "parking_lot": [
                "Parking vehicles",
                "Loading/unloading items",
                "Entering/exiting vehicles"
            ],
            "park_area": [
                "Walking",
                "Relaxing outdoors",
                "Exercising",
                "Social gathering"
            ],
            "retail_store": [
                "Shopping",
                "Browsing products",
                "Purchasing items"
            ],
            "supermarket": [
                "Grocery shopping",
                "Selecting products",
                "Checking out"
            ],
            "upscale_dining": [
                "Fine dining",
                "Social gathering",
                "Special occasion meal",
                "Family dinner",
                "Business meeting",
                "Celebratory meal"
            ],
            "asian_commercial_street": [
                "Shopping",
                "Sightseeing",
                "Walking to destinations",
                "Visiting local shops",
                "Cultural exploration",
                "Urban commuting",
                "Meeting friends"
            ],
            "financial_district": [
                "Commuting",
                "Business travel",
                "Urban transit",
                "Sightseeing",
                "City navigation",
                "Professional activities",
                "Corporate meetings"
            ],
            "urban_intersection": [
                "Street crossing",
                "Waiting for signals",
                "Urban navigation",
                "Commuting",
                "Group movement",
                "Following traffic patterns",
                "Pedestrian coordination"
            ],
            "transit_hub": [
                "Commuting",
                "Waiting for transportation",
                "Transferring between vehicles",
                "Starting/ending journeys",
                "Meeting travelers",
                "Checking transit schedules",
                "Urban transportation"
            ],
            "shopping_district": [
                "Retail shopping",
                "Window browsing",
                "Social shopping",
                "Product comparison",
                "Making purchases",
                "Brand exploration",
                "Recreational shopping"
            ],
            "bus_stop": [
                "Waiting for the bus",
                "Checking schedules",
                "Boarding or alighting",
                "Standing under shelter"
            ],
            "bus_station": [
                "Navigating between platforms",
                "Handling luggage",
                "Boarding buses",
                "Gathering at waiting areas"
            ],
            "zoo": [
                "Watching animal exhibits",
                "Taking photos of wildlife",
                "Walking along enclosures",
                "Reading informational signs"
            ],
            "harbor": [
                "Observing docked boats",
                "Commuting by watercraft",
                "Loading or unloading cargo",
                "Strolling along the pier"
            ],
            "playground": [
                "Playing ball games",
                "Swinging or sliding",
                "Running around",
                "Socializing with friends"
            ],
            "sports_field": [
                "Practicing ball drills",
                "Competing in matches",
                "Warming up or stretching",
                "Team training sessions"
            ],
            "narrow_commercial_alley": [
                "Walking through alley",
                "Browsing storefronts",
                "Navigating light traffic",
                "Carrying shopping bags"
            ],
            "daytime_shopping_street": [
                "Shopping",
                "Window browsing",
                "Street photography",
                "Commuting by vehicle"
            ],
            "urban_pedestrian_crossing": [
                "Crossing the street",
                "Waiting for signal",
                "Following traffic rules",
                "Checking for vehicles"
            ],
            "aerial_view_intersection": [
                "Crossing multiple directions",
                "Following traffic signals",
                "Navigating pedestrian paths",
                "Traffic management",
                "Multi-directional movement",
                "Organized crossing patterns",
                "Waiting at signals"
            ],
            "aerial_view_commercial_area": [
                "Shopping district navigation",
                "Retail browsing",
                "Store-to-store movement",
                "Commercial zone foot traffic",
                "Shopping center traversal",
                "Retail area engagement",
                "Walking between stores"
            ],
            "aerial_view_plaza": [
                "Public gathering",
                "Open space traversal",
                "Community congregation",
                "Plaza navigation",
                "Public square activities",
                "Urban space utilization"
            ],
            "asian_night_market": [
                "Street food sampling",
                "Night market browsing",
                "Evening shopping",
                "Cultural food exploration",
                "Vendor interaction",
                "Social night dining",
                "Market stall hopping"
            ],
            "asian_temple_area": [
                "Temple visiting",
                "Cultural site exploration",
                "Spiritual observance",
                "Traditional rituals",
                "Historical site appreciation",
                "Religious tourism",
                "Cultural photography"
            ],
            "european_plaza": [
                "Urban sightseeing",
                "Historical appreciation",
                "Tourist photography",
                "Public space relaxation",
                "Casual strolling"
            ],
            "nighttime_street": [
                "Evening commuting",
                "Night walking",
                "After-hours travel",
                "Nighttime navigation",
                "Evening errands",
                "Late-night transportation",
                "Nocturnal urban movement"
            ],
            "nighttime_commercial_district": [
                "Evening shopping",
                "Nightlife participation",
                "Nighttime entertainment",
                "After-dark dining",
                "Evening social gathering",
                "Night market browsing",
                "Illumination appreciation"
            ],
            "indoor_outdoor_cafe": [
                "Al fresco dining",
                "Sidewalk coffee enjoyment",
                "Indoor-outdoor socializing",
                "Patio relaxation",
                "Open-air refreshment",
                "Transitional space usage",
                "Weather-dependent positioning"
            ],
            "transit_station_platform": [
                "Transit waiting",
                "Platform navigation",
                "Boarding preparation",
                "Arrival monitoring",
                "Schedule checking",
                "Departure positioning",
                "Platform traversal"
            ],
            "sports_stadium": [
                "Spectator viewing",
                "Sports fan cheering",
                "Game attendance",
                "Stadium navigation",
                "Athletic event watching",
                "Audience participation",
                "Sports appreciation"
            ],
            "construction_site": [
                "Construction work",
                "Building development",
                "Site management",
                "Material handling",
                "Construction supervision",
                "Safety monitoring",
                "Building process"
            ],
            "medical_facility": [
                "Healthcare consultation",
                "Medical treatment",
                "Patient waiting",
                "Healthcare delivery",
                "Medical examination",
                "Professional care",
                "Health monitoring"
            ],
            "educational_setting": [
                "Classroom learning",
                "Educational instruction",
                "Student participation",
                "Academic engagement",
                "Knowledge acquisition",
                "Educational discussion",
                "Scholastic activities"
            ],
            "beach_water_recreation": [
                "Surfing",
                "Sunbathing",
                "Beach volleyball",
                "Swimming",
                "Relaxing by the water",
                "Flying beach kites",
                "Beach picnicking",
                "Coastal walking"
            ],
            "sports_venue": [
                "Professional game playing",
                "Sports competition",
                "Athletic training",
                "Team practice",
                "Spectator viewing",
                "Sports coaching",
                "Tournament participation",
                "Athletic performance"
            ],
            "professional_kitchen": [
                "Professional cooking",
                "Food preparation",
                "Meal service coordination",
                "Kitchen operations",
                "Culinary production",
                "Chef activities",
                "Commercial food handling",
                "Restaurant meal preparation"
            ],
            "tourist_landmark": [
                "Sightseeing",
                "Photography",
                "Guided tours",
                "Learning about landmark history",
                "Souvenir shopping",
                "Cultural appreciation",
                "Architectural observation"
            ],
            "natural_landmark": [
                "Nature photography",
                "Scenic viewing",
                "Hiking",
                "Nature appreciation",
                "Wildlife watching",
                "Outdoor recreation",
                "Environmental education"
            ],
            "historical_monument": [
                "Historical tours",
                "Cultural heritage appreciation",
                "Educational visits",
                "Historical photography",
                "Learning about past events",
                "Architectural study",
                "Heritage tourism"
            ],
             "general_indoor_space": [
                "Engaging in general indoor activities",
                "Resting or relaxing in an indoor setting",
                "Possibly having a conversation or reading"
            ],
            "generic_street_view": [
                "People walking or commuting",
                "Vehicles driving on the road",
                "Observing street traffic and urban activity",
                "Waiting at a crosswalk or bus stop (if applicable objects present)"
            ],
            "desk_area_workspace": [
                "Working on a computer or laptop",
                "Studying or reading documents",
                "Writing or taking notes",
                "Participating in an online meeting (if computer present)"
            ],
            "outdoor_gathering_spot": [
                "People socializing outdoors",
                "Relaxing on a bench or in a park-like setting",
                "Engaging in light recreational activities",
                "Having a picnic (if food items or backpacks are present)"
            ],
            "kitchen_counter_or_utility_area": [
                "Preparing food or drinks",
                "Using kitchen appliances like a microwave or toaster",
                "Washing dishes or cleaning",
                "Storing food items"
            ]
        }

Writing activity_templates.py


In [ ]:
# %%writefile object_categories.py
OBJECT_CATEGORIES = {
                "furniture": [56, 57, 58, 59, 60, 61],
                "electronics": [62, 63, 64, 65, 66, 67, 68, 69, 70],
                "kitchen_items": [39, 40, 41, 42, 43, 44, 45],
                "food": [46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
                "vehicles": [1, 2, 3, 4, 5, 6, 7, 8],
                "personal_items": [24, 25, 26, 27, 28, 73, 78, 79]
            }

Writing object_categories.py


In [ ]:
# %%writefile lighting_conditions.py

LIGHTING_CONDITIONS = {
    "time_descriptions": {
        "day_clear": {
        "general": "The scene is captured during clear daylight hours with bright natural lighting.",
        "bright": "The scene is brightly lit with strong, clear daylight.",
        "medium": "The scene is illuminated with moderate daylight under clear conditions.",
        "dim": "The scene is captured in soft daylight on a clear day."
        },
        "day_cloudy": {
        "general": "The scene is captured during daytime under overcast conditions.",
        "bright": "The scene has the diffused bright lighting of an overcast day.",
        "medium": "The scene has even, soft lighting typical of a cloudy day.",
        "dim": "The scene has the muted lighting of a heavily overcast day."
        },
         "day_cloudy_gray": {
        "general": "The scene is captured during an overcast day with muted gray lighting.",
        "bright": "The scene has bright but diffused gray daylight from heavy cloud cover.",
        "medium": "The scene has even, muted lighting typical of a gray, overcast day.",
        "dim": "The scene has subdued lighting under thick gray clouds."
        },
        "indoor_residential_natural": {
            "general": "The scene is captured in a residential setting with natural window lighting.",
            "bright": "The residential space is brightly lit with abundant natural light from windows.",
            "medium": "The home interior has comfortable natural lighting complemented by artificial sources.",
            "dim": "The residential space has soft natural lighting creating a cozy atmosphere."
        },
        "indoor_designer_residential": {
            "general": "The scene is captured in a well-designed residential space with curated lighting.",
            "bright": "The residential interior features bright, designer lighting creating an elegant atmosphere.",
            "medium": "The home space has thoughtfully planned lighting balancing aesthetics and functionality.",
            "dim": "The residential area has sophisticated mood lighting enhancing the design elements."
        },
        "indoor_bright_natural_mix": {
            "general": "The scene is captured indoors with a blend of natural and artificial lighting.",
            "bright": "The indoor space combines bright natural window light with artificial illumination.",
            "medium": "The interior has balanced mixed lighting from windows and electric sources.",
            "dim": "The indoor area has gentle mixed lighting creating comfortable illumination."
        },
        "indoor_restaurant_bar": {
            "general": "The scene is captured inside a restaurant or bar with characteristic warm lighting.",
            "bright": "The dining establishment is well-lit with warm illumination emphasizing ambiance.",
            "medium": "The restaurant/bar has moderate warm lighting creating a comfortable social atmosphere.",
            "dim": "The establishment features soft, warm lighting creating an intimate dining or social atmosphere."
        },
        "sunset/sunrise": {
        "general": "The scene is captured during golden hour with warm lighting.",
        "bright": "The scene is illuminated with bright golden hour light with long shadows.",
        "medium": "The scene has the warm orange-yellow glow typical of sunset or sunrise.",
        "dim": "The scene has soft, warm lighting characteristic of early sunrise or late sunset."
        },
        "night": {
        "general": "The scene is captured at night with limited natural lighting.",
        "bright": "The scene is captured at night but well-lit with artificial lighting.",
        "medium": "The scene is captured at night with moderate artificial lighting.",
        "dim": "The scene is captured in low-light night conditions with minimal illumination."
        },
        "indoor_bright": {
        "general": "The scene is captured indoors with ample lighting.",
        "bright": "The indoor space is brightly lit, possibly with natural light from windows.",
        "medium": "The indoor space has good lighting conditions.",
        "dim": "The indoor space has adequate lighting."
        },
        "indoor_moderate": {
        "general": "The scene is captured indoors with moderate lighting.",
        "bright": "The indoor space has comfortable, moderate lighting.",
        "medium": "The indoor space has standard interior lighting.",
        "dim": "The indoor space has somewhat subdued lighting."
        },
        "indoor_dim": {
        "general": "The scene is captured indoors with dim or mood lighting.",
        "bright": "The indoor space has dim but sufficient lighting.",
        "medium": "The indoor space has low, atmospheric lighting.",
        "dim": "The indoor space has very dim, possibly mood-oriented lighting."
        },
        "beach_daylight": {
            "general": "The scene is captured during daytime at a beach with bright natural sunlight.",
            "bright": "The beach scene is intensely illuminated by direct sunlight.",
            "medium": "The coastal area has even natural daylight.",
            "dim": "The beach has softer lighting, possibly from a partially cloudy sky."
        },
        "sports_arena": {
            "general": "The scene is captured in a sports venue with specialized arena lighting.",
            "bright": "The sports facility is brightly illuminated with powerful overhead lights.",
            "medium": "The venue has standard sports event lighting providing clear visibility.",
            "dim": "The sports area has reduced illumination, possibly before or after an event."
        },
        "kitchen_working": {
            "general": "The scene is captured in a professional kitchen with task-oriented lighting.",
            "bright": "The kitchen is intensely illuminated with clear, functional lighting.",
            "medium": "The culinary space has standard working lights focused on preparation areas.",
            "dim": "The kitchen has reduced lighting, possibly during off-peak hours."
        },
        "unknown": {
        "general": "The lighting conditions in this scene are not easily determined."
        }
    },
    "template_modifiers": {
        "day_clear": "brightly-lit",
        "day_cloudy": "softly-lit",
        "sunset/sunrise": "warmly-lit",
        "night": "night-time",
        "indoor_bright": "well-lit indoor",
        "indoor_moderate": "indoor",
        "indoor_dim": "dimly-lit indoor",
        "indoor_commercial": "retail-lit",
        "indoor_restaurant": "atmospherically-lit",
        "neon_night": "neon-illuminated",
        "stadium_lighting": "flood-lit",
        "mixed_lighting": "transitionally-lit",
        "beach_lighting": "sun-drenched",
        "sports_venue_lighting": "arena-lit",
        "professional_kitchen_lighting": "kitchen-task lit",
        "day_cloudy_gray": "gray-lit",
        "indoor_residential_natural": "naturally-lit residential",
        "indoor_designer_residential": "designer-lit residential",
        "indoor_bright_natural_mix": "mixed-lit indoor",
        "unknown": ""
    },
    "activity_modifiers": {
        "day_clear": ["active", "lively", "busy"],
        "day_cloudy": ["calm", "relaxed", "casual"],
        "sunset/sunrise": ["peaceful", "transitional", "atmospheric"],
        "night": ["quiet", "subdued", "nocturnal"],
        "indoor_bright": ["focused", "productive", "engaged"],
        "indoor_moderate": ["comfortable", "social", "casual"],
        "indoor_dim": ["intimate", "relaxed", "private"],
        "indoor_commercial": ["shopping", "browsing", "consumer-oriented"],
        "indoor_restaurant": ["dining", "social", "culinary"],
        "neon_night": ["vibrant", "energetic", "night-life"],
        "stadium_lighting": ["event-focused", "spectator-oriented", "performance-based"],
        "mixed_lighting": ["transitional", "adaptable", "variable"],
        "unknown": []
    },
    "indoor_commercial": {
    "general": "The scene is captured inside a commercial setting with retail-optimized lighting.",
    "bright": "The space is brightly illuminated with commercial display lighting to highlight merchandise.",
    "medium": "The commercial interior has standard retail lighting that balances visibility and ambiance.",
    "dim": "The commercial space has subdued lighting creating an upscale or intimate shopping atmosphere."
    },
    "indoor_restaurant": {
        "general": "The scene is captured inside a restaurant with characteristic dining lighting.",
        "bright": "The restaurant is well-lit with clear illumination emphasizing food presentation.",
        "medium": "The dining space has moderate lighting striking a balance between functionality and ambiance.",
        "dim": "The restaurant features soft, low lighting creating an intimate dining atmosphere."
    },
    "neon_night": {
        "general": "The scene is captured at night with colorful neon lighting typical of entertainment districts.",
        "bright": "The night scene is illuminated by vibrant neon signs creating a lively, colorful atmosphere.",
        "medium": "The evening setting features moderate neon lighting creating a characteristic urban nightlife scene.",
        "dim": "The night area has subtle neon accents against the darkness, creating a moody urban atmosphere."
    },
    "stadium_lighting": {
        "general": "The scene is captured under powerful stadium lights designed for spectator events.",
        "bright": "The venue is intensely illuminated by stadium floodlights creating daylight-like conditions.",
        "medium": "The sports facility has standard event lighting providing clear visibility across the venue.",
        "dim": "The stadium has reduced illumination typical of pre-event or post-event conditions."
    },
    "mixed_lighting": {
        "general": "The scene features a mix of indoor and outdoor lighting creating transitional illumination.",
        "bright": "The space blends bright natural and artificial light sources across indoor-outdoor boundaries.",
        "medium": "The area combines moderate indoor lighting with outdoor illumination in a balanced way.",
        "dim": "The transition space features subtle lighting gradients between indoor and outdoor zones."
    },
    "stadium_or_floodlit_area": {
    "general": "The scene is captured under powerful floodlights creating uniform bright illumination.",
    "bright": "The area is intensely illuminated by floodlights, similar to stadium conditions.",
    "medium": "The space has even, powerful lighting typical of sports facilities or outdoor events.",
    "dim": "The area has moderate floodlight illumination providing consistent lighting across the space."
    }
}

Writing lighting_conditions.py


In [ ]:
# %%writefile viewpoint_templates.py

VIEWPOINT_TEMPLATES = {
    "eye_level": {
        "prefix": "From a standard eye-level perspective, ",
        "observation": "the scene shows {scene_elements} arranged in a typical front-facing view."
    },
    "aerial": {
        "prefix": "From an aerial perspective, ",
        "observation": "the scene shows {scene_elements} as viewed from above, revealing the spatial layout."
    },
    "elevated": {
        "prefix": "From an elevated viewpoint, ",
        "observation": "the scene presents {scene_elements} with a slight downward angle."
    },
    "low_angle": {
        "prefix": "From a low angle, ",
        "observation": "the scene depicts {scene_elements} from below, emphasizing vertical elements."
    }
}

Writing viewpoint_templates.py


In [ ]:
# %%writefile cultural_templates.py

CULTURAL_TEMPLATES = {
    "asian": {
        "elements": ["character signage", "lanterns", "dense urban layout"],
        "description": "The scene shows distinctive Asian cultural elements such as {elements}."
    },
    "european": {
        "elements": ["classical architecture", "cobblestone streets", "café terraces"],
        "description": "The environment has European characteristics including {elements}."
    },
    "middle_eastern": {
        "elements": ["ornate archways", "geometric patterns", "domed structures"],
        "description": "The scene contains Middle Eastern architectural features such as {elements}."
    },
    "north_american": {
        "elements": ["grid street pattern", "modern skyscrapers", "wide boulevards"],
        "description": "The layout shows typical North American urban design with {elements}."
    }
}

Overwriting cultural_templates.py


In [ ]:
# %%writefile region_analyzer.py

import logging
import traceback
from typing import Dict, List, Any

logger = logging.getLogger(__name__)

class RegionAnalyzer:
    """
    負責處理圖像區域劃分和基礎空間分析功能
    專注於3x3網格的區域劃分、物件分布分析和空間多樣性計算
    """

    def __init__(self):
        """初始化區域分析器，定義3x3網格區域"""
        try:
            # 定義圖像的3x3網格區域
            self.regions = {
                "top_left": (0, 0, 1/3, 1/3),
                "top_center": (1/3, 0, 2/3, 1/3),
                "top_right": (2/3, 0, 1, 1/3),
                "middle_left": (0, 1/3, 1/3, 2/3),
                "middle_center": (1/3, 1/3, 2/3, 2/3),
                "middle_right": (2/3, 1/3, 1, 2/3),
                "bottom_left": (0, 2/3, 1/3, 1),
                "bottom_center": (1/3, 2/3, 2/3, 1),
                "bottom_right": (2/3, 2/3, 1, 1)
            }
            logger.info("RegionAnalyzer initialized successfully with 3x3 grid regions")
        except Exception as e:
            logger.error(f"Failed to initialize RegionAnalyzer: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def determine_region(self, x: float, y: float) -> str:
        """
        判斷點位於哪個區域

        Args:
            x: 標準化x座標 (0-1)
            y: 標準化y座標 (0-1)

        Returns:
            區域名稱
        """
        try:
            for region_name, (x1, y1, x2, y2) in self.regions.items():
                if x1 <= x < x2 and y1 <= y < y2:
                    return region_name

            logger.warning(f"Point ({x}, {y}) does not fall into any defined region")
            return "unknown"

        except Exception as e:
            logger.error(f"Error determining region for point ({x}, {y}): {str(e)}")
            logger.error(traceback.format_exc())
            return "unknown"

    def get_spatial_description_phrase(self, region: str) -> str:
        """
        將region ID轉換為完整的空間描述短語，包含適當的介詞結構

        Args:
            region: 區域標識符（如 "middle_center", "top_left"）

        Returns:
            str: 完整的空間描述短語，空值時返回空字串
        """
        try:
            # 處理空值或無效輸入
            if not region or region.strip() == "" or region == "unknown":
                return "within the visible area"

            # 清理region格式，移除底線
            clean_region = region.replace('_', ' ').strip().lower()

            # 根據區域位置生成自然語言描述
            region_mappings = {
                "top left": "in the upper left area",
                "top center": "in the upper area",
                "top right": "in the upper right area",
                "middle left": "on the left side",
                "middle center": "in the center",
                "center": "in the center",
                "middle right": "on the right side",
                "bottom left": "in the lower left area",
                "bottom center": "in the lower area",
                "bottom right": "in the lower right area"
            }

            # 直接映射匹配
            if clean_region in region_mappings:
                return region_mappings[clean_region]

            # 模糊匹配方位的處理
            if "top" in clean_region and "left" in clean_region:
                return "in the upper left area"
            elif "top" in clean_region and "right" in clean_region:
                return "in the upper right area"
            elif "bottom" in clean_region and "left" in clean_region:
                return "in the lower left area"
            elif "bottom" in clean_region and "right" in clean_region:
                return "in the lower right area"
            elif "top" in clean_region:
                return "in the upper area"
            elif "bottom" in clean_region:
                return "in the lower area"
            elif "left" in clean_region:
                return "on the left side"
            elif "right" in clean_region:
                return "on the right side"
            elif "center" in clean_region or "middle" in clean_region:
                return "in the center"
            else:
                # 對於無法辨識的區域，返回通用描述
                return f"in the {clean_region} area"

        except Exception as e:
            logger.warning(f"Error generating spatial description for region '{region}': {str(e)}")
            return ""

    def get_contextual_spatial_description(self, region: str, object_type: str = "") -> str:
        """
        根據物件類型提供更具情境的空間描述

        Args:
            region: 區域標識符
            object_type: 物件類型，用於優化描述語境

        Returns:
            str: 情境化的空間描述短語
        """
        try:
            # 獲取基礎空間描述
            base_description = self.get_spatial_description_phrase(region)

            if not base_description:
                return ""

            # 根據物件類型調整描述語境
            if object_type:
                object_type_lower = object_type.lower()

                # 對於辨識到人相關，用更自然的位置描述
                if "person" in object_type_lower or "people" in object_type_lower:
                    if "center" in base_description:
                        return "in the central area"
                    elif "upper" in base_description:
                        return "in the background"
                    elif "lower" in base_description:
                        return "in the foreground"

                # 對於車輛，強調道路位置
                elif any(vehicle in object_type_lower for vehicle in ["car", "vehicle", "truck", "bus"]):
                    if "left" in base_description:
                        return "on the left side of the scene"
                    elif "right" in base_description:
                        return "on the right side of the scene"
                    elif "center" in base_description:
                        return "in the central area"

                # 對於交通設施，使用更具體的位置描述
                elif "traffic" in object_type_lower:
                    if "upper" in base_description:
                        return "positioned in the upper portion"
                    elif "center" in base_description:
                        return "centrally positioned"
                    else:
                        return base_description.replace("in the", "positioned in the")

            return base_description

        except Exception as e:
            logger.warning(f"Error generating contextual spatial description: {str(e)}")
            return self.get_spatial_description_phrase(region)


    def validate_region_input(self, region: str) -> bool:
        """
        驗證region輸入是否有效

        Args:
            region: 待驗證的區域標識符

        Returns:
            bool: 是否為有效的region
        """
        try:
            if not region or region.strip() == "":
                return False

            # 清理並檢查是否為已知區域
            clean_region = region.replace('_', ' ').strip().lower()

            known_regions = [
                "top left", "top center", "top right",
                "middle left", "middle center", "middle right",
                "bottom left", "bottom center", "bottom right",
                "center", "unknown"
            ]

            # 直接匹配或包含關鍵詞匹配
            if clean_region in known_regions:
                return True

            # 檢查是否包含有效的位置關鍵詞組合
            position_keywords = ["top", "bottom", "left", "right", "center", "middle"]
            has_valid_keyword = any(keyword in clean_region for keyword in position_keywords)

            return has_valid_keyword

        except Exception as e:
            logger.warning(f"Error validating region input '{region}': {str(e)}")
            return False

    def get_enhanced_directional_description(self, region: str) -> str:
        """
        增強版的方位描述生成，提供更豐富的方位資訊
        擴展原有的get_directional_description方法功能

        Args:
            region: 區域名稱

        Returns:
            str: 增強的方位描述字串
        """
        try:
            if not self.validate_region_input(region):
                return "central"

            region_lower = region.replace('_', ' ').strip().lower()

            # 用比較準確的方位映射
            direction_mappings = {
                "top left": "northwest",
                "top center": "north",
                "top right": "northeast",
                "middle left": "west",
                "middle center": "central",
                "center": "central",
                "middle right": "east",
                "bottom left": "southwest",
                "bottom center": "south",
                "bottom right": "southeast"
            }

            if region_lower in direction_mappings:
                return direction_mappings[region_lower]

            # 模糊匹配邏輯保持與原方法相同
            if "top" in region_lower and "left" in region_lower:
                return "northwest"
            elif "top" in region_lower and "right" in region_lower:
                return "northeast"
            elif "bottom" in region_lower and "left" in region_lower:
                return "southwest"
            elif "bottom" in region_lower and "right" in region_lower:
                return "southeast"
            elif "top" in region_lower:
                return "north"
            elif "bottom" in region_lower:
                return "south"
            elif "left" in region_lower:
                return "west"
            elif "right" in region_lower:
                return "east"
            else:
                return "central"

        except Exception as e:
            logger.error(f"Error getting enhanced directional description for region '{region}': {str(e)}")
            return "central"

    def analyze_regions(self, detected_objects: List[Dict]) -> Dict:
        """
        分析物件在各區域的分布情況

        Args:
            detected_objects: 包含位置資訊的檢測物件列表

        Returns:
            包含區域分析結果的字典
        """
        try:
            if not detected_objects:
                logger.warning("No detected objects provided for region analysis")
                return {
                    "counts": {region: 0 for region in self.regions.keys()},
                    "main_focus": [],
                    "objects_by_region": {region: [] for region in self.regions.keys()}
                }

            # 計算每個區域的物件數量
            region_counts = {region: 0 for region in self.regions.keys()}
            region_objects = {region: [] for region in self.regions.keys()}

            for obj in detected_objects:
                try:
                    region = obj.get("region", "unknown")
                    if region in region_counts:
                        region_counts[region] += 1
                        region_objects[region].append({
                            "class_id": obj.get("class_id"),
                            "class_name": obj.get("class_name")
                        })
                    else:
                        logger.warning(f"Unknown region '{region}' found in object")

                except Exception as e:
                    logger.error(f"Error processing object in region analysis: {str(e)}")
                    continue

            # 確定主要焦點區域（按物件數量排序的前1-2個區域）
            sorted_regions = sorted(region_counts.items(), key=lambda x: x[1], reverse=True)
            main_regions = [region for region, count in sorted_regions if count > 0][:2]

            result = {
                "counts": region_counts,
                "main_focus": main_regions,
                "objects_by_region": region_objects
            }

            logger.info(f"Region analysis completed. Main focus areas: {main_regions}")
            return result

        except Exception as e:
            logger.error(f"Error in region analysis: {str(e)}")
            logger.error(traceback.format_exc())
            # 返回空的結果結構而不是拋出異常
            return {
                "counts": {region: 0 for region in self.regions.keys()},
                "main_focus": [],
                "objects_by_region": {region: [] for region in self.regions.keys()}
            }

    def create_distribution_map(self, detected_objects: List[Dict]) -> Dict:
        """
        創建物件在各區域分布的詳細地圖，用於空間分析

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含各區域分布詳情的字典
        """
        try:
            if not detected_objects:
                logger.warning("No detected objects provided for distribution map creation")
                return self._get_empty_distribution_map()

            distribution = {}

            # 初始化所有區域
            for region in self.regions.keys():
                distribution[region] = {
                    "total": 0,
                    "objects": {},
                    "density": 0
                }

            # 填充分布資料
            for obj in detected_objects:
                try:
                    region = obj.get("region", "unknown")
                    class_id = obj.get("class_id")
                    class_name = obj.get("class_name", "unknown")

                    if region not in distribution:
                        logger.warning(f"Unknown region '{region}' found, skipping object")
                        continue

                    distribution[region]["total"] += 1

                    if class_id not in distribution[region]["objects"]:
                        distribution[region]["objects"][class_id] = {
                            "name": class_name,
                            "count": 0,
                            "positions": []
                        }

                    distribution[region]["objects"][class_id]["count"] += 1

                    # 儲存位置資訊用於空間關係分析
                    normalized_center = obj.get("normalized_center")
                    if normalized_center:
                        distribution[region]["objects"][class_id]["positions"].append(normalized_center)

                except Exception as e:
                    logger.error(f"Error processing object in distribution map: {str(e)}")
                    continue

            # 計算每個區域的物件密度
            for region, data in distribution.items():
                # 假設所有區域在網格中大小相等
                data["density"] = data["total"] / 1

            logger.info("Distribution map created successfully")
            return distribution

        except Exception as e:
            logger.error(f"Error creating distribution map: {str(e)}")
            logger.error(traceback.format_exc())
            return self._get_empty_distribution_map()

    def calculate_spatial_diversity(self, detected_objects: List[Dict]) -> float:
        """
        計算物件空間分布的多樣性
        評估物件是否分散在不同區域，避免所有物件集中在單一區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            空間多樣性評分 (0.0-1.0)
        """
        try:
            if not detected_objects:
                logger.warning("No detected objects provided for spatial diversity calculation")
                return 0.0

            regions = set()
            for obj in detected_objects:
                region = obj.get("region", "center")
                regions.add(region)

            unique_regions = len(regions)
            diversity_score = min(unique_regions / 2.0, 1.0)

            logger.info(f"Spatial diversity calculated: {diversity_score:.3f} (regions: {unique_regions})")
            return diversity_score

        except Exception as e:
            logger.error(f"Error calculating spatial diversity: {str(e)}")
            logger.error(traceback.format_exc())
            return 0.0

    def get_directional_description(self, region: str) -> str:
        """
        將區域名稱轉換為方位描述（東西南北）

        Args:
            region: 區域名稱

        Returns:
            方位描述字串
        """
        try:
            region_lower = region.lower()

            if "top" in region_lower and "left" in region_lower:
                return "northwest"
            elif "top" in region_lower and "right" in region_lower:
                return "northeast"
            elif "bottom" in region_lower and "left" in region_lower:
                return "southwest"
            elif "bottom" in region_lower and "right" in region_lower:
                return "southeast"
            elif "top" in region_lower:
                return "north"
            elif "bottom" in region_lower:
                return "south"
            elif "left" in region_lower:
                return "west"
            elif "right" in region_lower:
                return "east"
            else:
                return "central"

        except Exception as e:
            logger.error(f"Error getting directional description for region '{region}': {str(e)}")
            return "central"

    def _get_empty_distribution_map(self) -> Dict:
        """
        返回空的分布地圖結構

        Returns:
            空的分布地圖字典
        """
        distribution = {}
        for region in self.regions.keys():
            distribution[region] = {
                "total": 0,
                "objects": {},
                "density": 0
            }
        return distribution

Writing region_analyzer.py


In [ ]:
# %%writefile object_extractor.py

import logging
import traceback
from typing import Dict, List, Any, Optional

logger = logging.getLogger(__name__)

class ObjectExtractor:
    """
    專門處理物件檢測結果的提取和預處理
    負責從YOLO檢測結果提取物件資訊、物件分類和核心物件的辨識
    """

    def __init__(self, class_names: Dict[int, str] = None, object_categories: Dict[str, List[int]] = None):
        """
        初始化物件提取器

        Args:
            class_names: 類別ID到類別名稱的映射字典
            object_categories: 物件類別分組字典
        """
        try:
            self.class_names = class_names or {}
            self.object_categories = object_categories or {}

            # 1. 讀取並設定基本信心度門檻（如果外部沒傳，就預設 0.25）
            self.base_conf_threshold = 0.25

            # 2. 動態信心度調整映射表 (key: 小寫 class_name, value: 調整係數)
            #    最終的門檻 = base_conf_threshold * factor
            #    如果某個 class_name 沒在這裡，就直接用 base_conf_threshold（相當於 factor=1.0）
            self.dynamic_conf_map = {
                "traffic light": 0.6,  # 0.25 * 0.6 = 0.15
                "car": 0.8,            # 0.25 * 0.8 = 0.20
                "person": 0.7,         # 0.25 * 0.7 = 0.175

            }

            logger.info(f"ObjectExtractor initialized with {len(self.class_names)} class names and {len(self.object_categories)} object categories")

        except Exception as e:
            logger.error(f"Failed to initialize ObjectExtractor: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def _get_dynamic_threshold(self, class_name: str) -> float:
        """
        根據 class_name 從 dynamic_conf_map 拿到 factor，計算最終的信心度門檻：
            threshold = base_conf_threshold * factor

        如果 class_name 不在映射表裡，就回傳 base_conf_threshold。
        """
        # 使用小寫做匹配，確保在 dynamic_conf_map 裡的 key 也都用小寫
        key = class_name.lower()
        factor = self.dynamic_conf_map.get(key, 1.0)
        return self.base_conf_threshold * factor

    def extract_detected_objects(
            self,
            detection_result: Any,
            confidence_threshold: float = 0.25,
            region_analyzer=None
        ) -> List[Dict]:
            """
            從檢測結果中提取物件資訊，包含位置資訊

            Args:
                detection_result: YOLO檢測結果
                confidence_threshold: 改由動態門檻決定
                region_analyzer: 區域分析器實例，用於判斷物件所屬區域

            Returns:
                包含檢測物件資訊的字典列表
            """
            try:
                # 調試信息：記錄當前類別映射狀態
                logger.info(f"ObjectExtractor.extract_detected_objects called")
                logger.info(f"Current class_names keys: {list(self.class_names.keys()) if self.class_names else 'None'}")

                if detection_result is None:
                    logger.warning("Detection result is None")
                    return []

                if not hasattr(detection_result, 'boxes'):
                    logger.error("Detection result does not have boxes attribute")
                    return []

                boxes = detection_result.boxes.xyxy.cpu().numpy()
                classes = detection_result.boxes.cls.cpu().numpy().astype(int)
                confidences = detection_result.boxes.conf.cpu().numpy()

                # 獲取圖像尺寸
                img_height, img_width = detection_result.orig_shape[:2]

                detected_objects = []

                for box, class_id, confidence in zip(boxes, classes, confidences):
                    try:
                        # 1. 先拿到這筆偵測物件的 class_name
                        class_name = self.class_names.get(int(class_id), f"unknown_class_{class_id}")
                        # 2. 計算這個 class 應該採用的動態 threshold
                        dyn_thr = self._get_dynamic_threshold(class_name)  # e.g. 0.25 * factor
                        # 3. 如果 confidence < dyn_thr，就跳過這一筆
                        if confidence < dyn_thr:
                            continue

                        # 後面維持原本的座標、中心、大小、區域等資訊計算
                        x1, y1, x2, y2 = box
                        width = x2 - x1
                        height = y2 - y1

                        # 中心點計算
                        center_x = (x1 + x2) / 2
                        center_y = (y1 + y2) / 2

                        # 標準化位置 (0-1)
                        norm_x = center_x / img_width
                        norm_y = center_y / img_height
                        norm_width = width / img_width
                        norm_height = height / img_height

                        # 面積計算
                        area = width * height
                        norm_area = area / (img_width * img_height)

                        # 區域判斷
                        object_region = "unknown"
                        if region_analyzer:
                            object_region = region_analyzer.determine_region(norm_x, norm_y)

                        # 調試信息：記錄映射過程
                        if class_name.startswith("unknown_class_"):
                            logger.warning(
                                f"Class ID {class_id} not found in class_names. "
                                f"Available keys: {list(self.class_names.keys())}"
                            )
                        else:
                            logger.debug(f"Successfully mapped class ID {class_id} to '{class_name}'")

                        detected_objects.append({
                            "class_id": int(class_id),
                            "class_name": class_name,
                            "confidence": float(confidence),
                            "box": [float(x1), float(y1), float(x2), float(y2)],
                            "center": [float(center_x), float(center_y)],
                            "normalized_center": [float(norm_x), float(norm_y)],
                            "size": [float(width), float(height)],
                            "normalized_size": [float(norm_width), float(norm_height)],
                            "area": float(area),
                            "normalized_area": float(norm_area),
                            "region": object_region
                        })

                    except Exception as e:
                        logger.error(f"Error processing object with class_id {class_id}: {str(e)}")
                        continue

                logger.info(f"Extracted {len(detected_objects)} objects from detection result")
                return detected_objects

            except Exception as e:
                logger.error(f"Error extracting detected objects: {str(e)}")
                logger.error(traceback.format_exc())
                return []

    def update_class_names(self, class_names: Dict[int, str]):
        """
        動態更新類別名稱映射

        Args:
            class_names: 新的類別名稱映射字典
        """
        try:
            self.class_names = class_names or {}
            logger.info(f"Class names updated: {len(self.class_names)} classes")
            logger.debug(f"Updated class names: {self.class_names}")
        except Exception as e:
            logger.error(f"Failed to update class names: {str(e)}")

    def categorize_object(self, obj: Dict) -> str:
        """
        將檢測到的物件分類到功能類別中，用於區域識別

        Args:
            obj: 物件字典

        Returns:
            物件功能類別字串
        """
        try:
            class_id = obj.get("class_id", -1)
            class_name = obj.get("class_name", "").lower()

            # 使用現有的類別映射（如果可用）
            if self.object_categories:
                for category, ids in self.object_categories.items():
                    if class_id in ids:
                        return category

            # 基於COCO類別名稱的後備分類
            furniture_items = ["chair", "couch", "bed", "dining table", "toilet"]
            plant_items = ["potted plant"]
            electronic_items = ["tv", "laptop", "mouse", "remote", "keyboard", "cell phone"]
            vehicle_items = ["bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat"]
            person_items = ["person"]
            kitchen_items = ["bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
                            "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
                            "pizza", "donut", "cake", "refrigerator", "oven", "toaster", "sink", "microwave"]
            sports_items = ["frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                        "baseball glove", "skateboard", "surfboard", "tennis racket"]
            personal_items = ["handbag", "tie", "suitcase", "umbrella", "backpack"]

            if any(item in class_name for item in furniture_items):
                return "furniture"
            elif any(item in class_name for item in plant_items):
                return "plant"
            elif any(item in class_name for item in electronic_items):
                return "electronics"
            elif any(item in class_name for item in vehicle_items):
                return "vehicle"
            elif any(item in class_name for item in person_items):
                return "person"
            elif any(item in class_name for item in kitchen_items):
                return "kitchen_items"
            elif any(item in class_name for item in sports_items):
                return "sports"
            elif any(item in class_name for item in personal_items):
                return "personal_items"
            else:
                return "misc"

        except Exception as e:
            logger.error(f"Error categorizing object: {str(e)}")
            logger.error(traceback.format_exc())
            return "misc"

    def get_object_categories(self, detected_objects: List[Dict]) -> set:
        """
        從檢測到的物件中取得唯一的物件類別

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            唯一物件類別的集合
        """
        try:
            object_categories = set()
            for obj in detected_objects:
                category = self.categorize_object(obj)
                if category:
                    object_categories.add(category)

            logger.info(f"Found {len(object_categories)} unique object categories")
            return object_categories

        except Exception as e:
            logger.error(f"Error getting object categories: {str(e)}")
            logger.error(traceback.format_exc())
            return set()

    def identify_core_objects_for_scene(self, detected_objects: List[Dict], scene_type: str) -> List[Dict]:
        """
        識別定義特定場景類型的核心物件

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            場景的核心物件列表
        """
        try:
            core_objects = []

            # 場景核心物件映射
            scene_core_mapping = {
                "bedroom": [59],  # bed
                "kitchen": [68, 69, 71, 72],  # microwave, oven, sink, refrigerator
                "living_room": [57, 58, 62],  # sofa, chair, tv
                "dining_area": [60, 46, 47],  # dining table, fork, knife
                "office_workspace": [63, 64, 66, 73]  # laptop, mouse, keyboard, book
            }

            if scene_type in scene_core_mapping:
                core_class_ids = scene_core_mapping[scene_type]
                for obj in detected_objects:
                    if obj.get("class_id") in core_class_ids and obj.get("confidence", 0) >= 0.4:
                        core_objects.append(obj)

            logger.info(f"Identified {len(core_objects)} core objects for scene type '{scene_type}'")
            return core_objects

        except Exception as e:
            logger.error(f"Error identifying core objects for scene '{scene_type}': {str(e)}")
            logger.error(traceback.format_exc())
            return []

    def group_objects_by_category_and_region(self, detected_objects: List[Dict]) -> Dict:
        """
        將物件按類別和區域分組

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            按類別和區域分組的物件字典
        """
        try:
            category_regions = {}

            for obj in detected_objects:
                category = self.categorize_object(obj)
                if not category:
                    continue

                if category not in category_regions:
                    category_regions[category] = {}

                region = obj.get("region", "center")
                if region not in category_regions[category]:
                    category_regions[category][region] = []

                category_regions[category][region].append(obj)

            logger.info(f"Grouped objects into {len(category_regions)} categories across regions")
            return category_regions

        except Exception as e:
            logger.error(f"Error grouping objects by category and region: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def filter_objects_by_confidence(self, detected_objects: List[Dict], min_confidence: float) -> List[Dict]:
        """
        根據信心度過濾物件

        Args:
            detected_objects: 檢測到的物件列表
            min_confidence: 最小信心度閾值

        Returns:
            過濾後的物件列表
        """
        try:
            filtered_objects = [
                obj for obj in detected_objects
                if obj.get("confidence", 0) >= min_confidence
            ]

            logger.info(f"Filtered {len(detected_objects)} objects to {len(filtered_objects)} objects with confidence >= {min_confidence}")
            return filtered_objects

        except Exception as e:
            logger.error(f"Error filtering objects by confidence: {str(e)}")
            logger.error(traceback.format_exc())
            return detected_objects  # 發生錯誤時返回原始列表

Writing object_extractor.py


In [ ]:
# %%writefile scene_viewpoint_analyzer.py

import logging
import traceback
import numpy as np
from typing import Dict, List, Any, Optional, Tuple

logger = logging.getLogger(__name__)

class SceneViewpointAnalyzer:
    """
    負責場景視角的檢測和模式辨識
    專注於檢測場景視角（俯視、平視等）並識別特殊場景模式（如十字路口、人流方向等）
    提供詳細的場景空間分析和視角相關的場景理解功能
    """

    def __init__(self, enhanced_scene_describer=None):
        """
        初始化場景視角分析器

        Args:
            enhanced_scene_describer: 增強場景描述器，用於基本視角檢測
        """
        try:
            self.enhanced_scene_describer = enhanced_scene_describer
            logger.info("SceneViewpointAnalyzer initialized successfully")

        except Exception as e:
            logger.error(f"Failed to initialize SceneViewpointAnalyzer: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def detect_viewpoint(self, detected_objects: List[Dict]) -> str:
        """
        檢測圖像視角類型

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            str: 檢測到的視角類型
        """
        try:
            # 使用內部的場景視角檢測方法
            viewpoint_info = self.detect_scene_viewpoint(detected_objects)
            return viewpoint_info.get("viewpoint", "eye_level")
        except Exception as e:
            logger.warning(f"Error detecting viewpoint: {str(e)}")
            return "eye_level"

    def get_viewpoint_confidence(self, detected_objects: List[Dict]) -> Tuple[str, float]:
        """
        獲取視角檢測結果及其信心度

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            Tuple[str, float]: (視角類型, 信心度)
        """
        try:
            viewpoint_info = self.detect_scene_viewpoint(detected_objects)
            viewpoint = viewpoint_info.get("viewpoint", "eye_level")

            # 根據檢測到的模式計算信心度
            patterns = viewpoint_info.get("patterns", [])
            confidence = 0.5  # 基礎信心度

            if "crosswalk_intersection" in patterns:
                confidence += 0.3
            if "consistent_object_size" in patterns:
                confidence += 0.2
            if "multi_directional_movement" in patterns:
                confidence += 0.1

            confidence = min(confidence, 1.0)
            return viewpoint, confidence

        except Exception as e:
            logger.error(f"Error getting viewpoint confidence: {str(e)}")
            return "eye_level", 0.5

    def detect_scene_viewpoint(self, detected_objects: List[Dict]) -> Dict:
        """
        檢測場景視角並識別特殊場景模式

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含視角和場景模式資訊的字典
        """
        try:
            if not detected_objects:
                logger.warning("No detected objects provided for viewpoint detection")
                return {"viewpoint": "eye_level", "patterns": []}

            # 從物件位置中提取資訊
            patterns = []

            # 檢測行人位置模式 - 篩選出所有行人物件
            pedestrian_objs = [obj for obj in detected_objects if obj.get("class_id") == 0]

            # 檢查是否有足夠的行人來辨識模式，至少需要4個行人才能進行模式分析
            if len(pedestrian_objs) >= 4:
                # 提取行人的標準化中心座標用於模式分析
                pedestrian_positions = [obj["normalized_center"] for obj in pedestrian_objs]

                # 檢測十字交叉模式，這通常出現在斑馬線交叉口的俯視圖
                if self._detect_cross_pattern(pedestrian_positions):
                    patterns.append("crosswalk_intersection")

                # 檢測多方向行人流 - 分析行人是否在多個方向移動
                directions = self._analyze_movement_directions(pedestrian_positions)
                if len(directions) >= 2:
                    patterns.append("multi_directional_movement")

            # 檢查物件的大小一致性，在空中俯視圖中，物件大小通常更一致
            # 因為距離相對均勻，不像地面視角會有遠近差異
            if len(detected_objects) >= 5:
                sizes = [obj.get("normalized_area", 0) for obj in detected_objects]
                # 計算標準化變異數，避免受平均值影響
                size_variance = np.var(sizes) / (np.mean(sizes) ** 2) if np.mean(sizes) > 0 else 0

                # 低變異表示大小一致，可能是俯視角度
                if size_variance < 0.3:
                    patterns.append("consistent_object_size")

            # 基本視角檢測，使用增強場景描述器進行基礎視角判斷
            viewpoint = "eye_level"  # 預設值
            if self.enhanced_scene_describer and hasattr(self.enhanced_scene_describer, '_detect_viewpoint'):
                viewpoint = self.enhanced_scene_describer._detect_viewpoint(detected_objects)

            # 根據檢測到的模式增強視角判斷
            # 如果檢測到斑馬線交叉但視角判斷不是空中視角，優先採用模式判斷
            if "crosswalk_intersection" in patterns and viewpoint != "aerial":
                viewpoint = "aerial"

            result = {
                "viewpoint": viewpoint,
                "patterns": patterns
            }

            logger.info(f"Viewpoint detection completed: {viewpoint}, patterns: {patterns}")
            return result

        except Exception as e:
            logger.error(f"Error in scene viewpoint detection: {str(e)}")
            logger.error(traceback.format_exc())
            return {"viewpoint": "eye_level", "patterns": []}

    def _detect_cross_pattern(self, positions: List[List[float]]) -> bool:
        """
        檢測位置中的十字交叉模式
        這種模式通常出現在十字路口的俯視圖中，行人分布呈現十字形

        Args:
            positions: 位置列表 [[x1, y1], [x2, y2], ...]

        Returns:
            是否檢測到十字交叉模式
        """
        try:
            if len(positions) < 8:  # 需要足夠多的點才能形成有意義的十字模式
                return False

            # 提取 x 和 y 座標進行分析
            x_coords = [pos[0] for pos in positions]
            y_coords = [pos[1] for pos in positions]

            # 計算座標的平均值，用於確定中心線位置
            x_mean = np.mean(x_coords)
            y_mean = np.mean(y_coords)

            # 計算在中心線附近的點數量
            # 如果有足夠多的點在垂直和水平中心線附近，可能是十字交叉
            near_x_center = sum(1 for x in x_coords if abs(x - x_mean) < 0.1)  # 10%的偏差
            near_y_center = sum(1 for y in y_coords if abs(y - y_mean) < 0.1)  # 10%的偏差

            # 十字交叉模式的判斷條件：垂直和水平方向都有足夠的點聚集
            is_cross_pattern = near_x_center >= 3 and near_y_center >= 3

            if is_cross_pattern:
                logger.info(f"Cross pattern detected with {near_x_center} points near vertical center and {near_y_center} points near horizontal center")

            return is_cross_pattern

        except Exception as e:
            logger.error(f"Error detecting cross pattern: {str(e)}")
            logger.error(traceback.format_exc())
            return False

    def _analyze_movement_directions(self, positions: List[List[float]]) -> List[str]:
        """
        分析位置中的移動方向
        通過分析座標分布範圍來推斷主要的移動方向

        Args:
            positions: 位置列表 [[x1, y1], [x2, y2], ...]

        Returns:
            檢測到的主要方向列表
        """
        try:
            if len(positions) < 6:  # 需要足夠的點才能分析方向性
                return []

            # 提取 x 和 y 座標
            x_coords = [pos[0] for pos in positions]
            y_coords = [pos[1] for pos in positions]

            directions = []

            # 水平移動分析（左右移動）
            # 計算x座標的標準差和範圍來判斷水平方向的分散程度
            x_std = np.std(x_coords)
            x_range = max(x_coords) - min(x_coords)

            # 垂直移動分析（上下移動）
            # 計算y座標的標準差和範圍來判斷垂直方向的分散程度
            y_std = np.std(y_coords)
            y_range = max(y_coords) - min(y_coords)

            # 足夠大的範圍表示該方向有明顯的運動或分散
            # 40%的圖像範圍被認為是有意義的移動範圍
            if x_range > 0.4:
                directions.append("horizontal")
                logger.debug(f"Horizontal movement detected with range: {x_range:.3f}")

            if y_range > 0.4:
                directions.append("vertical")
                logger.debug(f"Vertical movement detected with range: {y_range:.3f}")

            logger.info(f"Movement directions analyzed: {directions}")
            return directions

        except Exception as e:
            logger.error(f"Error analyzing movement directions: {str(e)}")
            logger.error(traceback.format_exc())
            return []

    def detect_aerial_view_indicators(self, detected_objects: List[Dict]) -> Dict:
        """
        檢測俯視角度的指標
        分析物件分布特徵來判斷是否為俯視角度

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含俯視角度指標的字典
        """
        try:
            indicators = {
                "consistent_sizing": False,
                "grid_like_distribution": False,
                "high_object_density": False,
                "aerial_score": 0.0
            }

            if not detected_objects:
                return indicators

            # 檢查物件大小的一致性
            sizes = [obj.get("normalized_area", 0) for obj in detected_objects]
            if len(sizes) >= 3:
                size_variance = np.var(sizes) / (np.mean(sizes) ** 2) if np.mean(sizes) > 0 else 1
                # 俯視角度通常物件大小較為一致
                indicators["consistent_sizing"] = size_variance < 0.3

            # 檢查是否有網格狀分布（如停車場的俯視圖）
            positions = [obj.get("normalized_center", [0.5, 0.5]) for obj in detected_objects]
            if len(positions) >= 6:
                # 簡化的網格檢測：檢查是否有規律的行列分布
                x_coords = [pos[0] for pos in positions]
                y_coords = [pos[1] for pos in positions]

                # 計算座標的分布是否接近規律網格
                x_unique = len(set([round(x, 1) for x in x_coords]))  # 四捨五入到小數第一位
                y_unique = len(set([round(y, 1) for y in y_coords]))

                # 如果x和y方向都有多個不同的規律位置，可能是網格分布
                indicators["grid_like_distribution"] = x_unique >= 3 and y_unique >= 3

            # 檢查物件密度
            total_objects = len(detected_objects)
            # 俯視角度通常能看到更多物件
            indicators["high_object_density"] = total_objects >= 8

            # 計算俯視角度評分
            score = 0
            if indicators["consistent_sizing"]:
                score += 0.4
            if indicators["grid_like_distribution"]:
                score += 0.4
            if indicators["high_object_density"]:
                score += 0.2

            indicators["aerial_score"] = score

            logger.info(f"Aerial view indicators: score={score:.2f}, consistent_sizing={indicators['consistent_sizing']}, grid_distribution={indicators['grid_like_distribution']}, high_density={indicators['high_object_density']}")
            return indicators

        except Exception as e:
            logger.error(f"Error detecting aerial view indicators: {str(e)}")
            logger.error(traceback.format_exc())
            return {
                "consistent_sizing": False,
                "grid_like_distribution": False,
                "high_object_density": False,
                "aerial_score": 0.0
            }

Writing scene_viewpoint_analyzer.py


In [ ]:
# %%writefile zone_evaluator.py

import logging
import traceback
import numpy as np
from typing import Dict, List, Any, Optional

logger = logging.getLogger(__name__)

class ZoneEvaluator:
    """
    負責功能區域辨識的可行性評估和物件關聯性計算
    評估是否應該進行區域劃分以及計算物件間的功能關聯性
    """

    def __init__(self):
        """初始化區域評估器"""
        try:
            # 定義物件間的功能關聯性評分表
            # 分數越高表示兩個物件在功能上越相關，更可能出現在同一功能區域
            self.relationship_pairs = {
                # 家具組合關係 - 這些組合通常出現在特定功能區域
                frozenset([56, 60]): 1.0,  # 椅子+桌子 (dining/work area)
                frozenset([57, 62]): 0.9,  # 沙發+電視 (living area)
                frozenset([59, 58]): 0.7,  # 床+植物 (bedroom decor)

                # 工作相關組合 - 工作環境的典型配置
                frozenset([63, 66]): 0.9,  # 筆電+鍵盤 (workspace)
                frozenset([63, 64]): 0.8,  # 筆電+滑鼠 (workspace)
                frozenset([60, 63]): 0.8,  # 桌子+筆電 (workspace)

                # 廚房相關組合 - 廚房設備的常見的物品
                frozenset([68, 72]): 0.9,  # 微波爐+冰箱 (kitchen)
                frozenset([69, 71]): 0.8,  # 烤箱+水槽 (kitchen)

                # 用餐相關組合 - 餐廳或用餐區域的典型物品
                frozenset([60, 40]): 0.8,  # 桌子+酒杯 (dining)
                frozenset([60, 41]): 0.8,  # 桌子+杯子 (dining)
                frozenset([56, 40]): 0.7,  # 椅子+酒杯 (dining)

                # 交通相關組合 - 城市交通的環境
                frozenset([2, 9]): 0.8,   # 汽車+交通燈 (traffic)
                frozenset([0, 9]): 0.7,   # 行人+交通燈 (crosswalk)
            }

            logger.info("ZoneEvaluator initialized with predefined relationship pairs")

        except Exception as e:
            logger.error(f"Failed to initialize ZoneEvaluator: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def evaluate_zone_identification_feasibility(self, detected_objects: List[Dict], scene_type: str) -> bool:
        """
        基於物件關聯性和分布特徵的彈性可行性評估
        決定是否應該進行功能區域劃分

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            是否適合進行區域識別
        """
        try:
            if len(detected_objects) < 2:
                logger.info("Insufficient objects for zone identification (minimum 2 required)")
                return False

            # 計算不同置信度層級的物件分布
            # 高信心度物件更可靠，用於核心區域判斷
            high_conf_objects = [obj for obj in detected_objects if obj.get("confidence", 0) >= 0.6]
            # 中等置信度物件提供補充資訊
            medium_conf_objects = [obj for obj in detected_objects if obj.get("confidence", 0) >= 0.4]

            # 基礎條件：至少需要一定數量的可信物件才值得進行區域分析
            if len(medium_conf_objects) < 2:
                logger.info("Insufficient medium confidence objects for zone identification")
                return False

            # 評估物件間的功能關聯性，關聯性高的物件更適合劃分功能區域
            functional_relationships = self.calculate_functional_relationships(detected_objects)

            # 評估空間分布多樣性 - 物件分散在多個區域才有劃分的意義
            spatial_diversity = self.calculate_spatial_diversity(detected_objects)

            # 綜合評分機制，用各項指標加權計算最終可行性評分
            feasibility_score = 0

            # 物件數量的貢獻（權重30%）- 更多物件提供更多劃分依據
            object_count_score = min(len(detected_objects) / 5.0, 1.0) * 0.3

            # 信心度質量貢獻（權重25%）- 高置信度物件比例影響可靠性
            confidence_score = len(high_conf_objects) / max(len(detected_objects), 1) * 0.25

            # 功能關聯性貢獻（權重25%）- 有功能關聯的物件更適合劃分區域
            relationship_score = functional_relationships * 0.25

            # 空間多樣性貢獻（權重20%）- 分散的物件才需要區域劃分
            diversity_score = spatial_diversity * 0.20

            feasibility_score = object_count_score + confidence_score + relationship_score + diversity_score

            # 動態閾值：根據場景複雜度調整可行性標準
            complexity_threshold = self.get_complexity_threshold(scene_type)

            is_feasible = feasibility_score >= complexity_threshold

            logger.info(f"Zone identification feasibility: {is_feasible} (score: {feasibility_score:.3f}, threshold: {complexity_threshold:.3f})")
            logger.debug(f"Score breakdown - objects: {object_count_score:.3f}, confidence: {confidence_score:.3f}, relationships: {relationship_score:.3f}, diversity: {diversity_score:.3f}")

            return is_feasible

        except Exception as e:
            logger.error(f"Error evaluating zone identification feasibility: {str(e)}")
            logger.error(traceback.format_exc())
            return False

    def calculate_functional_relationships(self, detected_objects: List[Dict]) -> float:
        """
        計算物件間的功能關聯性評分
        基於常見的物件組合模式評估功能相關性

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            功能關聯性評分 (0.0-1.0)
        """
        try:
            detected_class_ids = set(obj.get("class_id") for obj in detected_objects)
            max_possible_score = 0
            actual_score = 0

            # 遍歷所有預定義的關聯性組合，計算實際場景中的關聯性評分
            for pair, score in self.relationship_pairs.items():
                max_possible_score += score
                # 如果檢測到的物件中包含這個關聯組合，累加其評分
                if pair.issubset(detected_class_ids):
                    actual_score += score
                    logger.debug(f"Found functional relationship: {pair} with score {score}")

            # 標準化評分：實際評分除以最大可能評分
            relationship_score = actual_score / max_possible_score if max_possible_score > 0 else 0

            logger.info(f"Functional relationships calculated: {relationship_score:.3f} (found {actual_score:.1f}/{max_possible_score:.1f} possible relationships)")
            return relationship_score

        except Exception as e:
            logger.error(f"Error calculating functional relationships: {str(e)}")
            logger.error(traceback.format_exc())
            return 0

    def calculate_spatial_diversity(self, detected_objects: List[Dict]) -> float:
        """
        計算物件空間分布的多樣性
        評估物件是否分散在不同區域，避免所有物件集中在單一區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            空間多樣性評分 (0.0-1.0)
        """
        try:
            # 收集所有物件所在的不同區域
            regions = set(obj.get("region", "center") for obj in detected_objects)
            unique_regions = len(regions)

            # 標準化多樣性評分：假設理想情況是物件分散在2個以上區域
            # 更多區域意味著更高的空間多樣性，更適合進行區域劃分
            diversity_score = min(unique_regions / 2.0, 1.0)

            logger.info(f"Spatial diversity calculated: {diversity_score:.3f} (objects distributed across {unique_regions} regions)")
            return diversity_score

        except Exception as e:
            logger.error(f"Error calculating spatial diversity: {str(e)}")
            logger.error(traceback.format_exc())
            return 0

    def get_complexity_threshold(self, scene_type: str) -> float:
        """
        根據場景類型返回適當的複雜度閾值
        平衡不同場景的區域劃分需求

        Args:
            scene_type: 場景類型

        Returns:
            複雜度閾值 (0.0-1.0)
        """
        try:
            # 較簡單場景需要較高分數才進行區域劃分
            # 這些場景通常功能較為單純，不太需要細分
            simple_scenes = ["bedroom", "bathroom", "closet"]

            # 較複雜場景可以較低分數進行區域劃分
            # 這些場景通常有多種功能，適合劃分不同區域
            complex_scenes = ["living_room", "kitchen", "office_workspace", "dining_area"]

            if scene_type in simple_scenes:
                threshold = 0.65  # 較高閾值，避免過度細分
                logger.debug(f"Using high threshold {threshold} for simple scene: {scene_type}")
            elif scene_type in complex_scenes:
                threshold = 0.45  # 較低閾值，允許合理劃分
                logger.debug(f"Using low threshold {threshold} for complex scene: {scene_type}")
            else:
                threshold = 0.55  # 中等閾值，平衡策略
                logger.debug(f"Using medium threshold {threshold} for scene: {scene_type}")

            return threshold

        except Exception as e:
            logger.error(f"Error getting complexity threshold for scene '{scene_type}': {str(e)}")
            logger.error(traceback.format_exc())
            return 0.55  # 預設中等閾值

    def analyze_object_clustering(self, detected_objects: List[Dict]) -> Dict:
        """
        分析物件的聚集模式
        識別物件是否形成明顯的聚集群組，這有助於功能區域的劃分

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含聚集分析結果的字典
        """
        try:
            clustering_result = {
                "has_clusters": False,
                "cluster_count": 0,
                "cluster_regions": [],
                "clustering_score": 0.0
            }

            if len(detected_objects) < 3:
                logger.info("Insufficient objects for clustering analysis")
                return clustering_result

            # 統計每個區域的物件數量
            region_counts = {}
            for obj in detected_objects:
                region = obj.get("region", "unknown")
                region_counts[region] = region_counts.get(region, 0) + 1

            # 找出有顯著物件聚集的區域（物件數量 >= 2）
            significant_regions = [region for region, count in region_counts.items() if count >= 2]

            # 計算聚集：聚集區域數量與總區域數量的比例
            total_regions_with_objects = len([count for count in region_counts.values() if count > 0])
            clustering_score = len(significant_regions) / max(total_regions_with_objects, 1)

            clustering_result.update({
                "has_clusters": len(significant_regions) >= 2,
                "cluster_count": len(significant_regions),
                "cluster_regions": significant_regions,
                "clustering_score": clustering_score
            })

            logger.info(f"Object clustering analysis: {len(significant_regions)} clusters found in regions {significant_regions}")
            return clustering_result

        except Exception as e:
            logger.error(f"Error analyzing object clustering: {str(e)}")
            logger.error(traceback.format_exc())
            return {
                "has_clusters": False,
                "cluster_count": 0,
                "cluster_regions": [],
                "clustering_score": 0.0
            }

Overwriting zone_evaluator.py


In [ ]:
# %%writefile scene_zone_identifier.py

import logging
import traceback
import numpy as np
from typing import Dict, List, Any, Optional

logger = logging.getLogger(__name__)

class SceneZoneIdentifier:
    """
    負責不同場景類型的區域識別邏輯
    專注於根據場景類型執行相應的功能區域識別策略
    """

    def __init__(self):
        """初始化場景區域辨識器"""
        try:
            logger.info("SceneZoneIdentifier initialized successfully")

        except Exception as e:
            logger.error(f"Failed to initialize SceneZoneIdentifier: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def identify_indoor_zones(self, category_regions: Dict, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        平衡化的室內功能區域識別並標準化命名
        採用通用的物件關聯性分析，避免只針對特定場景

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            識別出的室內功能區域字典，使用描述性鍵名
        """
        try:
            zones = {}

            # 主要功能區域（基於物件關聯性而非場景類型）
            primary_zone = self._identify_primary_functional_area(detected_objects)
            if primary_zone:
                # 基於區域內容生成描述性鍵名
                descriptive_key = self._generate_descriptive_zone_key_from_data(primary_zone, "primary")
                zones[descriptive_key] = primary_zone

            # 只有明確證據且物件數量足夠時創建次要功能區域
            if len(zones) >= 1 and len(detected_objects) >= 6:
                secondary_zone = self._identify_secondary_functional_area(detected_objects, zones)
                if secondary_zone:
                    # 基於區域內容生成描述性鍵名
                    descriptive_key = self._generate_descriptive_zone_key_from_data(secondary_zone, "secondary")
                    zones[descriptive_key] = secondary_zone

            logger.info(f"Identified {len(zones)} indoor zones for scene type '{scene_type}'")
            return zones

        except Exception as e:
            logger.error(f"Error identifying indoor zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _generate_descriptive_zone_key_from_data(self, zone_data: Dict, priority_level: str) -> str:
        """
        基於區域數據生成描述性鍵名

        Args:
            zone_data: 區域數據字典
            priority_level: 優先級別（primary/secondary）

        Returns:
            str: 描述性區域鍵名
        """
        try:
            objects = zone_data.get("objects", [])
            region = zone_data.get("region", "")
            description = zone_data.get("description", "")

            # 基於物件內容確定功能類型
            if any("dining" in obj.lower() or "table" in obj.lower() for obj in objects):
                base_name = "dining area"
            elif any("chair" in obj.lower() or "sofa" in obj.lower() for obj in objects):
                base_name = "seating area"
            elif any("bed" in obj.lower() for obj in objects):
                base_name = "sleeping area"
            elif any("laptop" in obj.lower() or "keyboard" in obj.lower() for obj in objects):
                base_name = "workspace area"
            elif any("plant" in obj.lower() or "vase" in obj.lower() for obj in objects):
                base_name = "decorative area"
            elif any("refrigerator" in obj.lower() or "microwave" in obj.lower() for obj in objects):
                base_name = "kitchen area"
            else:
                # 基於描述內容推斷
                if "dining" in description.lower():
                    base_name = "dining area"
                elif "seating" in description.lower() or "relaxation" in description.lower():
                    base_name = "seating area"
                elif "work" in description.lower():
                    base_name = "workspace area"
                elif "decorative" in description.lower():
                    base_name = "decorative area"
                else:
                    base_name = "functional area"

            # 為次要區域添加位置標識以區分
            if priority_level == "secondary" and region:
                spatial_context = self._get_spatial_context_description(region)
                if spatial_context:
                    return f"{spatial_context} {base_name}"

            return base_name

        except Exception as e:
            logger.warning(f"Error generating descriptive zone key: {str(e)}")
            return "activity area"

    def _get_spatial_context_description(self, region: str) -> str:
        """
        獲取空間上下文描述

        Args:
            region: 區域位置標識

        Returns:
            str: 空間上下文描述
        """
        try:
            spatial_mapping = {
                "top_left": "upper left",
                "top_center": "upper",
                "top_right": "upper right",
                "middle_left": "left side",
                "middle_center": "central",
                "middle_right": "right side",
                "bottom_left": "lower left",
                "bottom_center": "lower",
                "bottom_right": "lower right"
            }

            return spatial_mapping.get(region, "")

        except Exception as e:
            logger.warning(f"Error getting spatial context for region '{region}': {str(e)}")
            return ""

    def identify_outdoor_general_zones(self, category_regions: Dict, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        識別一般戶外場景的功能區域

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表
            scene_type: 特定戶外場景類型

        Returns:
            戶外功能區域字典
        """
        try:
            zones = {}

            # 識別行人區域
            people_objs = [obj for obj in detected_objects if obj["class_id"] == 0]
            if people_objs:
                people_regions = {}
                for obj in people_objs:
                    region = obj["region"]
                    if region not in people_regions:
                        people_regions[region] = []
                    people_regions[region].append(obj)

                if people_regions:
                    # 找到主要的行人活動區域
                    main_people_regions = sorted(people_regions.items(),
                                            key=lambda x: len(x[1]),
                                            reverse=True)[:2]  # 取前2個區域

                    for idx, (region, objs) in enumerate(main_people_regions):
                        if len(objs) > 0:
                            # 生成基於位置的描述性鍵名
                            spatial_desc = self._get_directional_description(region)
                            if spatial_desc and spatial_desc != "central":
                                zone_key = f"{spatial_desc} pedestrian area"
                            else:
                                zone_key = "main pedestrian area" if idx == 0 else "secondary pedestrian area"

                            zones[zone_key] = {
                                "region": region,
                                "objects": ["person"] * len(objs),
                                "description": f"Pedestrian area with {len(objs)} {'people' if len(objs) > 1 else 'person'}"
                            }

            # 識別車輛區域，適用於街道和停車場
            vehicle_objs = [obj for obj in detected_objects if obj["class_id"] in [1, 2, 3, 5, 6, 7]]
            if vehicle_objs:
                vehicle_regions = {}
                for obj in vehicle_objs:
                    region = obj["region"]
                    if region not in vehicle_regions:
                        vehicle_regions[region] = []
                    vehicle_regions[region].append(obj)

                if vehicle_regions:
                    main_vehicle_region = max(vehicle_regions.items(),
                                        key=lambda x: len(x[1]),
                                        default=(None, []))

                    if main_vehicle_region[0] is not None:
                        vehicle_types = [obj["class_name"] for obj in main_vehicle_region[1]]
                        zones["vehicle_zone"] = {
                            "region": main_vehicle_region[0],
                            "objects": vehicle_types,
                            "description": f"Traffic area with {', '.join(list(set(vehicle_types))[:3])}"
                        }

            # 針對公園區域的特殊處理
            if scene_type == "park_area":
                zones.update(self._identify_park_recreational_zones(detected_objects))

            # 針對停車場的特殊處理
            if scene_type == "parking_lot":
                zones.update(self._identify_parking_zones(detected_objects))

            logger.info(f"Identified {len(zones)} outdoor zones for scene type '{scene_type}'")
            return zones

        except Exception as e:
            logger.error(f"Error identifying outdoor general zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_intersection_zones(self, category_regions: Dict, detected_objects: List[Dict], viewpoint: str) -> Dict:
        """
        辨識城市十字路口的功能區域，無論是否有行人，只要偵測到紅綠燈就一定顯示 Traffic Control Area；
        若有行人，則額外建立 Crossing Zone 並把行人 + 同 region 的紅綠燈歸在一起。

        Args:
            category_regions: 按類別和 region 分組的物件字典
            detected_objects: YOLO 檢測到的所有物件列表
            viewpoint: 偵測到的視角字串

        Returns:
            zones: 最終的十字路口功能區域字典
        """
        try:
            zones = {}

            # 1. 按 class_id 分出行人、車輛、紅綠燈
            pedestrian_objs    = [obj for obj in detected_objects if obj["class_id"] == 0]
            vehicle_objs       = [obj for obj in detected_objects if obj["class_id"] in [1, 2, 3, 5, 7]]
            traffic_light_objs = [obj for obj in detected_objects if obj["class_id"] == 9]

            # 2. Step A: 無條件建立 Traffic Control Area
            #    把每個 region 下的紅綠燈都先分群，生成對應 zone，確保「只要偵測到紅綠燈就一定顯示」
            signal_regions_all = {}
            for t in traffic_light_objs:
                region = t["region"]
                signal_regions_all.setdefault(region, []).append(t)

            for idx, (region, signals) in enumerate(signal_regions_all.items()):
                # 先決定 zone_key (依 direction 或 primary/auxiliary)
                direction = self._get_directional_description(region)
                if direction and direction != "central":
                    zone_key = f"{direction} traffic control area"
                else:
                    zone_key = "primary traffic control area" if idx == 0 else "auxiliary traffic control area"

                # 確保命名不衝突
                if zone_key in zones:
                    suffix = 1
                    new_key = f"{zone_key} ({suffix})"
                    while new_key in zones:
                        suffix += 1
                        new_key = f"{zone_key} ({suffix})"
                    zone_key = new_key

                zones[zone_key] = {
                    "region": region,
                    "objects": ["traffic light"] * len(signals),
                    "description": f"Traffic control area with {len(signals)} traffic lights in {region}"
                }

            # (用於後面計算 Crossing 使用掉的 traffic light)
            used_tl_count_per_region = dict.fromkeys(signal_regions_all.keys(), 0)

            # 3. Step B: 如果有行人，就建立 Crossing Zone，並移除已被打包的紅綠燈
            if pedestrian_objs:
                # 先呼叫 _analyze_crossing_patterns，讓它回傳「行人 + 同 region 的紅綠燈」區
                crossing_zones = self._analyze_crossing_patterns(pedestrian_objs, traffic_light_objs)

                # 把 Crossing Zone 加到最終 zones，並同時記錄已使用掉的紅綠燈數量
                for zone_key, zone_info in crossing_zones.items():
                    region = zone_info.get("region", "")
                    obj_list = zone_info.get("objects", [])

                    # 如果該 zone_info["objects"] 裡含有紅綠燈，就累加到 used_tl_count_per_region
                    count_in_zone = obj_list.count("traffic light")
                    if count_in_zone > 0:
                        used_tl_count_per_region[region] = used_tl_count_per_region.get(region, 0) + count_in_zone

                    # 加入最終結果
                    # 如果 key 重複，也可以在此加上 index，或直接覆蓋
                    if zone_key in zones:
                        suffix = 1
                        new_key = f"{zone_key} ({suffix})"
                        while new_key in zones:
                            suffix += 1
                            new_key = f"{zone_key} ({suffix})"
                        zone_key = new_key

                    zones[zone_key] = {
                        "region": region,
                        "objects": obj_list,
                        "description": zone_info.get("description", "")
                    }

            # 4. Step C: 計算並顯示 debug 資訊 (Total / Used / Remaining)
            for region, signals in signal_regions_all.items():
                total = len(signals)
                used = used_tl_count_per_region.get(region, 0)
                remaining = total - used
                # print(f"[DEBUG] Region '{region}': Total TL = {total}, Used in crossing = {used}, Remaining = {remaining}")

            # 5. Step D: 分析車輛交通區域（Vehicle Zones）
            if vehicle_objs:
                traffic_zones = self._analyze_traffic_zones(vehicle_objs)
                # _analyze_traffic_zones 內部已用英文 debug，直接更新
                for zone_key, zone_info in traffic_zones.items():
                    if zone_key in zones:
                        suffix = 1
                        new_key = f"{zone_key} ({suffix})"
                        while new_key in zones:
                            suffix += 1
                            new_key = f"{zone_key} ({suffix})"
                        zone_key = new_key
                    zones[zone_key] = zone_info

            logger.info(f"Identified {len(zones)} intersection zones")
            return zones

        except Exception as e:
            logger.error(f"Error in identify_intersection_zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_aerial_view_zones(self, category_regions: Dict, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        辨識空中視角場景的功能區域
        專注於模式和流動而非特定區域

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表
            scene_type: 特定場景類型

        Returns:
            空中視角功能區域字典
        """
        try:
            zones = {}

            # 識別行人模式
            people_objs = [obj for obj in detected_objects if obj["class_id"] == 0]
            if people_objs:
                # 將位置轉換為數組進行模式分析
                positions = np.array([obj["normalized_center"] for obj in people_objs])

                if len(positions) >= 3:
                    # 計算分布指標
                    x_coords = positions[:, 0]
                    y_coords = positions[:, 1]

                    x_mean = np.mean(x_coords)
                    y_mean = np.mean(y_coords)
                    x_std = np.std(x_coords)
                    y_std = np.std(y_coords)

                    # 判斷人群是否組織成線性模式
                    if x_std < 0.1 or y_std < 0.1:
                        # 沿一個軸的線性分布
                        pattern_direction = "vertical" if x_std < y_std else "horizontal"

                        zones["pedestrian_pattern"] = {
                            "region": "central",
                            "objects": ["person"] * len(people_objs),
                            "description": f"Aerial view shows a {pattern_direction} pedestrian movement pattern"
                        }
                    else:
                        # 更分散的模式
                        zones["pedestrian_distribution"] = {
                            "region": "wide",
                            "objects": ["person"] * len(people_objs),
                            "description": f"Aerial view shows pedestrians distributed across the area"
                        }

            # 識別車輛模式進行交通分析
            vehicle_objs = [obj for obj in detected_objects if obj["class_id"] in [1, 2, 3, 5, 6, 7]]
            if vehicle_objs:
                zones.update(self._analyze_aerial_traffic_patterns(vehicle_objs))

            # 針對十字路口特定空中視角的處理
            if "intersection" in scene_type:
                zones.update(self._identify_aerial_intersection_features(detected_objects))

            # 針對廣場空中視角的處理
            if "plaza" in scene_type:
                zones.update(self._identify_aerial_plaza_features(people_objs))

            logger.info(f"Identified {len(zones)} aerial view zones")
            return zones

        except Exception as e:
            logger.error(f"Error identifying aerial view zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_asian_cultural_zones(self, category_regions: Dict, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        辨識有亞洲文化背景的場景功能區域

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表
            scene_type: 特定場景類型

        Returns:
            亞洲文化功能區域字典
        """
        try:
            zones = {}

            # 識別店面區域
            # 由於店面不能直接檢測，從情境推斷
            # 例如，尋找有標誌、行人和小物件的區域
            storefront_regions = {}
            for obj in detected_objects:
                if obj["class_id"] == 0:  # Person
                    region = obj["region"]
                    if region not in storefront_regions:
                        storefront_regions[region] = []
                    storefront_regions[region].append(obj)

            # 將人最多的區域作為店面區域
            if storefront_regions:
                main_storefront_regions = sorted(storefront_regions.items(),
                                            key=lambda x: len(x[1]),
                                            reverse=True)[:2]  # 前2個區域

                for idx, (region, objs) in enumerate(main_storefront_regions):
                    # 生成基於位置的描述性鍵名
                    spatial_desc = self._get_directional_description(region)
                    if spatial_desc and spatial_desc != "central":
                        zone_key = f"{spatial_desc} commercial area"
                    else:
                        zone_key = "main commercial area" if idx == 0 else "secondary commercial area"

                    zones[zone_key] = {
                        "region": region,
                        "objects": [obj["class_name"] for obj in objs],
                        "description": f"Asian commercial storefront with pedestrian activity"
                    }

            # 辨識行人通道
            zones.update(self._identify_asian_pedestrian_pathway(detected_objects))

            # 辨識攤販區域（小攤/商店 - 從情境推斷）
            zones.update(self._identify_vendor_zones(detected_objects))

            # 針對夜市的特殊處理
            if scene_type == "asian_night_market":
                zones["food_stall_zone"] = {
                    "region": "middle_center",
                    "objects": ["inferred food stalls"],
                    "description": "Food stall area typical of Asian night markets"
                }

            logger.info(f"Identified {len(zones)} Asian cultural zones")
            return zones

        except Exception as e:
            logger.error(f"Error identifying Asian cultural zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_upscale_dining_zones(self, category_regions: Dict, detected_objects: List[Dict]) -> Dict:
        """
        辨識高級餐飲設置的功能區域

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表

        Returns:
            高級餐飲功能區域字典
        """
        try:
            zones = {}

            # 辨識餐桌區域
            dining_items = []
            dining_regions = {}

            for obj in detected_objects:
                if obj["class_id"] in [40, 41, 42, 43, 44, 45, 60]:  # Wine glass, cup, fork, knife, spoon, bowl, table
                    region = obj["region"]
                    if region not in dining_regions:
                        dining_regions[region] = []
                    dining_regions[region].append(obj)
                    dining_items.append(obj["class_name"])

            if dining_items:
                main_dining_region = max(dining_regions.items(),
                                    key=lambda x: len(x[1]),
                                    default=(None, []))

                if main_dining_region[0] is not None:
                    zones["formal_dining_zone"] = {
                        "region": main_dining_region[0],
                        "objects": list(set(dining_items)),
                        "description": f"Formal dining area with {', '.join(list(set(dining_items))[:3])}"
                    }

            # 識別裝飾區域，增強檢測
            zones.update(self._identify_upscale_decorative_zones(detected_objects))

            # 識別座位安排區域
            zones.update(self._identify_dining_seating_zones(detected_objects))

            # 識別服務區域（如果與餐飲區域不同）
            zones.update(self._identify_serving_zones(detected_objects, zones))

            logger.info(f"Identified {len(zones)} upscale dining zones")
            return zones

        except Exception as e:
            logger.error(f"Error identifying upscale dining zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_financial_district_zones(self, category_regions: Dict, detected_objects: List[Dict]) -> Dict:
        """
        金融區場景的功能區域

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表

        Returns:
            金融區功能區域字典
        """
        try:
            zones = {}

            # 識別交通區域
            traffic_items = []
            traffic_regions = {}

            for obj in detected_objects:
                if obj["class_id"] in [1, 2, 3, 5, 6, 7, 9]:  # 各種車輛和交通燈
                    region = obj["region"]
                    if region not in traffic_regions:
                        traffic_regions[region] = []
                    traffic_regions[region].append(obj)
                    traffic_items.append(obj["class_name"])

            if traffic_items:
                main_traffic_region = max(traffic_regions.items(),
                                    key=lambda x: len(x[1]),
                                    default=(None, []))

                if main_traffic_region[0] is not None:
                    zones["traffic_zone"] = {
                        "region": main_traffic_region[0],
                        "objects": list(set(traffic_items)),
                        "description": f"Urban traffic area with {', '.join(list(set(traffic_items))[:3])}"
                    }

            # 側邊建築區域（從場景情境推斷）
            zones.update(self._identify_building_zones(detected_objects))

            # 行人區域
            zones.update(self._identify_financial_pedestrian_zones(detected_objects))

            logger.info(f"Identified {len(zones)} financial district zones")
            return zones

        except Exception as e:
            logger.error(f"Error identifying financial district zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def identify_landmark_zones(self, landmark_objects: List[Dict]) -> Dict:
        """
        辨識與地標相關的功能區域

        Args:
            landmark_objects: 被辨識為地標的物體列表

        Returns:
            地標相關的功能區域字典
        """
        try:
            landmark_zones = {}

            # 如果沒有任何地標，就直接回空字典
            if not landmark_objects:
                logger.warning("No landmark objects provided to identify_landmark_zones")
                return landmark_zones

            # 只取第一個地標來示範：至少產生一個地標
            landmark = landmark_objects[0]
            # 確保傳入的 landmark 是 dict
            if not isinstance(landmark, dict):
                logger.warning("First landmark object is not a dict")
                return landmark_zones

            # 從 landmark dict 拿出必要欄位
            landmark_id = landmark.get("landmark_id", "unknown_landmark")
            landmark_name = landmark.get("class_name", "Landmark")
            landmark_type = landmark.get("landmark_type", "architectural")
            landmark_region = landmark.get("region", "middle_center")

            # 如果 location 沒提供，就給預設 "this area"
            location = landmark.get("location")
            if not location:
                location = "this area"

            # 為地標創建主要觀景區
            zone_id = f"{landmark_name.lower().replace(' ', '_')}_viewing_area"
            zone_name = f"{landmark_name} Viewing Area"

            # 根據地標類型調整描述，並確保帶入地點
            if landmark_type == "natural":
                zone_description = (
                    f"Scenic viewpoint for observing {landmark_name}, "
                    f"a notable natural landmark in {location}."
                )
                primary_function = "Nature observation and photography"
            elif landmark_type == "monument":
                zone_description = (
                    f"Viewing area around {landmark_name}, "
                    f"a significant monument in {location}."
                )
                primary_function = "Historical appreciation and cultural tourism"
            else:  # architectural
                zone_description = (
                    f"Area centered around {landmark_name}, "
                    f"where visitors can observe and appreciate this iconic structure in {location}."
                )
                primary_function = "Architectural tourism and photography"

            # 確定與地標相關的物體（如果被偵測到）
            related_objects = []
            for o in landmark_objects:
                cn = o.get("class_name", "").lower()
                if cn in ["person", "camera", "cell phone", "backpack"]:
                    related_objects.append(cn)

            # 建立地標功能區
            landmark_zones[zone_id] = {
                "name": zone_name,
                "description": zone_description,
                "objects": ["landmark"] + related_objects,
                "region": landmark_region,
                "primary_function": primary_function
            }

            # 創建相關輔助功能區，如攝影區、紀念品販賣區
            auxiliary_zones = self._create_landmark_auxiliary_zones(landmark, 0)
            if auxiliary_zones:
                landmark_zones.update(auxiliary_zones)

            logger.info(f"Identified {len(landmark_zones)} landmark zones")
            return landmark_zones

        except Exception as e:
            logger.error(f"Error in identify_landmark_zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}


    def _identify_primary_functional_area(self, detected_objects: List[Dict]) -> Dict:
        """
        識別主要功能區域，基於最強的物件關聯性組合
        採用通用邏輯處理各種室內場景

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            主要功能區域字典或None
        """
        try:
            # 用餐區域檢測（桌椅組合）
            dining_area = self._detect_functional_combination(
                detected_objects,
                primary_objects=[60],  # dining table
                supporting_objects=[56, 40, 41, 42, 43],  # chair, wine glass, cup, fork, knife
                min_supporting=2,
                description_template="Dining area with table and seating arrangement"
            )
            if dining_area:
                return dining_area

            # 休息區域檢測（沙發電視組合或床）
            seating_area = self._detect_functional_combination(
                detected_objects,
                primary_objects=[57, 59],  # sofa, bed
                supporting_objects=[62, 58, 56],  # tv, potted plant, chair
                min_supporting=1,
                description_template="Seating and relaxation area"
            )
            if seating_area:
                return seating_area

            # 工作區域檢測（電子設備與家具組合）
            work_area = self._detect_functional_combination(
                detected_objects,
                primary_objects=[63, 66],  # laptop, keyboard
                supporting_objects=[60, 56, 64],  # dining table, chair, mouse
                min_supporting=2,
                description_template="Workspace area with electronics and furniture"
            )
            if work_area:
                return work_area

            return None

        except Exception as e:
            logger.error(f"Error identifying primary functional area: {str(e)}")
            logger.error(traceback.format_exc())
            return None

    def _identify_secondary_functional_area(self, detected_objects: List[Dict], existing_zones: Dict) -> Dict:
        """
        識別次要功能區域，避免與主要區域重疊

        Args:
            detected_objects: 檢測到的物件列表
            existing_zones: 已存在的功能區域

        Returns:
            次要功能區域字典或None
        """
        try:
            # 獲取已使用的區域
            used_regions = set(zone.get("region") for zone in existing_zones.values())

            # 裝飾區域檢測（植物集中區域）
            decorative_area = self._detect_functional_combination(
                detected_objects,
                primary_objects=[58],  # potted plant
                supporting_objects=[75],  # vase
                min_supporting=0,
                min_primary=3,  # 至少需要3個植物
                description_template="Decorative area with plants and ornamental items",
                exclude_regions=used_regions
            )
            if decorative_area:
                return decorative_area

            # 儲存區域檢測（廚房電器組合）
            storage_area = self._detect_functional_combination(
                detected_objects,
                primary_objects=[72, 68, 69],  # refrigerator, microwave, oven
                supporting_objects=[71],  # sink
                min_supporting=0,
                min_primary=2,
                description_template="Kitchen appliance and storage area",
                exclude_regions=used_regions
            )
            if storage_area:
                return storage_area

            return None

        except Exception as e:
            logger.error(f"Error identifying secondary functional area: {str(e)}")
            logger.error(traceback.format_exc())
            return None

    def _detect_functional_combination(self, detected_objects: List[Dict], primary_objects: List[int],
                                    supporting_objects: List[int], min_supporting: int,
                                    description_template: str, min_primary: int = 1,
                                    exclude_regions: set = None) -> Dict:
        """
        通用的功能組合檢測方法
        基於主要物件和支持物件的組合判斷功能區域

        Args:
            detected_objects: 檢測到的物件列表
            primary_objects: 主要物件的class_id列表
            supporting_objects: 支持物件的class_id列表
            min_supporting: 最少需要的支持物件數量
            description_template: 描述模板
            min_primary: 最少需要的主要物件數量
            exclude_regions: 需要排除的區域集合

        Returns:
            功能區域資訊字典，如果不符合條件則返回None
        """
        try:
            if exclude_regions is None:
                exclude_regions = set()

            # 收集主要物件
            primary_objs = [obj for obj in detected_objects
                        if obj.get("class_id") in primary_objects and obj.get("confidence", 0) >= 0.4]

            # 收集支持物件
            supporting_objs = [obj for obj in detected_objects
                            if obj.get("class_id") in supporting_objects and obj.get("confidence", 0) >= 0.4]

            # 檢查是否滿足最少數量要求
            if len(primary_objs) < min_primary or len(supporting_objs) < min_supporting:
                return None

            # 按區域組織物件
            region_combinations = {}
            all_relevant_objs = primary_objs + supporting_objs

            for obj in all_relevant_objs:
                region = obj.get("region")

                # 排除指定區域
                if region in exclude_regions:
                    continue

                if region not in region_combinations:
                    region_combinations[region] = {"primary": [], "supporting": [], "all": []}

                region_combinations[region]["all"].append(obj)

                if obj.get("class_id") in primary_objects:
                    region_combinations[region]["primary"].append(obj)
                else:
                    region_combinations[region]["supporting"].append(obj)

            # 找到最佳區域組合
            best_region = None
            best_score = 0

            for region, objs in region_combinations.items():
                # 計算該區域的評分
                primary_count = len(objs["primary"])
                supporting_count = len(objs["supporting"])

                # 必須滿足最低要求
                if primary_count < min_primary or supporting_count < min_supporting:
                    continue

                # 計算組合評分（主要物件權重較高）
                score = primary_count * 2 + supporting_count

                if score > best_score:
                    best_score = score
                    best_region = region

            if best_region is None:
                return None

            best_combination = region_combinations[best_region]
            all_objects = [obj["class_name"] for obj in best_combination["all"]]

            return {
                "region": best_region,
                "objects": all_objects,
                "description": description_template
            }

        except Exception as e:
            logger.error(f"Error detecting functional combination: {str(e)}")
            logger.error(traceback.format_exc())
            return None

    def _analyze_crossing_patterns(self, pedestrians: List[Dict], traffic_lights: List[Dict]) -> Dict:
        """
        Analyze pedestrian crossing patterns to identify crossing zones.
        若同一 region 中同時有行人與紅綠燈，則將兩者都放入該區域的 objects。

        Args:
            pedestrians: 行人物件列表（每個 obj 應包含 'class_id', 'region', 'confidence' 等）
            traffic_lights: 紅綠燈物件列表（每個 obj 應包含 'class_id', 'region', 'confidence' 等）

        Returns:
            crossing_zones: 字典，key 為 zone 名稱，value 包含 'region', 'objects', 'description'
        """
        try:
            crossing_zones = {}

            # 如果沒有任何行人，就不辨識任何 crossing zone
            if not pedestrians:
                return crossing_zones

            # (1) 按照 region 分組行人
            pedestrian_regions = {}
            for p in pedestrians:
                region = p["region"]
                pedestrian_regions.setdefault(region, []).append(p)

            # (2) 針對每個 region，看是否同時有紅綠燈
            # 建立一個 mapping： region -> { "pedestrians": [...], "traffic_lights": [...] }
            combined_regions = {}
            for region, peds in pedestrian_regions.items():
                # 取得該 region 下所有紅綠燈
                tls_in_region = [t for t in traffic_lights if t["region"] == region]
                combined_regions[region] = {
                    "pedestrians": peds,
                    "traffic_lights": tls_in_region
                }

            # (3) 按照行人數量排序，找出前兩個需要建立 crossing zone 的 region
            sorted_regions = sorted(
                combined_regions.items(),
                key=lambda x: len(x[1]["pedestrians"]),
                reverse=True
            )

            # (4) 將前兩個 region 建立 Crossing Zone，objects 同時包含行人與紅綠燈
            for idx, (region, group) in enumerate(sorted_regions[:2]):
                peds = group["pedestrians"]
                tls  = group["traffic_lights"]
                has_nearby_signals = len(tls) > 0

                # 生成 zone_name（基於 region 方向 + idx 決定主/次 crossing）
                direction = self._get_directional_description(region)
                if direction and direction != "central":
                    zone_name = f"{direction} crossing area"
                else:
                    zone_name = "main crossing area" if idx == 0 else "secondary crossing area"

                # 組合 description
                description = f"Pedestrian crossing area with {len(peds)} "
                description += "person" if len(peds) == 1 else "people"
                if direction:
                    description += f" in {direction} direction"
                if has_nearby_signals:
                    description += " near traffic signals"

                # ======= 將行人 + 同區紅綠燈一併放入 objects =======
                obj_list = ["pedestrian"] * len(peds)
                if has_nearby_signals:
                    obj_list += ["traffic light"] * len(tls)

                crossing_zones[zone_name] = {
                    "region": region,
                    "objects": obj_list,
                    "description": description
                }

            return crossing_zones

        except Exception as e:
            logger.error(f"Error in _analyze_crossing_patterns: {str(e)}")
            logger.error(traceback.format_exc())
            return {}


    def _analyze_traffic_zones(self, vehicles: List[Dict]) -> Dict:
        """
        分析車輛分布以識別具有方向感知的交通區域

        Args:
            vehicles: 車輛物件列表

        Returns:
            識別出的交通區域字典
        """
        try:
            traffic_zones = {}

            if not vehicles:
                return traffic_zones

            # 按區域分組車輛
            vehicle_regions = {}
            for v in vehicles:
                region = v["region"]
                if region not in vehicle_regions:
                    vehicle_regions[region] = []
                vehicle_regions[region].append(v)

            # 為有車輛的區域創建交通區域
            main_traffic_region = max(vehicle_regions.items(), key=lambda x: len(x[1]), default=(None, []))

            if main_traffic_region[0] is not None:
                region = main_traffic_region[0]
                vehicles_in_region = main_traffic_region[1]

                # 獲取車輛類型列表用於描述
                vehicle_types = [v["class_name"] for v in vehicles_in_region]
                unique_types = list(set(vehicle_types))

                # 獲取方向描述
                direction = self._get_directional_description(region)

                # 創建描述性區域
                traffic_zones["vehicle_zone"] = {
                    "region": region,
                    "objects": vehicle_types,
                    "description": f"Vehicle traffic area with {', '.join(unique_types[:3])}" +
                                (f" in {direction} area" if direction else "")
                }

                # 如果車輛分布在多個區域，創建次要區域
                if len(vehicle_regions) > 1:
                    # 獲取第二大車輛聚集區域
                    sorted_regions = sorted(vehicle_regions.items(), key=lambda x: len(x[1]), reverse=True)
                    if len(sorted_regions) > 1:
                        second_region, second_vehicles = sorted_regions[1]
                        direction = self._get_directional_description(second_region)
                        vehicle_types = [v["class_name"] for v in second_vehicles]
                        unique_types = list(set(vehicle_types))

                        traffic_zones["secondary_vehicle_zone"] = {
                            "region": second_region,
                            "objects": vehicle_types,
                            "description": f"Secondary traffic area with {', '.join(unique_types[:2])}" +
                                        (f" in {direction} direction" if direction else "")
                        }

            return traffic_zones

        except Exception as e:
            logger.error(f"Error analyzing traffic zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _get_directional_description(self, region: str) -> str:
        """
        將區域名稱轉換為方位描述（東西南北）

        Args:
            region: 區域名稱

        Returns:
            方位描述字串
        """
        try:
            region_lower = region.lower()

            if "top" in region_lower and "left" in region_lower:
                return "northwest"
            elif "top" in region_lower and "right" in region_lower:
                return "northeast"
            elif "bottom" in region_lower and "left" in region_lower:
                return "southwest"
            elif "bottom" in region_lower and "right" in region_lower:
                return "southeast"
            elif "top" in region_lower:
                return "north"
            elif "bottom" in region_lower:
                return "south"
            elif "left" in region_lower:
                return "west"
            elif "right" in region_lower:
                return "east"
            else:
                return "central"

        except Exception as e:
            logger.error(f"Error getting directional description for region '{region}': {str(e)}")
            return "central"

    def _identify_park_recreational_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別公園的休閒活動區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            休閒區域字典
        """
        try:
            zones = {}

            # 尋找休閒物件（運動球、風箏等）
            rec_items = []
            rec_regions = {}

            for obj in detected_objects:
                if obj["class_id"] in [32, 33, 34, 35, 38]:  # sports ball, kite, baseball bat, glove, tennis racket
                    region = obj["region"]
                    if region not in rec_regions:
                        rec_regions[region] = []
                    rec_regions[region].append(obj)
                    rec_items.append(obj["class_name"])

            if rec_items:
                main_rec_region = max(rec_regions.items(),
                                key=lambda x: len(x[1]),
                                default=(None, []))

                if main_rec_region[0] is not None:
                    zones["recreational_zone"] = {
                        "region": main_rec_region[0],
                        "objects": list(set(rec_items)),
                        "description": f"Recreational area with {', '.join(list(set(rec_items)))}"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error identifying park recreational zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_parking_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        停車場的停車區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            停車區域字典
        """
        try:
            zones = {}

            # 尋找停放的汽車
            car_objs = [obj for obj in detected_objects if obj["class_id"] == 2]  # cars

            if len(car_objs) >= 3:
                # 檢查汽車是否按模式排列（簡化）
                car_positions = [obj["normalized_center"] for obj in car_objs]

                # 通過分析垂直位置檢查行模式
                y_coords = [pos[1] for pos in car_positions]
                y_clusters = {}

                # 簡化聚類 - 按相似y坐標分組汽車
                for i, y in enumerate(y_coords):
                    assigned = False
                    for cluster_y in y_clusters.keys():
                        if abs(y - cluster_y) < 0.1:  # 圖像高度的10%內
                            y_clusters[cluster_y].append(i)
                            assigned = True
                            break

                    if not assigned:
                        y_clusters[y] = [i]

                # 如果有行模式
                if max(len(indices) for indices in y_clusters.values()) >= 2:
                    zones["parking_row"] = {
                        "region": "central",
                        "objects": ["car"] * len(car_objs),
                        "description": f"Organized parking area with vehicles arranged in rows"
                    }
                else:
                    zones["parking_area"] = {
                        "region": "wide",
                        "objects": ["car"] * len(car_objs),
                        "description": f"Parking area with {len(car_objs)} vehicles"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error identifying parking zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _analyze_aerial_traffic_patterns(self, vehicle_objs: List[Dict]) -> Dict:
        """
        分析空中視角的車輛交通模式

        Args:
            vehicle_objs: 車輛物件列表

        Returns:
            交通模式區域字典
        """
        try:
            zones = {}

            if not vehicle_objs:
                return zones

            # 將位置轉換為數組進行模式分析
            positions = np.array([obj["normalized_center"] for obj in vehicle_objs])

            if len(positions) >= 2:
                # 計算分布指標
                x_coords = positions[:, 0]
                y_coords = positions[:, 1]

                x_mean = np.mean(x_coords)
                y_mean = np.mean(y_coords)
                x_std = np.std(x_coords)
                y_std = np.std(y_coords)

                # 判斷車輛是否組織成車道
                if x_std < y_std * 0.5:
                    # 車輛垂直對齊 - 表示南北交通
                    zones["vertical_traffic_flow"] = {
                        "region": "central_vertical",
                        "objects": [obj["class_name"] for obj in vehicle_objs[:5]],
                        "description": "North-south traffic flow visible from aerial view"
                    }
                elif y_std < x_std * 0.5:
                    # 車輛水平對齊 - 表示東西交通
                    zones["horizontal_traffic_flow"] = {
                        "region": "central_horizontal",
                        "objects": [obj["class_name"] for obj in vehicle_objs[:5]],
                        "description": "East-west traffic flow visible from aerial view"
                    }
                else:
                    # 車輛多方向 - 表示十字路口
                    zones["intersection_traffic"] = {
                        "region": "central",
                        "objects": [obj["class_name"] for obj in vehicle_objs[:5]],
                        "description": "Multi-directional traffic at intersection visible from aerial view"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error analyzing aerial traffic patterns: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_aerial_intersection_features(self, detected_objects: List[Dict]) -> Dict:
        """
        空中視角十字路口特徵

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            十字路口特徵區域字典
        """
        try:
            zones = {}

            # 檢查交通信號
            traffic_light_objs = [obj for obj in detected_objects if obj["class_id"] == 9]
            if traffic_light_objs:
                zones["traffic_control_pattern"] = {
                    "region": "intersection",
                    "objects": ["traffic light"] * len(traffic_light_objs),
                    "description": f"Intersection traffic control with {len(traffic_light_objs)} signals visible from above"
                }

            # 人行道從空中視角的情境推斷
            zones["crossing_pattern"] = {
                "region": "central",
                "objects": ["inferred crosswalk"],
                "description": "Crossing pattern visible from aerial perspective"
            }

            return zones

        except Exception as e:
            logger.error(f"Error identifying aerial intersection features: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_aerial_plaza_features(self, people_objs: List[Dict]) -> Dict:
        """
        識別空中視角廣場特徵

        Args:
            people_objs: 行人物件列表

        Returns:
            廣場特徵區域字典
        """
        try:
            zones = {}

            if people_objs:
                # 檢查人群是否聚集在中央區域
                central_people = [obj for obj in people_objs
                                if "middle" in obj["region"]]

                if central_people:
                    zones["central_gathering"] = {
                        "region": "middle_center",
                        "objects": ["person"] * len(central_people),
                        "description": f"Central plaza gathering area with {len(central_people)} people viewed from above"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error identifying aerial plaza features: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_asian_pedestrian_pathway(self, detected_objects: List[Dict]) -> Dict:
        """
        亞洲文化場景中的行人通道

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            行人通道區域字典
        """
        try:
            zones = {}

            pathway_items = []
            pathway_regions = {}

            # 提取人群用於通道分析
            people_objs = [obj for obj in detected_objects if obj["class_id"] == 0]

            # 分析人群是否形成線形（商業街的特徵）
            people_positions = [obj["normalized_center"] for obj in people_objs]

            structured_path = False
            path_direction = "meandering"

            if len(people_positions) >= 3:
                # 檢查人群是否沿相似y坐標排列（水平路徑）
                y_coords = [pos[1] for pos in people_positions]
                y_mean = sum(y_coords) / len(y_coords)
                y_variance = sum((y - y_mean)**2 for y in y_coords) / len(y_coords)

                horizontal_path = y_variance < 0.05  # 低變異表示水平對齊

                # 檢查人群是否沿相似x坐標排列（垂直路徑）
                x_coords = [pos[0] for pos in people_positions]
                x_mean = sum(x_coords) / len(x_coords)
                x_variance = sum((x - x_mean)**2 for x in x_coords) / len(x_coords)

                vertical_path = x_variance < 0.05  # 低變異表示垂直對齊

                structured_path = horizontal_path or vertical_path
                path_direction = "horizontal" if horizontal_path else "vertical" if vertical_path else "meandering"

            # 收集通道物件（人、自行車、摩托車在中間區域）
            for obj in detected_objects:
                if obj["class_id"] in [0, 1, 3]:  # Person, bicycle, motorcycle
                    y_pos = obj["normalized_center"][1]
                    # 按垂直位置分組（圖像中間可能是通道）
                    if 0.25 <= y_pos <= 0.75:
                        region = obj["region"]
                        if region not in pathway_regions:
                            pathway_regions[region] = []
                        pathway_regions[region].append(obj)
                        pathway_items.append(obj["class_name"])

            if pathway_items:
                path_desc = "Pedestrian walkway with people moving through the commercial area"
                if structured_path:
                    path_desc = f"{path_direction.capitalize()} pedestrian walkway with organized foot traffic"

                zones["pedestrian_pathway"] = {
                    "region": "middle_center",  # 假設：通道通常在中間
                    "objects": list(set(pathway_items)),
                    "description": path_desc
                }

            return zones

        except Exception as e:
            logger.error(f"Error identifying Asian pedestrian pathway: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_vendor_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別攤販區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            攤販區域字典
        """
        try:
            zones = {}

            # 識別攤販區域（小攤/商店 - 從情境推斷）
            has_small_objects = any(obj["class_id"] in [24, 26, 39, 41] for obj in detected_objects)  # bags, bottles, cups
            has_people = any(obj["class_id"] == 0 for obj in detected_objects)

            if has_small_objects and has_people:
                # 可能的攤販區域是人群和小物件聚集的地方
                small_obj_regions = {}

                for obj in detected_objects:
                    if obj["class_id"] in [24, 26, 39, 41, 67]:  # bags, bottles, cups, phones
                        region = obj["region"]
                        if region not in small_obj_regions:
                            small_obj_regions[region] = []
                        small_obj_regions[region].append(obj)

                if small_obj_regions:
                    main_vendor_region = max(small_obj_regions.items(),
                                        key=lambda x: len(x[1]),
                                        default=(None, []))

                    if main_vendor_region[0] is not None:
                        vendor_items = [obj["class_name"] for obj in main_vendor_region[1]]
                        zones["vendor_zone"] = {
                            "region": main_vendor_region[0],
                            "objects": list(set(vendor_items)),
                            "description": "Vendor or market stall area with small merchandise"
                        }

            return zones

        except Exception as e:
            logger.error(f"Error identifying vendor zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_upscale_decorative_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別高級餐飲的裝飾區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            裝飾區域字典
        """
        try:
            zones = {}

            decor_items = []
            decor_regions = {}

            # 尋找裝飾元素（花瓶、酒杯、未使用的餐具）
            for obj in detected_objects:
                if obj["class_id"] in [75, 40]:  # Vase, wine glass
                    region = obj["region"]
                    if region not in decor_regions:
                        decor_regions[region] = []
                    decor_regions[region].append(obj)
                    decor_items.append(obj["class_name"])

            if decor_items:
                main_decor_region = max(decor_regions.items(),
                                    key=lambda x: len(x[1]),
                                    default=(None, []))

                if main_decor_region[0] is not None:
                    zones["decorative_zone"] = {
                        "region": main_decor_region[0],
                        "objects": list(set(decor_items)),
                        "description": f"Decorative area with {', '.join(list(set(decor_items)))}"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error identifying upscale decorative zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_dining_seating_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別餐廳座位安排區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            座位區域字典
        """
        try:
            zones = {}

            # 識別座位安排區域
            chairs = [obj for obj in detected_objects if obj["class_id"] == 56]  # chairs
            if len(chairs) >= 2:
                chair_regions = {}
                for obj in chairs:
                    region = obj["region"]
                    if region not in chair_regions:
                        chair_regions[region] = []
                    chair_regions[region].append(obj)

                if chair_regions:
                    main_seating_region = max(chair_regions.items(),
                                        key=lambda x: len(x[1]),
                                        default=(None, []))

                    if main_seating_region[0] is not None:
                        zones["dining_seating_zone"] = {
                            "region": main_seating_region[0],
                            "objects": ["chair"] * len(main_seating_region[1]),
                            "description": f"Formal dining seating arrangement with {len(main_seating_region[1])} chairs"
                        }

            return zones

        except Exception as e:
            logger.error(f"Error identifying dining seating zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_serving_zones(self, detected_objects: List[Dict], existing_zones: Dict) -> Dict:
        """
        識別服務區域

        Args:
            detected_objects: 檢測到的物件列表
            existing_zones: 已存在的功能區域

        Returns:
            服務區域字典
        """
        try:
            zones = {}

            serving_items = []
            serving_regions = {}

            # 服務區域可能有瓶子、碗、容器
            for obj in detected_objects:
                if obj["class_id"] in [39, 45]:  # Bottle, bowl
                    # 檢查是否在與主餐桌不同的區域
                    if "formal_dining_zone" in existing_zones and obj["region"] != existing_zones["formal_dining_zone"]["region"]:
                        region = obj["region"]
                        if region not in serving_regions:
                            serving_regions[region] = []
                        serving_regions[region].append(obj)
                        serving_items.append(obj["class_name"])

            if serving_items:
                main_serving_region = max(serving_regions.items(),
                                    key=lambda x: len(x[1]),
                                    default=(None, []))

                if main_serving_region[0] is not None:
                    zones["serving_zone"] = {
                        "region": main_serving_region[0],
                        "objects": list(set(serving_items)),
                        "description": f"Serving or sideboard area with {', '.join(list(set(serving_items)))}"
                    }

            return zones

        except Exception as e:
            logger.error(f"Error identifying serving zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_building_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別建築區域（從場景情境推斷）

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            建築區域字典
        """
        try:
            zones = {}

            # 側邊建築區域（從場景情境推斷）
            # 檢查是否有實際可能包含建築物的區域
            left_side_regions = ["top_left", "middle_left", "bottom_left"]
            right_side_regions = ["top_right", "middle_right", "bottom_right"]

            # 檢查左側
            left_building_evidence = True
            for region in left_side_regions:
                # 如果此區域有很多車輛或人群，不太可能是建築物
                vehicle_in_region = any(obj["region"] == region and obj["class_id"] in [1, 2, 3, 5, 7]
                                    for obj in detected_objects)
                people_in_region = any(obj["region"] == region and obj["class_id"] == 0
                                    for obj in detected_objects)

                if vehicle_in_region or people_in_region:
                    left_building_evidence = False
                    break

            # 檢查右側
            right_building_evidence = True
            for region in right_side_regions:
                # 如果此區域有很多車輛或人群，不太可能是建築物
                vehicle_in_region = any(obj["region"] == region and obj["class_id"] in [1, 2, 3, 5, 7]
                                    for obj in detected_objects)
                people_in_region = any(obj["region"] == region and obj["class_id"] == 0
                                    for obj in detected_objects)

                if vehicle_in_region or people_in_region:
                    right_building_evidence = False
                    break

            # 如果證據支持，添加建築區域
            if left_building_evidence:
                zones["building_zone_left"] = {
                    "region": "middle_left",
                    "objects": ["building"],  # 推斷
                    "description": "Tall buildings line the left side of the street"
                }

            if right_building_evidence:
                zones["building_zone_right"] = {
                    "region": "middle_right",
                    "objects": ["building"],  # 推斷
                    "description": "Tall buildings line the right side of the street"
                }

            return zones

        except Exception as e:
            logger.error(f"Error identifying building zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _identify_financial_pedestrian_zones(self, detected_objects: List[Dict]) -> Dict:
        """
        識別金融區的行人區域

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            行人區域字典
        """
        try:
            zones = {}

            # 識別行人區域（如果有人群）
            people_objs = [obj for obj in detected_objects if obj["class_id"] == 0]
            if people_objs:
                people_regions = {}
                for obj in people_objs:
                    region = obj["region"]
                    if region not in people_regions:
                        people_regions[region] = []
                    people_regions[region].append(obj)

                if people_regions:
                    main_pedestrian_region = max(people_regions.items(),
                                            key=lambda x: len(x[1]),
                                            default=(None, []))

                    if main_pedestrian_region[0] is not None:
                        zones["pedestrian_zone"] = {
                            "region": main_pedestrian_region[0],
                            "objects": ["person"] * len(main_pedestrian_region[1]),
                            "description": f"Pedestrian area with {len(main_pedestrian_region[1])} people navigating the financial district"
                        }

            return zones

        except Exception as e:
            logger.error(f"Error identifying financial pedestrian zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _create_landmark_auxiliary_zones(self, landmark: Dict, index: int) -> Dict:
        """
        創建地標相關的輔助區域（攝影區、紀念品區等）

        Args:
            landmark: 地標物件字典
            index: 地標索引

        Returns:
            輔助區域字典
        """
        try:
            auxiliary_zones = {}
            landmark_region = landmark.get("region", "middle_center")
            landmark_name = landmark.get("class_name", "Landmark")

            # 創建攝影區
            # 根據地標位置調整攝影區位置（地標前方通常是攝影區）
            region_mapping = {
                "top_left": "bottom_right",
                "top_center": "bottom_center",
                "top_right": "bottom_left",
                "middle_left": "middle_right",
                "middle_center": "bottom_center",
                "middle_right": "middle_left",
                "bottom_left": "top_right",
                "bottom_center": "top_center",
                "bottom_right": "top_left"
            }

            photo_region = region_mapping.get(landmark_region, landmark_region)

            photo_key = f"{landmark_name.lower().replace(' ', '_')}_photography_spot"
            auxiliary_zones[photo_key] = {
                "name": f"{landmark_name} Photography Spot",
                "description": f"Popular position for photographing {landmark_name} with optimal viewing angle.",
                "objects": ["camera", "person", "cell phone"],
                "region": photo_region,
                "primary_function": "Tourist photography"
            }

            # 如果是著名地標，可能有紀念品販售區
            if landmark.get("confidence", 0) > 0.7:  # 高置信度地標更可能有紀念品區
                # 根據地標位置找到適合的紀念品區位置（通常在地標附近但不直接在地標上）
                adjacent_regions = {
                    "top_left": ["top_center", "middle_left"],
                    "top_center": ["top_left", "top_right"],
                    "top_right": ["top_center", "middle_right"],
                    "middle_left": ["top_left", "bottom_left"],
                    "middle_center": ["middle_left", "middle_right"],
                    "middle_right": ["top_right", "bottom_right"],
                    "bottom_left": ["middle_left", "bottom_center"],
                    "bottom_center": ["bottom_left", "bottom_right"],
                    "bottom_right": ["bottom_center", "middle_right"]
                }

                if landmark_region in adjacent_regions:
                    souvenir_region = adjacent_regions[landmark_region][0]  # 選擇第一個相鄰區域

                    souvenir_key = f"{landmark_name.lower().replace(' ', '_')}_souvenir_area"
                    auxiliary_zones[souvenir_key] = {
                        "name": f"{landmark_name} Souvenir Area",
                        "description": f"Area where visitors can purchase souvenirs and memorabilia related to {landmark_name}.",
                        "objects": ["person", "handbag", "backpack"],
                        "region": souvenir_region,
                        "primary_function": "Tourism commerce"
                    }

            return auxiliary_zones

        except Exception as e:
            logger.error(f"Error creating landmark auxiliary zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

Overwriting scene_zone_identifier.py


In [ ]:
# %%writefile functional_zone_identifier.py

import logging
import traceback
from typing import Dict, List, Any, Optional

logger = logging.getLogger(__name__)

class FunctionalZoneIdentifier:
    """
    作為功能區域辨識的主要窗口
    整合區域評估和場景特定的區域辨識邏輯，提供統一的功能區域辨識接口
    """

    def __init__(self, zone_evaluator=None, scene_zone_identifier=None, scene_viewpoint_analyzer=None):
        """
        初始化功能區域識別器

        Args:
            zone_evaluator: 區域評估器實例
            scene_zone_identifier: 場景區域辨識器實例
            scene_viewpoint_analyzer: 場景視角分析器
        """
        try:
            self.zone_evaluator = zone_evaluator
            self.scene_zone_identifier = scene_zone_identifier

            self.scene_viewpoint_analyzer = scene_viewpoint_analyzer
            self.viewpoint_detector = scene_viewpoint_analyzer

            logger.info("FunctionalZoneIdentifier initialized successfully with SceneViewpointAnalyzer")

        except Exception as e:
            logger.error(f"Failed to initialize FunctionalZoneIdentifier: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def identify_functional_zones(self, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        識別場景內的功能區域，具有針對不同視角和文化背景的改進檢測能力。
        如果偵測到 is_landmark=True 的物件，則優先直接呼叫 identify_landmark_zones 並回傳結果。
        """

        try:
            # 1. 如果沒有啟用地標功能，就先把所有有 is_landmark=True 的物件過濾掉
            if not getattr(self, 'enable_landmark', True):
                detected_objects = [obj for obj in detected_objects if not obj.get("is_landmark", False)]

            # 2. 只要檢測到任何 is_landmark=True 的物件，立即優先使用 identify_landmark_zones
            landmark_objects = [obj for obj in detected_objects if obj.get("is_landmark", False)]
            if landmark_objects and self.scene_zone_identifier:
                lm_zones = self.scene_zone_identifier.identify_landmark_zones(landmark_objects)
                return self._standardize_zone_keys_and_descriptions(lm_zones)

            # 3. city_street
            if scene_type in ["tourist_landmark", "natural_landmark", "historical_monument"]:
                scene_type = "city_street"

            # 4.  判斷與物件數量檢查
            if self.zone_evaluator:
                should_identify = self.zone_evaluator.evaluate_zone_identification_feasibility(
                    detected_objects, scene_type
                )
                if not should_identify:
                    logger.info(f"Zone identification not feasible for scene type '{scene_type}'")
                    return {}
            else:
                if len(detected_objects) < 2:
                    logger.info("Insufficient objects for zone identification")
                    return {}

            # 5. 建立 category_regions
            category_regions = self._build_category_regions_mapping(detected_objects)
            zones = {}

            # 6. 檢測場景視角
            viewpoint_info = {"viewpoint": "eye_level"}
            if self.scene_viewpoint_analyzer:
                viewpoint_info = self.scene_viewpoint_analyzer.detect_scene_viewpoint(detected_objects)

            # 7. 根據不同 scene_type 使用各種自己的區域辨識
            if scene_type in ["living_room", "bedroom", "dining_area", "kitchen", "office_workspace", "meeting_room"]:
                if self.scene_zone_identifier:
                    raw_zones = self.scene_zone_identifier.identify_indoor_zones(
                        category_regions, detected_objects, scene_type
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(raw_zones))

            elif scene_type in ["city_street", "parking_lot", "park_area"]:
                if self.scene_zone_identifier:
                    raw_zones = self.scene_zone_identifier.identify_outdoor_general_zones(
                        category_regions, detected_objects, scene_type
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(raw_zones))

            elif "aerial" in scene_type or viewpoint_info.get("viewpoint") == "aerial":
                if self.scene_zone_identifier:
                    raw_zones = self.scene_zone_identifier.identify_aerial_view_zones(
                        category_regions, detected_objects, scene_type
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(raw_zones))

            elif "asian" in scene_type:
                if self.scene_zone_identifier:
                    asian_zones = self.scene_zone_identifier.identify_asian_cultural_zones(
                        category_regions, detected_objects, scene_type
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(asian_zones))

            elif scene_type == "urban_intersection":
                if self.scene_zone_identifier:
                    raw_zones = self.scene_zone_identifier.identify_intersection_zones(
                        category_regions, detected_objects, viewpoint_info.get("viewpoint")
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(raw_zones))
                    used_tl_count_per_region = {}
                    for zone_info in raw_zones.values():
                        obj_list = zone_info.get("objects", [])
                        if "traffic light" in obj_list:
                            rg = zone_info.get("region", "")
                            count_in_zone = obj_list.count("traffic light")
                            used_tl_count_per_region[rg] = used_tl_count_per_region.get(rg, 0) + count_in_zone

                    signal_regions = {}
                    for t in [obj for obj in detected_objects if obj.get("class_id") == 9]:
                        region = t.get("region", "")
                        signal_regions.setdefault(region, []).append(t)

                    for idx, (region, signals) in enumerate(signal_regions.items()):
                        total_in_region = len(signals)
                        used_in_region = used_tl_count_per_region.get(region, 0)
                        remaining_in_region = total_in_region - used_in_region

                        if remaining_in_region > 0:
                            direction = self._get_directional_description(region)
                            if direction and direction != "central":
                                zone_key = f"{direction} traffic control area"
                            else:
                                zone_key = "primary traffic control area" if idx == 0 else "auxiliary traffic control area"

                            if zone_key in zones:
                                suffix = 1
                                new_key = f"{zone_key} ({suffix})"
                                while new_key in zones:
                                    suffix += 1
                                    new_key = f"{zone_key} ({suffix})"
                                zone_key = new_key

                            zones[zone_key] = {
                                "region": region,
                                "objects": ["traffic light"] * remaining_in_region,
                                "description": f"Traffic control area with {remaining_in_region} traffic lights in {region}"
                            }

                    for region, signals in signal_regions.items():
                        used = used_tl_count_per_region.get(region, 0)
                        total = len(signals)
                        remaining = total - used
                        # print(f"[DEBUG] Region '{region}': Total TL = {total}, Used in crossing = {used}, Remaining = {remaining}")

            elif scene_type == "financial_district":
                if self.scene_zone_identifier:
                    fd_zones = self.scene_zone_identifier.identify_financial_district_zones(
                        category_regions, detected_objects
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(fd_zones))

            elif scene_type == "upscale_dining":
                if self.scene_zone_identifier:
                    ud_zones = self.scene_zone_identifier.identify_upscale_dining_zones(
                        category_regions, detected_objects
                    )
                    zones.update(self._standardize_zone_keys_and_descriptions(ud_zones))

            else:
                # 如果不是上述任何一種場景，就用「預設功能區」
                default_zones = self._identify_default_zones(category_regions, detected_objects)
                zones.update(self._standardize_zone_keys_and_descriptions(default_zones))

            # 8. 如果此時 zones 仍為空，就會變成 default → basic → fallback
            if not zones:
                default_zones = self._identify_default_zones(category_regions, detected_objects)
                if default_zones:
                    zones.update(self._standardize_zone_keys_and_descriptions(default_zones))
                else:
                    basic_zones = self._create_basic_zones_from_objects(detected_objects, scene_type)
                    zones.update(self._standardize_zone_keys_and_descriptions(basic_zones))

            # 通用 fallback：把所有還沒被列出的 (class_name, region) 通通補進去
            fallback_zones = self._generate_category_fallback_zones(detected_objects, zones)
            zones.update(fallback_zones)

            # Debug: 列印出各功能區的 traffic light 統計
            total_tl_in_zones = 0
            for zone_key, zone_info in zones.items():
                if isinstance(zone_info, dict):
                    sub_objs = zone_info.get("objects", [])
                else:
                    sub_objs = []
                t_in_zone = [obj for obj in sub_objs if obj == "traffic light"]
                # print(f"[DEBUG] identify_functional_zones - Zone '{zone_key}' has {len(t_in_zone)} traffic light(s).")
                total_tl_in_zones += len(t_in_zone)
            # print(f"[DEBUG] identify_functional_zones - Total traffic lights in zones: {total_tl_in_zones}")

            logger.info(f"Identified {len(zones)} functional zones for scene type '{scene_type}'")
            return zones

        except Exception as e:
            logger.error(f"Error identifying functional zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _standardize_zone_keys_and_descriptions(self, raw_zones: Dict) -> Dict:
        """
        標準化區域鍵名和描述，將內部標識符轉換為描述性名稱

        Args:
            raw_zones: 原始區域識別結果

        Returns:
            Dict: 標準化後的區域字典
        """
        try:
            standardized_zones = {}

            for zone_key, zone_data in raw_zones.items():
                # 生成描述性的區域鍵名
                descriptive_key = self._generate_descriptive_zone_key(zone_key, zone_data)

                # 確保區域描述也經過標準化
                if isinstance(zone_data, dict) and "description" in zone_data:
                    zone_data["description"] = self._enhance_zone_description(zone_data["description"], zone_data)

                standardized_zones[descriptive_key] = zone_data

            return standardized_zones

        except Exception as e:
            logger.error(f"Error standardizing zone keys and descriptions: {str(e)}")
            return raw_zones

    def _generate_descriptive_zone_key(self, original_key: str, zone_data: Dict) -> str:
        """
        基於區域內容生成描述性的鍵名
        核心修改：只要該區域內有任一個 'traffic light'，就優先回傳 'traffic control zone'，
        """
        try:
            objects = zone_data.get("objects", [])
            region = zone_data.get("region", "")

            # 優先檢查是否含有 traffic light
            if any(obj == "traffic light" or "traffic light" in obj for obj in objects):
                return "traffic control zone"

            # 如果沒有 traffic light，才繼續分析「主要物件」順序
            primary_objects = self._analyze_primary_objects(objects)

            # 依序檢查人、車、家具、紅綠燈等
            if "person" in primary_objects:
                if len([o for o in objects if o == "person"]) > 1:
                    return "pedestrian activity area"
                else:
                    return "individual activity zone"
            elif any(vehicle in primary_objects for vehicle in ["car", "truck", "bus", "motorcycle"]):
                return "vehicle movement area"
            elif any(furniture in primary_objects for furniture in ["chair", "table", "sofa", "bed"]):
                return "furniture arrangement area"

            # 若上述都不符合，改用「基於位置」做 fallback
            position_descriptions = {
                "top_left": "upper left area",
                "top_center": "upper central area",
                "top_right": "upper right area",
                "middle_left": "left side area",
                "middle_center": "main crossing area",
                "middle_right": "right side area",
                "bottom_left": "lower left area",
                "bottom_center": "lower central area",
                "bottom_right": "lower right area"
            }
            if region in position_descriptions:
                return position_descriptions[region]

            # 再次檢查主要物件，給出另一種 fallback 命名
            if primary_objects:
                if "traffic light" in primary_objects:
                    return "traffic control zone"
                elif any(vehicle in primary_objects for vehicle in ["car", "truck", "bus"]):
                    return "vehicle movement area"
                elif "person" in primary_objects:
                    return "pedestrian activity area"

            # 最後最後的備用名稱
            return "activity area"

        except Exception as e:
            logger.warning(f"Error generating descriptive key for '{original_key}': {str(e)}")
            return "activity area"

    def _analyze_primary_objects(self, objects: List[str]) -> List[str]:
        """
        分析區域中的主要物件類型

        Args:
            objects: 物件名稱列表

        Returns:
            List[str]: 主要物件類型列表
        """
        try:
            # 計算物件出現頻率
            object_counts = {}
            for obj in objects:
                normalized_obj = obj.replace('_', ' ').lower().strip()
                object_counts[normalized_obj] = object_counts.get(normalized_obj, 0) + 1

            # 按出現頻率排序，返回前三個主要物件
            sorted_objects = sorted(object_counts.items(), key=lambda x: x[1], reverse=True)
            return [obj[0] for obj in sorted_objects[:3]]

        except Exception as e:
            logger.warning(f"Error analyzing primary objects: {str(e)}")
            return []

    def _enhance_zone_description(self, original_description: str, zone_data: Dict) -> str:
        """
        增強區域描述的自然性和完整性
        """
        try:
            if not original_description or not original_description.strip():
                return self._generate_fallback_description(zone_data)

            import re
            enhanced = original_description.strip()

            # 改善技術性表達為自然語言
            enhanced = re.sub(r'\bin central direction\b', 'in the center', enhanced)
            enhanced = re.sub(r'\bin west area\b', 'on the left side', enhanced)
            enhanced = re.sub(r'\bin east direction\b', 'on the right side', enhanced)
            enhanced = re.sub(r'\bnear traffic signals\b', 'near the traffic lights', enhanced)
            enhanced = re.sub(r'\bwith (\d+) (\w+)\b', r'where \1 \2 can be seen', enhanced)

            # 移除重複和冗餘表達
            enhanced = re.sub(r'\barea with.*?in.*?area\b', lambda m: m.group(0).split(' in ')[0], enhanced)
            enhanced = enhanced.replace('traffic area', 'area').replace('crossing area', 'crossing')

            # 標準化描述結構
            if enhanced.startswith('Pedestrian'):
                enhanced = re.sub(r'^Pedestrian crossing area', 'The main pedestrian crossing', enhanced)
            elif enhanced.startswith('Vehicle'):
                enhanced = re.sub(r'^Vehicle traffic area', 'The vehicle movement area', enhanced)
            elif enhanced.startswith('Traffic control'):
                enhanced = re.sub(r'^Traffic control area', 'Traffic management elements', enhanced)

            # 移除內部標識符格式
            enhanced = re.sub(r'\b\w+_\w+(?:_\w+)*\b', lambda m: m.group(0).replace('_', ' '), enhanced)

            # 確保描述的完整性
            if not enhanced.endswith('.'):
                enhanced += '.'

            # 改善描述的自然性
            enhanced = enhanced.replace('with with', 'with')
            enhanced = re.sub(r'\s{2,}', ' ', enhanced)

            return enhanced

        except Exception as e:
            logger.warning(f"Error enhancing zone description: {str(e)}")
            return original_description if original_description else "A functional area within the scene."

    def _generate_fallback_description(self, zone_data: Dict) -> str:
        """
        為缺少描述的區域生成備用描述

        Args:
            zone_data: 區域數據

        Returns:
            str: 備用描述
        """
        try:
            objects = zone_data.get("objects", [])
            region = zone_data.get("region", "")

            if objects:
                object_count = len(objects)
                unique_objects = list(set(objects))

                if object_count == 1:
                    return f"Area containing {unique_objects[0].replace('_', ' ')}."
                elif len(unique_objects) <= 3:
                    obj_list = ", ".join([obj.replace('_', ' ') for obj in unique_objects])
                    return f"Area featuring {obj_list}."
                else:
                    return f"Multi-functional area with {object_count} elements including various objects."

            return "Functional area within the scene."

        except Exception as e:
            logger.warning(f"Error generating fallback description: {str(e)}")
            return "Activity area."

    def _build_category_regions_mapping(self, detected_objects: List[Dict]) -> Dict:
        """
        建立物件按類別和區域的分組映射

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            按類別和區域分組的物件字典
        """
        try:
            category_regions = {}

            for obj in detected_objects:
                category = self._categorize_object(obj)
                if not category:
                    continue

                if category not in category_regions:
                    category_regions[category] = {}

                region = obj.get("region", "center")
                if region not in category_regions[category]:
                    category_regions[category][region] = []

                category_regions[category][region].append(obj)

            logger.debug(f"Built category regions mapping with {len(category_regions)} categories")
            return category_regions

        except Exception as e:
            logger.error(f"Error building category regions mapping: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _categorize_object(self, obj: Dict) -> str:
        """
        將檢測到的物件分類到功能類別中，用於區域識別

        Args:
            obj: 物件字典

        Returns:
            物件功能類別字串
        """
        try:
            class_id = obj.get("class_id", -1)
            class_name = obj.get("class_name", "").lower()

            # 使用現有的類別映射（如果可用）
            if hasattr(self, 'OBJECT_CATEGORIES') and self.OBJECT_CATEGORIES:
                for category, ids in self.OBJECT_CATEGORIES.items():
                    if class_id in ids:
                        return category

            # 基於COCO類別名稱的後備分類
            furniture_items = ["chair", "couch", "bed", "dining table", "toilet"]
            plant_items = ["potted plant"]
            electronic_items = ["tv", "laptop", "mouse", "remote", "keyboard", "cell phone"]
            vehicle_items = ["bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat"]
            person_items = ["person"]
            kitchen_items = ["bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
                            "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
                            "pizza", "donut", "cake", "refrigerator", "oven", "toaster", "sink", "microwave"]
            sports_items = ["frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                        "baseball glove", "skateboard", "surfboard", "tennis racket"]
            personal_items = ["handbag", "tie", "suitcase", "umbrella", "backpack"]

            if any(item in class_name for item in furniture_items):
                return "furniture"
            elif any(item in class_name for item in plant_items):
                return "plant"
            elif any(item in class_name for item in electronic_items):
                return "electronics"
            elif any(item in class_name for item in vehicle_items):
                return "vehicle"
            elif any(item in class_name for item in person_items):
                return "person"
            elif any(item in class_name for item in kitchen_items):
                return "kitchen_items"
            elif any(item in class_name for item in sports_items):
                return "sports"
            elif any(item in class_name for item in personal_items):
                return "personal_items"
            else:
                return "misc"

        except Exception as e:
            logger.error(f"Error categorizing object: {str(e)}")
            logger.error(traceback.format_exc())
            return "misc"

    def _identify_default_zones(self, category_regions: Dict, detected_objects: List[Dict]) -> Dict:
        """
        當沒有匹配到特定場景類型時的一般功能區域識別

        Args:
            category_regions: 按類別和區域分組的物件字典
            detected_objects: 檢測到的物件列表

        Returns:
            預設功能區域字典
        """
        try:
            zones = {}

            # 按類別分組物件並找到主要集中區域
            for category, regions in category_regions.items():
                if not regions:
                    continue

                # 找到此類別中物件最多的區域
                main_region = max(regions.items(),
                            key=lambda x: len(x[1]),
                            default=(None, []))

                if main_region[0] is None or len(main_region[1]) < 2:
                    continue

                # 創建基於物件類別的區域
                zone_objects = [obj["class_name"] for obj in main_region[1]]

                # 如果物件太少，跳過
                if len(zone_objects) < 2:
                    continue

                # 根據類別創建區域名稱和描述
                if category == "furniture":
                    zones["furniture arrangement area"] = {
                        "region": main_region[0],
                        "objects": zone_objects,
                        "description": f"Furniture arrangement area featuring {self._format_object_list_naturally(zone_objects[:3])}"
                    }
                elif category == "electronics":
                    zones["electronics area"] = {
                        "region": main_region[0],
                        "objects": zone_objects,
                        "description": f"Electronics area containing {self._format_object_list_naturally(zone_objects[:3])}"
                    }
                elif category == "kitchen_items":
                    zones["dining_zone"] = {
                        "region": main_region[0],
                        "objects": zone_objects,
                        "description": f"Dining or food area with {', '.join(zone_objects[:3])}"
                    }
                elif category == "vehicle":
                    zones["vehicle_zone"] = {
                        "region": main_region[0],
                        "objects": zone_objects,
                        "description": f"Area with vehicles including {', '.join(zone_objects[:3])}"
                    }
                elif category == "personal_items":
                    zones["personal_items_zone"] = {
                        "region": main_region[0],
                        "objects": zone_objects,
                        "description": f"Area with personal items including {', '.join(zone_objects[:3])}"
                    }

            # 檢查人群聚集
            people_objs = [obj for obj in detected_objects if obj["class_id"] == 0]
            if len(people_objs) >= 2:
                people_regions = {}
                for obj in people_objs:
                    region = obj["region"]
                    if region not in people_regions:
                        people_regions[region] = []
                    people_regions[region].append(obj)

                if people_regions:
                    main_people_region = max(people_regions.items(),
                                        key=lambda x: len(x[1]),
                                        default=(None, []))

                    if main_people_region[0] is not None:
                        zones["people_zone"] = {
                            "region": main_people_region[0],
                            "objects": ["person"] * len(main_people_region[1]),
                            "description": f"Area with {len(main_people_region[1])} people"
                        }

            logger.debug(f"Identified {len(zones)} default zones")
            return zones

        except Exception as e:
            logger.error(f"Error identifying default zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _format_object_list_naturally(self, object_list: List[str]) -> str:
        """
        將物件列表格式化為自然語言表達

        Args:
            object_list: 物件名稱列表

        Returns:
            str: 自然語言格式的物件列表
        """
        try:
            if not object_list:
                return "various items"

            # 標準化物件名稱
            normalized_objects = []
            for obj in object_list:
                normalized = obj.replace('_', ' ').strip()
                if normalized:
                    normalized_objects.append(normalized)

            if not normalized_objects:
                return "various items"

            # 格式化列表
            if len(normalized_objects) == 1:
                return normalized_objects[0]
            elif len(normalized_objects) == 2:
                return f"{normalized_objects[0]} and {normalized_objects[1]}"
            else:
                return ", ".join(normalized_objects[:-1]) + f", and {normalized_objects[-1]}"

        except Exception as e:
            logger.warning(f"Error formatting object list naturally: {str(e)}")
            return "various items"

    def _create_basic_zones_from_objects(self, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        從個別高置信度物件創建基本功能區域
        這是標準區域識別失敗時的後備方案

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            基本區域字典
        """
        try:
            zones = {}

            # 專注於高置信度物件
            high_conf_objects = [obj for obj in detected_objects if obj.get("confidence", 0) >= 0.6]

            if not high_conf_objects:
                high_conf_objects = detected_objects  # 後備到所有物件

            # 基於個別重要物件創建區域
            processed_objects = set()  # 避免重複處理相同類型的物件

            for obj in high_conf_objects[:3]:  # 限制為前3個物件
                class_name = obj["class_name"]
                region = obj.get("region", "center")

                # 避免為同一類型物件創建多個區域
                if class_name in processed_objects:
                    continue
                processed_objects.add(class_name)

                # 基於物件類型創建描述性區域
                zone_description = self._get_basic_zone_description(class_name, scene_type)
                descriptive_key = self._generate_object_based_zone_key(class_name, region)

                if zone_description and descriptive_key:
                    zones[descriptive_key] = {
                        "region": region,
                        "objects": [class_name],
                        "description": zone_description
                    }

            logger.debug(f"Created {len(zones)} basic zones from high confidence objects")
            return zones

        except Exception as e:
            logger.error(f"Error creating basic zones from objects: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _generate_object_based_zone_key(self, class_name: str, region: str) -> str:
        """
        基於物件類型和位置生成描述性的區域鍵名

        Args:
            class_name: 物件類別名稱
            region: 區域位置

        Returns:
            str: 描述性區域鍵名
        """
        try:
            # 標準化物件名稱
            normalized_class = class_name.replace('_', ' ').lower().strip()

            # 物件類型對應的區域描述
            object_zone_mapping = {
                'person': 'activity area',
                'car': 'vehicle area',
                'truck': 'vehicle area',
                'bus': 'vehicle area',
                'motorcycle': 'vehicle area',
                'bicycle': 'cycling area',
                'traffic light': 'traffic control area',
                'chair': 'seating area',
                'sofa': 'seating area',
                'bed': 'rest area',
                'dining table': 'dining area',
                'tv': 'entertainment area',
                'laptop': 'workspace area',
                'potted plant': 'decorative area'
            }

            base_description = object_zone_mapping.get(normalized_class, f"{normalized_class} area")

            # 添加位置信息以提供更具體的描述
            position_modifiers = {
                'top_left': 'upper left',
                'top_center': 'upper central',
                'top_right': 'upper right',
                'middle_left': 'left side',
                'middle_center': 'central',
                'middle_right': 'right side',
                'bottom_left': 'lower left',
                'bottom_center': 'lower central',
                'bottom_right': 'lower right'
            }

            if region in position_modifiers:
                return f"{position_modifiers[region]} {base_description}"

            return base_description

        except Exception as e:
            logger.warning(f"Error generating object-based zone key for '{class_name}': {str(e)}")
            return "activity area"

    def _get_basic_zone_description(self, class_name: str, scene_type: str) -> str:
        """
        基於物件和場景類型生成基本區域描述

        Args:
            class_name: 物件類別名稱
            scene_type: 場景類型

        Returns:
            區域描述字串
        """
        try:
            # 物件特定描述
            descriptions = {
                "bed": "Sleeping and rest area",
                "sofa": "Seating and relaxation area",
                "chair": "Seating area",
                "dining table": "Dining and meal area",
                "tv": "Entertainment and media area",
                "laptop": "Work and computing area",
                "potted plant": "Decorative and green space area",
                "refrigerator": "Food storage and kitchen area",
                "car": "Vehicle and transportation area",
                "person": "Activity and social area"
            }

            return descriptions.get(class_name, f"Functional area with {class_name}")

        except Exception as e:
            logger.error(f"Error getting basic zone description for '{class_name}': {str(e)}")
            return f"Functional area with {class_name}"


    def _generate_category_fallback_zones(self, all_detected_objects: List[Dict], current_zones: Dict) -> Dict:
        """
        通用 fallback：針對 all_detected_objects 裡，每一個 (class_name, region) 組合是否已經
        在 current_zones 裡出現過。如果還沒，就為它們產生一個 fallback zone。
        """
        general_fallback = {
                0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus',
                6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant',
                11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat',
                16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear',
                22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag',
                27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard',
                32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove',
                36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle',
                40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
                46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
                51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair',
                57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet',
                62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard',
                67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink',
                72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors',
                77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'

        }

        # 1. 統計 current_zones 裡，已使用掉的 (class_name, region) 次數
        used_count = {}
        for zone_info in current_zones.values():
            rg = zone_info.get("region", "")
            for obj_name in zone_info.get("objects", []):
                key = (obj_name, rg)
                used_count[key] = used_count.get(key, 0) + 1

        # 2. 統計 all_detected_objects 裡的 (class_name, region) 總次數
        total_count = {}
        for obj in all_detected_objects:
            cname = obj.get("class_name", "")
            rg = obj.get("region", "")
            key = (cname, rg)
            total_count[key] = total_count.get(key, 0) + 1

        # 3. 把 default_classes 轉換成「class_name → fallback 區域 type」的對照表
        category_to_fallback = {
            # 行人與交通工具
            "person":        "pedestrian area",
            "bicycle":       "vehicle movement area",
            "car":           "vehicle movement area",
            "motorcycle":    "vehicle movement area",
            "airplane":      "vehicle movement area",
            "bus":           "vehicle movement area",
            "train":         "vehicle movement area",
            "truck":         "vehicle movement area",
            "boat":          "vehicle movement area",
            "traffic light": "traffic control area",
            "fire hydrant":  "traffic control area",
            "stop sign":     "traffic control area",
            "parking meter": "traffic control area",
            "bench":         "public furniture area",

            # 動物類、鳥類
            "bird":          "animal area",
            "cat":           "animal area",
            "dog":           "animal area",
            "horse":         "animal area",
            "sheep":         "animal area",
            "cow":           "animal area",
            "elephant":      "animal area",
            "bear":          "animal area",
            "zebra":         "animal area",
            "giraffe":       "animal area",

            # 托運與行李
            "backpack":      "personal items area",
            "umbrella":      "personal items area",
            "handbag":       "personal items area",
            "tie":           "personal items area",
            "suitcase":      "personal items area",

            # 運動器材
            "frisbee":       "sports area",
            "skis":          "sports area",
            "snowboard":     "sports area",
            "sports ball":   "sports area",
            "kite":          "sports area",
            "baseball bat":  "sports area",
            "baseball glove":"sports area",
            "skateboard":    "sports area",
            "surfboard":     "sports area",
            "tennis racket": "sports area",

            # 廚房與食品（Kitchen）
            "bottle":        "kitchen area",
            "wine glass":    "kitchen area",
            "cup":           "kitchen area",
            "fork":          "kitchen area",
            "knife":         "kitchen area",
            "spoon":         "kitchen area",
            "bowl":          "kitchen area",
            "banana":        "kitchen area",
            "apple":         "kitchen area",
            "sandwich":      "kitchen area",
            "orange":        "kitchen area",
            "broccoli":      "kitchen area",
            "carrot":        "kitchen area",
            "hot dog":       "kitchen area",
            "pizza":         "kitchen area",
            "donut":         "kitchen area",
            "cake":          "kitchen area",
            "dining table":  "furniture arrangement area",
            "refrigerator":  "kitchen area",
            "oven":          "kitchen area",
            "microwave":     "kitchen area",
            "toaster":       "kitchen area",
            "sink":          "kitchen area",
            "book":          "miscellaneous area",
            "clock":         "miscellaneous area",
            "vase":          "decorative area",
            "scissors":      "miscellaneous area",
            "teddy bear":    "miscellaneous area",
            "hair drier":    "miscellaneous area",
            "toothbrush":    "miscellaneous area",

            # 電子產品
            "tv":            "electronics area",
            "laptop":        "electronics area",
            "mouse":         "electronics area",
            "remote":        "electronics area",
            "keyboard":      "electronics area",
            "cell phone":    "electronics area",

            # 家具類
            "chair":         "furniture arrangement area",
            "couch":         "furniture arrangement area",
            "bed":           "furniture arrangement area",
            "toilet":        "furniture arrangement area",

            # 植物（室內植物或戶外綠化）
            "potted plant":  "decorative area",
        }

        # 4. 計算缺少的 (class_name, region) 並建立 fallback zone
        for (cname, rg), total in total_count.items():
            used = used_count.get((cname, rg), 0)
            missing = total - used
            if missing <= 0:
                continue

            # (A) 決定這個 cname 在 fallback 裡屬於哪個大 class（zone_type）
            zone_type = category_to_fallback.get(cname, "miscellaneous area")

            # (B) 根據 region 與 zone_type 組合成 fallback_key
            fallback_key = f"{rg} {zone_type}"

            # (C) 如果名稱重複，就在後面加 (1),(2),… 避免掉衝突
            if fallback_key in current_zones or fallback_key in general_fallback:
                suffix = 1
                new_key = f"{fallback_key} ({suffix})"
                while new_key in current_zones or new_key in general_fallback:
                    suffix += 1
                    new_key = f"{fallback_key} ({suffix})"
                fallback_key = new_key

            # (D) 建立這支 fallback zone，objects 裡放 missing 個 cname
            general_fallback[fallback_key] = {
                "region": rg,
                "objects": [cname] * missing,
                "description": f"{missing} {cname}(s) placed in fallback {zone_type} for region {rg}"
            }

        return general_fallback

Overwriting functional_zone_identifier.py


In [ ]:
# %%writefile spatial_analyzer.py

import os
import numpy as np
import logging
import traceback
from typing import Dict, List, Tuple, Any, Optional

# from region_analyzer import RegionAnalyzer
# from object_extractor import ObjectExtractor
# from scene_viewpoint_analyzer import SceneViewpointAnalyzer
# from zone_evaluator import ZoneEvaluator
# from scene_zone_identifier import SceneZoneIdentifier
# from functional_zone_identifier import FunctionalZoneIdentifier

logger = logging.getLogger(__name__)

class SpatialAnalyzer:
    """
    分析圖像中物件間空間關係的主要類別
    處理區域分配、物件定位和功能區域識別
    使用Facade模式整合多個子組件，保持外部接口的穩定性
    """

    def __init__(self, class_names: Dict[int, str] = None, object_categories=None):
        """
        初始化空間分析器，包含圖像區域定義

        Args:
            class_names: 類別ID到類別名稱的映射字典
            object_categories: 物件類別分組字典
        """
        try:
            # 初始化所有子組件
            self.region_analyzer = RegionAnalyzer()
            self.object_extractor = ObjectExtractor(class_names, object_categories)

            self.scene_viewpoint_analyzer = SceneViewpointAnalyzer()

            self.zone_evaluator = ZoneEvaluator()
            self.scene_zone_identifier = SceneZoneIdentifier()
            self.functional_zone_identifier = FunctionalZoneIdentifier(
                zone_evaluator=self.zone_evaluator,
                scene_zone_identifier=self.scene_zone_identifier,
                scene_viewpoint_analyzer=self.scene_viewpoint_analyzer
            )

            self.class_names = class_names
            self.OBJECT_CATEGORIES = object_categories or {}

            self.enhance_descriptor = None

            # 接近分析的距離閾值（標準化）
            self.proximity_threshold = 0.2

            logger.info("SpatialAnalyzer initialized successfully with all sub-components")

        except Exception as e:
            logger.error(f"Failed to initialize SpatialAnalyzer: {str(e)}")
            logger.error(traceback.format_exc())
            raise

    def update_class_names(self, class_names: Dict[int, str]):
        """
        更新類別名稱映射並傳遞給 ObjectExtractor

        Args:
            class_names: 新的類別名稱映射字典
        """
        try:
            self.class_names = class_names
            if hasattr(self, 'object_extractor') and self.object_extractor:
                self.object_extractor.update_class_names(class_names)
                logger.info(f"Updated class names in SpatialAnalyzer and ObjectExtractor")
        except Exception as e:
            logger.error(f"Failed to update class names in SpatialAnalyzer: {str(e)}")

    def _determine_region(self, x: float, y: float) -> str:
        """
        判斷點位於哪個區域

        Args:
            x: 標準化x座標 (0-1)
            y: 標準化y座標 (0-1)

        Returns:
            區域名稱
        """
        try:
            return self.region_analyzer.determine_region(x, y)
        except Exception as e:
            logger.error(f"Error in _determine_region: {str(e)}")
            logger.error(traceback.format_exc())
            return "unknown"

    def _analyze_regions(self, detected_objects: List[Dict]) -> Dict:
        """
        分析物件在各區域的分布情況

        Args:
            detected_objects: 包含位置資訊的檢測物件列表

        Returns:
            包含區域分析結果的字典
        """
        try:
            return self.region_analyzer.analyze_regions(detected_objects)
        except Exception as e:
            logger.error(f"Error in _analyze_regions: {str(e)}")
            logger.error(traceback.format_exc())
            return {
                "counts": {},
                "main_focus": [],
                "objects_by_region": {}
            }

    def _extract_detected_objects(self, detection_result: Any, confidence_threshold: float = 0.25) -> List[Dict]:
        """
        從檢測結果中提取物件資訊，包含位置資訊

        Args:
            detection_result: YOLOv8檢測結果
            confidence_threshold: 最小信心度閾值

        Returns:
            包含檢測物件資訊的字典列表
        """
        try:
            return self.object_extractor.extract_detected_objects(
                detection_result,
                confidence_threshold,
                region_analyzer=self.region_analyzer
            )
        except Exception as e:
            logger.error(f"Error in _extract_detected_objects: {str(e)}")
            logger.error(traceback.format_exc())
            return []

    def _detect_scene_viewpoint(self, detected_objects: List[Dict]) -> Dict:
        """
        檢測場景視角並識別特殊場景模式

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含視角和場景模式資訊的字典
        """
        try:
            # 委託給新的場景視角分析器
            return self.scene_viewpoint_analyzer.detect_scene_viewpoint(detected_objects)
        except Exception as e:
            logger.error(f"Error in _detect_scene_viewpoint: {str(e)}")
            logger.error(traceback.format_exc())
            return {"viewpoint": "eye_level", "patterns": []}

    def _identify_functional_zones(self, detected_objects: List[Dict], scene_type: str) -> Dict:
        """
        識別場景內的功能區域，具有針對不同視角和文化背景的改進檢測能力

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 識別出的場景類型

        Returns:
            包含功能區域及其描述的字典
        """
        try:
            return self.functional_zone_identifier.identify_functional_zones(detected_objects, scene_type)
        except Exception as e:
            logger.error(f"Error in _identify_functional_zones: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _categorize_object(self, obj: Dict) -> str:
        """
        將檢測到的物件分類到功能類別中，用於區域識別
        確保所有返回值都使用自然語言格式，避免底線或技術性標識符
        """
        try:
            class_id = obj.get("class_id", -1)
            class_name = obj.get("class_name", "").lower().strip()

            # 優先處理 traffic light
            # 只要 class_id == 9 或 class_name 包含 "traffic light"，就分類為 "traffic light"
            if class_id == 9 or "traffic light" in class_name:
                return "traffic light"

            # 如果有自訂的 OBJECT_CATEGORIES 映射，優先使用它
            if hasattr(self, 'OBJECT_CATEGORIES') and self.OBJECT_CATEGORIES:
                for category, ids in self.OBJECT_CATEGORIES.items():
                    if class_id in ids:
                        # 確保返回的類別名稱使用自然語言格式
                        return self._clean_category_name(category)

            # COCO class default name
            furniture_items = ["chair", "couch", "bed", "dining table", "toilet"]
            plant_items = ["potted plant"]
            electronic_items = ["tv", "laptop", "mouse", "remote", "keyboard", "cell phone"]
            vehicle_items = ["bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat"]
            person_items = ["person"]
            kitchen_items = [
                "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
                "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
                "pizza", "donut", "cake", "refrigerator", "oven", "toaster", "sink", "microwave"
            ]
            sports_items = [
                "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                "baseball glove", "skateboard", "surfboard", "tennis racket"
            ]
            personal_items = ["handbag", "tie", "suitcase", "umbrella", "backpack"]

            # fallback natural language
            if any(item in class_name for item in furniture_items):
                return "furniture"
            elif any(item in class_name for item in plant_items):
                return "plant"
            elif any(item in class_name for item in electronic_items):
                return "electronics"
            elif any(item in class_name for item in vehicle_items):
                return "vehicle"
            elif any(item in class_name for item in person_items):
                return "person"
            elif any(item in class_name for item in kitchen_items):
                return "kitchen items"  # 移除底線
            elif any(item in class_name for item in sports_items):
                return "sports"
            elif any(item in class_name for item in personal_items):
                return "personal items"  # 移除底線
            else:
                return "misc"

        except Exception as e:
            logger.error(f"Error categorizing object: {str(e)}")
            logger.error(traceback.format_exc())
            return "misc"

    def _clean_category_name(self, category: str) -> str:
        """
        清理類別名稱，移除底線並轉換為較自然的格式

        Args:
            category: 原始類別名稱

        Returns:
            str: 清理後的類別名稱
        """
        try:
            if not category:
                return "misc"

            # 將底線替換為空格
            cleaned = category.replace('_', ' ')

            # 處理常見的技術性命名模式
            replacements = {
                'kitchen items': 'kitchen items',
                'personal items': 'personal items',
                'traffic light': 'traffic light',
                'misc items': 'misc'
            }

            # 應用特定的替換規則
            for old_term, new_term in replacements.items():
                if cleaned == old_term:
                    return new_term

            return cleaned.strip()

        except Exception as e:
            logger.warning(f"Error cleaning category name '{category}': {str(e)}")
            return "misc"

    def _get_object_categories(self, detected_objects: List[Dict]) -> set:
        """
        從檢測到的物件中獲取唯一的物件類別

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            唯一物件類別的集合
        """
        try:
            return self.object_extractor.get_object_categories(detected_objects)
        except Exception as e:
            logger.error(f"Error in _get_object_categories: {str(e)}")
            logger.error(traceback.format_exc())
            return set()

    def _identify_core_objects_for_scene(self, detected_objects: List[Dict], scene_type: str) -> List[Dict]:
        """
        識別定義特定場景類型的核心物件

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            場景的核心物件列表
        """
        try:
            return self.object_extractor.identify_core_objects_for_scene(detected_objects, scene_type)
        except Exception as e:
            logger.error(f"Error in _identify_core_objects_for_scene: {str(e)}")
            logger.error(traceback.format_exc())
            return []

    def _evaluate_zone_identification_feasibility(self, detected_objects: List[Dict], scene_type: str) -> bool:
        """
        基於物件關聯性和分布特徵的彈性可行性評估

        Args:
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            是否適合進行區域識別
        """
        try:
            return self.zone_evaluator.evaluate_zone_identification_feasibility(detected_objects, scene_type)
        except Exception as e:
            logger.error(f"Error in _evaluate_zone_identification_feasibility: {str(e)}")
            logger.error(traceback.format_exc())
            return False

    def _calculate_functional_relationships(self, detected_objects: List[Dict]) -> float:
        """
        計算物件間的功能關聯性評分

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            功能關聯性評分 (0.0-1.0)
        """
        try:
            return self.zone_evaluator.calculate_functional_relationships(detected_objects)
        except Exception as e:
            logger.error(f"Error in _calculate_functional_relationships: {str(e)}")
            logger.error(traceback.format_exc())
            return 0.0

    def _calculate_spatial_diversity(self, detected_objects: List[Dict]) -> float:
        """
        計算物件空間分布的多樣性

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            空間多樣性評分 (0.0-1.0)
        """
        try:
            return self.zone_evaluator.calculate_spatial_diversity(detected_objects)
        except Exception as e:
            logger.error(f"Error in _calculate_spatial_diversity: {str(e)}")
            logger.error(traceback.format_exc())
            return 0.0

    def _get_complexity_threshold(self, scene_type: str) -> float:
        """
        根據場景類型返回適當的複雜度閾值

        Args:
            scene_type: 場景類型

        Returns:
            複雜度閾值 (0.0-1.0)
        """
        try:
            return self.zone_evaluator.get_complexity_threshold(scene_type)
        except Exception as e:
            logger.error(f"Error in _get_complexity_threshold: {str(e)}")
            logger.error(traceback.format_exc())
            return 0.55

    def _create_distribution_map(self, detected_objects: List[Dict]) -> Dict:
        """
        創建物件在各區域分布的詳細地圖，用於空間分析

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            包含各區域分布詳情的字典
        """
        try:
            return self.region_analyzer.create_distribution_map(detected_objects)
        except Exception as e:
            logger.error(f"Error in _create_distribution_map: {str(e)}")
            logger.error(traceback.format_exc())
            return {}

    def _find_main_region(self, region_objects_dict: Dict) -> str:
        """
        找到物件最多的主要區域

        Args:
            region_objects_dict: 區域物件字典

        Returns:
            主要區域名稱
        """
        try:
            if not region_objects_dict:
                return "unknown"

            return max(region_objects_dict.items(),
                    key=lambda x: len(x[1]),
                    default=("unknown", []))[0]
        except Exception as e:
            logger.error(f"Error in _find_main_region: {str(e)}")
            logger.error(traceback.format_exc())
            return "unknown"

    def _detect_cross_pattern(self, positions):
        """檢測位置中的十字交叉模式 - 委託給SceneViewpointAnalyzer"""
        try:
            return self.scene_viewpoint_analyzer._detect_cross_pattern(positions)
        except Exception as e:
            logger.error(f"Error in _detect_cross_pattern: {str(e)}")
            return False

    def _analyze_movement_directions(self, positions):
        """分析位置中的移動方向 - 委託給SceneViewpointAnalyzer"""
        try:
            return self.scene_viewpoint_analyzer._analyze_movement_directions(positions)
        except Exception as e:
            logger.error(f"Error in _analyze_movement_directions: {str(e)}")
            return []

    def _get_directional_description(self, region: str) -> str:
        """將區域名稱轉換為方位描述 - 委託給RegionAnalyzer"""
        try:
            return self.region_analyzer.get_directional_description(region)
        except Exception as e:
            logger.error(f"Error in _get_directional_description: {str(e)}")
            return "central"

    @property
    def regions(self):
        """提供對區域定義的向後兼容訪問"""
        return self.region_analyzer.regions

Overwriting spatial_analyzer.py


In [ ]:
# %%writefile places365_model.py
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from typing import Dict, List, Tuple, Optional, Any
import logging

class Places365Model:
    """
    Places365 scene classification model wrapper for scene understanding integration.
    Provides scene classification and scene attribute prediction capabilities.
    """

    def __init__(self, model_name: str = 'resnet50_places365', device: Optional[str] = None):
        """
        Initialize Places365 model with configurable architecture and device.

        Args:
            model_name: Model architecture name (默認 resnet50)
            device: Target device for inference (auto-detected if None)
        """
        self.logger = logging.getLogger(self.__class__.__name__)

        # Device configuration with fallback logic
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        self.model_name = model_name
        self.model = None
        self.scene_classes = []
        self.scene_attributes = []

        # Model configuration mapping
        self.model_configs = {
            'resnet18_places365': {
                'arch': 'resnet18',
                'num_classes': 365,
                'url': 'http://places2.csail.mit.edu/models_places365/resnet18_places365.pth.tar'
            },
            'resnet50_places365': {
                'arch': 'resnet50',
                'num_classes': 365,
                'url': 'http://places2.csail.mit.edu/models_places365/resnet50_places365.pth.tar'
            },
            'densenet161_places365': {
                'arch': 'densenet161',
                'num_classes': 365,
                'url': 'http://places2.csail.mit.edu/models_places365/densenet161_places365.pth.tar'
            }
        }

        self._load_model()
        self._load_class_names()
        self._setup_scene_mapping()

    def _load_model(self):
        """載入與初始化 Places365 model"""
        try:
            if self.model_name not in self.model_configs:
                raise ValueError(f"Unsupported model name: {self.model_name}")

            config = self.model_configs[self.model_name]

            # Import model architecture
            if config['arch'].startswith('resnet'):
                import torchvision.models as models
                if config['arch'] == 'resnet18':
                    self.model = models.resnet18(num_classes=config['num_classes'])
                elif config['arch'] == 'resnet50':
                    self.model = models.resnet50(num_classes=config['num_classes'])
            elif config['arch'] == 'densenet161':
                import torchvision.models as models
                self.model = models.densenet161(num_classes=config['num_classes'])

            # Load pretrained weights
            checkpoint = torch.hub.load_state_dict_from_url(
                config['url'],
                map_location=self.device,
                progress=True
            )

            # Handle different checkpoint formats
            if 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
                # Remove 'module.' prefix if present
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            else:
                state_dict = checkpoint

            self.model.load_state_dict(state_dict)
            self.model.to(self.device)
            self.model.eval()

            self.logger.info(f"Places365 model {self.model_name} loaded successfully on {self.device}")

        except Exception as e:
            self.logger.error(f"Error loading Places365 model: {str(e)}")
            raise

    def _load_class_names(self):
        """Load Places365 class names and scene attributes."""
        try:
            # Load scene class names (365 categories)
            import urllib.request

            class_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
            class_file = urllib.request.urlopen(class_url)

            self.scene_classes = []
            for line in class_file:
                class_name = line.decode('utf-8').strip().split(' ')[0][3:]  # Remove /x/ prefix
                self.scene_classes.append(class_name)

            # Load scene attributes (optional, for enhanced description)
            attr_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
            try:
                attr_file = urllib.request.urlopen(attr_url)
                self.scene_attributes = []
                for line in attr_file:
                    attr_name = line.decode('utf-8').strip()
                    self.scene_attributes.append(attr_name)
            except:
                self.logger.warning("Scene attributes not loaded, continuing with basic classification")
                self.scene_attributes = []

            self.logger.info(f"Loaded {len(self.scene_classes)} scene classes and {len(self.scene_attributes)} attributes")

        except Exception as e:
            self.logger.error(f"Error loading class names: {str(e)}")
            # Fallback to basic class names if download fails
            self.scene_classes = [f"scene_class_{i}" for i in range(365)]
            self.scene_attributes = []

    def _setup_scene_mapping(self):
        """Setup mapping from Places365 classes to common scene types."""
        # 建立Places365類別到通用場景類型的映射關係
        self.scene_type_mapping = {
            # Indoor scenes
            'living_room': 'living_room',
            'bedroom': 'bedroom',
            'kitchen': 'kitchen',
            'dining_room': 'dining_area',
            'bathroom': 'bathroom',
            'office': 'office_workspace',
            'conference_room': 'office_workspace',
            'classroom': 'educational_setting',
            'library': 'library',
            'restaurant': 'restaurant',
            'cafe': 'cafe',
            'bar': 'bar',
            'hotel_room': 'hotel_room',
            'hospital_room': 'medical_facility',
            'gym': 'gym',
            'supermarket': 'retail_store',
            'clothing_store': 'retail_store',

            # Outdoor urban scenes
            'street': 'city_street',
            'crosswalk': 'intersection',
            'parking_lot': 'parking_lot',
            'gas_station': 'gas_station',
            'bus_station': 'bus_stop',
            'train_station': 'train_station',
            'airport_terminal': 'airport',
            'subway_station': 'subway_station',
            'bridge': 'bridge',
            'highway': 'highway',
            'downtown': 'commercial_district',
            'shopping_mall': 'shopping_mall',

            # Natural outdoor scenes
            'park': 'park_area',
            'beach': 'beach',
            'forest': 'forest',
            'mountain': 'mountain',
            'lake': 'lake',
            'river': 'river',
            'ocean': 'ocean',
            'desert': 'desert',
            'field': 'field',
            'garden': 'garden',

            # Landmark and tourist areas
            'castle': 'historical_monument',
            'palace': 'historical_monument',
            'temple': 'temple',
            'church': 'church',
            'mosque': 'mosque',
            'museum': 'museum',
            'art_gallery': 'art_gallery',
            'tower': 'tourist_landmark',
            'monument': 'historical_monument',

            # Sports and entertainment
            'stadium': 'stadium',
            'basketball_court': 'sports_field',
            'tennis_court': 'sports_field',
            'swimming_pool': 'swimming_pool',
            'playground': 'playground',
            'amusement_park': 'amusement_park',
            'theater': 'theater',
            'concert_hall': 'concert_hall',

            # Transportation
            'airplane_cabin': 'airplane_cabin',
            'train_interior': 'train_interior',
            'car_interior': 'car_interior',

            # Construction and industrial
            'construction_site': 'construction_site',
            'factory': 'factory',
            'warehouse': 'warehouse'
        }

        # Indoor/outdoor classification helper
        self.indoor_classes = {
            'living_room', 'bedroom', 'kitchen', 'dining_room', 'bathroom', 'office',
            'conference_room', 'classroom', 'library', 'restaurant', 'cafe', 'bar',
            'hotel_room', 'hospital_room', 'gym', 'supermarket', 'clothing_store',
            'airplane_cabin', 'train_interior', 'car_interior', 'theater', 'concert_hall',
            'museum', 'art_gallery', 'shopping_mall'
        }

        self.outdoor_classes = {
            'street', 'crosswalk', 'parking_lot', 'gas_station', 'bus_station',
            'train_station', 'airport_terminal', 'bridge', 'highway', 'downtown',
            'park', 'beach', 'forest', 'mountain', 'lake', 'river', 'ocean',
            'desert', 'field', 'garden', 'stadium', 'basketball_court', 'tennis_court',
            'swimming_pool', 'playground', 'amusement_park', 'construction_site',
            'factory', 'warehouse', 'castle', 'palace', 'temple', 'church', 'mosque',
            'tower', 'monument'
        }

    def preprocess(self, image_pil: Image.Image) -> torch.Tensor:
        """
        Preprocess PIL image for Places365 model inference.

        Args:
            image_pil: Input PIL image

        Returns:
            torch.Tensor: Preprocessed image tensor
        """
        # Places365 standard preprocessing
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Convert to RGB if needed
        if image_pil.mode != 'RGB':
            image_pil = image_pil.convert('RGB')

        # Apply preprocessing
        input_tensor = transform(image_pil).unsqueeze(0)
        return input_tensor.to(self.device)

    def predict(self, image_pil: Image.Image) -> Dict[str, Any]:
        """
        Predict scene classification and attributes for input image.

        Args:
            image_pil: Input PIL image

        Returns:
            Dict containing scene predictions and confidence scores
        """
        try:
            # Preprocess image
            input_tensor = self.preprocess(image_pil)

            # Model inference
            with torch.no_grad():
                outputs = self.model(input_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)

            # 返回最有可能的項目
            top_k = min(10, len(self.scene_classes))  # Configurable top-k
            top_probs, top_indices = torch.topk(probabilities, top_k, dim=1)

            # Extract results
            top_probs = top_probs.cpu().numpy()[0]
            top_indices = top_indices.cpu().numpy()[0]

            # Build prediction results
            predictions = []
            for i in range(top_k):
                class_idx = top_indices[i]
                confidence = float(top_probs[i])
                scene_class = self.scene_classes[class_idx]

                predictions.append({
                    'class_name': scene_class,
                    'class_index': class_idx,
                    'confidence': confidence
                })

            # Get primary prediction
            primary_prediction = predictions[0]
            primary_class = primary_prediction['class_name']

            # 確認是 indoor/outdoor
            is_indoor = self._classify_indoor_outdoor(primary_class)

            # Map to common scene type
            mapped_scene_type = self._map_places365_to_scene_types(primary_class)

            # Determine scene attributes (basic inference based on class)
            scene_attributes = self._infer_scene_attributes(primary_class)

            result = {
                'scene_label': primary_class,
                'mapped_scene_type': mapped_scene_type,
                'confidence': primary_prediction['confidence'],
                'is_indoor': is_indoor,
                'attributes': scene_attributes,
                'top_predictions': predictions,
                'all_probabilities': probabilities.cpu().numpy()[0].tolist()
            }

            return result

        except Exception as e:
            self.logger.error(f"Error in Places365 prediction: {str(e)}")
            return {
                'scene_label': 'unknown',
                'mapped_scene_type': 'unknown',
                'confidence': 0.0,
                'is_indoor': None,
                'attributes': [],
                'top_predictions': [],
                'error': str(e)
            }

    def _classify_indoor_outdoor(self, scene_class: str) -> Optional[bool]:
        """
        Classify if scene is indoor or outdoor based on Places365 class.

        Args:
            scene_class: Places365 scene class name

        Returns:
            bool or None: True for indoor, False for outdoor, None if uncertain
        """
        if scene_class in self.indoor_classes:
            return True
        elif scene_class in self.outdoor_classes:
            return False
        else:
            # For ambiguous classes, use heuristics
            indoor_keywords = ['room', 'office', 'store', 'shop', 'hall', 'interior', 'indoor']
            outdoor_keywords = ['street', 'road', 'park', 'field', 'beach', 'mountain', 'outdoor']

            scene_lower = scene_class.lower()
            if any(keyword in scene_lower for keyword in indoor_keywords):
                return True
            elif any(keyword in scene_lower for keyword in outdoor_keywords):
                return False
            else:
                return None

    def _map_places365_to_scene_types(self, places365_class: str) -> str:
        """
        Map Places365 class to common scene type used by the system.

        Args:
            places365_class: Places365 scene class name

        Returns:
            str: Mapped scene type
        """
        # Direct mapping lookup
        if places365_class in self.scene_type_mapping:
            return self.scene_type_mapping[places365_class]

        # Fuzzy matching for similar classes
        places365_lower = places365_class.lower()

        # Indoor fuzzy matching
        if any(keyword in places365_lower for keyword in ['living', 'bedroom', 'kitchen']):
            return 'general_indoor_space'
        elif any(keyword in places365_lower for keyword in ['office', 'conference', 'meeting']):
            return 'office_workspace'
        elif any(keyword in places365_lower for keyword in ['dining', 'restaurant', 'cafe']):
            return 'dining_area'
        elif any(keyword in places365_lower for keyword in ['store', 'shop', 'market']):
            return 'retail_store'
        elif any(keyword in places365_lower for keyword in ['school', 'class', 'library']):
            return 'educational_setting'

        # Outdoor fuzzy matching
        elif any(keyword in places365_lower for keyword in ['street', 'road', 'crosswalk']):
            return 'city_street'
        elif any(keyword in places365_lower for keyword in ['park', 'garden', 'plaza']):
            return 'park_area'
        elif any(keyword in places365_lower for keyword in ['beach', 'ocean', 'lake']):
            return 'beach'
        elif any(keyword in places365_lower for keyword in ['mountain', 'forest', 'desert']):
            return 'natural_outdoor_area'
        elif any(keyword in places365_lower for keyword in ['parking', 'garage']):
            return 'parking_lot'
        elif any(keyword in places365_lower for keyword in ['station', 'terminal', 'airport']):
            return 'transportation_hub'

        # Landmark fuzzy matching
        elif any(keyword in places365_lower for keyword in ['castle', 'palace', 'monument', 'temple']):
            return 'historical_monument'
        elif any(keyword in places365_lower for keyword in ['tower', 'landmark']):
            return 'tourist_landmark'
        elif any(keyword in places365_lower for keyword in ['museum', 'gallery']):
            return 'cultural_venue'

        # Default fallback based on indoor/outdoor
        is_indoor = self._classify_indoor_outdoor(places365_class)
        if is_indoor is True:
            return 'general_indoor_space'
        elif is_indoor is False:
            return 'generic_street_view'
        else:
            return 'unknown'

    def _infer_scene_attributes(self, scene_class: str) -> List[str]:
        """
        Infer basic scene attributes from Places365 class.

        Args:
            scene_class: Places365 scene class name

        Returns:
            List[str]: Inferred scene attributes
        """
        attributes = []
        scene_lower = scene_class.lower()

        # Lighting attributes
        if any(keyword in scene_lower for keyword in ['outdoor', 'street', 'park', 'beach']):
            attributes.append('natural_lighting')
        elif any(keyword in scene_lower for keyword in ['indoor', 'room', 'office']):
            attributes.append('artificial_lighting')

        # Functional attributes
        if any(keyword in scene_lower for keyword in ['commercial', 'store', 'shop', 'restaurant']):
            attributes.append('commercial')
        elif any(keyword in scene_lower for keyword in ['residential', 'home', 'living', 'bedroom']):
            attributes.append('residential')
        elif any(keyword in scene_lower for keyword in ['office', 'conference', 'meeting']):
            attributes.append('workplace')
        elif any(keyword in scene_lower for keyword in ['recreation', 'park', 'playground', 'stadium']):
            attributes.append('recreational')
        elif any(keyword in scene_lower for keyword in ['educational', 'school', 'library', 'classroom']):
            attributes.append('educational')

        # Spatial attributes
        if any(keyword in scene_lower for keyword in ['open', 'field', 'plaza', 'stadium']):
            attributes.append('open_space')
        elif any(keyword in scene_lower for keyword in ['enclosed', 'room', 'interior']):
            attributes.append('enclosed_space')

        return attributes

    def get_scene_probabilities(self, image_pil: Image.Image) -> Dict[str, float]:
        """
        Get probability distribution over all scene classes.

        Args:
            image_pil: Input PIL image

        Returns:
            Dict mapping scene class names to probabilities
        """
        try:
            input_tensor = self.preprocess(image_pil)

            with torch.no_grad():
                outputs = self.model(input_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)

            probs = probabilities.cpu().numpy()[0]

            return {
                self.scene_classes[i]: float(probs[i])
                for i in range(len(self.scene_classes))
            }

        except Exception as e:
            self.logger.error(f"Error getting scene probabilities: {str(e)}")
            return {}

Writing places365_model.py


In [ ]:
# %%writefile viewpoint_detector.py
import logging
import traceback
from typing import Dict, List, Tuple, Optional
import numpy as np

class ViewpointDetectionError(Exception):
    """Custom exception for errors during viewpoint detection."""
    pass


class ViewpointDetector:
    """
    視角檢測器 - 分析物體分布模式以識別圖像視角類型

    此class負責通過分析檢測到的物體在圖像中的空間分布、大小變化和位置模式，
    來確定圖像的拍攝視角。特別針對行人密集的十字路口場景進行了優化。
    """

    def __init__(self,
                 aerial_threshold: float = 0.7,
                 aerial_size_variance_threshold: float = 0.15,
                 low_angle_threshold: float = 0.3,
                 vertical_size_ratio_threshold: float = 1.8,
                 elevated_threshold: float = 0.6,
                 elevated_top_threshold: float = 0.3,
                 crosswalk_position_tolerance: float = 0.1,
                 crosswalk_axis_tolerance: float = 0.15,
                 min_people_for_crosswalk: int = 8,
                 min_people_for_aerial: int = 10):
        """
        初始化視角檢測器

        Args:
            aerial_threshold: 空中視角檢測的物體密度閾值
            aerial_size_variance_threshold: 空中視角的大小變異閾值
            low_angle_threshold: 低角度視角的底部分布閾值
            vertical_size_ratio_threshold: 垂直大小比例閾值
            elevated_threshold: 高位視角的物體分布閾值
            elevated_top_threshold: 高位視角的頂部物體閾值
            crosswalk_position_tolerance: 十字路口位置容差
            crosswalk_axis_tolerance: 十字路口軸線容差
            min_people_for_crosswalk: 檢測十字路口所需的最少人數
            min_people_for_aerial: 檢測空中視角所需的最少人數
        """
        self.logger = logging.getLogger(self.__class__.__name__)

        # 視角檢測參數配置
        self.viewpoint_params = {
            "aerial_threshold": aerial_threshold,
            "aerial_size_variance_threshold": aerial_size_variance_threshold,
            "low_angle_threshold": low_angle_threshold,
            "vertical_size_ratio_threshold": vertical_size_ratio_threshold,
            "elevated_threshold": elevated_threshold,
            "elevated_top_threshold": elevated_top_threshold,
            "crosswalk_position_tolerance": crosswalk_position_tolerance,
            "crosswalk_axis_tolerance": crosswalk_axis_tolerance,
            "min_people_for_crosswalk": min_people_for_crosswalk,
            "min_people_for_aerial": min_people_for_aerial
        }

        self.logger.info("ViewpointDetector initialized with parameters: %s", self.viewpoint_params)

    def detect_viewpoint(self, detected_objects: List[Dict]) -> str:
        """
        檢測圖像視角類型

        Args:
            detected_objects: 檢測到的物體列表，每個物體應包含位置、大小等信息

        Returns:
            str: 檢測到的視角類型 ('aerial', 'low_angle', 'elevated', 'eye_level')
        """
        try:
            if not detected_objects:
                self.logger.warning("No detected objects provided for viewpoint detection")
                return "eye_level"

            self.logger.info(f"Starting viewpoint detection with {len(detected_objects)} objects")

            # 優先檢測十字路口模式（通常為空中視角）
            if self._detect_crosswalk_pattern(detected_objects):
                self.logger.info("Crosswalk pattern detected - returning aerial viewpoint")
                return "aerial"

            # 檢測基於行人分布的空中視角
            if self._detect_aerial_from_pedestrian_distribution(detected_objects):
                self.logger.info("Aerial viewpoint detected from pedestrian distribution")
                return "aerial"

            # 標準視角檢測流程
            return self._detect_standard_viewpoint(detected_objects)

        except Exception as e:
            error_msg = f"Error during viewpoint detection: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            return "eye_level"  # 返回默認值

    def _detect_crosswalk_pattern(self, detected_objects: List[Dict]) -> bool:
        """
        檢測十字路口/斑馬線模式

        Args:
            detected_objects: 檢測到的物體列表

        Returns:
            bool: 是否檢測到十字路口模式
        """
        try:
            people_objs = [obj for obj in detected_objects if obj.get("class_id") == 0]

            if len(people_objs) < self.viewpoint_params["min_people_for_crosswalk"]:
                return False

            # 提取行人位置
            people_positions = []
            for obj in people_objs:
                if "normalized_center" in obj:
                    people_positions.append(obj["normalized_center"])

            if len(people_positions) < 4:
                return False

            # 檢測十字形分布
            if self._detect_cross_pattern(people_positions):
                self.logger.debug("Cross pattern detected in pedestrian positions")
                return True

            # 檢測線性聚類分布
            if self._detect_linear_crosswalk_clusters(people_positions):
                self.logger.debug("Linear crosswalk clusters detected")
                return True

            return False

        except Exception as e:
            self.logger.warning(f"Error in crosswalk pattern detection: {str(e)}")
            return False

    def _detect_cross_pattern(self, positions: List[Tuple[float, float]]) -> bool:
        """
        檢測十字形分布模式

        Args:
            positions: 物體位置列表 [(x, y), ...]

        Returns:
            bool: 是否檢測到十字形模式
        """
        try:
            x_coords = [pos[0] for pos in positions]
            y_coords = [pos[1] for pos in positions]

            x_range = max(x_coords) - min(x_coords)
            y_range = max(y_coords) - min(y_coords)

            # 檢查 x 和 y 方向都有較大範圍且範圍相似
            if x_range <= 0.5 or y_range <= 0.5:
                return False

            if not (0.7 < (x_range / y_range) < 1.3):
                return False

            # 計算到中心點的距離並檢查軸線分布
            center_x = np.mean(x_coords)
            center_y = np.mean(y_coords)

            close_to_axis_count = 0
            axis_tolerance = self.viewpoint_params["crosswalk_axis_tolerance"]

            for x, y in positions:
                x_distance_to_center = abs(x - center_x)
                y_distance_to_center = abs(y - center_y)

                # 檢查是否接近水平或垂直軸線
                if x_distance_to_center < axis_tolerance or y_distance_to_center < axis_tolerance:
                    close_to_axis_count += 1

            # 如果足夠多的點接近軸線，認為是十字路口
            axis_ratio = close_to_axis_count / len(positions)
            return axis_ratio >= 0.6

        except Exception as e:
            self.logger.warning(f"Error detecting cross pattern: {str(e)}")
            return False

    def _detect_linear_crosswalk_clusters(self, positions: List[Tuple[float, float]]) -> bool:
        """
        檢測線性聚類分布（交叉的斑馬線）

        Args:
            positions: 物體位置列表

        Returns:
            bool: 是否檢測到線性交叉模式
        """
        try:
            x_coords = [pos[0] for pos in positions]
            y_coords = [pos[1] for pos in positions]

            # 檢測 x 和 y 方向的聚類
            x_clusters = self._detect_linear_clusters(x_coords)
            y_clusters = self._detect_linear_clusters(y_coords)

            # 如果在 x 和 y 方向上都有多個聚類，可能是交叉的斑馬線
            return len(x_clusters) >= 2 and len(y_clusters) >= 2

        except Exception as e:
            self.logger.warning(f"Error detecting linear crosswalk clusters: {str(e)}")
            return False

    def _detect_linear_clusters(self, coords: List[float], threshold: float = 0.05) -> List[List[float]]:
        """
        檢測坐標中的線性聚類

        Args:
            coords: 一維坐標列表
            threshold: 聚類閾值

        Returns:
            List[List[float]]: 聚類列表
        """
        if not coords:
            return []

        try:
            sorted_coords = sorted(coords)
            clusters = []
            current_cluster = [sorted_coords[0]]

            for i in range(1, len(sorted_coords)):
                if sorted_coords[i] - sorted_coords[i-1] < threshold:
                    current_cluster.append(sorted_coords[i])
                else:
                    if len(current_cluster) >= 2:
                        clusters.append(current_cluster)
                    current_cluster = [sorted_coords[i]]

            # 添加最後一個聚類
            if len(current_cluster) >= 2:
                clusters.append(current_cluster)

            return clusters

        except Exception as e:
            self.logger.warning(f"Error in linear cluster detection: {str(e)}")
            return []

    def _detect_aerial_from_pedestrian_distribution(self, detected_objects: List[Dict]) -> bool:
        """
        基於行人分布檢測空中視角

        Args:
            detected_objects: 檢測到的物體列表

        Returns:
            bool: 是否為空中視角
        """
        try:
            people_objs = [obj for obj in detected_objects if obj.get("class_id") == 0]

            if len(people_objs) < self.viewpoint_params["min_people_for_aerial"]:
                return False

            # 統計不同區域的行人數量
            people_region_counts = {}
            for obj in people_objs:
                region = obj.get("region", "unknown")
                people_region_counts[region] = people_region_counts.get(region, 0) + 1

            # 檢查行人是否分布在多個區域
            regions_with_multiple_people = sum(1 for count in people_region_counts.values() if count >= 2)

            if regions_with_multiple_people < 4:
                return False

            # 檢查行人分布的均勻性
            region_counts = list(people_region_counts.values())
            if not region_counts:
                return False

            region_counts_variance = np.var(region_counts)
            region_counts_mean = np.mean(region_counts)

            if region_counts_mean > 0:
                variation_coefficient = region_counts_variance / region_counts_mean
                return variation_coefficient < 0.5

            return False

        except Exception as e:
            self.logger.warning(f"Error in aerial detection from pedestrian distribution: {str(e)}")
            return False

    def _detect_standard_viewpoint(self, detected_objects: List[Dict]) -> str:
        """
        標準視角檢測流程

        Args:
            detected_objects: 檢測到的物體列表

        Returns:
            str: 檢測到的視角類型
        """
        try:
            # 計算基本統計指標
            metrics = self._calculate_viewpoint_metrics(detected_objects)

            # 基於計算的指標判斷視角類型
            if self._is_aerial_viewpoint(metrics):
                return "aerial"
            elif self._is_low_angle_viewpoint(metrics):
                return "low_angle"
            elif self._is_elevated_viewpoint(metrics):
                return "elevated"
            else:
                return "eye_level"

        except Exception as e:
            self.logger.warning(f"Error in standard viewpoint detection: {str(e)}")
            return "eye_level"

    def _calculate_viewpoint_metrics(self, detected_objects: List[Dict]) -> Dict:
        """
        計算視角檢測所需的各項指標

        Args:
            detected_objects: 檢測到的物體列表

        Returns:
            Dict: 包含各項指標的字典
        """
        total_objects = len(detected_objects)
        top_region_count = 0
        bottom_region_count = 0
        sizes = []
        height_width_ratios = []

        try:
            for obj in detected_objects:
                # 統計頂部和底部區域的物體數量
                region = obj.get("region", "")
                if "top" in region:
                    top_region_count += 1
                elif "bottom" in region:
                    bottom_region_count += 1

                # 收集大小信息
                if "normalized_area" in obj:
                    sizes.append(obj["normalized_area"])

                # 計算高寬比
                if "normalized_size" in obj:
                    width, height = obj["normalized_size"]
                    if width > 0:
                        height_width_ratios.append(height / width)

            # 計算比例
            top_ratio = top_region_count / total_objects if total_objects > 0 else 0
            bottom_ratio = bottom_region_count / total_objects if total_objects > 0 else 0

            # 計算大小變異係數
            size_variance_coefficient = 0
            if sizes and len(sizes) > 1:
                mean_size = np.mean(sizes)
                if mean_size > 0:
                    size_variance = np.var(sizes)
                    size_variance_coefficient = size_variance / (mean_size ** 2)

            # 計算平均高寬比
            avg_height_width_ratio = np.mean(height_width_ratios) if height_width_ratios else 1.0

            metrics = {
                "top_ratio": top_ratio,
                "bottom_ratio": bottom_ratio,
                "size_variance_coefficient": size_variance_coefficient,
                "avg_height_width_ratio": avg_height_width_ratio,
                "total_objects": total_objects
            }

            self.logger.debug(f"Calculated viewpoint metrics: {metrics}")
            return metrics

        except Exception as e:
            self.logger.error(f"Error calculating viewpoint metrics: {str(e)}")
            return {
                "top_ratio": 0,
                "bottom_ratio": 0,
                "size_variance_coefficient": 0,
                "avg_height_width_ratio": 1.0,
                "total_objects": total_objects
            }

    def _is_aerial_viewpoint(self, metrics: Dict) -> bool:
        """判斷是否為空中視角"""
        return (metrics["size_variance_coefficient"] < self.viewpoint_params["aerial_size_variance_threshold"] and
                metrics["bottom_ratio"] < 0.3 and
                metrics["top_ratio"] > self.viewpoint_params["aerial_threshold"])

    def _is_low_angle_viewpoint(self, metrics: Dict) -> bool:
        """判斷是否為低角度視角"""
        return (metrics["avg_height_width_ratio"] > self.viewpoint_params["vertical_size_ratio_threshold"] and
                metrics["top_ratio"] > self.viewpoint_params["low_angle_threshold"])

    def _is_elevated_viewpoint(self, metrics: Dict) -> bool:
        """判斷是否為高位視角"""
        return (metrics["bottom_ratio"] > self.viewpoint_params["elevated_threshold"] and
                metrics["top_ratio"] < self.viewpoint_params["elevated_top_threshold"])

    def get_viewpoint_confidence(self, detected_objects: List[Dict]) -> Tuple[str, float]:
        """
        獲取視角檢測結果及其信心度

        Args:
            detected_objects: 檢測到的物體列表

        Returns:
            Tuple[str, float]: (視角類型, 信心度)
        """
        try:
            viewpoint = self.detect_viewpoint(detected_objects)

            # 基於檢測條件計算信心度
            if viewpoint == "aerial" and self._detect_crosswalk_pattern(detected_objects):
                confidence = 0.95  # 十字路口模式有很高信心度
            elif viewpoint == "aerial":
                confidence = 0.8
            elif viewpoint == "eye_level":
                confidence = 0.7  # 默認視角信心度較低
            else:
                confidence = 0.85

            self.logger.info(f"Viewpoint detection result: {viewpoint} (confidence: {confidence:.2f})")
            return viewpoint, confidence

        except Exception as e:
            self.logger.warning("Using fallback viewpoint due to detection error")
            return "eye_level", 0.3

Writing viewpoint_detector.py


In [ ]:
# %%writefile template_manager.py
import logging
import traceback
import re
import random
from typing import Dict, List, Optional, Any
import json

# from scene_detail_templates import SCENE_DETAIL_TEMPLATES
# from object_template_fillers import OBJECT_TEMPLATE_FILLERS
# from viewpoint_templates import VIEWPOINT_TEMPLATES
# from cultural_templates import CULTURAL_TEMPLATES
# from lighting_conditions import LIGHTING_CONDITIONS
# from confidence_templates import CONFIDENCE_TEMPLATES

class TemplateLoadingError(Exception):
    """模板載入或處理相關錯誤的自訂例外"""
    pass

class TemplateFillError(Exception):
    pass

class TemplateManager:
    """
    模板管理器 - 負責描述模板的載入、管理和填充

    此class 管理所有用於場景描述生成的模板資源，提供模板填充功能，
    並根據場景類型、物體檢測結果和上下文的資訊給出適當的描述內容。
    """

    def __init__(self, custom_templates_db: Optional[Dict] = None):
        """
        初始化模板管理器

        Args:
            custom_templates_db: 可選的自定義模板數據庫，如果提供則會與默認模板合併
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.template_registry = {}

        try:
            # 載入模板數據庫
            self.templates = self._load_templates()

            # 如果提供了自定義模板，則進行合併
            if custom_templates_db:
                self._merge_custom_templates(custom_templates_db)

            # 驗證模板完整性
            self._validate_templates()

            self.logger.info("TemplateManager initialized successfully with %d template categories",
                        len(self.templates))

        except Exception as e:
            error_msg = f"Failed to initialize TemplateManager: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            # 初始化基本的空模板
            self.templates = self._initialize_fallback_templates()

    def _load_templates(self) -> Dict:
        """
        載入所有描述模板

        Returns:
            Dict: 包含所有模板類別的字典
        """
        try:
            templates = {}

            # 載入場景詳細描述模板
            self.logger.debug("Loading scene detail templates")
            try:
                templates["scene_detail_templates"] = SCENE_DETAIL_TEMPLATES
            except NameError:
                self.logger.warning("SCENE_DETAIL_TEMPLATES not defined, using empty dict")
                templates["scene_detail_templates"] = {}

            # 載入物體模板填充器
            self.logger.debug("Loading object template fillers")
            try:
                templates["object_template_fillers"] = OBJECT_TEMPLATE_FILLERS
            except NameError:
                self.logger.warning("OBJECT_TEMPLATE_FILLERS not defined, using empty dict")
                templates["object_template_fillers"] = {}

            # 載入視角模板
            self.logger.debug("Loading viewpoint templates")
            try:
                templates["viewpoint_templates"] = VIEWPOINT_TEMPLATES
            except NameError:
                self.logger.warning("VIEWPOINT_TEMPLATES not defined, using empty dict")
                templates["viewpoint_templates"] = {}

            # 載入文化模板
            self.logger.debug("Loading cultural templates")
            try:
                templates["cultural_templates"] = CULTURAL_TEMPLATES
            except NameError:
                self.logger.warning("CULTURAL_TEMPLATES not defined, using empty dict")
                templates["cultural_templates"] = {}

            # 從照明條件模組載入照明模板
            self.logger.debug("Loading lighting templates")
            try:
                templates["lighting_templates"] = self._extract_lighting_templates()
            except Exception as e:
                self.logger.warning(f"Failed to extract lighting templates: {str(e)}")
                templates["lighting_templates"] = {}

            # 載入信心度模板
            self.logger.debug("Loading confidence templates")
            try:
                templates["confidence_templates"] = CONFIDENCE_TEMPLATES
            except NameError:
                self.logger.warning("CONFIDENCE_TEMPLATES not defined, using empty dict")
                templates["confidence_templates"] = {}

            # 初始化默認模板（當成備份）
            self._initialize_default_templates(templates)

            self.logger.info("Successfully loaded %d template categories", len(templates))
            return templates

        except Exception as e:
            error_msg = f"Unexpected error during template loading: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            # 返回基本模板
            return self._initialize_fallback_templates()

    def _initialize_template_registry(self) -> Dict[str, Dict[str, Any]]:
        """
        初始化模板，包含各種場景類型的結構化模板

        Returns:
            Dict[str, Dict[str, Any]]: 模板註冊表字典
        """
        try:
            template_registry = {
                "indoor_detailed": {
                    "scene_type": "indoor",
                    "complexity": "high",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "This indoor scene presents a comprehensive view of a well-organized living space."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "functional_areas",
                            "detail_level": "detailed"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_category",
                            "include_counts": True
                        },
                        {
                            "type": "conclusion",
                            "style": "analytical"
                        }
                    ]
                },

                "indoor_moderate": {
                    "scene_type": "indoor",
                    "complexity": "medium",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "The indoor environment displays organized functional areas."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "main_areas",
                            "detail_level": "moderate"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_function",
                            "include_counts": False
                        },
                        {
                            "type": "conclusion",
                            "style": "descriptive"
                        }
                    ]
                },

                "indoor_simple": {
                    "scene_type": "indoor",
                    "complexity": "low",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "An indoor space with visible furniture and household items."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "basic_areas",
                            "detail_level": "simple"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "general",
                            "include_counts": False
                        }
                    ]
                },

                "outdoor_detailed": {
                    "scene_type": "outdoor",
                    "complexity": "high",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "This outdoor scene captures a dynamic urban environment with multiple activity zones."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "activity_areas",
                            "detail_level": "detailed"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_location",
                            "include_counts": True
                        },
                        {
                            "type": "conclusion",
                            "style": "environmental"
                        }
                    ]
                },

                "outdoor_moderate": {
                    "scene_type": "outdoor",
                    "complexity": "medium",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "The outdoor scene shows organized public spaces and pedestrian areas."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "public_areas",
                            "detail_level": "moderate"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_type",
                            "include_counts": False
                        },
                        {
                            "type": "conclusion",
                            "style": "observational"
                        }
                    ]
                },

                "outdoor_simple": {
                    "scene_type": "outdoor",
                    "complexity": "low",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "An outdoor area with pedestrians and urban elements."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "basic_areas",
                            "detail_level": "simple"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "general",
                            "include_counts": False
                        }
                    ]
                },

                "commercial_detailed": {
                    "scene_type": "commercial",
                    "complexity": "high",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "This commercial environment demonstrates organized retail and customer service areas."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "service_areas",
                            "detail_level": "detailed"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_function",
                            "include_counts": True
                        },
                        {
                            "type": "conclusion",
                            "style": "business"
                        }
                    ]
                },

                "transportation_detailed": {
                    "scene_type": "transportation",
                    "complexity": "high",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "This transportation hub features organized passenger facilities and transit infrastructure."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "transit_areas",
                            "detail_level": "detailed"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "by_transit_function",
                            "include_counts": True
                        },
                        {
                            "type": "conclusion",
                            "style": "infrastructure"
                        }
                    ]
                },

                "default": {
                    "scene_type": "general",
                    "complexity": "medium",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "The scene displays various elements organized across functional areas."
                        },
                        {
                            "type": "zone_analysis",
                            "priority": "general_areas",
                            "detail_level": "moderate"
                        },
                        {
                            "type": "object_summary",
                            "grouping": "general",
                            "include_counts": False
                        },
                        {
                            "type": "conclusion",
                            "style": "general"
                        }
                    ]
                }
            }

            self.logger.debug(f"Initialized template registry with {len(template_registry)} templates")
            return template_registry

        except Exception as e:
            error_msg = f"Error initializing template registry: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            # 返回最基本的註冊表
            return {
                "default": {
                    "scene_type": "general",
                    "complexity": "low",
                    "structure": [
                        {
                            "type": "opening",
                            "content": "Scene analysis completed with identified objects and areas."
                        }
                    ]
                }
            }

    def get_template_by_scene_type(self, scene_type: str, detected_objects: List[Dict],
                              functional_zones: Dict) -> str:
        """
        根據場景類型選擇合適的模板並進行標準化處理

        Args:
            scene_type: 場景類型
            detected_objects: 檢測到的物件列表
            functional_zones: 功能區域字典

        Returns:
            str: 標準化後的模板字符串
        """
        try:
            # 獲取場景的物件統計信息
            object_stats = self._analyze_scene_composition(detected_objects)
            zone_count = len(functional_zones) if functional_zones else 0

            # 根據場景複雜度和類型選擇模板
            if scene_type in self.templates:
                scene_templates = self.templates[scene_type]

                # 根據複雜度選擇合適的模板變體
                if zone_count >= 3 and object_stats.get("total_objects", 0) >= 10:
                    template_key = "complex"
                elif zone_count >= 2 or object_stats.get("total_objects", 0) >= 5:
                    template_key = "moderate"
                else:
                    template_key = "simple"

                if template_key in scene_templates:
                    raw_template = scene_templates[template_key]
                else:
                    raw_template = scene_templates.get("default", scene_templates[list(scene_templates.keys())[0]])
            else:
                # 如果沒有特定場景的模板，使用通用模板
                raw_template = self._get_generic_template(object_stats, zone_count)

            # 標準化模板中的佔位符和格式
            standardized_template = self._standardize_template_format(raw_template)
            return standardized_template

        except Exception as e:
            logger.error(f"Error selecting template for scene type '{scene_type}': {str(e)}")
            return self._get_fallback_template()

    def _analyze_scene_composition(self, detected_objects: List[Dict]) -> Dict:
        """
        分析場景組成以確定模板複雜度

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            Dict: 場景組成統計信息
        """
        try:
            total_objects = len(detected_objects)

            # 統計不同類型的物件
            object_categories = {}
            for obj in detected_objects:
                class_name = obj.get("class_name", "unknown")
                object_categories[class_name] = object_categories.get(class_name, 0) + 1

            # 計算場景多樣性
            unique_categories = len(object_categories)

            return {
                "total_objects": total_objects,
                "unique_categories": unique_categories,
                "category_distribution": object_categories,
                "complexity_score": min(total_objects * 0.3 + unique_categories * 0.7, 10)
            }

        except Exception as e:
            logger.warning(f"Error analyzing scene composition: {str(e)}")
            return {"total_objects": 0, "unique_categories": 0, "complexity_score": 0}

    def _get_generic_template(self, object_stats: Dict, zone_count: int) -> str:
        """
        獲取通用模板

        Args:
            object_stats: 物件統計信息
            zone_count: 功能區域數量

        Returns:
            str: 通用模板字符串
        """
        try:
            complexity_score = object_stats.get("complexity_score", 0)

            if complexity_score >= 7 or zone_count >= 3:
                return "This scene presents a comprehensive view featuring {functional_area} with {primary_objects}. The spatial organization demonstrates {spatial_arrangement} across multiple {activity_areas}, creating a dynamic environment with diverse elements and clear functional zones."
            elif complexity_score >= 4 or zone_count >= 2:
                return "The scene displays {functional_area} containing {primary_objects}. The arrangement shows {spatial_organization} with distinct areas serving different purposes within the overall space."
            else:
                return "A {scene_description} featuring {primary_objects} arranged in {basic_layout} within the visible area."

        except Exception as e:
            logger.warning(f"Error getting generic template: {str(e)}")
            return self._get_fallback_template()

    def _get_fallback_template(self) -> str:
        """
        獲取備用模板

        Returns:
            str: 備用模板字符串
        """
        return "A scene featuring various elements and organized areas of activity within the visible space."

    def _standardize_template_format(self, template: str) -> str:
        """
        標準化模板格式，確保佔位符和表達方式符合自然語言要求

        Args:
            template: 原始模板字符串

        Returns:
            str: 標準化後的模板字符串
        """
        try:
            if not template:
                return self._get_fallback_template()

            import re
            standardized = template

            # 標準化佔位符格式，移除技術性標記
            placeholder_mapping = {
                r'\{zone_\d+\}': '{functional_area}',
                r'\{object_group_\d+\}': '{primary_objects}',
                r'\{region_\d+\}': '{spatial_area}',
                r'\{category_\d+\}': '{object_category}',
                r'\{area_\d+\}': '{activity_area}',
                r'\{section_\d+\}': '{scene_section}'
            }

            for pattern, replacement in placeholder_mapping.items():
                standardized = re.sub(pattern, replacement, standardized)

            # 標準化常見的技術性術語
            term_replacements = {
                'functional_zones': 'areas of activity',
                'object_detection': 'visible elements',
                'category_regions': 'organized sections',
                'spatial_distribution': 'arrangement throughout the space',
                'viewpoint_analysis': 'perspective view'
            }

            for tech_term, natural_term in term_replacements.items():
                standardized = standardized.replace(tech_term, natural_term)

            # 確保模板語法的自然性
            standardized = self._improve_template_readability(standardized)

            return standardized

        except Exception as e:
            logger.warning(f"Error standardizing template format: {str(e)}")
            return template if template else self._get_fallback_template()

    def _improve_template_readability(self, template: str) -> str:
        """
        改善模板的可讀性和自然性

        Args:
            template: 模板字符串

        Returns:
            str: 改善後的模板字符串
        """
        try:
            import re

            # 移除多餘的空格和換行
            improved = re.sub(r'\s+', ' ', template).strip()

            # 改善句子連接
            improved = improved.replace(' . ', '. ')
            improved = improved.replace(' , ', ', ')
            improved = improved.replace(' ; ', '; ')

            # 確保適當的句號結尾
            if improved and not improved.endswith(('.', '!', '?')):
                improved += '.'

            # 改善常見的表達問題
            readability_fixes = [
                (r'\bthe the\b', 'the'),
                (r'\ba a\b', 'a'),
                (r'\ban an\b', 'an'),
                (r'\bwith with\b', 'with'),
                (r'\bin in\b', 'in'),
                (r'\bof of\b', 'of'),
                (r'\band and\b', 'and')
            ]

            for pattern, replacement in readability_fixes:
                improved = re.sub(pattern, replacement, improved, flags=re.IGNORECASE)

            return improved

        except Exception as e:
            logger.warning(f"Error improving template readability: {str(e)}")
            return template

    def _extract_lighting_templates(self) -> Dict:
        """
        從照明條件模組提取照明描述模板

        Returns:
            Dict: 照明模板字典
        """
        try:
            lighting_templates = {}

            # 從 LIGHTING_CONDITIONS 提取時間描述
            time_descriptions = LIGHTING_CONDITIONS.get("time_descriptions", {})

            for time_key, time_data in time_descriptions.items():
                if isinstance(time_data, dict) and "general" in time_data:
                    lighting_templates[time_key] = time_data["general"]
                else:
                    # 如果數據結構不符合預期，使用備用描述
                    lighting_templates[time_key] = f"The scene is captured during {time_key.replace('_', ' ')}."

            # 確保至少有基本的照明模板
            if not lighting_templates:
                self.logger.warning("No lighting templates found, using defaults")
                lighting_templates = self._get_default_lighting_templates()

            self.logger.debug("Extracted %d lighting templates", len(lighting_templates))
            return lighting_templates

        except Exception as e:
            self.logger.warning(f"Error extracting lighting templates: {str(e)}, using defaults")
            return self._get_default_lighting_templates()

    def _get_default_lighting_templates(self) -> Dict:
        """獲取默認照明模板"""
        return {
            "day_clear": "The scene is captured during clear daylight conditions.",
            "day_overcast": "The scene is captured during overcast daylight.",
            "night": "The scene is captured at night with artificial lighting.",
            "dawn": "The scene is captured during dawn with soft natural lighting.",
            "dusk": "The scene is captured during dusk with diminishing natural light.",
            "unknown": "The lighting conditions are not clearly identifiable."
        }

    def _initialize_default_templates(self, templates: Dict):
        """
        初始化默認模板作為備份機制

        Args:
            templates: 要檢查和補充的模板字典
        """
        try:
            # 置信度模板備份
            if "confidence_templates" not in templates or not templates["confidence_templates"]:
                templates["confidence_templates"] = {
                    "high": "{description} {details}",
                    "medium": "This appears to be {description} {details}",
                    "low": "This might be {description}, but the confidence is low. {details}"
                }

            # 場景詳細模板備份
            if "scene_detail_templates" not in templates or not templates["scene_detail_templates"]:
                templates["scene_detail_templates"] = {
                    "default": ["A scene with various elements and objects."]
                }

            # 物體填充模板備份
            if "object_template_fillers" not in templates or not templates["object_template_fillers"]:
                templates["object_template_fillers"] = {
                    "default": ["various items", "different objects", "multiple elements"]
                }

            # 視角模板備份
            if "viewpoint_templates" not in templates or not templates["viewpoint_templates"]:
                templates["viewpoint_templates"] = {
                    "eye_level": {
                        "prefix": "From eye level, ",
                        "observation": "the scene is viewed straight ahead.",
                        "short_desc": "at eye level"
                    },
                    "aerial": {
                        "prefix": "From above, ",
                        "observation": "the scene is viewed from a bird's-eye perspective.",
                        "short_desc": "from above"
                    },
                    "low_angle": {
                        "prefix": "From a low angle, ",
                        "observation": "the scene is viewed from below looking upward.",
                        "short_desc": "from below"
                    },
                    "elevated": {
                        "prefix": "From an elevated position, ",
                        "observation": "the scene is viewed from a higher vantage point.",
                        "short_desc": "from an elevated position"
                    }
                }

            # 文化模板備份
            if "cultural_templates" not in templates or not templates["cultural_templates"]:
                templates["cultural_templates"] = {
                    "asian": {
                        "elements": ["traditional architectural elements", "cultural signage", "Asian design features"],
                        "description": "The scene displays distinctive Asian cultural characteristics with {elements}."
                    },
                    "european": {
                        "elements": ["classical architecture", "European design elements", "historic features"],
                        "description": "The scene exhibits European architectural and cultural elements including {elements}."
                    }
                }

            self.logger.debug("Default templates initialized as backup")

        except Exception as e:
            self.logger.error(f"Error initializing default templates: {str(e)}")

    def _merge_custom_templates(self, custom_templates: Dict):
        """
        合併自定義模板到現有模板庫

        Args:
            custom_templates: 自定義模板字典
        """
        try:
            for template_category, custom_content in custom_templates.items():
                if template_category in self.templates:
                    if isinstance(self.templates[template_category], dict) and isinstance(custom_content, dict):
                        self.templates[template_category].update(custom_content)
                        self.logger.debug(f"Merged custom templates for category: {template_category}")
                    else:
                        self.templates[template_category] = custom_content
                        self.logger.debug(f"Replaced templates for category: {template_category}")
                else:
                    self.templates[template_category] = custom_content
                    self.logger.debug(f"Added new template category: {template_category}")

            self.logger.info("Successfully merged custom templates")

        except Exception as e:
            self.logger.warning(f"Error merging custom templates: {str(e)}")

    def _validate_templates(self):
        """
        驗證模板完整性和有效性
        """
        try:
            required_categories = [
                "scene_detail_templates",
                "object_template_fillers",
                "viewpoint_templates",
                "cultural_templates",
                "lighting_templates",
                "confidence_templates"
            ]

            missing_categories = []
            for category in required_categories:
                if category not in self.templates:
                    missing_categories.append(category)
                elif not self.templates[category]:
                    self.logger.warning(f"Template category '{category}' is empty")

            if missing_categories:
                error_msg = f"Missing required template categories: {missing_categories}"
                self.logger.warning(error_msg)
                # 為缺失的類別創建空模板
                for category in missing_categories:
                    self.templates[category] = {}

            # 驗證視角模板結構
            self._validate_viewpoint_templates()

            # 驗證文化模板結構
            self._validate_cultural_templates()

            self.logger.debug("Template validation completed successfully")

        except Exception as e:
            error_msg = f"Template validation failed: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")

    def _validate_viewpoint_templates(self):
        """驗證視角模板結構"""
        viewpoint_templates = self.templates.get("viewpoint_templates", {})

        for viewpoint, template_data in viewpoint_templates.items():
            if not isinstance(template_data, dict):
                self.logger.warning(f"Invalid viewpoint template structure for '{viewpoint}'")
                continue

            required_keys = ["prefix", "observation"]
            for key in required_keys:
                if key not in template_data:
                    self.logger.warning(f"Missing '{key}' in viewpoint template '{viewpoint}'")

    def _validate_cultural_templates(self):
        """驗證文化模板結構"""
        cultural_templates = self.templates.get("cultural_templates", {})

        for culture, template_data in cultural_templates.items():
            if not isinstance(template_data, dict):
                self.logger.warning(f"Invalid cultural template structure for '{culture}'")
                continue

            if "elements" not in template_data or "description" not in template_data:
                self.logger.warning(f"Missing required keys in cultural template '{culture}'")

    def get_template(self, category: str, key: Optional[str] = None) -> Any:
        """
        獲取指定類別的模板

        Args:
            category: 模板類別名稱
            key: 可選的具體模板鍵值

        Returns:
            Any: 請求的模板內容，如果不存在則返回空字典或空字符串
        """
        try:
            if category not in self.templates:
                self.logger.warning(f"Template category '{category}' not found")
                return {} if key is None else ""

            if key is None:
                return self.templates[category]

            category_templates = self.templates[category]
            if not isinstance(category_templates, dict):
                self.logger.warning(f"Template category '{category}' is not a dictionary")
                return ""

            if key not in category_templates:
                self.logger.warning(f"Template key '{key}' not found in category '{category}'")
                return ""

            return category_templates[key]

        except Exception as e:
            error_msg = f"Error retrieving template {category}.{key}: {str(e)}"
            self.logger.error(error_msg)
            return {} if key is None else ""

    def fill_template(self, template: str, detected_objects: List[Dict], scene_type: str,
             places365_info: Optional[Dict] = None,
             object_statistics: Optional[Dict] = None) -> str:
        """
        填充模板中的佔位符，增強容錯處理

        Args:
            template: 包含佔位符的模板字符串
            detected_objects: 檢測到的物體列表
            scene_type: 場景類型
            places365_info: Places365場景分類信息
            object_statistics: 物體統計信息

        Returns:
            str: 填充後的模板字符串，確保語法正確
        """
        try:
            self.logger.debug(f"Filling template for scene_type: {scene_type}")

            if not template or not template.strip():
                return "A scene with various elements."

            # 預處理模板，移除可能的問題模式
            template = self._preprocess_template(template)

            # 查找模板中的佔位符
            placeholders = re.findall(r'\{([^}]+)\}', template)
            filled_template = template

            # 獲取模板填充器
            fillers = self.templates.get("object_template_fillers", {})

            # 基於物體統計信息生成替換內容
            statistics_based_replacements = self._generate_statistics_replacements(object_statistics)

            # 生成默認替換內容
            default_replacements = self._generate_default_replacements()

            # 添加Places365上下文信息
            places365_replacements = self._generate_places365_replacements(places365_info)

            # 添加功能區域信息到場景數據中以便後續使用
            scene_functional_zones = None
            if hasattr(self, '_current_functional_zones'):
                scene_functional_zones = self._current_functional_zones

            # 合併所有替換內容（優先順序是統計信息 > Places365 > 默認）
            all_replacements = {**default_replacements, **places365_replacements, **statistics_based_replacements}

            # 填充每個佔位符
            for placeholder in placeholders:
                try:
                    replacement = self._get_placeholder_replacement(
                        placeholder, fillers, all_replacements, detected_objects, scene_type
                    )

                    # 確保替換內容不為空且有意義
                    if not replacement or not replacement.strip():
                        replacement = self._get_emergency_replacement(placeholder)

                    filled_template = filled_template.replace(f"{{{placeholder}}}", replacement)

                except Exception as placeholder_error:
                    self.logger.warning(f"Failed to replace placeholder '{placeholder}': {str(placeholder_error)}")
                    # 使用緊急替換值
                    emergency_replacement = self._get_emergency_replacement(placeholder)
                    filled_template = filled_template.replace(f"{{{placeholder}}}", emergency_replacement)

            # 修復可能的語法問題
            filled_template = self._postprocess_filled_template(filled_template)

            self.logger.debug("Template filling completed successfully")
            return filled_template

        except Exception as e:
            error_msg = f"Error filling template: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            # 返回安全的備用內容
            return self._generate_fallback_description(scene_type, detected_objects)

    def _preprocess_template(self, template: str) -> str:
        """
        預處理模板，修復常見問題

        Args:
            template: 原始模板字符串

        Returns:
            str: 預處理後的模板
        """
        try:
            # 移除可能導致問題的模式
            template = re.sub(r'\{[^}]*\}\s*,\s*\{[^}]*\}', '{combined_elements}', template)

            # 確保模板不以逗號開始
            template = re.sub(r'^[,\s]*', '', template)

            return template.strip()

        except Exception as e:
            self.logger.warning(f"Error preprocessing template: {str(e)}")
            return template

    def _get_emergency_replacement(self, placeholder: str) -> str:
        """
        獲取緊急替換值，確保不會產生語法錯誤

        Args:
            placeholder: 佔位符名稱

        Returns:
            str: 安全的替換值
        """
        emergency_replacements = {
            "crossing_pattern": "pedestrian walkways",
            "pedestrian_behavior": "people moving through the area",
            "traffic_pattern": "vehicle movement",
            "scene_setting": "this location",
            "urban_elements": "city features",
            "street_elements": "urban components"
        }

        if placeholder in emergency_replacements:
            return emergency_replacements[placeholder]

        # 基於佔位符名稱生成合理的替換
        cleaned = placeholder.replace('_', ' ')
        if len(cleaned.split()) > 1:
            return cleaned
        else:
            return f"various {cleaned}"

    def _postprocess_filled_template(self, filled_template: str) -> str:
        """
        後處理填充完成的模板，修復語法問題

        Args:
            filled_template: 填充後的模板字符串

        Returns:
            str: 修復後的模板字符串
        """
        try:
            # 修復 "In , " 模式
            filled_template = re.sub(r'\bIn\s*,\s*', 'In this scene, ', filled_template)
            filled_template = re.sub(r'\bAt\s*,\s*', 'At this location, ', filled_template)
            filled_template = re.sub(r'\bWithin\s*,\s*', 'Within this area, ', filled_template)

            # 修復連續逗號
            filled_template = re.sub(r',\s*,', ',', filled_template)

            # 修復開頭的逗號
            filled_template = re.sub(r'^[,\s]*', '', filled_template)

            # 確保首字母大寫
            if filled_template and not filled_template[0].isupper():
                filled_template = filled_template[0].upper() + filled_template[1:]

            # 確保以句號結尾
            if filled_template and not filled_template.endswith(('.', '!', '?')):
                filled_template += '.'

            return filled_template.strip()

        except Exception as e:
            self.logger.warning(f"Error postprocessing filled template: {str(e)}")
            return filled_template

    def _generate_fallback_description(self, scene_type: str, detected_objects: List[Dict]) -> str:
        """
        生成備用描述，當模板填充完全失敗時使用

        Args:
            scene_type: 場景類型
            detected_objects: 檢測到的物體列表

        Returns:
            str: 備用描述
        """
        try:
            object_count = len(detected_objects)

            if object_count == 0:
                return f"A {scene_type.replace('_', ' ')} scene."
            elif object_count == 1:
                return f"A {scene_type.replace('_', ' ')} scene with one visible element."
            else:
                return f"A {scene_type.replace('_', ' ')} scene with {object_count} visible elements."

        except Exception as e:
            self.logger.warning(f"Error generating fallback description: {str(e)}")
            return "A scene with various elements."

    def _generate_statistics_replacements(self, object_statistics: Optional[Dict]) -> Dict[str, str]:
        """
        基於物體統計信息生成模板替換內容

        Args:
            object_statistics: 物體統計信息

        Returns:
            Dict[str, str]: 統計信息基礎的替換內容
        """
        replacements = {}

        if not object_statistics:
            return replacements

        try:
            # 處理植物元素
            if "potted plant" in object_statistics:
                count = object_statistics["potted plant"]["count"]
                if count == 1:
                    replacements["plant_elements"] = "a potted plant"
                elif count <= 3:
                    replacements["plant_elements"] = f"{count} potted plants"
                else:
                    replacements["plant_elements"] = f"multiple potted plants ({count} total)"

            # 處理座位
            if "chair" in object_statistics:
                count = object_statistics["chair"]["count"]
                if count == 1:
                    replacements["seating"] = "a chair"
                elif count <= 4:
                    replacements["seating"] = f"{count} chairs"
                else:
                    replacements["seating"] = f"numerous chairs ({count} total)"

            # 處理人員
            if "person" in object_statistics:
                count = object_statistics["person"]["count"]
                if count == 1:
                    replacements["people_and_vehicles"] = "a person"
                    replacements["pedestrian_flow"] = "an individual walking"
                elif count <= 5:
                    replacements["people_and_vehicles"] = f"{count} people"
                    replacements["pedestrian_flow"] = f"{count} people walking"
                else:
                    replacements["people_and_vehicles"] = f"many people ({count} individuals)"
                    replacements["pedestrian_flow"] = f"a crowd of {count} people"

            # 處理桌子設置
            if "dining table" in object_statistics:
                count = object_statistics["dining table"]["count"]
                if count == 1:
                    replacements["table_setup"] = "a dining table"
                    replacements["table_description"] = "a dining surface"
                else:
                    replacements["table_setup"] = f"{count} dining tables"
                    replacements["table_description"] = f"{count} dining surfaces"

            self.logger.debug(f"Generated {len(replacements)} statistics-based replacements")

        except Exception as e:
            self.logger.warning(f"Error generating statistics replacements: {str(e)}")

        return replacements

    def _generate_places365_replacements(self, places365_info: Optional[Dict]) -> Dict[str, str]:
        """
        基於Places365信息生成模板替換內容

        Args:
            places365_info: Places365場景分類信息

        Returns:
            Dict[str, str]: Places365基礎的替換內容
        """
        replacements = {}

        if not places365_info or places365_info.get('confidence', 0) <= 0.35:
            replacements["places365_context"] = ""
            replacements["places365_atmosphere"] = ""
            return replacements

        try:
            scene_label = places365_info.get('scene_label', '').replace('_', ' ')
            attributes = places365_info.get('attributes', [])

            # 生成場景上下文
            if scene_label:
                replacements["places365_context"] = f"characteristic of a {scene_label}"
            else:
                replacements["places365_context"] = ""

            # 生成氛圍描述
            if 'natural_lighting' in attributes:
                replacements["places365_atmosphere"] = "with natural illumination"
            elif 'artificial_lighting' in attributes:
                replacements["places365_atmosphere"] = "under artificial lighting"
            else:
                replacements["places365_atmosphere"] = ""

            self.logger.debug("Generated Places365-based replacements")

        except Exception as e:
            self.logger.warning(f"Error generating Places365 replacements: {str(e)}")
            replacements["places365_context"] = ""
            replacements["places365_atmosphere"] = ""

        return replacements

    def _generate_default_replacements(self) -> Dict[str, str]:
        """
        生成默認的模板替換內容

        Returns:
            Dict[str, str]: 默認替換內容
        """
        return {

            "scene_introduction": "this scene",
            "location_prefix": "this location",
            "setting_description": "this setting",
            "area_description": "this area",
            "environment_description": "this environment",
            "spatial_introduction": "this space",

            # 室內相關
            "furniture": "various furniture pieces",
            "seating": "comfortable seating",
            "electronics": "entertainment devices",
            "bed_type": "a bed",
            "bed_location": "room",
            "bed_description": "sleeping arrangements",
            "extras": "personal items",
            "table_setup": "a dining table and chairs",
            "table_description": "a dining surface",
            "dining_items": "dining furniture and tableware",
            "appliances": "kitchen appliances",
            "kitchen_items": "cooking utensils and dishware",
            "cooking_equipment": "cooking equipment",
            "office_equipment": "work-related furniture and devices",
            "desk_setup": "a desk and chair",
            "computer_equipment": "electronic devices",

            # 室外/城市相關
            "traffic_description": "vehicles and pedestrians",
            "people_and_vehicles": "people and various vehicles",
            "street_elements": "urban infrastructure",
            "park_features": "benches and greenery",
            "outdoor_elements": "natural features",
            "park_description": "outdoor amenities",
            "store_elements": "merchandise displays",
            "shopping_activity": "customers browse and shop",
            "store_items": "products for sale",

            # 高級餐廳相關
            "design_elements": "elegant decor",
            "lighting": "stylish lighting fixtures",

            # 亞洲商業街相
            "storefront_features": "compact shops",
            "pedestrian_flow": "people walking",
            "asian_elements": "distinctive cultural elements",
            "cultural_elements": "traditional design features",
            "signage": "colorful signs",
            "street_activities": "busy urban activity",

            # 金融區相關
            "buildings": "tall buildings",
            "traffic_elements": "vehicles",
            "skyscrapers": "high-rise buildings",
            "road_features": "wide streets",
            "architectural_elements": "modern architecture",
            "city_landmarks": "prominent structures",

            # 十字路口相關
            "crossing_pattern": "clearly marked pedestrian crossings",
            "pedestrian_behavior": "careful pedestrian movement",
            "pedestrian_density": "multiple groups of pedestrians",
            "traffic_pattern": "well-regulated traffic flow",
            "pedestrian_flow": "steady pedestrian movement",
            "traffic_description": "active urban traffic",
            "people_and_vehicles": "pedestrians and vehicles",
            "street_elements": "urban infrastructure elements",

            # 交通相關
            "transit_vehicles": "public transportation vehicles",
            "passenger_activity": "commuter movement",
            "transportation_modes": "various transit options",
            "passenger_needs": "waiting areas",
            "transit_infrastructure": "transit facilities",
            "passenger_movement": "commuter flow",

            # 購物區相關
            "retail_elements": "shops and displays",
            "store_types": "various retail establishments",
            "walkway_features": "pedestrian pathways",
            "commercial_signage": "store signs",
            "consumer_behavior": "shopping activities",

            # 空中視角相關
            "commercial_layout": "organized retail areas",
            "pedestrian_pattern": "people movement patterns",
            "gathering_features": "public gathering spaces",
            "movement_pattern": "crowd flow patterns",
            "urban_elements": "city infrastructure",
            "public_activity": "social interaction",

            # 文化特定元素
            "stall_elements": "vendor booths",
            "lighting_features": "decorative lights",
            "food_elements": "food offerings",
            "vendor_stalls": "market stalls",
            "nighttime_activity": "evening commerce",
            "cultural_lighting": "traditional lighting",
            "night_market_sounds": "lively market sounds",
            "evening_crowd_behavior": "nighttime social activity",
            "architectural_elements": "cultural buildings",
            "religious_structures": "sacred buildings",
            "decorative_features": "ornamental designs",
            "cultural_practices": "traditional activities",
            "temple_architecture": "religious structures",
            "sensory_elements": "atmospheric elements",
            "visitor_activities": "cultural experiences",
            "ritual_activities": "ceremonial practices",
            "cultural_symbols": "meaningful symbols",
            "architectural_style": "historical buildings",
            "historic_elements": "traditional architecture",
            "urban_design": "city planning elements",
            "social_behaviors": "public interactions",
            "european_features": "European architectural details",
            "tourist_activities": "visitor activities",
            "local_customs": "regional practices",

            # 時間特定元素
            "lighting_effects": "artificial lighting",
            "shadow_patterns": "light and shadow",
            "urban_features": "city elements",
            "illuminated_elements": "lit structures",
            "evening_activities": "nighttime activities",
            "light_sources": "lighting points",
            "lit_areas": "illuminated spaces",
            "shadowed_zones": "darker areas",
            "illuminated_signage": "bright signs",
            "colorful_lighting": "multicolored lights",
            "neon_elements": "neon signs",
            "night_crowd_behavior": "evening social patterns",
            "light_displays": "lighting installations",
            "building_features": "architectural elements",
            "nightlife_activities": "evening entertainment",
            "lighting_modifier": "bright",

            # 混合環境元素
            "transitional_elements": "connecting features",
            "indoor_features": "interior elements",
            "outdoor_setting": "exterior spaces",
            "interior_amenities": "inside comforts",
            "exterior_features": "outside elements",
            "inside_elements": "interior design",
            "outside_spaces": "outdoor areas",
            "dual_environment_benefits": "combined settings",
            "passenger_activities": "waiting behaviors",
            "transportation_types": "transit vehicles",
            "sheltered_elements": "covered areas",
            "exposed_areas": "open sections",
            "waiting_behaviors": "passenger activities",
            "indoor_facilities": "inside services",
            "platform_features": "transit platform elements",
            "transit_routines": "transportation procedures",

            # 專門場所元素
            "seating_arrangement": "spectator seating",
            "playing_surface": "athletic field",
            "sporting_activities": "sports events",
            "spectator_facilities": "viewer accommodations",
            "competition_space": "sports arena",
            "sports_events": "athletic competitions",
            "viewing_areas": "audience sections",
            "field_elements": "field markings and equipment",
            "game_activities": "competitive play",
            "construction_equipment": "building machinery",
            "building_materials": "construction supplies",
            "construction_activities": "building work",
            "work_elements": "construction tools",
            "structural_components": "building structures",
            "site_equipment": "construction gear",
            "raw_materials": "building supplies",
            "construction_process": "building phases",
            "medical_elements": "healthcare equipment",
            "clinical_activities": "medical procedures",
            "facility_design": "healthcare layout",
            "healthcare_features": "medical facilities",
            "patient_interactions": "care activities",
            "equipment_types": "medical devices",
            "care_procedures": "health services",
            "treatment_spaces": "clinical areas",
            "educational_furniture": "learning furniture",
            "learning_activities": "educational practices",
            "instructional_design": "teaching layout",
            "classroom_elements": "school equipment",
            "teaching_methods": "educational approaches",
            "student_engagement": "learning participation",
            "learning_spaces": "educational areas",
            "educational_tools": "teaching resources",
            "knowledge_transfer": "learning exchanges"
        }

    def _generate_objects_summary(self, detected_objects: List[Dict]) -> str:
        """
        基於檢測物件生成自然語言摘要，按重要性排序

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            str: 物件摘要描述
        """
        try:
            # detected_objects 裡有幾個 traffic light)
            tl_count = len([obj for obj in detected_objects if obj.get("class_name","") == "traffic light"])
            # print(f"[DEBUG] _generate_objects_summary 傳入的 detected_objects 中 traffic light: {tl_count} 個")
            for obj in detected_objects:
                if obj.get("class_name","") == "traffic light":
                    print(f"    - conf={obj.get('confidence',0):.4f}, bbox={obj.get('bbox')}, region={obj.get('region')}")

            if not detected_objects:
                return "various elements"

            # calculate object statistic
            object_counts = {}
            total_confidence = 0

            for obj in detected_objects:
                class_name = obj.get("class_name", "unknown")
                confidence = obj.get("confidence", 0.5)

                if class_name not in object_counts:
                    object_counts[class_name] = {"count": 0, "total_confidence": 0}

                object_counts[class_name]["count"] += 1
                object_counts[class_name]["total_confidence"] += confidence
                total_confidence += confidence

            # 計算平均置信度並排序
            sorted_objects = []
            for class_name, stats in object_counts.items():
                avg_confidence = stats["total_confidence"] / stats["count"]
                count = stats["count"]

                # 重要性評分：結合數量和置信度
                importance_score = (count * 0.6) + (avg_confidence * 0.4)
                sorted_objects.append((class_name, count, importance_score))

            # 按重要性排序，取前5個最重要的物件
            sorted_objects.sort(key=lambda x: x[2], reverse=True)
            top_objects = sorted_objects[:5]

            # 生成自然語言描述
            descriptions = []
            for class_name, count, _ in top_objects:
                clean_name = class_name.replace('_', ' ')
                if count == 1:
                    article = "an" if clean_name[0].lower() in 'aeiou' else "a"
                    descriptions.append(f"{article} {clean_name}")
                else:
                    descriptions.append(f"{count} {clean_name}s")

            # 組合描述
            if len(descriptions) == 1:
                return descriptions[0]
            elif len(descriptions) == 2:
                return f"{descriptions[0]} and {descriptions[1]}"
            else:
                return ", ".join(descriptions[:-1]) + f", and {descriptions[-1]}"

        except Exception as e:
            self.logger.warning(f"Error generating objects summary: {str(e)}")
            return "various elements"

    def _get_placeholder_replacement(self, placeholder: str, fillers: Dict,
                           all_replacements: Dict, detected_objects: List[Dict],
                           scene_type: str) -> str:
        """
        獲取特定佔位符的替換內容，確保永遠不返回空值
        """
        try:
            # 優先處理動態內容生成的佔位符
            dynamic_placeholders = [
                'primary_objects', 'detected_objects_summary', 'main_objects',
                'functional_area', 'functional_zones_description', 'scene_elements'
            ]

            if placeholder in dynamic_placeholders:
                dynamic_content = self._generate_objects_summary(detected_objects)
                if dynamic_content and dynamic_content.strip():
                    return dynamic_content.strip()

            # 檢查預定義替換內容
            if placeholder in all_replacements:
                replacement = all_replacements[placeholder]
                if replacement and replacement.strip():
                    return replacement.strip()

            # 檢查物體模板填充器
            if placeholder in fillers:
                options = fillers[placeholder]
                if options and isinstance(options, list):
                    valid_options = [opt.strip() for opt in options if opt and str(opt).strip()]
                    if valid_options:
                        num_items = min(len(valid_options), random.randint(1, 3))
                        selected_items = random.sample(valid_options, num_items)

                        if len(selected_items) == 1:
                            return selected_items[0]
                        elif len(selected_items) == 2:
                            return f"{selected_items[0]} and {selected_items[1]}"
                        else:
                            return ", ".join(selected_items[:-1]) + f", and {selected_items[-1]}"

            # 基於檢測對象生成動態內容
            scene_specific_replacement = self._generate_scene_specific_content(
                placeholder, detected_objects, scene_type
            )
            if scene_specific_replacement and scene_specific_replacement.strip():
                return scene_specific_replacement.strip()

            # 通用備用字典 - 擴展版本
            fallback_replacements = {
                # 交通和城市相關
                "crossing_pattern": "pedestrian crosswalks",
                "pedestrian_behavior": "people moving carefully",
                "traffic_pattern": "vehicle movement",
                "urban_elements": "city infrastructure",
                "street_elements": "urban features",
                "intersection_features": "traffic management systems",
                "pedestrian_density": "groups of people",
                "pedestrian_flow": "pedestrian movement",
                "traffic_description": "vehicle traffic",
                "people_and_vehicles": "pedestrians and cars",

                # 場景設置相關
                "scene_setting": "this urban environment",
                "location_context": "the area",
                "spatial_context": "the scene",
                "environmental_context": "this location",

                # 常見的家具和設備
                "furniture": "various furniture pieces",
                "seating": "seating arrangements",
                "electronics": "electronic devices",
                "appliances": "household appliances",

                # 活動和行為
                "activities": "various activities",
                "interactions": "people interacting",
                "movement": "movement patterns",

                # 照明和氛圍
                "lighting_conditions": "ambient lighting",
                "atmosphere": "the overall atmosphere",
                "ambiance": "environmental ambiance",

                # 空間描述
                "spatial_arrangement": "spatial organization",
                "layout": "the layout",
                "composition": "visual composition",

                # 物體和元素
                "objects": "various objects",
                "elements": "scene elements",
                "features": "notable features",
                "details": "observable details"
            }

            if placeholder in fallback_replacements:
                return fallback_replacements[placeholder]

            # 基於場景類型的智能默認值
            scene_based_defaults = self._get_scene_based_default(placeholder, scene_type)
            if scene_based_defaults:
                return scene_based_defaults

            # 最終備用：將下劃線轉換為有意義的短語
            cleaned_placeholder = placeholder.replace('_', ' ')

            # 對常見模式提供更好的默認值
            if placeholder.endswith('_pattern'):
                return f"{cleaned_placeholder.replace(' pattern', '')} arrangement"
            elif placeholder.endswith('_behavior'):
                return f"{cleaned_placeholder.replace(' behavior', '')} activity"
            elif placeholder.endswith('_description'):
                return f"{cleaned_placeholder.replace(' description', '')} elements"
            elif placeholder.endswith('_elements'):
                return cleaned_placeholder
            elif placeholder.endswith('_features'):
                return cleaned_placeholder
            else:
                return cleaned_placeholder if cleaned_placeholder != placeholder else "various elements"

        except Exception as e:
            self.logger.warning(f"Error getting replacement for placeholder '{placeholder}': {str(e)}")
            # 確保即使在異常情況下也返回有意義的內容
            return placeholder.replace('_', ' ') if placeholder else "scene elements"

    def _get_scene_based_default(self, placeholder: str, scene_type: str) -> Optional[str]:
        """
        基於場景類型提供智能默認值

        Args:
            placeholder: 佔位符名稱
            scene_type: 場景類型

        Returns:
            Optional[str]: 場景特定的默認值或None
        """
        try:
            # 針對不同場景類型的特定默認值
            scene_defaults = {
                "urban_intersection": {
                    "crossing_pattern": "marked crosswalks",
                    "pedestrian_behavior": "pedestrians crossing carefully",
                    "traffic_pattern": "controlled traffic flow"
                },
                "city_street": {
                    "traffic_description": "urban vehicle traffic",
                    "street_elements": "city infrastructure",
                    "people_and_vehicles": "pedestrians and vehicles"
                },
                "living_room": {
                    "furniture": "comfortable living room furniture",
                    "seating": "sofas and chairs",
                    "electronics": "entertainment equipment"
                },
                "kitchen": {
                    "appliances": "kitchen appliances",
                    "cooking_equipment": "cooking tools and equipment"
                },
                "office_workspace": {
                    "office_equipment": "work furniture and devices",
                    "desk_setup": "desk and office chair"
                }
            }

            if scene_type in scene_defaults and placeholder in scene_defaults[scene_type]:
                return scene_defaults[scene_type][placeholder]

            return None

        except Exception as e:
            self.logger.warning(f"Error getting scene-based default for '{placeholder}' in '{scene_type}': {str(e)}")
            return None

    def _generate_scene_specific_content(self, placeholder: str, detected_objects: List[Dict],
                                       scene_type: str) -> Optional[str]:
        """
        基於場景特定邏輯生成佔位符內容

        Args:
            placeholder: 佔位符名稱
            detected_objects: 檢測到的物體列表
            scene_type: 場景類型

        Returns:
            Optional[str]: 生成的內容或None
        """
        try:
            if placeholder == "furniture":
                # 提取家具物品
                furniture_ids = [56, 57, 58, 59, 60, 61]  # 家具類別ID
                furniture_objects = [obj for obj in detected_objects if obj.get("class_id") in furniture_ids]

                if furniture_objects:
                    furniture_names = [obj.get("class_name", "furniture") for obj in furniture_objects[:3]]
                    unique_names = list(set(furniture_names))
                    return ", ".join(unique_names) if len(unique_names) > 1 else unique_names[0]
                return "various furniture items"

            elif placeholder == "electronics":
                # 提取電子設備
                electronics_ids = [62, 63, 64, 65, 66, 67, 68, 69, 70]  # 電子設備類別ID
                electronics_objects = [obj for obj in detected_objects if obj.get("class_id") in electronics_ids]

                if electronics_objects:
                    electronics_names = [obj.get("class_name", "electronic device") for obj in electronics_objects[:3]]
                    unique_names = list(set(electronics_names))
                    return ", ".join(unique_names) if len(unique_names) > 1 else unique_names[0]
                return "electronic devices"

            elif placeholder == "people_count":
                # 計算人數
                people_count = len([obj for obj in detected_objects if obj.get("class_id") == 0])

                if people_count == 0:
                    return "no people"
                elif people_count == 1:
                    return "one person"
                elif people_count < 5:
                    return f"{people_count} people"
                else:
                    return "several people"

            elif placeholder == "seating":
                # 提取座位物品
                seating_ids = [56, 57]  # chair, sofa
                seating_objects = [obj for obj in detected_objects if obj.get("class_id") in seating_ids]

                if seating_objects:
                    seating_names = [obj.get("class_name", "seating") for obj in seating_objects[:2]]
                    unique_names = list(set(seating_names))
                    return ", ".join(unique_names) if len(unique_names) > 1 else unique_names[0]
                return "seating arrangements"

            # 如果沒有匹配的特定邏輯，返回None
            return None

        except Exception as e:
            self.logger.warning(f"Error generating scene-specific content for '{placeholder}': {str(e)}")
            return None

    def get_confidence_template(self, confidence_level: str) -> str:
        """
        獲取指定信心度級別的模板

        Args:
            confidence_level: 信心度級別 ('high', 'medium', 'low')

        Returns:
            str: 信心度模板字符串
        """
        try:
            confidence_templates = self.templates.get("confidence_templates", {})

            if confidence_level in confidence_templates:
                return confidence_templates[confidence_level]

            # 備用模板
            fallback_templates = {
                "high": "{description} {details}",
                "medium": "This appears to be {description} {details}",
                "low": "This might be {description}, but the confidence is low. {details}"
            }

            return fallback_templates.get(confidence_level, "{description} {details}")

        except Exception as e:
            self.logger.warning(f"Error getting confidence template for '{confidence_level}': {str(e)}")
            return "{description} {details}"

    def get_lighting_template(self, lighting_type: str) -> str:
        """
        獲取指定照明類型的模板

        Args:
            lighting_type: 照明類型

        Returns:
            str: 照明描述模板
        """
        try:
            lighting_templates = self.templates.get("lighting_templates", {})

            if lighting_type in lighting_templates:
                return lighting_templates[lighting_type]

            # 備用模板
            return f"The scene is captured with {lighting_type.replace('_', ' ')} lighting conditions."

        except Exception as e:
            self.logger.warning(f"Error getting lighting template for '{lighting_type}': {str(e)}")
            return "The lighting conditions are not clearly identifiable."

    def get_viewpoint_template(self, viewpoint: str) -> Dict[str, str]:
        """
        獲取指定視角的模板

        Args:
            viewpoint: 視角類型

        Returns:
            Dict[str, str]: 包含prefix、observation等鍵的視角模板字典
        """
        try:
            viewpoint_templates = self.templates.get("viewpoint_templates", {})

            if viewpoint in viewpoint_templates:
                return viewpoint_templates[viewpoint]

            # 備用模板
            fallback_templates = {
                "eye_level": {
                    "prefix": "From eye level, ",
                    "observation": "the scene is viewed straight ahead.",
                    "short_desc": "at eye level"
                },
                "aerial": {
                    "prefix": "From above, ",
                    "observation": "the scene is viewed from a bird's-eye perspective.",
                    "short_desc": "from above"
                },
                "low_angle": {
                    "prefix": "From a low angle, ",
                    "observation": "the scene is viewed from below looking upward.",
                    "short_desc": "from below"
                },
                "elevated": {
                    "prefix": "From an elevated position, ",
                    "observation": "the scene is viewed from a higher vantage point.",
                    "short_desc": "from an elevated position"
                }
            }

            return fallback_templates.get(viewpoint, fallback_templates["eye_level"])

        except Exception as e:
            self.logger.warning(f"Error getting viewpoint template for '{viewpoint}': {str(e)}")
            return {
                "prefix": "",
                "observation": "the scene is viewed normally.",
                "short_desc": "normally"
            }

    def get_cultural_template(self, cultural_context: str) -> Dict[str, Any]:
        """
        獲取指定文化語境的模板

        Args:
            cultural_context: 文化語境

        Returns:
            Dict[str, Any]: 文化模板字典
        """
        try:
            cultural_templates = self.templates.get("cultural_templates", {})

            if cultural_context in cultural_templates:
                return cultural_templates[cultural_context]

            # 備用模板
            return {
                "elements": ["cultural elements"],
                "description": f"The scene displays {cultural_context} cultural characteristics."
            }

        except Exception as e:
            self.logger.warning(f"Error getting cultural template for '{cultural_context}': {str(e)}")
            return {
                "elements": ["various elements"],
                "description": "The scene displays cultural characteristics."
            }

    def get_scene_detail_templates(self, scene_type: str, viewpoint: Optional[str] = None) -> List[str]:
        """
        獲取場景詳細描述模板

        Args:
            scene_type: 場景類型
            viewpoint: 可選的視角類型

        Returns:
            List[str]: 場景描述模板列表
        """
        try:
            scene_templates = self.templates.get("scene_detail_templates", {})

            # 首先嘗試獲取特定視角的模板
            if viewpoint:
                viewpoint_key = f"{scene_type}_{viewpoint}"
                if viewpoint_key in scene_templates:
                    return scene_templates[viewpoint_key]

            # 然後嘗試獲取場景類型的通用模板
            if scene_type in scene_templates:
                return scene_templates[scene_type]

            # 最後使用默認模板
            if "default" in scene_templates:
                return scene_templates["default"]

            # 備用模板
            return ["A scene with various elements and objects."]

        except Exception as e:
            self.logger.warning(f"Error getting scene detail templates for '{scene_type}': {str(e)}")
            return ["A scene with various elements and objects."]

    def reload_templates(self):
        """
        重新載入所有模板
        """
        try:
            self.template_manager.reload_templates()
            self.logger.info("Templates reloaded successfully")
        except Exception as e:
            self.logger.error(f"Error reloading templates: {str(e)}")

    def get_template_categories(self) -> List[str]:
        """
        獲取所有可用的模板類別名稱

        Returns:
            List[str]: 模板類別名稱列表
        """
        return list(self.templates.keys())

    def template_exists(self, category: str, key: Optional[str] = None) -> bool:
        """
        檢查模板是否存在

        Args:
            category: 模板類別
            key: 可選的模板鍵值

        Returns:
            bool: 模板是否存在
        """
        try:
            if category not in self.templates:
                return False

            if key is None:
                return True

            category_templates = self.templates[category]
            if isinstance(category_templates, dict):
                return key in category_templates

            return False

        except Exception as e:
            self.logger.warning(f"Error checking template existence for {category}.{key}: {str(e)}")
            return False

    def apply_template(self, template: Union[str, Dict[str, Any]], scene_data: Dict[str, Any]) -> str:
        """
        應用選定的模板來生成場景描述

        Args:
            template: 模板字符串或模板內容字典
            scene_data: 場景分析的資料字典

        Returns:
            str: 最終生成的場景描述
        """
        try:
            # 如果傳入的是字符串模板，直接使用填充邏輯
            if isinstance(template, str):
                self.logger.debug("Processing string template directly")

                # 提取場景數據
                detected_objects = scene_data.get("detected_objects", [])
                scene_type = scene_data.get("scene_type", "general")
                places365_info = scene_data.get("places365_info")
                object_statistics = scene_data.get("object_statistics")
                functional_zones = scene_data.get("functional_zones", {})

                # 暫存功能區域資訊供填充邏輯使用
                self._current_functional_zones = functional_zones

                # 使用現有的填充邏輯
                filled_description = self.fill_template(
                    template,
                    detected_objects,
                    scene_type,
                    places365_info,
                    object_statistics
                )

                # 清理暫存資訊
                if hasattr(self, '_current_functional_zones'):
                    delattr(self, '_current_functional_zones')

                return filled_description

            # 如果傳入的是字典結構模板
            elif isinstance(template, dict):
                self.logger.debug("Processing structured template")
                return self._process_structured_template(template, scene_data)

            # 如果是模板名稱字符串且需要從registry獲取
            elif hasattr(self, 'template_registry') and template in self.template_registry:
                template_dict = self.template_registry[template]
                return self._process_structured_template(template_dict, scene_data)

            else:
                self.logger.warning(f"Invalid template format or template not found: {type(template)}")
                return self._generate_fallback_scene_description(scene_data)

        except Exception as e:
            self.logger.error(f"Error applying template: {str(e)}")
            return self._generate_fallback_scene_description(scene_data)

    def _process_structured_template(self, template: Dict[str, Any], scene_data: Dict[str, Any]) -> str:
        """
        處理結構化模板字典

        Args:
            template: 結構化模板字典
            scene_data: 場景分析資料

        Returns:
            str: 生成的場景描述
        """
        try:
            # 提取 scene_data 中各區塊資料
            zone_data = scene_data.get("functional_zones", scene_data.get("zones", {}))
            object_data = scene_data.get("detected_objects", [])
            scene_context = scene_data.get("scene_context", "")

            # 獲取模板結構
            structure = template.get("structure", [])
            if not structure:
                self.logger.warning("Template has no structure defined")
                return self._generate_fallback_scene_description(scene_data)

            description_parts = []

            # 按照模板結構生成描述
            for section in structure:
                section_type = section.get("type", "")
                content = section.get("content", "")

                if section_type == "opening":
                    description_parts.append(content)

                elif section_type == "zone_analysis":
                    zone_descriptions = self._generate_zone_descriptions(zone_data, section)
                    if zone_descriptions:
                        description_parts.extend(zone_descriptions)

                elif section_type == "object_summary":
                    object_summary = self._generate_object_summary(object_data, section)
                    if object_summary:
                        description_parts.append(object_summary)

                elif section_type == "conclusion":
                    conclusion = self._generate_conclusion(template, zone_data, object_data)
                    if conclusion:
                        description_parts.append(conclusion)

            # 合併並標準化輸出
            final_description = self._standardize_final_description(" ".join(description_parts))
            self.logger.info("Successfully applied structured template")
            return final_description

        except Exception as e:
            self.logger.error(f"Error processing structured template: {str(e)}")
            return self._generate_fallback_scene_description(scene_data)

    def _generate_fallback_scene_description(self, scene_data: Dict[str, Any]) -> str:
        """
        生成備用場景描述

        Args:
            scene_data: 場景分析資料

        Returns:
            str: 備用場景描述
        """
        try:
            detected_objects = scene_data.get("detected_objects", [])
            zones = scene_data.get("functional_zones", scene_data.get("zones", {}))
            scene_type = scene_data.get("scene_type", "general")

            object_count = len(detected_objects)
            zone_count = len(zones)

            if zone_count > 0 and object_count > 0:
                return f"Scene analysis completed with {zone_count} functional areas containing {object_count} identified objects."
            elif object_count > 0:
                return f"Scene analysis identified {object_count} objects in this {scene_type.replace('_', ' ')} environment."
            else:
                return f"Scene analysis completed for this {scene_type.replace('_', ' ')} environment."

        except Exception as e:
            self.logger.warning(f"Error generating fallback description: {str(e)}")
            return "Scene analysis completed with detected objects and functional areas."


    def _generate_zone_descriptions(self, zone_data: Dict[str, Any], section: Dict[str, Any]) -> List[str]:
        """
        生成功能區域描述
        """
        try:
            descriptions = []

            if not zone_data:
                return descriptions

            # 直接處理區域資料（zone_data 本身就是區域字典）
            sorted_zones = sorted(zone_data.items(),
                                key=lambda x: len(x[1].get("objects", [])),
                                reverse=True)

            for zone_name, zone_info in sorted_zones:
                description = zone_info.get("description", "")
                objects = zone_info.get("objects", [])

                if objects:
                    # 使用現有描述或生成基於物件的描述
                    if description and not any(tech in description.lower() for tech in ['zone', 'area', 'region']):
                        zone_desc = description
                    else:
                        # 生成更自然的區域描述
                        clean_zone_name = zone_name.replace('_', ' ').replace(' area', '').replace(' zone', '')
                        object_list = ', '.join(objects[:3])

                        if 'crossing' in zone_name or 'pedestrian' in zone_name:
                            zone_desc = f"In the central crossing area, there are {object_list}."
                        elif 'vehicle' in zone_name or 'traffic' in zone_name:
                            zone_desc = f"The vehicle movement area includes {object_list}."
                        elif 'control' in zone_name:
                            zone_desc = f"Traffic control elements include {object_list}."
                        else:
                            zone_desc = f"The {clean_zone_name} contains {object_list}."

                        if len(objects) > 3:
                            zone_desc += f" Along with {len(objects) - 3} additional elements."

                    descriptions.append(zone_desc)

            return descriptions

        except Exception as e:
            logger.error(f"Error generating zone descriptions: {str(e)}")
            return []

    def _generate_object_summary(self, object_data: List[Dict], section: Dict[str, Any]) -> str:
        """
        生成物件摘要描述
        """
        try:
            if not object_data:
                return ""

            # 統計物件類型並計算重要性
            object_stats = {}
            for obj in object_data:
                class_name = obj.get("class_name", "unknown")
                confidence = obj.get("confidence", 0.5)

                if class_name not in object_stats:
                    object_stats[class_name] = {"count": 0, "total_confidence": 0}

                object_stats[class_name]["count"] += 1
                object_stats[class_name]["total_confidence"] += confidence

            # 按重要性排序（結合數量和置信度）
            sorted_objects = []
            for class_name, stats in object_stats.items():
                count = stats["count"]
                avg_confidence = stats["total_confidence"] / count
                importance = count * 0.6 + avg_confidence * 0.4
                sorted_objects.append((class_name, count, importance))

            sorted_objects.sort(key=lambda x: x[2], reverse=True)

            # 生成自然語言描述
            descriptions = []
            for class_name, count, _ in sorted_objects[:5]:
                clean_name = class_name.replace('_', ' ')
                if count == 1:
                    article = "an" if clean_name[0].lower() in 'aeiou' else "a"
                    descriptions.append(f"{article} {clean_name}")
                else:
                    descriptions.append(f"{count} {clean_name}s")

            if len(descriptions) == 1:
                return f"The scene features {descriptions[0]}."
            elif len(descriptions) == 2:
                return f"The scene features {descriptions[0]} and {descriptions[1]}."
            else:
                main_items = ", ".join(descriptions[:-1])
                return f"The scene features {main_items}, and {descriptions[-1]}."

        except Exception as e:
            self.logger.error(f"Error generating object summary: {str(e)}")
            return ""

    def _generate_conclusion(self, template: Dict[str, Any], zone_data: Dict[str, Any],
                            object_data: List[Dict]) -> str:
        """
        生成結論描述
        """
        try:
            scene_type = template.get("scene_type", "general")
            zones_count = len(zone_data)
            objects_count = len(object_data)

            if scene_type == "indoor":
                conclusion = f"This indoor environment demonstrates clear functional organization with {zones_count} distinct areas and {objects_count} identified objects."
            elif scene_type == "outdoor":
                conclusion = f"This outdoor scene shows dynamic activity patterns across {zones_count} functional zones with {objects_count} detected elements."
            else:
                conclusion = f"The scene analysis reveals {zones_count} functional areas containing {objects_count} identifiable objects."

            return conclusion

        except Exception as e:
            logger.error(f"Error generating conclusion: {str(e)}")
            return ""

    def _standardize_final_description(self, description: str) -> str:
        """
        對最終描述進行標準化處理

        Args:
            description: 原始描述文本

        Returns:
            str: 標準化後的描述文本
        """
        try:
            # 移除多餘空格
            description = " ".join(description.split())

            # 確保句子間有適當間距
            description = description.replace(". ", ". ")

            # 移除任何殘留的技術性標識符
            technical_patterns = [
                r'zone_\d+', r'area_\d+', r'region_\d+',
                r'_zone', r'_area', r'_region'
            ]

            for pattern in technical_patterns:
                description = re.sub(pattern, '', description, flags=re.IGNORECASE)

            return description.strip()

        except Exception as e:
            logger.error(f"Error standardizing final description: {str(e)}")
            return description

Writing template_manager.py


In [ ]:
# %%writefile object_description_generator.py
import logging
import traceback
from typing import Dict, List, Tuple, Optional, Any
import numpy as np

class ObjectDescriptionError(Exception):
    """物件描述生成過程中的自定義異常"""
    pass


class ObjectDescriptionGenerator:
    """
    物件描述生成器 - 負責將檢測到的物件轉換為自然語言描述

    該類別處理物件相關的所有描述生成邏輯，包括重要物件的識別、
    空間位置描述、物件列表格式化以及描述文本的優化。
    """

    def __init__(self,
                 min_prominence_score: float = 0.1,
                 max_categories_to_return: int = 5,
                 max_total_objects: int = 7,
                 confidence_threshold_for_description: float = 0.25,
                 region_analyzer: Optional[Any] = None):
        """
        初始化物件描述生成器

        Args:
            min_prominence_score: 物件顯著性的最低分數閾值
            max_categories_to_return: 返回的物件類別最大數量
            max_total_objects: 返回的物件總數上限
            confidence_threshold_for_description: 用於描述的置信度閾值
        """
        self.logger = logging.getLogger(self.__class__.__name__)

        self.min_prominence_score = min_prominence_score
        self.max_categories_to_return = max_categories_to_return
        self.max_total_objects = max_total_objects
        self.confidence_threshold_for_description = confidence_threshold_for_description
        self.region_analyzer = region_analyzer

        self.logger.info("ObjectDescriptionGenerator initialized with prominence_score=%.2f, "
                        "max_categories=%d, max_objects=%d, confidence_threshold=%.2f",
                        min_prominence_score, max_categories_to_return,
                        max_total_objects, confidence_threshold_for_description)

    def get_prominent_objects(self, detected_objects: List[Dict],
                          min_prominence_score: float = 0.5,
                          max_categories_to_return: Optional[int] = None) -> List[Dict]:
        """
        獲取最重要的物件，基於置信度、大小和位置計算重要性評分

        Args:
            detected_objects: 檢測到的物件列表
            min_prominence_score: 最小重要性分數閾值，範圍 0.0-1.0
            max_categories_to_return: 可選的最大返回類別數量限制

        Returns:
            List[Dict]: 按重要性排序的物件列表
        """
        try:
            if not detected_objects:
                return []

            prominent_objects = []

            for obj in detected_objects:
                # 計算重要性評分
                prominence_score = self._calculate_prominence_score(obj)

                # 只保留超過閾值的物件
                if prominence_score >= min_prominence_score:
                    obj_copy = obj.copy()
                    obj_copy['prominence_score'] = prominence_score
                    prominent_objects.append(obj_copy)

            # 按重要性評分排序（從高到低）
            prominent_objects.sort(key=lambda x: x.get('prominence_score', 0), reverse=True)

            # 如果指定了最大類別數量限制，進行過濾
            if max_categories_to_return is not None and max_categories_to_return > 0:
                categories_seen = set()
                filtered_objects = []

                for obj in prominent_objects:
                    class_name = obj.get("class_name", "unknown")

                    # 如果是新類別且未達到限制
                    if class_name not in categories_seen:
                        if len(categories_seen) < max_categories_to_return:
                            categories_seen.add(class_name)
                            filtered_objects.append(obj)
                    else:
                        # 已見過的類別，直接添加
                        filtered_objects.append(obj)

                return filtered_objects

            return prominent_objects

        except Exception as e:
            self.logger.error(f"Error calculating prominent objects: {str(e)}")
            return []

    def set_region_analyzer(self, region_analyzer: Any) -> None:
        """
        設置RegionAnalyzer，用於標準化空間描述生成

        Args:
            region_analyzer: RegionAnalyzer實例
        """
        try:
            self.region_analyzer = region_analyzer
            self.logger.info("RegionAnalyzer instance set for ObjectDescriptionGenerator")
        except Exception as e:
            self.logger.warning(f"Error setting RegionAnalyzer: {str(e)}")

    def _get_standardized_spatial_description(self, obj: Dict) -> str:
        """
        使用RegionAnalyzer生成標準化空間描述的內部方法

        Args:
            obj: 物件字典

        Returns:
            str: 標準化空間描述，失敗時返回空字串
        """
        try:
            if hasattr(self, 'region_analyzer') and self.region_analyzer:
                region = obj.get("region", "")
                object_type = obj.get("class_name", "")

                if hasattr(self.region_analyzer, 'get_contextual_spatial_description'):
                    return self.region_analyzer.get_contextual_spatial_description(region, object_type)
                elif hasattr(self.region_analyzer, 'get_spatial_description_phrase'):
                    return self.region_analyzer.get_spatial_description_phrase(region)

            return ""

        except Exception as e:
            self.logger.warning(f"Error getting standardized spatial description: {str(e)}")
            if object_type:
                return f"visible in the scene"
            return "present in the view"

    def _calculate_prominence_score(self, obj: Dict) -> float:
        """
        計算物件的重要性評分

        Args:
            obj: 物件字典，包含檢測信息

        Returns:
            float: 重要性評分 (0.0-1.0)
        """
        try:
            # 基礎置信度評分 (權重: 40%)
            confidence = obj.get("confidence", 0.5)
            confidence_score = confidence * 0.4

            # 大小評分 (權重: 30%)
            normalized_area = obj.get("normalized_area", 0.1)
            # 使用對數縮放避免過大物件主導評分
            size_score = min(np.log(normalized_area * 10 + 1) / np.log(11), 1.0) * 0.3

            # 位置評分 (權重: 20%)
            # 中心區域的物件通常更重要
            center_x, center_y = obj.get("normalized_center", [0.5, 0.5])
            distance_from_center = np.sqrt((center_x - 0.5)**2 + (center_y - 0.5)**2)
            position_score = (1 - min(distance_from_center * 2, 1.0)) * 0.2

            # 類別重要性評分 (權重: 10%)
            class_importance = self._get_class_importance(obj.get("class_name", "unknown"))
            class_score = class_importance * 0.1

            total_score = confidence_score + size_score + position_score + class_score

            # 確保評分在有效範圍內
            return max(0.0, min(1.0, total_score))

        except Exception as e:
            self.logger.warning(f"Error calculating prominence score for object: {str(e)}")
            return 0.5  # 返回中等評分作為備用

    def _get_class_importance(self, class_name: str) -> float:
        """
        根據物件類別返回重要性係數

        Args:
            class_name: 物件類別名稱

        Returns:
            float: 類別重要性係數 (0.0-1.0)
        """
        # 高重要性物件（人、車輛、建築）
        high_importance = ["person", "car", "truck", "bus", "motorcycle", "bicycle", "building"]

        # 中等重要性物件（家具、電器）
        medium_importance = ["chair", "couch", "tv", "laptop", "refrigerator", "dining table", "bed"]

        # 低重要性物件（小物品、配件）
        low_importance = ["handbag", "backpack", "umbrella", "cell phone", "remote", "mouse"]

        class_name_lower = class_name.lower()

        if any(item in class_name_lower for item in high_importance):
            return 1.0
        elif any(item in class_name_lower for item in medium_importance):
            return 0.7
        elif any(item in class_name_lower for item in low_importance):
            return 0.4
        else:
            return 0.6  # 預設中等重要性

    def format_object_list_for_description(self,
                                          objects: List[Dict],
                                          use_indefinite_article_for_one: bool = False,
                                          count_threshold_for_generalization: int = -1,
                                          max_types_to_list: int = 5) -> str:
        """
        將物件列表格式化為人類可讀的字符串，包含計數信息

        Args:
            objects: 物件字典列表，每個應包含 'class_name'
            use_indefinite_article_for_one: 單個物件是否使用 "a/an"，否則使用 "one"
            count_threshold_for_generalization: 超過此計數時使用通用術語，-1表示精確計數
            max_types_to_list: 列表中包含的不同物件類型最大數量

        Returns:
            str: 格式化的物件描述字符串
        """
        try:
            if not objects:
                return "no specific objects clearly identified"

            counts: Dict[str, int] = {}
            for obj in objects:
                name = obj.get("class_name", "unknown object")
                if name == "unknown object" or not name:
                    continue
                counts[name] = counts.get(name, 0) + 1

            if not counts:
                return "no specific objects clearly identified"

            descriptions = []
            # 按計數降序然後按名稱升序排序，限制物件類型數量
            sorted_counts = sorted(counts.items(), key=lambda item: (-item[1], item[0]))[:max_types_to_list]

            for name, count in sorted_counts:
                if count == 1:
                    if use_indefinite_article_for_one:
                        if name[0].lower() in 'aeiou':
                            descriptions.append(f"an {name}")
                        else:
                            descriptions.append(f"a {name}")
                    else:
                        descriptions.append(f"one {name}")
                else:
                    # 處理複數形式
                    plural_name = name
                    if name.endswith("y") and not name.lower().endswith(("ay", "ey", "iy", "oy", "uy")):
                        plural_name = name[:-1] + "ies"
                    elif name.endswith(("s", "sh", "ch", "x", "z")):
                        plural_name = name + "es"
                    elif not name.endswith("s"):
                        plural_name = name + "s"

                    if count_threshold_for_generalization != -1 and count > count_threshold_for_generalization:
                        if count <= count_threshold_for_generalization + 3:
                            descriptions.append(f"several {plural_name}")
                        else:
                            descriptions.append(f"many {plural_name}")
                    else:
                        descriptions.append(f"{count} {plural_name}")

            if not descriptions:
                return "no specific objects clearly identified"

            if len(descriptions) == 1:
                return descriptions[0]
            elif len(descriptions) == 2:
                return f"{descriptions[0]} and {descriptions[1]}"
            else:
                # 使用牛津逗號格式
                return ", ".join(descriptions[:-1]) + f", and {descriptions[-1]}"

        except Exception as e:
            self.logger.warning(f"Error formatting object list: {str(e)}")
            return "various objects"

    def get_spatial_description(self, obj: Dict, image_width: Optional[int] = None,
                           image_height: Optional[int] = None,
                           region_analyzer: Optional[Any] = None) -> str:
        """
        為物件生成空間位置描述

        Args:
            obj: 物件字典
            image_width: 可選的圖像寬度
            image_height: 可選的圖像高度
            region_analyzer: 可選的RegionAnalyzer實例，用於生成標準化描述

        Returns:
            str: 空間描述字符串，空值region時返回空字串
        """
        try:
            region = obj.get("region") or ""

            # 處理空值或無效region，直接返回空字串避免不完整描述
            if not region.strip() or region == "unknown":
                # 根據物件類型提供合適的預設位置描述
                if object_type and any(vehicle in object_type.lower() for vehicle in ["car", "truck", "bus"]):
                    return "positioned in the scene"
                elif object_type and "person" in object_type.lower():
                    return "present in the area"
                else:
                    return "located in the scene"

            # 如果提供了RegionAnalyzer實例，使用其標準化方法
            if region_analyzer and hasattr(region_analyzer, 'get_spatial_description_phrase'):
                object_type = obj.get("class_name", "")
                if hasattr(region_analyzer, 'get_contextual_spatial_description'):
                    spatial_desc = region_analyzer.get_contextual_spatial_description(region, object_type)
                else:
                    spatial_desc = region_analyzer.get_spatial_description_phrase(region)

                if spatial_desc:
                    return spatial_desc

            # 備用邏輯：使用改進的內建映射
            clean_region = region.replace('_', ' ').strip().lower()

            region_map = {
                "top left": "in the upper left area",
                "top center": "in the upper area",
                "top right": "in the upper right area",
                "middle left": "on the left side",
                "middle center": "in the center",
                "center": "in the center",
                "middle right": "on the right side",
                "bottom left": "in the lower left area",
                "bottom center": "in the lower area",
                "bottom right": "in the lower right area"
            }

            # 直接映射匹配
            if clean_region in region_map:
                return region_map[clean_region]

            # 模糊匹配處理
            if "top" in clean_region and "left" in clean_region:
                return "in the upper left area"
            elif "top" in clean_region and "right" in clean_region:
                return "in the upper right area"
            elif "bottom" in clean_region and "left" in clean_region:
                return "in the lower left area"
            elif "bottom" in clean_region and "right" in clean_region:
                return "in the lower right area"
            elif "top" in clean_region:
                return "in the upper area"
            elif "bottom" in clean_region:
                return "in the lower area"
            elif "left" in clean_region:
                return "on the left side"
            elif "right" in clean_region:
                return "on the right side"
            elif "center" in clean_region or "middle" in clean_region:
                return "in the center"

            # 如果region無法識別，使用normalized_center作為最後備用
            norm_center = obj.get("normalized_center")
            if norm_center and image_width and image_height:
                x_norm, y_norm = norm_center
                h_pos = "left" if x_norm < 0.4 else "right" if x_norm > 0.6 else "center"
                v_pos = "upper" if y_norm < 0.4 else "lower" if y_norm > 0.6 else "center"

                if h_pos == "center" and v_pos == "center":
                    return "in the center"
                return f"in the {v_pos} {h_pos} area"

            # 如果所有方法都失敗，返回空字串
            return ""

        except Exception as e:
            self.logger.warning(f"Error generating spatial description: {str(e)}")
            return ""

    def optimize_object_description(self, description: str) -> str:
        """
        優化物件描述，避免重複列舉相同物件

        Args:
            description: 原始描述文本

        Returns:
            str: 優化後的描述文本
        """
        try:
            import re

            # 處理床鋪重複描述
            if "bed in the room" in description:
                description = description.replace("a bed in the room", "a bed")

            # 處理重複的物件列表
            object_lists = re.findall(r'with ([^\.]+?)(?:\.|\band\b)', description)

            for obj_list in object_lists:
                # 計算每個物件出現次數
                items = re.findall(r'([a-zA-Z\s]+)(?:,|\band\b|$)', obj_list)
                item_counts = {}

                for item in items:
                    item = item.strip()
                    if item and item not in ["and", "with"]:
                        if item not in item_counts:
                            item_counts[item] = 0
                        item_counts[item] += 1

                # 生成優化後的物件列表
                if item_counts:
                    new_items = []
                    for item, count in item_counts.items():
                        if count > 1:
                            new_items.append(f"{count} {item}s")
                        else:
                            new_items.append(item)

                    # 格式化新列表
                    if len(new_items) == 1:
                        new_list = new_items[0]
                    elif len(new_items) == 2:
                        new_list = f"{new_items[0]} and {new_items[1]}"
                    else:
                        new_list = ", ".join(new_items[:-1]) + f", and {new_items[-1]}"

                    # 替換原始列表
                    description = description.replace(obj_list, new_list)

            return description

        except Exception as e:
            self.logger.warning(f"Error optimizing object description: {str(e)}")
            return description

    def generate_dynamic_everyday_description(self,
                                            detected_objects: List[Dict],
                                            lighting_info: Optional[Dict] = None,
                                            viewpoint: str = "eye_level",
                                            spatial_analysis: Optional[Dict] = None,
                                            image_dimensions: Optional[Tuple[int, int]] = None,
                                            places365_info: Optional[Dict] = None,
                                            object_statistics: Optional[Dict] = None) -> str:
        """
        為日常場景動態生成描述，基於所有相關的檢測物件、計數和上下文

        Args:
            detected_objects: 檢測到的物件列表
            lighting_info: 照明信息
            viewpoint: 視角類型
            spatial_analysis: 空間分析結果
            image_dimensions: 圖像尺寸
            places365_info: Places365場景分類信息
            object_statistics: 物件統計信息

        Returns:
            str: 動態生成的場景描述
        """
        try:
            description_segments = []
            image_width, image_height = image_dimensions if image_dimensions else (None, None)

            self.logger.debug(f"Generating dynamic description for {len(detected_objects)} objects, "
                            f"viewpoint: {viewpoint}, lighting: {lighting_info is not None}")

            # 1. 整體氛圍（照明和視角）
            ambiance_parts = []
            if lighting_info:
                time_of_day = lighting_info.get("time_of_day", "unknown lighting")
                is_indoor = lighting_info.get("is_indoor")
                ambiance_statement = "This is"
                if is_indoor is True:
                    ambiance_statement += " an indoor scene"
                elif is_indoor is False:
                    ambiance_statement += " an outdoor scene"
                else:
                    ambiance_statement += " a scene"

                # remove underline
                readable_lighting = f"with {time_of_day.replace('_', ' ')} lighting conditions"
                ambiance_statement += f", likely {readable_lighting}."
                ambiance_parts.append(ambiance_statement)

            if viewpoint and viewpoint != "eye_level":
                if not ambiance_parts:
                    ambiance_parts.append(f"From {viewpoint.replace('_', ' ')}, the general layout of the scene is observed.")
                else:
                    ambiance_parts[-1] = ambiance_parts[-1].rstrip('.') + f", viewed from {viewpoint.replace('_', ' ')}."

            if ambiance_parts:
                description_segments.append(" ".join(ambiance_parts))

            # 2. 描述所有檢測到的物件，按類別分組，使用準確計數和位置
            if not detected_objects:
                if not description_segments:
                    description_segments.append("A general scene is visible, but no specific objects were clearly identified.")
                else:
                    description_segments.append("Within this setting, no specific objects were clearly identified.")
            else:
                objects_by_class: Dict[str, List[Dict]] = {}

                # 使用置信度過濾
                confident_objects = [obj for obj in detected_objects
                                   if obj.get("confidence", 0) >= self.confidence_threshold_for_description]

                if not confident_objects:
                    no_confident_obj_msg = "While some elements might be present, no objects were identified with sufficient confidence for a detailed description."
                    if not description_segments:
                        description_segments.append(no_confident_obj_msg)
                    else:
                        description_segments.append(no_confident_obj_msg.lower().capitalize())
                else:
                    if object_statistics:
                        # 使用預計算的統計信息，採用動態的信心度
                        for class_name, stats in object_statistics.items():
                            count = stats.get("count", 0)
                            avg_confidence = stats.get("avg_confidence", 0)

                            # 動態調整置信度閾值
                            dynamic_threshold = self.confidence_threshold_for_description
                            if class_name in ["potted plant", "vase", "clock", "book"]:
                                dynamic_threshold = max(0.15, self.confidence_threshold_for_description * 0.6)
                            elif count >= 3:
                                dynamic_threshold = max(0.2, self.confidence_threshold_for_description * 0.8)

                            if count > 0 and avg_confidence >= dynamic_threshold:
                                matching_objects = [obj for obj in confident_objects if obj.get("class_name") == class_name]
                                if not matching_objects:
                                    matching_objects = [obj for obj in detected_objects
                                                      if obj.get("class_name") == class_name and obj.get("confidence", 0) >= dynamic_threshold]

                                if matching_objects:
                                    actual_count = min(stats["count"], len(matching_objects))
                                    objects_by_class[class_name] = matching_objects[:actual_count]
                    else:
                        # 備用邏輯，同樣使用動態閾值
                        for obj in confident_objects:
                            name = obj.get("class_name", "unknown object")
                            if name == "unknown object" or not name:
                                continue
                            if name not in objects_by_class:
                                objects_by_class[name] = []
                            objects_by_class[name].append(obj)

                    if not objects_by_class:
                        description_segments.append("No common objects were confidently identified for detailed description.")
                    else:
                        # 物件組排序函數
                        def sort_key_object_groups(item_tuple: Tuple[str, List[Dict]]):
                            class_name_key, obj_group_list = item_tuple
                            priority = 3
                            count = len(obj_group_list)

                            # 確保類別名稱已標準化
                            normalized_class_name = self._normalize_object_class_name(class_name_key)

                            # 動態優先級
                            if normalized_class_name == "person":
                                priority = 0
                            elif normalized_class_name in ["dining table", "chair", "sofa", "bed"]:
                                priority = 1
                            elif normalized_class_name in ["car", "bus", "truck", "traffic light"]:
                                priority = 2
                            elif count >= 3:
                                priority = max(1, priority - 1)
                            elif normalized_class_name in ["potted plant", "vase", "clock", "book"] and count >= 2:
                                priority = 2

                            avg_area = sum(o.get("normalized_area", 0.0) for o in obj_group_list) / len(obj_group_list) if obj_group_list else 0
                            quantity_bonus = min(count / 5.0, 1.0)

                            return (priority, -len(obj_group_list), -avg_area, -quantity_bonus)

                        # remove duplicate
                        deduplicated_objects_by_class = {}
                        processed_positions = []

                        for class_name, group_of_objects in objects_by_class.items():
                            unique_objects = []

                            for obj in group_of_objects:
                                obj_position = obj.get("normalized_center", [0.5, 0.5])
                                is_duplicate = False

                                for processed_pos in processed_positions:
                                    position_distance = abs(obj_position[0] - processed_pos[0]) + abs(obj_position[1] - processed_pos[1])
                                    if position_distance < 0.15:
                                        is_duplicate = True
                                        break

                                if not is_duplicate:
                                    unique_objects.append(obj)
                                    processed_positions.append(obj_position)

                            if unique_objects:
                                deduplicated_objects_by_class[class_name] = unique_objects

                        objects_by_class = deduplicated_objects_by_class
                        sorted_object_groups = sorted(objects_by_class.items(), key=sort_key_object_groups)

                        object_clauses = []

                        for class_name, group_of_objects in sorted_object_groups:
                            count = len(group_of_objects)
                            if count == 0:
                                continue

                            # 標準化class name
                            normalized_class_name = self._normalize_object_class_name(class_name)

                            # 使用統計信息確保準確的數量描述
                            if object_statistics and class_name in object_statistics:
                                actual_count = object_statistics[class_name]["count"]
                                formatted_name_with_exact_count = self._format_object_count_description(
                                    normalized_class_name, actual_count
                                )
                            else:
                                formatted_name_with_exact_count = self._format_object_count_description(
                                    normalized_class_name, count
                                )

                            if formatted_name_with_exact_count == "no specific objects clearly identified" or not formatted_name_with_exact_count:
                                continue

                            # 確定群組的集體位置
                            location_description_suffix = ""
                            if count == 1:
                                spatial_desc = self.get_spatial_description(group_of_objects[0], image_width, image_height, self.region_analyzer)
                                if spatial_desc:
                                    location_description_suffix = f"is {spatial_desc}"
                                else:
                                    distinct_regions = sorted(list(set(obj.get("region", "") for obj in group_of_objects if obj.get("region"))))
                                    valid_regions = [r for r in distinct_regions if r and r != "unknown" and r.strip()]
                                    if not valid_regions:
                                        location_description_suffix = "is positioned in the scene"
                                    elif len(valid_regions) == 1:
                                        spatial_desc = self.get_spatial_description_phrase(valid_regions[0])
                                        location_description_suffix = f"is primarily {spatial_desc}" if spatial_desc else "is positioned in the scene"
                                    elif len(valid_regions) == 2:
                                        clean_region1 = valid_regions[0].replace('_', ' ')
                                        clean_region2 = valid_regions[1].replace('_', ' ')
                                        location_description_suffix = f"is mainly across the {clean_region1} and {clean_region2} areas"
                                    else:
                                        location_description_suffix = "is distributed in various parts of the scene"
                            else:
                                distinct_regions = sorted(list(set(obj.get("region", "") for obj in group_of_objects if obj.get("region"))))
                                valid_regions = [r for r in distinct_regions if r and r != "unknown" and r.strip()]
                                if not valid_regions:
                                    location_description_suffix = "are visible in the scene"
                                elif len(valid_regions) == 1:
                                    clean_region = valid_regions[0].replace('_', ' ')
                                    location_description_suffix = f"are primarily in the {clean_region} area"
                                elif len(valid_regions) == 2:
                                    clean_region1 = valid_regions[0].replace('_', ' ')
                                    clean_region2 = valid_regions[1].replace('_', ' ')
                                    location_description_suffix = f"are mainly across the {clean_region1} and {clean_region2} areas"
                                else:
                                    location_description_suffix = "are distributed in various parts of the scene"

                            # 首字母大寫
                            formatted_name_capitalized = formatted_name_with_exact_count[0].upper() + formatted_name_with_exact_count[1:]
                            object_clauses.append(f"{formatted_name_capitalized} {location_description_suffix}")

                        if object_clauses:
                            if not description_segments:
                                if object_clauses:
                                    first_clause = object_clauses.pop(0)
                                    description_segments.append(first_clause + ".")
                            else:
                                if object_clauses:
                                    description_segments.append("The scene features:")

                            if object_clauses:
                                joined_object_clauses = ". ".join(object_clauses)
                                if joined_object_clauses and not joined_object_clauses.endswith("."):
                                    joined_object_clauses += "."
                                description_segments.append(joined_object_clauses)

                        elif not description_segments:
                            return "The image depicts a scene, but specific objects could not be described with confidence or detail."

            # 最終組裝和格式化
            raw_description = ""
            for i, segment in enumerate(filter(None, description_segments)):
                segment = segment.strip()
                if not segment:
                    continue

                if not raw_description:
                    raw_description = segment
                else:
                    if not raw_description.endswith(('.', '!', '?')):
                        raw_description += "."
                    raw_description += " " + (segment[0].upper() + segment[1:] if len(segment) > 1 else segment.upper())

            if raw_description and not raw_description.endswith(('.', '!', '?')):
                raw_description += "."

            if not raw_description or len(raw_description.strip()) < 20:
                if 'confident_objects' in locals() and confident_objects:
                    return "The scene contains several detected objects, but a detailed textual description could not be fully constructed."
                else:
                    return "A general scene is depicted with no objects identified with high confidence."

            return raw_description

        except Exception as e:
            error_msg = f"Error generating dynamic everyday description: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise ObjectDescriptionError(error_msg) from e

    def _format_object_count_description(self, class_name: str, count: int) -> str:
        """
        格式化物件數量描述，提供多樣化的表達方式

        Args:
            class_name: 標準化後的類別名稱
            count: 物件數量

        Returns:
            str: 格式化的數量描述
        """
        try:
            if count <= 0:
                return ""

            # 單數情況
            if count == 1:
                article = "an" if class_name[0].lower() in 'aeiou' else "a"
                return f"{article} {class_name}"

            # 複數情況
            plural_form = self._get_plural_form(class_name)

            # 根據數量選擇不同的表達方式
            if count == 2:
                return f"two {plural_form}"
            elif count == 3:
                return f"three {plural_form}"
            elif count <= 5:
                return f"{count} {plural_form}"
            elif count <= 10:
                return f"several {plural_form}"
            else:
                return f"numerous {plural_form}"

        except Exception as e:
            self.logger.warning(f"Error formatting object count for '{class_name}': {str(e)}")
            return f"{count} {class_name}s" if count > 1 else class_name

    def _get_plural_form(self, word: str) -> str:
        """
        獲取詞彙的複數形式

        Args:
            word: 單數詞彙

        Returns:
            str: 複數形式
        """
        try:
            # 特殊複數形式
            irregular_plurals = {
                'person': 'people',
                'child': 'children',
                'foot': 'feet',
                'tooth': 'teeth',
                'mouse': 'mice',
                'man': 'men',
                'woman': 'women'
            }

            if word.lower() in irregular_plurals:
                return irregular_plurals[word.lower()]

            # 規則複數形式
            if word.endswith(('s', 'sh', 'ch', 'x', 'z')):
                return word + 'es'
            elif word.endswith('y') and word[-2] not in 'aeiou':
                return word[:-1] + 'ies'
            elif word.endswith('f'):
                return word[:-1] + 'ves'
            elif word.endswith('fe'):
                return word[:-2] + 'ves'
            else:
                return word + 's'

        except Exception as e:
            self.logger.warning(f"Error getting plural form for '{word}': {str(e)}")
            return word + 's'

    def _normalize_object_class_name(self, class_name: str) -> str:
        """
        標準化物件類別名稱，確保輸出自然語言格式

        Args:
            class_name: 原始類別名稱

        Returns:
            str: 標準化後的類別名稱
        """
        try:
            if not class_name or not isinstance(class_name, str):
                return "object"

            # 移除可能的技術性前綴或後綴
            import re
            normalized = re.sub(r'^(class_|id_|type_)', '', class_name.lower())
            normalized = re.sub(r'(_class|_id|_type)$', '', normalized)

            # 將下劃線和連字符替換為空格
            normalized = normalized.replace('_', ' ').replace('-', ' ')

            # 移除多餘空格
            normalized = ' '.join(normalized.split())

            # 特殊類別名稱的標準化映射
            class_name_mapping = {
                'traffic light': 'traffic light',
                'stop sign': 'stop sign',
                'fire hydrant': 'fire hydrant',
                'dining table': 'dining table',
                'potted plant': 'potted plant',
                'tv monitor': 'television',
                'cell phone': 'mobile phone',
                'wine glass': 'wine glass',
                'hot dog': 'hot dog',
                'teddy bear': 'teddy bear',
                'hair drier': 'hair dryer',
                'toothbrush': 'toothbrush'
            }

            return class_name_mapping.get(normalized, normalized)

        except Exception as e:
            self.logger.warning(f"Error normalizing class name '{class_name}': {str(e)}")
            return class_name if isinstance(class_name, str) else "object"

    def generate_basic_details(self, scene_type: str, detected_objects: List[Dict]) -> str:
        """
        當模板不可用時生成基本詳細信息

        Args:
            scene_type: 識別的場景類型
            detected_objects: 檢測到的物件列表

        Returns:
            str: 基本場景詳細信息
        """
        try:
            # 處理特定場景類型的自定義邏輯
            if scene_type == "living_room":
                tv_objs = [obj for obj in detected_objects if obj.get("class_id") == 62]  # TV
                sofa_objs = [obj for obj in detected_objects if obj.get("class_id") == 57]  # Sofa

                if tv_objs and sofa_objs:
                    tv_region = tv_objs[0].get("region", "center")
                    sofa_region = sofa_objs[0].get("region", "center")

                    arrangement = f"The TV is in the {tv_region.replace('_', ' ')} of the image, "
                    arrangement += f"while the sofa is in the {sofa_region.replace('_', ' ')}. "

                    return f"{arrangement}This appears to be a space designed for relaxation and entertainment."

            elif scene_type == "bedroom":
                bed_objs = [obj for obj in detected_objects if obj.get("class_id") == 59]  # Bed

                if bed_objs:
                    bed_region = bed_objs[0].get("region", "center")
                    extra_items = []

                    for obj in detected_objects:
                        if obj.get("class_id") == 74:  # Clock
                            extra_items.append("clock")
                        elif obj.get("class_id") == 73:  # Book
                            extra_items.append("book")

                    extras = ""
                    if extra_items:
                        extras = f" There is also a {' and a '.join(extra_items)} visible."

                    return f"The bed is located in the {bed_region.replace('_', ' ')} of the image.{extras}"

            elif scene_type in ["dining_area", "kitchen"]:
                # 計算食物和餐飲相關物品
                food_items = []
                for obj in detected_objects:
                    if obj.get("class_id") in [39, 41, 42, 43, 44, 45]:  # 廚房物品
                        food_items.append(obj.get("class_name", "kitchen item"))

                food_str = ""
                if food_items:
                    unique_items = list(set(food_items))
                    if len(unique_items) <= 3:
                        food_str = f" with {', '.join(unique_items)}"
                    else:
                        food_str = f" with {', '.join(unique_items[:3])} and other items"

                return f"{food_str}."

            elif scene_type == "city_street":
                # 計算人員和車輛
                people_count = len([obj for obj in detected_objects if obj.get("class_id") == 0])
                vehicle_count = len([obj for obj in detected_objects
                                   if obj.get("class_id") in [1, 2, 3, 5, 7]])  # Bicycle, car, motorbike, bus, truck

                traffic_desc = ""
                if people_count > 0 and vehicle_count > 0:
                    traffic_desc = f" with {people_count} {'people' if people_count > 1 else 'person'} and "
                    traffic_desc += f"{vehicle_count} {'vehicles' if vehicle_count > 1 else 'vehicle'}"
                elif people_count > 0:
                    traffic_desc = f" with {people_count} {'people' if people_count > 1 else 'person'}"
                elif vehicle_count > 0:
                    traffic_desc = f" with {vehicle_count} {'vehicles' if vehicle_count > 1 else 'vehicle'}"

                return f"{traffic_desc}."

            elif scene_type == "asian_commercial_street":
                # 尋找關鍵城市元素
                people_count = len([obj for obj in detected_objects if obj.get("class_id") == 0])
                vehicle_count = len([obj for obj in detected_objects if obj.get("class_id") in [1, 2, 3]])

                # 分析行人分布
                people_positions = []
                for obj in detected_objects:
                    if obj.get("class_id") == 0:  # Person
                        people_positions.append(obj.get("normalized_center", (0.5, 0.5)))

                # 檢查人員是否沿線分布（表示步行路徑）
                structured_path = False
                if len(people_positions) >= 3:
                    # 簡化檢查 - 查看多個人員的y坐標是否相似
                    y_coords = [pos[1] for pos in people_positions]
                    y_mean = sum(y_coords) / len(y_coords)
                    y_variance = sum((y - y_mean)**2 for y in y_coords) / len(y_coords)
                    if y_variance < 0.05:  # 低變異數表示線性排列
                        structured_path = True

                street_desc = "A commercial street with "
                if people_count > 0:
                    street_desc += f"{people_count} {'pedestrians' if people_count > 1 else 'pedestrian'}"
                    if vehicle_count > 0:
                        street_desc += f" and {vehicle_count} {'vehicles' if vehicle_count > 1 else 'vehicle'}"
                elif vehicle_count > 0:
                    street_desc += f"{vehicle_count} {'vehicles' if vehicle_count > 1 else 'vehicle'}"
                else:
                    street_desc += "various commercial elements"

                if structured_path:
                    street_desc += ". The pedestrians appear to be following a defined walking path"

                # 添加文化元素
                street_desc += ". The signage and architectural elements suggest an Asian urban setting."

                return street_desc

            # 默認通用描述
            return "The scene contains various elements characteristic of this environment."

        except Exception as e:
            self.logger.warning(f"Error generating basic details for scene_type '{scene_type}': {str(e)}")
            return "The scene contains various elements characteristic of this environment."

    def generate_placeholder_content(self, placeholder: str, detected_objects: List[Dict], scene_type: str) -> str:
        """
        為模板佔位符生成內容

        Args:
            placeholder: 模板佔位符
            detected_objects: 檢測到的物件列表
            scene_type: 場景類型

        Returns:
            str: 生成的佔位符內容
        """
        try:
            # 處理不同類型的佔位符與自定義邏輯
            if placeholder == "furniture":
                # 提取家具物品
                furniture_ids = [56, 57, 58, 59, 60, 61]  # 家具類別ID示例
                furniture_objects = [obj for obj in detected_objects if obj.get("class_id") in furniture_ids]

                if furniture_objects:
                    furniture_names = []
                    for obj in furniture_objects[:3]:
                        raw_name = obj.get("class_name", "furniture")
                        normalized_name = self._normalize_object_class_name(raw_name)
                        furniture_names.append(normalized_name)

                    unique_names = list(set(furniture_names))
                    if len(unique_names) == 1:
                        return unique_names[0]
                    elif len(unique_names) == 2:
                        return f"{unique_names[0]} and {unique_names[1]}"
                    else:
                        return ", ".join(unique_names[:-1]) + f", and {unique_names[-1]}"
                return "various furniture items"

            elif placeholder == "electronics":
                # 提取電子物品
                electronics_ids = [62, 63, 64, 65, 66, 67, 68, 69, 70]  # 電子設備類別ID示例
                electronics_objects = [obj for obj in detected_objects if obj.get("class_id") in electronics_ids]

                if electronics_objects:
                    electronics_names = [obj.get("class_name", "electronic device") for obj in electronics_objects[:3]]
                    return ", ".join(set(electronics_names))
                return "electronic devices"

            elif placeholder == "people_count":
                # 計算人數
                people_count = len([obj for obj in detected_objects if obj.get("class_id") == 0])

                if people_count == 0:
                    return "no people"
                elif people_count == 1:
                    return "one person"
                elif people_count < 5:
                    return f"{people_count} people"
                else:
                    return "several people"

            elif placeholder == "seating":
                # 提取座位物品
                seating_ids = [56, 57]  # chair, sofa
                seating_objects = [obj for obj in detected_objects if obj.get("class_id") in seating_ids]

                if seating_objects:
                    seating_names = [obj.get("class_name", "seating") for obj in seating_objects[:2]]
                    return ", ".join(set(seating_names))
                return "seating arrangements"

            # 默認情況 - 空字符串
            return ""

        except Exception as e:
            self.logger.warning(f"Error generating placeholder content for '{placeholder}': {str(e)}")
            return ""

    def describe_functional_zones(self, functional_zones: Dict) -> str:
        """
        生成場景功能區域的描述，優化處理行人區域、人數統計和物品重複問題

        Args:
            functional_zones: 識別出的功能區域字典

        Returns:
            str: 功能區域描述
        """
        try:
            if not functional_zones:
                return ""

            # 處理不同類型的 functional_zones 參數
            if isinstance(functional_zones, list):
                # 如果是列表，轉換為字典格式
                zones_dict = {}
                for i, zone in enumerate(functional_zones):
                    if isinstance(zone, dict) and 'name' in zone:
                        zone_name = self._normalize_zone_name(zone['name'])
                    else:
                        zone_name = f"functional area {i+1}"
                    zones_dict[zone_name] = zone if isinstance(zone, dict) else {"description": str(zone)}
                functional_zones = zones_dict
            elif not isinstance(functional_zones, dict):
                return ""

            # 標準化所有區域鍵名，移除內部標識符格式
            normalized_zones = {}
            for zone_key, zone_data in functional_zones.items():
                normalized_key = self._normalize_zone_name(zone_key)
                normalized_zones[normalized_key] = zone_data
            functional_zones = normalized_zones

            # 計算場景中的總人數
            total_people_count = 0
            people_by_zone = {}

            # 計算每個區域的人數並累計總人數
            for zone_name, zone_info in functional_zones.items():
                if "objects" in zone_info:
                    zone_people_count = zone_info["objects"].count("person")
                    people_by_zone[zone_name] = zone_people_count
                    total_people_count += zone_people_count

            # 分類區域為行人區域和其他區域
            pedestrian_zones = []
            other_zones = []

            for zone_name, zone_info in functional_zones.items():
                # 檢查是否是行人相關區域
                if any(keyword in zone_name.lower() for keyword in ["pedestrian", "crossing", "people"]):
                    pedestrian_zones.append((zone_name, zone_info))
                else:
                    other_zones.append((zone_name, zone_info))

            # 獲取最重要的行人區域和其他區域
            main_pedestrian_zones = sorted(pedestrian_zones,
                                        key=lambda z: people_by_zone.get(z[0], 0),
                                        reverse=True)[:1]  # 最多1個主要行人區域

            top_other_zones = sorted(other_zones,
                                key=lambda z: len(z[1].get("objects", [])),
                                reverse=True)[:2]  # 最多2個其他區域

            # 合併區域
            top_zones = main_pedestrian_zones + top_other_zones

            if not top_zones:
                return ""

            # 生成匯總描述
            summary = ""
            max_mentioned_people = 0  # 追蹤已經提到的最大人數

            # 如果總人數顯著且還沒在主描述中提到，添加總人數描述
            if total_people_count > 5:
                summary = f"The scene contains a significant number of pedestrians ({total_people_count} people). "
                max_mentioned_people = total_people_count  # 更新已提到的最大人數

            # 處理每個區域的描述，確保人數信息的一致性
            processed_zones = []

            for zone_name, zone_info in top_zones:
                zone_desc = zone_info.get("description", "a functional zone")
                zone_people_count = people_by_zone.get(zone_name, 0)

                # 檢查描述中是否包含人數資訊
                contains_people_info = "with" in zone_desc and ("person" in zone_desc.lower() or "people" in zone_desc.lower())

                # 如果描述包含人數信息，且人數較小（小於已提到的最大人數），則修改描述
                if contains_people_info and zone_people_count < max_mentioned_people:
                    parts = zone_desc.split("with")
                    if len(parts) > 1:
                        # 移除人數部分
                        zone_desc = parts[0].strip() + " area"

                processed_zones.append((zone_name, {"description": zone_desc}))

            # 根據處理後的區域數量生成最終描述
            final_desc = ""

            if len(processed_zones) == 1:
                _, zone_info = processed_zones[0]
                zone_desc = zone_info["description"]
                final_desc = summary + f"The scene includes {zone_desc}."
            elif len(processed_zones) == 2:
                _, zone1_info = processed_zones[0]
                _, zone2_info = processed_zones[1]
                zone1_desc = zone1_info["description"]
                zone2_desc = zone2_info["description"]
                final_desc = summary + f"The scene is divided into two main areas: {zone1_desc} and {zone2_desc}."
            else:
                zones_desc = ["The scene contains multiple functional areas including"]
                zone_descriptions = [z[1]["description"] for z in processed_zones]

                # 格式化最終的多區域描述
                if len(zone_descriptions) == 3:
                    formatted_desc = f"{zone_descriptions[0]}, {zone_descriptions[1]}, and {zone_descriptions[2]}"
                else:
                    formatted_desc = ", ".join(zone_descriptions[:-1]) + f", and {zone_descriptions[-1]}"

                final_desc = summary + f"{zones_desc[0]} {formatted_desc}."

            return self.optimize_object_description(final_desc)

        except Exception as e:
            self.logger.warning(f"Error describing functional zones: {str(e)}")
            return ""

    def _normalize_zone_name(self, zone_name: str) -> str:
        """
        將內部區域鍵名標準化為自然語言描述

        Args:
            zone_name: 原始區域名稱

        Returns:
            str: 標準化後的區域名稱
        """
        try:
            if not zone_name or not isinstance(zone_name, str):
                return "functional area"

            # 移除數字後綴（如 crossing_zone_1 -> crossing_zone）
            import re
            base_name = re.sub(r'_\d+$', '', zone_name)

            # 將下劃線替換為空格
            normalized = base_name.replace('_', ' ')

            # 標準化常見的區域類型名稱
            zone_type_mapping = {
                'crossing zone': 'pedestrian crossing area',
                'vehicle zone': 'vehicle movement area',
                'pedestrian zone': 'pedestrian activity area',
                'traffic zone': 'traffic flow area',
                'waiting zone': 'waiting area',
                'seating zone': 'seating area',
                'dining zone': 'dining area',
                'furniture zone': 'furniture arrangement area',
                'electronics zone': 'electronics area',
                'people zone': 'social activity area',
                'functional area': 'activity area'
            }

            # 檢查是否有對應的標準化名稱
            for pattern, replacement in zone_type_mapping.items():
                if pattern in normalized.lower():
                    return replacement

            # 如果沒有特定映射，使用通用格式
            if 'zone' in normalized.lower():
                normalized = normalized.replace('zone', 'area')
            elif not any(keyword in normalized.lower() for keyword in ['area', 'space', 'region']):
                normalized += ' area'

            return normalized.strip()

        except Exception as e:
            self.logger.warning(f"Error normalizing zone name '{zone_name}': {str(e)}")
            return "activity area"

    def get_configuration(self) -> Dict[str, Any]:
        """
        獲取當前配置參數

        Returns:
            Dict[str, Any]: 配置參數字典
        """
        return {
            "min_prominence_score": self.min_prominence_score,
            "max_categories_to_return": self.max_categories_to_return,
            "max_total_objects": self.max_total_objects,
            "confidence_threshold_for_description": self.confidence_threshold_for_description
        }

    def update_configuration(self, **kwargs):
        """
        更新配置參數

        Args:
            **kwargs: 要更新的配置參數
        """
        try:
            for key, value in kwargs.items():
                if hasattr(self, key):
                    old_value = getattr(self, key)
                    setattr(self, key, value)
                    self.logger.info(f"Updated {key}: {old_value} -> {value}")
                else:
                    self.logger.warning(f"Unknown configuration parameter: {key}")

        except Exception as e:
            self.logger.error(f"Error updating configuration: {str(e)}")
            raise ObjectDescriptionError(f"Failed to update configuration: {str(e)}") from e

Writing object_description_generator.py


In [ ]:
# %%writefile cultural_context_analyzer.py
import logging
import traceback
import random
from typing import Dict, List, Optional, Any

# from cultural_templates import CULTURAL_TEMPLATES

class CulturalContextError(Exception):
    """文化語境分析過程中的自定義異常"""
    pass


class CulturalContextAnalyzer:
    """
    文化語境分析器 - 檢測場景中的文化特徵並生成相關的描述

    該類別負責識別場景中的文化語境線索，包括建築風格、標誌特徵
    和物件配置，然後生成適當的文化描述元素。
    """

    def __init__(self, cultural_templates: Optional[Dict] = None):
        """
        初始化文化語境分析器

        Args:
            cultural_templates: 可選的自定義文化模板，如果提供則會與默認模板合併
        """
        self.logger = logging.getLogger(self.__class__.__name__)

        try:
            # 載入文化模板
            self.cultural_templates = self._load_cultural_templates()

            # 如果提供了自定義模板，進行合併
            if cultural_templates:
                self._merge_custom_templates(cultural_templates)

            # 初始化場景類型到文化語境的映射
            self.scene_cultural_mapping = self._initialize_scene_cultural_mapping()

            self.logger.info("CulturalContextAnalyzer initialized with %d cultural templates",
                           len(self.cultural_templates))

        except Exception as e:
            error_msg = f"Failed to initialize CulturalContextAnalyzer: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise CulturalContextError(error_msg) from e

    def _load_cultural_templates(self) -> Dict:
        """
        載入文化模板

        Returns:
            Dict: 文化模板字典

        Raises:
            CulturalContextError: 當模板載入失敗時
        """
        try:
            self.logger.debug("Loading cultural templates")

            # 從配置模組載入文化模板
            templates = CULTURAL_TEMPLATES.copy()

            # 確保模板結構正確
            self._validate_cultural_templates(templates)

            # 如果沒有載入到模板，使用默認模板
            if not templates:
                self.logger.warning("No cultural templates loaded, using defaults")
                templates = self._get_default_cultural_templates()

            self.logger.debug("Successfully loaded %d cultural template categories", len(templates))
            return templates

        except ImportError as e:
            self.logger.warning(f"Failed to import cultural templates: {str(e)}, using defaults")
            return self._get_default_cultural_templates()
        except Exception as e:
            error_msg = f"Error loading cultural templates: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise CulturalContextError(error_msg) from e

    def _get_default_cultural_templates(self) -> Dict:
        """
        獲取默認文化模板

        Returns:
            Dict: 默認文化模板字典
        """
        return {
            "asian": {
                "elements": [
                    "traditional architectural elements",
                    "cultural signage",
                    "Asian design features",
                    "oriental decorative patterns",
                    "traditional building materials",
                    "characteristic roofline styles",
                    "cultural landscaping elements"
                ],
                "description": "The scene displays distinctive Asian cultural characteristics with {elements}."
            },
            "european": {
                "elements": [
                    "classical architecture",
                    "European design elements",
                    "historic features",
                    "traditional stonework",
                    "characteristic window styles",
                    "ornamental facades",
                    "heritage building elements"
                ],
                "description": "The scene exhibits European architectural and cultural elements including {elements}."
            },
            "american": {
                "elements": [
                    "modern architectural styles",
                    "contemporary design features",
                    "commercial signage",
                    "urban planning elements",
                    "standardized building designs"
                ],
                "description": "The scene shows American urban characteristics featuring {elements}."
            },
            "mediterranean": {
                "elements": [
                    "coastal architectural styles",
                    "warm climate adaptations",
                    "traditional building colors",
                    "characteristic outdoor spaces"
                ],
                "description": "The scene reflects Mediterranean cultural influences with {elements}."
            }
        }

    def _validate_cultural_templates(self, templates: Dict):
        """
        驗證文化模板結構

        Args:
            templates: 要驗證的模板字典

        Raises:
            CulturalContextError: 當模板結構無效時
        """
        try:
            for culture, template_data in templates.items():
                if not isinstance(template_data, dict):
                    self.logger.warning(f"Invalid cultural template structure for '{culture}': not a dictionary")
                    continue

                required_keys = ["elements", "description"]
                for key in required_keys:
                    if key not in template_data:
                        self.logger.warning(f"Missing required key '{key}' in cultural template '{culture}'")

                # 驗證元素列表
                if "elements" in template_data:
                    if not isinstance(template_data["elements"], list):
                        self.logger.warning(f"Cultural template '{culture}' elements should be a list")
                    elif not template_data["elements"]:
                        self.logger.warning(f"Cultural template '{culture}' has empty elements list")

                # 驗證描述模板
                if "description" in template_data:
                    if not isinstance(template_data["description"], str):
                        self.logger.warning(f"Cultural template '{culture}' description should be a string")
                    elif "{elements}" not in template_data["description"]:
                        self.logger.warning(f"Cultural template '{culture}' description missing {{elements}} placeholder")

            self.logger.debug("Cultural templates validation completed")

        except Exception as e:
            self.logger.warning(f"Error validating cultural templates: {str(e)}")

    def _merge_custom_templates(self, custom_templates: Dict):
        """
        合併自定義文化模板

        Args:
            custom_templates: 自定義模板字典
        """
        try:
            for culture, template_data in custom_templates.items():
                if culture in self.cultural_templates:
                    # 合併現有文化的模板
                    if isinstance(self.cultural_templates[culture], dict) and isinstance(template_data, dict):
                        # 合併元素列表
                        if "elements" in template_data and "elements" in self.cultural_templates[culture]:
                            existing_elements = self.cultural_templates[culture]["elements"]
                            new_elements = template_data["elements"]
                            if isinstance(existing_elements, list) and isinstance(new_elements, list):
                                self.cultural_templates[culture]["elements"] = existing_elements + new_elements

                        # 更新其他鍵值
                        for key, value in template_data.items():
                            if key != "elements":
                                self.cultural_templates[culture][key] = value
                    else:
                        self.cultural_templates[culture] = template_data
                else:
                    # 添加新的文化模板
                    self.cultural_templates[culture] = template_data

                self.logger.debug(f"Merged custom template for culture: {culture}")

            self.logger.info("Successfully merged custom cultural templates")

        except Exception as e:
            self.logger.warning(f"Error merging custom cultural templates: {str(e)}")

    def _initialize_scene_cultural_mapping(self) -> Dict[str, str]:
        """
        初始化場景類型到文化語境的display

        Returns:
            Dict[str, str]: 場景類型到文化語境的映射字典
        """
        return {
            "asian_commercial_street": "asian",
            "asian_night_market": "asian",
            "asian_temple_area": "asian",
            "chinese_restaurant": "asian",
            "japanese_restaurant": "asian",
            "korean_restaurant": "asian",
            "european_plaza": "european",
            "european_cafe": "european",
            "mediterranean_restaurant": "mediterranean",
            "american_diner": "american",
            "american_fast_food": "american"
        }

    def detect_cultural_context(self, scene_type: str, detected_objects: List[Dict]) -> Optional[str]:
        """
        檢測場景的文化語境

        Args:
            scene_type: 識別的場景類型
            detected_objects: 檢測到的物件列表

        Returns:
            Optional[str]: 檢測到的文化語境（asian, european等）或None
        """
        try:
            self.logger.debug(f"Detecting cultural context for scene_type: {scene_type}")

            # 檢查場景類型是否直接指示文化語境
            if scene_type in self.scene_cultural_mapping:
                cultural_context = self.scene_cultural_mapping[scene_type]
                self.logger.debug(f"Direct cultural mapping found: {scene_type} -> {cultural_context}")
                return cultural_context

            # 基於場景類型名稱的模式匹配
            cultural_context = self._detect_from_scene_name_patterns(scene_type)
            if cultural_context:
                self.logger.debug(f"Cultural context detected from name patterns: {cultural_context}")
                return cultural_context

            # 基於檢測物件的文化特徵分析
            cultural_context = self._detect_from_object_analysis(detected_objects)
            if cultural_context:
                self.logger.debug(f"Cultural context detected from object analysis: {cultural_context}")
                return cultural_context

            # 沒有檢測到特定文化語境
            self.logger.debug("No specific cultural context detected")
            return None

        except Exception as e:
            self.logger.warning(f"Error detecting cultural context: {str(e)}")
            return None

    def _detect_from_scene_name_patterns(self, scene_type: str) -> Optional[str]:
        """
        基於場景類型名稱模式檢測文化語境

        Args:
            scene_type: 場景類型名稱

        Returns:
            Optional[str]: 檢測到的文化語境或None
        """
        try:
            scene_lower = scene_type.lower()

            # Asia
            asian_keywords = [
                "asian", "chinese", "japanese", "korean", "thai", "vietnamese",
                "temple", "pagoda", "zen", "oriental", "bamboo", "tatami"
            ]

            # Europe
            european_keywords = [
                "european", "french", "italian", "spanish", "german", "british",
                "plaza", "piazza", "cathedral", "gothic", "baroque", "renaissance",
                "cafe", "bistro", "pub"
            ]

            # 地中海文化
            mediterranean_keywords = [
                "mediterranean", "greek", "turkish", "coastal", "terrace",
                "villa", "courtyard"
            ]

            # 美國
            american_keywords = [
                "american", "diner", "fast_food", "mall", "suburban",
                "downtown", "strip_mall"
            ]

            # 檢查各文化的key word
            if any(keyword in scene_lower for keyword in asian_keywords):
                return "asian"
            elif any(keyword in scene_lower for keyword in european_keywords):
                return "european"
            elif any(keyword in scene_lower for keyword in mediterranean_keywords):
                return "mediterranean"
            elif any(keyword in scene_lower for keyword in american_keywords):
                return "american"

            return None

        except Exception as e:
            self.logger.warning(f"Error detecting cultural context from scene name patterns: {str(e)}")
            return None

    def _detect_from_object_analysis(self, detected_objects: List[Dict]) -> Optional[str]:
        """
        基於檢測物件分析文化特徵

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            Optional[str]: 檢測到的文化語境或None
        """
        try:
            if not detected_objects:
                return None

            # 統計文化相關物件
            cultural_indicators = {
                "asian": 0,
                "european": 0,
                "american": 0,
                "mediterranean": 0
            }

            for obj in detected_objects:
                class_name = obj.get("class_name", "").lower()

                # Asia 特色
                if any(indicator in class_name for indicator in [
                    "lantern", "chopsticks", "rice", "noodles", "tea",
                    "bamboo", "pagoda", "shrine", "torii"
                ]):
                    cultural_indicators["asian"] += 1

                # 歐洲的特色
                elif any(indicator in class_name for indicator in [
                    "wine", "cheese", "bread", "fountain", "column",
                    "statue", "cathedral", "clock_tower"
                ]):
                    cultural_indicators["european"] += 1

                # 地中海的特色
                elif any(indicator in class_name for indicator in [
                    "olive", "terracotta", "pergola", "villa",
                    "coastal", "maritime"
                ]):
                    cultural_indicators["mediterranean"] += 1

                # 美國的特色
                elif any(indicator in class_name for indicator in [
                    "burger", "pizza", "hotdog", "soda",
                    "drive_through", "parking_lot"
                ]):
                    cultural_indicators["american"] += 1

            # 找出得分最高的文化語境
            if max(cultural_indicators.values()) > 0:
                dominant_culture = max(cultural_indicators.items(), key=lambda x: x[1])[0]
                max_score = cultural_indicators[dominant_culture]

                # 需要至少2個指標物件才算有效檢測
                if max_score >= 2:
                    return dominant_culture

            return None

        except Exception as e:
            self.logger.warning(f"Error detecting cultural context from object analysis: {str(e)}")
            return None

    def generate_cultural_elements(self, cultural_context: str) -> str:
        """
        為檢測到的文化語境生成描述元素

        Args:
            cultural_context: 檢測到的文化語境

        Returns:
            str: 文化元素描述

        Raises:
            CulturalContextError: 當文化元素生成失敗時
        """
        try:
            if not cultural_context:
                return ""

            self.logger.debug(f"Generating cultural elements for context: {cultural_context}")

            # 獲取該文化語境的模板
            if cultural_context not in self.cultural_templates:
                self.logger.warning(f"No template found for cultural context: {cultural_context}")
                return ""

            template = self.cultural_templates[cultural_context]
            elements = template.get("elements", [])

            if not elements:
                self.logger.warning(f"No elements found for cultural context: {cultural_context}")
                return ""

            # 選擇1-2個隨機元素
            num_elements = min(len(elements), random.randint(1, 2))
            selected_elements = random.sample(elements, num_elements)

            # 格式化元素列表
            if len(selected_elements) == 1:
                elements_text = selected_elements[0]
            else:
                elements_text = " and ".join(selected_elements)

            # 填充模板
            description_template = template.get("description", "")
            if not description_template:
                return f"The scene displays {cultural_context} cultural characteristics."

            # 替換佔位符
            cultural_description = description_template.format(elements=elements_text)

            self.logger.debug(f"Generated cultural description: {cultural_description}")
            return cultural_description

        except Exception as e:
            error_msg = f"Error generating cultural elements for context '{cultural_context}': {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise CulturalContextError(error_msg) from e

    def get_cultural_template(self, cultural_context: str) -> Dict[str, Any]:
        """
        獲取指定文化語境的模板

        Args:
            cultural_context: 文化語境名稱

        Returns:
            Dict[str, Any]: 文化模板字典
        """
        try:
            if cultural_context in self.cultural_templates:
                return self.cultural_templates[cultural_context].copy()

            # 返回備用模板
            self.logger.warning(f"Cultural template not found for '{cultural_context}', using fallback")
            return {
                "elements": ["various cultural elements"],
                "description": f"The scene displays {cultural_context} cultural characteristics."
            }

        except Exception as e:
            self.logger.warning(f"Error getting cultural template for '{cultural_context}': {str(e)}")
            return {
                "elements": ["various elements"],
                "description": "The scene displays cultural characteristics."
            }

    def add_cultural_template(self, cultural_context: str, template: Dict[str, Any]):
        """
        添加或更新文化模板

        Args:
            cultural_context: 文化語境名稱
            template: 文化模板字典

        Raises:
            CulturalContextError: 當模板格式無效時
        """
        try:
            # 驗證模板格式
            if not isinstance(template, dict):
                raise CulturalContextError("Template must be a dictionary")

            required_keys = ["elements", "description"]
            for key in required_keys:
                if key not in template:
                    raise CulturalContextError(f"Template missing required key: {key}")

            if not isinstance(template["elements"], list):
                raise CulturalContextError("Template 'elements' must be a list")

            if not isinstance(template["description"], str):
                raise CulturalContextError("Template 'description' must be a string")

            # 添加模板
            self.cultural_templates[cultural_context] = template.copy()

            self.logger.info(f"Added cultural template for context: {cultural_context}")

        except CulturalContextError:
            raise
        except Exception as e:
            error_msg = f"Error adding cultural template for '{cultural_context}': {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise CulturalContextError(error_msg) from e

    def get_supported_cultures(self) -> List[str]:
        """
        獲取所有支援的文化語境列表

        Returns:
            List[str]: 支援的文化語境名稱列表
        """
        return list(self.cultural_templates.keys())

    def has_cultural_context(self, cultural_context: str) -> bool:
        """
        檢查是否支援指定的文化語境

        Args:
            cultural_context: 文化語境名稱

        Returns:
            bool: 是否支援該文化語境
        """
        return cultural_context in self.cultural_templates

    def analyze_cultural_diversity(self, detected_objects: List[Dict]) -> Dict[str, int]:
        """
        分析場景中的文化多樣性

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            Dict[str, int]: 各文化語境的指標物件計數
        """
        try:
            cultural_scores = {culture: 0 for culture in self.cultural_templates.keys()}

            if not detected_objects:
                return cultural_scores

            for obj in detected_objects:
                class_name = obj.get("class_name", "").lower()

                # 為每個文化語境計算指標分數
                for culture in cultural_scores:
                    if self._is_cultural_indicator(class_name, culture):
                        cultural_scores[culture] += 1

            self.logger.debug(f"Cultural diversity analysis: {cultural_scores}")
            return cultural_scores

        except Exception as e:
            self.logger.warning(f"Error analyzing cultural diversity: {str(e)}")
            return {culture: 0 for culture in self.cultural_templates.keys()}

    def _is_cultural_indicator(self, object_name: str, culture: str) -> bool:
        """
        檢查物件名稱是否為特定文化的指標

        Args:
            object_name: 物件名稱
            culture: 文化語境

        Returns:
            bool: 是否為該文化的指標物件
        """
        try:
            cultural_keywords = {
                "asian": [
                    "lantern", "chopsticks", "rice", "noodles", "tea",
                    "bamboo", "pagoda", "shrine", "torii", "kimono",
                    "sushi", "ramen", "dim_sum"
                ],
                "european": [
                    "wine", "cheese", "bread", "fountain", "column",
                    "statue", "cathedral", "clock_tower", "baguette",
                    "croissant", "espresso", "gelato"
                ],
                "mediterranean": [
                    "olive", "terracotta", "pergola", "villa",
                    "coastal", "maritime", "cypress", "vineyard"
                ],
                "american": [
                    "burger", "pizza", "hotdog", "soda",
                    "drive_through", "parking_lot", "diner",
                    "strip_mall", "suburb"
                ]
            }

            if culture not in cultural_keywords:
                return False

            keywords = cultural_keywords[culture]
            return any(keyword in object_name for keyword in keywords)

        except Exception as e:
            self.logger.warning(f"Error checking cultural indicator for {object_name}, {culture}: {str(e)}")
            return False

    def get_template_summary(self) -> Dict[str, Dict[str, Any]]:
        """
        獲取所有文化模板的摘要信息

        Returns:
            Dict[str, Dict[str, Any]]: 文化模板摘要
        """
        try:
            summary = {}

            for culture, template in self.cultural_templates.items():
                summary[culture] = {
                    "element_count": len(template.get("elements", [])),
                    "has_description": bool(template.get("description", "")),
                    "sample_elements": template.get("elements", [])[:3]  # 前3個元素作為樣本
                }

            return summary

        except Exception as e:
            self.logger.warning(f"Error generating template summary: {str(e)}")
            return {}

Writing cultural_context_analyzer.py


In [ ]:
# %%writefile text_formatter.py
import logging
import traceback
import re
from typing import Dict, List, Optional

# from landmark_data import ALL_LANDMARKS

class TextFormattingError(Exception):
    """文本格式化過程中的自定義異常"""
    pass


class TextFormatter:
    """
    文本格式化器 - 負責文本拼接、格式化和最終輸出優化

    該類別處理所有與文本格式化相關的邏輯，包括智能文本拼接、
    標點符號處理、大小寫規範化以及地標引用的過濾功能。
    """

    def __init__(self):
        """
        初始化文本格式化器
        """
        self.logger = logging.getLogger(self.__class__.__name__)

        try:
            # 載入地標數據用於引用過濾
            self.landmark_data = self._load_landmark_data()

            self.logger.info("TextFormatter initialized successfully")

        except Exception as e:
            error_msg = f"Failed to initialize TextFormatter: {str(e)}"
            self.logger.error(f"{error_msg}\n{traceback.format_exc()}")
            raise TextFormattingError(error_msg) from e

    def _load_landmark_data(self) -> Dict:
        """
        載入地標數據

        Returns:
            Dict: 地標數據字典
        """
        try:
            return ALL_LANDMARKS
        except ImportError:
            self.logger.warning("Failed to import landmark data, landmark filtering will be disabled")
            return {}
        except Exception as e:
            self.logger.warning(f"Error loading landmark data: {str(e)}")
            return {}

    def smart_append(self, current_text: str, new_fragment: str) -> str:
        """
        將新文本片段附加到現有文本，處理標點符號和大小寫

        Args:
            current_text: 要加到的現有文本
            new_fragment: 要加的新文本片段

        Returns:
            str: 合併後的文本，具有適當的格式化
        """
        try:
            # 處理空值情況
            if not new_fragment:
                return current_text

            if not current_text:
                # 確保第一個字符大寫
                return new_fragment[0].upper() + new_fragment[1:] if new_fragment else ""

            # 清理現有文本
            current_text = current_text.rstrip()

            # 檢查結尾標點符號
            ends_with_sentence = current_text.endswith(('.', '!', '?'))
            ends_with_comma = current_text.endswith(',')

            # 特別處理 "A xxx A yyy" 模式
            if (current_text.startswith("A ") or current_text.startswith("An ")) and \
               (new_fragment.startswith("A ") or new_fragment.startswith("An ")):
                return current_text + ". " + new_fragment

            # 檢查新片段是否包含地標名稱（通常為專有名詞）
            has_landmark_name = any(word[0].isupper() for word in new_fragment.split()
                                  if len(word) > 2 and not word.startswith(("A ", "An ", "The ")))

            # 決定如何連接文本
            if ends_with_sentence:
                # 句子後，以大寫開始並添加適當間距
                joined_text = current_text + " " + (new_fragment[0].upper() + new_fragment[1:])
            elif ends_with_comma:
                # 逗號後，要保持流暢性，除非是專有名詞或特殊情況
                if new_fragment.startswith(('I ', 'I\'', 'A ', 'An ', 'The ')) or new_fragment[0].isupper() or has_landmark_name:
                    joined_text = current_text + " " + new_fragment
                else:
                    joined_text = current_text + " " + new_fragment[0].lower() + new_fragment[1:]
            elif "scene is" in new_fragment.lower() or "scene includes" in new_fragment.lower():
                # 加關於場景的新句子時，使用句號
                joined_text = current_text + ". " + new_fragment
            else:
                # 其他情況，根據內容決定
                if self._is_related_phrases(current_text, new_fragment):
                    if new_fragment.startswith(('I ', 'I\'', 'A ', 'An ', 'The ')) or new_fragment[0].isupper() or has_landmark_name:
                        joined_text = current_text + ", " + new_fragment
                    else:
                        joined_text = current_text + ", " + new_fragment[0].lower() + new_fragment[1:]
                else:
                    # 對不相關的短語使用句號
                    joined_text = current_text + ". " + (new_fragment[0].upper() + new_fragment[1:])

            return joined_text

        except Exception as e:
            self.logger.warning(f"Error in smart_append: {str(e)}")
            # 備用簡單拼接
            return f"{current_text} {new_fragment}" if current_text else new_fragment

    def _is_related_phrases(self, text1: str, text2: str) -> bool:
        """
        判斷兩個短語是否相關，應該用逗號

        Args:
            text1: 第一個文本片段
            text2: 要加的第二個文本片段

        Returns:
            bool: 短語是否相關
        """
        try:
            # 檢查兩個短語是否都以 "A" 或 "An" 開始 - 這些是獨立的描述
            if (text1.startswith("A ") or text1.startswith("An ")) and \
               (text2.startswith("A ") or text2.startswith("An ")):
                return False  # 這些是獨立的描述，不是相關短語

            # 檢查第二個短語是否以連接詞開始
            connecting_words = ["which", "where", "who", "whom", "whose", "with", "without",
                              "this", "these", "that", "those", "and", "or", "but"]

            first_word = text2.split()[0].lower() if text2 else ""
            if first_word in connecting_words:
                return True

            # 檢查第一個短語是否以暗示連續性的內容結尾
            ending_patterns = ["such as", "including", "like", "especially", "particularly",
                             "for example", "for instance", "namely", "specifically"]

            for pattern in ending_patterns:
                if text1.lower().endswith(pattern):
                    return True

            # 檢查兩個短語是否都關於場景
            if "scene" in text1.lower() and "scene" in text2.lower():
                return False  # 關於場景的獨立陳述應該是分開的句子

            return False

        except Exception as e:
            self.logger.warning(f"Error checking phrase relationship: {str(e)}")
            return False

    def format_final_description(self, text: str) -> str:
        """
        格式化最終描述文本，確保正確的標點符號、大小寫和間距

        Args:
            text: 要格式化的文本

        Returns:
            str: 格式化後的文本
        """
        try:
            if not text or not text.strip():
                return ""

            # 首先修剪前導/尾隨空白
            text = text.strip()

            # 1. 處理連續的 "A/An" 段落（可能將它們分成句子）
            text = re.sub(r'(A\s+[^.!?]+?[\w\.])\s+(A\s+)', r'\1. \2', text, flags=re.IGNORECASE)
            text = re.sub(r'(An\s+[^.!?]+?[\w\.])\s+(An?\s+)', r'\1. \2', text, flags=re.IGNORECASE)

            # 2. 確保整個文本的第一個字符大寫
            if text:
                text = text[0].upper() + text[1:]

            # 3. 規範化空白：多個空格變為一個
            text = re.sub(r'\s{2,}', ' ', text)

            # 4. 句子結尾標點符號後大寫
            def capitalize_after_punctuation(match):
                return match.group(1) + match.group(2).upper()
            text = re.sub(r'([.!?]\s+)([a-z])', capitalize_after_punctuation, text)

            # 5. 處理逗號後的大小寫
            def fix_capitalization_after_comma(match):
                leading_comma_space = match.group(1)  # (,\s+)
                word_after_comma = match.group(2)     # ([A-Z][a-zA-Z]*)

                proper_nouns_exceptions = ["I", "I'm", "I've", "I'd", "I'll",
                                         "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday",
                                         "January", "February", "March", "April", "May", "June", "July",
                                         "August", "September", "October", "November", "December"]

                if word_after_comma in proper_nouns_exceptions:
                    return match.group(0)

                # 如果詞看起來像專有名詞（已經大寫且不是常用詞），保持不變
                if len(word_after_comma) > 2 and word_after_comma[0].isupper() and word_after_comma.lower() not in ["this", "that", "these", "those", "they", "their", "then", "thus"]:
                    return match.group(0)  # 如果看起來已經是專有名詞則保持不變

                return leading_comma_space + word_after_comma[0].lower() + word_after_comma[1:]
            text = re.sub(r'(,\s+)([A-Z][a-zA-Z\'\-]+)', fix_capitalization_after_comma, text)

            # 6. 修正標點符號周圍的間距
            text = re.sub(r'\s*([.,;:!?])\s*', r'\1 ', text)  # 確保標點符號後有一個空格，前面沒有
            text = text.replace(' .', '.').replace(' ,', ',')  # 清理標點符號前可能的空格

            # 7. 合併多個句子結尾標點符號
            text = re.sub(r'[.!?]{2,}', '.', text)  # 將多個轉換為單個句號
            text = re.sub(r',+', ',', text)  # 多個逗號變為一個

            # 8. 確保文本以單個句子結尾標點符號結尾
            text = text.strip()  # 檢查最後一個字符前移除尾隨空白
            if text and not text[-1] in '.!?':
                text += '.'

            # 9. 處理空的佔位符和前導標點符號
            text = re.sub(r'\bIn\s*,\s*', 'In this scene, ', text)  # 修復 "In , " 問題
            text = re.sub(r'\s*,\s*([A-Z])', r'. \1', text)  # 修復逗號後直接跟大寫字母的問題
            text = re.sub(r'^[.,;:!?\s]+', '', text)  # 移除前導標點符號

            # 10. 第一個字母大寫的最終檢查
            if text:
                text = text[0].upper() + text[1:]

            # 11. 移除最終標點符號前的空格（如果規則7意外添加）
            text = re.sub(r'\s+([.!?])$', r'\1', text)

            return text.strip()  # 最終修剪

        except Exception as e:
            self.logger.warning(f"Error formatting final description: {str(e)}")
            # 備用基本格式化
            if text:
                text = text.strip()
                if text and not text.endswith(('.', '!', '?')):
                    text += '.'
                if text:
                    text = text[0].upper() + text[1:]
                return text
            return ""

    def filter_landmark_references(self, text: str, enable_landmark: bool = True) -> str:
        """
        動態過濾文本中的地標引用

        Args:
            text: 需要過濾的文本
            enable_landmark: 是否啟用地標功能

        Returns:
            str: 過濾後的文本
        """
        try:
            if enable_landmark or not text:
                return text

            # 動態收集所有地標名稱和位置
            landmark_names = []
            locations = []

            for landmark_id, info in self.landmark_data.items():
                # 收集地標名稱及其別名
                landmark_names.append(info["name"])
                landmark_names.extend(info.get("aliases", []))

                # 收集地理位置
                if "location" in info:
                    location = info["location"]
                    locations.append(location)

                    # 處理分離的城市和國家名稱
                    parts = location.split(",")
                    if len(parts) >= 1:
                        locations.append(parts[0].strip())
                    if len(parts) >= 2:
                        locations.append(parts[1].strip())

            # 替換所有地標名稱
            for name in landmark_names:
                if name and len(name) > 2:  # 避免過短的名稱
                    text = re.sub(r'\b' + re.escape(name) + r'\b', "tall structure", text, flags=re.IGNORECASE)

            # 動態替換所有位置引用
            for location in locations:
                if location and len(location) > 2:
                    # 替換常見位置表述模式
                    text = re.sub(r'in ' + re.escape(location), "in the urban area", text, flags=re.IGNORECASE)
                    text = re.sub(r'of ' + re.escape(location), "of the urban area", text, flags=re.IGNORECASE)
                    text = re.sub(r'\b' + re.escape(location) + r'\b', "the urban area", text, flags=re.IGNORECASE)

            # 通用地標描述模式替換
            landmark_patterns = [
                (r'a (tourist|popular|famous) landmark', r'an urban structure'),
                (r'an iconic structure in ([A-Z][a-zA-Z\s,]+)', r'an urban structure in the area'),
                (r'a famous (monument|tower|landmark) in ([A-Z][a-zA-Z\s,]+)', r'an urban structure in the area'),
                (r'(centered|built|located|positioned) around the ([A-Z][a-zA-Z\s]+? (Tower|Monument|Landmark))', r'located in this area'),
                (r'(sightseeing|guided tours|cultural tourism) (at|around|near) (this landmark|the [A-Z][a-zA-Z\s]+)', r'\1 in this area'),
                (r'this (famous|iconic|historic|well-known) (landmark|monument|tower|structure)', r'this urban structure'),
                (r'([A-Z][a-zA-Z\s]+) Tower', r'tall structure'),
                (r'a (tower|structure) in ([A-Z][a-zA-Z\s,]+)', r'a \1 in the area'),
                (r'landmark scene', r'urban scene'),
                (r'tourist destination', r'urban area'),
                (r'tourist attraction', r'urban area')
            ]

            for pattern, replacement in landmark_patterns:
                text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

            return text

        except Exception as e:
            self.logger.warning(f"Error filtering landmark references: {str(e)}")
            return text

    def optimize_text_flow(self, text: str) -> str:
        """
        優化文本流暢性，減少重複和改善可讀性

        Args:
            text: 要優化的文本

        Returns:
            str: 優化後的文本
        """
        try:
            if not text:
                return text

            # 移除重複的短語
            text = self._remove_duplicate_phrases(text)

            # 優化連接詞使用
            text = self._optimize_connectors(text)

            # 平衡句子長度
            text = self._balance_sentence_length(text)

            return text

        except Exception as e:
            self.logger.warning(f"Error optimizing text flow: {str(e)}")
            return text

    def _remove_duplicate_phrases(self, text: str) -> str:
        """
        移除文本中的重複短語

        Args:
            text: 輸入文本

        Returns:
            str: 移除重複後的文本
        """
        try:
            # 分割成句子
            sentences = re.split(r'[.!?]+', text)
            unique_sentences = []
            seen_content = set()

            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue

                # 規範化以進行比較（移除額外空白和標點符號）
                normalized = re.sub(r'\s+', ' ', sentence.lower().strip())

                # 檢查是否實質相似
                is_duplicate = False
                for seen in seen_content:
                    if self._sentences_similar(normalized, seen):
                        is_duplicate = True
                        break

                if not is_duplicate:
                    unique_sentences.append(sentence)
                    seen_content.add(normalized)

            return '. '.join(unique_sentences) + '.' if unique_sentences else ""

        except Exception as e:
            self.logger.warning(f"Error removing duplicate phrases: {str(e)}")
            return text

    def _sentences_similar(self, sent1: str, sent2: str) -> bool:
        """
        檢查兩個句子是否相似

        Args:
            sent1: 第一個句子
            sent2: 第二個句子

        Returns:
            bool: 句子是否相似
        """
        try:
            # 簡單的相似性檢查：如果80%的詞彙重疊
            words1 = set(sent1.split())
            words2 = set(sent2.split())

            if not words1 or not words2:
                return False

            intersection = len(words1 & words2)
            union = len(words1 | words2)

            similarity = intersection / union if union > 0 else 0
            return similarity > 0.8

        except Exception as e:
            self.logger.warning(f"Error checking sentence similarity: {str(e)}")
            return False

    def _optimize_connectors(self, text: str) -> str:
        """
        優化連接詞的使用

        Args:
            text: 輸入文本

        Returns:
            str: 優化連接詞後的文本
        """
        try:
            # 替換重複的連接詞
            text = re.sub(r'\band\s+and\b', 'and', text, flags=re.IGNORECASE)
            text = re.sub(r'\bwith\s+with\b', 'with', text, flags=re.IGNORECASE)

            # 改善過度使用 "and" 的情況
            text = re.sub(r'(\w+),\s+and\s+(\w+),\s+and\s+(\w+)', r'\1, \2, and \3', text)

            return text

        except Exception as e:
            self.logger.warning(f"Error optimizing connectors: {str(e)}")
            return text

    def _balance_sentence_length(self, text: str) -> str:
        """
        平衡句子長度，分割過長的句子

        Args:
            text: 輸入文本

        Returns:
            str: 平衡句子長度後的文本
        """
        try:
            sentences = re.split(r'([.!?]+)', text)
            balanced_text = ""

            for i in range(0, len(sentences), 2):
                if i + 1 < len(sentences):
                    sentence = sentences[i]
                    punctuation = sentences[i + 1]

                    # 如果句子太長（超過150個字符），嘗試在適當位置分割
                    if len(sentence) > 150:
                        # 在逗號或連接詞處分割
                        split_points = [m.start() for m in re.finditer(r',\s+(?:and|but|or|while|when|where)', sentence)]
                        if split_points:
                            mid_point = split_points[len(split_points) // 2]
                            first_part = sentence[:mid_point].strip()
                            second_part = sentence[mid_point + 1:].strip()
                            if second_part and not second_part[0].isupper():
                                second_part = second_part[0].upper() + second_part[1:]
                            balanced_text += first_part + ". " + second_part + punctuation + " "
                        else:
                            balanced_text += sentence + punctuation + " "
                    else:
                        balanced_text += sentence + punctuation + " "

            return balanced_text.strip()

        except Exception as e:
            self.logger.warning(f"Error balancing sentence length: {str(e)}")
            return text

    def validate_text_quality(self, text: str) -> Dict[str, bool]:
        """
        驗證文本質量

        Args:
            text: 要驗證的文本

        Returns:
            Dict[str, bool]: 質量檢查結果
        """
        try:
            quality_checks = {
                "has_content": bool(text and text.strip()),
                "proper_capitalization": bool(text and text[0].isupper()) if text else False,
                "ends_with_punctuation": bool(text and text.strip()[-1] in '.!?') if text else False,
                "no_double_spaces": "  " not in text if text else True,
                "no_leading_punctuation": not bool(re.match(r'^[.,;:!?]', text.strip())) if text else True,
                "reasonable_length": 20 <= len(text) <= 1000 if text else False
            }

            return quality_checks

        except Exception as e:
            self.logger.warning(f"Error validating text quality: {str(e)}")
            return {"error": True}

    def get_text_statistics(self, text: str) -> Dict[str, int]:
        """
        獲取文本統計信息

        Args:
            text: 要分析的文本

        Returns:
            Dict[str, int]: 文本統計信息
        """
        try:
            if not text:
                return {"characters": 0, "words": 0, "sentences": 0}

            characters = len(text)
            words = len(text.split())
            sentences = len(re.findall(r'[.!?]+', text))

            return {
                "characters": characters,
                "words": words,
                "sentences": sentences
            }

        except Exception as e:
            self.logger.warning(f"Error getting text statistics: {str(e)}")
            return {"characters": 0, "words": 0, "sentences": 0}

Writing text_formatter.py


In [ ]:
# %%writefile enhanced_scene_describer.py
import os
import re
import json
import logging
import random
import numpy as np
from typing import Dict, List, Tuple, Any, Optional

# from scene_type import SCENE_TYPES
# from scene_detail_templates import SCENE_DETAIL_TEMPLATES
# from object_template_fillers import OBJECT_TEMPLATE_FILLERS
# from lighting_conditions import LIGHTING_CONDITIONS
# from viewpoint_templates import VIEWPOINT_TEMPLATES
# from cultural_templates import CULTURAL_TEMPLATES
# from confidence_templates import CONFIDENCE_TEMPLATES
# from landmark_data import ALL_LANDMARKS
# from region_analyzer import RegionAnalyzer
# from viewpoint_detector import ViewpointDetector, ViewpointDetectionError
# from template_manager import TemplateManager, TemplateLoadingError, TemplateFillError
# from object_description_generator import ObjectDescriptionGenerator, ObjectDescriptionError
# from cultural_context_analyzer import CulturalContextAnalyzer, CulturalContextError
# from text_formatter import TextFormatter, TextFormattingError

class EnhancedSceneDescriberError(Exception):
    """場景描述生成過程中的自定義異常"""
    pass

class EnhancedSceneDescriber:
    """
    增強場景描述器 - 提供詳細自然語言場景描述的主要窗口，其他相關class匯集於此

    此class會協調多個專門組件來生成高質量的場景描述，包括視角檢測、
    模板管理、物件描述、文化語境分析和文本格式化。
    """

    def __init__(self, templates_db: Optional[Dict] = None, scene_types: Optional[Dict] = None, spatial_analyzer_instance: Optional[Any] = None):
        """
        初始化增強場景描述器

        Args:
            templates_db: 可選的自定義模板數據庫
            scene_types: 場景類型定義字典
            spatial_analyzer_instance: 空間分析器實例（保持兼容性）
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.setLevel(logging.INFO)

        # 如果沒有logger，就加一個
        if not self.logger.hasHandlers():
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)

        try:
            # 載入場景類型定義
            self.scene_types = scene_types or self._load_default_scene_types()

            # 初始化子組件
            self._initialize_components(templates_db)

            # 保存空間分析器實例以保持兼容性
            self.spatial_analyzer_instance = spatial_analyzer_instance

            self.logger.info("EnhancedSceneDescriber initialized successfully with %d scene types",
                           len(self.scene_types))

        except Exception as e:
            error_msg = f"Failed to initialize EnhancedSceneDescriber: {str(e)}"
            self.logger.error(f"{error_msg}\n{e.__class__.__name__}: {str(e)}")
            raise EnhancedSceneDescriberError(error_msg) from e

    def _load_default_scene_types(self) -> Dict:
        """
        載入默認場景類型

        Returns:
            Dict: 場景類型定義
        """
        try:
            return SCENE_TYPES
        except Exception as e:
            self.logger.error(f"Failed to import SCENE_TYPES: {str(e)}")
            return {}  # 返回空字典

    def _initialize_components(self, templates_db: Optional[Dict]):
        """
        初始化所有子組件

        Args:
            templates_db: 可選的模板數據庫
        """
        try:
            # 初始化視角檢測器
            self.viewpoint_detector = ViewpointDetector()

            # 初始化區域分析器
            self.region_analyzer = RegionAnalyzer()

            # 初始化模板管理器
            self.template_manager = TemplateManager(custom_templates_db=templates_db)

            # 初始化物件描述生成器，傳入區域分析器
            self.object_description_generator = ObjectDescriptionGenerator(
                region_analyzer=self.region_analyzer
            )

            # 初始化文化語境分析器
            self.cultural_context_analyzer = CulturalContextAnalyzer()

            # 初始化文本格式化器
            self.text_formatter = TextFormatter()

            self.logger.debug("All components initialized successfully")

        except Exception as e:
            error_msg = f"Component initialization failed: {str(e)}"
            self.logger.error(error_msg)
            # 初始化基本組件而不是拋出異常
            self._initialize_fallback_components()


    def generate_description(self, scene_type: str, detected_objects: List[Dict], confidence: float,
                           lighting_info: Dict, functional_zones: List[str], enable_landmark: bool = True,
                           scene_scores: Optional[Dict] = None, spatial_analysis: Optional[Dict] = None,
                           image_dimensions: Optional[Tuple[int, int]] = None, # 改為 Tuple
                           places365_info: Optional[Dict] = None,
                           object_statistics: Optional[Dict] = None) -> str:
        try:
            traffic_list = [obj for obj in detected_objects if obj.get("class_name", "") == "traffic light"]
            # print(f"[DEBUG] generate_description 一開始接收到的 traffic light 數量: {len(traffic_list)}") # 原始的 print
            self.logger.debug(f"Initial traffic light count in generate_description: {len(traffic_list)}") # 改用 logger
            # for idx, tl in enumerate(traffic_list): # 這部分 log 可能過於詳細，先註解
            #     self.logger.debug(f"    idx={idx}, confidence={tl.get('confidence', 0):.4f}, bbox={tl.get('bbox')}, region={tl.get('region')}")

            if scene_type == "unknown" or confidence < 0.4:
                generic_desc = self._generate_generic_description(detected_objects, lighting_info)
                return self.text_formatter.format_final_description(generic_desc)

            current_detected_objects = detected_objects
            if not enable_landmark:
                current_detected_objects = [obj for obj in detected_objects if not obj.get("is_landmark", False)]

            places365_context = ""
            if places365_info and places365_info.get('confidence', 0) > 0.3:
                scene_label = places365_info.get('scene_label', '')
                attributes = places365_info.get('attributes', [])
                is_indoor = places365_info.get('is_indoor', None)
                if scene_label:
                    places365_context = f"Scene context: {scene_label}"
                    if attributes:
                        places365_context += f" with characteristics: {', '.join(attributes[:3])}"
                    if is_indoor is not None:
                        indoor_outdoor = "indoor" if is_indoor else "outdoor"
                        places365_context += f" ({indoor_outdoor} environment)"
                self.logger.debug(f"Enhanced description incorporating Places365 context: {places365_context}")

            landmark_objects_in_scene = [obj for obj in current_detected_objects if obj.get("is_landmark", False)]
            has_landmark_in_scene = len(landmark_objects_in_scene) > 0

            if enable_landmark and (scene_type in ["tourist_landmark", "natural_landmark", "historical_monument"] or has_landmark_in_scene):
                landmark_desc = self._generate_landmark_description(
                    scene_type, current_detected_objects, confidence,
                    lighting_info, functional_zones, landmark_objects_in_scene
                )
                return self.text_formatter.format_final_description(landmark_desc)

            viewpoint = self.viewpoint_detector.detect_viewpoint(current_detected_objects)
            current_scene_type = scene_type

            if viewpoint == "aerial":
                if "intersection" in current_scene_type.lower() or self._is_intersection(current_detected_objects):
                    current_scene_type = "aerial_view_intersection"
                elif any(keyword in current_scene_type.lower() for keyword in ["commercial", "shopping", "retail"]):
                    current_scene_type = "aerial_view_commercial_area"
                elif any(keyword in current_scene_type.lower() for keyword in ["plaza", "square"]):
                    current_scene_type = "aerial_view_plaza"
                else:
                    current_scene_type = "aerial_view_general"

            current_scene_type = self._sanitize_scene_type_for_description(current_scene_type)

            # 偵測文化背景資訊
            cultural_context = None
            if viewpoint != "aerial":
                cultural_context = self.cultural_context_analyzer.detect_cultural_context(current_scene_type, current_detected_objects)

             # 設定基礎描述
            base_description = "A scene"
            if viewpoint == "aerial":
                if current_scene_type in self.scene_types: # 確保 self.scene_types 已有
                    base_description = self.scene_types.get(current_scene_type, {}).get("description", "An aerial view showing the layout and movement patterns from above")
                else:
                    base_description = "An aerial view showing the layout and movement patterns from above"
            elif current_scene_type in self.scene_types: # 確保 self.scene_types 已有
                 base_description = self.scene_types.get(current_scene_type, {}).get("description", "A scene")

            # 假設 template_manager 內部可以處理 List[str] 的 functional_zones
            selected_template = self.template_manager.get_template_by_scene_type(
                scene_type=current_scene_type,
                detected_objects=current_detected_objects,
                functional_zones=functional_zones or [] # 傳入 List[str]
            )

            # 用於 fill_template 中的某些佔位符
            processed_functional_zones = {}
            if functional_zones:
                if isinstance(functional_zones, dict): # 如果外部傳入的就是dict
                     processed_functional_zones = functional_zones
                elif isinstance(functional_zones, list): # 如果是 list of strings
                     processed_functional_zones = {f"zone_{i}": {"description": zone_desc} for i, zone_desc in enumerate(functional_zones)}


            # 組織場景資料
            scene_data = {
                "detected_objects": current_detected_objects,
                "functional_zones": processed_functional_zones, # 傳入處理過的字典
                "scene_type": current_scene_type,
                "object_statistics": object_statistics or {},
                "lighting_info": lighting_info,
                "spatial_analysis": spatial_analysis,
                "places365_info": places365_info
            }

            # 應用模板產生核心場景描述
            core_scene_details = self.template_manager.apply_template(selected_template, scene_data)

            # 組合基礎描述與核心場景細節
            description = base_description
            if core_scene_details and core_scene_details.strip():
                cleaned_scene_details = self._validate_and_clean_scene_details(core_scene_details)
                if base_description.lower() == "a scene" and len(cleaned_scene_details) > len(base_description):
                    description = cleaned_scene_details
                else:
                    description = self.text_formatter.smart_append(description, cleaned_scene_details)
            elif not core_scene_details and not description: # 如果兩者都為空
                description = self._generate_generic_description(current_detected_objects, lighting_info)

            # 添加次要描述資訊
            if current_scene_type in self.scene_types and "secondary_description" in self.scene_types[current_scene_type]:
                secondary_desc = self.scene_types[current_scene_type]["secondary_description"]
                if secondary_desc:
                    description = self.text_formatter.smart_append(description, secondary_desc)

            # 處理人物相關的描述
            people_objs = [obj for obj in current_detected_objects if obj.get("class_id") == 0]
            if people_objs:
                people_count = len(people_objs)
                if people_count == 1: people_phrase = "a single person"
                elif 1 < people_count <= 3: people_phrase = f"{people_count} people"
                elif 3 < people_count <= 7: people_phrase = "several people"
                else: people_phrase = "multiple people"
                if not any(p_word in description.lower() for p_word in ["person", "people", "pedestrian"]):
                    description = self.text_formatter.smart_append(description, f"The scene includes {people_phrase}.")

            # 添加文化背景元素(非空中視角）
            if cultural_context and viewpoint != "aerial":
                cultural_elements = self.cultural_context_analyzer.generate_cultural_elements(cultural_context)
                if cultural_elements:
                    description = self.text_formatter.smart_append(description, cultural_elements)

            # 處理光照條件描述
            lighting_description_text = ""
            if lighting_info and "time_of_day" in lighting_info:
                lighting_type = lighting_info["time_of_day"]
                lighting_desc_template = self.template_manager.get_lighting_template(lighting_type)
                if lighting_desc_template: lighting_description_text = lighting_desc_template
            if lighting_description_text and lighting_description_text.lower() not in description.lower():
                description = self.text_formatter.smart_append(description, lighting_description_text)

             # 添加視角特定的觀察描述
            if viewpoint != "eye_level":
                viewpoint_template = self.template_manager.get_viewpoint_template(viewpoint)
                prefix = viewpoint_template.get('prefix', '')
                observation_template = viewpoint_template.get("observation", "")
                scene_elements_for_vp = "the overall layout and objects"
                if viewpoint == "aerial": scene_elements_for_vp = "crossing patterns and general layout"
                viewpoint_observation_text = observation_template.format(scene_elements=scene_elements_for_vp)
                full_viewpoint_text = ""
                if prefix:
                    full_viewpoint_text = prefix.strip() + " "
                    if viewpoint_observation_text and viewpoint_observation_text[0].islower():
                        full_viewpoint_text += viewpoint_observation_text
                    elif viewpoint_observation_text:
                        full_viewpoint_text = prefix + (viewpoint_observation_text[0].lower() + viewpoint_observation_text[1:] if description else viewpoint_observation_text)
                elif viewpoint_observation_text:
                    full_viewpoint_text = viewpoint_observation_text[0].upper() + viewpoint_observation_text[1:]
                if full_viewpoint_text and full_viewpoint_text.lower() not in description.lower():
                    description = self.text_formatter.smart_append(description, full_viewpoint_text)

            # 需要轉換或調整 describe_functional_zones
            if functional_zones and len(functional_zones) > 0:
                if isinstance(functional_zones, dict):
                     zones_desc_text = self.object_description_generator.describe_functional_zones(functional_zones)
                else: # 如果是 list of strings
                     temp_zones_dict = {f"area_{i}": {"description": desc} for i, desc in enumerate(functional_zones)}
                     zones_desc_text = self.object_description_generator.describe_functional_zones(temp_zones_dict)

                if zones_desc_text:
                    description = self.text_formatter.smart_append(description, zones_desc_text)

            # 避免重複提到
            if hasattr(self.text_formatter, 'deduplicate_sentences_in_description'):
                deduplicated_description = self.text_formatter.deduplicate_sentences_in_description(description)
                self.logger.info(f"Description before pre-LLM deduplication (len {len(description)}): '{description[:150]}...'")
                self.logger.info(f"Description after pre-LLM deduplication (len {len(deduplicated_description)}): '{deduplicated_description[:150]}...'")
                description = deduplicated_description # 更新 description 為去除重複後的版本
            else:
                self.logger.warning("TextFormatter does not have 'deduplicate_sentences_in_description'. Skipping pre-LLM deduplication of the internally generated description.")

            # 格式化最終描述
            final_formatted_description = self.text_formatter.format_final_description(description)

            # 如果禁用地標，過濾地標引用
            if not enable_landmark:
                final_formatted_description = self.text_formatter.filter_landmark_references(final_formatted_description, enable_landmark=False)

            # 如果描述為空，使用備用描述
            if not final_formatted_description.strip() or final_formatted_description.strip() == ".":
                self.logger.warning(f"Description for scene_type '{current_scene_type}' became empty after processing. Falling back.")
                final_formatted_description = self.text_formatter.format_final_description(
                    self._generate_generic_description(current_detected_objects, lighting_info)
                )

            return final_formatted_description

        except Exception as e:
            error_msg = f"Error generating scene description: {str(e)}"
            self.logger.error(f"{error_msg}\n{e.__class__.__name__}: {str(e)}")
            try:
                fallback_desc = self._generate_generic_description(detected_objects, lighting_info)
                return self.text_formatter.format_final_description(fallback_desc)
            except:
                return "A scene with various elements is visible."

    def deduplicate_sentences_in_description(self, description: str, similarity_threshold: float = 0.80) -> str:
        """
        從一段描述文本中移除重複或高度相似的句子。
        此方法會嘗試保留更長、資訊更豐富的句子版本。

        Args:
            description (str): 原始描述文本。
            similarity_threshold (float): 判斷句子是否相似的 Jaccard 相似度閾值 (0 到 1)。
                                         預設為 0.8，表示詞彙重疊度達到80%即視為相似。

        Returns:
            str: 移除了重複或高度相似句子後的文本。
        """
        try:
            if not description or not description.strip():
                self.logger.debug("deduplicate_sentences_in_description: Received empty or blank description.")
                return ""

            # 使用正則表達式分割句子，保留句尾標點符號
            sentences = re.split(r'(?<=[.!?])\s+', description.strip())

            if not sentences:
                self.logger.debug("deduplicate_sentences_in_description: No sentences found after splitting.")
                return ""

            unique_sentences_data = []  # 存儲 (原始句子文本, 該句子的詞彙集合)

            for current_sentence_text in sentences:
                current_sentence_text = current_sentence_text.strip()
                if not current_sentence_text:
                    continue

                # 預處理當前句子以進行比較：轉小寫、移除標點、分割成詞彙集合
                simplified_current_text = re.sub(r'[^\w\s\d]', '', current_sentence_text.lower()) # 保留數字
                current_sentence_words = set(simplified_current_text.split())

                if not current_sentence_words: # 如果處理後是空集合 (例如句子只包含標點)
                    # 如果原始句子有內容（例如只有一個標點），就保留它
                    if current_sentence_text and not unique_sentences_data: # 避免在開頭加入孤立標點
                         unique_sentences_data.append((current_sentence_text, current_sentence_words))
                    continue

                is_subsumed_or_highly_similar = False
                index_to_replace = -1

                for i, (kept_sentence_text, kept_sentence_words) in enumerate(unique_sentences_data):
                    if not kept_sentence_words: # 跳過已保留的空詞彙集合
                        continue

                    # 計算 Jaccard 相似度
                    intersection_len = len(current_sentence_words.intersection(kept_sentence_words))
                    union_len = len(current_sentence_words.union(kept_sentence_words))

                    jaccard_similarity = 0.0
                    if union_len > 0:
                        jaccard_similarity = intersection_len / union_len
                    elif not current_sentence_words and not kept_sentence_words: # 兩個都是空的
                        jaccard_similarity = 1.0


                    if jaccard_similarity >= similarity_threshold:
                        # 如果當前句子比已保留的句子長，則標記替換舊的
                        if len(current_sentence_words) > len(kept_sentence_words):
                            self.logger.debug(f"Deduplication: Replacing shorter \"{kept_sentence_text[:50]}...\" "
                                              f"with longer similar \"{current_sentence_text[:50]}...\" (Jaccard: {jaccard_similarity:.2f})")
                            index_to_replace = i
                            break # 找到一個可以被替換的，就跳出內層循環
                        # 如果當前句子比已保留的句子短，或者長度相近但內容高度相似，則標記當前句子為重複
                        else: # current_sentence_words is shorter or of similar length
                            is_subsumed_or_highly_similar = True
                            self.logger.debug(f"Deduplication: Current sentence \"{current_sentence_text[:50]}...\" "
                                              f"is subsumed by or highly similar to \"{kept_sentence_text[:50]}...\" (Jaccard: {jaccard_similarity:.2f}). Skipping.")
                            break

                if index_to_replace != -1:
                    unique_sentences_data[index_to_replace] = (current_sentence_text, current_sentence_words)
                elif not is_subsumed_or_highly_similar:
                    unique_sentences_data.append((current_sentence_text, current_sentence_words))

            # 從 unique_sentences_data 中提取最終的句子文本
            final_sentences = [s_data[0] for s_data in unique_sentences_data]

            # 重組句子，確保每個句子以標點符號結尾，並且句子間有空格
            reconstructed_response = ""
            for i, s_text in enumerate(final_sentences):
                s_text = s_text.strip()
                if not s_text:
                    continue
                # 確保句子以標點結尾
                if not re.search(r'[.!?]$', s_text):
                    s_text += "."

                reconstructed_response += s_text
                if i < len(final_sentences) - 1: # 如果不是最後一句，添加空格
                    reconstructed_response += " "

            self.logger.debug(f"Deduplicated description (len {len(reconstructed_response.strip())}): '{reconstructed_response.strip()[:150]}...'")
            return reconstructed_response.strip()

        except Exception as e:
            self.logger.error(f"Error in deduplicate_sentences_in_description: {str(e)}")
            self.logger.error(traceback.format_exc())
            return description # 發生錯誤時返回原始描述

    def _extract_placeholders(self, template: str) -> List[str]:
        """提取模板中的佔位符"""
        import re
        return re.findall(r'\{([^}]+)\}', template)

    def _generate_placeholder_content(self, placeholder: str, detected_objects: List[Dict],
                                    functional_zones: List, scene_type: str,
                                    object_statistics: Dict) -> str:
        """生成佔位符內容"""
        all_replacements = self._generate_default_replacements()
        return self._get_placeholder_replacement(
            placeholder, {}, all_replacements, detected_objects, scene_type
        )

    def _preprocess_functional_zones(self, functional_zones: List) -> Dict:
        """預處理功能區域數據"""
        if isinstance(functional_zones, list):
            # 將列表轉換為字典格式
            zones_dict = {}
            for i, zone in enumerate(functional_zones):
                if isinstance(zone, str):
                    zones_dict[f"area {i+1}"] = {"description": zone}
                elif isinstance(zone, dict):
                    zones_dict[f"area {i+1}"] = zone
            return zones_dict
        elif isinstance(functional_zones, dict):
            return functional_zones
        else:
            return {}

    def _standardize_placeholder_content(self, content: str, placeholder_type: str) -> str:
        """標準化佔位符內容"""
        if not content:
            return "various elements"
        return content.strip()

    def _finalize_description_output(self, description: str) -> str:
        """最終化描述輸出"""
        if not description:
            return "A scene featuring various elements and organized areas of activity."

        # 基本清理
        import re
        finalized = re.sub(r'\s+', ' ', description).strip()

        # 確保適當結尾
        if finalized and not finalized.endswith(('.', '!', '?')):
            finalized += '.'

        # 首字母大寫
        if finalized:
            finalized = finalized[0].upper() + finalized[1:] if len(finalized) > 1 else finalized.upper()

        return finalized

    def _sanitize_scene_type_for_description(self, scene_type: str) -> str:
        """
        清理場景類型名稱，確保不包含內部標識符格式

        Args:
            scene_type: 原始場景類型名稱

        Returns:
            str: 清理後的場景類型名稱
        """
        try:
            # 移除下劃線並轉換為空格分隔的自然語言
            cleaned_type = scene_type.replace('_', ' ')

            # 確保不直接在描述中使用技術性場景類型名稱
            return cleaned_type

        except Exception as e:
            self.logger.warning(f"Error sanitizing scene type '{scene_type}': {str(e)}")
            return "general scene"

    def _validate_and_clean_scene_details(self, scene_details: str) -> str:
        """
        驗證並清理場景詳細信息，移除可能的模板填充錯誤

        Args:
            scene_details: 原始場景詳細信息

        Returns:
            str: 清理後的場景詳細信息
        """
        try:
            if not scene_details or not scene_details.strip():
                return ""

            cleaned = scene_details.strip()

            # 移除常見的模板填充錯誤模式
            import re

            # 修復 "In ," 類型的錯誤
            cleaned = re.sub(r'\bIn\s*,\s*', 'In this scene, ', cleaned)
            cleaned = re.sub(r'\bAt\s*,\s*', 'At this location, ', cleaned)
            cleaned = re.sub(r'\bWithin\s*,\s*', 'Within this area, ', cleaned)

            # 移除內部標識符格式
            cleaned = re.sub(r'\b\w+_\w+(?:_\w+)*\b(?!\s+(area|zone|region))',
                            lambda m: m.group(0).replace('_', ' '), cleaned)

            # 確保句子完整性
            if cleaned and not cleaned.endswith(('.', '!', '?')):
                cleaned += '.'

            return cleaned

        except Exception as e:
            self.logger.warning(f"Error validating scene details: {str(e)}")
            return scene_details if scene_details else ""

    def _generate_landmark_description(self,
                                     scene_type: str,
                                     detected_objects: List[Dict],
                                     confidence: float,
                                     lighting_info: Optional[Dict] = None,
                                     functional_zones: Optional[Dict] = None,
                                     landmark_objects: Optional[List[Dict]] = None) -> str:
        """
        生成包含地標信息的場景描述

        Args:
            scene_type: 識別的場景類型
            detected_objects: 檢測到的物件列表
            confidence: 場景分類置信度
            lighting_info: 照明條件信息
            functional_zones: 功能區域信息
            landmark_objects: 識別為地標的物件列表

        Returns:
            str: 包含地標信息的自然語言場景描述
        """
        try:
            # 如果沒有提供地標物件，從檢測物件中篩選
            if landmark_objects is None:
                landmark_objects = [obj for obj in detected_objects if obj.get("is_landmark", False)]

            # 如果沒有地標，退回到標準描述
            if not landmark_objects:
                if scene_type in ["tourist_landmark", "natural_landmark", "historical_monument"]:
                    base_description = "A scenic area that appears to be a tourist destination, though specific landmarks are not clearly identifiable."
                else:
                    return self.text_formatter.format_final_description(self._generate_scene_details(
                        scene_type,
                        detected_objects,
                        lighting_info,
                        self.viewpoint_detector.detect_viewpoint(detected_objects)
                    ))
            else:
                # 獲取主要地標
                primary_landmark = max(landmark_objects, key=lambda x: x.get("confidence", 0))
                landmark_name = primary_landmark.get("class_name", "landmark")
                # 先取原生 location
                landmark_location = primary_landmark.get("location", "")
                # 如果 location 為空，就從全域 ALL_LANDMARKS 補上
                lm_id = primary_landmark.get("landmark_id")
                if not landmark_location and lm_id and lm_id in ALL_LANDMARKS:
                    landmark_location = ALL_LANDMARKS[lm_id].get("location", "")

                # 根據地標類型選擇適當的描述模板，並插入 location
                if scene_type == "natural_landmark" or primary_landmark.get("landmark_type") == "natural":
                    base_description = f"A natural landmark scene featuring {landmark_name} in {landmark_location}."
                elif scene_type == "historical_monument" or primary_landmark.get("landmark_type") == "monument":
                    base_description = f"A historical monument scene showcasing {landmark_name}, a significant landmark in {landmark_location}."
                else:
                    base_description = f"A tourist landmark scene centered around {landmark_name}, an iconic structure in {landmark_location}."

            # 添加地標的額外信息
            landmark_details = []
            for landmark in landmark_objects:
                details = []

                if "year_built" in landmark:
                    details.append(f"built in {landmark['year_built']}")

                if "architectural_style" in landmark:
                    details.append(f"featuring {landmark['architectural_style']} architectural style")

                if "significance" in landmark:
                    details.append(landmark["significance"])

                # 補 location（如果該物件沒有 location，就再從 ALL_LANDMARKS 撈一次）
                loc = landmark.get("location", "")
                lm_id_iter = landmark.get("landmark_id")
                if not loc and lm_id_iter and lm_id_iter in ALL_LANDMARKS:
                    loc = ALL_LANDMARKS[lm_id_iter].get("location", "")
                if loc:
                    details.append(f"located in {loc}")

                if details:
                    landmark_details.append(f"{landmark['class_name']} ({', '.join(details)})")

            # 將詳細信息添加到基本描述中
            if landmark_details:
                description = base_description + " The scene features " + ", ".join(landmark_details) + "."
            else:
                description = base_description

            # 獲取視角
            viewpoint = self.viewpoint_detector.detect_viewpoint(detected_objects)

            # 生成人員活動描述
            people_count = len([obj for obj in detected_objects if obj["class_id"] == 0])

            if people_count > 0:
                if people_count == 1:
                    people_description = "There is one person in the scene, likely a tourist or visitor."
                elif people_count < 5:
                    people_description = f"There are {people_count} people in the scene, possibly tourists visiting the landmark."
                else:
                    people_description = f"The scene includes a group of {people_count} people, indicating this is a popular tourist destination."

                description = self.text_formatter.smart_append(description, people_description)

            # 添加照明信息
            if lighting_info and "time_of_day" in lighting_info:
                lighting_type = lighting_info["time_of_day"]
                lighting_description = self.template_manager.get_lighting_template(lighting_type)
                description = self.text_formatter.smart_append(description, lighting_description)

            # 添加視角描述
            if viewpoint != "eye_level":
                viewpoint_template = self.template_manager.get_viewpoint_template(viewpoint)

                prefix = viewpoint_template.get('prefix', '')
                if prefix and not description.startswith(prefix):
                    if description and description[0].isupper():
                        description = prefix + description[0].lower() + description[1:]
                    else:
                        description = prefix + description

                viewpoint_desc = viewpoint_template.get("observation", "").format(
                    scene_elements="the landmark and surrounding area"
                )

                if viewpoint_desc and viewpoint_desc not in description:
                    description = self.text_formatter.smart_append(description, viewpoint_desc)

            # 添加功能區域描述
            if functional_zones and len(functional_zones) > 0:
                zones_desc = self.object_description_generator.describe_functional_zones(functional_zones)
                if zones_desc:
                    description = self.text_formatter.smart_append(description, zones_desc)

            # 描述可能的活動
            landmark_activities = []

            if scene_type == "natural_landmark" or any(obj.get("landmark_type") == "natural" for obj in landmark_objects):
                landmark_activities = [
                    "nature photography",
                    "scenic viewing",
                    "hiking or walking",
                    "guided nature tours",
                    "outdoor appreciation"
                ]
            elif scene_type == "historical_monument" or any(obj.get("landmark_type") == "monument" for obj in landmark_objects):
                landmark_activities = [
                    "historical sightseeing",
                    "educational tours",
                    "cultural appreciation",
                    "photography of historical architecture",
                    "learning about historical significance"
                ]
            else:
                landmark_activities = [
                    "sightseeing",
                    "taking photographs",
                    "guided tours",
                    "cultural tourism",
                    "souvenir shopping"
                ]

            # 添加活動描述
            if landmark_activities:
                activities_text = "Common activities at this location include " + ", ".join(landmark_activities[:3]) + "."
                description = self.text_formatter.smart_append(description, activities_text)

            return self.text_formatter.format_final_description(description)

        except Exception as e:
            self.logger.warning(f"Error generating landmark description: {str(e)}")
            # 備用處理
            return self.text_formatter.format_final_description(
                "A landmark scene with notable architectural or natural features."
            )


    def _is_intersection(self, detected_objects: List[Dict]) -> bool:
        """
        通過分析物件分布來判斷場景是否為十字路口

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            bool: 是否為十字路口
        """
        try:
            pedestrians = [obj for obj in detected_objects if obj.get("class_id") == 0]

            if len(pedestrians) >= 8:
                positions = [obj.get("normalized_center", (0, 0)) for obj in pedestrians]

                x_coords = [pos[0] for pos in positions]
                y_coords = [pos[1] for pos in positions]

                x_variance = np.var(x_coords) if len(x_coords) > 1 else 0
                y_variance = np.var(y_coords) if len(y_coords) > 1 else 0

                x_range = max(x_coords) - min(x_coords)
                y_range = max(y_coords) - min(y_coords)

                if x_range > 0.5 and y_range > 0.5 and 0.7 < (x_range / y_range) < 1.3:
                    return True

            return False

        except Exception as e:
            self.logger.warning(f"Error detecting intersection: {str(e)}")
            return False

    def _generate_generic_description(self, detected_objects: List[Dict], lighting_info: Optional[Dict] = None) -> str:
        """
        當場景類型未知或置信度極低時生成通用描述

        Args:
            detected_objects: 檢測到的物件列表
            lighting_info: 可選的照明條件信息

        Returns:
            str: 基於檢測物件的通用描述
        """
        try:
            obj_counts = {}
            for obj in detected_objects:
                class_name = obj.get("class_name", "unknown object")
                if class_name not in obj_counts:
                    obj_counts[class_name] = 0
                obj_counts[class_name] += 1

            top_objects = sorted(obj_counts.items(), key=lambda x: x[1], reverse=True)[:5]

            if not top_objects:
                base_desc = "This scene displays various elements, though specific objects are not clearly identifiable."
            else:
                objects_text = []
                for name, count in top_objects:
                    # 確保物件名稱不包含技術性格式
                    clean_name = name.replace('_', ' ') if isinstance(name, str) else str(name)
                    if count > 1:
                        objects_text.append(f"{count} {clean_name}s")
                    else:
                        objects_text.append(f"a {clean_name}" if clean_name[0].lower() not in 'aeiou' else f"an {clean_name}")

                if len(objects_text) == 1:
                    objects_list = objects_text[0]
                elif len(objects_text) == 2:
                    objects_list = f"{objects_text[0]} and {objects_text[1]}"
                else:
                    objects_list = ", ".join(objects_text[:-1]) + f", and {objects_text[-1]}"

                base_desc = f"This scene features {objects_list}."

            # 添加照明信息
            if lighting_info and "time_of_day" in lighting_info:
                lighting_type = lighting_info["time_of_day"]
                lighting_desc = self.template_manager.get_lighting_template(lighting_type)
                base_desc += f" {lighting_desc}"

            return base_desc

        except Exception as e:
            self.logger.warning(f"Error generating generic description: {str(e)}")
            return "A general scene is visible with various elements."

    def _generate_scene_details(self,
                              scene_type: str,
                              detected_objects: List[Dict],
                              lighting_info: Optional[Dict] = None,
                              viewpoint: str = "eye_level",
                              spatial_analysis: Optional[Dict] = None,
                              image_dimensions: Optional[Tuple[int, int]] = None,
                              places365_info: Optional[Dict] = None,
                              object_statistics: Optional[Dict] = None) -> str:
        """
        基於場景類型和檢測物件生成詳細描述

        Args:
            scene_type: 識別的場景類型
            detected_objects: 檢測到的物件列表
            lighting_info: 可選的照明條件信息
            viewpoint: 檢測到的視角
            spatial_analysis: 可選的空間分析結果
            image_dimensions: 可選的圖像尺寸
            places365_info: 可選的 Places365 場景分類結果
            object_statistics: 可選的詳細物件統計信息

        Returns:
            str: 詳細場景描述
        """
        try:
            scene_details = ""

            # 日常場景類型列表
            everyday_scene_types = [
                "general_indoor_space", "generic_street_view",
                "desk_area_workspace", "outdoor_gathering_spot",
                "kitchen_counter_or_utility_area", "unknown"
            ]

            # 預處理場景類型以避免內部格式洩漏
            processed_scene_type = self._sanitize_scene_type_for_description(scene_type)

            # 確定場景描述方法
            is_confident_specific_scene = scene_type not in everyday_scene_types and scene_type in self.template_manager.get_scene_detail_templates(scene_type)
            treat_as_everyday = scene_type in everyday_scene_types

            if hasattr(self, 'enable_landmark') and not self.enable_landmark:
                if scene_type not in ["kitchen", "bedroom", "living_room", "office_workspace", "dining_area", "professional_kitchen"]:
                    treat_as_everyday = True

            if treat_as_everyday or not is_confident_specific_scene:
                self.logger.debug(f"Generating dynamic description for scene_type: {scene_type}")
                scene_details = self.object_description_generator.generate_dynamic_everyday_description(
                    detected_objects,
                    lighting_info,
                    viewpoint,
                    spatial_analysis,
                    image_dimensions,
                    places365_info,
                    object_statistics
                )
            else:
                self.logger.debug(f"Using template for scene_type: {scene_type}")
                templates_list = self.template_manager.get_scene_detail_templates(scene_type, viewpoint)

                if templates_list:
                    detail_template = random.choice(templates_list)
                    scene_details = self.template_manager.fill_template(
                        detail_template,
                        detected_objects,
                        scene_type,
                        places365_info,
                        object_statistics
                    )
                else:
                    scene_details = self.object_description_generator.generate_dynamic_everyday_description(
                        detected_objects, lighting_info, viewpoint, spatial_analysis,
                        image_dimensions, places365_info, object_statistics
                    )

            # 如果禁用地標檢測，過濾地標引用
            if hasattr(self, 'enable_landmark') and not self.enable_landmark:
                scene_details = self.text_formatter.filter_landmark_references(scene_details, enable_landmark=False)

            return scene_details if scene_details else "A scene with some visual elements."

        except Exception as e:
            self.logger.warning(f"Error generating scene details: {str(e)}")
            return "A scene with various elements."

    def filter_landmark_references(self, text, enable_landmark=True):
        """
        動態過濾文本中的地標引用

        Args:
            text: 需要過濾的文本
            enable_landmark: 是否啟用地標功能

        Returns:
            str: 過濾後的文本
        """
        return self.text_formatter.filter_landmark_references(text, enable_landmark)

    def get_prominent_objects(self, detected_objects: List[Dict],
                          min_prominence_score: float = 0.5,
                          max_categories_to_return: Optional[int] = None,
                          max_total_objects: Optional[int] = None) -> List[Dict]:
        """
        獲取最重要的物件

        Args:
            detected_objects: 檢測到的物件列表
            min_prominence_score: 最小重要性分數閾值，預設為0.5
            max_categories_to_return: 可選的最大返回類別數量限制
            max_total_objects: 可選的最大返回物件總數限制

        Returns:
            List[Dict]: 重要物件列表
        """
        try:
            # 傳遞所有參數
            prominent_objects = self.object_description_generator.get_prominent_objects(
                detected_objects,
                min_prominence_score,
                max_categories_to_return
            )

            # 如果指定了最大物件總數限制，進行額外過濾
            if max_total_objects is not None and max_total_objects > 0:
                # 限制總物件數量，保持重要性排序
                prominent_objects = prominent_objects[:max_total_objects]

            # 如果指定了最大類別數量限制，則進行額外過濾
            if max_categories_to_return is not None and max_categories_to_return > 0:
                # 按類別分組物件
                categories_seen = set()
                filtered_objects = []

                for obj in prominent_objects:
                    class_name = obj.get("class_name", "unknown")
                    if class_name not in categories_seen:
                        categories_seen.add(class_name)
                        filtered_objects.append(obj)

                        # 如果已達到最大類別數量，停止添加新類別
                        if len(categories_seen) >= max_categories_to_return:
                            break
                    elif class_name in categories_seen:
                        # 如果是已見過的類別，仍然添加該物件
                        filtered_objects.append(obj)

                return filtered_objects

            return prominent_objects

        except Exception as e:
            self.logger.warning(f"Error getting prominent objects: {str(e)}")
            return []

    def detect_viewpoint(self, detected_objects: List[Dict]) -> str:
        """
        檢測圖像視角類型

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            str: 檢測到的視角類型
        """
        try:
            return self.viewpoint_detector.detect_viewpoint(detected_objects)
        except Exception as e:
            self.logger.warning(f"Error detecting viewpoint: {str(e)}")
            return "eye_level"

    def detect_cultural_context(self, scene_type: str, detected_objects: List[Dict]) -> Optional[str]:
        """
        檢測場景的文化語境

        Args:
            scene_type: 識別的場景類型
            detected_objects: 檢測到的物件列表

        Returns:
            Optional[str]: 檢測到的文化語境或None
        """
        try:
            return self.cultural_context_analyzer.detect_cultural_context(scene_type, detected_objects)
        except CulturalContextError as e:
            self.logger.warning(f"Error detecting cultural context: {str(e)}")
            return None

    def generate_cultural_elements(self, cultural_context: str) -> str:
        """
        為檢測到的文化語境生成描述元素

        Args:
            cultural_context: 檢測到的文化語境

        Returns:
            str: 文化元素描述
        """
        try:
            return self.cultural_context_analyzer.generate_cultural_elements(cultural_context)
        except CulturalContextError as e:
            self.logger.warning(f"Error generating cultural elements: {str(e)}")
            return ""

    def format_object_list_for_description(self, objects: List[Dict],
                                         use_indefinite_article_for_one: bool = False,
                                         count_threshold_for_generalization: int = -1,
                                         max_types_to_list: int = 5) -> str:
        """
        將物件列表格式化為人類可讀的字符串

        Args:
            objects: 物件字典列表
            use_indefinite_article_for_one: 單個物件是否使用 "a/an"
            count_threshold_for_generalization: 計數閾值
            max_types_to_list: 最大物件類型數量

        Returns:
            str: 格式化的物件描述字符串
        """
        try:
            return self.object_description_generator.format_object_list_for_description(
                objects, use_indefinite_article_for_one, count_threshold_for_generalization, max_types_to_list
            )
        except ObjectDescriptionError as e:
            self.logger.warning(f"Error formatting object list: {str(e)}")
            return "various objects"

    def get_spatial_description(self, obj: Dict, image_width: Optional[int] = None,
                              image_height: Optional[int] = None) -> str:
        """
        為物件生成空間位置描述

        Args:
            obj: 物件字典
            image_width: 可選的圖像寬度
            image_height: 可選的圖像高度

        Returns:
            str: 空間描述字符串
        """
        try:
            return self.object_description_generator.get_spatial_description(obj, image_width, image_height)
        except ObjectDescriptionError as e:
            self.logger.warning(f"Error generating spatial description: {str(e)}")
            return "in the scene"

    def optimize_object_description(self, description: str) -> str:
        """
        優化物件描述，避免重複列舉相同物件

        Args:
            description: 原始描述文本

        Returns:
            str: 優化後的描述文本
        """
        try:
            return self.object_description_generator.optimize_object_description(description)
        except ObjectDescriptionError as e:
            self.logger.warning(f"Error optimizing object description: {str(e)}")
            return description

    def describe_functional_zones(self, functional_zones: Dict) -> str:
        """
        生成場景功能區域的描述

        Args:
            functional_zones: 識別出的功能區域字典

        Returns:
            str: 功能區域描述
        """
        try:
            return self.object_description_generator.describe_functional_zones(functional_zones)
        except ObjectDescriptionError as e:
            self.logger.warning(f"Error describing functional zones: {str(e)}")
            return ""

    def smart_append(self, current_text: str, new_fragment: str) -> str:
        """
        智能地將新文本片段附加到現有文本

        Args:
            current_text: 要附加到的現有文本
            new_fragment: 要附加的新文本片段

        Returns:
            str: 合併後的文本
        """
        try:
            return self.text_formatter.smart_append(current_text, new_fragment)
        except TextFormattingError as e:
            self.logger.warning(f"Error in smart append: {str(e)}")
            return f"{current_text} {new_fragment}" if current_text else new_fragment

    def format_final_description(self, text: str) -> str:
        """
        格式化最終描述文本

        Args:
            text: 要格式化的文本

        Returns:
            str: 格式化後的文本
        """
        try:
            return self.text_formatter.format_final_description(text)
        except TextFormattingError as e:
            self.logger.warning(f"Error formatting final description: {str(e)}")
            return text

    def get_template(self, category: str, key: Optional[str] = None):
        """
        獲取指定類別的模板

        Args:
            category: 模板類別名稱
            key: 可選的具體模板鍵值

        Returns:
            模板內容
        """
        try:
            return self.template_manager.get_template(category, key)
        except (TemplateLoadingError, TemplateFillError) as e:
            self.logger.warning(f"Error getting template: {str(e)}")
            return None

    def get_viewpoint_confidence(self, detected_objects: List[Dict]) -> Tuple[str, float]:
        """
        獲取視角檢測結果及其信心度

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            Tuple[str, float]: (視角類型, 信心度)
        """
        try:
            return self.viewpoint_detector.get_viewpoint_confidence(detected_objects)
        except ViewpointDetectionError as e:
            self.logger.warning(f"Error getting viewpoint confidence: {str(e)}")
            return "eye_level", 0.5

    def get_supported_cultures(self) -> List[str]:
        """
        獲取所有支援的文化語境列表

        Returns:
            List[str]: 支援的文化語境名稱列表
        """
        return self.cultural_context_analyzer.get_supported_cultures()

    def has_cultural_context(self, cultural_context: str) -> bool:
        """
        檢查是否支援指定的文化語境

        Args:
            cultural_context: 文化語境名稱

        Returns:
            bool: 是否支援該文化語境
        """
        return self.cultural_context_analyzer.has_cultural_context(cultural_context)

    def validate_text_quality(self, text: str) -> Dict[str, bool]:
        """
        驗證文本質量

        Args:
            text: 要驗證的文本

        Returns:
            Dict[str, bool]: 質量檢查結果
        """
        try:
            return self.text_formatter.validate_text_quality(text)
        except TextFormattingError as e:
            self.logger.warning(f"Error validating text quality: {str(e)}")
            return {"error": True}

    def get_text_statistics(self, text: str) -> Dict[str, int]:
        """
        獲取文本統計信息

        Args:
            text: 要分析的文本

        Returns:
            Dict[str, int]: 文本統計信息
        """
        try:
            return self.text_formatter.get_text_statistics(text)
        except TextFormattingError as e:
            self.logger.warning(f"Error getting text statistics: {str(e)}")
            return {"characters": 0, "words": 0, "sentences": 0}

    def reload_templates(self):
        """
        重新載入所有模板
        """
        try:
            self.template_manager.reload_templates()
            self.logger.info("Templates reloaded successfully")
        except (TemplateLoadingError, TemplateFillError) as e:
            self.logger.error(f"Error reloading templates: {str(e)}")
            raise EnhancedSceneDescriberError(f"Failed to reload templates: {str(e)}") from e

    def get_configuration(self) -> Dict[str, Any]:
        """
        獲取當前配置信息

        Returns:
            Dict[str, Any]: 配置信息字典
        """
        try:
            return {
                "scene_types_count": len(self.scene_types),
                "viewpoint_detector_config": self.viewpoint_detector.viewpoint_params,
                "object_generator_config": self.object_description_generator.get_configuration(),
                "supported_cultures": self.cultural_context_analyzer.get_supported_cultures(),
                "template_categories": self.template_manager.get_template_categories()
            }
        except Exception as e:
            self.logger.warning(f"Error getting configuration: {str(e)}")
            return {"error": str(e)}

    def _initialize_fallback_components(self):
        """備用組件初始化"""
        try:
            self.region_analyzer = RegionAnalyzer()
            self.object_description_generator = ObjectDescriptionGenerator(
                region_analyzer=self.region_analyzer
            )
        except Exception as e:
            self.logger.error(f"Fallback component initialization failed: {str(e)}")

Writing enhanced_scene_describer.py


In [ ]:
# %%writefile configuration_manager.py
from typing import Dict, Any, List, Tuple, Optional, Union
import json
import os
from dataclasses import dataclass, field
from pathlib import Path


@dataclass
class FeatureThresholds:
    """Configuration class for feature extraction thresholds."""
    dark_pixel_threshold: float = 50.0
    bright_pixel_threshold: float = 220.0
    sky_blue_hue_min: float = 95.0
    sky_blue_hue_max: float = 135.0
    sky_blue_sat_min: float = 40.0
    sky_blue_val_min: float = 90.0
    gray_sat_max: float = 70.0
    gray_val_min: float = 60.0
    gray_val_max: float = 220.0
    light_source_abs_thresh: float = 220.0


@dataclass
class IndoorOutdoorThresholds:
    """Configuration class for indoor/outdoor classification thresholds."""
    sky_blue_dominance_thresh: float = 0.18
    sky_brightness_ratio_thresh: float = 1.25
    openness_top_thresh: float = 0.68
    sky_texture_complexity_thresh: float = 0.35
    ceiling_likelihood_thresh: float = 0.4
    boundary_clarity_thresh: float = 0.38
    brightness_uniformity_thresh_indoor: float = 0.6
    brightness_uniformity_thresh_outdoor: float = 0.40
    many_bright_spots_thresh: int = 6
    dim_scene_for_spots_thresh: float = 115.0
    home_pattern_thresh_strong: float = 2.0
    home_pattern_thresh_moderate: float = 1.0
    warm_indoor_max_brightness_thresh: float = 135.0
    aerial_top_dark_ratio_thresh: float = 0.9
    aerial_top_complex_thresh: float = 0.60
    aerial_min_avg_brightness_thresh: float = 65.0


@dataclass
class LightingThresholds:
    """Configuration class for lighting condition analysis thresholds."""
    outdoor_night_thresh_brightness: float = 80.0
    outdoor_night_lights_thresh: int = 2
    outdoor_dusk_dawn_thresh_brightness: float = 130.0
    outdoor_dusk_dawn_color_thresh: float = 0.10
    outdoor_day_bright_thresh: float = 140.0
    outdoor_day_blue_thresh: float = 0.05
    outdoor_day_cloudy_thresh: float = 120.0
    outdoor_day_gray_thresh: float = 0.18
    indoor_bright_thresh: float = 130.0
    indoor_moderate_thresh: float = 95.0
    commercial_min_brightness_thresh: float = 105.0
    commercial_min_spots_thresh: int = 3
    stadium_min_spots_thresh: int = 6
    neon_yellow_orange_thresh: float = 0.12
    neon_bright_spots_thresh: int = 4
    neon_avg_saturation_thresh: float = 60.0


@dataclass
class WeightingFactors:
    """Configuration class for feature weighting factors."""
    # Sky/Openness weights (negative values push towards outdoor)
    sky_blue_dominance_w: float = 3.5
    sky_brightness_ratio_w: float = 3.0
    openness_top_w: float = 2.8
    sky_texture_w: float = 2.0

    # Ceiling/Enclosure weights (positive values push towards indoor)
    ceiling_likelihood_w: float = 1.5
    boundary_clarity_w: float = 1.2

    # Brightness weights
    brightness_uniformity_w: float = 0.6
    brightness_non_uniformity_outdoor_w: float = 1.0
    brightness_non_uniformity_indoor_penalty_w: float = 0.1

    # Light source weights
    circular_lights_w: float = 1.2
    indoor_light_score_w: float = 0.8
    many_bright_spots_indoor_w: float = 0.3

    # Color atmosphere weights
    warm_atmosphere_indoor_w: float = 0.15

    # Environment pattern weights
    home_env_strong_w: float = 1.5
    home_env_moderate_w: float = 0.7

    # Structural pattern weights
    aerial_street_w: float = 2.5
    places365_outdoor_scene_w: float = 4.0
    places365_indoor_scene_w: float = 3.0
    places365_attribute_w: float = 1.5


@dataclass
class OverrideFactors:
    """Configuration class for override and reduction factors."""
    sky_override_factor_ceiling: float = 0.1
    sky_override_factor_boundary: float = 0.2
    sky_override_factor_uniformity: float = 0.15
    sky_override_factor_lights: float = 0.05
    sky_override_factor_p365_indoor_decision: float = 0.3
    aerial_enclosure_reduction_factor: float = 0.75
    ceiling_sky_override_factor: float = 0.1
    p365_outdoor_reduces_enclosure_factor: float = 0.3
    p365_indoor_boosts_ceiling_factor: float = 1.5


@dataclass
class ColorRanges:
    """Configuration class for color range definitions."""
    warm_hue_ranges: List[Tuple[float, float]] = field(
        default_factory=lambda: [(0, 50), (330, 360)]
    )
    cool_hue_ranges: List[Tuple[float, float]] = field(
        default_factory=lambda: [(90, 270)]
    )


@dataclass
class AlgorithmParameters:
    """Configuration class for algorithm-specific parameters."""
    indoor_score_sigmoid_scale: float = 0.3
    indoor_decision_threshold: float = 0.5
    places365_high_confidence_thresh: float = 0.75
    places365_moderate_confidence_thresh: float = 0.5
    places365_attribute_confidence_thresh: float = 0.6
    include_diagnostics: bool = True


class ConfigurationManager:
    """
    這主要是管理光線分析的參數，會有很多不同情況, 做parameters配置

    This class provides type-safe access to all configuration parameters,
    supports loading from external files, and includes validation mechanisms.
    """

    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize the configuration manager.

        Args:
            config_path: Optional path to external configuration file.
                        If None, uses default configuration.
        """
        self._feature_thresholds = FeatureThresholds()
        self._indoor_outdoor_thresholds = IndoorOutdoorThresholds()
        self._lighting_thresholds = LightingThresholds()
        self._weighting_factors = WeightingFactors()
        self._override_factors = OverrideFactors()
        self._color_ranges = ColorRanges()
        self._algorithm_parameters = AlgorithmParameters()

        if config_path is not None:
            self.load_from_file(config_path)

    @property
    def feature_thresholds(self) -> FeatureThresholds:
        """Get feature extraction thresholds."""
        return self._feature_thresholds

    @property
    def indoor_outdoor_thresholds(self) -> IndoorOutdoorThresholds:
        """Get indoor/outdoor classification thresholds."""
        return self._indoor_outdoor_thresholds

    @property
    def lighting_thresholds(self) -> LightingThresholds:
        """Get lighting condition analysis thresholds."""
        return self._lighting_thresholds

    @property
    def weighting_factors(self) -> WeightingFactors:
        """Get feature weighting factors."""
        return self._weighting_factors

    @property
    def override_factors(self) -> OverrideFactors:
        """Get override and reduction factors."""
        return self._override_factors

    @property
    def color_ranges(self) -> ColorRanges:
        """Get color range definitions."""
        return self._color_ranges

    @property
    def algorithm_parameters(self) -> AlgorithmParameters:
        """Get algorithm-specific parameters."""
        return self._algorithm_parameters

    def get_legacy_config_dict(self) -> Dict[str, Any]:
        """
        Generate legacy configuration dictionary for backward compatibility.

        Returns:
            Dictionary containing all configuration parameters in the original format.
        """
        config_dict = {}

        # Feature thresholds
        for field_name, field_value in self._feature_thresholds.__dict__.items():
            config_dict[field_name] = field_value

        # Indoor/outdoor thresholds
        for field_name, field_value in self._indoor_outdoor_thresholds.__dict__.items():
            config_dict[field_name] = field_value

        # Lighting thresholds
        for field_name, field_value in self._lighting_thresholds.__dict__.items():
            config_dict[field_name] = field_value

        # Override factors
        for field_name, field_value in self._override_factors.__dict__.items():
            config_dict[field_name] = field_value

        # Color ranges
        for field_name, field_value in self._color_ranges.__dict__.items():
            config_dict[field_name] = field_value

        # Algorithm parameters
        for field_name, field_value in self._algorithm_parameters.__dict__.items():
            config_dict[field_name] = field_value

        # Weighting factors - stored under 'indoor_outdoor_weights' key
        config_dict["indoor_outdoor_weights"] = self._weighting_factors.__dict__.copy()

        return config_dict

    def load_from_file(self, config_path: Union[str, Path]) -> None:
        """
        Load configuration from external JSON file.

        Args:
            config_path: Path to the configuration file.

        Raises:
            FileNotFoundError: If the configuration file doesn't exist.
            ValueError: If the configuration file contains invalid data.
        """
        config_path = Path(config_path)

        if not config_path.exists():
            raise FileNotFoundError(f"Configuration file not found: {config_path}")

        try:
            with open(config_path, 'r', encoding='utf-8') as file:
                config_data = json.load(file)

            self._update_from_dict(config_data)

        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON in configuration file: {e}")
        except Exception as e:
            raise ValueError(f"Error loading configuration: {e}")

    def save_to_file(self, config_path: Union[str, Path]) -> None:
        """
        Save current configuration to JSON file.

        Args:
            config_path: Path where to save the configuration file.
        """
        config_path = Path(config_path)
        config_path.parent.mkdir(parents=True, exist_ok=True)

        config_dict = self.get_legacy_config_dict()

        with open(config_path, 'w', encoding='utf-8') as file:
            json.dump(config_dict, file, indent=2, ensure_ascii=False)

    def _update_from_dict(self, config_data: Dict[str, Any]) -> None:
        """
        Update configuration from dictionary data.

        Args:
            config_data: Dictionary containing configuration parameters.
        """
        # Update feature thresholds
        self._update_dataclass_from_dict(self._feature_thresholds, config_data)

        # Update indoor/outdoor thresholds
        self._update_dataclass_from_dict(self._indoor_outdoor_thresholds, config_data)

        # Update lighting thresholds
        self._update_dataclass_from_dict(self._lighting_thresholds, config_data)

        # Update override factors
        self._update_dataclass_from_dict(self._override_factors, config_data)

        # Update color ranges
        self._update_dataclass_from_dict(self._color_ranges, config_data)

        # Update algorithm parameters
        self._update_dataclass_from_dict(self._algorithm_parameters, config_data)

        # Update weighting factors from nested dictionary
        if "indoor_outdoor_weights" in config_data:
            self._update_dataclass_from_dict(
                self._weighting_factors,
                config_data["indoor_outdoor_weights"]
            )

    def _update_dataclass_from_dict(self, dataclass_instance: object, data_dict: Dict[str, Any]) -> None:
        """
        Update dataclass instance fields from dictionary.

        Args:
            dataclass_instance: The dataclass instance to update.
            data_dict: Dictionary containing the update values.
        """
        for field_name, field_value in data_dict.items():
            if hasattr(dataclass_instance, field_name):
                # Type validation could be added here
                setattr(dataclass_instance, field_name, field_value)

    def validate_configuration(self) -> List[str]:
        """
        Validate the current configuration for logical consistency.

        Returns:
            List of validation error messages. Empty list if configuration is valid.
        """
        errors = []

        # Validate threshold ranges
        ft = self._feature_thresholds
        if ft.dark_pixel_threshold >= ft.bright_pixel_threshold:
            errors.append("Dark pixel threshold must be less than bright pixel threshold")

        if ft.sky_blue_hue_min >= ft.sky_blue_hue_max:
            errors.append("Sky blue hue min must be less than sky blue hue max")

        if ft.gray_val_min >= ft.gray_val_max:
            errors.append("Gray value min must be less than gray value max")

        # Validate probability thresholds
        ap = self._algorithm_parameters
        if not (0.0 <= ap.indoor_decision_threshold <= 1.0):
            errors.append("Indoor decision threshold must be between 0 and 1")

        if not (0.0 <= ap.places365_high_confidence_thresh <= 1.0):
            errors.append("Places365 high confidence threshold must be between 0 and 1")

        # Validate color ranges
        for warm_range in self._color_ranges.warm_hue_ranges:
            if warm_range[0] >= warm_range[1]:
                errors.append(f"Invalid warm hue range: {warm_range}")

        for cool_range in self._color_ranges.cool_hue_ranges:
            if cool_range[0] >= cool_range[1]:
                errors.append(f"Invalid cool hue range: {cool_range}")

        return errors

    def get_threshold_value(self, threshold_name: str) -> Any:
        """
        Get a specific threshold value by name.

        Args:
            threshold_name: Name of the threshold parameter.

        Returns:
            The threshold value.

        Raises:
            AttributeError: If the threshold name doesn't exist.
        """
        # Search through all configuration sections
        for config_section in [
            self._feature_thresholds,
            self._indoor_outdoor_thresholds,
            self._lighting_thresholds,
            self._override_factors,
            self._algorithm_parameters
        ]:
            if hasattr(config_section, threshold_name):
                return getattr(config_section, threshold_name)

        # Check weighting factors
        if hasattr(self._weighting_factors, threshold_name):
            return getattr(self._weighting_factors, threshold_name)

        raise AttributeError(f"Threshold '{threshold_name}' not found")

    def update_threshold(self, threshold_name: str, value: Any) -> None:
        """
        Update a specific threshold value.

        Args:
            threshold_name: Name of the threshold parameter.
            value: New value for the threshold.

        Raises:
            AttributeError: If the threshold name doesn't exist.
        """
        # Search through all configuration sections
        for config_section in [
            self._feature_thresholds,
            self._indoor_outdoor_thresholds,
            self._lighting_thresholds,
            self._override_factors,
            self._algorithm_parameters,
            self._weighting_factors
        ]:
            if hasattr(config_section, threshold_name):
                setattr(config_section, threshold_name, value)
                return

        raise AttributeError(f"Threshold '{threshold_name}' not found")

Writing configuration_manager.py


In [ ]:
# %%writefile feature_extractor.py
import numpy as np
import cv2
import logging
import traceback
from typing import Dict, Any, Optional
# from configuration_manager import ConfigurationManager


class FeatureExtractor:
    """
    Extracts comprehensive lighting and scene features from images.（主要從圖片提取光線資訊)

    This class handles all basic feature computation including brightness analysis,
    color characteristics, texture complexity, and structural features for
    lighting analysis and scene understanding.
    """

    def __init__(self, config_manager: ConfigurationManager):
        """
        Initialize the feature extractor.

        Args:
            config_manager: Configuration manager instance for accessing thresholds.
        """
        self.config_manager = config_manager
        self.logger = self._setup_logger()

    def _setup_logger(self) -> logging.Logger:
        """Set up logger for feature extraction operations."""
        logger = logging.getLogger(f"{__name__}.FeatureExtractor")
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        return logger

    def extract_features(self, image_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Extract all features from an RGB image.

        Args:
            image_rgb: Input image as numpy array in RGB format.

        Returns:
            Dictionary containing all extracted features.
        """
        try:
            # Validate input image
            if not self._validate_image(image_rgb):
                return self._get_default_features()

            # Get image dimensions and prepare processing parameters
            height, width = image_rgb.shape[:2]
            scale_factor = self._calculate_scale_factor(height, width)

            # Create processed image versions
            small_rgb = cv2.resize(
                image_rgb,
                (width // scale_factor, height // scale_factor),
                interpolation=cv2.INTER_AREA
            )
            hsv_img = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
            gray_img = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
            small_gray = cv2.cvtColor(small_rgb, cv2.COLOR_RGB2GRAY)

            # Extract features by category
            brightness_features = self.compute_brightness_features(hsv_img, height, width)
            color_features = self.compute_color_features(hsv_img, height, width)
            texture_features = self.compute_texture_features(small_gray, gray_img, height, width)
            structure_features = self.compute_structure_features(
                small_gray, gray_img, hsv_img, height, width, scale_factor
            )

            # Combine all features
            features = {**brightness_features, **color_features, **texture_features, **structure_features}

            # Add compatibility features for legacy code
            legacy_features = self._compute_legacy_compatibility_features(
                hsv_img, small_gray, features, scale_factor
            )
            features.update(legacy_features)

            self.logger.debug(f"Successfully extracted {len(features)} features from image")
            return features

        except Exception as e:
            self.logger.error(f"Error in feature extraction: {str(e)}")
            self.logger.error(f"Traceback: {traceback.format_exc()}")
            return self._get_default_features()

    def compute_brightness_features(self, hsv_img: np.ndarray, height: int, width: int) -> Dict[str, float]:
        """
        Compute brightness-related features from HSV image.

        Args:
            hsv_img: Image in HSV color space.
            height: Image height.
            width: Image width.

        Returns:
            Dictionary containing brightness features.
        """
        try:
            v_channel = hsv_img[:, :, 2]  # Value channel represents brightness

            # 基本的亮度統計
            avg_brightness = float(np.mean(v_channel))
            brightness_std = float(np.std(v_channel))

            # Pixel ratio calculations
            dark_threshold = self.config_manager.feature_thresholds.dark_pixel_threshold
            bright_threshold = self.config_manager.feature_thresholds.bright_pixel_threshold

            total_pixels = height * width
            dark_pixel_ratio = float(np.sum(v_channel < dark_threshold) / total_pixels)
            bright_pixel_ratio = float(np.sum(v_channel > bright_threshold) / total_pixels)

            # Brightness uniformity
            brightness_uniformity = 1.0 - min(1.0, brightness_std / max(avg_brightness, 1e-5))

            return {
                "avg_brightness": avg_brightness,
                "brightness_std": brightness_std,
                "dark_pixel_ratio": dark_pixel_ratio,
                "bright_pixel_ratio": bright_pixel_ratio,
                "brightness_uniformity": brightness_uniformity
            }

        except Exception as e:
            self.logger.error(f"Error computing brightness features: {str(e)}")
            return {
                "avg_brightness": 100.0,
                "brightness_std": 50.0,
                "dark_pixel_ratio": 0.0,
                "bright_pixel_ratio": 0.0,
                "brightness_uniformity": 0.5
            }

    def compute_color_features(self, hsv_img: np.ndarray, height: int, width: int) -> Dict[str, Any]:
        """
        Compute color-related features from HSV image.

        Args:
            hsv_img: Image in HSV color space.
            height: Image height.
            width: Image width.

        Returns:
            Dictionary containing color features.
        """
        try:
            h_channel, s_channel, v_channel = cv2.split(hsv_img)
            total_pixels = height * width

            # Color ratio calculations
            color_features = {}

            # Blue color detection (general and sky-specific)
            blue_mask = ((h_channel >= 90) & (h_channel <= 140))
            color_features["blue_ratio"] = float(np.sum(blue_mask) / total_pixels)

            # Sky-like blue detection
            ft = self.config_manager.feature_thresholds
            sky_blue_mask = (
                (h_channel >= ft.sky_blue_hue_min) & (h_channel <= ft.sky_blue_hue_max) &
                (s_channel > ft.sky_blue_sat_min) & (v_channel > ft.sky_blue_val_min)
            )
            color_features["sky_like_blue_ratio"] = float(np.sum(sky_blue_mask) / total_pixels)

            # Yellow-orange detection
            yellow_orange_mask = ((h_channel >= 15) & (h_channel <= 45))
            color_features["yellow_orange_ratio"] = float(np.sum(yellow_orange_mask) / total_pixels)

            # Gray detection
            gray_mask = (
                (s_channel < ft.gray_sat_max) &
                (v_channel > ft.gray_val_min) &
                (v_channel < ft.gray_val_max)
            )
            color_features["gray_ratio"] = float(np.sum(gray_mask) / total_pixels)

            # Saturation statistics
            color_features["avg_saturation"] = float(np.mean(s_channel))

            # Sky region analysis
            sky_region_features = self._analyze_sky_region(h_channel, s_channel, v_channel, height)
            color_features.update(sky_region_features)

            # Color atmosphere analysis
            atmosphere_features = self._analyze_color_atmosphere(h_channel, s_channel, total_pixels)
            color_features.update(atmosphere_features)

            return color_features

        except Exception as e:
            self.logger.error(f"Error computing color features: {str(e)}")
            return self._get_default_color_features()

    def compute_texture_features(self, small_gray: np.ndarray, gray_img: np.ndarray,
                                height: int, width: int) -> Dict[str, float]:
        """
        Compute texture and gradient features.

        Args:
            small_gray: Downscaled grayscale image for efficient processing.
            gray_img: Full-resolution grayscale image.
            height: Original image height.
            width: Original image width.

        Returns:
            Dictionary containing texture features.
        """
        try:
            # Compute gradients on small image for efficiency
            gx = cv2.Sobel(small_gray, cv2.CV_32F, 1, 0, ksize=3)
            gy = cv2.Sobel(small_gray, cv2.CV_32F, 0, 1, ksize=3)

            avg_abs_gx = float(np.mean(np.abs(gx)))
            avg_abs_gy = float(np.mean(np.abs(gy)))

            # Gradient ratio (vertical to horizontal)
            gradient_ratio_vertical_horizontal = avg_abs_gy / max(avg_abs_gx, 1e-5)

            # Top region texture complexity
            small_top_third_height = small_gray.shape[0] // 3
            small_sky_region_gray = small_gray[:small_top_third_height, :]

            if small_sky_region_gray.size > 0:
                laplacian_var_sky = cv2.Laplacian(small_sky_region_gray, cv2.CV_64F).var()
                top_region_texture_complexity = min(1.0, laplacian_var_sky / 1000.0)
            else:
                top_region_texture_complexity = 0.5

            # Shadow clarity estimation
            brightness_std = float(np.std(gray_img))
            avg_brightness = float(np.mean(gray_img))
            dark_pixel_ratio = float(np.sum(gray_img < 50) / (height * width))

            if brightness_std > 60 and dark_pixel_ratio < 0.15 and avg_brightness > 100:
                shadow_clarity_score = 0.7
            elif brightness_std < 30 and dark_pixel_ratio > 0.1:
                shadow_clarity_score = 0.3
            else:
                shadow_clarity_score = 0.5

            # Edge density
            edges_density = min(1.0, (avg_abs_gx + avg_abs_gy) / 100.0)

            return {
                "gradient_ratio_vertical_horizontal": gradient_ratio_vertical_horizontal,
                "top_region_texture_complexity": top_region_texture_complexity,
                "shadow_clarity_score": shadow_clarity_score,
                "vertical_strength": avg_abs_gy,
                "horizontal_strength": avg_abs_gx,
                "edges_density": edges_density
            }

        except Exception as e:
            self.logger.error(f"Error computing texture features: {str(e)}")
            return {
                "gradient_ratio_vertical_horizontal": 1.0,
                "top_region_texture_complexity": 0.5,
                "shadow_clarity_score": 0.5,
                "vertical_strength": 0.0,
                "horizontal_strength": 0.0,
                "edges_density": 0.0
            }

    def compute_structure_features(self, small_gray: np.ndarray, gray_img: np.ndarray,
                                  hsv_img: np.ndarray, height: int, width: int,
                                  scale_factor: int) -> Dict[str, float]:
        """
        Compute structural features including ceiling likelihood and boundary clarity.

        Args:
            small_gray: Downscaled grayscale image.
            gray_img: Full-resolution grayscale image.
            hsv_img: HSV image for brightness analysis.
            height: Original image height.
            width: Original image width.
            scale_factor: Downscaling factor used.

        Returns:
            Dictionary containing structural features.
        """
        try:
            # Compute gradients
            gx = cv2.Sobel(small_gray, cv2.CV_32F, 1, 0, ksize=3)
            gy = cv2.Sobel(small_gray, cv2.CV_32F, 0, 1, ksize=3)
            avg_abs_gx = float(np.mean(np.abs(gx)))
            avg_abs_gy = float(np.mean(np.abs(gy)))

            # Ceiling likelihood analysis
            ceiling_features = self._analyze_ceiling_likelihood(
                small_gray, hsv_img, gx, avg_abs_gx, height, scale_factor
            )

            # Boundary clarity analysis
            boundary_clarity = self._compute_boundary_clarity(small_gray, avg_abs_gx, avg_abs_gy)

            # Openness analysis
            openness_top_edge = self._compute_openness_top_edge(gy, avg_abs_gy)

            # Legacy compatibility features
            legacy_structure = self._compute_legacy_structure_features(gray_img, height)

            structure_features = {
                "ceiling_likelihood": ceiling_features["ceiling_likelihood"],
                "boundary_clarity": boundary_clarity,
                "openness_top_edge": openness_top_edge,
                **legacy_structure
            }

            return structure_features

        except Exception as e:
            self.logger.error(f"Error computing structure features: {str(e)}")
            return {
                "ceiling_likelihood": 0.0,
                "boundary_clarity": 0.0,
                "openness_top_edge": 0.5,
                "ceiling_uniformity": 0.5,
                "horizontal_line_ratio": 0.0
            }

    def _analyze_sky_region(self, h_channel: np.ndarray, s_channel: np.ndarray,
                           v_channel: np.ndarray, height: int) -> Dict[str, float]:
        """Analyze features specific to the sky region (top third of image)."""
        try:
            top_third_height = height // 3
            sky_region_v = v_channel[:top_third_height, :]
            sky_region_s = s_channel[:top_third_height, :]
            sky_region_h = h_channel[:top_third_height, :]

            if sky_region_v.size == 0:
                return self._get_default_sky_features()

            # Sky region brightness analysis
            sky_region_avg_brightness = float(np.mean(sky_region_v))
            overall_avg_brightness = float(np.mean(v_channel))
            sky_region_brightness_ratio = sky_region_avg_brightness / max(overall_avg_brightness, 1e-5)
            sky_region_saturation = float(np.mean(sky_region_s))

            # Sky blue dominance in sky region
            ft = self.config_manager.feature_thresholds
            sky_region_blue_pixels = np.sum(
                (sky_region_h >= ft.sky_blue_hue_min) & (sky_region_h <= ft.sky_blue_hue_max) &
                (sky_region_s > ft.sky_blue_sat_min) & (sky_region_v > ft.sky_blue_val_min)
            )
            sky_region_blue_dominance = float(sky_region_blue_pixels / max(1, sky_region_v.size))

            return {
                "sky_region_brightness_ratio": sky_region_brightness_ratio,
                "sky_region_saturation": sky_region_saturation,
                "sky_region_blue_dominance": sky_region_blue_dominance,
                "sky_brightness": sky_region_avg_brightness
            }

        except Exception as e:
            self.logger.error(f"Error analyzing sky region: {str(e)}")
            return self._get_default_sky_features()

    def _analyze_color_atmosphere(self, h_channel: np.ndarray, s_channel: np.ndarray,
                                 total_pixels: int) -> Dict[str, Any]:
        """Analyze warm/cool color atmosphere."""
        try:
            cr = self.config_manager.color_ranges

            # Warm colors detection
            warm_mask = np.zeros_like(h_channel, dtype=bool)
            for h_min, h_max in cr.warm_hue_ranges:
                warm_mask |= ((h_channel >= h_min) & (h_channel <= h_max))
            warm_ratio = float(np.sum(warm_mask & (s_channel > 30)) / total_pixels)

            # Cool colors detection
            cool_mask = np.zeros_like(h_channel, dtype=bool)
            for h_min, h_max in cr.cool_hue_ranges:
                cool_mask |= ((h_channel >= h_min) & (h_channel <= h_max))
            cool_ratio = float(np.sum(cool_mask & (s_channel > 30)) / total_pixels)

            # Determine overall atmosphere
            if warm_ratio > cool_ratio and warm_ratio > 0.3:
                color_atmosphere = "warm"
            elif cool_ratio > warm_ratio and cool_ratio > 0.3:
                color_atmosphere = "cool"
            else:
                color_atmosphere = "neutral"

            return {
                "warm_ratio": warm_ratio,
                "cool_ratio": cool_ratio,
                "color_atmosphere": color_atmosphere
            }

        except Exception as e:
            self.logger.error(f"Error analyzing color atmosphere: {str(e)}")
            return {
                "warm_ratio": 0.0,
                "cool_ratio": 0.0,
                "color_atmosphere": "neutral"
            }

    def _analyze_ceiling_likelihood(self, small_gray: np.ndarray, hsv_img: np.ndarray,
                                   gx: np.ndarray, avg_abs_gx: float, height: int,
                                   scale_factor: int) -> Dict[str, float]:
        """Analyze likelihood of ceiling presence."""
        try:
            ceiling_likelihood = 0.0
            config = self.config_manager.indoor_outdoor_thresholds

            # Get sky region brightness for analysis
            v_channel = hsv_img[:, :, 2]
            top_third_height = height // 3
            sky_region_v = v_channel[:top_third_height, :]
            sky_region_avg_brightness = float(np.mean(sky_region_v)) if sky_region_v.size > 0 else 0

            # Get top region texture complexity
            small_top_third_height = small_gray.shape[0] // 3
            small_sky_region_gray = small_gray[:small_top_third_height, :]

            if small_sky_region_gray.size > 0:
                laplacian_var = cv2.Laplacian(small_sky_region_gray, cv2.CV_64F).var()
                top_region_texture_complexity = min(1.0, laplacian_var / 1000.0)
            else:
                top_region_texture_complexity = 0.5

            # Condition 1: Simple texture and moderate brightness
            ceiling_texture_thresh = getattr(config, 'ceiling_texture_thresh', 0.4)
            ceiling_brightness_min = getattr(config, 'ceiling_brightness_min', 60)
            ceiling_brightness_max = getattr(config, 'ceiling_brightness_max', 230)

            if (top_region_texture_complexity < ceiling_texture_thresh and
                ceiling_brightness_min < sky_region_avg_brightness < ceiling_brightness_max):
                ceiling_likelihood += 0.45

            # Condition 2: Horizontal line strength
            top_horizontal_lines_strength = float(np.mean(np.abs(gx[:small_gray.shape[0]//3, :])))
            ceiling_horizontal_line_factor = getattr(config, 'ceiling_horizontal_line_factor', 1.15)

            if top_horizontal_lines_strength > avg_abs_gx * ceiling_horizontal_line_factor:
                ceiling_likelihood += 0.35

            # Condition 3: Central bright spot (lamp detection)
            center_y_sm, center_x_sm = small_gray.shape[0]//2, small_gray.shape[1]//2
            lamp_check_radius_y = small_gray.shape[0] // 8
            lamp_check_radius_x = small_gray.shape[1] // 8

            center_region = small_gray[
                max(0, center_y_sm - lamp_check_radius_y):min(small_gray.shape[0], center_y_sm + lamp_check_radius_y),
                max(0, center_x_sm - lamp_check_radius_x):min(small_gray.shape[1], center_x_sm + lamp_check_radius_x)
            ]

            if center_region.size > 0:
                avg_brightness = float(np.mean(small_gray))
                center_brightness = float(np.mean(center_region))
                ceiling_center_bright_factor = getattr(config, 'ceiling_center_bright_factor', 1.25)

                if center_brightness > avg_brightness * ceiling_center_bright_factor:
                    ceiling_likelihood += 0.30

            # Sky dominance analysis for penalty
            sky_region_blue_dominance = self._compute_sky_blue_dominance(hsv_img, height)
            sky_region_brightness_ratio = sky_region_avg_brightness / max(float(np.mean(v_channel)), 1e-5)

            # Penalties for strong sky signals
            ceiling_max_sky_blue_thresh = getattr(config, 'ceiling_max_sky_blue_thresh', 0.08)
            ceiling_max_sky_brightness_ratio = getattr(config, 'ceiling_max_sky_brightness_ratio', 1.15)

            if (sky_region_blue_dominance < ceiling_max_sky_blue_thresh and
                sky_region_brightness_ratio < ceiling_max_sky_brightness_ratio):
                ceiling_likelihood += 0.15

            # Strong sky override
            sky_blue_dominance_strong_thresh = getattr(config, 'sky_blue_dominance_strong_thresh', 0.25)
            sky_brightness_strong_thresh = getattr(config, 'sky_brightness_strong_thresh', 1.25)
            ceiling_sky_override_factor = getattr(config, 'ceiling_sky_override_factor', 0.1)

            if (sky_region_blue_dominance > sky_blue_dominance_strong_thresh and
                sky_region_brightness_ratio > sky_brightness_strong_thresh):
                ceiling_likelihood *= ceiling_sky_override_factor

            ceiling_likelihood = min(1.0, ceiling_likelihood)

            return {"ceiling_likelihood": ceiling_likelihood}

        except Exception as e:
            self.logger.error(f"Error analyzing ceiling likelihood: {str(e)}")
            return {"ceiling_likelihood": 0.0}

    def _compute_sky_blue_dominance(self, hsv_img: np.ndarray, height: int) -> float:
        """Compute blue dominance in sky region."""
        try:
            h_channel, s_channel, v_channel = cv2.split(hsv_img)
            top_third_height = height // 3
            sky_region_h = h_channel[:top_third_height, :]
            sky_region_s = s_channel[:top_third_height, :]
            sky_region_v = v_channel[:top_third_height, :]

            if sky_region_h.size == 0:
                return 0.0

            ft = self.config_manager.feature_thresholds
            sky_region_blue_pixels = np.sum(
                (sky_region_h >= ft.sky_blue_hue_min) & (sky_region_h <= ft.sky_blue_hue_max) &
                (sky_region_s > ft.sky_blue_sat_min) & (sky_region_v > ft.sky_blue_val_min)
            )

            return float(sky_region_blue_pixels / max(1, sky_region_h.size))

        except Exception as e:
            self.logger.error(f"Error computing sky blue dominance: {str(e)}")
            return 0.0

    def _compute_boundary_clarity(self, small_gray: np.ndarray, avg_abs_gx: float,
                                 avg_abs_gy: float) -> float:
        """Compute boundary clarity score."""
        try:
            edge_width_sm = max(1, small_gray.shape[1] // 10)
            edge_height_sm = max(1, small_gray.shape[0] // 10)

            # Edge gradients
            left_edge_grad_x = 0.0
            right_edge_grad_x = 0.0
            top_edge_grad_y = 0.0

            if small_gray.shape[1] > edge_width_sm:
                left_edge = small_gray[:, :edge_width_sm]
                right_edge = small_gray[:, -edge_width_sm:]
                left_edge_grad_x = float(np.mean(np.abs(cv2.Sobel(left_edge, cv2.CV_32F, 1, 0, ksize=3))))
                right_edge_grad_x = float(np.mean(np.abs(cv2.Sobel(right_edge, cv2.CV_32F, 1, 0, ksize=3))))

            if small_gray.shape[0] > edge_height_sm:
                top_edge = small_gray[:edge_height_sm, :]
                top_edge_grad_y = float(np.mean(np.abs(cv2.Sobel(top_edge, cv2.CV_32F, 0, 1, ksize=3))))

            # Normalize against average gradients
            boundary_clarity = (left_edge_grad_x + right_edge_grad_x + top_edge_grad_y) / (
                3 * max(avg_abs_gx, avg_abs_gy, 1e-5)
            )
            boundary_clarity = min(1.0, boundary_clarity / 1.5)

            return boundary_clarity

        except Exception as e:
            self.logger.error(f"Error computing boundary clarity: {str(e)}")
            return 0.0

    def _compute_openness_top_edge(self, gy: np.ndarray, avg_abs_gy: float) -> float:
        """Compute openness of top edge."""
        try:
            top_edge_strip_gy = float(np.mean(np.abs(gy[:max(1, gy.shape[0]//20), :])))
            openness_top_edge = 1.0 - min(1.0, top_edge_strip_gy / max(avg_abs_gy, 1e-5) / 0.5)
            return openness_top_edge
        except Exception as e:
            self.logger.error(f"Error computing top edge openness: {str(e)}")
            return 0.5

    def _compute_legacy_compatibility_features(self, hsv_img: np.ndarray, small_gray: np.ndarray,
                                             features: Dict[str, Any], scale_factor: int) -> Dict[str, Any]:
        """Compute additional features for backward compatibility."""
        try:
            v_channel = hsv_img[:, :, 2]

            # Light source detection
            light_features = self._detect_light_sources(v_channel, features["avg_brightness"],
                                                       features["brightness_std"], scale_factor)

            # Street line detection
            street_score = self._compute_street_line_score(small_gray)

            # Additional legacy features
            legacy_features = {
                **light_features,
                "street_line_score": street_score,
                "sky_blue_ratio": features.get("sky_like_blue_ratio", 0.0),  # Alias
                "gradient_ratio": features.get("gradient_ratio_vertical_horizontal", 1.0)  # Alias
            }

            return legacy_features

        except Exception as e:
            self.logger.error(f"Error computing legacy compatibility features: {str(e)}")
            return {}

    def _detect_light_sources(self, v_channel: np.ndarray, avg_brightness: float,
                             brightness_std: float, scale_factor: int) -> Dict[str, float]:
        """Detect artificial light sources in the image."""
        try:
            # Sample pixels for efficiency
            sampled_v = v_channel[::scale_factor*2, ::scale_factor*2]

            # Light threshold
            light_threshold = min(
                self.config_manager.feature_thresholds.light_source_abs_thresh,
                avg_brightness + 2 * brightness_std
            )

            is_bright_spots = sampled_v > light_threshold
            bright_spot_count = int(np.sum(is_bright_spots))

            # Initialize light features
            circular_light_count = 0
            indoor_light_score = 0.0
            light_distribution_uniformity = 0.5

            # Analyze light distribution if spots are found
            if 1 < bright_spot_count < 20:
                bright_y, bright_x = np.where(is_bright_spots)
                if len(bright_y) > 1:
                    mean_x, mean_y = np.mean(bright_x), np.mean(bright_y)
                    dist_from_center = np.sqrt((bright_x - mean_x)**2 + (bright_y - mean_y)**2)

                    if np.std(dist_from_center) < np.mean(dist_from_center):
                        circular_light_count = min(3, len(bright_y) // 2)
                        light_distribution_uniformity = 0.7

                    if np.mean(bright_y) < sampled_v.shape[0] / 2:
                        indoor_light_score = 0.6
                    else:
                        indoor_light_score = 0.3

            return {
                "bright_spot_count": bright_spot_count,
                "circular_light_count": circular_light_count,
                "indoor_light_score": indoor_light_score,
                "light_distribution_uniformity": light_distribution_uniformity
            }

        except Exception as e:
            self.logger.error(f"Error detecting light sources: {str(e)}")
            return {
                "bright_spot_count": 0,
                "circular_light_count": 0,
                "indoor_light_score": 0.0,
                "light_distribution_uniformity": 0.5
            }

    def _compute_street_line_score(self, small_gray: np.ndarray) -> float:
        """Compute street line detection score."""
        try:
            street_line_score = 0.0
            bottom_half_sm = small_gray[small_gray.shape[0]//2:, :]

            if bottom_half_sm.size > 0:
                bottom_vert_gradient = cv2.Sobel(bottom_half_sm, cv2.CV_32F, 0, 1, ksize=3)
                strong_vert_lines = np.abs(bottom_vert_gradient) > 50

                if np.sum(strong_vert_lines) > (bottom_half_sm.size * 0.05):
                    street_line_score = 0.7

            return street_line_score

        except Exception as e:
            self.logger.error(f"Error computing street line score: {str(e)}")
            return 0.0

    def _compute_legacy_structure_features(self, gray_img: np.ndarray, height: int) -> Dict[str, float]:
        """Compute legacy structure features for backward compatibility."""
        try:
            # Top region analysis for ceiling uniformity
            top_region = gray_img[:height//4, :]
            top_region_std = float(np.std(top_region)) if top_region.size > 0 else 0.0
            ceiling_uniformity = 1.0 - min(1.0, top_region_std / max(float(np.mean(top_region)) if top_region.size > 0 else 1e-5, 1e-5))

            # Horizontal line detection in top region
            if top_region.size > 0:
                top_gradients = np.abs(cv2.Sobel(top_region, cv2.CV_32F, 0, 1, ksize=3))
                horizontal_lines_strength = float(np.mean(top_gradients))
                horizontal_line_ratio = min(1.0, horizontal_lines_strength / 40.0)
            else:
                horizontal_line_ratio = 0.0

            # Boundary edge score computation
            boundary_edge_score = self._compute_legacy_boundary_score(gray_img)

            return {
                "ceiling_uniformity": ceiling_uniformity,
                "horizontal_line_ratio": horizontal_line_ratio,
                "top_region_std": top_region_std,
                "boundary_edge_score": boundary_edge_score
            }

        except Exception as e:
            self.logger.error(f"Error computing legacy structure features: {str(e)}")
            return {
                "ceiling_uniformity": 0.5,
                "horizontal_line_ratio": 0.0,
                "top_region_std": 0.0,
                "boundary_edge_score": 0.0
            }

    def _compute_legacy_boundary_score(self, gray_img: np.ndarray) -> float:
        """Compute legacy boundary edge score."""
        try:
            height, width = gray_img.shape

            # Create small version for boundary analysis
            small_height, small_width = height // 4, width // 4
            small_gray = cv2.resize(gray_img, (small_width, small_height), interpolation=cv2.INTER_AREA)

            # Edge regions
            left_edge_sm = small_gray[:, :small_width//6] if small_width > 6 else small_gray
            right_edge_sm = small_gray[:, 5*small_width//6:] if small_width > 6 else small_gray
            top_edge_sm = small_gray[:small_height//6, :] if small_height > 6 else small_gray

            # Compute gradients for each edge
            left_gradient = float(np.mean(np.abs(cv2.Sobel(left_edge_sm, cv2.CV_32F, 1, 0, ksize=3)))) if left_edge_sm.size > 0 else 0
            right_gradient = float(np.mean(np.abs(cv2.Sobel(right_edge_sm, cv2.CV_32F, 1, 0, ksize=3)))) if right_edge_sm.size > 0 else 0
            top_gradient = float(np.mean(np.abs(cv2.Sobel(top_edge_sm, cv2.CV_32F, 0, 1, ksize=3)))) if top_edge_sm.size > 0 else 0

            # Combine and normalize
            boundary_edge_score = (min(1.0, left_gradient/50) + min(1.0, right_gradient/50) + min(1.0, top_gradient/50)) / 3

            return boundary_edge_score

        except Exception as e:
            self.logger.error(f"Error computing legacy boundary score: {str(e)}")
            return 0.0

    def _validate_image(self, image_rgb: np.ndarray) -> bool:
        """Validate input image format and dimensions."""
        try:
            if not isinstance(image_rgb, np.ndarray):
                self.logger.error("Input is not a numpy array")
                return False

            if len(image_rgb.shape) != 3 or image_rgb.shape[2] != 3:
                self.logger.error(f"Invalid image shape: {image_rgb.shape}. Expected (H, W, 3)")
                return False

            height, width = image_rgb.shape[:2]
            if height == 0 or width == 0:
                self.logger.error(f"Invalid image dimensions: {height}x{width}")
                return False

            return True

        except Exception as e:
            self.logger.error(f"Error validating image: {str(e)}")
            return False

    def _calculate_scale_factor(self, height: int, width: int) -> int:
        """Calculate appropriate scale factor for image processing efficiency."""
        try:
            base_scale = 4
            scale_factor = base_scale + min(8, max(0, int((height * width) / (1000 * 1000)) if height * width > 0 else 0))
            return max(1, scale_factor)
        except Exception as e:
            self.logger.error(f"Error calculating scale factor: {str(e)}")
            return 4

    def _get_default_features(self) -> Dict[str, Any]:
        """Return default feature values in case of processing errors."""
        return {
            "avg_brightness": 100.0,
            "brightness_std": 50.0,
            "dark_pixel_ratio": 0.0,
            "bright_pixel_ratio": 0.0,
            "brightness_uniformity": 0.5,
            "blue_ratio": 0.0,
            "sky_like_blue_ratio": 0.0,
            "yellow_orange_ratio": 0.0,
            "gray_ratio": 0.0,
            "avg_saturation": 100.0,
            "sky_region_brightness_ratio": 1.0,
            "sky_region_saturation": 0.0,
            "sky_region_blue_dominance": 0.0,
            "sky_brightness": 100.0,
            "warm_ratio": 0.0,
            "cool_ratio": 0.0,
            "color_atmosphere": "neutral",
            "gradient_ratio_vertical_horizontal": 1.0,
            "top_region_texture_complexity": 0.5,
            "shadow_clarity_score": 0.5,
            "vertical_strength": 0.0,
            "horizontal_strength": 0.0,
            "edges_density": 0.0,
            "ceiling_likelihood": 0.0,
            "boundary_clarity": 0.0,
            "openness_top_edge": 0.5,
            "ceiling_uniformity": 0.5,
            "horizontal_line_ratio": 0.0,
            "top_region_std": 0.0,
            "boundary_edge_score": 0.0,
            "bright_spot_count": 0,
            "circular_light_count": 0,
            "indoor_light_score": 0.0,
            "light_distribution_uniformity": 0.5,
            "street_line_score": 0.0,
            "sky_blue_ratio": 0.0,
            "gradient_ratio": 1.0
        }

    def _get_default_color_features(self) -> Dict[str, Any]:
        """Return default color feature values."""
        return {
            "blue_ratio": 0.0,
            "sky_like_blue_ratio": 0.0,
            "yellow_orange_ratio": 0.0,
            "gray_ratio": 0.0,
            "avg_saturation": 100.0,
            "sky_region_brightness_ratio": 1.0,
            "sky_region_saturation": 0.0,
            "sky_region_blue_dominance": 0.0,
            "sky_brightness": 100.0,
            "warm_ratio": 0.0,
            "cool_ratio": 0.0,
            "color_atmosphere": "neutral"
        }

    def _get_default_sky_features(self) -> Dict[str, float]:
        """Return default sky region feature values."""
        return {
            "sky_region_brightness_ratio": 1.0,
            "sky_region_saturation": 0.0,
            "sky_region_blue_dominance": 0.0,
            "sky_brightness": 100.0
        }

Overwriting feature_extractor.py


In [ ]:
# %%writefile indoor_outdoor_classifier.py
import numpy as np
import logging
import traceback
from typing import Dict, Any, Optional, List
# from configuration_manager import ConfigurationManager


class IndoorOutdoorClassifier:
    """
    Classifies scenes as indoor or outdoor based on visual features and Places365 context.(判斷室內室外)
    此class會融入PLACES365，使判斷更準確

    This class implements sophisticated decision logic that combines multiple evidence sources
    including visual scene analysis, structural features, and external scene classification
    data to determine whether a scene is indoor or outdoor.
    """

    def __init__(self, config_manager: ConfigurationManager):
        """
        Initialize the indoor/outdoor classifier.

        Args:
            config_manager: Configuration manager instance for accessing thresholds and weights.
        """
        self.config_manager = config_manager
        self.logger = self._setup_logger()

        # Internal threshold constants for Places365 confidence levels
        self.P365_HIGH_CONF_THRESHOLD = 0.65
        self.P365_MODERATE_CONF_THRESHOLD = 0.4

        # 以下是絕對室內/室外的基本情況
        self.DEFINITELY_OUTDOOR_KEYWORDS_P365 = [
            "street", "road", "highway", "park", "beach", "mountain", "forest", "field",
            "outdoor", "sky", "coast", "courtyard", "square", "plaza", "bridge",
            "parking_lot", "playground", "stadium", "construction_site", "river", "ocean",
            "desert", "garden", "trail", "intersection", "crosswalk", "sidewalk", "pathway",
            "avenue", "boulevard", "downtown", "city_center", "market_outdoor"
        ]

        self.DEFINITELY_INDOOR_KEYWORDS_P365 = [
            "bedroom", "office", "kitchen", "library", "classroom", "conference_room", "living_room",
            "bathroom", "hospital", "hotel_room", "cabin", "interior", "museum", "gallery",
            "mall", "market_indoor", "basement", "corridor", "lobby", "restaurant_indoor",
            "bar_indoor", "shop_indoor", "gym_indoor"
        ]

    def _setup_logger(self) -> logging.Logger:
        """Set up logger for classification operations."""
        logger = logging.getLogger(f"{__name__}.IndoorOutdoorClassifier")
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        return logger

    def classify(self, features: Dict[str, Any], places365_info: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Classify scene as indoor or outdoor based on features and Places365 context.

        Args:
            features: Dictionary containing extracted image features.
            places365_info: Optional Places365 classification information.

        Returns:
            Dictionary containing classification results including decision, probability,
            feature contributions, and diagnostic information.
        """
        try:
            self.logger.debug("Starting indoor/outdoor classification")

            # Initialize classification components
            visual_score = 0.0
            feature_contributions = {}
            diagnostics = {}

            # Extract Places365 information
            p365_context = self._extract_places365_context(places365_info, diagnostics)

            # Compute visual evidence score
            visual_analysis = self._analyze_visual_evidence(features, diagnostics)
            visual_score = visual_analysis["visual_score"]
            feature_contributions.update(visual_analysis["contributions"])

            # Incorporate Places365 influence
            p365_analysis = self._analyze_places365_influence(
                p365_context, visual_analysis.get("strong_sky_signal", False), diagnostics
            )
            p365_influence_score = p365_analysis["influence_score"]
            if abs(p365_influence_score) > 0.01:
                feature_contributions["places365_influence_score"] = round(p365_influence_score, 2)

            # Calculate final score and probability
            final_indoor_score = visual_score + p365_influence_score
            classification_result = self._compute_final_classification(
                final_indoor_score, visual_score, p365_influence_score, diagnostics
            )

            # Apply Places365 override if conditions are met
            override_result = self._apply_places365_override(
                classification_result, p365_context, diagnostics
            )

            # Ensure default values for missing contributions
            self._ensure_default_contributions(feature_contributions)

            # 最終結果
            result = {
                "is_indoor": override_result["is_indoor"],
                "indoor_probability": override_result["indoor_probability"],
                "indoor_score_raw": override_result["final_score"],
                "feature_contributions": feature_contributions,
                "diagnostics": diagnostics
            }

            self.logger.debug(f"Classification complete: indoor={result['is_indoor']}, "
                            f"probability={result['indoor_probability']:.3f}")

            return result

        except Exception as e:
            self.logger.error(f"Error in indoor/outdoor classification: {str(e)}")
            self.logger.error(f"Traceback: {traceback.format_exc()}")
            return self._get_default_classification_result()

    def _extract_places365_context(self, places365_info: Optional[Dict],
                                  diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Extract and validate Places365 context information."""
        context = {
            "mapped_scene": "unknown",
            "is_indoor_from_classification": None,
            "attributes": [],
            "confidence": 0.0,
            "is_indoor": None
        }

        if places365_info:
            context["mapped_scene"] = places365_info.get('mapped_scene_type', 'unknown').lower()
            context["attributes"] = [attr.lower() for attr in places365_info.get('attributes', [])]
            context["confidence"] = places365_info.get('confidence', 0.0)
            context["is_indoor_from_classification"] = places365_info.get('is_indoor_from_classification', None)
            context["is_indoor"] = places365_info.get('is_indoor', None)

            diagnostics["p365_context_received"] = (
                f"P365 Scene: {context['mapped_scene']}, P365 SceneConf: {context['confidence']:.2f}, "
                f"P365 DirectIndoor: {context['is_indoor_from_classification']}, "
                f"P365 Attrs: {context['attributes']}"
            )

        return context

    def _analyze_visual_evidence(self, features: Dict[str, Any],
                                diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze visual evidence for indoor/outdoor classification."""
        visual_score = 0.0
        contributions = {}
        strong_sky_signal = False

        # Sky and openness analysis
        sky_analysis = self._analyze_sky_evidence(features, diagnostics)
        visual_score += sky_analysis["score"]
        if sky_analysis["score"] != 0:
            contributions["sky_openness_features_visual"] = round(sky_analysis["score"], 2)
        strong_sky_signal = sky_analysis["strong_signal"]

        # Enclosure and structural analysis
        enclosure_analysis = self._analyze_enclosure_evidence(features, strong_sky_signal, diagnostics)
        visual_score += enclosure_analysis["score"]
        if enclosure_analysis["score"] != 0:
            contributions["enclosure_features"] = round(enclosure_analysis["score"], 2)

        # Brightness uniformity analysis
        uniformity_analysis = self._analyze_brightness_uniformity(features, strong_sky_signal, diagnostics)
        visual_score += uniformity_analysis["score"]
        if uniformity_analysis["score"] != 0:
            contributions["brightness_uniformity_contribution"] = round(uniformity_analysis["score"], 2)

        # Light source analysis
        light_analysis = self._analyze_light_sources(features, strong_sky_signal, diagnostics)
        visual_score += light_analysis["score"]
        if light_analysis["score"] != 0:
            contributions["light_source_features"] = round(light_analysis["score"], 2)

        # Color atmosphere analysis
        atmosphere_analysis = self._analyze_color_atmosphere(features, strong_sky_signal, diagnostics)
        visual_score += atmosphere_analysis["score"]
        if atmosphere_analysis["score"] != 0:
            contributions["warm_atmosphere_indoor_visual_contrib"] = round(atmosphere_analysis["score"], 2)

        # Home environment pattern analysis
        home_analysis = self._analyze_home_environment_pattern(features, strong_sky_signal, diagnostics)
        visual_score += home_analysis["score"]
        if home_analysis["score"] != 0:
            contributions["home_environment_pattern_visual"] = round(home_analysis["score"], 2)

        # Aerial street pattern analysis
        aerial_analysis = self._analyze_aerial_street_pattern(features, strong_sky_signal, contributions, diagnostics)
        visual_score += aerial_analysis["score"]
        if aerial_analysis["score"] != 0:
            contributions["aerial_street_pattern_visual"] = round(aerial_analysis["score"], 2)

        diagnostics["visual_indoor_score_subtotal"] = round(visual_score, 3)

        return {
            "visual_score": visual_score,
            "contributions": contributions,
            "strong_sky_signal": strong_sky_signal
        }

    def _analyze_sky_evidence(self, features: Dict[str, Any],
                             diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze sky-related evidence for outdoor classification."""
        sky_evidence_score = 0.0
        strong_sky_signal = False

        # Extract relevant features
        sky_blue_dominance = features.get("sky_region_blue_dominance", 0.0)
        sky_brightness_ratio = features.get("sky_region_brightness_ratio", 1.0)
        texture_complexity = features.get("top_region_texture_complexity", 0.5)
        openness_top_edge = features.get("openness_top_edge", 0.5)

        # Get thresholds
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors

        # Strong blue sky signal
        if sky_blue_dominance > thresholds.sky_blue_dominance_thresh:
            sky_evidence_score -= weights.sky_blue_dominance_w * sky_blue_dominance
            diagnostics["sky_detection_reason_visual"] = f"Visual: Strong sky-like blue ({sky_blue_dominance:.2f})"
            strong_sky_signal = True

        # Bright top region with low texture
        elif (sky_brightness_ratio > getattr(thresholds, 'sky_brightness_ratio_strong_thresh', 1.35) and
              texture_complexity < getattr(thresholds, 'sky_texture_complexity_clear_thresh', 0.25)):
            outdoor_push = weights.sky_brightness_ratio_w * (sky_brightness_ratio - 1.0)
            sky_evidence_score -= outdoor_push
            sky_evidence_score -= weights.sky_texture_w
            diagnostics["sky_detection_reason_visual"] = (
                f"Visual: Top brighter (ratio:{sky_brightness_ratio:.2f}) & low texture."
            )
            strong_sky_signal = True

        # High top edge openness
        elif openness_top_edge > getattr(thresholds, 'openness_top_strong_thresh', 0.80):
            sky_evidence_score -= weights.openness_top_w * openness_top_edge
            diagnostics["sky_detection_reason_visual"] = (
                f"Visual: Very high top edge openness ({openness_top_edge:.2f})."
            )
            strong_sky_signal = True

        # Weak sky signal (cloudy conditions)
        elif (not strong_sky_signal and
              texture_complexity < getattr(thresholds, 'sky_texture_complexity_cloudy_thresh', 0.20) and
              sky_brightness_ratio > getattr(thresholds, 'sky_brightness_ratio_cloudy_thresh', 0.95)):
            sky_evidence_score -= weights.sky_texture_w * (1.0 - texture_complexity) * 0.5
            diagnostics["sky_detection_reason_visual"] = (
                f"Visual: Weak sky signal (low texture, brightish top: {texture_complexity:.2f}), less weight."
            )

        if strong_sky_signal:
            diagnostics["strong_sky_signal_visual_detected"] = True

        return {
            "score": sky_evidence_score,
            "strong_signal": strong_sky_signal
        }

    def _analyze_enclosure_evidence(self, features: Dict[str, Any], strong_sky_signal: bool,
                                   diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze enclosure evidence for indoor classification."""
        enclosure_score = 0.0

        # Extract features
        ceiling_likelihood = features.get("ceiling_likelihood", 0.0)
        boundary_clarity = features.get("boundary_clarity", 0.0)
        texture_complexity = features.get("top_region_texture_complexity", 0.5)
        openness_top_edge = features.get("openness_top_edge", 0.5)

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors
        override_factors = self.config_manager.override_factors

        # Ceiling likelihood analysis
        if ceiling_likelihood > thresholds.ceiling_likelihood_thresh:
            current_ceiling_score = weights.ceiling_likelihood_w * ceiling_likelihood
            if strong_sky_signal:
                current_ceiling_score *= override_factors.sky_override_factor_ceiling
            enclosure_score += current_ceiling_score
            diagnostics["indoor_reason_ceiling_visual"] = (
                f"Visual Ceiling: {ceiling_likelihood:.2f}, ScoreCont: {current_ceiling_score:.2f}"
            )

        # Boundary clarity analysis
        if boundary_clarity > thresholds.boundary_clarity_thresh:
            current_boundary_score = weights.boundary_clarity_w * boundary_clarity
            if strong_sky_signal:
                current_boundary_score *= override_factors.sky_override_factor_boundary
            enclosure_score += current_boundary_score
            diagnostics["indoor_reason_boundary_visual"] = (
                f"Visual Boundary: {boundary_clarity:.2f}, ScoreCont: {current_boundary_score:.2f}"
            )

        # Complex urban top detection
        if (not strong_sky_signal and texture_complexity > 0.7 and
            openness_top_edge < 0.3 and ceiling_likelihood < 0.35):
            diagnostics["complex_urban_top_visual"] = True
            if boundary_clarity > 0.5:
                enclosure_score *= 0.5
                diagnostics["reduced_enclosure_for_urban_top_visual"] = True

        return {"score": enclosure_score}

    def _analyze_brightness_uniformity(self, features: Dict[str, Any], strong_sky_signal: bool,
                                      diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze brightness uniformity patterns."""
        uniformity_score = 0.0

        # Calculate brightness uniformity
        brightness_std = features.get("brightness_std", 50.0)
        avg_brightness = features.get("avg_brightness", 100.0)
        brightness_uniformity = 1.0 - min(1.0, brightness_std / max(avg_brightness, 1e-5))
        shadow_clarity = features.get("shadow_clarity_score", 0.5)

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors
        override_factors = self.config_manager.override_factors

        # High uniformity (indoor indicator)
        if brightness_uniformity > thresholds.brightness_uniformity_thresh_indoor:
            uniformity_score = weights.brightness_uniformity_w * brightness_uniformity
            if strong_sky_signal:
                uniformity_score *= override_factors.sky_override_factor_uniformity

        # Low uniformity (potential outdoor indicator)
        elif brightness_uniformity < thresholds.brightness_uniformity_thresh_outdoor:
            if shadow_clarity > 0.65:
                uniformity_score = -weights.brightness_non_uniformity_outdoor_w * (1.0 - brightness_uniformity)
            elif not strong_sky_signal:
                uniformity_score = weights.brightness_non_uniformity_indoor_penalty_w * (1.0 - brightness_uniformity)

        return {"score": uniformity_score}

    def _analyze_light_sources(self, features: Dict[str, Any], strong_sky_signal: bool,
                              diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze artificial light source patterns."""
        light_score = 0.0

        # Extract light features
        indoor_light_score = features.get("indoor_light_score", 0.0)
        circular_light_count = features.get("circular_light_count", 0)
        bright_spot_count = features.get("bright_spot_count", 0)
        avg_brightness = features.get("avg_brightness", 100.0)
        gradient_ratio = features.get("gradient_ratio_vertical_horizontal", 1.0)
        edges_density = features.get("edges_density", 0.0)

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors
        override_factors = self.config_manager.override_factors

        # Circular lights detection
        if circular_light_count >= 1 and not strong_sky_signal:
            light_score += weights.circular_lights_w * circular_light_count

        # Indoor light score
        elif indoor_light_score > 0.55 and not strong_sky_signal:
            light_score += weights.indoor_light_score_w * indoor_light_score

        # Many bright spots in dim scenes
        elif (bright_spot_count > thresholds.many_bright_spots_thresh and
              avg_brightness < thresholds.dim_scene_for_spots_thresh and
              not strong_sky_signal):
            light_score += weights.many_bright_spots_indoor_w * min(bright_spot_count / 10.0, 1.5)

        # Street structure detection
        is_likely_street_structure = (0.7 < gradient_ratio < 1.5) and edges_density > 0.15

        if is_likely_street_structure and bright_spot_count > 3 and not strong_sky_signal:
            light_score *= 0.2
            diagnostics["street_lights_heuristic_visual"] = True
        elif strong_sky_signal:
            light_score *= override_factors.sky_override_factor_lights

        return {"score": light_score}

    def _analyze_color_atmosphere(self, features: Dict[str, Any], strong_sky_signal: bool,
                                 diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze color atmosphere patterns."""
        atmosphere_score = 0.0

        # Extract features
        color_atmosphere = features.get("color_atmosphere", "neutral")
        avg_brightness = features.get("avg_brightness", 100.0)
        avg_saturation = features.get("avg_saturation", 100.0)
        gradient_ratio = features.get("gradient_ratio_vertical_horizontal", 1.0)
        edges_density = features.get("edges_density", 0.0)
        indoor_light_score = features.get("indoor_light_score", 0.0)

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors

        # Warm atmosphere analysis
        if (color_atmosphere == "warm" and
            avg_brightness < thresholds.warm_indoor_max_brightness_thresh):

            # Check exclusion conditions
            is_likely_street_structure = (0.7 < gradient_ratio < 1.5) and edges_density > 0.15
            is_complex_urban_top = diagnostics.get("complex_urban_top_visual", False)

            if (not strong_sky_signal and not is_complex_urban_top and
                not (is_likely_street_structure and avg_brightness > 80) and
                avg_saturation < 160):

                if indoor_light_score > 0.05:
                    atmosphere_score = weights.warm_atmosphere_indoor_w

        return {"score": atmosphere_score}

    def _analyze_home_environment_pattern(self, features: Dict[str, Any], strong_sky_signal: bool,
                                         diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze home/residential environment patterns."""
        home_score = 0.0

        if strong_sky_signal:
            diagnostics["skipped_home_env_visual_due_to_sky"] = True
            return {"score": 0.0}

        # Calculate bedroom/home indicators
        bedroom_indicators = 0.0
        brightness_uniformity = features.get("brightness_uniformity", 0.0)
        boundary_clarity = features.get("boundary_clarity", 0.0)
        ceiling_likelihood = features.get("ceiling_likelihood", 0.0)
        bright_spot_count = features.get("bright_spot_count", 0)
        circular_light_count = features.get("circular_light_count", 0)
        warm_ratio = features.get("warm_ratio", 0.0)
        avg_saturation = features.get("avg_saturation", 100.0)

        # Accumulate indicators
        if brightness_uniformity > 0.65 and boundary_clarity > 0.40:
            bedroom_indicators += 1.1

        if ceiling_likelihood > 0.35 and (bright_spot_count > 0 or circular_light_count > 0):
            bedroom_indicators += 1.1

        if warm_ratio > 0.55 and brightness_uniformity > 0.65:
            bedroom_indicators += 1.0

        if brightness_uniformity > 0.70 and avg_saturation < 60:
            bedroom_indicators += 0.7

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors

        # Apply scoring based on indicator strength
        if bedroom_indicators >= thresholds.home_pattern_thresh_strong:
            home_score = weights.home_env_strong_w
        elif bedroom_indicators >= thresholds.home_pattern_thresh_moderate:
            home_score = weights.home_env_moderate_w

        if bedroom_indicators > 0:
            diagnostics["home_environment_pattern_visual_indicators"] = round(bedroom_indicators, 1)

        return {"score": home_score}

    def _analyze_aerial_street_pattern(self, features: Dict[str, Any], strong_sky_signal: bool,
                                      contributions: Dict[str, float],
                                      diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze aerial view street patterns."""
        aerial_score = 0.0

        # Extract features
        sky_brightness_ratio = features.get("sky_region_brightness_ratio", 1.0)
        texture_complexity = features.get("top_region_texture_complexity", 0.5)
        avg_brightness = features.get("avg_brightness", 100.0)

        # Get configuration
        thresholds = self.config_manager.indoor_outdoor_thresholds
        weights = self.config_manager.weighting_factors

        # Aerial street pattern detection
        if (sky_brightness_ratio < thresholds.aerial_top_dark_ratio_thresh and
            texture_complexity > thresholds.aerial_top_complex_thresh and
            avg_brightness > thresholds.aerial_min_avg_brightness_thresh and
            not strong_sky_signal):

            aerial_score = -weights.aerial_street_w
            diagnostics["aerial_street_pattern_visual_detected"] = True

            # Reduce enclosure features if aerial pattern detected
            if ("enclosure_features" in contributions and
                contributions["enclosure_features"] > 0):

                reduction_factor = self.config_manager.override_factors.aerial_enclosure_reduction_factor
                positive_enclosure_score = max(0, contributions["enclosure_features"])
                reduction_amount = positive_enclosure_score * reduction_factor

                contributions["enclosure_features_reduced_by_aerial"] = round(-reduction_amount, 2)
                contributions["enclosure_features"] = round(
                    contributions["enclosure_features"] - reduction_amount, 2
                )

        return {"score": aerial_score}

    def _analyze_places365_influence(self, p365_context: Dict[str, Any],
                                    strong_sky_signal: bool,
                                    diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze Places365 influence on classification."""
        p365_influence_score = 0.0

        if not p365_context or p365_context["confidence"] < self.P365_MODERATE_CONF_THRESHOLD:
            return {"influence_score": 0.0}

        # Places365 direct classification influence
        if p365_context["is_indoor_from_classification"] is not None:
            p365_influence_score += self._compute_direct_classification_influence(
                p365_context, strong_sky_signal, diagnostics
            )

        # Places365 scene context influence
        elif p365_context["confidence"] >= self.P365_MODERATE_CONF_THRESHOLD:
            p365_influence_score += self._compute_scene_context_influence(
                p365_context, strong_sky_signal, diagnostics
            )

        # Places365 attributes influence
        if p365_context["attributes"] and p365_context["confidence"] > 0.5:
            p365_influence_score += self._compute_attributes_influence(
                p365_context, strong_sky_signal, diagnostics
            )

        # High confidence street scene boost
        if (p365_context["confidence"] >= 0.85 and
            any(kw in p365_context["mapped_scene"] for kw in ["intersection", "crosswalk", "street", "road"])):

            additional_outdoor_push = -3.0 * p365_context["confidence"]
            p365_influence_score += additional_outdoor_push
            diagnostics["p365_street_scene_boost"] = (
                f"Additional outdoor push: {additional_outdoor_push:.2f} for street scene: "
                f"{p365_context['mapped_scene']}"
            )
            self.logger.debug(f"High confidence street scene detected - "
                            f"{p365_context['mapped_scene']} with confidence {p365_context['confidence']:.3f}")

        return {"influence_score": p365_influence_score}

    def _compute_direct_classification_influence(self, p365_context: Dict[str, Any],
                                               strong_sky_signal: bool,
                                               diagnostics: Dict[str, Any]) -> float:
        """Compute influence from Places365 direct indoor/outdoor classification."""
        P365_DIRECT_INDOOR_WEIGHT = 3.5
        P365_DIRECT_OUTDOOR_WEIGHT = 4.0

        confidence = p365_context["confidence"]
        is_indoor = p365_context["is_indoor_from_classification"]
        mapped_scene = p365_context["mapped_scene"]

        if is_indoor is True:
            current_contrib = P365_DIRECT_INDOOR_WEIGHT * confidence
            diagnostics["p365_influence_source"] = (
                f"P365_DirectIndoor(True,Conf:{confidence:.2f},Scene:{mapped_scene})"
            )
        else:
            current_contrib = -P365_DIRECT_OUTDOOR_WEIGHT * confidence
            diagnostics["p365_influence_source"] = (
                f"P365_DirectIndoor(False,Conf:{confidence:.2f},Scene:{mapped_scene})"
            )

        # Apply sky override for indoor predictions
        if strong_sky_signal and current_contrib > 0:
            sky_override_factor = self.config_manager.override_factors.sky_override_factor_p365_indoor_decision
            current_contrib *= sky_override_factor
            diagnostics["p365_indoor_push_reduced_by_visual_sky"] = f"Reduced to {current_contrib:.2f}"

        return current_contrib

    def _compute_scene_context_influence(self, p365_context: Dict[str, Any],
                                        strong_sky_signal: bool,
                                        diagnostics: Dict[str, Any]) -> float:
        """Compute influence from Places365 scene context."""
        P365_SCENE_CONTEXT_INDOOR_WEIGHT = 2.0
        P365_SCENE_CONTEXT_OUTDOOR_WEIGHT = 2.5

        confidence = p365_context["confidence"]
        mapped_scene = p365_context["mapped_scene"]

        is_def_indoor = any(kw in mapped_scene for kw in self.DEFINITELY_INDOOR_KEYWORDS_P365)
        is_def_outdoor = any(kw in mapped_scene for kw in self.DEFINITELY_OUTDOOR_KEYWORDS_P365)

        current_contrib = 0.0

        if is_def_indoor and not is_def_outdoor:
            current_contrib = P365_SCENE_CONTEXT_INDOOR_WEIGHT * confidence
            diagnostics["p365_influence_source"] = (
                f"P365_SceneContext(Indoor: {mapped_scene}, Conf:{confidence:.2f})"
            )
        elif is_def_outdoor and not is_def_indoor:
            current_contrib = -P365_SCENE_CONTEXT_OUTDOOR_WEIGHT * confidence
            diagnostics["p365_influence_source"] = (
                f"P365_SceneContext(Outdoor: {mapped_scene}, Conf:{confidence:.2f})"
            )

        # Apply sky override for indoor predictions
        if strong_sky_signal and current_contrib > 0:
            sky_override_factor = self.config_manager.override_factors.sky_override_factor_p365_indoor_decision
            current_contrib *= sky_override_factor
            diagnostics["p365_context_indoor_push_reduced_by_visual_sky"] = f"Reduced to {current_contrib:.2f}"

        return current_contrib

    def _compute_attributes_influence(self, p365_context: Dict[str, Any],
                                     strong_sky_signal: bool,
                                     diagnostics: Dict[str, Any]) -> float:
        """Compute influence from Places365 attributes."""
        P365_ATTRIBUTE_INDOOR_WEIGHT = 1.0
        P365_ATTRIBUTE_OUTDOOR_WEIGHT = 1.5

        confidence = p365_context["confidence"]
        attributes = p365_context["attributes"]

        attr_contrib = 0.0

        if "indoor" in attributes and "outdoor" not in attributes:
            attr_contrib += P365_ATTRIBUTE_INDOOR_WEIGHT * (confidence * 0.5)
            diagnostics["p365_attr_influence"] = f"+{attr_contrib:.2f} (indoor attr)"
        elif "outdoor" in attributes and "indoor" not in attributes:
            attr_contrib -= P365_ATTRIBUTE_OUTDOOR_WEIGHT * (confidence * 0.5)
            diagnostics["p365_attr_influence"] = f"{attr_contrib:.2f} (outdoor attr)"

        # Apply sky override for indoor attributes
        if strong_sky_signal and attr_contrib > 0:
            sky_override_factor = self.config_manager.override_factors.sky_override_factor_p365_indoor_decision
            attr_contrib *= sky_override_factor

        return attr_contrib

    def _compute_final_classification(self, final_indoor_score: float, visual_score: float,
                                     p365_influence_score: float, diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Compute final classification probability and decision."""
        # Record score breakdown
        diagnostics["final_indoor_score_value"] = round(final_indoor_score, 3)
        diagnostics["final_score_breakdown"] = (
            f"VisualScore: {visual_score:.2f}, P365Influence: {p365_influence_score:.2f}"
        )

        # Apply sigmoid transformation
        sigmoid_scale = self.config_manager.algorithm_parameters.indoor_score_sigmoid_scale
        indoor_probability = 1 / (1 + np.exp(-final_indoor_score * sigmoid_scale))

        # Make decision
        decision_threshold = self.config_manager.algorithm_parameters.indoor_decision_threshold
        is_indoor = indoor_probability > decision_threshold

        return {
            "is_indoor": is_indoor,
            "indoor_probability": indoor_probability,
            "final_score": final_indoor_score
        }

    def _apply_places365_override(self, classification_result: Dict[str, Any],
                                 p365_context: Dict[str, Any],
                                 diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Apply Places365 high-confidence override if conditions are met."""
        is_indoor = classification_result["is_indoor"]
        indoor_probability = classification_result["indoor_probability"]
        final_score = classification_result["final_score"]

        # Check for override conditions
        if not p365_context or p365_context["confidence"] < 0.5:
            diagnostics["final_indoor_probability_calculated"] = round(indoor_probability, 3)
            diagnostics["final_is_indoor_decision"] = bool(is_indoor)
            return classification_result

        p365_is_indoor_decision = p365_context.get("is_indoor", None)
        confidence = p365_context["confidence"]

        self.logger.debug(f"Override check: is_indoor={is_indoor}, p365_conf={confidence}, "
                         f"p365_raw_is_indoor={p365_is_indoor_decision}")

        # Apply override for high confidence Places365 decisions
        if p365_is_indoor_decision is not None:
            if p365_is_indoor_decision == False:
                self.logger.debug(f"Applying outdoor override. Original: {is_indoor}")
                original_decision = f"Indoor:{is_indoor}, Prob:{indoor_probability:.3f}, Score:{final_score:.2f}"

                is_indoor = False
                indoor_probability = 0.02
                final_score = -8.0

                diagnostics["p365_force_override_applied"] = (
                    f"P365 FORCED OUTDOOR (is_indoor: {p365_is_indoor_decision}, Conf: {confidence:.3f})"
                )
                diagnostics["p365_override_original_decision"] = original_decision
                self.logger.info(f"Places365 FORCED OUTDOOR override applied. New is_indoor: {is_indoor}")

            elif p365_is_indoor_decision == True:
                self.logger.debug(f"Applying indoor override. Original: {is_indoor}")
                original_decision = f"Indoor:{is_indoor}, Prob:{indoor_probability:.3f}, Score:{final_score:.2f}"

                is_indoor = True
                indoor_probability = 0.98
                final_score = 8.0

                diagnostics["p365_force_override_applied"] = (
                    f"P365 FORCED INDOOR (is_indoor: {p365_is_indoor_decision}, Conf: {confidence:.3f})"
                )
                diagnostics["p365_override_original_decision"] = original_decision
                self.logger.info(f"Places365 FORCED INDOOR override applied. New is_indoor: {is_indoor}")

        # Record final values
        diagnostics["final_indoor_probability_calculated"] = round(indoor_probability, 3)
        diagnostics["final_is_indoor_decision"] = bool(is_indoor)

        self.logger.debug(f"Final classification: is_indoor={is_indoor}, score={final_score}, prob={indoor_probability}")

        return {
            "is_indoor": is_indoor,
            "indoor_probability": indoor_probability,
            "final_score": final_score
        }

    def _ensure_default_contributions(self, feature_contributions: Dict[str, float]) -> None:
        """Ensure all expected feature contribution keys have default values."""
        default_keys = [
            "sky_openness_features", "enclosure_features",
            "brightness_uniformity_contribution", "light_source_features"
        ]

        for key in default_keys:
            if key not in feature_contributions:
                feature_contributions[key] = 0.0

    def _get_default_classification_result(self) -> Dict[str, Any]:
        """Return default classification result in case of errors."""
        return {
            "is_indoor": False,
            "indoor_probability": 0.5,
            "indoor_score_raw": 0.0,
            "feature_contributions": {
                "sky_openness_features": 0.0,
                "enclosure_features": 0.0,
                "brightness_uniformity_contribution": 0.0,
                "light_source_features": 0.0
            },
            "diagnostics": {
                "error": "Classification failed, using default values"
            }
        }

Writing indoor_outdoor_classifier.py


In [ ]:
# %%writefile lighting_condition_analyzer.py
import numpy as np
import logging
import traceback
from typing import Dict, Any, Optional, List, Tuple
# from configuration_manager import ConfigurationManager


class LightingConditionAnalyzer:
    """
    Determines specific lighting conditions and time of day based on scene analysis.
    此class 會判斷一些光線的特定場景

    This class analyzes lighting characteristics including natural and artificial illumination,
    color temperature patterns, and temporal indicators to classify scenes into specific
    lighting categories such as day clear, night with lights, indoor artificial, etc.
    """

    def __init__(self, config_manager: ConfigurationManager):
        """
        Initialize the lighting condition analyzer.

        Args:
            config_manager: Configuration manager instance for accessing thresholds and parameters.
        """
        self.config_manager = config_manager
        self.logger = self._setup_logger()

        # Internal threshold constants for Places365 analysis
        self.P365_ATTRIBUTE_CONF_THRESHOLD = 0.60
        self.P365_SCENE_MODERATE_CONF_THRESHOLD = 0.45
        self.P365_SCENE_HIGH_CONF_THRESHOLD = 0.70

        # Scene type keyword definitions
        self.P365_OUTDOOR_SCENE_KEYWORDS = [
            "street", "road", "highway", "park", "beach", "mountain", "forest", "field",
            "outdoor", "sky", "coast", "courtyard", "square", "plaza", "bridge",
            "parking", "playground", "stadium", "construction", "river", "ocean", "desert",
            "garden", "trail", "natural_landmark", "airport_outdoor", "train_station_outdoor",
            "bus_station_outdoor", "intersection", "crosswalk", "sidewalk", "pathway"
        ]

        self.P365_INDOOR_RESTAURANT_KEYWORDS = [
            "restaurant", "bar", "cafe", "dining_room", "pub", "bistro", "eatery"
        ]

    def _setup_logger(self) -> logging.Logger:
        """Set up logger for lighting condition analysis operations."""
        logger = logging.getLogger(f"{__name__}.LightingConditionAnalyzer")
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        return logger

    def analyze_lighting_conditions(self, features: Dict[str, Any], is_indoor: bool,
                                   places365_info: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Determine specific lighting conditions based on features and scene context.

        Args:
            features: Dictionary containing extracted image features.
            is_indoor: Boolean indicating whether the scene is indoor (from previous classification).
            places365_info: Optional Places365 classification information.

        Returns:
            Dictionary containing lighting analysis results including time_of_day, confidence,
            and diagnostic information.
        """
        try:
            self.logger.debug(f"Starting lighting analysis for {'indoor' if is_indoor else 'outdoor'} scene")

            # Initialize analysis results
            time_of_day = "unknown"
            confidence = 0.5
            diagnostics = {}

            # Extract Places365 context
            p365_context = self._extract_places365_context(places365_info, diagnostics)

            # Priority 1: Use Places365 attributes if highly confident
            attribute_result = self._analyze_places365_attributes(
                p365_context, is_indoor, features, diagnostics
            )

            if attribute_result["determined"] and attribute_result["confidence"] >= 0.75:
                self.logger.debug(f"High-confidence Places365 attribute determination: {attribute_result['time_of_day']}")
                return {
                    "time_of_day": attribute_result["time_of_day"],
                    "confidence": attribute_result["confidence"],
                    "diagnostics": diagnostics
                }

            # Priority 2: Visual feature analysis with Places365 scene context
            visual_result = self._analyze_visual_features(
                features, is_indoor, p365_context, diagnostics
            )

            time_of_day = visual_result["time_of_day"]
            confidence = visual_result["confidence"]

            # Combine with attribute result if it exists but wasn't decisive
            if attribute_result["determined"]:
                combined_result = self._combine_attribute_and_visual_results(
                    attribute_result, visual_result, diagnostics
                )
                time_of_day = combined_result["time_of_day"]
                confidence = combined_result["confidence"]

            # Priority 3: Special lighting refinement (neon, sodium vapor)
            refined_result = self._apply_special_lighting_refinement(
                time_of_day, confidence, features, is_indoor, p365_context, diagnostics
            )

            time_of_day = refined_result["time_of_day"]
            confidence = refined_result["confidence"]

            # Final confidence clamping
            confidence = min(0.95, max(0.50, confidence))

            # Record final results
            diagnostics["final_lighting_time_of_day"] = time_of_day
            diagnostics["final_lighting_confidence"] = round(confidence, 3)

            self.logger.debug(f"Lighting analysis complete: {time_of_day} (confidence: {confidence:.3f})")

            return {
                "time_of_day": time_of_day,
                "confidence": confidence,
                "diagnostics": diagnostics
            }

        except Exception as e:
            self.logger.error(f"Error in lighting condition analysis: {str(e)}")
            self.logger.error(f"Traceback: {traceback.format_exc()}")
            return self._get_default_lighting_result()

    def _extract_places365_context(self, places365_info: Optional[Dict],
                                  diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Extract and validate Places365 context information for lighting analysis."""
        context = {
            "mapped_scene": "unknown",
            "attributes": [],
            "confidence": 0.0
        }

        if places365_info:
            context["mapped_scene"] = places365_info.get('mapped_scene_type', 'unknown').lower()
            context["attributes"] = [attr.lower() for attr in places365_info.get('attributes', [])]
            context["confidence"] = places365_info.get('confidence', 0.0)

            diagnostics["p365_context_for_lighting"] = (
                f"P365 Scene: {context['mapped_scene']}, Attrs: {context['attributes']}, "
                f"Conf: {context['confidence']:.2f}"
            )

        return context

    def _analyze_places365_attributes(self, p365_context: Dict[str, Any], is_indoor: bool,
                                     features: Dict[str, Any], diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze Places365 attributes for lighting condition determination."""
        if (not p365_context["attributes"] or
            p365_context["confidence"] <= self.P365_ATTRIBUTE_CONF_THRESHOLD):
            return {"determined": False, "time_of_day": "unknown", "confidence": 0.5}

        confidence = p365_context["confidence"]
        attributes = p365_context["attributes"]
        mapped_scene = p365_context["mapped_scene"]

        # Outdoor attribute analysis
        if not is_indoor:
            outdoor_result = self._analyze_outdoor_attributes(
                attributes, mapped_scene, confidence, diagnostics
            )
            if outdoor_result["determined"]:
                return outdoor_result

        # Indoor attribute analysis
        if is_indoor:
            indoor_result = self._analyze_indoor_attributes(
                attributes, mapped_scene, features, confidence, diagnostics
            )
            if indoor_result["determined"]:
                return indoor_result

        return {"determined": False, "time_of_day": "unknown", "confidence": 0.5}

    def _analyze_outdoor_attributes(self, attributes: List[str], mapped_scene: str,
                                   confidence: float, diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze Places365 attributes for outdoor lighting conditions."""
        base_confidence_boost = (confidence - self.P365_ATTRIBUTE_CONF_THRESHOLD) * 0.25

        if "sunny" in attributes or "clear sky" in attributes:
            final_confidence = 0.85 + base_confidence_boost
            diagnostics["reason"] = "P365 attribute: sunny/clear sky (Outdoor)."
            return {
                "determined": True,
                "time_of_day": "day_clear",
                "confidence": final_confidence
            }

        elif "nighttime" in attributes or "night" in attributes:
            if ("artificial lighting" in attributes or "man-made lighting" in attributes or
                any(kw in mapped_scene for kw in ["street", "city", "road", "urban", "downtown"])):
                final_confidence = 0.82 + base_confidence_boost * 0.8
                diagnostics["reason"] = "P365 attribute: nighttime with artificial/street lights (Outdoor)."
                return {
                    "determined": True,
                    "time_of_day": "night_with_lights",
                    "confidence": final_confidence
                }
            else:
                final_confidence = 0.78 + base_confidence_boost * 0.8
                diagnostics["reason"] = "P365 attribute: nighttime, dark (Outdoor)."
                return {
                    "determined": True,
                    "time_of_day": "night_dark",
                    "confidence": final_confidence
                }

        elif "cloudy" in attributes or "overcast" in attributes:
            final_confidence = 0.80 + base_confidence_boost
            diagnostics["reason"] = "P365 attribute: cloudy/overcast (Outdoor)."
            return {
                "determined": True,
                "time_of_day": "day_cloudy_overcast",
                "confidence": final_confidence
            }

        return {"determined": False, "time_of_day": "unknown", "confidence": 0.5}

    def _analyze_indoor_attributes(self, attributes: List[str], mapped_scene: str,
                                  features: Dict[str, Any], confidence: float,
                                  diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze Places365 attributes for indoor lighting conditions."""
        base_confidence_boost = (confidence - self.P365_ATTRIBUTE_CONF_THRESHOLD) * 0.20
        avg_brightness = features.get("avg_brightness", 128.0)

        if "artificial lighting" in attributes or "man-made lighting" in attributes:
            base_indoor_conf = 0.70 + base_confidence_boost
            thresholds = self.config_manager.lighting_thresholds

            if avg_brightness > thresholds.indoor_bright_thresh:
                time_of_day = "indoor_bright_artificial"
                final_confidence = base_indoor_conf + 0.10
            elif avg_brightness > thresholds.indoor_moderate_thresh:
                time_of_day = "indoor_moderate_artificial"
                final_confidence = base_indoor_conf
            else:
                time_of_day = "indoor_dim_artificial"
                final_confidence = base_indoor_conf - 0.05

            diagnostics["reason"] = (
                f"P365 attribute: artificial lighting (Indoor), "
                f"brightness based category: {time_of_day}."
            )
            return {
                "determined": True,
                "time_of_day": time_of_day,
                "confidence": final_confidence
            }

        elif "natural lighting" in attributes:
            is_applicable_scene = (
                self._check_home_environment_pattern(features) or
                any(kw in mapped_scene for kw in ["living_room", "bedroom", "sunroom"])
            )
            if is_applicable_scene:
                final_confidence = 0.80 + base_confidence_boost
                diagnostics["reason"] = "P365 attribute: natural lighting in residential/applicable indoor scene."
                return {
                    "determined": True,
                    "time_of_day": "indoor_residential_natural",
                    "confidence": final_confidence
                }

        return {"determined": False, "time_of_day": "unknown", "confidence": 0.5}

    def _analyze_visual_features(self, features: Dict[str, Any], is_indoor: bool,
                                p365_context: Dict[str, Any], diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze visual features for lighting condition determination."""
        if is_indoor:
            return self._analyze_indoor_visual_features(features, p365_context, diagnostics)
        else:
            return self._analyze_outdoor_visual_features(features, p365_context, diagnostics)

    def _analyze_indoor_visual_features(self, features: Dict[str, Any], p365_context: Dict[str, Any],
                                       diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze visual features for indoor lighting conditions."""
        avg_brightness = features.get("avg_brightness", 128.0)
        thresholds = self.config_manager.lighting_thresholds

        # Extract relevant features
        sky_blue_in_sky_region = features.get("sky_region_blue_dominance", 0.0)
        sky_region_is_brighter = features.get("sky_region_brightness_ratio", 1.0) > 1.05
        is_likely_home_environment = self._check_home_environment_pattern(features)

        # Lighting and structural features
        circular_lights = features.get("circular_light_count", 0)
        bright_spots_overall = features.get("bright_spot_count", 0)
        brightness_uniformity = features.get("brightness_uniformity", 0.0)
        warm_ratio = features.get("warm_ratio", 0.0)

        # Natural light hints calculation
        natural_light_hints = 0.0
        if sky_blue_in_sky_region > 0.05 and sky_region_is_brighter:
            natural_light_hints += 1.0
        if brightness_uniformity > 0.65 and features.get("brightness_std", 100.0) < 70:
            natural_light_hints += 1.0
        if warm_ratio > 0.15 and avg_brightness > 110:
            natural_light_hints += 0.5

        # Designer lighting detection
        is_designer_lit = (
            (circular_lights > 0 or bright_spots_overall > 2) and
            brightness_uniformity > 0.6 and warm_ratio > 0.2 and avg_brightness > 90
        )

        # Brightness-based classification
        if avg_brightness > thresholds.indoor_bright_thresh:
            return self._classify_bright_indoor(
                features, natural_light_hints, is_designer_lit, is_likely_home_environment,
                p365_context, diagnostics
            )
        elif avg_brightness > thresholds.indoor_moderate_thresh:
            return self._classify_moderate_indoor(
                features, is_designer_lit, is_likely_home_environment, p365_context, diagnostics
            )
        else:
            return self._classify_dim_indoor(features, diagnostics)

    def _classify_bright_indoor(self, features: Dict[str, Any], natural_light_hints: float,
                               is_designer_lit: bool, is_likely_home_environment: bool,
                               p365_context: Dict[str, Any], diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify bright indoor lighting conditions."""
        mapped_scene = p365_context["mapped_scene"]
        sky_blue_in_sky_region = features.get("sky_region_blue_dominance", 0.0)
        sky_region_is_brighter = features.get("sky_region_brightness_ratio", 1.0) > 1.05

        # Natural residential lighting
        if (natural_light_hints >= 1.5 and
            (is_likely_home_environment or any(kw in mapped_scene for kw in ["home", "residential", "living", "bedroom"]))):
            return {
                "time_of_day": "indoor_residential_natural",
                "confidence": 0.82
            }

        # Designer residential lighting
        elif (is_designer_lit and
              (is_likely_home_environment or any(kw in mapped_scene for kw in ["home", "designer", "modern_interior"]))):
            return {
                "time_of_day": "indoor_designer_residential",
                "confidence": 0.85
            }

        # Mixed natural/artificial lighting
        elif sky_blue_in_sky_region > 0.03 and sky_region_is_brighter:
            return {
                "time_of_day": "indoor_bright_natural_mix",
                "confidence": 0.78
            }

        # Pure artificial lighting
        else:
            return {
                "time_of_day": "indoor_bright_artificial",
                "confidence": 0.75
            }

    def _classify_moderate_indoor(self, features: Dict[str, Any], is_designer_lit: bool,
                                 is_likely_home_environment: bool, p365_context: Dict[str, Any],
                                 diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify moderate brightness indoor lighting conditions."""
        mapped_scene = p365_context["mapped_scene"]
        confidence = p365_context["confidence"]
        warm_ratio = features.get("warm_ratio", 0.0)
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)

        # Designer residential lighting
        if (is_designer_lit and
            (is_likely_home_environment or any(kw in mapped_scene for kw in ["home", "designer"]))):
            return {
                "time_of_day": "indoor_designer_residential",
                "confidence": 0.78
            }

        # Restaurant/bar lighting
        elif warm_ratio > 0.35 and yellow_orange_ratio > 0.1:
            return self._classify_restaurant_bar_lighting(
                p365_context, features, diagnostics
            )

        # Standard moderate artificial
        else:
            return {
                "time_of_day": "indoor_moderate_artificial",
                "confidence": 0.70
            }

    def _classify_restaurant_bar_lighting(self, p365_context: Dict[str, Any],
                                         features: Dict[str, Any],
                                         diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify restaurant/bar specific lighting conditions."""
        mapped_scene = p365_context["mapped_scene"]
        confidence = p365_context["confidence"]

        # Strong P365 restaurant/bar confirmation
        if (any(kw in mapped_scene for kw in self.P365_INDOOR_RESTAURANT_KEYWORDS) and
            confidence > self.P365_SCENE_MODERATE_CONF_THRESHOLD):
            diagnostics["visual_analysis_reason"] = (
                "Visual: Moderate warm tones. P365 context confirms restaurant/bar."
            )
            return {
                "time_of_day": "indoor_restaurant_bar",
                "confidence": 0.80 + confidence * 0.15
            }

        # P365 outdoor conflict detection
        elif (any(kw in mapped_scene for kw in self.P365_OUTDOOR_SCENE_KEYWORDS) and
              confidence > self.P365_SCENE_MODERATE_CONF_THRESHOLD):
            diagnostics["visual_analysis_reason"] = (
                "Visual: Moderate warm. CONFLICT: LA says indoor but P365 scene is outdoor. "
                "Defaulting to general indoor artificial."
            )
            diagnostics["conflict_is_indoor_vs_p365_scene_for_restaurant_bar"] = True
            return {
                "time_of_day": "indoor_moderate_artificial",
                "confidence": 0.55
            }

        # Neutral P365 context
        else:
            diagnostics["visual_analysis_reason"] = (
                "Visual: Moderate warm tones, typical of restaurant/bar. P365 context neutral or weak."
            )
            return {
                "time_of_day": "indoor_restaurant_bar",
                "confidence": 0.70
            }

    def _classify_dim_indoor(self, features: Dict[str, Any],
                            diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify dim indoor lighting conditions."""
        warm_ratio = features.get("warm_ratio", 0.0)
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)

        if warm_ratio > 0.45 and yellow_orange_ratio > 0.15:
            return {
                "time_of_day": "indoor_dim_warm",
                "confidence": 0.75
            }
        else:
            return {
                "time_of_day": "indoor_dim_general",
                "confidence": 0.70
            }

    def _analyze_outdoor_visual_features(self, features: Dict[str, Any], p365_context: Dict[str, Any],
                                        diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze visual features for outdoor lighting conditions."""
        avg_brightness = features.get("avg_brightness", 128.0)
        thresholds = self.config_manager.lighting_thresholds

        # P365 enhanced street scene analysis
        street_result = self._analyze_p365_enhanced_street_scenes(
            features, p365_context, diagnostics
        )
        if street_result["determined"]:
            return street_result

        # Brightness-based outdoor classification
        if avg_brightness < thresholds.outdoor_night_thresh_brightness:
            return self._classify_night_outdoor(features, diagnostics)
        elif (avg_brightness < thresholds.outdoor_dusk_dawn_thresh_brightness and
              self._check_warm_sunset_conditions(features)):
            return self._classify_sunset_sunrise(features, p365_context, diagnostics)
        elif avg_brightness > thresholds.outdoor_day_bright_thresh:
            return self._classify_bright_day_outdoor(features, diagnostics)
        elif avg_brightness > thresholds.outdoor_day_cloudy_thresh:
            return self._classify_cloudy_day_outdoor(features, diagnostics)
        else:
            return self._classify_general_outdoor(features, diagnostics)

    def _analyze_p365_enhanced_street_scenes(self, features: Dict[str, Any], p365_context: Dict[str, Any],
                                            diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze outdoor scenes with Places365 street context enhancement."""
        mapped_scene = p365_context["mapped_scene"]
        confidence = p365_context["confidence"]
        thresholds = self.config_manager.lighting_thresholds

        # Check for street scene with warm lighting
        is_street_scene = (
            any(kw in mapped_scene for kw in ["street", "city", "road", "urban", "downtown", "intersection"]) and
            confidence > self.P365_SCENE_MODERATE_CONF_THRESHOLD and
            features.get("color_atmosphere") == "warm"
        )

        if not is_street_scene:
            return {"determined": False, "time_of_day": "unknown", "confidence": 0.5}

        avg_brightness = features.get("avg_brightness", 128.0)
        bright_spots_overall = features.get("bright_spot_count", 0)

        # Night with street lights
        if (avg_brightness < thresholds.outdoor_night_thresh_brightness and
            bright_spots_overall > thresholds.outdoor_night_lights_thresh):
            diagnostics["visual_analysis_reason"] = (
                f"P365 outdoor scene '{mapped_scene}' + visual low-warm light with spots -> night_with_lights."
            )
            return {
                "determined": True,
                "time_of_day": "night_with_lights",
                "confidence": 0.88 + confidence * 0.1
            }

        # Sunset/sunrise conditions
        elif avg_brightness >= thresholds.outdoor_night_thresh_brightness:
            diagnostics["visual_analysis_reason"] = (
                f"P365 outdoor scene '{mapped_scene}' + visual moderate-warm light -> sunset/sunrise."
            )
            return {
                "determined": True,
                "time_of_day": "sunset_sunrise",
                "confidence": 0.88 + confidence * 0.1
            }

        # Very dark conditions
        else:
            diagnostics["visual_analysis_reason"] = (
                f"P365 outdoor scene '{mapped_scene}' + visual very low light -> night_dark."
            )
            return {
                "determined": True,
                "time_of_day": "night_dark",
                "confidence": 0.75 + confidence * 0.1
            }

    def _classify_night_outdoor(self, features: Dict[str, Any],
                               diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify nighttime outdoor conditions."""
        bright_spots_overall = features.get("bright_spot_count", 0)
        dark_pixel_ratio = features.get("dark_pixel_ratio", 0.0)
        thresholds = self.config_manager.lighting_thresholds

        if bright_spots_overall > thresholds.outdoor_night_lights_thresh:
            confidence = 0.82 + min(0.13, dark_pixel_ratio / 2.5)
            diagnostics["visual_analysis_reason"] = "Visual: Low brightness with light sources (street/car lights)."
            return {
                "time_of_day": "night_with_lights",
                "confidence": confidence
            }
        else:
            confidence = 0.78 + min(0.17, dark_pixel_ratio / 1.8)
            diagnostics["visual_analysis_reason"] = "Visual: Very low brightness outdoor, deep night."
            return {
                "time_of_day": "night_dark",
                "confidence": confidence
            }

    def _classify_sunset_sunrise(self, features: Dict[str, Any], p365_context: Dict[str, Any],
                                diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify sunset/sunrise outdoor conditions."""
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)
        confidence = 0.75 + min(0.20, yellow_orange_ratio / 1.5)

        diagnostics["visual_analysis_reason"] = "Visual: Moderate brightness, warm tones -> sunset/sunrise."

        # P365 natural scene boost
        mapped_scene = p365_context["mapped_scene"]
        p365_confidence = p365_context["confidence"]

        if (any(kw in mapped_scene for kw in ["beach", "mountain", "lake", "ocean", "desert", "field", "natural_landmark", "sky"]) and
            p365_confidence > self.P365_SCENE_MODERATE_CONF_THRESHOLD):
            confidence = min(0.95, confidence + 0.15)
            diagnostics["visual_analysis_reason"] += f" P365 natural scene '{mapped_scene}' supports."

        return {
            "time_of_day": "sunset_sunrise",
            "confidence": confidence
        }

    def _classify_bright_day_outdoor(self, features: Dict[str, Any],
                                    diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify bright daytime outdoor conditions."""
        sky_like_blue_in_sky_region = features.get("sky_region_blue_dominance", 0.0)
        sky_region_brightness_ratio = features.get("sky_region_brightness_ratio", 1.0)
        texture_complexity = features.get("top_region_texture_complexity", 0.5)
        thresholds = self.config_manager.lighting_thresholds

        # Clear sky conditions
        if (sky_like_blue_in_sky_region > thresholds.outdoor_day_blue_thresh or
            (sky_region_brightness_ratio > 1.05 and texture_complexity < 0.4)):

            confidence = 0.80 + min(0.15, sky_like_blue_in_sky_region * 2 +
                                  (sky_like_blue_in_sky_region * 1.5 if sky_region_brightness_ratio > 1.05 else 0))
            diagnostics["visual_analysis_reason"] = "Visual: High brightness with blue/sky tones or bright smooth top."

            return {
                "time_of_day": "day_clear",
                "confidence": confidence
            }

        # Stadium/floodlit detection
        brightness_uniformity = features.get("brightness_uniformity", 0.0)
        bright_spots_overall = features.get("bright_spot_count", 0)

        if (brightness_uniformity > 0.70 and
            bright_spots_overall > thresholds.stadium_min_spots_thresh):
            diagnostics["visual_analysis_reason"] = (
                "Visual: Very bright, uniform lighting with multiple sources, suggests floodlights (Outdoor)."
            )
            return {
                "time_of_day": "stadium_or_floodlit_area",
                "confidence": 0.78
            }

        # General bright day
        diagnostics["visual_analysis_reason"] = "Visual: High brightness outdoor, specific sky features unclear."
        return {
            "time_of_day": "day_bright_general",
            "confidence": 0.68
        }

    def _classify_cloudy_day_outdoor(self, features: Dict[str, Any],
                                    diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify cloudy daytime outdoor conditions."""
        sky_region_brightness_ratio = features.get("sky_region_brightness_ratio", 1.0)
        texture_complexity = features.get("top_region_texture_complexity", 0.5)
        avg_saturation = features.get("avg_saturation", 100.0)
        gray_ratio = features.get("gray_ratio", 0.0)
        brightness_uniformity = features.get("brightness_uniformity", 0.0)
        thresholds = self.config_manager.lighting_thresholds

        # Overcast conditions
        if (sky_region_brightness_ratio > 1.05 and texture_complexity < 0.45 and avg_saturation < 70):
            confidence = 0.75 + min(0.20, gray_ratio / 1.5 + (brightness_uniformity - 0.5) / 1.5)
            diagnostics["visual_analysis_reason"] = (
                "Visual: Good brightness, uniform bright top, lower saturation -> overcast."
            )
            return {
                "time_of_day": "day_cloudy_overcast",
                "confidence": confidence
            }

        # Gray cloudy conditions
        elif gray_ratio > thresholds.outdoor_day_gray_thresh:
            confidence = 0.72 + min(0.23, gray_ratio / 1.8)
            diagnostics["visual_analysis_reason"] = "Visual: Good brightness with higher gray tones."
            return {
                "time_of_day": "day_cloudy_gray",
                "confidence": confidence
            }

        # General bright outdoor
        else:
            diagnostics["visual_analysis_reason"] = "Visual: Bright outdoor, specific type less clear."
            return {
                "time_of_day": "day_bright_general",
                "confidence": 0.68
            }

    def _classify_general_outdoor(self, features: Dict[str, Any],
                                 diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Classify general outdoor conditions when specific patterns are unclear."""
        color_atmosphere = features.get("color_atmosphere", "neutral")
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)
        sky_like_blue_in_sky_region = features.get("sky_region_blue_dominance", 0.0)

        # Potential sunset/sunrise with low confidence
        if color_atmosphere == "warm" and yellow_orange_ratio > 0.08:
            diagnostics["visual_analysis_reason"] = (
                "Visual: Outdoor, specific conditions less clear; broader visual cues suggest warm lighting."
            )
            return {
                "time_of_day": "sunset_sunrise_low_confidence",
                "confidence": 0.62
            }

        # Potential hazy day conditions
        elif sky_like_blue_in_sky_region > 0.02:
            diagnostics["visual_analysis_reason"] = (
                "Visual: Outdoor, specific conditions less clear; some blue tones suggest daylight."
            )
            return {
                "time_of_day": "day_hazy_or_partly_cloudy",
                "confidence": 0.62
            }

        # Unknown outdoor daylight
        else:
            diagnostics["visual_analysis_reason"] = (
                "Visual: Outdoor, specific conditions less clear; broader visual cues."
            )
            return {
                "time_of_day": "outdoor_unknown_daylight",
                "confidence": 0.58
            }

    def _apply_commercial_indoor_refinement(self, features: Dict[str, Any], p365_context: Dict[str, Any],
                                           time_of_day: str, confidence: float) -> Dict[str, Any]:
        """Apply commercial indoor lighting refinement if conditions are met."""
        # Skip if already classified as residential, restaurant, or bar
        if any(category in time_of_day for category in ["residential", "restaurant", "bar"]):
            return {"time_of_day": time_of_day, "confidence": confidence}

        # Skip if P365 suggests home environment
        mapped_scene = p365_context["mapped_scene"]
        if any(kw in mapped_scene for kw in ["home", "residential"]):
            return {"time_of_day": time_of_day, "confidence": confidence}

        # Check commercial lighting indicators
        avg_brightness = features.get("avg_brightness", 100.0)
        bright_spots_overall = features.get("bright_spot_count", 0)
        light_dist_uniformity = features.get("light_distribution_uniformity", 0.5)
        ceiling_likelihood = features.get("ceiling_likelihood", 0.0)
        thresholds = self.config_manager.lighting_thresholds

        if (avg_brightness > thresholds.commercial_min_brightness_thresh and
            bright_spots_overall > thresholds.commercial_min_spots_thresh and
            (light_dist_uniformity > 0.5 or ceiling_likelihood > 0.4)):

            refined_confidence = 0.70 + min(0.2, bright_spots_overall * 0.02)
            return {
                "time_of_day": "indoor_commercial",
                "confidence": refined_confidence
            }

        return {"time_of_day": time_of_day, "confidence": confidence}

    def _apply_special_lighting_refinement(self, time_of_day: str, confidence: float,
                                          features: Dict[str, Any], is_indoor: bool,
                                          p365_context: Dict[str, Any],
                                          diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Apply special lighting refinement for neon and sodium vapor lighting."""
        # Apply commercial refinement for indoor scenes first
        if is_indoor:
            commercial_result = self._apply_commercial_indoor_refinement(
                features, p365_context, time_of_day, confidence
            )
            time_of_day = commercial_result["time_of_day"]
            confidence = commercial_result["confidence"]

        # Check for neon/sodium vapor lighting conditions
        is_current_night_or_dim_warm = "night" in time_of_day or time_of_day == "indoor_dim_warm"

        if not is_current_night_or_dim_warm:
            return {"time_of_day": time_of_day, "confidence": confidence}

        # Extract features for neon detection
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)
        bright_spots_overall = features.get("bright_spot_count", 0)
        color_atmosphere = features.get("color_atmosphere", "neutral")
        avg_saturation = features.get("avg_saturation", 0.0)

        # Get neon detection thresholds
        thresholds = self.config_manager.lighting_thresholds

        # Check neon lighting conditions
        if (yellow_orange_ratio > thresholds.neon_yellow_orange_thresh and
            bright_spots_overall > thresholds.neon_bright_spots_thresh and
            color_atmosphere == "warm" and
            avg_saturation > thresholds.neon_avg_saturation_thresh):

            old_time_of_day = time_of_day
            old_confidence = confidence

            # Check P365 context for neon scenes
            mapped_scene = p365_context["mapped_scene"]
            attributes = p365_context["attributes"]
            is_p365_neon_context = (
                any(kw in mapped_scene for kw in ["neon", "nightclub", "bar_neon"]) or
                "neon" in attributes
            )

            if is_indoor:
                if (is_p365_neon_context or
                    any(kw in mapped_scene for kw in self.P365_INDOOR_RESTAURANT_KEYWORDS)):
                    time_of_day = "indoor_neon_lit"
                    confidence = max(confidence, 0.80)
                else:
                    time_of_day = "indoor_dim_warm_neon_accent"
                    confidence = max(confidence, 0.77)
            else:
                if (is_p365_neon_context or
                    any(kw in mapped_scene for kw in ["street_night", "city_night", "downtown_night"])):
                    time_of_day = "neon_or_sodium_vapor_night"
                    confidence = max(confidence, 0.82)
                else:
                    time_of_day = "night_with_neon_lights"
                    confidence = max(confidence, 0.79)

            # Record the refinement
            diagnostics["special_lighting_detected"] = (
                f"Refined from {old_time_of_day} (Conf:{old_confidence:.2f}) "
                f"to {time_of_day} (Conf:{confidence:.2f}) due to neon/sodium vapor light characteristics. "
                f"P365 Context: {mapped_scene if is_p365_neon_context else 'N/A'}."
            )

        return {"time_of_day": time_of_day, "confidence": confidence}

    def _combine_attribute_and_visual_results(self, attribute_result: Dict[str, Any],
                                             visual_result: Dict[str, Any],
                                             diagnostics: Dict[str, Any]) -> Dict[str, Any]:
        """Combine Places365 attribute and visual analysis results."""
        # If visual analysis provided a different and potentially more nuanced result
        if (attribute_result["time_of_day"] != visual_result["time_of_day"] and
            visual_result["confidence"] > 0.65):

            diagnostics["final_decision_source"] = "Visual features (potentially P365-context-refined)."
            diagnostics["p365_attr_overridden_by_visual"] = (
                f"P365 Attr ToD {attribute_result['time_of_day']} "
                f"(Conf {attribute_result['confidence']:.2f}) was less certain or overridden by "
                f"visual logic result {visual_result['time_of_day']} (Conf {visual_result['confidence']:.2f})."
            )
            return visual_result

        # Use attribute result if it was more confident
        elif attribute_result["confidence"] >= visual_result["confidence"]:
            diagnostics["final_decision_source"] = "High-confidence P365 attribute."
            return attribute_result

        # Use visual result
        else:
            diagnostics["final_decision_source"] = "Visual features (potentially P365-context-refined)."
            return visual_result

    def _check_home_environment_pattern(self, features: Dict[str, Any]) -> bool:
        """Check if features indicate a home/residential environment pattern."""
        thresholds = self.config_manager.indoor_outdoor_thresholds
        return features.get("home_environment_pattern", 0.0) > thresholds.home_pattern_thresh_moderate * 0.7

    def _check_warm_sunset_conditions(self, features: Dict[str, Any]) -> bool:
        """Check if features indicate warm sunset/sunrise lighting conditions."""
        thresholds = self.config_manager.lighting_thresholds
        yellow_orange_ratio = features.get("yellow_orange_ratio", 0.0)
        color_atmosphere = features.get("color_atmosphere", "neutral")
        sky_brightness_ratio = features.get("sky_region_brightness_ratio", 1.0)

        return (yellow_orange_ratio > thresholds.outdoor_dusk_dawn_color_thresh and
                color_atmosphere == "warm" and
                sky_brightness_ratio < 1.5)

    def _get_default_lighting_result(self) -> Dict[str, Any]:
        """Return default lighting analysis result in case of errors."""
        return {
            "time_of_day": "unknown",
            "confidence": 0.5,
            "diagnostics": {
                "error": "Lighting analysis failed, using default values"
            }
        }

Writing lighting_condition_analyzer.py


In [ ]:
# %%writefile lighting_analyzer.py
import numpy as np
import cv2
import logging
import traceback
from typing import Dict, Any, Optional

# from configuration_manager import ConfigurationManager
# from feature_extractor import FeatureExtractor
# from indoor_outdoor_classifier import IndoorOutdoorClassifier
# from lighting_condition_analyzer import LightingConditionAnalyzer


class LightingAnalyzer:
    """
    Comprehensive lighting analysis system facade that coordinates feature extraction,
    indoor/outdoor classification, and lighting condition determination.
    此class是一個總窗口，主要匯總各式光線分析相關的class

    This facade class maintains the original interface while internally delegating
    work to specialized components for improved maintainability and modularity.
    """

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """
        Initialize the lighting analyzer with configuration.

        Args:
            config: Optional configuration dictionary. If None, uses default configuration.
        """
        self.logger = self._setup_logger()

        try:
            # Initialize configuration manager
            self.config_manager = ConfigurationManager()

            # Override default configuration if provided
            if config is not None:
                self._update_configuration(config)

            # Initialize specialized components
            self.feature_extractor = FeatureExtractor(self.config_manager)
            self.indoor_outdoor_classifier = IndoorOutdoorClassifier(self.config_manager)
            self.lighting_condition_analyzer = LightingConditionAnalyzer(self.config_manager)

            # Legacy configuration access for backward compatibility
            self.config = self.config_manager.get_legacy_config_dict()

            self.logger.info("LightingAnalyzer initialized successfully")

        except Exception as e:
            self.logger.error(f"Error initializing LightingAnalyzer: {str(e)}")
            self.logger.error(f"Traceback: {traceback.format_exc()}")
            raise

    def _setup_logger(self) -> logging.Logger:
        """Set up logger for lighting analysis operations."""
        logger = logging.getLogger(f"{__name__}.LightingAnalyzer")
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)
            logger.setLevel(logging.INFO)
        return logger

    def _update_configuration(self, config: Dict[str, Any]) -> None:
        """
        Update configuration manager with provided configuration dictionary.

        Args:
            config: Configuration dictionary to update existing configuration.
        """
        try:
            # Update configuration through the manager's internal method
            self.config_manager._update_from_dict(config)
            self.logger.debug("Configuration updated successfully")

        except Exception as e:
            self.logger.warning(f"Error updating configuration: {str(e)}")
            self.logger.warning("Continuing with default configuration")

    def analyze(self, image, places365_info: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Analyze lighting conditions of an image.

        This is the main entry point that maintains compatibility with the original interface
        while leveraging the new modular architecture internally.

        Args:
            image: Input image (numpy array or PIL Image).
            places365_info: Optional Places365 classification information containing
                          scene type, confidence, attributes, and indoor/outdoor classification.

        Returns:
            Dictionary containing comprehensive lighting analysis results including:
            - time_of_day: Specific lighting condition classification
            - confidence: Confidence score for the classification
            - is_indoor: Boolean indicating indoor/outdoor classification
            - indoor_probability: Probability score for indoor classification
            - brightness: Brightness analysis metrics
            - color_info: Color characteristic analysis
            - texture_info: Texture and gradient analysis
            - structure_info: Structural feature analysis
            - diagnostics: Detailed diagnostic information (if enabled)
        """
        try:
            self.logger.debug("Starting comprehensive lighting analysis")

            # Step 1: Validate and preprocess input image
            processed_image = self._preprocess_image(image)
            if processed_image is None:
                return self._get_error_result("Invalid image input")

            # Step 2: Extract comprehensive features
            self.logger.debug("Extracting image features")
            features = self.feature_extractor.extract_features(processed_image)

            if not features or "avg_brightness" not in features:
                return self._get_error_result("Feature extraction failed")

            # Step 3: Classify indoor/outdoor with Places365 integration
            self.logger.debug("Performing indoor/outdoor classification")
            indoor_outdoor_result = self.indoor_outdoor_classifier.classify(
                features, places365_info
            )

            is_indoor = indoor_outdoor_result["is_indoor"]
            indoor_probability = indoor_outdoor_result["indoor_probability"]

            # Step 4: Determine specific lighting conditions
            self.logger.debug(f"Analyzing lighting conditions for {'indoor' if is_indoor else 'outdoor'} scene")
            lighting_result = self.lighting_condition_analyzer.analyze_lighting_conditions(
                features, is_indoor, places365_info
            )

            # Step 5: Consolidate comprehensive results
            result = self._consolidate_analysis_results(
                lighting_result, indoor_outdoor_result, features
            )

            self.logger.info(f"Analysis complete: {result['time_of_day']} "
                           f"({'indoor' if result['is_indoor'] else 'outdoor'}) "
                           f"confidence: {result['confidence']:.3f}")

            return result

        except Exception as e:
            self.logger.error(f"Error in lighting analysis: {str(e)}")
            self.logger.error(f"Traceback: {traceback.format_exc()}")
            return self._get_error_result(str(e))

    def _preprocess_image(self, image) -> Optional[np.ndarray]:
        """
        Preprocess input image to ensure consistent format for analysis.

        Args:
            image: Input image in various possible formats.

        Returns:
            Preprocessed image as RGB numpy array, or None if preprocessing failed.
        """
        try:
            # Convert to numpy array if needed
            if not isinstance(image, np.ndarray):
                image_np = np.array(image)
            else:
                image_np = image.copy()

            # Validate basic image properties
            if len(image_np.shape) < 2:
                self.logger.error("Image must have at least 2 dimensions")
                return None

            height, width = image_np.shape[:2]
            if height == 0 or width == 0:
                self.logger.error(f"Invalid image dimensions: {height}x{width}")
                return None

            # Handle different color formats and convert to RGB
            if len(image_np.shape) == 2:
                # 灰階 to RGB
                image_rgb = cv2.cvtColor(image_np, cv2.COLOR_GRAY2RGB)
            elif image_np.shape[2] == 3:
                # Handle BGR vs RGB
                if not isinstance(image, np.ndarray):
                    # PIL images are typically RGB
                    image_rgb = image_np
                else:
                    # OpenCV arrays are typically BGR, convert to RGB
                    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            elif image_np.shape[2] == 4:
                # RGBA to RGB
                if not isinstance(image, np.ndarray):
                    # PIL RGBA to RGB
                    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
                else:
                    # OpenCV BGRA to RGB
                    image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGRA2RGB)
            else:
                self.logger.error(f"Unsupported image format with shape: {image_np.shape}")
                return None

            # Ensure uint8 data type
            if image_rgb.dtype != np.uint8:
                if image_rgb.dtype in [np.float32, np.float64]:
                    # Assume normalized float values
                    if image_rgb.max() <= 1.0:
                        image_rgb = (image_rgb * 255).astype(np.uint8)
                    else:
                        image_rgb = image_rgb.astype(np.uint8)
                else:
                    image_rgb = image_rgb.astype(np.uint8)

            self.logger.debug(f"Preprocessed image: {image_rgb.shape}, dtype: {image_rgb.dtype}")
            return image_rgb

        except Exception as e:
            self.logger.error(f"Error preprocessing image: {str(e)}")
            return None

    def _consolidate_analysis_results(self, lighting_result: Dict[str, Any],
                                     indoor_outdoor_result: Dict[str, Any],
                                     features: Dict[str, Any]) -> Dict[str, Any]:
        """
        Consolidate results from all analysis components into final output format.

        Args:
            lighting_result: Results from lighting condition analysis.
            indoor_outdoor_result: Results from indoor/outdoor classification.
            features: Extracted image features.

        Returns:
            Consolidated analysis results in the expected output format.
        """
        # Extract core results
        time_of_day = lighting_result["time_of_day"]
        confidence = lighting_result["confidence"]
        is_indoor = indoor_outdoor_result["is_indoor"]
        indoor_probability = indoor_outdoor_result["indoor_probability"]

        # Organize brightness information
        brightness_info = {
            "average": float(features.get("avg_brightness", 0.0)),
            "std_dev": float(features.get("brightness_std", 0.0)),
            "dark_ratio": float(features.get("dark_pixel_ratio", 0.0)),
            "bright_ratio": float(features.get("bright_pixel_ratio", 0.0))
        }

        # Organize color information
        color_info = {
            "blue_ratio": float(features.get("blue_ratio", 0.0)),
            "sky_like_blue_ratio": float(features.get("sky_like_blue_ratio", 0.0)),
            "yellow_orange_ratio": float(features.get("yellow_orange_ratio", 0.0)),
            "gray_ratio": float(features.get("gray_ratio", 0.0)),
            "avg_saturation": float(features.get("avg_saturation", 0.0)),
            "sky_region_brightness_ratio": float(features.get("sky_region_brightness_ratio", 1.0)),
            "sky_region_saturation": float(features.get("sky_region_saturation", 0.0)),
            "sky_region_blue_dominance": float(features.get("sky_region_blue_dominance", 0.0)),
            "color_atmosphere": features.get("color_atmosphere", "neutral"),
            "warm_ratio": float(features.get("warm_ratio", 0.0)),
            "cool_ratio": float(features.get("cool_ratio", 0.0))
        }

        # Organize texture information
        texture_info = {
            "gradient_ratio_vertical_horizontal": float(features.get("gradient_ratio_vertical_horizontal", 0.0)),
            "top_region_texture_complexity": float(features.get("top_region_texture_complexity", 0.0)),
            "shadow_clarity_score": float(features.get("shadow_clarity_score", 0.5))
        }

        # Organize structure information
        structure_info = {
            "ceiling_likelihood": float(features.get("ceiling_likelihood", 0.0)),
            "boundary_clarity": float(features.get("boundary_clarity", 0.0)),
            "openness_top_edge": float(features.get("openness_top_edge", 0.5))
        }

        # Compile final result
        result = {
            "time_of_day": time_of_day,
            "confidence": float(confidence),
            "is_indoor": is_indoor,
            "indoor_probability": float(indoor_probability),
            "brightness": brightness_info,
            "color_info": color_info,
            "texture_info": texture_info,
            "structure_info": structure_info
        }

        # Add diagnostic information if enabled
        if self.config_manager.algorithm_parameters.include_diagnostics:
            diagnostics = {}

            # Combine diagnostics from all components
            if "diagnostics" in lighting_result:
                diagnostics["lighting_diagnostics"] = lighting_result["diagnostics"]

            if "diagnostics" in indoor_outdoor_result:
                diagnostics["indoor_outdoor_diagnostics"] = indoor_outdoor_result["diagnostics"]

            if "feature_contributions" in indoor_outdoor_result:
                diagnostics["feature_contributions"] = indoor_outdoor_result["feature_contributions"]

            result["diagnostics"] = diagnostics

        return result

    def _get_error_result(self, error_message: str) -> Dict[str, Any]:
        """
        Generate standardized error result format.

        Args:
            error_message: Description of the error that occurred.

        Returns:
            Dictionary containing error result with safe default values.
        """
        return {
            "time_of_day": "unknown",
            "confidence": 0.0,
            "is_indoor": False,
            "indoor_probability": 0.5,
            "brightness": {
                "average": 100.0,
                "std_dev": 50.0,
                "dark_ratio": 0.0,
                "bright_ratio": 0.0
            },
            "color_info": {
                "blue_ratio": 0.0,
                "sky_like_blue_ratio": 0.0,
                "yellow_orange_ratio": 0.0,
                "gray_ratio": 0.0,
                "avg_saturation": 100.0,
                "sky_region_brightness_ratio": 1.0,
                "sky_region_saturation": 0.0,
                "sky_region_blue_dominance": 0.0,
                "color_atmosphere": "neutral",
                "warm_ratio": 0.0,
                "cool_ratio": 0.0
            },
            "texture_info": {
                "gradient_ratio_vertical_horizontal": 1.0,
                "top_region_texture_complexity": 0.5,
                "shadow_clarity_score": 0.5
            },
            "structure_info": {
                "ceiling_likelihood": 0.0,
                "boundary_clarity": 0.0,
                "openness_top_edge": 0.5
            },
            "error": error_message
        }

    def get_configuration(self) -> Dict[str, Any]:
        """
        Get current configuration as dictionary for backward compatibility.

        Returns:
            Dictionary containing all current configuration parameters.
        """
        return self.config_manager.get_legacy_config_dict()

    def update_configuration(self, config_updates: Dict[str, Any]) -> None:
        """
        Update configuration parameters.

        Args:
            config_updates: Dictionary containing configuration parameters to update.
        """
        try:
            self.config_manager._update_from_dict(config_updates)
            # Update legacy config reference
            self.config = self.config_manager.get_legacy_config_dict()
            self.logger.info("Configuration updated successfully")

        except Exception as e:
            self.logger.error(f"Error updating configuration: {str(e)}")
            raise

    def validate_configuration(self) -> bool:
        """
        Validate current configuration for logical consistency.

        Returns:
            True if configuration is valid, False otherwise.
        """
        try:
            validation_errors = self.config_manager.validate_configuration()

            if validation_errors:
                self.logger.error("Configuration validation failed:")
                for error in validation_errors:
                    self.logger.error(f"  - {error}")
                return False

            self.logger.info("Configuration validation passed")
            return True

        except Exception as e:
            self.logger.error(f"Error during configuration validation: {str(e)}")
            return False

    def save_configuration(self, filepath: str) -> None:
        """
        Save current configuration to file.

        Args:
            filepath: Path where to save the configuration file.
        """
        try:
            self.config_manager.save_to_file(filepath)
            self.logger.info(f"Configuration saved to {filepath}")

        except Exception as e:
            self.logger.error(f"Error saving configuration: {str(e)}")
            raise

    def load_configuration(self, filepath: str) -> None:
        """
        Load configuration from file.

        Args:
            filepath: Path to the configuration file to load.
        """
        try:
            self.config_manager.load_from_file(filepath)
            # Update legacy config reference
            self.config = self.config_manager.get_legacy_config_dict()
            self.logger.info(f"Configuration loaded from {filepath}")

        except Exception as e:
            self.logger.error(f"Error loading configuration: {str(e)}")
            raise

Writing lighting_analyzer.py


In [ ]:
# %%writefile scene_description.py
import os
import json
from typing import Dict, List, Tuple, Any, Optional

# from scene_type import SCENE_TYPES
# from scene_detail_templates import SCENE_DETAIL_TEMPLATES
# from object_template_fillers import OBJECT_TEMPLATE_FILLERS
# from activity_templates import ACTIVITY_TEMPLATES
# from safety_templates import SAFETY_TEMPLATES
# from confidence_templates import CONFIDENCE_TEMPLATES

class SceneDescriptor:
    """
    Generates natural language descriptions of scenes.
    Handles scene descriptions, activity inference, and safety concerns identification.
    """

    def __init__(self, scene_types=None, object_categories=None):
        """
        Initialize the scene descriptor

        Args:
            scene_types: Dictionary of scene type definitions
        """
        self.scene_types = scene_types or {}
        self.SCENE_TYPES = scene_types or {}

        if object_categories:
            self.OBJECT_CATEGORIES = object_categories
        else:
            # 從 JSON 加載或使用默認值
            self.OBJECT_CATEGORIES = self._load_json_data("object_categories") or {
                "furniture": [56, 57, 58, 59, 60, 61],
                "electronics": [62, 63, 64, 65, 66, 67, 68, 69, 70],
                "kitchen_items": [39, 40, 41, 42, 43, 44, 45],
                "food": [46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
                "vehicles": [1, 2, 3, 4, 5, 6, 7, 8],
                "personal_items": [24, 25, 26, 27, 28, 73, 78, 79]
            }

        # 加載所有模板數據
        self._load_templates()

    def _load_templates(self):
        """Load all template data from script or fallback to imported defaults"""
        self.confidence_templates = CONFIDENCE_TEMPLATES
        self.scene_detail_templates = SCENE_DETAIL_TEMPLATES
        self.object_template_fillers = OBJECT_TEMPLATE_FILLERS
        self.safety_templates = SAFETY_TEMPLATES
        self.activity_templates = ACTIVITY_TEMPLATES


    def _initialize_fallback_templates(self):
        """Initialize fallback templates when no external data is available"""
        # 只在無法從文件或導入加載時使用
        self.confidence_templates = {
            "high": "{description} {details}",
            "medium": "This appears to be {description} {details}",
            "low": "This might be {description}, but the confidence is low. {details}"
        }

        # 只提供最基本的模板作為後備
        self.scene_detail_templates = {
            "default": ["A space with various objects."]
        }

        self.object_template_fillers = {
            "default": ["various items"]
        }

        self.safety_templates = {
            "general": "Pay attention to {safety_element}."
        }

        self.activity_templates = {
            "default": ["General activity"]
        }

    def _get_alternative_scenes(self, scene_scores: Dict[str, float],
                            threshold: float, top_k: int = 2) -> List[Dict]:
        """
        Get alternative scene interpretations with their scores.

        Args:
            scene_scores: Dictionary of scene type scores
            threshold: Minimum confidence threshold
            top_k: Number of alternatives to return

        Returns:
            List of dictionaries with alternative scenes
        """
        # Sort scenes by score in descending order
        sorted_scenes = sorted(scene_scores.items(), key=lambda x: x[1], reverse=True)

        # Skip the first one (best match) and take the next top_k
        alternatives = []
        for scene_type, score in sorted_scenes[1:1+top_k]:
            if score >= threshold:
                alternatives.append({
                    "type": scene_type,
                    "name": self.SCENE_TYPES.get(scene_type, {}).get("name", "Unknown"),
                    "confidence": score
                })

        return alternatives


    def _infer_possible_activities(self, scene_type: str, detected_objects: List[Dict], enable_landmark: bool = True, scene_scores: Optional[Dict] = None) -> List[str]:
        """
        Infer possible activities based on scene type and detected objects.

        Args:
            scene_type: Identified scene type
            detected_objects: List of detected objects
            enable_landmark: Whether landmark detection is enabled
            scene_scores: Optional dictionary of scene type scores

        Returns:
            List of possible activities
        """
        activities = []

        # Dynamically replace landmark scene types when landmark detection is disabled
        if not enable_landmark and scene_type in ["tourist_landmark", "natural_landmark", "historical_monument"]:
            alternative_scene_type = self._get_alternative_scene_type(scene_type, detected_objects, scene_scores)
            print(f"Replacing landmark scene type '{scene_type}' with '{alternative_scene_type}' for activity inference")
            scene_type = alternative_scene_type

        # Process aerial view scenes
        if scene_type.startswith("aerial_view_"):
            if scene_type == "aerial_view_intersection":
                # Use predefined intersection activities
                activities.extend(self.activity_templates.get("aerial_view_intersection", []))

                # Add pedestrian and vehicle specific activities
                pedestrians = [obj for obj in detected_objects if obj["class_id"] == 0]
                vehicles = [obj for obj in detected_objects if obj["class_id"] in [2, 5, 7]]  # Car, bus, truck

                if pedestrians and vehicles:
                    activities.append("Waiting for an opportunity to cross the street")
                    activities.append("Obeying traffic signals")

            elif scene_type == "aerial_view_commercial_area":
                activities.extend(self.activity_templates.get("aerial_view_commercial_area", []))

            elif scene_type == "aerial_view_plaza":
                activities.extend(self.activity_templates.get("aerial_view_plaza", []))

            else:
                # Handle other undefined aerial view scenes
                aerial_activities = [
                    "Street crossing",
                    "Waiting for signals",
                    "Following traffic rules",
                    "Pedestrian movement"
                ]
                activities.extend(aerial_activities)

        # Add scene-specific activities from templates
        if scene_type in self.activity_templates:
            activities.extend(self.activity_templates[scene_type])
        elif "default" in self.activity_templates:
            activities.extend(self.activity_templates["default"])

        # Filter out landmark-related activities when landmark detection is disabled
        if not enable_landmark:
            filtered_activities = []
            landmark_keywords = ["sightseeing", "landmark", "tourist", "monument", "historical",
                                "guided tour", "photography", "cultural tourism", "heritage"]

            for activity in activities:
                if not any(keyword in activity.lower() for keyword in landmark_keywords):
                    filtered_activities.append(activity)

            activities = filtered_activities

        # If we filtered out all activities, add some generic ones based on scene type
        if not activities:
            generic_activities = {
                "city_street": ["Walking", "Commuting", "Shopping"],
                "intersection": ["Crossing the street", "Waiting for traffic signals"],
                "commercial_district": ["Shopping", "Walking", "Dining"],
                "pedestrian_area": ["Walking", "Socializing", "Shopping"],
                "park_area": ["Relaxing", "Walking", "Exercise"],
                "outdoor_natural_area": ["Walking", "Nature observation", "Relaxation"],
                "urban_architecture": ["Walking", "Urban exploration", "Photography"]
            }

            activities.extend(generic_activities.get(scene_type, ["Walking", "Observing surroundings"]))

        # Add activities based on detected objects
        detected_class_ids = [obj["class_id"] for obj in detected_objects]

        # Add activities based on specific object combinations
        if 62 in detected_class_ids and 57 in detected_class_ids:  # TV and sofa
            activities.append("Watching shows or movies")

        if 63 in detected_class_ids:  # laptop
            activities.append("Using a computer/laptop")

        if 67 in detected_class_ids:  # cell phone
            activities.append("Using a mobile phone")

        if 73 in detected_class_ids:  # book
            activities.append("Reading")

        if any(food_id in detected_class_ids for food_id in [46, 47, 48, 49, 50, 51, 52, 53, 54, 55]):
            activities.append("Eating or preparing food")

        # Person-specific activities
        if 0 in detected_class_ids:  # Person
            if any(vehicle in detected_class_ids for vehicle in [1, 2, 3, 5, 7]):  # Vehicles
                activities.append("Commuting or traveling")

            if 16 in detected_class_ids:  # Dog
                activities.append("Walking a dog")

            if 24 in detected_class_ids or 26 in detected_class_ids:  # Backpack or handbag
                activities.append("Carrying personal items")

            # Add more person count-dependent activities
            person_count = detected_class_ids.count(0)
            if person_count > 3:
                activities.append("Group gathering")
            elif person_count > 1:
                activities.append("Social interaction")

        # Add additional activities based on significant objects
        if 43 in detected_class_ids:  # cup
            activities.append("Drinking beverages")

        if 32 in detected_class_ids:  # sports ball
            activities.append("Playing sports")

        if 25 in detected_class_ids:  # umbrella
            activities.append("Sheltering from weather")

        # Add location-specific activities based on environment objects
        if any(furniture in detected_class_ids for furniture in [56, 57, 58, 59, 60]):  # furniture items
            activities.append("Using indoor facilities")

        if any(outdoor_item in detected_class_ids for outdoor_item in [13, 14, 15]):  # bench, outdoor items
            activities.append("Enjoying outdoor spaces")

        # Remove duplicates and ensure reasonable number of activities
        unique_activities = list(set(activities))

        # Limit to reasonable number (maximum 8 activities)
        if len(unique_activities) > 8:
            # Prioritize more specific activities over general ones
            general_activities = ["Walking", "Observing surroundings", "Commuting", "Using indoor facilities"]
            specific_activities = [a for a in unique_activities if a not in general_activities]

            # Take all specific activities first, then fill with general ones if needed
            if len(specific_activities) <= 8:
                result = specific_activities + general_activities[:8-len(specific_activities)]
            else:
                result = specific_activities[:8]
        else:
            result = unique_activities

        return result

    def _identify_safety_concerns(self, detected_objects: List[Dict], scene_type: str) -> List[str]:
        """
        Identify potential safety concerns based on objects and scene type.

        Args:
            detected_objects: List of detected objects
            scene_type: Identified scene type

        Returns:
            List of potential safety concerns
        """
        concerns = []
        detected_class_ids = [obj["class_id"] for obj in detected_objects]

        # General safety concerns
        if 42 in detected_class_ids or 43 in detected_class_ids:  # Fork or knife
            concerns.append("Sharp utensils present")

        if 76 in detected_class_ids:  # Scissors
            concerns.append("Cutting tools present")

        # Traffic-related concerns
        if scene_type in ["city_street", "parking_lot"]:
            if 0 in detected_class_ids:  # Person
                if any(vehicle in detected_class_ids for vehicle in [2, 3, 5, 7, 8]):  # Vehicles
                    concerns.append("Pedestrians near vehicles")

            if 9 in detected_class_ids:  # Traffic light
                concerns.append("Monitor traffic signals")

        # Identify crowded scenes
        person_count = detected_class_ids.count(0)
        if person_count > 5:
            concerns.append(f"Crowded area with multiple people ({person_count})")

        # Scene-specific concerns
        if scene_type == "kitchen":
            if 68 in detected_class_ids or 69 in detected_class_ids:  # Microwave or oven
                concerns.append("Hot cooking equipment")

        # Potentially unstable objects
        for obj in detected_objects:
            if obj["class_id"] in [39, 40, 41, 45]:  # Bottle, wine glass, cup, bowl
                if obj["region"] in ["top_left", "top_center", "top_right"] and obj["normalized_area"] > 0.05:
                    concerns.append(f"Elevated {obj['class_name']} might be unstable")

        # Upscale dining safety concerns
        if scene_type == "upscale_dining":
            # Check for fragile items
            if 40 in detected_class_ids:  # Wine glass
                concerns.append("Fragile glassware present")

            # Check for lit candles (can't directly detect but can infer from context)
            # Look for small bright spots that might be candles
            if any(obj["class_id"] == 41 for obj in detected_objects):  # Cup (which might include candle holders)
                # We can't reliably detect candles, but if the scene appears to be formal dining,
                # we can suggest this as a possibility
                concerns.append("Possible lit candles or decorative items requiring care")

            # Check for overcrowded table
            table_objs = [obj for obj in detected_objects if obj["class_id"] == 60]  # Dining table
            if table_objs:
                table_region = table_objs[0]["region"]
                items_on_table = 0

                for obj in detected_objects:
                    if obj["class_id"] in [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]:
                        if obj["region"] == table_region:
                            items_on_table += 1

                if items_on_table > 8:
                    concerns.append("Dining table has multiple items which should be handled with care")

        # Asian commercial street safety concerns
        elif scene_type == "asian_commercial_street":
            # Check for crowded walkways
            if 0 in detected_class_ids:  # Person
                person_count = detected_class_ids.count(0)
                if person_count > 3:
                    # Calculate person density (simplified)
                    person_positions = []
                    for obj in detected_objects:
                        if obj["class_id"] == 0:
                            person_positions.append(obj["normalized_center"])

                    if len(person_positions) >= 2:
                        # Calculate average distance between people
                        total_distance = 0
                        count = 0
                        for i in range(len(person_positions)):
                            for j in range(i+1, len(person_positions)):
                                p1 = person_positions[i]
                                p2 = person_positions[j]
                                distance = ((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)**0.5
                                total_distance += distance
                                count += 1

                        if count > 0:
                            avg_distance = total_distance / count
                            if avg_distance < 0.1:  # Close proximity
                                concerns.append("Crowded walkway with limited personal space")

            # Check for motorcycles/bicycles near pedestrians
            if (1 in detected_class_ids or 3 in detected_class_ids) and 0 in detected_class_ids:  # Bicycle/motorcycle and person
                concerns.append("Two-wheeled vehicles in pedestrian areas")

            # Check for potential trip hazards
            if scene_type == "asian_commercial_street" and "bottom" in " ".join([obj["region"] for obj in detected_objects if obj["class_id"] == 0]):
                # If people are in bottom regions, they might be walking on uneven surfaces
                concerns.append("Potential uneven walking surfaces in commercial area")

        # Financial district safety concerns
        elif scene_type == "financial_district":
            # Check for heavy traffic conditions
            vehicle_count = sum(1 for obj_id in detected_class_ids if obj_id in [2, 5, 7])  # Car, bus, truck
            if vehicle_count > 5:
                concerns.append("Heavy vehicle traffic in urban area")

            # Check for pedestrians crossing busy streets
            if 0 in detected_class_ids:  # Person
                person_count = detected_class_ids.count(0)
                vehicle_nearby = any(vehicle in detected_class_ids for vehicle in [2, 3, 5, 7])

                if person_count > 0 and vehicle_nearby:
                    concerns.append("Pedestrians navigating busy urban traffic")

            # Check for traffic signals
            if 9 in detected_class_ids:  # Traffic light
                concerns.append("Observe traffic signals when navigating this area")
            else:
                # If no traffic lights detected but it's a busy area, it's worth noting
                if vehicle_count > 3:
                    concerns.append("Busy traffic area potentially without visible traffic signals in view")

            # Time of day considerations
            vehicle_objs = [obj for obj in detected_objects if obj["class_id"] in [2, 5, 7]]
            if vehicle_objs and any("lighting_conditions" in obj for obj in detected_objects):
                # If vehicles are present and it might be evening/night
                concerns.append("Reduced visibility conditions during evening commute")

        # Urban intersection safety concerns
        elif scene_type == "urban_intersection":
            # Check for pedestrians in crosswalks
            pedestrian_objs = [obj for obj in detected_objects if obj["class_id"] == 0]
            vehicle_objs = [obj for obj in detected_objects if obj["class_id"] in [2, 3, 5, 7]]

            if pedestrian_objs:
                # Calculate distribution of pedestrians to see if they're crossing
                pedestrian_positions = [obj["normalized_center"] for obj in pedestrian_objs]

                # Simplified check for pedestrians in crossing pattern
                if len(pedestrian_positions) >= 3:
                    # Check if pedestrians are distributed across different regions
                    pedestrian_regions = set(obj["region"] for obj in pedestrian_objs)
                    if len(pedestrian_regions) >= 2:
                        concerns.append("Multiple pedestrians crossing the intersection")

            # Check for traffic signal observation
            if 9 in detected_class_ids:  # Traffic light
                concerns.append("Observe traffic signals when crossing")

            # Check for busy intersection
            if len(vehicle_objs) > 3:
                concerns.append("Busy intersection with multiple vehicles")

            # Check for pedestrians potentially jay-walking
            if pedestrian_objs and not 9 in detected_class_ids:  # People but no traffic lights
                concerns.append("Pedestrians should use designated crosswalks")

            # Visibility concerns based on lighting
            # This would be better with actual lighting data
            pedestrian_count = len(pedestrian_objs)
            if pedestrian_count > 5:
                concerns.append("High pedestrian density at crossing points")

        # Transit hub safety concerns
        elif scene_type == "transit_hub":
            # These would be for transit areas like train stations or bus terminals
            if 0 in detected_class_ids:  # Person
                person_count = detected_class_ids.count(0)
                if person_count > 8:
                    concerns.append("Crowded transit area requiring careful navigation")

            # Check for luggage/bags that could be trip hazards
            if 24 in detected_class_ids or 28 in detected_class_ids:  # Backpack or suitcase
                concerns.append("Luggage and personal items may create obstacles")

            # Public transportation vehicles
            if any(vehicle in detected_class_ids for vehicle in [5, 6, 7]):  # Bus, train, truck
                concerns.append("Stay clear of arriving and departing transit vehicles")

        # Shopping district safety concerns
        elif scene_type == "shopping_district":
            # Check for crowded shopping areas
            if 0 in detected_class_ids:  # Person
                person_count = detected_class_ids.count(0)
                if person_count > 5:
                    concerns.append("Crowded shopping area with multiple people")

            # Check for shopping bags and personal items
            if 24 in detected_class_ids or 26 in detected_class_ids:  # Backpack or handbag
                concerns.append("Mind personal belongings in busy retail environment")

            # Check for store entrances/exits which might have automatic doors
            # We can't directly detect this, but can infer from context
            if scene_type == "shopping_district" and 0 in detected_class_ids:
                concerns.append("Be aware of store entrances and exits with potential automatic doors")

        return concerns

Writing scene_description.py


In [ ]:
# %%writefile clip_prompts.py

# 場景類型提示
SCENE_TYPE_PROMPTS = {
    # 基本室內場景
    "living_room": "A photo of a living room with furniture and entertainment systems.",
    "bedroom": "A photo of a bedroom with a bed and personal items.",
    "dining_area": "A photo of a dining area with a table and chairs for meals.",
    "kitchen": "A photo of a kitchen with cooking appliances and food preparation areas.",
    "office_workspace": "A photo of an office workspace with desk, computer and work equipment.",
    "meeting_room": "A photo of a meeting room with a conference table and multiple chairs.",

    # 基本室外/城市場景
    "city_street": "A photo of a city street with traffic, pedestrians and urban buildings.",
    "parking_lot": "A photo of a parking lot with multiple parked vehicles.",
    "park_area": "A photo of a park or recreational area with greenery and outdoor facilities.",
    "retail_store": "A photo of a retail store with merchandise displays and shopping areas.",
    "supermarket": "A photo of a supermarket with food items, aisles and shopping carts.",

    # 特殊室內場景
    "upscale_dining": "A photo of an upscale dining area with elegant furniture and refined decor.",
    "conference_room": "A photo of a professional conference room with presentation equipment and seating.",
    "classroom": "A photo of a classroom with desks, chairs and educational equipment.",
    "library": "A photo of a library with bookshelves, reading areas and study spaces.",

    # 亞洲特色場景
    "asian_commercial_street": "A photo of an Asian commercial street with dense signage, shops and pedestrians.",
    "asian_night_market": "A photo of an Asian night market with food stalls, crowds and colorful lights.",
    "asian_temple_area": "A photo of an Asian temple with traditional architecture and cultural elements.",

    # 交通相關場景
    "financial_district": "A photo of a financial district with tall office buildings and business activity.",
    "urban_intersection": "A photo of an urban intersection with crosswalks, traffic lights and pedestrians crossing.",
    "transit_hub": "A photo of a transportation hub with multiple modes of public transit and passengers.",
    "bus_stop": "A photo of a bus stop with people waiting and buses arriving or departing.",
    "bus_station": "A photo of a bus terminal with multiple buses and traveler facilities.",
    "train_station": "A photo of a train station with platforms, trains and passenger activity.",
    "airport": "A photo of an airport with planes, terminals and traveler activity.",

    # 商業場景
    "shopping_district": "A photo of a shopping district with multiple retail stores and consumer activity.",
    "cafe": "A photo of a cafe with coffee service, seating and casual dining.",
    "restaurant": "A photo of a restaurant with dining tables, food service and eating areas.",

    # 空中視角場景
    "aerial_view_intersection": "An aerial view of an intersection showing crosswalks and traffic patterns from above.",
    "aerial_view_commercial_area": "An aerial view of a commercial area showing shopping districts from above.",
    "aerial_view_plaza": "An aerial view of a public plaza or square showing patterns of people movement from above.",

    # 娛樂場景
    "zoo": "A photo of a zoo with animal enclosures, exhibits and visitors.",
    "playground": "A photo of a playground with recreational equipment and children playing.",
    "sports_field": "A photo of a sports field with playing surfaces and athletic equipment.",
    "sports_stadium": "A photo of a sports stadium with spectator seating and athletic facilities.",

    # 水相關場景
    "harbor": "A photo of a harbor with boats, docks and waterfront activity.",
    "beach_water_recreation": "A photo of a beach area with water activities, sand and recreational equipment like surfboards.",

    # 文化時間特定場景
    "nighttime_street": "A photo of a street at night with artificial lighting and evening activity.",
    "nighttime_commercial_district": "A photo of a commercial district at night with illuminated signs and evening shopping.",
    "european_plaza": "A photo of a European-style plaza with historic architecture and public gathering spaces.",

    # 混合環境場景
    "indoor_outdoor_cafe": "A photo of a cafe with both indoor seating and outdoor patio areas.",
    "transit_station_platform": "A photo of a transit station platform with waiting areas and arriving vehicles.",

    # 工作場景
    "construction_site": "A photo of a construction site with building materials, equipment and workers.",
    "medical_facility": "A photo of a medical facility with healthcare equipment and professional staff.",
    "educational_setting": "A photo of an educational setting with learning spaces and academic resources.",
    "professional_kitchen": "A photo of a professional commercial kitchen with industrial cooking equipment and food preparation stations."
}

# 文化特定場景提示
CULTURAL_SCENE_PROMPTS = {
    "asian_commercial_street": [
        "A busy Asian shopping street with neon signs and dense storefronts.",
        "A commercial street in Asia with multi-level signage and narrow walkways.",
        "A street scene in Taiwan or Hong Kong with vertical signage and compact shops.",
        "A crowded commercial alley in an Asian city with signs in Chinese characters.",
        "A narrow shopping street in Asia with small shops on both sides.",
        "An outdoor shopping district in an East Asian city with electronic billboards.",
        "A bustling commercial street in Taiwan with food vendors and retail shops.",
        "A pedestrian shopping area with Korean or Chinese signs and storefronts.",
        "A daytime shopping street in an Asian urban center with vertical development."
    ],
    "asian_night_market": [
        "A vibrant night market in Asia with food stalls and large crowds.",
        "An evening street market in Taiwan with street food vendors and bright lights.",
        "A busy night bazaar in Asia with illuminated stalls and local food.",
        "A crowded night street food market in an Asian city with vendor carts.",
        "An Asian night market with steam from cooking food and hanging lanterns.",
        "A nocturnal food street in East Asia with vendor canopies and neon lights.",
        "A bustling evening market with rows of food stalls and plastic stools.",
        "A lively Asian street food scene at night with cooking stations and crowds."
    ],
    "asian_temple_area": [
        "A traditional Asian temple with ornate roof details and religious symbols.",
        "A Buddhist temple complex in East Asia with multiple pavilions and prayer areas.",
        "A sacred site in Asia with incense burners and ceremonial elements.",
        "A temple courtyard with stone statues and traditional Asian architecture.",
        "A spiritual center in East Asia with pagoda-style structures and visitors.",
        "An ancient temple site with Asian architectural elements and cultural symbols.",
        "A religious compound with characteristic Asian roof curves and decorative features."
    ],
    "european_plaza": [
        "A historic European city square with classical architecture and cafes.",
        "An old-world plaza in Europe with cobblestone paving and historic buildings.",
        "A public square in a European city with fountains and surrounding architecture.",
        "A central plaza in Europe with outdoor seating areas and historic monuments.",
        "A traditional European town square with surrounding shops and restaurants.",
        "A historic gathering space in Europe with distinctive architecture and pedestrians."
    ]
}

# 對比類別提示
COMPARATIVE_PROMPTS = {
    "indoor_vs_outdoor": [
        "An indoor shopping mall corridor with controlled lighting and storefronts.",
        "An outdoor commercial street with natural lighting and urban storefronts.",
        "An enclosed shopping gallery with artificial lighting and climate control.",
        "An open-air market street with natural light and weather exposure."
    ],
    "professional_vs_home": [
        "A professional commercial kitchen with stainless steel equipment and workstations.",
        "A home kitchen with residential appliances and family cooking space.",
        "A restaurant kitchen with multiple cooking stations and chef activity.",
        "A family kitchen with standard household equipment and personal touches."
    ],
    "sports_venue_vs_park": [
        "A professional sports stadium with designated playing areas and audience seating.",
        "A public park with casual recreation space and community greenery.",
        "An athletic venue with specialized sports equipment and competitive playing surfaces.",
        "An outdoor community space with general purpose areas and natural elements."
    ],
    "asian_vs_western_commercial": [
        "An Asian shopping street with vertical signage and compact multi-level shops.",
        "A Western commercial street with horizontal storefronts and wider sidewalks.",
        "An East Asian retail area with dense signage in Asian scripts and narrow walkways."
        "A Western shopping district with uniform building heights and Latin alphabetic signs."
    ],
    "daytime_vs_nighttime": [
        "A daytime urban scene with natural sunlight illuminating streets and buildings.",
        "A nighttime city scene with artificial lighting from stores, signs and streetlights.",
        "A commercial district during daylight hours with natural shadows and visibility.",
        "An evening urban setting with illuminated storefronts and light patterns on streets."
    ],
    "aerial_vs_street_level": [
        "An aerial view showing urban patterns and layouts from above.",
        "A street-level view showing pedestrian perspective and immediate surroundings.",
        "A bird's-eye view of city organization and movement patterns from high above.",
        "An eye-level perspective showing direct human interaction with urban elements."
    ]
}

# 環境條件文本提示
LIGHTING_CONDITION_PROMPTS = {
    "day_clear": "A photo taken during daytime with clear skies and direct sunlight.",
    "day_cloudy": "A photo taken during daytime with overcast conditions and diffused light.",
    "sunset/sunrise": "A photo taken during sunset or sunrise with warm golden lighting and long shadows.",
    "night": "A photo taken at night with minimal natural light and artificial illumination.",
    "indoor_bright": "An indoor photo with bright, even artificial lighting throughout the space.",
    "indoor_moderate": "An indoor photo with moderate lighting creating a balanced indoor atmosphere.",
    "indoor_dim": "An indoor photo with low lighting levels creating a subdued environment.",
    "neon_night": "A night scene with colorful neon lighting creating vibrant illumination patterns.",
    "indoor_commercial": "An indoor retail environment with directed display lighting highlighting products.",
    "indoor_restaurant": "An indoor dining space with ambient mood lighting for atmosphere.",
    "stadium_lighting": "A sports venue with powerful floodlights creating intense, even illumination.",
    "mixed_lighting": "A scene with combined natural and artificial light sources creating transition zones.",
    "beach_daylight": "A photo taken at a beach with bright natural sunlight and reflections from water.",
    "sports_arena_lighting": "A photo of a sports venue illuminated by powerful overhead lighting systems.",
    "kitchen_task_lighting": "A photo of a kitchen with focused lighting concentrated on work surfaces."
}

# 針對新場景類型的特殊提示
SPECIALIZED_SCENE_PROMPTS = {
    "beach_water_recreation": [
        "A coastal beach scene with people surfing and sunbathing on sandy shores.",
        "Active water sports participants at a beach with surfboards and swimming areas.",
        "A sunny beach destination with recreational water equipment and beachgoers.",
        "A shoreline recreation area with surf gear and coastal activities.",
        "An oceanfront scene with people engaging in water sports and beach leisure.",
        "A popular beach spot with swimming areas and surfing zones.",
        "A coastal recreation setting with beach umbrellas and water activities."
    ],
    "sports_venue": [
        "An indoor sports arena with professional equipment and competition spaces.",
        "A sports stadium with marked playing areas and spectator seating arrangement.",
        "A specialized athletic venue with competition equipment and performance areas.",
        "A professional sports facility with game-related apparatus and audience zones.",
        "An organized sports center with competitive play areas and athletic equipment.",
        "A competition venue with sport-specific markings and professional setup.",
        "A formal athletic facility with standardized equipment and playing surfaces."
    ],
    "professional_kitchen": [
        "A commercial restaurant kitchen with multiple cooking stations and food prep areas.",
        "A professional culinary workspace with industrial appliances and chef activity.",
        "A busy restaurant back-of-house with stainless steel equipment and meal preparation.",
        "A commercial food service kitchen with chef workstations and specialized zones.",
        "An industrial kitchen facility with specialized cooking equipment and prep surfaces.",
        "A high-volume food production kitchen with professional-grade appliances.",
        "A restaurant kitchen with distinct cooking areas and culinary workflow design."
    ],
    "urban_intersection": [
        "A city intersection with crosswalks and traffic signals controlling movement.",
        "A busy urban crossroad with pedestrian crossings and vehicle traffic.",
        "A regulated street intersection with crosswalk markings and waiting pedestrians.",
        "A metropolitan junction with traffic lights and pedestrian crossing zones.",
        "A city street crossing with safety features for pedestrians and traffic flow.",
        "A controlled urban intersection with movement patterns for vehicles and people.",
        "A city center crossroad with traffic management features and pedestrian areas."
    ],
    "financial_district": [
        "A downtown business area with tall office buildings and commercial activity.",
        "An urban financial center with skyscrapers and professional environment.",
        "A city's business district with corporate headquarters and office towers.",
        "A metropolitan financial zone with high-rise buildings and business traffic.",
        "A corporate district in a city center with professional architecture.",
        "An urban area dominated by office buildings and business establishments.",
        "A city's economic center with banking institutions and corporate offices."
    ],
    "aerial_view_intersection": [
        "A bird's-eye view of a city intersection showing crossing patterns from above.",
        "An overhead perspective of an urban crossroad showing traffic organization.",
        "A top-down view of a street intersection revealing pedestrian crosswalks.",
        "An aerial shot of a city junction showing the layout of roads and crossings.",
        "A high-angle view of an intersection showing traffic and pedestrian flow patterns.",
        "A drone perspective of urban crossing design viewed from directly above.",
        "A vertical view of a street intersection showing crossing infrastructure."
    ]
}

VIEWPOINT_PROMPTS = {
    "eye_level": "A photo taken from normal human eye level showing a direct frontal perspective.",
    "aerial": "A photo taken from high above looking directly down at the scene below.",
    "elevated": "A photo taken from a higher than normal position looking down at an angle.",
    "low_angle": "A photo taken from a low position looking upward at the scene.",
    "bird_eye": "A photo taken from very high above showing a complete overhead perspective.",
    "street_level": "A photo taken from the perspective of someone standing on the street.",
    "interior": "A photo taken from inside a building showing the internal environment.",
    "vehicular": "A photo taken from inside or mounted on a moving vehicle."
}

OBJECT_COMBINATION_PROMPTS = {
    "dining_setting": "A scene with tables, chairs, plates, and eating utensils arranged for meals.",
    "office_setup": "A scene with desks, chairs, computers, and office supplies for work.",
    "living_space": "A scene with sofas, coffee tables, TVs, and comfortable seating arrangements.",
    "transportation_hub": "A scene with vehicles, waiting areas, passengers, and transit information.",
    "retail_environment": "A scene with merchandise displays, shoppers, and store fixtures.",
    "crosswalk_scene": "A scene with street markings, pedestrians crossing, and traffic signals.",
    "cooking_area": "A scene with stoves, prep surfaces, cooking utensils, and food items.",
    "recreational_space": "A scene with sports equipment, play areas, and activity participants."
}

ACTIVITY_PROMPTS = {
    "shopping": "People looking at merchandise, carrying shopping bags, and browsing stores.",
    "dining": "People eating food, sitting at tables, and using dining utensils.",
    "commuting": "People waiting for transportation, boarding vehicles, and traveling.",
    "working": "People using computers, attending meetings, and engaged in professional tasks.",
    "exercising": "People engaged in physical activities, using sports equipment, and training.",
    "cooking": "People preparing food, using kitchen equipment, and creating meals.",
    "crossing_street": "People walking across designated crosswalks and navigating intersections.",
    "recreational_activity": "People engaged in leisure activities, games, and social recreation."
}

Writing clip_prompts.py


In [ ]:
# %%writefile clip_analyzer.py
import torch
import clip
import numpy as np
from PIL import Image
from typing import Dict, List, Tuple, Any, Optional, Union

# from clip_prompts import (
#     SCENE_TYPE_PROMPTS,
#     CULTURAL_SCENE_PROMPTS,
#     COMPARATIVE_PROMPTS,
#     LIGHTING_CONDITION_PROMPTS,
#     SPECIALIZED_SCENE_PROMPTS,
#     VIEWPOINT_PROMPTS,
#     OBJECT_COMBINATION_PROMPTS,
#     ACTIVITY_PROMPTS
# )

class CLIPAnalyzer:
    """
    Use Clip to intergrate scene understanding function
    """

    def __init__(self, model_name: str = "ViT-B/16", device: str = None):
        """
        初始化 CLIP 分析器。

        Args:
            model_name: CLIP Model name, 默認 "ViT-B/16"
            device: Use GPU if it can use
        """
        # 自動選擇設備
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        print(f"Loading CLIP model {model_name} on {self.device}...")
        try:
            self.model, self.preprocess = clip.load(model_name, device=self.device)
            print(f"CLIP model loaded successfully.")
        except Exception as e:
            print(f"Error loading CLIP model: {e}")
            raise

        self.scene_type_prompts = SCENE_TYPE_PROMPTS
        self.cultural_scene_prompts = CULTURAL_SCENE_PROMPTS
        self.comparative_prompts = COMPARATIVE_PROMPTS
        self.lighting_condition_prompts = LIGHTING_CONDITION_PROMPTS
        self.specialized_scene_prompts = SPECIALIZED_SCENE_PROMPTS
        self.viewpoint_prompts = VIEWPOINT_PROMPTS
        self.object_combination_prompts = OBJECT_COMBINATION_PROMPTS
        self.activity_prompts = ACTIVITY_PROMPTS

        # turn to CLIP format
        self._prepare_text_prompts()

    def _prepare_text_prompts(self):
        """準備所有文本提示的 CLIP 特徵並存儲到 self.text_features_cache 中"""
        self.text_features_cache = {}

        # 處理基礎場景類型 (SCENE_TYPE_PROMPTS)
        if hasattr(self, 'scene_type_prompts') and self.scene_type_prompts:
            scene_texts = [prompt for scene_type, prompt in self.scene_type_prompts.items()]
            if scene_texts:
                self.text_features_cache["scene_type_keys"] = list(self.scene_type_prompts.keys())
                try:
                    self.text_features_cache["scene_type_tokens"] = clip.tokenize(scene_texts).to(self.device)
                except Exception as e:
                    print(f"Warning: Error tokenizing scene_type_prompts: {e}")
                    self.text_features_cache["scene_type_tokens"] = None # 標記錯誤或空
            else:
                self.text_features_cache["scene_type_keys"] = []
                self.text_features_cache["scene_type_tokens"] = None
        else:
            self.text_features_cache["scene_type_keys"] = []
            self.text_features_cache["scene_type_tokens"] = None

        # 處理文化場景 (CULTURAL_SCENE_PROMPTS)
        # cultural_tokens_dict 存儲的是 tokenized prompts
        cultural_tokens_dict_val = {}
        if hasattr(self, 'cultural_scene_prompts') and self.cultural_scene_prompts:
            for scene_type, prompts in self.cultural_scene_prompts.items():
                if prompts and isinstance(prompts, list) and all(isinstance(p, str) for p in prompts):
                    try:
                        cultural_tokens_dict_val[scene_type] = clip.tokenize(prompts).to(self.device)
                    except Exception as e:
                        print(f"Warning: Error tokenizing cultural_scene_prompts for {scene_type}: {e}")
                        cultural_tokens_dict_val[scene_type] = None # 標記錯誤或空
                else:
                    cultural_tokens_dict_val[scene_type] = None # prompts 不合規
        self.text_features_cache["cultural_tokens_dict"] = cultural_tokens_dict_val

        # 處理光照條件 (LIGHTING_CONDITION_PROMPTS)
        if hasattr(self, 'lighting_condition_prompts') and self.lighting_condition_prompts:
            lighting_texts = [prompt for cond, prompt in self.lighting_condition_prompts.items()]
            if lighting_texts:
                self.text_features_cache["lighting_condition_keys"] = list(self.lighting_condition_prompts.keys())
                try:
                    self.text_features_cache["lighting_tokens"] = clip.tokenize(lighting_texts).to(self.device)
                except Exception as e:
                    print(f"Warning: Error tokenizing lighting_condition_prompts: {e}")
                    self.text_features_cache["lighting_tokens"] = None
            else:
                self.text_features_cache["lighting_condition_keys"] = []
                self.text_features_cache["lighting_tokens"] = None
        else:
            self.text_features_cache["lighting_condition_keys"] = []
            self.text_features_cache["lighting_tokens"] = None

        # 處理特殊場景 (SPECIALIZED_SCENE_PROMPTS)
        specialized_tokens_dict_val = {}
        if hasattr(self, 'specialized_scene_prompts') and self.specialized_scene_prompts:
            for scene_type, prompts in self.specialized_scene_prompts.items():
                if prompts and isinstance(prompts, list) and all(isinstance(p, str) for p in prompts):
                    try:
                        specialized_tokens_dict_val[scene_type] = clip.tokenize(prompts).to(self.device)
                    except Exception as e:
                        print(f"Warning: Error tokenizing specialized_scene_prompts for {scene_type}: {e}")
                        specialized_tokens_dict_val[scene_type] = None
                else:
                    specialized_tokens_dict_val[scene_type] = None
        self.text_features_cache["specialized_tokens_dict"] = specialized_tokens_dict_val

        # 處理視角 (VIEWPOINT_PROMPTS)
        if hasattr(self, 'viewpoint_prompts') and self.viewpoint_prompts:
            viewpoint_texts = [prompt for viewpoint, prompt in self.viewpoint_prompts.items()]
            if viewpoint_texts:
                self.text_features_cache["viewpoint_keys"] = list(self.viewpoint_prompts.keys())
                try:
                    self.text_features_cache["viewpoint_tokens"] = clip.tokenize(viewpoint_texts).to(self.device)
                except Exception as e:
                    print(f"Warning: Error tokenizing viewpoint_prompts: {e}")
                    self.text_features_cache["viewpoint_tokens"] = None
            else:
                self.text_features_cache["viewpoint_keys"] = []
                self.text_features_cache["viewpoint_tokens"] = None
        else:
            self.text_features_cache["viewpoint_keys"] = []
            self.text_features_cache["viewpoint_tokens"] = None

        # 處理物件組合 (OBJECT_COMBINATION_PROMPTS)
        if hasattr(self, 'object_combination_prompts') and self.object_combination_prompts:
            object_combination_texts = [prompt for combo, prompt in self.object_combination_prompts.items()]
            if object_combination_texts:
                self.text_features_cache["object_combination_keys"] = list(self.object_combination_prompts.keys())
                try:
                    self.text_features_cache["object_combination_tokens"] = clip.tokenize(object_combination_texts).to(self.device)
                except Exception as e:
                    print(f"Warning: Error tokenizing object_combination_prompts: {e}")
                    self.text_features_cache["object_combination_tokens"] = None
            else:
                self.text_features_cache["object_combination_keys"] = []
                self.text_features_cache["object_combination_tokens"] = None
        else:
            self.text_features_cache["object_combination_keys"] = []
            self.text_features_cache["object_combination_tokens"] = None

        # 處理活動 (ACTIVITY_PROMPTS)
        if hasattr(self, 'activity_prompts') and self.activity_prompts:
            activity_texts = [prompt for activity, prompt in self.activity_prompts.items()]
            if activity_texts:
                self.text_features_cache["activity_keys"] = list(self.activity_prompts.keys())
                try:
                    self.text_features_cache["activity_tokens"] = clip.tokenize(activity_texts).to(self.device)
                except Exception as e:
                    print(f"Warning: Error tokenizing activity_prompts: {e}")
                    self.text_features_cache["activity_tokens"] = None
            else:
                self.text_features_cache["activity_keys"] = []
                self.text_features_cache["activity_tokens"] = None
        else:
            self.text_features_cache["activity_keys"] = []
            self.text_features_cache["activity_tokens"] = None

        self.scene_type_tokens = self.text_features_cache["scene_type_tokens"]
        self.lighting_tokens = self.text_features_cache["lighting_tokens"]
        self.viewpoint_tokens = self.text_features_cache["viewpoint_tokens"]
        self.object_combination_tokens = self.text_features_cache["object_combination_tokens"]
        self.activity_tokens = self.text_features_cache["activity_tokens"]
        self.cultural_tokens_dict = self.text_features_cache["cultural_tokens_dict"]
        self.specialized_tokens_dict = self.text_features_cache["specialized_tokens_dict"]

        print("CLIP text_features_cache prepared.")

    def analyze_image(self, image, include_cultural_analysis=True, exclude_categories=None, enable_landmark=True, places365_guidance=None):
        """
        分析圖像，預測場景類型和光照條件。

        Args:
            image: 輸入圖像 (PIL Image 或 numpy array)
            include_cultural_analysis: 是否包含文化場景的詳細分析
            exclude_categories: 要排除的類別列表
            enable_landmark: 是否啟用地標檢測功能
            places365_guidance: Places365 提供的場景指導信息 (可選)


        Returns:
            Dict: 包含場景類型預測和光照條件的分析結果
        """
        try:
            self.enable_landmark = enable_landmark # 更新實例的 enable_landmark 狀態
            # 確保圖像是 PIL 格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 預處理圖像
            image_input = self.preprocess(image).unsqueeze(0).to(self.device)

            # 獲取圖像特徵
            with torch.no_grad():
                image_features = self.model.encode_image(image_input)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)

            places365_focus_areas = []
            places365_scene_context = "" # 用於存儲 Places365 提供的場景描述

            if places365_guidance and isinstance(places365_guidance, dict) and places365_guidance.get('confidence', 0) > 0.4:
                mapped_scene = places365_guidance.get('mapped_scene_type', '')
                scene_label = places365_guidance.get('scene_label', '')
                # is_indoor = places365_guidance.get('is_indoor', None) # 未使用，可註釋
                attributes = places365_guidance.get('attributes', [])

                places365_scene_context = f"Scene identified by Places365 as {scene_label}" # 更新上下文描述

                # Adjust CLIP analysis focus based on Places365 scene type
                if mapped_scene in ['kitchen', 'dining_area', 'restaurant']:
                    places365_focus_areas.extend(['food preparation', 'dining setup', 'kitchen appliances'])
                elif mapped_scene in ['office_workspace', 'educational_setting', 'library', 'conference_room']:
                    places365_focus_areas.extend(['work environment', 'professional setting', 'learning space', 'study area'])
                elif mapped_scene in ['retail_store', 'shopping_mall', 'market', 'supermarket']: # 擴展匹配
                    places365_focus_areas.extend(['commercial space', 'shopping environment', 'retail display', 'goods for sale'])
                elif mapped_scene in ['park_area', 'beach', 'natural_outdoor_area', 'playground', 'sports_field']: # 擴展匹配
                    places365_focus_areas.extend(['outdoor recreation', 'natural environment', 'leisure activity', 'open space'])

                # 根據屬性添加更通用的 focus areas
                if isinstance(attributes, list): # 確保 attributes 是列表
                    if 'commercial' in attributes:
                        places365_focus_areas.append('business activity')
                    if 'recreational' in attributes:
                        places365_focus_areas.append('entertainment or leisure')
                    if 'residential' in attributes:
                        places365_focus_areas.append('living space')

                # 去重
                places365_focus_areas = list(set(places365_focus_areas))

                if places365_focus_areas: # 只有在確實有 focus areas 時才打印
                    print(f"CLIP analysis guided by Places365: {places365_scene_context}, focus areas: {places365_focus_areas}")

            # 分析場景類型，傳遞 enable_landmark 參數和 Places365 指導
            scene_scores = self._analyze_scene_type(image_features,
                                                  enable_landmark=self.enable_landmark, # 使用更新後的實例屬性
                                                  places365_focus=places365_focus_areas)

            # 如果禁用地標功能，確保排除地標相關類別
            current_exclude_categories = list(exclude_categories) if exclude_categories is not None else []
            if not self.enable_landmark: # 使用更新後的實例屬性
                landmark_related_terms = ["landmark", "monument", "tower", "tourist", "attraction", "historical", "famous", "iconic"]
                for term in landmark_related_terms:
                    if term not in current_exclude_categories:
                        current_exclude_categories.append(term)

            if current_exclude_categories:
                filtered_scores = {}
                for scene, score in scene_scores.items():
                    # 檢查 scene 的鍵名（通常是英文）是否包含任何排除詞彙
                    if not any(cat.lower() in scene.lower() for cat in current_exclude_categories):
                        filtered_scores[scene] = score

                if filtered_scores:
                    total_score = sum(filtered_scores.values())
                    if total_score > 1e-5: # 避免除以零或非常小的數
                        scene_scores = {k: v / total_score for k, v in filtered_scores.items()}
                    else: # 如果總分趨近於0，則保持原樣或設為0
                        scene_scores = {k: 0.0 for k in filtered_scores.keys()} # 或者 scene_scores = filtered_scores
                else: # 如果過濾後沒有場景了
                    scene_scores = {k: (0.0 if any(cat.lower() in k.lower() for cat in current_exclude_categories) else v) for k,v in scene_scores.items()}
                    if not any(s > 1e-5 for s in scene_scores.values()): # 如果還是全0
                         scene_scores = {"unknown": 1.0} # 給一個默認值避免空字典

            lighting_scores = self._analyze_lighting_condition(image_features)
            cultural_analysis = {}
            if include_cultural_analysis and self.enable_landmark: # 使用更新後的實例屬性
                for scene_type_cultural_key in self.text_features_cache.get("cultural_tokens_dict", {}).keys():
                     # 確保 scene_type_cultural_key 是 SCENE_TYPE_PROMPTS 中的鍵，或者有一個映射關係
                    if scene_type_cultural_key in scene_scores and scene_scores[scene_type_cultural_key] > 0.2:
                        cultural_analysis[scene_type_cultural_key] = self._analyze_cultural_scene(
                            image_features, scene_type_cultural_key
                        )

            specialized_analysis = {}
            for scene_type_specialized_key in self.text_features_cache.get("specialized_tokens_dict", {}).keys():
                if scene_type_specialized_key in scene_scores and scene_scores[scene_type_specialized_key] > 0.2:
                    specialized_analysis[scene_type_specialized_key] = self._analyze_specialized_scene(
                        image_features, scene_type_specialized_key
                    )

            viewpoint_scores = self._analyze_viewpoint(image_features)
            object_combination_scores = self._analyze_object_combinations(image_features)
            activity_scores = self._analyze_activities(image_features)

            if scene_scores: # 確保 scene_scores 不是空的
                top_scene = max(scene_scores.items(), key=lambda x: x[1])
                 # 如果禁用地標，再次確認 top_scene 不是地標相關
                if not self.enable_landmark and any(cat.lower() in top_scene[0].lower() for cat in current_exclude_categories):
                    non_excluded_scores = {k:v for k,v in scene_scores.items() if not any(cat.lower() in k.lower() for cat in current_exclude_categories)}
                    if non_excluded_scores:
                        top_scene = max(non_excluded_scores.items(), key=lambda x: x[1])
                    else:
                        top_scene = ("unknown", 0.0) # 或其他合適的默認值
            else:
                top_scene = ("unknown", 0.0)


            result = {
                "scene_scores": scene_scores,
                "top_scene": top_scene,
                "lighting_condition": max(lighting_scores.items(), key=lambda x: x[1]) if lighting_scores else ("unknown", 0.0),
                "embedding": image_features.cpu().numpy().tolist()[0], # 簡化
                "viewpoint": max(viewpoint_scores.items(), key=lambda x: x[1]) if viewpoint_scores else ("unknown", 0.0),
                "object_combinations": sorted(object_combination_scores.items(), key=lambda x: x[1], reverse=True)[:3] if object_combination_scores else [],
                "activities": sorted(activity_scores.items(), key=lambda x: x[1], reverse=True)[:3] if activity_scores else []
            }

            if places365_guidance and isinstance(places365_guidance, dict) and places365_focus_areas: # 檢查 places365_focus_areas 是否被填充
                result["places365_guidance"] = {
                    "scene_context": places365_scene_context,
                    "focus_areas": places365_focus_areas, # 現在這個會包含基於 guidance 的內容
                    "guided_analysis": True,
                    "original_places365_scene": places365_guidance.get('scene_label', 'N/A'),
                    "original_places365_confidence": places365_guidance.get('confidence', 0.0)
                }

            if cultural_analysis and self.enable_landmark:
                result["cultural_analysis"] = cultural_analysis

            if specialized_analysis:
                result["specialized_analysis"] = specialized_analysis

            return result

        except Exception as e:
            print(f"Error analyzing image with CLIP: {e}")
            import traceback
            traceback.print_exc()
            return {"error": str(e), "scene_scores": {}, "top_scene": ("error", 0.0)}

    def _analyze_scene_type(self, image_features: torch.Tensor, enable_landmark: bool = True, places365_focus: List[str] = None) -> Dict[str, float]:
        """
        分析圖像特徵與各場景類型的相似度，並可選擇性地排除地標相關場景

        Args:
            image_features: 經過 CLIP 編碼的圖像特徵
            enable_landmark: 是否啟用地標識別功能

        Returns:
            Dict[str, float]: 各場景類型的相似度分數字典
        """
        with torch.no_grad():
            # 計算場景類型文本特徵
            text_features = self.model.encode_text(self.scene_type_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # Apply Places365 guidance if available
            if places365_focus and len(places365_focus) > 0:
                # Create enhanced prompts that incorporate Places365 guidance
                enhanced_prompts = []
                for scene_type in self.scene_type_prompts.keys():
                    base_prompt = self.scene_type_prompts[scene_type]

                    # Check if this scene type should be emphasized based on Places365 guidance
                    scene_lower = scene_type.lower()
                    should_enhance = False

                    for focus_area in places365_focus:
                        if any(keyword in scene_lower for keyword in focus_area.split()):
                            should_enhance = True
                            enhanced_prompts.append(f"{base_prompt} with {focus_area}")
                            break

                    if not should_enhance:
                        enhanced_prompts.append(base_prompt)

                # Re-tokenize and encode enhanced prompts
                enhanced_tokens = clip.tokenize(enhanced_prompts).to(self.device)
                text_features = self.model.encode_text(enhanced_tokens)
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 建立場景分數字典
            scene_scores = {}
            for i, scene_type in enumerate(self.scene_type_prompts.keys()):
                # 如果未啟用地標功能，則跳過地標相關場景類型
                if not enable_landmark and scene_type in ["tourist_landmark", "natural_landmark", "historical_monument"]:
                    scene_scores[scene_type] = 0.0  # 將地標場景分數設為零
                else:
                    base_score = float(similarity[i])

                    # Apply Places365 guidance boost if applicable
                    if places365_focus:
                        scene_lower = scene_type.lower()
                        boost_factor = 1.0

                        for focus_area in places365_focus:
                            if any(keyword in scene_lower for keyword in focus_area.split()):
                                boost_factor = 1.15  # 15% boost for matching scenes
                                break

                        scene_scores[scene_type] = base_score * boost_factor
                    else:
                        scene_scores[scene_type] = base_score

            # 如果禁用地標功能，確保重新歸一化剩餘場景分數
            if not enable_landmark:
                # 獲取所有非零分數
                non_zero_scores = {k: v for k, v in scene_scores.items() if v > 0}
                if non_zero_scores:
                    # 計算總和並歸一化
                    total_score = sum(non_zero_scores.values())
                    if total_score > 0:
                        for scene_type in non_zero_scores:
                            scene_scores[scene_type] = non_zero_scores[scene_type] / total_score

            return scene_scores

    def _analyze_lighting_condition(self, image_features: torch.Tensor) -> Dict[str, float]:
        """分析圖像的光照條件"""
        with torch.no_grad():
            # 計算光照條件文本特徵
            text_features = self.model.encode_text(self.lighting_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 建立光照條件分數字典
            lighting_scores = {}
            for i, lighting_type in enumerate(self.lighting_condition_prompts.keys()):
                lighting_scores[lighting_type] = float(similarity[i])

            return lighting_scores

    def _analyze_cultural_scene(self, image_features: torch.Tensor, scene_type: str) -> Dict[str, Any]:
        """針對特定文化場景進行深入分析"""
        if scene_type not in self.cultural_tokens_dict:
            return {"error": f"No cultural analysis available for {scene_type}"}

        with torch.no_grad():
            # 獲取特定文化場景的文本特徵
            cultural_tokens = self.cultural_tokens_dict[scene_type]
            text_features = self.model.encode_text(cultural_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 找到最匹配的文化描述
            prompts = self.cultural_scene_prompts[scene_type]
            scores = [(prompts[i], float(similarity[i])) for i in range(len(prompts))]
            scores.sort(key=lambda x: x[1], reverse=True)

            return {
                "best_description": scores[0][0],
                "confidence": scores[0][1],
                "all_matches": scores
            }

    def _analyze_specialized_scene(self, image_features: torch.Tensor, scene_type: str) -> Dict[str, Any]:
        """針對特定專門場景進行深入分析"""
        if scene_type not in self.specialized_tokens_dict:
            return {"error": f"No specialized analysis available for {scene_type}"}

        with torch.no_grad():
            # 獲取特定專門場景的文本特徵
            specialized_tokens = self.specialized_tokens_dict[scene_type]
            text_features = self.model.encode_text(specialized_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 找到最匹配的專門描述
            prompts = self.specialized_scene_prompts[scene_type]
            scores = [(prompts[i], float(similarity[i])) for i in range(len(prompts))]
            scores.sort(key=lambda x: x[1], reverse=True)

            return {
                "best_description": scores[0][0],
                "confidence": scores[0][1],
                "all_matches": scores
            }

    def _analyze_viewpoint(self, image_features: torch.Tensor) -> Dict[str, float]:
        """分析圖像的拍攝視角"""
        with torch.no_grad():
            # 計算視角文本特徵
            text_features = self.model.encode_text(self.viewpoint_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 建立視角分數字典
            viewpoint_scores = {}
            for i, viewpoint in enumerate(self.viewpoint_prompts.keys()):
                viewpoint_scores[viewpoint] = float(similarity[i])

            return viewpoint_scores

    def _analyze_object_combinations(self, image_features: torch.Tensor) -> Dict[str, float]:
        """分析圖像中的物體組合"""
        with torch.no_grad():
            # 計算物體組合文本特徵
            text_features = self.model.encode_text(self.object_combination_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 建立物體組合分數字典
            combination_scores = {}
            for i, combination in enumerate(self.object_combination_prompts.keys()):
                combination_scores[combination] = float(similarity[i])

            return combination_scores

    def _analyze_activities(self, image_features: torch.Tensor) -> Dict[str, float]:
        """分析圖像中的活動"""
        with torch.no_grad():
            # 計算活動文本特徵
            text_features = self.model.encode_text(self.activity_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # 計算相似度分數
            similarity = (100 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

            # 建立活動分數字典
            activity_scores = {}
            for i, activity in enumerate(self.activity_prompts.keys()):
                activity_scores[activity] = float(similarity[i])

            return activity_scores

    def get_image_embedding(self, image) -> np.ndarray:
        """
        獲取圖像的 CLIP 嵌入表示

        Args:
            image: PIL Image 或 numpy array

        Returns:
            np.ndarray: 圖像的 CLIP 特徵向量
        """
        # 確保圖像是 PIL 格式
        if not isinstance(image, Image.Image):
            if isinstance(image, np.ndarray):
                image = Image.fromarray(image)
            else:
                raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

        # 預處理並編碼
        image_input = self.preprocess(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            image_features = self.model.encode_image(image_input)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # 轉換為 numpy 並返回
        return image_features.cpu().numpy()[0] if self.device == "cuda" else image_features.numpy()[0]

    def text_to_embedding(self, text: str) -> np.ndarray:
        """
        將文本轉換為 CLIP 嵌入表示

        Args:
            text: 輸入文本

        Returns:
            np.ndarray: 文本的 CLIP 特徵向量
        """
        text_token = clip.tokenize([text]).to(self.device)

        with torch.no_grad():
            text_features = self.model.encode_text(text_token)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        return text_features.cpu().numpy()[0] if self.device == "cuda" else text_features.numpy()[0]

    def calculate_similarity(self, image, text_queries: List[str]) -> Dict[str, float]:
        """
        計算圖像與多個文本查詢的相似度

        Args:
            image: PIL Image 或 numpy array
            text_queries: 文本查詢列表

        Returns:
            Dict: 每個查詢的相似度分數
        """
        # 獲取圖像嵌入
        if isinstance(image, np.ndarray) and len(image.shape) == 1:
            # 已經是嵌入向量
            image_features = torch.tensor(image).unsqueeze(0).to(self.device)
        else:
            # 是圖像，需要提取嵌入
            image_features = torch.tensor(self.get_image_embedding(image)).unsqueeze(0).to(self.device)

        # calulate similarity
        text_tokens = clip.tokenize(text_queries).to(self.device)

        with torch.no_grad():
            text_features = self.model.encode_text(text_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            similarity = similarity.cpu().numpy()[0] if self.device == "cuda" else similarity.numpy()[0]

        # display results
        result = {}
        for i, query in enumerate(text_queries):
            result[query] = float(similarity[i])

        return result

    def get_clip_instance(self):
        """
        獲取初始化好的CLIP模型實例，便於其他模組重用

        Returns:
            tuple: (模型實例, 預處理函數, 設備名稱)
        """
        return self.model, self.preprocess, self.device

Writing clip_analyzer.py


In [ ]:
# %%writefile landmark_data.py

"""
Landmark database for zero-shot classification using CLIP
"""

LANDMARK_DATA = {
    # 亞洲地標
    "asia": {
        "taipei_101": {
            "name": "Taipei 101",
            "aliases": ["Taipei 101 Tower", "Taipei Financial Center", "台北101大樓", "Taipei World Financial Center"],
            "location": "Taipei, Taiwan",
            "prompts": [
                "the iconic green bamboo-shaped Taipei 101 skyscraper against city skyline",
                "Taipei 101 tower with its distinctive stacked pagoda design and vibrant green glass facade",
                "world-famous Taipei 101 skyscraper with its segmented exterior resembling a bamboo stalk and prominent observation deck",
                "close-up of Taipei 101's architectural details showing the ruyi-inspired motifs and bamboo-inspired segments",
                "aerial view of Taipei 101 standing tall as a landmark in Taipei's urban landscape",
                "Taipei 101 illuminated at night with a spectrum of colorful light displays, often themed for holidays",
                "Taipei 101 seen from street level, emphasizing its immense height and unique green-tinted curtain wall",
                "the 508-meter tall Taipei 101 tower, an engineering marvel with its unique tiered design and spire",
                "Taipei 101's distinctive green glass exterior reflecting the sky, with visible bamboo segment patterns",
                "famous Taipei skyscraper, its characteristic pagoda-like stacked sections symbolizing growth and prosperity",
                "Taipei 101 featuring its massive tuned mass damper, visible through architectural openings, designed to counteract wind and seismic activity",
                "modern Asian skyscraper Taipei 101, showcasing a fusion of contemporary architecture and traditional Asian symbolism",
                "Taipei's iconic 101-story skyscraper with its tapered pinnacle reaching towards the sky, often against a backdrop of mountains or blue sky",
                "Taipei 101 viewed from Elephant Mountain, offering a classic panoramic shot of the tower and city",
                "close view of Taipei 101's signature green glass exterior with its intricate geometric patterns and reflective surface",
                "Taipei 101 with its distinct floor segments that resemble ancient Chinese ingots or a blossoming bamboo",
                "Taiwan's tallest building, Taipei 101, with its square base, tapered form, and prominent antenna",
                "the tall, slender green Taipei 101 skyscraper, a prominent feature in the city's skyline, day or night",
                "Taipei 101 with its unique green segmented tower design, a masterpiece of modern engineering and cultural symbolism, against a clear blue sky",
                "distinctive green glass Taipei 101 tower, a symbol of Taiwan's modernity, dominating the city view",
                "Taipei 101's iconic green tiered skyscraper structure with its high-speed elevators and 360-degree observation deck"
            ]
        },
        "taroko_gorge": {
            "name": "Taroko Gorge",
            "aliases": ["Taroko National Park", "太魯閣國家公園"],
            "location": "Hualien, Taiwan",
            "prompts": [
                "a photo of Taroko Gorge in Taiwan, showcasing its deep marble canyons and the Liwu River",
                "sheer marble cliffs of Taroko Gorge, with tunnels carved through the rock like the Tunnel of Nine Turns",
                "Taroko Gorge national park landscape with turquoise river, lush vegetation, and towering rock walls",
                "Swallow Grotto (Yanzikou) trail in Taroko Gorge, with views of the river and cliff formations",
                "the Cihmu Bridge, a distinctive red suspension bridge within Taroko Gorge"
            ]
        },
        "sun_moon_lake": {
            "name": "Sun Moon Lake",
            "aliases": ["日月潭", "Lake Candidius"],
            "location": "Nantou, Taiwan",
            "prompts": [
                "a photo of Sun Moon Lake in Taiwan, showing its calm, clear waters and surrounding mountains",
                "the serene Sun Moon Lake with Lalu Island, its sacred ancestral ground, dividing the lake into sun and moon shapes",
                "Sun Moon Lake surrounded by mist-covered mountains, with traditional temples like Wenwu Temple or Ci En Pagoda visible",
                "boats and ferries on Sun Moon Lake, with cyclists on lakeside paths",
                "aerial view of Sun Moon Lake highlighting its unique shape and the lush greenery of the region"
            ]
        },
        "jiufen_old_street": {
            "name": "Jiufen Old Street",
            "aliases": ["九份老街", "Chiufen"],
            "location": "New Taipei City, Taiwan",
            "prompts": [
                "narrow, winding alleys of Jiufen Old Street, lined with glowing red lanterns, traditional teahouses, and local food stalls",
                "atmospheric Jiufen Old Street at dusk or night, with countless red lanterns illuminating the historic hillside town and views of the coast",
                "historic gold mining town architecture in Jiufen, with wooden buildings and steep staircases, reminiscent of scenes from 'Spirited Away'",
                "A-Mei Tea House in Jiufen, a famous landmark with its traditional facade and lantern decorations",
                "street food and bustling crowds in the vibrant Jiufen Old Street market"
            ]
        },
        "kenting_national_park": {
            "name": "Kenting National Park",
            "aliases": ["墾丁國家公園"],
            "location": "Pingtung, Taiwan",
            "prompts": [
                "tropical beaches with white sand and clear blue water in Kenting National Park, southern Taiwan",
                "Eluanbi Lighthouse, the iconic white lighthouse at Taiwan's southernmost point, within Kenting National Park",
                "diverse coastal landscapes of Kenting, including coral reefs, rock formations like Sail Rock (Chuanfanshi), and lush forests",
                "vibrant marine life and water activities like snorkeling or surfing in Kenting National Park",
                "Longpan Park in Kenting, featuring dramatic grassy cliffs and coastline views"
            ]
        },
        "national_palace_museum_tw": { # Added _tw to differentiate from Beijing's
            "name": "National Palace Museum (Taipei)",
            "aliases": ["國立故宮博物院", "Taipei Palace Museum"],
            "location": "Taipei, Taiwan",
            "prompts": [
                "the grand exterior of the National Palace Museum in Taipei, a traditional Chinese palace-style building housing a vast collection of imperial artifacts",
                "iconic exhibits like the Jadeite Cabbage, Meat-shaped Stone, and Mao Gong Ding at the National Palace Museum in Taipei",
                "classical Chinese palace architecture of the National Palace Museum building in Taipei, with green tiled roofs and moon gates",
                "interior halls of the National Palace Museum displaying ancient Chinese ceramics, bronzes, and calligraphy",
                "gardens surrounding the National Palace Museum in Taipei, such as Zhishan Garden"
            ]
        },
        "alishan_national_scenic_area": {
            "name": "Alishan National Scenic Area",
            "aliases": ["阿里山國家風景區", "Mount Ali"],
            "location": "Chiayi, Taiwan",
            "prompts": [
                "sea of clouds phenomenon at Alishan National Scenic Area, viewed from high mountain peaks at sunrise or sunset",
                "Alishan Forest Railway trains, with their distinctive red carriages, winding through misty forests of ancient cypress and cedar trees",
                "sunrise views over Yushan (Jade Mountain), Taiwan's highest peak, from Alishan's Chushan or Ogasawara Mountain observation points",
                "tea plantations on the rolling hills of Alishan, known for its high-mountain oolong tea",
                "hiking trails through Alishan's giant tree groves and serene forests, like the Sister Ponds"
            ]
        },
        "shilin_night_market": {
            "name": "Shilin Night Market",
            "aliases": ["士林夜市"],
            "location": "Taipei, Taiwan",
            "prompts": [
                "bustling and vibrant atmosphere of Shilin Night Market in Taipei, one of Taiwan's largest and most famous night markets, packed with people",
                "a wide variety of Taiwanese street food stalls offering delicacies like oyster omelets, stinky tofu, and giant fried chicken cutlets at Shilin Night Market",
                "crowds of people exploring the maze-like alleys of Shilin Night Market, filled with food vendors, game stalls, and small shops",
                "brightly lit signs and food aromas filling the air at the lively Shilin Night Market",
                "underground food court area of Shilin Night Market offering a diverse range of local dishes"
            ]
        },
        "tokyo_tower": {
            "name": "Tokyo Tower",
            "aliases": ["東京タワー", "Tokyo Tower Landmark"],
            "location": "Tokyo, Japan",
            "prompts": [
                "a photo of Tokyo Tower in Japan, its distinctive red and white lattice structure inspired by the Eiffel Tower",
                "Tokyo Tower with its vibrant orange and white paint scheme, standing out against the Tokyo skyline",
                "the iconic Tokyo Tower illuminated at night, often with seasonal or special light displays",
                "view from Tokyo Tower's observation deck overlooking the sprawling metropolis of Tokyo",
                "Tokyo Tower as a symbol of post-war Japan's rebirth and a prominent communications tower"
            ]
        },
        "mount_fuji": {
            "name": "Mount Fuji",
            "aliases": ["富士山", "Fujisan"],
            "location": "Honshu, Japan",
            "prompts": [
                "a photo of Mount Fuji in Japan, its perfectly symmetrical snow-capped conical volcanic peak",
                "the snow-capped peak of Mount Fuji, often with a clear blue sky or surrounded by clouds",
                "Mount Fuji with cherry blossoms (sakura) in the foreground during spring, or reflected in one of the Fuji Five Lakes (Fujigoko)",
                "iconic view of Mount Fuji from the Chureito Pagoda",
                "Mount Fuji, Japan's highest mountain and an active stratovolcano, a symbol of Japan"
            ]
        },
        "kinkaku_ji": {
            "name": "Kinkaku-ji",
            "aliases": ["Golden Pavilion", "金閣寺", "Rokuon-ji"],
            "location": "Kyoto, Japan",
            "prompts": [
                "a photo of Kinkaku-ji temple (Golden Pavilion) in Kyoto, its top two floors completely covered in gold leaf",
                "the Golden Pavilion Kinkaku-ji reflected perfectly in the Mirror Pond (Kyōko-chi) surrounding it",
                "Kinkaku-ji, a Zen Buddhist temple in Kyoto, set within a beautiful Japanese stroll garden",
                "the three-tiered structure of Kinkaku-ji, each floor representing a different architectural style",
                "Kinkaku-ji in autumn with colorful foliage, or in winter dusted with snow"
            ]
        },
        "fushimi_inari_shrine": {
            "name": "Fushimi Inari Shrine",
            "aliases": ["伏見稲荷大社", "Thousand Torii Gates"],
            "location": "Kyoto, Japan",
            "prompts": [
                "a photo of Fushimi Inari Shrine in Kyoto, famous for its thousands of vibrant red/orange torii gates",
                "thousands of vermilion torii gates forming tunnels along a network of trails at Fushimi Inari Taisha",
                "pathways lined with densely packed red torii gates winding up a mountainside at Fushimi Inari Shrine",
                "fox statues (kitsune), messengers of Inari, found throughout Fushimi Inari Shrine",
                "the main shrine buildings of Fushimi Inari at the base of the mountain, with more torii gates leading upwards"
            ]
        },
        "shibuya_crossing": {
            "name": "Shibuya Crossing",
            "aliases": ["Shibuya Scramble Crossing", "澀谷十字路口"],
            "location": "Tokyo, Japan",
            "prompts": [
                "massive pedestrian scramble at Shibuya Crossing, Tokyo, with crowds of people crossing from all directions, surrounded by neon signs and large video screens",
                "bird's-eye view of the crowded Shibuya intersection with its iconic starburst pedestrian walkways and Hachiko statue nearby",
                "throngs of people crossing the multi-directional Shibuya intersection surrounded by modern buildings, department stores, and vibrant advertisements",
                "Shibuya Crossing at night with dazzling, vibrant lights from billboards and a continuous sea of people",
                "the energetic and iconic Shibuya Crossing, a symbol of modern Tokyo and urban life"
            ]
        },
        "tokyo_skytree": {
            "name": "Tokyo Skytree",
            "aliases": ["東京晴空塔"],
            "location": "Tokyo, Japan",
            "prompts": [
                "the slender, futuristic Tokyo Skytree, a broadcasting and observation tower in Sumida, Tokyo, with its lattice steel structure",
                "Tokyo Skytree illuminated in its signature pale blue (Iki) or purple (Miyabi) lights against the night sky",
                "modern lattice structure of the Tokyo Skytree, the world's tallest freestanding tower, dominating the city's skyline",
                "panoramic view from Tokyo Skytree's Tembo Deck or Tembo Galleria overlooking the sprawling city of Tokyo and beyond",
                "Tokyo Skytree with the Sumida River and surrounding urban landscape"
            ]
        },
        "senso_ji_temple": {
            "name": "Senso-ji Temple",
            "aliases": ["淺草寺", "Asakusa Kannon Temple"],
            "location": "Tokyo, Japan",
            "prompts": [
                "the vibrant red Senso-ji Temple in Asakusa, Tokyo, with its iconic Kaminarimon (Thunder Gate) featuring a massive red paper lantern",
                "traditional Japanese temple architecture of Senso-ji, Tokyo's oldest temple, including its five-story pagoda and main hall",
                "incense smoke billowing from a large cauldron and worshippers at Senso-ji Temple",
                "Nakamise-dori, the bustling market street leading to Senso-ji Temple, lined with traditional souvenir stalls and food vendors",
                "the Hozomon Gate with its giant waraji (straw sandals) at Senso-ji Temple"
            ]
        },
        "osaka_castle": {
            "name": "Osaka Castle",
            "aliases": ["大阪城", "Osaka-jo"],
            "location": "Osaka, Japan",
            "prompts": [
                "the majestic Osaka Castle with its distinctive white walls, green tiled roofs, golden embellishments, and surrounding moat and massive stone walls",
                "Osaka Castle Park with the imposing castle keep (tenshukaku) in the background, especially beautiful during cherry blossom season or autumn foliage",
                "historic Japanese castle, Osaka Castle, reconstructed with modern interiors, showcasing its historical significance",
                "Osaka Castle illuminated at night, reflecting in its expansive moat, creating a stunning visual",
                "view of Osaka Castle from Nishinomaru Garden, offering a picturesque perspective"
            ]
        },
        "dotonbori": {
            "name": "Dotonbori",
            "aliases": ["道頓堀"],
            "location": "Osaka, Japan",
            "prompts": [
                "the vibrant and eclectic Dotonbori entertainment district in Osaka, famous for its extravagant, oversized 3D signage like the Glico Running Man and Kani Doraku crab",
                "canal view of Dotonbori at night with a dazzling array of neon lights from billboards and signs reflecting on the Dotonbori River",
                "colorful and brightly lit billboards, including the iconic Glico Running Man, lining the Dotonbori canal, a symbol of Osaka's energy",
                "bustling atmosphere of Dotonbori with street food stalls offering takoyaki and okonomiyaki, and throngs of people",
                "the Ebisu Bridge over the Dotonbori canal, a popular meeting spot and photo location"
            ]
        },
        "arashiyama_bamboo_grove": {
            "name": "Arashiyama Bamboo Grove",
            "aliases": ["嵐山竹林", "Sagano Bamboo Forest"],
            "location": "Kyoto, Japan",
            "prompts": [
                "towering stalks of green bamboo creating a dense, immersive canopy over a pathway in Arashiyama Bamboo Grove, Kyoto",
                "sunlight filtering magically through the leaves of the tall, closely packed bamboo forest in Arashiyama",
                "serene and tranquil walking path through the iconic Arashiyama Bamboo Grove, with the distinctive sound of rustling bamboo leaves",
                "people in traditional kimono or yukata walking through the Arashiyama Bamboo Grove",
                "the unique, otherworldly atmosphere of the Arashiyama Bamboo Grove, a natural wonder"
            ]
        },
        "itsukushima_shrine": {
            "name": "Itsukushima Shrine",
            "aliases": ["嚴島神社", "Miyajima Shrine"],
            "location": "Miyajima Island, Hiroshima, Japan",
            "prompts": [
                "the iconic vermilion 'floating' torii gate of Itsukushima Shrine in Miyajima, appearing to float on the water during high tide",
                "Itsukushima Shrine complex, a UNESCO World Heritage site, built on stilts over the sea, with its distinctive red-lacquered corridors and halls",
                "sacred wild deer roaming freely around Miyajima Island with the Itsukushima Shrine and its torii gate in the background",
                "Itsukushima Shrine and its torii gate illuminated at night, creating a mystical scene",
                "view of Itsukushima Shrine from Mount Misen or from a ferry approaching Miyajima Island"
            ]
        },
        "gyeongbokgung_palace": {
            "name": "Gyeongbokgung Palace",
            "aliases": ["경복궁", "Gyeongbok Palace"],
            "location": "Seoul, South Korea",
            "prompts": [
                "a photo of Gyeongbokgung Palace in Seoul, the largest and most stunning of Seoul's five grand Joseon Dynasty palaces",
                "the main royal palace of the Joseon dynasty, Gyeongbokgung, with its grand Gwanghwamun Gate and Heungnyemun Gate",
                "traditional Korean architecture at Gyeongbokgung Palace, featuring intricate Dancheong (colorful painted patterns) on wooden structures and elegant curved roofs",
                "the impressive Geunjeongjeon Hall (Imperial Throne Hall) and the picturesque Gyeonghoeru Pavilion (Royal Banquet Hall) on a pond at Gyeongbokgung Palace",
                "Changing of the Royal Guard ceremony (Sumunjang Gyedaeui) taking place at Gyeongbokgung Palace"
            ]
        },
        "n_seoul_tower": {
            "name": "N Seoul Tower",
            "aliases": ["N서울타워", "YTN Seoul Tower", "Namsan Tower"],
            "location": "Seoul, South Korea",
            "prompts": [
                "a photo of N Seoul Tower perched atop Namsan Mountain, offering panoramic views of Seoul",
                "N Seoul Tower illuminated with vibrant LED lights at night, changing colors and patterns, visible from across the city",
                "the iconic N Seoul Tower in South Korea, a popular landmark with its observation deck and 'love locks' fences",
                "view from the N Seoul Tower looking down on the sprawling cityscape of Seoul, day or night",
                "cable car ascending Namsan Mountain towards the N Seoul Tower"
            ]
        },
        "bukchon_hanok_village": {
            "name": "Bukchon Hanok Village",
            "aliases": ["북촌한옥마을", "Traditional Korean Village"],
            "location": "Seoul, South Korea",
            "prompts": [
                "a photo of Bukchon Hanok Village in Seoul, a preserved traditional Korean village with hundreds of hanok (traditional Korean houses)",
                "traditional Korean houses (hanok) in Bukchon Village, featuring distinctive tiled roofs, wooden beams, and courtyards, nestled on a hillside",
                "narrow, winding alleyways and cobblestone streets of Bukchon Hanok Village, offering glimpses into historic Seoul",
                "view of modern Seoul skyline contrasting with the traditional rooftops of Bukchon Hanok Village",
                "people wearing Hanbok (traditional Korean attire) walking through Bukchon Hanok Village"
            ]
        },
        "myeongdong_shopping_street": {
            "name": "Myeongdong Shopping Street",
            "aliases": ["명동 쇼핑거리", "Myeong-dong"],
            "location": "Seoul, South Korea",
            "prompts": [
                "bustling Myeongdong shopping street in Seoul, a paradise for cosmetics, fashion, and K-pop merchandise, packed with shoppers and vibrant storefronts",
                "crowds of shoppers navigating the pedestrian-friendly streets of Myeongdong, with bright neon signs and music from stores creating a lively atmosphere, especially at night",
                "vibrant street food scene in Myeongdong, Seoul, with numerous stalls offering popular Korean snacks like tteokbokki, gyeranppang, and tornado potatoes",
                "Myeongdong Cathedral, a historic Gothic-style church, standing amidst the bustling modern shopping district",
                "large department stores and international brands alongside local boutiques in Myeongdong"
            ]
        },
        "dmz_korea": {
            "name": "DMZ (Korean Demilitarized Zone)",
            "aliases": ["韓國非軍事區", "Panmunjom", "JSA"],
            "location": "Gyeonggi-do, South Korea / North Korea",
            "prompts": [
                "the heavily fortified Korean Demilitarized Zone (DMZ) separating North and South Korea, with barbed wire fences and guard posts",
                "Joint Security Area (JSA) or Panmunjom at the DMZ, with soldiers from North and South Korea facing each other across the Military Demarcation Line",
                "the iconic blue conference buildings straddling the border within the JSA at the DMZ",
                "tense and somber atmosphere of the DMZ, a symbol of the Korean War and the divided peninsula",
                "observation posts like Dora Observatory overlooking North Korean territory from the DMZ"
            ]
        },
        "busan_gamcheon_culture_village": {
            "name": "Busan Gamcheon Culture Village",
            "aliases": ["부산 감천문화마을", "Machu Picchu of Busan", "Taegukdo Village"],
            "location": "Busan, South Korea",
            "prompts": [
                "colorful houses built in terraced fashion on a steep hillside in Gamcheon Culture Village, Busan, often called the 'Machu Picchu of Busan'",
                "narrow, winding alleyways adorned with vibrant street art, colorful murals, and art installations in Gamcheon Culture Village",
                "panoramic view of the brightly painted houses of Gamcheon Culture Village cascading down to the sea, creating a unique urban landscape",
                "sculptures like 'The Little Prince and the Desert Fox' overlooking the village in Gamcheon Culture Village",
                "artistic and quirky atmosphere of Gamcheon Culture Village, a regenerated slum transformed into an art hub"
            ]
        },
        "jeju_island": {
            "name": "Jeju Island",
            "aliases": ["제주도", "Jejudo"],
            "location": "Jeju Province, South Korea",
            "prompts": [
                "volcanic landscape of Jeju Island, South Korea, a UNESCO World Heritage site, featuring black basalt rock formations (like Jusangjeolli Cliff) and lush greenery",
                "Seongsan Ilchulbong (Sunrise Peak), a dramatic tuff cone crater rising from the sea on the eastern coast of Jeju Island",
                "beautiful sandy beaches like Hyeopjae Beach, waterfalls such as Cheonjeyeon Falls, and lava tubes like Manjanggul Cave on Jeju Island",
                "Hallasan National Park, home to South Korea's highest mountain, Hallasan, a dormant shield volcano, on Jeju Island",
                "Dol Hareubang (stone grandfathers), iconic black lava stone statues found throughout Jeju Island"
            ]
        },
        "changdeokgung_palace_secret_garden": {
            "name": "Changdeokgung Palace & Secret Garden",
            "aliases": ["창덕궁과 후원", "Donggwol (East Palace)"],
            "location": "Seoul, South Korea",
            "prompts": [
                "the beautiful Changdeokgung Palace, a UNESCO World Heritage site in Seoul, known for its harmonious design that blends with the natural landscape",
                "traditional Korean palace architecture of Changdeokgung, including the Injeongjeon (main throne hall) and Donhwamun (main gate)",
                "the serene and picturesque Secret Garden (Huwon) of Changdeokgung Palace, a vast rear garden with pavilions, ponds, and ancient trees, requiring a guided tour",
                "Buyongjeong Pavilion and Buyongji Pond in the Secret Garden of Changdeokgung Palace",
                "Changdeokgung Palace as one of the best-preserved Joseon Dynasty palaces"
            ]
        },
        "great_wall": {
            "name": "Great Wall of China",
            "aliases": ["长城", "The Great Wall", "萬里長城"],
            "location": "China",
            "prompts": [
                "a photo of the Great Wall of China, the massive ancient fortification winding across rugged mountains and diverse terrains",
                "the Great Wall stretching for thousands of miles, with watchtowers and battlements, a testament to ancient Chinese engineering",
                "sections of the Great Wall like Badaling or Mutianyu, showing its impressive scale and historical significance",
                "the Great Wall of China snaking along steep mountain ridges, often covered in snow or surrounded by lush greenery",
                "iconic man-made structure, the Great Wall, visible from afar, symbolizing China's strength and history"
            ]
        },
        "forbidden_city": {
            "name": "Forbidden City",
            "aliases": ["紫禁城", "Palace Museum", "故宫博物院"],
            "location": "Beijing, China",
            "prompts": [
                "a photo of the Forbidden City in Beijing, the vast imperial palace complex with its iconic red walls and yellow-tiled roofs",
                "the imperial palace complex of the Forbidden City, former home to Chinese emperors, showcasing classical Chinese palatial architecture",
                "courtyards, halls, and gates of the Forbidden City, such as the Hall of Supreme Harmony, arranged along a central axis",
                "intricate details and symbolic ornamentation of the Forbidden City's buildings, reflecting imperial power and Chinese cosmology",
                "the Meridian Gate (Wumen), the main entrance to the Forbidden City, or the Corner Towers with their complex roof structures"
            ]
        },
        "terracotta_army": {
            "name": "Terracotta Army",
            "aliases": ["兵马俑", "Terracotta Warriors and Horses"],
            "location": "Xi'an, China",
            "prompts": [
                "a photo of the Terracotta Army in Xi'an, rows of life-sized terracotta warriors, archers, and chariots in vast pits",
                "thousands of life-sized terracotta warriors and horses, each with unique facial expressions and armor, part of Qin Shi Huang's mausoleum",
                "archaeological site of the Terracotta Army, a UNESCO World Heritage site, showcasing the incredible artistry and scale of ancient Chinese burial practices",
                "close-up of the detailed faces and uniforms of the Terracotta Warriors",
                "excavation pits containing the Terracotta Army, demonstrating the ongoing discovery and preservation efforts"
            ]
        },
        "the_bund": {
            "name": "The Bund",
            "aliases": ["外滩", "Waitan"],
            "location": "Shanghai, China",
            "prompts": [
                "a photo of The Bund waterfront in Shanghai, with its impressive collection of historic colonial-era buildings in various architectural styles",
                "skyline of The Bund featuring iconic early 20th-century buildings, contrasting with the modern skyscrapers of Pudong across the Huangpu River",
                "The Bund overlooking the Huangpu River, with boats and ferries, and the Oriental Pearl Tower and Shanghai Tower visible on the Pudong side",
                "pedestrian promenade along The Bund, a popular spot for tourists and locals to view Shanghai's historic and modern skylines",
                "The Bund at night, with both the historic buildings and the Pudong skyline brilliantly illuminated"
            ]
        },
        "li_river_guilin": {
            "name": "Li River, Guilin",
            "aliases": ["漓江", "Guilin Karst Landscape"],
            "location": "Guilin, Guangxi, China",
            "prompts": [
                "surreal karst mountain landscape along the Li River in Guilin, China, with picturesque limestone peaks rising dramatically from the flat terrain",
                "bamboo rafts and cruise boats navigating the scenic Li River, surrounded by mist-shrouded, unusually shaped karst formations",
                "iconic view of the Li River, often depicted in traditional Chinese landscape paintings, particularly the scene on the 20 Yuan banknote (Yellow Cloth Shoal)",
                "cormorant fishermen on bamboo rafts on the Li River",
                "lush green vegetation covering the steep karst hills along the banks of the winding Li River"
            ]
        },
        "potala_palace": {
            "name": "Potala Palace",
            "aliases": ["布達拉宮"],
            "location": "Lhasa, Tibet, China",
            "prompts": [
                "the majestic Potala Palace, former winter residence of the Dalai Lama, dramatically situated on Red Hill (Marpo Ri) in Lhasa, Tibet",
                "distinctive white (White Palace) and red (Red Palace) architecture of the Potala Palace against a clear blue sky and surrounding mountains",
                "the imposing Potala Palace, a UNESCO World Heritage site, a symbol of Tibetan Buddhism and a marvel of Tibetan architecture with its many chapels and statues",
                "Potala Palace with prayer flags fluttering in the foreground",
                "view of Potala Palace from afar, showcasing its grand scale and dominant position over Lhasa"
            ]
        },
        "zhangjiajie_national_forest_park": {
            "name": "Zhangjiajie National Forest Park",
            "aliases": ["張家界國家森林公園", "Avatar Mountains"],
            "location": "Hunan, China",
            "prompts": [
                "tall, pillar-like quartz-sandstone formations, often shrouded in mist, in Zhangjiajie National Forest Park, inspiration for the 'Avatar Hallelujah Mountains'",
                "the 'Avatar Hallelujah Mountains' (formerly Southern Sky Column) in Zhangjiajie, known for their gravity-defying appearance and lush vegetation",
                "glass bridges like the Zhangjiajie Grand Canyon Glass Bridge, and cable cars offering stunning, vertigo-inducing views of the unique landscape of Zhangjiajie",
                "Bailong Elevator (Hundred Dragons Elevator), a massive glass elevator built onto the side of a cliff in Zhangjiajie",
                "dense forests and deep ravines characterize the otherworldly scenery of Zhangjiajie National Forest Park"
            ]
        },
        "west_lake_hangzhou": {
            "name": "West Lake, Hangzhou",
            "aliases": ["西湖"],
            "location": "Hangzhou, China",
            "prompts": [
                "the serene and beautiful West Lake in Hangzhou, China, a UNESCO World Heritage site, famous for its scenic beauty, pagodas, islands, and causeways",
                "iconic landmarks of West Lake such as the Broken Bridge (Duan Qiao), Leifeng Pagoda, Su Causeway, and Bai Causeway",
                "traditional Chinese pavilions, arched bridges, and gardens surrounding West Lake, often depicted in Chinese art and poetry",
                "boats gently gliding on West Lake, with willow trees lining its shores and lotus flowers blooming in summer",
                "Three Ponds Mirroring the Moon, one of the most famous sights of West Lake"
            ]
        },
        "summer_palace_beijing": {
            "name": "Summer Palace, Beijing",
            "aliases": ["頤和園", "Yiheyuan"],
            "location": "Beijing, China",
            "prompts": [
                "the vast imperial garden of the Summer Palace in Beijing, a UNESCO World Heritage site, featuring Kunming Lake and Longevity Hill",
                "ornate palaces, temples, bridges, and pavilions within the Summer Palace, such as the Marble Boat, the Long Corridor (Chang Lang), and the Tower of Buddhist Incense (Foxiang Ge)",
                "traditional Chinese landscape garden design of the Summer Palace, showcasing harmony between man-made structures and nature",
                "Kunming Lake in the Summer Palace, with the Seventeen-Arch Bridge leading to Nanhu Island",
                "Longevity Hill at the Summer Palace, crowned by impressive imperial buildings"
            ]
        },
        "petronas_towers": {
            "name": "Petronas Twin Towers",
            "aliases": ["KLCC Twin Towers", "Menara Petronas", "Petronas Towers"],
            "location": "Kuala Lumpur, Malaysia",
            "prompts": [
                "a photo of the Petronas Twin Towers in Kuala Lumpur, the world's tallest twin skyscrapers with their distinctive postmodern Islamic architectural motifs",
                "the iconic Petronas Towers connected by a double-decker skybridge on the 41st and 42nd floors",
                "Petronas Twin Towers brilliantly illuminated at night, a striking landmark in Kuala Lumpur's skyline",
                "the multi-faceted, tapering design of the Petronas Twin Towers, inspired by Islamic geometric patterns",
                "view of the Petronas Twin Towers from KLCC Park, with its fountains and gardens"
            ]
        },
        "marina_bay_sands": {
            "name": "Marina Bay Sands",
            "aliases": ["MBS", "Singapore Skypark"],
            "location": "Singapore",
            "prompts": [
                "a photo of Marina Bay Sands in Singapore, the iconic integrated resort featuring three soaring hotel towers topped by a massive cantilevered SkyPark",
                "the Marina Bay Sands resort with its distinctive three towers and the ship-like Sands SkyPark housing an infinity pool, observation deck, and gardens",
                "Marina Bay Sands overlooking the Singapore skyline and Marina Bay, often seen with the ArtScience Museum in the foreground",
                "the impressive architecture of Marina Bay Sands, a symbol of modern Singapore, illuminated at night",
                "Spectra light and water show in front of Marina Bay Sands"
            ]
        },
        "gardens_by_the_bay": {
            "name": "Gardens by the Bay",
            "aliases": ["Singapore Gardens", "Supertree Grove"],
            "location": "Singapore",
            "prompts": [
                "a photo of Gardens by the Bay in Singapore, a futuristic nature park known for its iconic Supertree Grove",
                "the towering Supertree Grove at Gardens by the Bay, tree-like vertical gardens that light up spectacularly during the Garden Rhapsody show at night",
                "Flower Dome and Cloud Forest, two massive cooled conservatories at Gardens by the Bay, housing diverse plant life from around the world",
                "the OCBC Skyway, an aerial walkway offering panoramic views of the Supertrees and surrounding gardens",
                "lush landscapes and innovative horticultural displays within Gardens by the Bay"
            ]
        },
        "taj_mahal": {
            "name": "Taj Mahal",
            "aliases": ["Crown of Palaces", "Mumtaz Mahal"],
            "location": "Agra, India",
            "prompts": [
                "a photo of the Taj Mahal in Agra, the iconic ivory-white marble mausoleum renowned for its beauty and symmetry",
                "the ivory-white marble mausoleum of the Taj Mahal, a UNESCO World Heritage site, built by Mughal emperor Shah Jahan",
                "Taj Mahal reflected perfectly in its long सामने (frontal) water feature (reflecting pool), flanked by symmetrical gardens",
                "the intricate marble inlay work (pietra dura) and calligraphy on the facade of the Taj Mahal",
                "the central dome and four minarets of the Taj Mahal, a masterpiece of Mughal architecture"
            ]
        },
        "angkor_wat": {
            "name": "Angkor Wat",
            "aliases": ["吳哥窟", "City of Temples"],
            "location": "Siem Reap, Cambodia",
            "prompts": [
                "the magnificent temple complex of Angkor Wat in Cambodia, a UNESCO World Heritage site, with its iconic five lotus-bud shaped towers reflecting in a surrounding moat",
                "intricate bas-reliefs and stone carvings depicting Hindu epics like the Ramayana and Mahabharata, and apsaras (celestial dancers) at Angkor Wat",
                "sunrise over Angkor Wat, casting a golden glow on its ancient stone structures, the world's largest religious monument",
                "the grand scale and symmetrical design of Angkor Wat, an architectural marvel of the Khmer Empire",
                "causeway leading to the main entrance of Angkor Wat, often with monks in saffron robes"
            ]
        },
        "ha_long_bay": {
            "name": "Ha Long Bay",
            "aliases": ["下龍灣", "Vịnh Hạ Long"],
            "location": "Quảng Ninh Province, Vietnam",
            "prompts": [
                "thousands of towering limestone karsts and islets rising dramatically from the emerald green waters of Ha Long Bay, Vietnam, a UNESCO World Heritage site",
                "traditional Vietnamese junk boats sailing majestically through the stunning seascape of Ha Long Bay",
                "caves and grottoes, such as Thien Cung Cave or Dau Go Cave, hidden within the limestone islands of Ha Long Bay",
                "floating fishing villages and pearl farms nestled among the karsts in Ha Long Bay",
                "misty and ethereal atmosphere of Ha Long Bay, especially at dawn or dusk"
            ]
        },
        "mount_everest": {
            "name": "Mount Everest",
            "aliases": ["聖母峰", "Sagarmatha", "Chomolungma"],
            "location": "Mahalangur Himal, Nepal/China border",
            "prompts": [
                "the snow-covered, pyramid-shaped peak of Mount Everest, the world's highest mountain, towering above the Himalayan range",
                "climbers and expeditions at Mount Everest Base Camp, with prayer flags and views of the Khumbu Icefall",
                "majestic and formidable landscape of Mount Everest and surrounding Himalayan peaks like Lhotse and Nuptse, often with wispy clouds",
                "the challenging south face or north face routes leading to the summit of Mount Everest",
                "breathtaking aerial view of Mount Everest, highlighting its immense scale and icy terrain"
            ]
        },
        "bagan": {
            "name": "Bagan",
            "aliases": ["蒲甘", "Pagan"],
            "location": "Mandalay Region, Myanmar",
            "prompts": [
                "thousands of ancient Buddhist temples, pagodas, and stupas, mostly in reddish-brown brick, spread across the plains of Bagan, Myanmar, a UNESCO World Heritage site",
                "hot air balloons drifting gracefully over the temple-studded landscape of Bagan at sunrise or sunset, creating a magical scene",
                "archaeological zone of Bagan with its diverse stupas (like Shwezigon Pagoda) and temples (like Ananda Temple) dating back to the 9th-13th centuries",
                "sunlight illuminating the ancient temples of Bagan, with intricate carvings and Buddha statues inside",
                "view from a high temple in Bagan, offering a panoramic vista of countless religious structures"
            ]
        },
        "grand_palace_wat_phra_kaew": {
            "name": "The Grand Palace & Wat Phra Kaew",
            "aliases": ["曼谷大皇宮與玉佛寺", "Royal Palace Bangkok"],
            "location": "Bangkok, Thailand",
            "prompts": [
                "ornate and glittering architecture of the Grand Palace in Bangkok, former residence of the Kings of Siam, with its intricate details and golden spires",
                "Wat Phra Kaew (Temple of the Emerald Buddha) within the Grand Palace complex, housing the highly revered Emerald Buddha statue carved from a single jade stone",
                "intricate details, golden chedis (stupas), colorful mosaics made of glass and porcelain, and mythical guardian statues (yakshas) of Thai traditional architecture at the Grand Palace",
                "the dazzling exteriors of the royal halls and temples within the Grand Palace grounds",
                "crowds of visitors and worshippers at the magnificent Grand Palace and Wat Phra Kaew"
            ]
        }
    },

    # 歐洲地標
    "europe": {
        "eiffel_tower": {
            "name": "Eiffel Tower",
            "aliases": ["Tour Eiffel", "The Iron Lady"],
            "location": "Paris, France",
            "prompts": [
                "a photo of the Eiffel Tower in Paris, the iconic wrought-iron lattice tower on the Champ de Mars",
                "the iconic Eiffel Tower structure, its intricate ironwork and graceful curves against the Paris skyline",
                "Eiffel Tower illuminated at night with its sparkling light show, a beacon in the City of Lights",
                "view from the top of the Eiffel Tower overlooking Paris, including the Seine River and landmarks like the Arc de Triomphe",
                "Eiffel Tower seen from the Trocadéro, providing a classic photographic angle"
            ]
        },
        "louvre_museum": {
            "name": "Louvre Museum",
            "aliases": ["Musée du Louvre", "The Louvre Pyramid"],
            "location": "Paris, France",
            "prompts": [
                "a photo of the Louvre Museum in Paris, with its iconic glass pyramid designed by I. M. Pei contrasting with the historic Louvre Palace",
                "the Louvre Pyramid at the entrance of the museum, reflecting the sky and surrounding palace wings",
                "exterior of the historic Louvre Palace, a former royal palace, now one of the world's largest art museums",
                "crowds of visitors entering the Louvre Museum through the pyramid or its underground entrance",
                "the Louvre Museum at night, with the pyramid and palace illuminated"
            ]
        },
        "mont_saint_michel": {
            "name": "Mont Saint-Michel",
            "aliases": ["Saint Michael's Mount (France)", "Abbaye du Mont-Saint-Michel"],
            "location": "Normandy, France",
            "prompts": [
                "a photo of Mont Saint-Michel in France, the stunning tidal island commune topped by a medieval Benedictine abbey",
                "the tidal island and abbey of Mont Saint-Michel rising dramatically from the bay, surrounded by water at high tide or sand flats at low tide",
                "Mont Saint-Michel at high tide, appearing as a fairytale castle floating on the water",
                "the Gothic architecture of the Abbey of Mont Saint-Michel, with its towering spire and fortified walls",
                "narrow winding streets and historic buildings leading up to the abbey on Mont Saint-Michel"
            ]
        },
        "arc_de_triomphe": {
            "name": "Arc de Triomphe",
            "aliases": ["Triumphal Arch of the Star", "Arc de Triomphe de l'Étoile"],
            "location": "Paris, France",
            "prompts": [
                "a photo of the Arc de Triomphe in Paris, the monumental triumphal arch at the center of Place Charles de Gaulle (Place de l'Étoile)",
                "the iconic Arc de Triomphe at the western end of the Champs-Élysées, adorned with intricate sculptures depicting Napoleonic victories",
                "view from the top of the Arc de Triomphe, looking down the twelve radiating avenues, including the Champs-Élysées towards the Louvre",
                "the Eternal Flame burning beneath the Arc de Triomphe at the Tomb of the Unknown Soldier",
                "Arc de Triomphe illuminated at night, a symbol of French national pride"
            ]
        },
        "big_ben": {
            "name": "Big Ben (Elizabeth Tower)", # Clarified name
            "aliases": ["Elizabeth Tower", "Westminster Clock Tower", "Clock Tower, Palace of Westminster"],
            "location": "London, UK",
            "prompts": [
                "a photo of Big Ben (Elizabeth Tower) clock tower with the Houses of Parliament (Palace of Westminster) in London, on the bank of the River Thames",
                "the iconic Gothic Revival clock tower Big Ben with its four massive, illuminated clock faces in London",
                "Big Ben tower with its distinctive golden clock face, intricate stonework, and cast-iron spire, a symbol of London and the UK",
                "the famous Westminster clock tower Big Ben in Gothic revival style, meticulously restored with its original blue clock hands",
                "close-up of Big Ben's clock face showing the Roman numerals and detailed craftsmanship"
            ]
        },
        "stonehenge": {
            "name": "Stonehenge",
            "aliases": ["Prehistoric Monument", "Ring of Stones"],
            "location": "Wiltshire, UK",
            "prompts": [
                "a photo of Stonehenge in the UK, the mysterious prehistoric monument of massive standing stones arranged in a circular formation",
                "the prehistoric stone circle of Stonehenge, a UNESCO World Heritage site, with its sarsens and bluestones",
                "Stonehenge at sunset or sunrise, with dramatic lighting casting long shadows over the ancient stones",
                "the unique trilithons (two upright stones topped by a lintel) of Stonehenge",
                "the enigmatic landscape surrounding Stonehenge on Salisbury Plain"
            ]
        },
        "tower_of_london": {
            "name": "Tower of London",
            "aliases": ["His Majesty's Royal Palace and Fortress of the Tower of London", "The Tower"],
            "location": "London, UK",
            "prompts": [
                "a photo of the Tower of London, the historic medieval castle on the north bank of the River Thames",
                "the imposing White Tower, the central keep of the Tower of London, with its Norman architecture",
                "Crown Jewels of the United Kingdom on display within the Tower of London",
                "Yeoman Warders ('Beefeaters') in their traditional Tudor uniforms at the Tower of London",
                "Traitors' Gate at the Tower of London, a famous water gate leading from the Thames"
            ]
        },
        "buckingham_palace": {
            "name": "Buckingham Palace",
            "aliases": ["British Royal Residence", "The Palace"],
            "location": "London, UK",
            "prompts": [
                "a photo of Buckingham Palace in London, the official residence and administrative headquarters of the monarch of the United Kingdom",
                "the iconic facade of Buckingham Palace with the Queen's Guard (King's Guard) in their red tunics and bearskin hats",
                "Changing of the Guard ceremony taking place in the forecourt of Buckingham Palace, a popular tourist attraction",
                "the Victoria Memorial statue in front of Buckingham Palace",
                "Buckingham Palace with the Royal Standard flag flying, indicating the monarch is in residence"
            ]
        },
        "colosseum": {
            "name": "Colosseum",
            "aliases": ["Roman Colosseum", "Flavian Amphitheatre", "Colosseo"],
            "location": "Rome, Italy",
            "prompts": [
                "a photo of the Colosseum in Rome, the massive ancient Roman amphitheater, an icon of Imperial Rome",
                "the ancient Roman Colosseum structure, with its elliptical shape, tiered seating, and arched exterior, partly in ruins",
                "historic Colosseum amphitheater in Italy, where gladiatorial contests and public spectacles were held",
                "the interior of the Colosseum, showing the hypogeum (underground structures) and remaining seating areas",
                "the Colosseum illuminated at night, a powerful symbol of Roman history"
            ]
        },
        "leaning_tower_of_pisa": {
            "name": "Leaning Tower of Pisa",
            "aliases": ["Torre pendente di Pisa", "Tower of Pisa"],
            "location": "Pisa, Tuscany, Italy",
            "prompts": [
                "a photo of the Leaning Tower of Pisa in Italy, the iconic white marble freestanding bell tower (campanile) of Pisa Cathedral, famous for its significant unintended tilt",
                "the world-famous leaning cylindrical campanile of Pisa Cathedral, built of white marble with Romanesque architecture and multiple tiers of arched colonnades, noticeably tilted to one side",
                "tourists taking humorous forced perspective photos with the Leaning Tower of Pisa",
                "the Leaning Tower of Pisa located in the Piazza dei Miracoli (Square of Miracles), alongside the Duomo (cathedral) and Baptistery",
                "the white marble cylindrical structure of the Leaning Tower of Pisa with its distinctive arched galleries, showing its dramatic lean",
                "the iconic leaning cylindrical bell tower with six tiers of open galleries with arches and columns, against a blue sky, in Pisa, Italy",
                "a global tourist landmark: Italy's Leaning Tower of Pisa, a tilted white marble tower with intricate Romanesque arcades",
                "side view emphasizing the dramatic four-degree angle of the Leaning Tower's tilt, showcasing its unique structural imbalance",
                "ornate white marble cylindrical tower with multiple levels of columns, visibly leaning to one side, a masterpiece of medieval engineering",
                "the famous tilted bell tower of Pisa with its many columned galleries viewed from below, highlighting its precarious stance",
                "detailed close-up of the Leaning Tower of Pisa's distinctive multi-tiered arched colonnades and ornate architectural details in white marble",
                "the freestanding belltower of Pisa Cathedral set on the green grass of Piazza dei Miracoli, with its visible foundation on the low side where it sinks into the ground",
                "a photo of the white marble Leaning Tower of Pisa, known for its nearly four-degree lean, a UNESCO World Heritage Site in Tuscany"
            ]
        },
        "trevi_fountain": {
            "name": "Trevi Fountain",
            "aliases": ["Fontana di Trevi"],
            "location": "Rome, Italy",
            "prompts": [
                "a photo of the Trevi Fountain in Rome, the largest Baroque fountain in the city and one of the most famous fountains in the world",
                "the spectacular Baroque Trevi Fountain with its grand sculptures, including Oceanus, tritons, and horses, set against the Palazzo Poli",
                "people throwing coins over their shoulders into the Trevi Fountain, a tradition said to ensure a return to Rome",
                "the Trevi Fountain illuminated at night, showcasing its dramatic statues and cascading water",
                "the vibrant turquoise water of the Trevi Fountain contrasting with its white travertine stone"
            ]
        },
        "st_peters_basilica": {
            "name": "St. Peter's Basilica",
            "aliases": ["Basilica di San Pietro", "Vatican Basilica"],
            "location": "Vatican City",
            "prompts": [
                "a photo of St. Peter's Basilica in Vatican City, the immense Renaissance church, one of the largest and most renowned churches in the world",
                "the magnificent dome of St. Peter's Basilica, designed by Michelangelo, dominating the skyline of Rome and Vatican City",
                "St. Peter's Square (Piazza San Pietro), designed by Bernini, with its grand colonnades, obelisk, and fountains, leading to the basilica",
                "the lavish interior of St. Peter's Basilica, featuring masterpieces like Michelangelo's Pietà and Bernini's Baldachin",
                "St. Peter's Basilica viewed from Via della Conciliazione or from the top of its dome"
            ]
        },
        "sagrada_familia": {
            "name": "Sagrada Familia",
            "aliases": ["Basílica de la Sagrada Família", "Gaudi's Church"],
            "location": "Barcelona, Spain",
            "prompts": [
                "a photo of Sagrada Familia in Barcelona, Antoni Gaudí's unfinished masterpiece of Catalan Modernism, a Roman Catholic minor basilica",
                "the unique and highly ornate architecture of Gaudí's Sagrada Familia, with its towering spires, intricate facades (Nativity, Passion, Glory), and organic forms",
                "construction cranes still present on the perpetually evolving Sagrada Familia",
                "the stunning interior of Sagrada Familia, with its tree-like columns and vibrant stained-glass windows creating a forest of light",
                "close-up details of the sculptural elements and symbolic decorations on the facades of Sagrada Familia"
            ]
        },
        "alhambra": {
            "name": "Alhambra",
            "aliases": ["Alhambra Palace", "The Red Fortress", "Alhambra of Granada"],
            "location": "Granada, Spain",
            "prompts": [
                "a photo of the Alhambra palace and fortress complex in Granada, Spain, a stunning example of Moorish architecture",
                "exquisite Moorish architecture of the Alhambra, featuring intricate stucco work, geometric tile patterns (azulejos), and delicate courtyards like the Court of the Lions",
                "courtyards, palaces (Nasrid Palaces), and gardens (Generalife) of the Alhambra, showcasing Islamic art and design",
                "the Alhambra perched on a hill overlooking the city of Granada, with the Sierra Nevada mountains in the background",
                "the red-hued walls of the Alcazaba fortress within the Alhambra complex"
            ]
        },
        "brandenburg_gate": {
            "name": "Brandenburg Gate",
            "aliases": ["Brandenburger Tor", "Berlin Gate"],
            "location": "Berlin, Germany",
            "prompts": [
                "a photo of the Brandenburg Gate in Berlin, the iconic neoclassical triumphal arch, a symbol of German reunification and peace",
                "the neoclassical Brandenburg Gate monument, with its Doric columns and the Quadriga (a chariot drawn by four horses) скульптура наверху",
                "Brandenburg Gate illuminated at night, standing at the end of Unter den Linden boulevard",
                "historical significance of the Brandenburg Gate, once a symbol of division during the Cold War",
                "Pariser Platz in front of the Brandenburg Gate, a lively public square"
            ]
        },
        "neuschwanstein_castle": {
            "name": "Neuschwanstein Castle",
            "aliases": ["Schloss Neuschwanstein", "Fairy Tale Castle", "Mad King Ludwig's Castle"],
            "location": "Bavaria, Germany",
            "prompts": [
                "a photo of Neuschwanstein Castle in Germany, the quintessential fairytale castle with its white limestone facade and blue turrets, inspiring Disney's Sleeping Beauty Castle",
                "the fairytale Neuschwanstein Castle dramatically nestled in the Bavarian Alps, perched on a rugged hill overlooking the Hohenschwangau valley",
                "Neuschwanstein Castle on a hill, often viewed from Marienbrücke (Mary's Bridge) for a classic postcard shot",
                "the Romanesque Revival architecture of Neuschwanstein Castle, commissioned by King Ludwig II of Bavaria",
                "Neuschwanstein Castle surrounded by autumn foliage or dusted with snow in winter"
            ]
        },
        "acropolis_of_athens": {
            "name": "Acropolis of Athens",
            "aliases": ["Ακρόπολη Αθηνών", "Parthenon", "Sacred Rock"],
            "location": "Athens, Greece",
            "prompts": [
                "a photo of the Acropolis of Athens in Greece, the ancient citadel located on a rocky outcrop above the city, crowned by the Parthenon",
                "the Parthenon, the iconic Doric temple dedicated to the goddess Athena, standing majestically on the Acropolis",
                "ancient ruins of the Acropolis overlooking the sprawling city of Athens, including the Erechtheion with its Porch of the Caryatids, and the Propylaea",
                "the Acropolis illuminated at night, a symbol of ancient Greek civilization and democracy",
                "view of the Acropolis from Filopappou Hill or Lycabettus Hill"
            ]
        },
        "santorini_oia": { # Specified Oia for iconic view
            "name": "Santorini (Oia)",
            "aliases": ["Thera", "Greek Islands", "Oia village Santorini"],
            "location": "Cyclades, Greece",
            "prompts": [
                "a photo of Oia village in Santorini island, Greece, famous for its whitewashed cave houses and blue-domed churches clinging to cliffs above the Aegean Sea",
                "iconic whitewashed villages with blue domes in Oia, Santorini, overlooking the volcanic caldera",
                "breathtaking sunset over the caldera in Oia, Santorini, a world-famous romantic spectacle",
                "narrow winding pathways and steps through the picturesque village of Oia",
                "bougainvillea flowers adding splashes of color to the white buildings of Santorini"
            ]
        },
        "canals_of_venice": {
            "name": "Canals of Venice",
            "aliases": ["Venetian Canals", "Rialto Bridge Venice"],
            "location": "Venice, Italy",
            "prompts": [
                "gondolas gliding gracefully through the narrow, winding canals of Venice, Italy, lined with historic, colorful buildings",
                "the Grand Canal in Venice, its main waterway, bustling with vaporettos (water buses), water taxis, and gondolas, spanned by the iconic Rialto Bridge",
                "romantic atmosphere of Venetian canals, reflecting the unique architecture of the sinking city, with smaller bridges connecting walkways",
                "picturesque scenes of Venice's canals, often with laundry hanging between buildings or flower boxes on windowsills",
                "Rio di Palazzo with the Bridge of Sighs connecting the Doge's Palace to the prisons"
            ]
        },
        "florence_cathedral_duomo": {
            "name": "Florence Cathedral (Duomo)",
            "aliases": ["Cattedrale di Santa Maria del Fiore", "Il Duomo di Firenze", "Brunelleschi's Dome"],
            "location": "Florence, Italy",
            "prompts": [
                "Brunelleschi's massive, iconic red-tiled dome atop the Florence Cathedral (Santa Maria del Fiore), dominating the city skyline of Florence",
                "the intricate Gothic and Renaissance facade of the Florence Duomo, made of white, green, and pink marble, alongside Giotto's Campanile (bell tower) and the Baptistery of St. John",
                "panoramic view of Florence from the top of Brunelleschi's Dome or Giotto's Campanile, showcasing the sea of red-tiled roofs and the Arno River",
                "the octagonal Florence Baptistery with its famous bronze doors, particularly Ghiberti's 'Gates of Paradise'",
                "the grand interior of Florence Cathedral, with its vast nave and frescoes, including Vasari's 'Last Judgment' inside the dome"
            ]
        },
        "anne_frank_house": {
            "name": "Anne Frank House",
            "aliases": ["Anne Frank Huis", "Secret Annex Amsterdam"],
            "location": "Amsterdam, Netherlands",
            "prompts": [
                "the unassuming exterior of the Anne Frank House, a canal house on the Prinsengracht in Amsterdam, where Anne Frank and her family hid during WWII",
                "the secret annex (Achterhuis) hidden behind a movable bookcase in the Anne Frank House, where the Frank family lived in hiding",
                "poignant historical site of the Anne Frank House, now a biographical museum dedicated to Jewish wartime diarist Anne Frank and the Holocaust",
                "long queues of visitors outside the Anne Frank House, waiting to enter the museum",
                "the preserved rooms and exhibits within the Anne Frank House, telling the story of Anne's life and diary"
            ]
        },
        "canals_of_amsterdam": {
            "name": "Canals of Amsterdam",
            "aliases": ["Grachtengordel Amsterdam", "Amsterdam Canal Ring"],
            "location": "Amsterdam, Netherlands",
            "prompts": [
                "picturesque canals of Amsterdam, part of the Grachtengordel (canal belt), a UNESCO World Heritage site, lined with narrow, gabled canal houses and houseboats",
                "bicycles parked along the charming bridges that cross Amsterdam's numerous canals, often adorned with flowers",
                "canal cruise boats navigating the historic waterways of Amsterdam, offering views of 17th-century architecture",
                "tree-lined canals of Amsterdam in different seasons, reflecting the elegant facades of the canal houses",
                "the distinctive architecture of Amsterdam's canal houses, with their narrow fronts and decorative gables"
            ]
        },
        "charles_bridge_prague": {
            "name": "Charles Bridge",
            "aliases": ["Karlův most", "Prague Stone Bridge"],
            "location": "Prague, Czech Republic",
            "prompts": [
                "the historic Charles Bridge in Prague, a medieval stone arch bridge adorned with 30 statues of saints, crossing the Vltava River",
                "view of Prague Castle and St. Vitus Cathedral from Charles Bridge, with artists, musicians, and vendors lining the bridge",
                "Gothic Old Town Bridge Tower and Lesser Town Bridge Towers guarding both ends of Charles Bridge",
                "Charles Bridge at dawn or dusk, with fewer crowds and atmospheric lighting, a symbol of Prague",
                "statues on Charles Bridge, such as the statue of St. John of Nepomuk, often touched for good luck"
            ]
        },
        "red_square_st_basils_cathedral": {
            "name": "Red Square & St. Basil's Cathedral",
            "aliases": ["Красная площадь", "Собор Василия Блаженного", "Moscow Kremlin"],
            "location": "Moscow, Russia",
            "prompts": [
                "the iconic, vibrantly colored onion domes of St. Basil's Cathedral (Cathedral of Vasily the Blessed) in Red Square, Moscow, a unique masterpiece of Russian architecture",
                "Red Square, the historic central square of Moscow, with Lenin's Mausoleum, the fortified walls of the Kremlin, the State Historical Museum, and the GUM department store",
                "historic and vast Red Square, a UNESCO World Heritage site, a focal point of Russian history and culture",
                "St. Basil's Cathedral illuminated at night, its swirling patterns and bright colors standing out against the dark sky",
                "the imposing Spasskaya Tower of the Moscow Kremlin overlooking Red Square"
            ]
        },
        "edinburgh_castle": {
            "name": "Edinburgh Castle",
            "aliases": ["Castle Rock Edinburgh"],
            "location": "Edinburgh, UK",
            "prompts": [
                "Edinburgh Castle perched dramatically atop Castle Rock, an extinct volcano, dominating the skyline of Edinburgh, Scotland",
                "historic Scottish fortress, Edinburgh Castle, with its ancient ramparts, Crown Jewels of Scotland (Honours of Scotland), and St. Margaret's Chapel",
                "view of the Royal Mile leading up to Edinburgh Castle, the historic heart of Edinburgh's Old Town",
                "the One O'Clock Gun firing from Edinburgh Castle, a daily tradition",
                "Edinburgh Castle illuminated at night, overlooking the city"
            ]
        },
        "matterhorn": {
            "name": "Matterhorn",
            "aliases": ["Monte Cervino", "The Horn"],
            "location": "Zermatt, Switzerland / Breuil-Cervinia, Italy",
            "prompts": [
                "the distinctive, sharply defined pyramidal peak of the Matterhorn mountain in the Pennine Alps on the border between Switzerland and Italy",
                "snow-covered Matterhorn against a clear blue sky, often reflected in a tranquil alpine lake like Riffelsee or Stellisee",
                "iconic alpine scenery surrounding the Matterhorn, a world-famous mountaineering challenge and symbol of the Alps",
                "the village of Zermatt, Switzerland, with traditional chalets and views of the Matterhorn",
                "Matterhorn at sunrise or sunset (alpenglow), when the peak is bathed in golden or reddish light"
            ]
        },
        "palace_of_versailles": {
            "name": "Palace of Versailles",
            "aliases": ["Château de Versailles", "Versailles Palace"],
            "location": "Versailles, France",
            "prompts": [
                "the opulent Palace of Versailles, former principal royal residence of France, with its grand Hall of Mirrors (Galerie des Glaces) and lavish state apartments",
                "expansive formal gardens of Versailles designed by André Le Nôtre, featuring geometric patterns, fountains (like the Latona Fountain), canals, and groves",
                "luxurious Baroque architecture and lavish interiors of the Palace of Versailles, a UNESCO World Heritage site symbolizing absolute monarchy",
                "the Grand Trianon and Petit Trianon, smaller palaces within the estate of Versailles, and Marie Antoinette's Hamlet",
                "the facade of the Palace of Versailles overlooking the Place d'Armes"
            ]
        }
    },

    # 北美地標
    "north_america": {
        "statue_of_liberty": {
            "name": "Statue of Liberty",
            "aliases": ["Liberty Enlightening the World", "Lady Liberty"],
            "location": "New York Harbor, New York, USA",
            "prompts": [
                "a photo of the Statue of Liberty in New York, the colossal neoclassical sculpture on Liberty Island, a symbol of freedom and democracy",
                "the iconic Statue of Liberty with her torch held high and tabula ansata (tablet), a gift from France to the USA",
                "Statue of Liberty on Liberty Island, with the Manhattan skyline or Ellis Island in the background",
                "close-up of Lady Liberty's crowned head or her copper-green patina",
                "ferry approaching the Statue of Liberty, offering panoramic views"
            ]
        },
        "golden_gate_bridge": {
            "name": "Golden Gate Bridge",
            "aliases": ["Golden Gate", "GGB"],
            "location": "San Francisco, California, USA",
            "prompts": [
                "a photo of the Golden Gate Bridge in San Francisco, its iconic Art Deco suspension bridge painted in 'International Orange'",
                "the vibrant red-orange Golden Gate Bridge spanning the Golden Gate strait, often partially shrouded in fog",
                "Golden Gate Bridge with its distinctive twin towers, soaring cables, and views of Alcatraz Island or the San Francisco skyline",
                "view of the Golden Gate Bridge from various vantage points like Battery Spencer, Vista Point, or Baker Beach",
                "cyclists or pedestrians crossing the Golden Gate Bridge"
            ]
        },
        "grand_canyon": {
            "name": "Grand Canyon",
            "aliases": ["Grand Canyon National Park", "The Canyon"],
            "location": "Arizona, USA",
            "prompts": [
                "a photo of the Grand Canyon in Arizona, a massive, steep-sided canyon carved by the Colorado River, showcasing layers of colorful rock",
                "vast, awe-inspiring landscape of the Grand Canyon, with its immense scale, depth, and intricate formations",
                "Colorado River flowing through the bottom of the Grand Canyon, visible from viewpoints like Mather Point or Yavapai Point on the South Rim",
                "sunset or sunrise over the Grand Canyon, painting the canyon walls in vibrant hues of red, orange, and purple",
                "hiking trails along the rim or into the Grand Canyon, such as Bright Angel Trail"
            ]
        },
        "hollywood_sign": {
            "name": "Hollywood Sign",
            "aliases": ["Hollywoodland Sign"],
            "location": "Mount Lee, Los Angeles, California, USA",
            "prompts": [
                "a photo of the Hollywood Sign in Los Angeles, the iconic white capital letters spelling out 'HOLLYWOOD' on the side of Mount Lee",
                "the iconic Hollywood Sign overlooking the sprawling cityscape of Hollywood and Los Angeles",
                "view of the Hollywood Sign from Griffith Observatory, Lake Hollywood Park, or a helicopter tour",
                "the large, distinctive letters of the Hollywood Sign, a symbol of the American film industry",
                "Hollywood Sign against a clear blue sky or at sunset with city lights below"
            ]
        },
        "white_house": {
            "name": "White House",
            "aliases": ["President's House", "Executive Mansion", "1600 Pennsylvania Avenue"],
            "location": "Washington D.C., USA",
            "prompts": [
                "a photo of the White House in Washington D.C., the official residence and workplace of the President of the United States",
                "the iconic neoclassical facade of the White House, with its white columns and porticoes (North Portico and South Portico)",
                "the North Portico of the White House facing Pennsylvania Avenue, or the South Lawn with the Oval Office view",
                "the White House surrounded by its meticulously manicured gardens and security fencing",
                "Marine One helicopter landing on the South Lawn of the White House"
            ]
        },
        "mount_rushmore": {
            "name": "Mount Rushmore",
            "aliases": ["Mount Rushmore National Memorial", "Presidents' Mountain"],
            "location": "Keystone, South Dakota, USA",
            "prompts": [
                "a photo of Mount Rushmore National Memorial, featuring the colossal carved faces of U.S. Presidents George Washington, Thomas Jefferson, Theodore Roosevelt, and Abraham Lincoln",
                "the sculpted faces of four presidents carved into the granite face of Mount Rushmore in the Black Hills of South Dakota",
                "Mount Rushmore with the Avenue of Flags leading to the Grand View Terrace",
                "the immense scale and detailed carving of the presidential heads on Mount Rushmore",
                "Mount Rushmore illuminated at night during the evening lighting ceremony"
            ]
        },
        "times_square": {
            "name": "Times Square",
            "aliases": ["The Crossroads of the World", "The Great White Way"],
            "location": "New York City, New York, USA",
            "prompts": [
                "a photo of Times Square in New York City, the bustling commercial intersection and entertainment hub, famous for its dazzling array of brightly lit billboards and advertisements",
                "bright, massive digital billboards and flashing neon lights of Times Square at night, creating a vibrant and energetic atmosphere",
                "bustling crowds of tourists and locals, yellow taxis, and costumed characters in Times Square",
                "the New Year's Eve ball drop ceremony in Times Square",
                "the TKTS booth with its red steps in Times Square, a popular spot for discounted Broadway tickets"
            ]
        },
        "cn_tower": {
            "name": "CN Tower",
            "aliases": ["Canadian National Tower", "Toronto Tower"],
            "location": "Toronto, Ontario, Canada",
            "prompts": [
                "a photo of the CN Tower in Toronto, the iconic slender communications and observation tower dominating the city's skyline",
                "the tall, freestanding CN Tower with its distinctive main pod housing observation decks, a revolving restaurant, and the EdgeWalk",
                "view from the top of the CN Tower, looking down through the glass floor or out at Lake Ontario and the Toronto Islands",
                "CN Tower illuminated at night with programmable LED lights, often changing colors for special occasions",
                "Toronto skyline featuring the CN Tower as its centerpiece"
            ]
        },
        "chichen_itza": {
            "name": "Chichen Itza",
            "aliases": ["El Castillo", "Pyramid of Kukulcan", "Chichén Itzá"],
            "location": "Yucatan Peninsula, Mexico",
            "prompts": [
                "a photo of Chichen Itza in Mexico, the ancient Mayan city and UNESCO World Heritage site, with its iconic El Castillo (Pyramid of Kukulcan)",
                "the massive step-pyramid El Castillo at Chichen Itza, famous for the serpent shadow effect during the equinoxes",
                "ancient ruins of Chichen Itza, including the Temple of Warriors, the Great Ball Court, and the Observatory (El Caracol)",
                "intricate stone carvings and Mayan hieroglyphs found on the structures of Chichen Itza",
                "the sacred cenote (sinkhole) at Chichen Itza, used for sacrifices"
            ]
        },
        "niagara_falls": {
            "name": "Niagara Falls",
            "aliases": ["Horseshoe Falls", "American Falls", "Bridal Veil Falls"],
            "location": "Ontario, Canada / New York, USA",
            "prompts": [
                "massive cascades of Niagara Falls, including the powerful Horseshoe Falls (Canadian Falls), the American Falls, and the smaller Bridal Veil Falls",
                "mist rising dramatically from the thundering Niagara Falls, with tour boats like the Maid of the Mist or Hornblower navigating the turbulent waters below",
                "Rainbow Bridge connecting Canada and the USA, with panoramic views of Niagara Falls",
                "Niagara Falls illuminated with colorful lights at night, or with fireworks displays",
                "Goat Island separating the American Falls and Horseshoe Falls, offering close-up views"
            ]
        },
        "central_park": {
            "name": "Central Park",
            "aliases": ["Manhattan Central Park"],
            "location": "New York City, New York, USA",
            "prompts": [
                "vast green expanse of Central Park in Manhattan, New York City, an urban oasis surrounded by the towering skyscrapers of the city skyline",
                "iconic locations within Central Park such as Bethesda Terrace and Fountain, Strawberry Fields (John Lennon memorial), Wollman Rink (ice skating), or the Central Park Carousel",
                "people enjoying recreational activities like picnicking, boating on The Lake, jogging, or horse-drawn carriage rides in Central Park",
                "lush lawns, wooded areas, walking paths, and picturesque bridges (like Bow Bridge or Gapstow Bridge) within Central Park",
                "aerial view of Central Park, highlighting its rectangular shape amidst the dense urban grid of Manhattan"
            ]
        },
        "las_vegas_strip": {
            "name": "Las Vegas Strip",
            "aliases": ["The Strip Las Vegas", "Las Vegas Boulevard South"],
            "location": "Las Vegas, Nevada, USA",
            "prompts": [
                "the dazzling Las Vegas Strip at night, a vibrant spectacle of illuminated mega-resorts, opulent casinos, and world-class entertainment venues",
                "iconic landmarks and themed hotels on the Las Vegas Strip such as the Bellagio fountains, the Eiffel Tower replica at Paris Las Vegas, the High Roller observation wheel, or the Venetian's canals",
                "bustling energy, flashing neon signs, and extravagant architecture characterizing the world-famous Las Vegas Strip",
                "pedestrians walking along the crowded sidewalks of the Las Vegas Strip, taking in the sights and sounds",
                "the Fountains of Bellagio water show on the Las Vegas Strip"
            ]
        },
        "yellowstone_national_park": {
            "name": "Yellowstone National Park",
            "aliases": ["Old Faithful Yellowstone", "Grand Prismatic Spring"],
            "location": "Wyoming, Montana, Idaho, USA",
            "prompts": [
                "geothermal features of Yellowstone National Park, including the iconic Old Faithful geyser erupting steam and hot water into the air",
                "the vibrant, rainbow-like colors of the Grand Prismatic Spring, the largest hot spring in the United States, in Yellowstone's Midway Geyser Basin",
                "wildlife such as bison herds, elk, bears, and wolves roaming freely in the diverse landscapes of Yellowstone National Park, including forests, meadows, and rivers",
                "the Grand Canyon of the Yellowstone, a dramatic canyon with impressive waterfalls like the Lower Falls",
                "Mammoth Hot Springs in Yellowstone, with its terraced travertine formations"
            ]
        },
        "banff_national_park_lake_louise": { # Specified Lake Louise
            "name": "Banff National Park (Lake Louise / Moraine Lake)",
            "aliases": ["Lake Louise Banff", "Moraine Lake Banff", "Canadian Rockies"],
            "location": "Alberta, Canada",
            "prompts": [
                "the stunning turquoise glacial waters of Lake Louise in Banff National Park, with the majestic Victoria Glacier and Fairmont Chateau Lake Louise in the background",
                "the equally breathtaking Moraine Lake in Banff National Park, with its vivid blue water backed by the rugged Valley of the Ten Peaks",
                "stunning Canadian Rockies mountain scenery in Banff National Park, a UNESCO World Heritage site, with snow-capped peaks, alpine meadows, and pristine forests",
                "canoeing or hiking around Lake Louise or Moraine Lake",
                "wildlife like elk or grizzly bears sometimes spotted in Banff National Park"
            ]
        },
        "space_needle_seattle": {
            "name": "Space Needle",
            "aliases": ["Seattle Space Needle"],
            "location": "Seattle, Washington, USA",
            "prompts": [
                "the futuristic Space Needle observation tower in Seattle, Washington, with its distinctive saucer-shaped top and slender hourglass silhouette",
                "panoramic view of Seattle's skyline, Puget Sound, Elliott Bay, and surrounding mountains like Mount Rainier from the Space Needle's observation deck or revolving restaurant",
                "Space Needle as an icon of the Pacific Northwest and a legacy of the 1962 World's Fair",
                "the Space Needle illuminated at night, a prominent feature of Seattle's cityscape",
                "Chihuly Garden and Glass exhibit located at the base of the Space Needle"
            ]
        }
    },

    # 南美地標
    "south_america": {
        "machu_picchu": {
            "name": "Machu Picchu",
            "aliases": ["Lost City of the Incas", "Machu Pikchu"],
            "location": "Cusco Region, Peru",
            "prompts": [
                "a photo of Machu Picchu in Peru, the breathtaking ancient Inca citadel set high in the Andes Mountains, often shrouded in mist",
                "the well-preserved ruins of the Inca city of Machu Picchu, with its intricate stone masonry, temples, plazas, and agricultural terraces",
                "panoramic view of Machu Picchu ruins with Huayna Picchu mountain rising prominently in the background",
                "llamas grazing among the ancient stone structures of Machu Picchu",
                "sunrise over Machu Picchu, revealing its mystical beauty and stunning mountain setting"
            ]
        },
        "christ_the_redeemer": {
            "name": "Christ the Redeemer",
            "aliases": ["Cristo Redentor", "Rio Jesus Statue", "Corcovado Statue"],
            "location": "Rio de Janeiro, Brazil",
            "prompts": [
                "a photo of Christ the Redeemer statue in Rio de Janeiro, the colossal Art Deco statue of Jesus Christ with outstretched arms, standing atop Corcovado Mountain",
                "the iconic soapstone statue of Jesus Christ on Corcovado Mountain, overlooking the city of Rio de Janeiro, Sugarloaf Mountain, and Guanabara Bay",
                "Christ the Redeemer as a symbol of Christianity and a global icon of Rio de Janeiro and Brazil",
                "view from the base of Christ the Redeemer statue, offering breathtaking panoramic vistas",
                "Christ the Redeemer statue illuminated at night or silhouetted against a vibrant sunset"
            ]
        },
        "iguazu_falls": {
            "name": "Iguazu Falls",
            "aliases": ["Iguaçu Falls", "Cataratas del Iguazú", "Cataratas do Iguaçu", "Devil's Throat Iguazu"],
            "location": "Misiones Province, Argentina / Paraná State, Brazil",
            "prompts": [
                "expansive network of hundreds of powerful waterfalls at Iguazu Falls, spanning the border of Argentina and Brazil, surrounded by lush subtropical rainforest",
                "the immense and thunderous Devil's Throat (Garganta del Diablo / Garganta do Diabo), the largest and most dramatic cataract of Iguazu Falls",
                "walkways and viewpoints offering close-up, immersive experiences of the mighty Iguazu Falls, often with rainbows forming in the mist",
                "boat tours venturing near the base of the waterfalls at Iguazu Falls",
                "diverse wildlife like coatis and colorful birds in the Iguazu National Park surrounding the falls"
            ]
        },
        "galapagos_islands": {
            "name": "Galapagos Islands",
            "aliases": ["Archipiélago de Colón", "Darwin's Islands"],
            "location": "Ecuador",
            "prompts": [
                "unique and fearless wildlife of the Galapagos Islands, such as giant tortoises roaming freely, marine iguanas basking on volcanic rocks, and blue-footed boobies performing their mating dance",
                "pristine volcanic landscapes, lava fields, and beautiful beaches (like Tortuga Bay) of the Galapagos Islands, a UNESCO World Heritage site that inspired Charles Darwin's theory of evolution",
                "snorkeling or diving with sea lions, penguins, and diverse marine life in the clear waters surrounding the Galapagos Islands",
                "various endemic species found only in the Galapagos, like Darwin's finches or flightless cormorants",
                "cruise ships or small yachts exploring the different islands of the Galapagos archipelago"
            ]
        },
        "torres_del_paine_national_park": {
            "name": "Torres del Paine National Park",
            "aliases": ["Parque Nacional Torres del Paine", "Paine Towers"],
            "location": "Patagonia, Chile",
            "prompts": [
                "the iconic granite peaks (Horns or Towers of Paine) of the Torres del Paine massif in Chilean Patagonia, often reflecting in turquoise glacial lakes like Pehoé or Nordenskjöld",
                "stunning and wild landscapes of glaciers (like Grey Glacier), vibrant blue lakes, rivers, and mountains in Torres del Paine National Park, a UNESCO Biosphere Reserve",
                "hiking trails like the 'W' trek or 'O' circuit offering breathtaking views of the dramatic Patagonian scenery in Torres del Paine",
                "guanacos, condors, and other Patagonian wildlife in their natural habitat within Torres del Paine",
                "the dramatic, windswept environment of Torres del Paine, known for its unpredictable weather"
            ]
        },
        "angel_falls": {
            "name": "Angel Falls",
            "aliases": ["Salto Ángel", "Kerepakupai Merú"],
            "location": "Canaima National Park, Venezuela",
            "prompts": [
                "Angel Falls, the world's tallest uninterrupted waterfall, cascading spectacularly from the sheer cliff face of Auyán-Tepui, a massive table-top mountain (tepui) in Venezuela's Canaima National Park",
                "remote and dramatic jungle landscape surrounding Angel Falls, with tepuis rising from the savanna and rainforest",
                "aerial view of Angel Falls plunging thousands of feet down the Auyán-Tepui, often shrouded in mist",
                "expeditions by boat and foot through the remote wilderness to reach the base of Angel Falls",
                "the sheer scale and pristine, untouched beauty of Angel Falls, a natural wonder"
            ]
        },
        "salar_de_uyuni": {
            "name": "Salar de Uyuni",
            "aliases": ["Uyuni Salt Flat"],
            "location": "Potosí, Bolivia",
            "prompts": [
                "vast, seemingly endless white expanse of the Salar de Uyuni salt flat in Bolivia, the world's largest salt desert, creating a surreal and minimalist landscape",
                "mirror-like reflections on Salar de Uyuni during the rainy season (December-April), transforming the salt flat into the world's largest natural mirror, blurring the horizon between sky and ground",
                "Isla Incahuasi (Fish Island) with its giant ancient cacti standing starkly against the white salt crust of Salar de Uyuni",
                "geometric patterns of salt polygons on the dry Salar de Uyuni",
                "creative forced perspective photographs taken by tourists on the Salar de Uyuni"
            ]
        }
    },

    # 中東/非洲地標
    "middle_east_africa": {
        "pyramids_of_giza": {
            "name": "Pyramids of Giza",
            "aliases": ["Great Pyramids", "Egyptian Pyramids", "Giza Necropolis"],
            "location": "Giza, Egypt (near Cairo)",
            "prompts": [
                "a photo of the Pyramids of Giza in Egypt, the ancient wonder of the world, with the Great Pyramid, Pyramid of Khafre, and Pyramid of Menkaure",
                "the ancient Egyptian pyramids on the Giza plateau, with the enigmatic Great Sphinx guarding them, against a desert backdrop or the Cairo skyline",
                "Great Pyramid of Giza, the largest of the three, with the smaller Queen's Pyramids nearby",
                "camels and horses carrying tourists around the Giza pyramid complex",
                "sunset or sunrise over the Pyramids of Giza, casting long shadows"
            ]
        },
        "burj_khalifa": {
            "name": "Burj Khalifa",
            "aliases": ["Khalifa Tower", "Dubai Tower", "World's Tallest Building"],
            "location": "Dubai, UAE",
            "prompts": [
                "a photo of Burj Khalifa in Dubai, the world's tallest building, a sleek, tapering skyscraper piercing the sky",
                "the ultra-modern skyscraper Burj Khalifa dominating the Dubai skyline with its impressive height and futuristic design",
                "Burj Khalifa skyscraper rising above the Dubai Fountain and surrounding modern architecture",
                "view from the observation deck ('At the Top') of Burj Khalifa, offering panoramic views of Dubai and the desert",
                "Burj Khalifa illuminated with spectacular light shows at night"
            ]
        },
        "petra_jordan": { # Added Jordan to differentiate from any other Petra
            "name": "Petra",
            "aliases": ["Rose City", "Lost City of Petra", "Al-Khazneh Petra"],
            "location": "Ma'an Governorate, Jordan",
            "prompts": [
                "a photo of Petra in Jordan, the ancient Nabataean city carved into rose-red sandstone cliffs, a UNESCO World Heritage site",
                "the iconic Treasury (Al-Khazneh) in Petra, with its ornate facade intricately carved into a sandstone rock face, revealed at the end of the Siq (narrow gorge)",
                "ancient city of Petra with its rock-cut architecture, including tombs, temples (like the Monastery, Ad Deir), and colonnaded streets",
                "the Siq, a narrow, winding gorge that serves as the dramatic main entrance to the city of Petra",
                "Bedouins with camels or donkeys in the ancient city of Petra"
            ]
        },
        "table_mountain": {
            "name": "Table Mountain",
            "aliases": ["Tafelberg", "Cape Town Table Mountain"],
            "location": "Cape Town, South Africa",
            "prompts": [
                "a photo of Table Mountain in Cape Town, the iconic flat-topped mountain majestically overlooking the city, Table Bay, and the Atlantic Ocean",
                "the flat-topped Table Mountain, often covered by a 'tablecloth' of clouds, with Devil's Peak and Lion's Head adjacent to it",
                "view from the rotating cable car ascending Table Mountain, or panoramic views from its summit",
                "Cape Town city nestled at the foot of Table Mountain, with the V&A Waterfront visible",
                "unique fynbos vegetation found on Table Mountain, part of the Cape Floral Kingdom"
            ]
        },
        "sheikh_zayed_grand_mosque": {
            "name": "Sheikh Zayed Grand Mosque",
            "aliases": ["Grand Mosque Abu Dhabi"],
            "location": "Abu Dhabi, UAE",
            "prompts": [
                "the stunning, pristine white marble Sheikh Zayed Grand Mosque in Abu Dhabi, with its numerous domes (82 of them) and four towering minarets",
                "intricate floral designs inlaid with semi-precious stones, gold accents, and massive reflective pools surrounding the Sheikh Zayed Grand Mosque",
                "the vast main prayer hall of Sheikh Zayed Grand Mosque, featuring the world's largest hand-knotted carpet and one of the world's largest Swarovski crystal chandeliers",
                "the gleaming white exterior and symmetrical courtyards of Sheikh Zayed Grand Mosque, a masterpiece of modern Islamic architecture",
                "Sheikh Zayed Grand Mosque illuminated at night with a unique lunar lighting system that changes with the phases of the moon"
            ]
        },
        "masai_mara_national_reserve": {
            "name": "Masai Mara National Reserve",
            "aliases": ["Maasai Mara", "The Mara"],
            "location": "Kenya",
            "prompts": [
                "vast, open savannas of the Masai Mara National Reserve in Kenya, teeming with iconic African wildlife like lions, elephants, giraffes, zebras, and wildebeest",
                "the Great Migration of millions of wildebeest and zebras crossing the Mara River, often facing crocodiles, during their annual journey (July-October)",
                "Masai people in their traditional vibrant red shuka robes, often seen near their villages or as safari guides in the Masai Mara",
                "hot air balloons drifting over the plains of the Masai Mara at sunrise, offering a unique perspective on the landscape and wildlife",
                "acacia trees silhouetted against a dramatic African sunset in the Masai Mara"
            ]
        },
        "victoria_falls": {
            "name": "Victoria Falls",
            "aliases": ["Mosi-oa-Tunya", "The Smoke that Thunders"],
            "location": "Livingstone, Zambia / Victoria Falls, Zimbabwe",
            "prompts": [
                "the spectacular, vast curtain of falling water at Victoria Falls, one of the largest waterfalls in the world by combined width and height, on the Zambezi River",
                "a plume of mist (the 'smoke') rising high above Victoria Falls, visible for miles, and frequent rainbows forming in the spray",
                "views of Victoria Falls from various viewpoints like the Knife-Edge Bridge, Danger Point, or from rainforest trails along the gorge",
                "adventure activities at Victoria Falls, such as bungee jumping from the Victoria Falls Bridge or white-water rafting on the Zambezi",
                "the Zambezi River plunging into the deep Batoka Gorge at Victoria Falls"
            ]
        },
        "kilimanjaro": {
            "name": "Mount Kilimanjaro",
            "aliases": ["Uhuru Peak", "Kibo Kilimanjaro", "Africa's Highest Mountain"],
            "location": "Tanzania",
            "prompts": [
                "the snow-capped, iconic peak of Mount Kilimanjaro, Africa's highest mountain and the world's tallest freestanding mountain, rising majestically from the plains of Tanzania",
                "the distinctive flat-topped dormant volcano, Kilimanjaro, with its three volcanic cones (Kibo, Mawenzi, and Shira), a popular and challenging climbing destination",
                "diverse ecosystems on the slopes of Kilimanjaro, transitioning from rainforest and moorland to alpine desert and arctic summit zones",
                "porters and climbers on one of the routes (e.g., Machame, Lemosho) ascending Mount Kilimanjaro",
                "Mount Kilimanjaro at sunrise or sunset, with its glaciers and snowfields gleaming"
            ]
        },
        "dead_sea": {
            "name": "Dead Sea",
            "aliases": ["Salt Sea"],
            "location": "Jordan / Israel / Palestine",
            "prompts": [
                "people floating effortlessly and buoyantly in the hyper-saline, turquoise waters of the Dead Sea, the lowest point on Earth's land surface",
                "mineral-rich black mud from the Dead Sea being applied to the skin for therapeutic benefits, with salt formations along the shores",
                "unique landscape of the Dead Sea, with its calm, dense waters reflecting the arid surrounding mountains and desert",
                "evaporation ponds for mineral extraction at the southern end of the Dead Sea",
                "salt crystals encrusting rocks and branches along the coastline of the Dead Sea"
            ]
        },
        "dome_of_the_rock": {
            "name": "Dome of the Rock",
            "aliases": ["Qubbat as-Sakhrah", "Temple Mount Jerusalem"],
            "location": "Old City of Jerusalem",
            "prompts": [
                "the iconic, gleaming golden dome of the Dome of the Rock, an Islamic shrine located on the Temple Mount (Haram al-Sharif) in the Old City of Jerusalem",
                "the octagonal structure of the Dome of the Rock, adorned with intricate blue and turquoise ceramic tilework, calligraphy, and mosaics",
                "iconic religious landmark, the Dome of the Rock, a site of great significance in Islam, Judaism, and Christianity, within the historic walls of Jerusalem",
                "view of the Dome of the Rock from the Mount of Olives, with the Old City in the background",
                "the interior of the Dome of the Rock (if permissible to depict), showing the Foundation Stone"
            ]
        }
    },

    # 大洋洲地標
    "oceania": {
        "sydney_opera_house": {
            "name": "Sydney Opera House",
            "aliases": ["Opera House Sydney", "Sydney Landmark"],
            "location": "Sydney, New South Wales, Australia",
            "prompts": [
                "a photo of the Sydney Opera House in Australia, its iconic white sail-shaped shells creating a distinctive silhouette on Sydney Harbour",
                "the multi-venue performing arts centre, Sydney Opera House, with its unique shell-like roof structures, a masterpiece of modern architecture",
                "Sydney Opera House with the Sydney Harbour Bridge in the background, a classic view of Sydney's landmarks",
                "the Sydney Opera House illuminated at night, often with colorful projections for events like Vivid Sydney",
                "close-up of the textured, chevron-patterned tiles covering the shells of the Sydney Opera House"
            ]
        },
        "uluru": {
            "name": "Uluru",
            "aliases": ["Ayers Rock", "The Rock", "Uluru-Kata Tjuta National Park"],
            "location": "Northern Territory, Australia",
            "prompts": [
                "a photo of Uluru (Ayers Rock) in Australia, the massive, sacred sandstone monolith rising from the flat desert landscape of the Red Centre",
                "the immense sandstone monolith of Uluru changing colors dramatically at sunrise or sunset, glowing in shades of red, orange, and purple",
                "Uluru in the Red Centre of Australia, a significant spiritual site for Indigenous Anangu people, with visible rock caves and ancient rock art",
                "the distinct shape and texture of Uluru, showing its weathered surface and gullies",
                "Kata Tjuta (The Olgas) formations visible in the distance from Uluru"
            ]
        },
        "great_barrier_reef": {
            "name": "Great Barrier Reef",
            "aliases": ["GBR", "World's Largest Coral Reef System"],
            "location": "Queensland, Australia",
            "prompts": [
                "an underwater photo of the Great Barrier Reef, showcasing its vibrant and diverse coral formations, colorful fish, and other marine life like sea turtles or manta rays",
                "colorful hard and soft coral gardens thriving in the clear turquoise waters of the Great Barrier Reef, the world's largest coral reef system",
                "aerial view of the Great Barrier Reef, revealing the intricate patterns of reefs, islands, and cays stretching along the Queensland coast",
                "scuba divers or snorkelers exploring the rich biodiversity of the Great Barrier Reef",
                "Heart Reef, a naturally formed heart-shaped coral formation in the Great Barrier Reef (often seen from the air)"
            ]
        },
        "hobbiton_movie_set": {
            "name": "Hobbiton Movie Set",
            "aliases": ["The Shire Tour", "Lord of the Rings Set NZ", "Hobbiton New Zealand"],
            "location": "Matamata, Waikato, New Zealand",
            "prompts": [
                "a photo of the Hobbiton Movie Set in New Zealand, the picturesque movie set for The Shire from 'The Lord of the Rings' and 'The Hobbit' trilogies",
                "charming, brightly colored circular Hobbit hole doors built into rolling green hills at the Hobbiton Movie Set, with meticulously tended gardens",
                "the Green Dragon Inn at Hobbiton, a faithfully reconstructed pub where visitors can enjoy a drink",
                "the Party Tree and Mill at Hobbiton, iconic locations from the movies, set within a lush pastoral landscape",
                "guided tour groups exploring the whimsical and detailed Hobbiton Movie Set"
            ]
        },
        "sydney_harbour_bridge": {
            "name": "Sydney Harbour Bridge",
            "aliases": ["The Coathanger Sydney"],
            "location": "Sydney, New South Wales, Australia",
            "prompts": [
                "the iconic steel through arch of the Sydney Harbour Bridge, affectionately known as 'The Coathanger', spanning Sydney Harbour",
                "people participating in the Sydney Harbour BridgeClimb, ascending the arch for panoramic views of the city and harbour",
                "view of Sydney Harbour featuring both the Opera House and the Harbour Bridge together, a quintessential Sydney scene",
                "ferries and sailboats passing under the massive Sydney Harbour Bridge",
                "fireworks display over the Sydney Harbour Bridge during New Year's Eve celebrations"
            ]
        },
        "fiordland_national_park_milford_sound": { # Specified Milford Sound
            "name": "Fiordland National Park (Milford Sound / Doubtful Sound)",
            "aliases": ["Milford Sound New Zealand", "Doubtful Sound Fiordland"],
            "location": "South Island, New Zealand",
            "prompts": [
                "dramatic fiords with sheer, towering cliffs and cascading waterfalls (like Stirling Falls or Bowen Falls) in Fiordland National Park, New Zealand, particularly Milford Sound",
                "the iconic Mitre Peak rising sharply from the dark, reflective waters of Milford Sound in Fiordland, often shrouded in mist",
                "boat cruises navigating through the stunning natural scenery of Milford Sound or the more remote Doubtful Sound, with seals, dolphins, or penguins sometimes spotted",
                "lush rainforest clinging to the steep mountain sides of Fiordland National Park",
                "the dramatic, moody atmosphere of Fiordland, known for its high rainfall and untouched wilderness"
            ]
        },
        "bondi_beach": {
            "name": "Bondi Beach",
            "aliases": ["Bondi Sydney"],
            "location": "Sydney, New South Wales, Australia",
            "prompts": [
                "the famous crescent-shaped Bondi Beach in Sydney, a popular destination with golden sand, turquoise waves, surfers, and sunbathers",
                "Bondi Icebergs Club swimming pool, an ocean pool with waves crashing into it, located at the southern end of Bondi Beach",
                "vibrant beach culture, surf lifesavers in their distinctive red and yellow uniforms, and bustling cafes along the promenade at Bondi Beach",
                "the Bondi to Coogee coastal walk, offering stunning ocean views starting from Bondi Beach",
                "aerial view of Bondi Beach, showcasing its iconic shape and lively atmosphere"
            ]
        },
        "aoraki_mount_cook_national_park": {
            "name": "Aoraki / Mount Cook National Park",
            "aliases": ["Mount Cook New Zealand", "Lake Pukaki Mount Cook", "Hooker Valley Track"],
            "location": "South Island, New Zealand",
            "prompts": [
                "the majestic, snow-capped pyramid peak of Aoraki / Mount Cook, New Zealand's highest mountain, dominating the Southern Alps",
                "glaciers like Tasman Glacier, alpine lakes with milky turquoise water (such as Lake Pukaki or Lake Tekapo often framed by colorful lupins in summer), and rugged mountain scenery in Aoraki / Mount Cook National Park",
                "stargazing in the Aoraki Mackenzie International Dark Sky Reserve, with the silhouette of Aoraki / Mount Cook against a starry night sky",
                "hiking trails like the Hooker Valley Track, offering spectacular views of Aoraki / Mount Cook, glaciers, and icebergs in Hooker Lake",
                "the Hermitage Hotel or other alpine lodges with views of Aoraki / Mount Cook"
            ]
        },
        "twelve_apostles_great_ocean_road": { # Specified Great Ocean Road
            "name": "The Twelve Apostles, Great Ocean Road",
            "aliases": ["Twelve Apostles Victoria", "Great Ocean Road Australia"],
            "location": "Victoria, Australia",
            "prompts": [
                "limestone stacks known as The Twelve Apostles (though fewer than twelve remain) rising dramatically from the Southern Ocean along the scenic Great Ocean Road in Victoria, Australia",
                "coastal scenery with rugged cliffs, powerful wave erosion, and the iconic sea stacks of The Twelve Apostles site, especially at sunrise or sunset",
                "sunset or sunrise over The Twelve Apostles rock formations, casting golden light and long shadows",
                "viewing platforms offering panoramic vistas of The Twelve Apostles and the surrounding coastline",
                "other nearby rock formations along the Great Ocean Road, like Loch Ard Gorge or London Arch (formerly London Bridge)"
            ]
        },
         "easter_island_moai": {
            "name": "Moai Statues, Easter Island",
            "aliases": [
                "Easter Island Heads",
                "Rapa Nui Moai",
                "復活節島摩艾石像",
                "拉帕努伊島石像",
                "摩艾"
            ],
            "location": "Easter Island (Rapa Nui), Chile",
            "prompts": [
                "colossal monolithic human figures known as moai, carved from volcanic rock, standing on Easter Island (Rapa Nui)",
                "iconic giant stone heads and torsos of the moai statues with long ears and stoic expressions, dotting the coastal and inland landscapes of Easter Island",
                "the mysterious moai statues of Rapa Nui, some with red scoria topknots (pukao), on ceremonial platforms called ahu",
                "Ahu Tongariki featuring a line-up of fifteen imposing moai statues against the Pacific Ocean, Easter Island",
                "moai statues in the Rano Raraku quarry, where they were carved, some partially buried or appearing to emerge from the hillside",
                "the unique archaeological site of Easter Island, showcasing hundreds of ancient monolithic moai, a testament to Rapa Nui culture",
                "weathered stone giants of Easter Island, their distinct profiles silhouetted against the sky or ocean, conveying a sense of ancient mystery",
                "close-up of a moai's characteristic features: heavy brow, elongated nose, thin lips, and often deep eye sockets, Easter Island",
                "a line of massive moai statues on an ahu platform overlooking the sea, representing deified ancestors of Rapa Nui",
                "the remote and windswept landscape of Easter Island, punctuated by the enigmatic presence of its ancient moai statues"
            ]
        }
    }
}

# 方便直接查詢所有地標
ALL_LANDMARKS = {}
for region, landmarks in LANDMARK_DATA.items():
    for landmark_id, landmark_info in landmarks.items():
        ALL_LANDMARKS[landmark_id] = landmark_info

# 獲取所有地標提示列表（用於CLIP分析）
def get_all_landmark_prompts():
    """
    返回所有地標的提示列表，用於CLIP分析

    Returns:
        list: 提示列表
    """
    prompts = []
    for landmark_id, landmark_info in ALL_LANDMARKS.items():
        # 使用第一個提示作為主要提示
        prompts.append(landmark_info["prompts"][0])
    return prompts

# 獲取地標名稱到ID的映射
def get_landmark_name_to_id_map():
    """
    返回地標名稱到ID的映射

    Returns:
        dict: {地標名稱: 地標ID}
    """
    name_to_id = {}
    for landmark_id, landmark_info in ALL_LANDMARKS.items():
        name_to_id[landmark_info["name"]] = landmark_id
        # 也添加所有別名
        for alias in landmark_info["aliases"]:
            name_to_id[alias] = landmark_id
    return name_to_id

# 獲取某個區域的所有地標ID
def get_landmarks_by_region(region):
    """
    返回指定區域的所有地標ID

    Args:
        region (str): 區域名稱

    Returns:
        list: 地標ID列表
    """
    if region not in LANDMARK_DATA:
        return []
    return list(LANDMARK_DATA[region].keys())

# 獲取每個地標的代表性提示
def get_landmark_prompt(landmark_id):
    """
    返回指定地標的代表性提示

    Args:
        landmark_id (str): 地標ID

    Returns:
        str: 提示文本
    """
    if landmark_id not in ALL_LANDMARKS:
        return None
    return ALL_LANDMARKS[landmark_id]["prompts"][0]

Writing landmark_data.py


In [ ]:
# %%writefile landmark_activities.py

"""
Activity suggestions for specific landmarks.
This module provides custom activity recommendations for recognized landmarks.
"""

LANDMARK_ACTIVITIES = {
    # 亞洲地標 (Asia)
    "taipei_101": [
        "Visiting the observation deck (89F, 91F, 101F) for panoramic city and mountain views",
        "Shopping at the luxury mall at the base (Taipei 101 Mall)",
        "Photographing the cityscape, especially at sunset or during the New Year's Eve fireworks display",
        "Dining at high-altitude restaurants within the tower (e.g., Din Tai Fung, Starbucks on 35F)",
        "Learning about the engineering marvels, including the tuned mass damper, through exhibits",
        "Admiring the public art installations around the building and mall"
    ],
    "taroko_gorge": [
        "Hiking scenic trails like Shakadang Trail (Mysterious Valley Trail), Baiyang Waterfall Trail (Water Curtain Cave), or Lushui Trail",
        "Photographing the sheer marble cliffs, tunnels, and the turquoise Liwu River",
        "Visiting the Eternal Spring Shrine (Changchun Shrine) and Bell Tower",
        "Exploring Swallow Grotto (Yanzikou) and the Tunnel of Nine Turns (Jiuqudong) for close-up gorge views",
        "Learning about the geology, ecology, and indigenous Truku culture at the visitor center",
        "Cycling along parts of the gorge road (with extreme caution due to traffic and tunnels)"
    ],
    "sun_moon_lake": [
        "Taking a boat tour across the lake, visiting Lalu Island, Xuanguang Temple, and Ita Thao Pier",
        "Cycling or walking on the dedicated bike paths encircling the lake (e.g., Xiangshan to Shuishe section)",
        "Visiting Wenwu Temple and Ci'en Pagoda for stunning lake views and cultural insights",
        "Riding the Sun Moon Lake Ropeway (cable car) for aerial views and access to the Formosan Aboriginal Culture Village",
        "Enjoying local Thao aboriginal cuisine and street food at Ita Thao village",
        "Photographing the serene lake landscapes, especially at sunrise, sunset, or when mist-covered"
    ],
    "jiufen_old_street": [
        "Wandering through the narrow, lantern-lined alleyways of Jiufen Old Street, especially Shuqi Road",
        "Sampling local Taiwanese snacks like taro balls, fish balls, and peanut ice cream rolls",
        "Visiting traditional teahouses (e.g., A-Mei Tea House) for tea and panoramic coastal views",
        "Photographing the nostalgic atmosphere, red lanterns, and views of Keelung Mountain and the coastline",
        "Shopping for local crafts, souvenirs, and ocarinas",
        "Learning about Jiufen's gold mining history at the Gold Museum in nearby Jinguashi (optional day trip extension)"
    ],
    "kenting_national_park": [
        "Relaxing, swimming, or surfing at popular beaches like Nanwan (South Bay), Baishawan (White Sand Bay), or Little Bay (Xiaowan)",
        "Visiting Eluanbi Park to see the iconic Eluanbi Lighthouse, Taiwan's southernmost point",
        "Exploring unique geological formations like Sail Rock (Chuanfanshi), Longpan Park's limestone cliffs, and Maobitou's coastal terrain",
        "Snorkeling or scuba diving in the coral-rich waters (e.g., Houbihu Marine Protected Area)",
        "Hiking trails in Sheding Nature Park or Kenting Forest Recreation Area",
        "Enjoying the vibrant atmosphere and street food at Kenting Night Market on Kenting Street"
    ],
    "national_palace_museum_tw": [
        "Viewing renowned masterpieces like the Jadeite Cabbage, Meat-shaped Stone, and Mao Gong Ding",
        "Exploring extensive collections of Chinese imperial artifacts, calligraphy, paintings, ceramics, and bronzes",
        "Taking a guided tour or using an audio guide to understand the historical and cultural context of the exhibits",
        "Admiring the classical Chinese palace-style architecture of the museum building",
        "Visiting the serene Zhishan Garden, a traditional Chinese garden adjacent to the museum",
        "Attending special exhibitions and cultural events hosted by the museum"
    ],
    "alishan_national_scenic_area": [
        "Watching the famous sunrise over a sea of clouds from Chushan or Ogasawara Mountain viewing platforms",
        "Riding the Alishan Forest Railway on one of its scenic mountain routes (e.g., to Shenmu 'Sacred Tree' Station)",
        "Hiking through misty forests of ancient giant trees (Taiwan red cypress and cedar) along trails like the Giant Tree Plank Trail",
        "Visiting the Sister Ponds (Jiemei Tan) and Shouzhen Temple",
        "Learning about Alishan's tea culture and sampling high-mountain oolong tea at local plantations",
        "Photographing the cherry blossoms in spring or fireflies in summer"
    ],
    "shilin_night_market": [
        "Sampling a wide variety of iconic Taiwanese street foods like oyster omelets, stinky tofu, giant fried chicken cutlets, and bubble tea",
        "Exploring the bustling maze of food stalls, clothing shops, and souvenir vendors in the outdoor and underground sections",
        "Playing traditional night market games like claw machines, balloon darts, and shrimp fishing",
        "Shopping for trendy apparel, accessories, and electronics at affordable prices",
        "Experiencing the vibrant and energetic atmosphere of one of Taipei's largest and most famous night markets",
        "Trying unique snacks like flame-torched beef cubes or cheese-filled potatoes"
    ],
    "tokyo_tower": [
        "Ascending to the main (150m) and top (250m) observation decks for panoramic views of Tokyo, including Mount Fuji on clear days",
        "Photographing the tower, especially when illuminated with its iconic orange lights or special seasonal displays",
        "Visiting the official 'FootTown' at the base, featuring souvenir shops, cafes, an aquarium, and an e-sports park",
        "Enjoying refreshments or a meal at the tower's cafes or restaurants with city views",
        "Learning about the tower's history, construction, and its role as a broadcasting tower",
        "Visiting nearby Zojoji Temple for a cultural contrast and photo opportunities with the tower in the background"
    ],
    "mount_fuji": [
        "Climbing to the summit during the official climbing season (typically early July to early September) via one of the four main trails",
        "Photographing the iconic snow-capped conical peak from various viewpoints like the Fuji Five Lakes (especially Lake Kawaguchiko or Lake Yamanakako), Chureito Pagoda, or Hakone",
        "Visiting the Fuji Five Lakes region for activities like boating, fishing, hot springs (onsen), and museums (e.g., Itchiku Kubota Art Museum)",
        "Hiking or nature walking around the lower slopes, Aokigahara forest (Jukai), or Oshino Hakkai (traditional village with eight spring ponds)",
        "Visiting Fuji-Q Highland amusement park for thrill rides with views of Mount Fuji",
        "Learning about its volcanic geology, cultural significance (UNESCO World Heritage site), and spiritual importance in Shintoism and Buddhism"
    ],
    "kinkaku_ji": [
        "Admiring and photographing the stunning Golden Pavilion (Shariden) meticulously covered in gold leaf, reflected in the Mirror Pond (Kyōko-chi)",
        "Strolling through the meticulously maintained Japanese Muromachi period stroll garden (kaiyū-shiki teien)",
        "Learning about the Zen Buddhist history of the temple, originally a retirement villa for Shogun Ashikaga Yoshimitsu",
        "Observing the distinct architectural styles of each of the three floors of the pavilion",
        "Making a wish by tossing coins at designated spots for good luck",
        "Enjoying matcha tea and traditional sweets at the Sekkatei Teahouse or nearby tea houses (check availability)"
    ],
    "fushimi_inari_shrine": [
        "Walking through the thousands of vibrant vermilion (shu-iro) torii gates that form tunnels along the mountain trails",
        "Hiking the entire 4km (2-3 hour) trail up the sacred Mount Inari, exploring the network of paths and smaller shrines",
        "Photographing the iconic, seemingly endless tunnels of torii gates, a symbol of prosperity and good fortune",
        "Visiting the main shrine buildings (Go-Honden) at the base and offering prayers to Inari, the Shinto god of rice, sake, and business",
        "Exploring smaller sub-shrines (otsuka) and atmospheric graveyards along the mountain trails",
        "Looking for numerous fox statues (kitsune), considered messengers of Inari, often holding keys or jewels in their mouths"
    ],
    "shibuya_crossing": [
        "Experiencing the 'scramble' by walking across the multi-directional intersection with hundreds of other pedestrians",
        "Photographing or video recording the iconic crossing from a high vantage point, such as the Starbucks in the Tsutaya building or Mag's Park rooftop at Magnet by Shibuya109",
        "People-watching and soaking in the vibrant, energetic atmosphere of modern Tokyo",
        "Visiting the Hachiko statue, a famous meeting spot dedicated to the loyal Akita dog, located outside Shibuya Station",
        "Shopping at the numerous department stores (e.g., Shibuya 109, Shibuya Sky) and trendy boutiques in the surrounding area",
        "Exploring nearby entertainment venues, restaurants, and nightlife options"
    ],
    "tokyo_skytree": [
        "Ascending to the Tembo Deck (350m) and Tembo Galleria (450m) for breathtaking panoramic views of Tokyo and beyond, including Mount Fuji on clear days",
        "Photographing the modern architectural marvel, especially when illuminated with its signature 'Iki' (sky blue) or 'Miyabi' (purple) lights",
        "Walking on the glass floor section of the Tembo Deck for a thrilling view downwards",
        "Shopping and dining at Tokyo Solamachi, the large entertainment complex at the base of the Skytree, which includes an aquarium and planetarium",
        "Learning about the tower's earthquake-resistant design and construction",
        "Visiting nearby Sumida River for a different perspective of the tower and leisurely walks"
    ],
    "senso_ji_temple": [
        "Entering through the Kaminarimon (Thunder Gate) with its giant red lantern and statues of Raijin and Fujin",
        "Walking along Nakamise-dori, the bustling market street leading to the temple, lined with stalls selling traditional snacks, crafts, and souvenirs",
        "Visiting the main hall (Hondo) dedicated to Kannon Bosatsu (Bodhisattva of Compassion) and the five-story pagoda",
        "Wafting incense smoke from the large cauldron (jokoro) over oneself for good health and luck",
        "Photographing the vibrant temple complex, its traditional architecture, and the lively atmosphere",
        "Exploring the quieter Asakusa Shrine (a Shinto shrine) located next to Senso-ji"
    ],
    "osaka_castle": [
        "Exploring the interior of the reconstructed castle keep (tenshukaku), which now serves as a museum detailing the castle's history and Toyotomi Hideyoshi",
        "Ascending to the observation deck on the top floor of the castle keep for panoramic views of Osaka city and Osaka Castle Park",
        "Strolling through Osaka Castle Park, enjoying the vast green spaces, moats, stone walls, and seasonal blooms (especially cherry blossoms in spring and plum blossoms in late winter)",
        "Photographing the majestic castle, its golden embellishments, and its reflection in the moat",
        "Visiting Nishinomaru Garden within the park for beautiful views of the castle (especially during cherry blossom season, requires separate admission)",
        "Learning about the historical sieges and significance of Osaka Castle in Japanese history"
    ],
    "dotonbori": [
        "Taking iconic photos with the Glico Running Man billboard and other extravagant 3D signs (e.g., Kani Doraku crab, Zubora-ya pufferfish)",
        "Strolling along the Dotonbori canal and its vibrant pedestrianized streets, especially at night when the neon lights are dazzling",
        "Sampling famous Osakan street food (kuidaore) like takoyaki, okonomiyaki, kushikatsu, and ramen from numerous stalls and restaurants",
        "Taking a Tonbori River Cruise for a different perspective of the district from the water",
        "Shopping for souvenirs, fashion, and unique Japanese goods in the area",
        "Experiencing the lively entertainment, including arcades, theaters (like Shochikuza for kabuki), and karaoke bars"
    ],
    "arashiyama_bamboo_grove": [
        "Walking or cycling along the enchanting pathway through the towering stalks of the Sagano Bamboo Forest",
        "Listening to the rustling sound of the bamboo leaves in the wind, a designated 'Soundscape of Japan'",
        "Photographing the magical light filtering through the dense green bamboo canopy",
        "Visiting nearby Tenryu-ji Temple (a UNESCO World Heritage site) with its beautiful garden that backs onto the bamboo grove",
        "Crossing the Togetsukyo Bridge ('Moon Crossing Bridge') over the Hozugawa River for scenic views of Arashiyama",
        "Renting a rowboat on the Hozugawa River or taking the Sagano Romantic Train for more scenic views of the area"
    ],
    "itsukushima_shrine": [
        "Photographing the iconic 'floating' vermilion O-Torii Gate, especially during high tide when it appears to float on the water, and at sunset",
        "Exploring the Itsukushima Shrine complex (a UNESCO World Heritage site), built on stilts over the water, with its distinctive red-lacquered corridors and halls",
        "Walking out to the O-Torii Gate during low tide to see it up close (check tide schedules)",
        "Interacting with the friendly wild sika deer that roam freely on Miyajima Island",
        "Taking the Miyajima Ropeway up Mount Misen for panoramic views of the Seto Inland Sea and hiking its trails",
        "Sampling local Miyajima delicacies like grilled oysters and momiji manju (maple leaf-shaped cakes)"
    ],
    "gyeongbokgung_palace": [
        "Exploring the main halls like Geunjeongjeon (Throne Hall) and pavilions like Gyeonghoeru (Royal Banquet Hall on a pond)",
        "Watching the impressive Royal Guard Changing Ceremony (Sumunjang Gyedaeui) held several times a day at Gwanghwamun and Heungnyemun Gates",
        "Renting a Hanbok (traditional Korean attire) from nearby shops to enter the palace for free and for an immersive photo experience",
        "Visiting the National Folk Museum of Korea and the National Palace Museum of Korea, both located within the palace grounds",
        "Taking a guided tour (available in multiple languages) to learn about the Joseon Dynasty's history and palace architecture",
        "Photographing the intricate Dancheong (colorful painted patterns) on the wooden structures and the serene Hyangwonjeong Pavilion"
    ],
    "n_seoul_tower": [
        "Taking a scenic cable car ride up Namsan Mountain to reach the N Seoul Tower",
        "Ascending to the observatory (digital and analog) for breathtaking 360-degree panoramic views of Seoul's cityscape, day and night",
        "Attaching a 'love lock' with a personal message to the famous railings and tree-like structures on the tower's outdoor terrace",
        "Dining at the revolving N.Grill restaurant or other cafes and eateries within the tower complex, enjoying the views",
        "Photographing the city skyline, especially during sunset or when the city lights twinkle at night; the tower itself is also beautifully illuminated",
        "Exploring Namsan Park surrounding the tower, enjoying its walking trails, botanical gardens, and the Namsan Beacon Mounds"
    ],
    "bukchon_hanok_village": [
        "Walking respectfully through the narrow, hilly alleyways lined with hundreds of well-preserved traditional Korean houses (Hanoks)",
        "Photographing the beautiful tiled roofs, wooden beams, and stone walls of the Hanoks, often with views of modern Seoul in the background",
        "Visiting small, private museums, cultural centers, and craft workshops (e.g., for Gahoe Museum, Donglim Knot Museum, or embroidery workshops) within the village",
        "Renting a Hanbok to enhance the experience and take memorable photos in the traditional setting",
        "Observing the 'Eight Scenic Spots of Bukchon' for the best photo opportunities and views (look for photo spot markers)",
        "Enjoying tea at a traditional Hanok teahouse or Browse unique artisan shops while being mindful of it being a residential area"
    ],
    "myeongdong_shopping_street": [
        "Shopping for Korean cosmetics and skincare products from numerous flagship stores and local brands",
        "Exploring trendy fashion boutiques, shoe stores, and accessory shops catering to K-fashion enthusiasts",
        "Indulging in a wide variety of delicious Korean street food from bustling stalls (e.g., tteokbokki, gyeranppang, tornado potato, grilled cheese lobster)",
        "Visiting large department stores like Lotte Department Store and Shinsegae Department Store for luxury goods and diverse shopping",
        "Catching K-pop related events, Browse K-pop merchandise stores, or spotting advertisements featuring K-pop idols",
        "Visiting Myeongdong Cathedral, a historic Gothic-style Catholic church, as a peaceful contrast to the shopping frenzy"
    ],
    "dmz_korea": [
        "Taking a guided tour to the Korean Demilitarized Zone (DMZ), including the Joint Security Area (JSA/Panmunjom) if accessible",
        "Visiting the Third Infiltration Tunnel, one of several tunnels dug by North Korea under the DMZ",
        "Looking into North Korea from observation posts like Dora Observatory",
        "Learning about the history of the Korean War, the division of Korea, and ongoing inter-Korean relations at the DMZ Exhibition Hall or Imjingak Park",
        "Seeing the Bridge of No Return and the Freedom House (within JSA)",
        "Reflecting on the poignant reminders of a divided nation and hopes for reunification"
    ],
    "busan_gamcheon_culture_village": [
        "Exploring the maze-like alleyways adorned with vibrant street art, colorful murals, and whimsical art installations",
        "Photographing the brightly painted terraced houses cascading down the hillside, often called the 'Machu Picchu of Busan' or 'Santorini of Korea'",
        "Following a stamp tour map to discover hidden artworks and viewpoints throughout the village",
        "Visiting small art galleries, quirky shops, and charming cafes run by local artists and residents",
        "Taking photos with iconic sculptures like 'The Little Prince and the Desert Fox' overlooking the village",
        "Enjoying panoramic views of the village and the port of Busan from various observation decks"
    ],
    "jeju_island": [
        "Hiking Hallasan, South Korea's highest mountain and a dormant volcano, or exploring its surrounding national park trails",
        "Visiting Seongsan Ilchulbong (Sunrise Peak), a UNESCO World Heritage tuff cone, especially for sunrise views",
        "Exploring Manjanggul Cave, one of the longest lava tubes in the world (UNESCO site)",
        "Relaxing on beautiful beaches like Hyeopjae Beach (with views of Biyangdo Island) or Jungmun Saekdal Beach (popular for surfing)",
        "Chasing waterfalls such as Cheonjeyeon Falls ('Pond of God') or Jeongbang Falls (falls directly into the ocean)",
        "Discovering unique geological formations like Jusangjeolli Cliffs (hexagonal lava pillars) and Yongduam Rock (Dragon Head Rock)"
    ],
    "changdeokgung_palace_secret_garden": [
        "Taking a mandatory guided tour of the Huwon (Secret Garden), a stunningly beautiful and expansive rear garden of Changdeokgung Palace (UNESCO World Heritage site)",
        "Admiring the harmonious blend of traditional Korean palace architecture (pavilions, halls) with the natural landscape (ponds, streams, ancient trees) within the Secret Garden",
        "Exploring the main palace buildings of Changdeokgung, such as Injeongjeon (main throne hall) and Donhwamun Gate",
        "Photographing the serene Buyongji Pond with Buyongjeong Pavilion and Juhamnu Pavilion in the Secret Garden",
        "Learning about the lives of the Joseon Dynasty royal family who used this palace and garden for leisure and study",
        "Appreciating the palace's design, which is considered more integrated with its natural surroundings than other Seoul palaces"
    ],
    "great_wall": [
        "Hiking or walking along various restored sections like Mutianyu (family-friendly, cable car/chairlift options), Badaling (most famous, can be crowded), or Jinshanling and Simatai (more rugged, great for photography, partially restored)",
        "Taking a cable car, chairlift, or toboggan ride at sections like Mutianyu for easier access and fun descent",
        "Photographing the vast structure snaking across diverse mountainous landscapes, with its watchtowers and fortifications",
        "Learning about the history of its construction (spanning many dynasties), its purpose as a defensive barrier, and the legends associated with it",
        "Participating in guided tours to understand the historical context and logistical details of visiting specific sections",
        "Considering a visit to less crowded, 'wild' sections (with caution and appropriate preparation) for a more adventurous experience"
    ],
    "forbidden_city": [
        "Exploring the vast imperial palace complex, walking from the Meridian Gate (Wumen) through numerous courtyards and halls to the Gate of Divine Might (Shenwumen)",
        "Admiring the magnificent traditional Chinese palatial architecture, intricate roof details, dragon motifs, and symbolic color schemes (yellow and red)",
        "Visiting key structures like the Hall of Supreme Harmony (Taihedian), Hall of Central Harmony (Zhonghedian), and Hall of Preserving Harmony (Baohedian) on the Outer Court",
        "Exploring the Inner Court, including the Palace of Heavenly Purity (Qianqinggong), Hall of Union (Jiaotaidian), and Palace of Earthly Tranquility (Kunninggong)",
        "Visiting the Palace Museum's extensive collections of imperial treasures, including ceramics, paintings, jade, and clocks, housed in various side halls",
        "Strolling through the Imperial Garden (Yuhuayuan) at the northern end of the complex"
    ],
    "terracotta_army": [
        "Viewing the thousands of life-sized terracotta warriors, archers, chariots, and horses arranged in battle formation in the three main excavation pits (Pit 1, Pit 2, Pit 3)",
        "Admiring the individual details of the soldiers, each with unique facial expressions, hairstyles, and armor, reflecting the incredible craftsmanship",
        "Visiting the Exhibition Hall of Bronze Chariots to see the two remarkably preserved and intricate bronze chariots and horses",
        "Learning about Emperor Qin Shi Huang, the first emperor of China, and his quest for immortality that led to the creation of this mausoleum army",
        "Taking a guided tour or using an audio guide for in-depth historical context and insights into the archaeological discoveries",
        "Photographing the impressive scale of the pits and the ranks of warriors (flash photography is usually prohibited)"
    ],
    "the_bund": [
        "Strolling along the wide waterfront promenade (Zhongshan East First Road) to admire the grand colonial-era buildings in various architectural styles (e.g., Gothic, Baroque, Art Deco) – the 'museum of international architecture'",
        "Photographing the spectacular modern skyline of Pudong across the Huangpu River, featuring the Oriental Pearl Tower, Shanghai Tower, and Shanghai World Financial Center, especially stunning at night",
        "Taking a Huangpu River cruise (day or night) for panoramic views of both the historic Bund and the futuristic Pudong skyline",
        "Visiting the interiors of some heritage buildings (many now house banks, hotels, or luxury shops) or admiring their detailed facades",
        "Dining at upscale restaurants or enjoying drinks at rooftop bars within the Bund buildings, offering magnificent views",
        "Observing locals engaging in morning tai chi, evening strolls, or public dancing along the promenade"
    ],
    "li_river_guilin": [
        "Taking a scenic boat cruise along the Li River from Guilin to Yangshuo (or vice versa) to admire the stunning karst mountain landscapes",
        "Photographing the picturesque limestone peaks, bamboo groves, rice paddies, and water buffalo along the riverbanks – scenes often depicted in traditional Chinese paintings",
        "Opting for a traditional bamboo raft ride (motorized) on sections of the Li River or the Yulong River (a tributary near Yangshuo) for a closer experience with the scenery",
        "Exploring the Reed Flute Cave or Seven Star Park in Guilin before or after the river cruise",
        "Visiting Xingping Town along the Li River, known for the landscape depicted on the 20 RMB banknote",
        "Cycling or hiking in the countryside around Yangshuo to further explore the karst scenery"
    ],
    "potala_palace": [
        "Exploring the majestic Potala Palace, a UNESCO World Heritage site, former winter residence of the Dalai Lamas, with its White Palace (administrative) and Red Palace (religious sections)",
        "Admiring the unique Tibetan architecture, including massive stone walls, golden roofs, intricate murals, thangkas, and numerous chapels and tombs of past Dalai Lamas",
        "Ascending the numerous staircases (visitors need to be acclimatized to the high altitude of Lhasa)",
        "Photographing the imposing palace from various viewpoints, such as Chakpori Hill (for sunrise/sunset views) or Potala Palace Square",
        "Learning about Tibetan Buddhism, the history of the Dalai Lamas, and the cultural significance of the palace",
        "Observing pilgrims performing kora (circumambulation) around the palace"
    ],
    "zhangjiajie_national_forest_park": [
        "Admiring the towering quartz-sandstone pillars and peaks, often shrouded in mist, that inspired the 'Hallelujah Mountains' in the movie Avatar (especially in the Yuanjiajie area)",
        "Riding the Bailong Elevator (Hundred Dragons Elevator), a massive glass elevator built onto the side of a cliff, for dramatic views",
        "Walking across the Zhangjiajie Grand Canyon Glass Bridge (requires separate entry/location from the main park but often combined in tours) for a thrilling experience",
        "Taking cable cars, like the one to Tianzi Mountain or Huangshizhai, for breathtaking panoramic vistas",
        "Hiking along scenic trails, such as the Golden Whip Stream, to enjoy the lush forests, clear streams, and unique rock formations from below",
        "Exploring other key areas like Tianmen Mountain (Heaven's Gate Mountain, often a separate visit) with its cable car, skywalk, and Tianmen Cave"
    ],
    "west_lake_hangzhou": [
        "Taking a leisurely boat ride on the serene West Lake to enjoy its scenic beauty and visit islands like Xiaoyingzhou (Three Ponds Mirroring the Moon)",
        "Walking or cycling along the Su Causeway (Sudi) and Bai Causeway (Baidi), lined with willow and peach trees",
        "Admiring iconic landmarks such as Leifeng Pagoda (rebuilt), Broken Bridge (Duan Qiao, famous in legend), and the Mid-Lake Pavilion",
        "Visiting traditional gardens and temples around the lake, like Lingyin Temple (Temple of the Soul's Retreat) and Guo's Villa",
        "Watching the 'Impression West Lake' outdoor musical performance on the lake at night (seasonal, created by Zhang Yimou)",
        "Photographing the picturesque landscapes, especially during different seasons (e.g., lotus blooms in summer, osmanthus in autumn)"
    ],
    "summer_palace_beijing": [
        "Strolling through the vast imperial garden, a masterpiece of Chinese landscape garden design, centered around Longevity Hill and Kunming Lake",
        "Taking a boat ride on Kunming Lake, especially a traditional dragon boat",
        "Walking along the Long Corridor (Chang Lang), a covered walkway decorated with thousands of paintings",
        "Visiting key structures like the Hall of Benevolence and Longevity (Renshoudian), Tower of Buddhist Incense (Foxiangge) on Longevity Hill, and the Marble Boat",
        "Exploring Suzhou Street, a recreated canal-side shopping street from imperial times",
        "Admiring the beautiful traditional Chinese architecture, bridges (like the Seventeen-Arch Bridge), and landscaped gardens"
    ],
    "petronas_towers": [
        "Visiting the Skybridge, the double-decker bridge connecting the two towers on the 41st and 42nd floors, for close-up views and a unique perspective (book tickets in advance)",
        "Ascending to the Observation Deck on the 86th floor of Tower 2 for breathtaking panoramic views of Kuala Lumpur's skyline",
        "Photographing the iconic twin skyscrapers, an architectural marvel, especially when brilliantly illuminated at night",
        "Shopping at the upscale Suria KLCC mall located at the base of the towers, featuring luxury brands and diverse retail options",
        "Relaxing or strolling in the beautifully landscaped KLCC Park, which includes a man-made lake (Lake Symphony) with water fountain shows, a jogging track, and a children's playground",
        "Visiting nearby attractions like Petrosains Discovery Centre (interactive science museum) or Aquaria KLCC (oceanarium)"
    ],
    "marina_bay_sands": [
        "Visiting the Sands SkyPark Observation Deck on the 57th floor for stunning 360-degree panoramic views of Singapore's skyline, Gardens by the Bay, and the Singapore Strait",
        "Swimming in the world-famous rooftop infinity pool (exclusive access for Marina Bay Sands hotel guests)",
        "Shopping at The Shoppes at Marina Bay Sands, a luxury mall featuring international brands, a canal with sampan rides, and the Digital Light Canvas",
        "Watching the 'Spectra – A Light & Water Show' in the evening, a spectacular outdoor multimedia presentation over Marina Bay",
        "Visiting the ArtScience Museum, with its iconic lotus-inspired design, showcasing art, science, culture, and technology exhibitions",
        "Dining at celebrity chef restaurants or exploring diverse culinary options within the integrated resort, and trying your luck at the casino"
    ],
    "gardens_by_the_bay": [
        "Exploring the two massive cooled conservatories: the Flower Dome (showcasing exotic plants from Mediterranean and semi-arid regions) and the Cloud Forest (featuring a stunning indoor waterfall and plants from tropical highlands)",
        "Walking along the OCBC Skyway, an aerial walkway suspended between two Supertrees, for panoramic views of the Supertree Grove and surrounding gardens",
        "Watching the 'Garden Rhapsody' light and sound show at Supertree Grove in the evening, where the Supertrees come alive with dazzling lights synchronized to music",
        "Strolling through the various outdoor themed gardens, such as the Heritage Gardens, Serene Garden, and the outdoor art sculptures",
        "Photographing the unique horticultural displays, futuristic Supertrees, and innovative sustainable architecture",
        "Letting children play at the Far East Organization Children's Garden with its water play areas and treehouses"
    ],
    "taj_mahal": [
        "Admiring the breathtaking beauty and perfect symmetry of the ivory-white marble mausoleum, a UNESCO World Heritage site and a symbol of eternal love",
        "Photographing the iconic structure from various angles, especially during sunrise or sunset when the light casts different hues on the marble, and its reflection in the central water channels",
        "Strolling through the formal Mughal gardens (Charbagh) that surround the mausoleum, with their pathways, fountains, and reflecting pools",
        "Learning about the poignant love story of Mughal emperor Shah Jahan and his wife Mumtaz Mahal, for whom the Taj Mahal was built",
        "Observing the intricate marble inlay work (pietra dura) featuring semi-precious stones, and the detailed calligraphy on the monument's facade",
        "Visiting the mosque and the guesthouse (jawab) that flank the main mausoleum, providing architectural balance"
    ],
    "angkor_wat": [
        "Exploring the vast temple complex of Angkor Wat, the world's largest religious monument, admiring its iconic five lotus-bud towers and intricate bas-reliefs",
        "Watching the sunrise over Angkor Wat, a magical experience as the temple silhouette emerges against the colorful sky (can be very crowded)",
        "Taking a guided tour to understand the history of the Khmer Empire, the Hindu and Buddhist symbolism, and the architectural features of the temple",
        "Examining the detailed stone carvings and extensive galleries depicting scenes from Hindu epics like the Ramayana and Mahabharata, and historical events",
        "Photographing the temple's reflection in the surrounding moat and its grandeur from different perspectives",
        "Visiting other nearby temples in the Angkor Archaeological Park, such as Angkor Thom (Bayon, Baphuon), Ta Prohm (Tomb Raider temple), and Banteay Srei"
    ],
    "ha_long_bay": [
        "Taking an overnight cruise or a day boat trip to explore the stunning seascape of thousands of limestone karsts and islets rising from the emerald green waters (UNESCO World Heritage site)",
        "Kayaking or taking a bamboo boat ride through hidden lagoons, caves, and around the towering rock formations for a closer experience",
        "Visiting impressive caves and grottoes, such as Thien Cung Cave (Heavenly Palace Cave) or Dau Go Cave (Wooden Stakes Cave), with their stalactites and stalagmites",
        "Swimming in the calm waters or relaxing on small, secluded beaches (e.g., Titop Island beach)",
        "Photographing the breathtaking and often misty scenery, especially at sunrise or sunset",
        "Learning about the local culture by visiting floating fishing villages or pearl farms"
    ],
    "mount_everest": [
        "Trekking to Everest Base Camp (South Base Camp in Nepal or North Base Camp in Tibet) for close-up views of Mount Everest and the surrounding Himalayan giants (requires significant preparation and acclimatization)",
        "Taking a scenic mountain flight from Kathmandu for breathtaking aerial views of Mount Everest and the Himalayan range (a popular option for non-trekkers)",
        "Photographing the world's highest peak, especially during sunrise or sunset when alpenglow illuminates the summit",
        "Learning about Sherpa culture and mountaineering history in villages like Namche Bazaar (on the Nepal EBC trek)",
        "Visiting monasteries like Tengboche Monastery (Nepal) for spiritual insights and stunning mountain backdrops",
        "For experienced mountaineers: attempting to summit Mount Everest (a highly challenging and expensive endeavor)"
    ],
    "bagan": [
        "Exploring the vast archaeological zone, home to thousands of ancient Buddhist temples, pagodas, and stupas dating from the 9th to 13th centuries (UNESCO World Heritage site)",
        "Renting an e-bike or horse cart to navigate the sandy tracks and discover both major temples (like Ananda, Shwezigon, Thatbyinnyu, Dhammayangyi) and smaller, hidden gems",
        "Watching the sunrise or sunset over the temple-strewn plains from a high vantage point (e.g., a designated viewing mound or a temple terrace where permitted)",
        "Taking a hot air balloon ride over Bagan at sunrise for an unforgettable panoramic view of the temples (seasonal, typically October to April)",
        "Photographing the unique landscape of ancient religious structures against the backdrop of the Irrawaddy River and distant hills",
        "Learning about the history of the Pagan Kingdom and Buddhist art and architecture through temple murals and local guides"
    ],
    "grand_palace_wat_phra_kaew": [
        "Exploring the magnificent Grand Palace complex, the former official residence of the Kings of Siam (Thailand), with its stunning traditional Thai architecture",
        "Visiting Wat Phra Kaew (Temple of the Emerald Buddha) within the palace grounds to see the highly revered Emerald Buddha statue, carved from a single jade stone",
        "Admiring the intricate details of the golden spires (chedis), colorful mosaics made of glass and porcelain, ornate gables, and mythical guardian statues (yakshas and kinnaras)",
        "Photographing the dazzling exteriors of the royal halls (e.g., Chakri Maha Prasat Hall), temples, and courtyards",
        "Learning about Thai history, royal traditions, and Buddhist art and iconography (dressing respectfully is required: no shorts, sleeveless shirts)",
        "Observing the Ramakien murals (Thai version of the Ramayana) that adorn the walls of the cloister surrounding Wat Phra Kaew"
    ],

    # 歐洲地標 (Europe)
    "eiffel_tower": [
        "Ascending to the different observation platforms (1st floor, 2nd floor, summit) for stunning panoramic views of Paris",
        "Enjoying a romantic meal or champagne at Le Jules Verne restaurant (2nd floor) or other tower eateries",
        "Picnicking on the Champ de Mars park with the Eiffel Tower as a magnificent backdrop",
        "Photographing the iconic structure day and night, especially during the hourly sparkling lights show after sunset",
        "Taking a Seine River cruise that offers unique perspectives of the tower from the water",
        "Learning about its history, engineering, and construction at the first-floor exhibition or through guided tours"
    ],
    "louvre_museum": [
        "Viewing iconic masterpieces like the Mona Lisa, Venus de Milo, Winged Victory of Samothrace, and Liberty Leading the People",
        "Exploring diverse and extensive art collections spanning from ancient civilizations (Egyptian, Greek, Roman) to 19th-century European painting and sculpture",
        "Taking a guided tour or using an audio guide to navigate the vast museum and focus on key exhibits or specific interests",
        "Admiring the architecture of the former royal palace (Louvre Palace) and the modern glass Louvre Pyramid designed by I. M. Pei",
        "Strolling through the Tuileries Garden (Jardin des Tuileries) adjacent to the museum, leading towards Place de la Concorde",
        "Photographing the impressive exterior, the pyramid, and select artworks (where permitted without flash)"
    ],
    "mont_saint_michel": [
        "Walking up the winding Grande Rue, lined with shops and restaurants, to reach the magnificent Benedictine Abbey at the summit",
        "Taking a guided tour (or audio guide) of the Mont Saint-Michel Abbey to explore its Romanesque and Gothic sections, cloister, and refectory",
        "Admiring the breathtaking views of the surrounding bay and tidal flats from the abbey terraces and ramparts",
        "Photographing the island commune at different times of the day and tides, especially during high tide when it's completely surrounded by water, or at sunrise/sunset",
        "Exploring the small museums, chapels, and historic houses within the village",
        "Witnessing the dramatic tidal changes (some of the highest in Europe) from a safe vantage point on the island or the mainland causeway"
    ],
    "arc_de_triomphe": [
        "Climbing the stairs (or taking the elevator part-way) to the rooftop observation deck for panoramic 360-degree views of Paris, including the Champs-Élysées, Eiffel Tower, Sacré-Cœur Basilica, and La Défense",
        "Admiring the intricate Neoclassical sculptures and reliefs on the arch's facade, depicting scenes from French military history, particularly Napoleonic victories",
        "Visiting the Tomb of the Unknown Soldier beneath the arch and witnessing the rekindling of the eternal flame each evening at 6:30 PM",
        "Photographing the monument, the twelve avenues radiating from Place Charles de Gaulle (formerly Place de l'Étoile), and the surrounding cityscape",
        "Learning about its historical significance and the events it commemorates through the small museum inside the arch",
        "Observing major events like the Bastille Day military parade (July 14th) which passes through the arch"
    ],
    "big_ben": [ # Elizabeth Tower
        "Photographing the iconic clock tower (officially Elizabeth Tower), the Great Bell (Big Ben), and the adjacent Houses of Parliament (Palace of Westminster) from Westminster Bridge or Parliament Square",
        "Hearing the famous chimes of Big Ben, especially the distinctive Westminster Quarters",
        "Taking a guided tour of the Houses of Parliament (when Parliament is not in session, book in advance) to see inside this historic building",
        "Walking along the South Bank of the River Thames for classic views of Big Ben and the Houses of Parliament, especially at dusk or night when illuminated",
        "Visiting nearby attractions such as Westminster Abbey, the Churchill War Rooms, and the London Eye",
        "Learning about its history, recent restoration, and the workings of the UK Parliament"
    ],
    "stonehenge": [
        "Walking the designated pathway around the prehistoric stone circle, observing the massive sarsen stones and smaller bluestones",
        "Listening to the informative audio guide (available in multiple languages) to learn about the history, construction phases, and various theories about Stonehenge's purpose (e.g., astronomical observatory, burial site, ceremonial center)",
        "Visiting the world-class exhibition and visitor centre to see archaeological finds, reconstructions of a Neolithic village, and interactive displays",
        "Photographing the mysterious ancient monument against the backdrop of Salisbury Plain, especially during sunrise or sunset for dramatic lighting (special access may be required for inner circle access during these times)",
        "Considering the astronomical alignments, particularly during the summer and winter solstices when special events are often held",
        "Exploring the surrounding landscape, which is rich in other Neolithic and Bronze Age sites, including Woodhenge and Durrington Walls"
    ],
    "tower_of_london": [
        "Taking a captivating tour led by a Yeoman Warder (Beefeater), who shares fascinating stories, historical anecdotes, and traditions of the Tower",
        "Viewing the spectacular Crown Jewels, a dazzling collection of royal regalia including crowns, orbs, and sceptres, housed securely in the Jewel House",
        "Exploring the White Tower, the oldest part of the castle (a Norman keep), which houses the Royal Armouries collection",
        "Walking the ancient ramparts and learning about the Tower's multifaceted history as a royal palace, fortress, prison, and place of execution",
        "Seeing the famous resident ravens and learning about the legend that the kingdom and Tower will fall if they leave",
        "Visiting sites of historical significance such as Traitors' Gate, the Bloody Tower, and the execution site on Tower Green"
    ],
    "buckingham_palace": [
        "Watching the iconic Changing of the Guard ceremony (check schedule and arrive early for a good view), a display of British pageantry with guards in red tunics and bearskin hats",
        "Photographing the grand facade of Buckingham Palace, the official London residence of the UK monarch, and the impressive Victoria Memorial in front",
        "Touring the magnificent State Rooms during the Summer Opening (typically July to September, when the King is not in residence, book tickets in advance)",
        "Visiting the Queen's Gallery to see rotating exhibitions of artworks and treasures from the Royal Collection",
        "Exploring the Royal Mews to see historic royal carriages and vehicles",
        "Strolling through St. James's Park or Green Park, adjacent to the palace, for pleasant walks and views"
    ],
    "colosseum": [
        "Taking a guided historical tour (or using an audio guide) to learn about the gladiatorial contests, wild animal hunts, public spectacles, and the lives of Roman citizens and emperors associated with the amphitheater",
        "Exploring the different levels of the ancient Flavian Amphitheatre, including the arena floor (if accessible with your ticket), the hypogeum (underground tunnels and chambers), and the upper tiers for panoramic views",
        "Photographing the imposing structure, a symbol of Imperial Rome, noting its architectural features like arches, columns, and the remaining sections of the outer wall",
        "Visiting the adjacent Roman Forum (Foro Romano) and Palatine Hill (Colle Palatino), which are usually included in a combined ticket, to explore the heart of ancient Roman life, politics, and mythology",
        "Learning about Roman engineering, architecture, and the social importance of the games held in the Colosseum",
        "Imagining the roar of the crowds and the dramatic events that once unfolded within its ancient walls, especially when illuminated at night"
    ],
    "leaning_tower_of_pisa": [
        "Climbing the spiral staircase of 294 steps to the top of the leaning bell tower (Torre Pendente di Pisa) for panoramic views of Pisa and the Piazza dei Miracoli (book tickets well in advance as numbers are limited)",
        "Taking the classic and fun forced-perspective photo where you appear to be 'holding up' or 'pushing over' the Leaning Tower",
        "Visiting the magnificent Pisa Cathedral (Duomo di Santa Maria Assunta) and the impressive Baptistery of St. John (Battistero di San Giovanni) located in the same Square of Miracles (Piazza dei Miracoli)",
        "Admiring the beautiful Romanesque architecture of white marble that characterizes the entire complex",
        "Strolling around the well-manicured lawns of the Piazza dei Miracoli and exploring the Camposanto Monumentale (monumental cemetery)",
        "Learning about the history of the tower's construction, the reasons for its unintended tilt, and the centuries of efforts to stabilize it"
    ],
    "trevi_fountain": [
        "Tossing a coin over your right shoulder with your left hand into the fountain – tradition holds that this ensures your return to Rome (toss two coins for a new romance, three for marriage)",
        "Admiring the spectacular Baroque sculptures depicting Oceanus (god of the sea) on a shell-shaped chariot pulled by sea horses and tritons, set against the backdrop of Palazzo Poli",
        "Photographing the magnificent fountain, one of the most famous in the world, especially beautiful when illuminated at night (be prepared for crowds)",
        "Enjoying delicious Italian gelato from a nearby gelateria while people-watching and soaking in the lively atmosphere",
        "Learning about the history and design of the fountain, completed by Nicola Salvi and Giuseppe Pannini, and its connection to ancient Roman aqueducts",
        "Visiting at different times of day, such as early morning, to experience it with fewer crowds and different lighting conditions"
    ],
    "st_peters_basilica": [
        "Exploring the vast and awe-inspiring interior of St. Peter's Basilica, the largest Christian church in the world and a masterpiece of Renaissance architecture",
        "Admiring Michelangelo's magnificent dome from the inside and considering climbing to the top (cupola) for breathtaking panoramic views of St. Peter's Square, Vatican City, and Rome (requires a ticket, involves stairs and an elevator option for part of the way)",
        "Viewing renowned masterpieces such as Michelangelo's Pietà (sculpture of Mary holding Christ's body), Bernini's bronze Baldachin over the papal altar, and various ornate chapels and papal tombs",
        "Visiting the Vatican Grottoes (Papal Tombs) located beneath the basilica, where many popes are interred",
        "Attending a Papal Mass, audience, or the Angelus prayer in St. Peter's Square if your visit coincides with these events (check Vatican schedules)",
        "Strolling through the immense St. Peter's Square (Piazza San Pietro), designed by Bernini, with its grand colonnades, central obelisk, and twin fountains"
    ],
    "sagrada_familia": [
        "Admiring the extraordinary and unique facades of Antoni Gaudí's unfinished masterpiece: the Nativity Facade (celebrating Christ's birth), the Passion Facade (depicting his suffering), and the still-under-construction Glory Facade",
        "Exploring the breathtaking interior, a forest of tree-like columns that branch out to support the roof, illuminated by vibrant stained-glass windows that create an ethereal play of light and color",
        "Taking a guided tour or using an audio guide (highly recommended) to understand Gaudí's visionary architectural concepts, complex symbolism, and the ongoing construction process (book tickets well in advance online)",
        "Ascending one of the towers (Nativity or Passion, requires separate ticket and booking) for close-up views of the spires and panoramic vistas of Barcelona",
        "Visiting the museum located underneath the basilica to learn about Gaudí's life, design models, construction techniques, and the history of the Sagrada Família",
        "Photographing the incredibly detailed architectural elements, both inside and out, reflecting Gaudí's deep connection with nature and spirituality"
    ],
    "alhambra": [
        "Exploring the Nasrid Palaces, the heart of the Alhambra, with their exquisite stucco work, intricate geometric tile mosaics (azulejos), delicate muqarnas (stalactite vaulting), and tranquil courtyards like the Court of the Lions and Court of the Myrtles (book tickets well in advance, as entry is timed and limited)",
        "Wandering through the serene Generalife gardens, the summer palace of the Nasrid rulers, with its beautiful patios, fountains, flowerbeds, and scenic pathways",
        "Visiting the Alcazaba, the oldest part of the Alhambra, a formidable fortress with towers offering panoramic views over the city of Granada and the surrounding landscape",
        "Admiring the Palace of Charles V, a contrasting Renaissance-style building within the Alhambra complex, now housing museums",
        "Photographing the stunning Moorish architecture, intricate details, and the interplay of light, water, and shadow throughout the complex",
        "Learning about the history of the last Muslim emirs in Spain, the Reconquista, and the subsequent Christian modifications to the palace"
    ],
    "brandenburg_gate": [
        "Walking through the iconic neoclassical triumphal arch, a potent symbol of German history, division, and reunification",
        "Photographing the gate, especially when illuminated at night, and the Quadriga statue (a chariot drawn by four horses, driven by Victoria, the Roman goddess of victory) that crowns it",
        "Visiting Pariser Platz, the grand square in front of the gate, which also houses embassies (e.g., US, French) and the Hotel Adlon",
        "Learning about its historical significance, from its construction under King Frederick William II of Prussia to its role during the Napoleonic Wars, the Nazi era, the Cold War (when it stood near the Berlin Wall), and German reunification",
        "Reflecting at nearby historical sites such as the Reichstag Building (German Parliament), the Memorial to the Murdered Jews of Europe, and the remnants of the Berlin Wall",
        "Observing the bustling atmosphere and often, street performers or public events taking place in Pariser Platz"
    ],
    "neuschwanstein_castle": [
        "Taking a mandatory guided tour of the fairytale-like interiors of King Ludwig II's dream castle, inspired by Richard Wagner's operas and medieval legends (tickets must be purchased in Hohenschwangau village and are timed; book online in advance to secure a spot)",
        "Photographing the castle from Marienbrücke (Mary's Bridge), a pedestrian bridge offering the classic, breathtaking postcard view of Neuschwanstein perched on its rugged hill with the Bavarian Alps in the background",
        "Hiking, taking a horse-drawn carriage, or riding a shuttle bus up the steep road from Hohenschwangau village to the castle entrance",
        "Admiring the scenic beauty of the Bavarian Alps, forests, and nearby lakes (Alpsee and Schwansee) surrounding the castle",
        "Learning about the eccentric 'Mad' King Ludwig II of Bavaria, his passion for art and architecture, and the romantic inspirations behind the castle's design",
        "Visiting the nearby Hohenschwangau Castle, King Ludwig II's childhood home, for more royal history and contrasting architectural style"
    ],
    "acropolis_of_athens": [
        "Exploring the Parthenon, the magnificent Doric temple dedicated to the goddess Athena Parthenos, an enduring symbol of Ancient Greece, democracy, and Western civilization",
        "Visiting other significant ancient structures on the Sacred Rock, such as the Erechtheion (with its iconic Porch of the Caryatids), the Propylaea (the monumental gateway), and the Temple of Athena Nike",
        "Walking to the ancient Theatre of Dionysus on the southern slope, considered the birthplace of Greek tragedy, and the well-preserved Odeon of Herodes Atticus (still used for performances)",
        "Enjoying breathtaking panoramic views of Athens, including the Plaka district, Mount Lycabettus, and the Saronic Gulf, from the summit of the Acropolis",
        "Visiting the Acropolis Museum, located at the foot of the Acropolis, to see original sculptures, friezes (including parts of the Parthenon Marbles), and artifacts found on the site, displayed in a modern architectural setting",
        "Photographing the ancient ruins against the backdrop of the modern city, especially beautiful during sunrise or sunset"
    ],
    "santorini_oia": [
        "Exploring the charming, narrow, winding pathways of Oia village, famous for its whitewashed cave houses (yposkafa), blue-domed churches, and vibrant bougainvillea",
        "Watching and photographing the world-renowned sunset over the caldera from Oia, typically from the ruins of the Venetian castle (Kastro) or designated sunset viewing spots (arrive very early to secure a good spot as it gets extremely crowded)",
        "Taking a boat tour to the volcanic islands of Nea Kameni (to hike on the crater) and Palea Kameni (to swim in the hot springs), and Thirassia island",
        "Relaxing on unique volcanic sand beaches around Santorini, such as Red Beach (Kokkini Paralia), Perissa Beach, or Kamari Beach (Oia itself is on cliffs, not a beach town)",
        "Hiking the scenic caldera trail between Fira (the capital) and Oia (approx. 3-4 hours) for stunning views",
        "Indulging in local Santorinian cuisine and distinctive Assyrtiko wines at cliffside restaurants with caldera views"
    ],
    "canals_of_venice": [
        "Taking a classic gondola ride through the narrow, picturesque canals, often accompanied by a serenading gondolier",
        "Exploring the city on foot, getting lost in the maze-like streets (calli) and discovering charming squares (campi) and countless bridges",
        "Cruising along the Grand Canal on a vaporetto (public water bus) to see famous landmarks like the Rialto Bridge, Doge's Palace, and Ca' d'Oro from the water",
        "Photographing the unique cityscape with its historic buildings seemingly rising from the water, colorful reflections, and bustling canal life",
        "Visiting iconic sights such as St. Mark's Square (Piazza San Marco), St. Mark's Basilica, and the Doge's Palace",
        "Enjoying cicchetti (Venetian tapas) and wine at traditional bacari (local bars)"
    ],
    "florence_cathedral_duomo": [ # Cattedrale di Santa Maria del Fiore
        "Admiring Brunelleschi's magnificent red-tiled dome, an architectural marvel of the Renaissance, and climbing to the top (463 steps, book well in advance) for breathtaking panoramic views of Florence",
        "Exploring the interior of the Florence Cathedral (Cattedrale di Santa Maria del Fiore), noting its Gothic architecture, stained glass windows, and Vasari's frescoes of the Last Judgment inside the dome",
        "Visiting Giotto's Campanile (bell tower) and climbing its 414 steps for another stunning perspective of the Duomo and the city",
        "Admiring the Florence Baptistery (Battistero di San Giovanni) with its famous bronze doors, especially Ghiberti's 'Gates of Paradise'",
        "Visiting the Museo dell'Opera del Duomo to see original artworks from the Duomo complex, including Ghiberti's original doors and Michelangelo's 'The Deposition' Pietà",
        "Photographing the intricate white, green, and pink marble facade of the Duomo, Campanile, and Baptistery in Piazza del Duomo"
    ],
    "anne_frank_house": [
        "Taking a poignant and reflective tour through the Secret Annex (Achterhuis) where Anne Frank and her family hid from Nazi persecution during World War II (book tickets online months in advance, as they sell out quickly)",
        "Viewing Anne Frank's original diary and other personal belongings displayed in the museum",
        "Learning about the lives of the people who hid in the annex, their helpers, and the historical context of the Holocaust and Jewish persecution in Amsterdam",
        "Reflecting on Anne's story, her writings, and the enduring messages of tolerance, human rights, and hope",
        "Visiting the museum exhibitions that provide further information about the persecution of Jews during the war and contemporary issues of discrimination",
        "Seeing the unassuming exterior of the canal house on Prinsengracht 263 and imagining the hidden life within"
    ],
    "canals_of_amsterdam": [
        "Taking a scenic canal cruise (day or evening) through the Grachtengordel (canal belt), a UNESCO World Heritage site, to admire the historic gabled canal houses, bridges, and houseboats",
        "Renting a private boat or pedal boat (canal bike) to explore the canals at your own pace",
        "Walking or cycling along the picturesque canals, such as Prinsengracht, Keizersgracht, and Herengracht, enjoying the charming atmosphere",
        "Photographing the iconic canal scenes, including narrow houses, beautiful bridges (like Magere Brug 'Skinny Bridge'), and reflections in the water",
        "Visiting canal house museums like Museum Willet-Holthuysen or Museum Van Loon to see how wealthy Amsterdammers lived in the Golden Age",
        "Enjoying a drink or meal at a waterside cafe or restaurant along the canals"
    ],
    "charles_bridge_prague": [
        "Walking across the historic Charles Bridge (Karlův most), a medieval stone arch bridge connecting the Old Town and Lesser Town (Malá Strana) over the Vltava River",
        "Admiring the 30 statues and statuaries of saints that line the bridge, including the famous statue of St. John of Nepomuk (touching the plaque is said to bring good luck or ensure a return to Prague)",
        "Enjoying panoramic views of Prague Castle, St. Vitus Cathedral, the Vltava River, and the surrounding city skyline from the bridge",
        "Photographing the bridge, its Gothic towers (Old Town Bridge Tower and Lesser Town Bridge Towers), and the bustling atmosphere, especially beautiful at dawn, dusk, or at night",
        "Observing street artists, musicians, and vendors who often line the bridge (can be very crowded during peak season)",
        "Climbing one of the bridge towers for a higher vantage point and stunning photos"
    ],
    "red_square_st_basils_cathedral": [
        "Admiring the iconic, vibrantly colored onion domes of St. Basil's Cathedral (Cathedral of Vasily the Blessed), a unique masterpiece of Russian architecture, located at the southern end of Red Square",
        "Exploring Red Square (Krasnaya Ploshchad), the historic central square of Moscow, and visiting other significant landmarks such as Lenin's Mausoleum, the fortified walls of the Moscow Kremlin, the State Historical Museum, and the GUM department store",
        "Taking a guided tour or visiting the interior of St. Basil's Cathedral (now a museum) to see its chapels and learn about its history",
        "Photographing the stunning ensemble of Red Square, especially the contrast between St. Basil's, the Kremlin towers, and the red brick of the Historical Museum",
        "Learning about the historical events that have taken place in Red Square, a UNESCO World Heritage site and a focal point of Russian history and culture",
        "Visiting the GUM department store for its unique architecture, high-end shops, and traditional Russian ice cream"
    ],
    "edinburgh_castle": [
        "Exploring the historic fortress perched atop Castle Rock, dominating the skyline of Edinburgh",
        "Viewing the Honours of Scotland (the Scottish Crown Jewels) and the Stone of Destiny, ancient symbols of Scottish royalty",
        "Visiting St. Margaret's Chapel, the oldest surviving building in Edinburgh, and the Great Hall with its impressive timber roof and collection of arms and armour",
        "Witnessing the firing of the One O'Clock Gun (Monday to Saturday), a time-honored tradition",
        "Walking along the castle ramparts for panoramic views of Edinburgh city, including the Royal Mile, Arthur's Seat, and the Firth of Forth",
        "Learning about the castle's rich and often turbulent history, its role in Scottish wars, and its famous residents through exhibitions and guided tours"
    ],
    "matterhorn": [
        "Photographing the iconic pyramidal peak of the Matterhorn, one of the world's most recognizable mountains, from Zermatt (Switzerland) or Breuil-Cervinia (Italy)",
        "Taking a cable car or cogwheel train to viewpoints like Gornergrat (for stunning Matterhorn and glacier views), Klein Matterhorn (Matterhorn Glacier Paradise - Europe's highest cable car station), or Sunnegga/Rothorn for different perspectives",
        "Hiking or mountain biking on the numerous trails around Zermatt that offer spectacular views of the Matterhorn and surrounding Alpine scenery (e.g., Riffelsee lake trail for reflections)",
        "Skiing or snowboarding in the Zermatt-Cervinia ski area, which offers year-round skiing on the Theodul Glacier with the Matterhorn as a backdrop",
        "Learning about the history of the first ascent and mountaineering in the region at the Matterhorn Museum (Zermatlantis) in Zermatt",
        "Enjoying Alpine cuisine and the charming car-free village atmosphere of Zermatt"
    ],
    "palace_of_versailles": [
        "Exploring the opulent State Apartments of the Palace of Versailles, including the magnificent Hall of Mirrors (Galerie des Glaces), the King's Grand Apartment, and the Queen's Grand Apartment",
        "Wandering through the vast and meticulously designed Gardens of Versailles (Jardins de Versailles) by André Le Nôtre, with their formal parterres, fountains (musical fountain shows are held on certain days), canals, and groves",
        "Visiting the Grand Trianon and Petit Trianon, smaller palaces within the estate, offering a more intimate glimpse into royal life, and Marie Antoinette's Hamlet (Le Hameau de la Reine), a rustic retreat",
        "Taking a guided tour or using an audio guide to learn about the history of the French monarchy (especially Louis XIV, Louis XV, and Louis XVI), courtly life, and the significant historical events that took place at Versailles (like the signing of the Treaty of Versailles)",
        "Renting a boat on the Grand Canal or exploring the gardens by bicycle or electric golf cart",
        "Photographing the lavish Baroque architecture, gilded interiors, impressive sculptures, and expansive landscapes"
    ],

    # 北美地標 (North America)
    "statue_of_liberty": [
        "Taking a ferry from Battery Park (Manhattan) or Liberty State Park (New Jersey) to Liberty Island",
        "Visiting the Statue of Liberty Museum to learn about its history, construction, symbolism, and Frederic Auguste Bartholdi's design",
        "Accessing the pedestal for closer views of the statue and panoramic views of New York Harbor and the Manhattan skyline (requires advance booking)",
        "Climbing to the crown for a unique, albeit small, viewing experience (requires very limited, advance booking months ahead)",
        "Combining the visit with a trip to Ellis Island and the National Museum of Immigration, which is usually included in the same ferry ticket",
        "Walking around the perimeter of Liberty Island for different photo opportunities of Lady Liberty and the surrounding views"
    ],
    "golden_gate_bridge": [
        "Walking or biking across the bridge on the dedicated pedestrian and bicycle paths (check access times for each side)",
        "Photographing the iconic Art Deco suspension bridge, painted in 'International Orange,' from various renowned viewpoints like Battery Spencer (Marin Headlands), Vista Point (north end), Fort Point National Historic Site (south end, underneath the bridge), or Baker Beach",
        "Visiting the Golden Gate Bridge Welcome Center (south end) to learn about the bridge's history, engineering marvels, and see exhibits",
        "Driving across the bridge for a classic experience (toll applies southbound into San Francisco)",
        "Exploring the Golden Gate National Recreation Area, which encompasses areas on both sides of the bridge offering trails and views",
        "Taking a bay cruise that sails under the Golden Gate Bridge and around Alcatraz Island for different perspectives"
    ],
    "grand_canyon": [
        "Hiking along the Rim Trail or venturing down into the canyon on trails like Bright Angel Trail or South Kaibab Trail (South Rim) or North Kaibab Trail (North Rim) – be aware of difficulty, heat, and need for water",
        "Taking a mule ride along the rim or into the canyon (book well in advance as these are very popular)",
        "Watching and photographing the spectacular sunrise or sunset over the canyon from popular viewpoints like Mather Point, Yavapai Point, Hopi Point (South Rim), or Bright Angel Point (North Rim) as the canyon walls change colors dramatically",
        "Visiting various viewpoints along the South Rim Drive (using the free park shuttle buses for much of the year) or the North Rim Scenic Drives",
        "Taking a helicopter or fixed-wing aircraft tour for a breathtaking aerial perspective of the vastness of the Grand Canyon (optional, can be costly)",
        "Learning about the geology, ecology, and human history (including Native American cultures) at visitor centers and ranger programs"
    ],
    "hollywood_sign": [
        "Hiking to viewpoints for relatively close-up views of the sign, such as from trails in Griffith Park (e.g., Hollyridge Trail, Wonder View Trail, or near Griffith Observatory) or from Lake Hollywood Park (great for ground-level photos with the sign)",
        "Photographing the iconic white capital letters on Mount Lee from various locations throughout Hollywood and Los Angeles, including the Hollywood & Highland Center observation decks",
        "Visiting Griffith Observatory for excellent panoramic views of the Hollywood Sign, the Los Angeles basin, and the Pacific Ocean (on clear days), as well as its science exhibits",
        "Learning about the history of the sign (originally 'HOLLYWOODLAND') and its significance as a symbol of the American film industry",
        "Driving through the Hollywood Hills for different perspectives (be mindful of residential areas and parking restrictions)",
        "Taking a guided Hollywood tour that often includes stops at optimal sign viewing locations"
    ],
    "white_house": [
        "Viewing and photographing the iconic exterior of the White House from the North Side (Pennsylvania Avenue, in front of Lafayette Square) or the South Side (from the Ellipse)",
        "Taking a public tour of select rooms in the East Wing (requires advance request through a U.S. Member of Congress for citizens, or through one's embassy for foreign nationals, months in advance; security is very strict)",
        "Visiting the White House Visitor Center (located nearby) for exhibits on the history of the White House, its architecture, furnishings, first families, and presidential life",
        "Exploring President's Park, which includes Lafayette Square to the north (with statues of revolutionary war heroes) and the Ellipse to the south",
        "Learning about the history of the U.S. presidency and the symbolic importance of the White House as the executive mansion",
        "Observing any official motorcades or security measures, which can be a unique part of the D.C. experience"
    ],
    "mount_rushmore": [
        "Viewing and photographing the colossal carved faces of U.S. Presidents George Washington, Thomas Jefferson, Theodore Roosevelt, and Abraham Lincoln in the granite of Mount Rushmore",
        "Walking the Presidential Trail, a boardwalk loop that provides closer views and different angles of the sculptures",
        "Visiting the Lincoln Borglum Visitor Center and Museum to learn about the history of the carving, the sculptor Gutzon Borglum, the tools and techniques used, and the depicted presidents",
        "Attending the evening lighting ceremony (held seasonally, typically May to September), which includes a ranger talk and the illumination of the monument",
        "Exploring the Sculptor's Studio to see original plaster models and tools used in the carving process",
        "Learning about the significance of each president chosen and the broader history of the United States represented by the memorial"
    ],
    "times_square": [
        "Experiencing the dazzling and overwhelming display of massive digital billboards, flashing neon lights, and advertisements that illuminate the square 24/7",
        "People-watching in one of the world's busiest and most famous pedestrian intersections, soaking in the vibrant and energetic atmosphere",
        "Attending a Broadway show in the renowned Theater District, which is centered around Times Square (purchase tickets in advance or try the TKTS booth for same-day discounts)",
        "Shopping at flagship stores of international brands, unique boutiques, and souvenir shops that line the square",
        "Dining at diverse restaurants, from themed eateries and fast-food chains to upscale dining options",
        "Photographing the iconic scenery, costumed characters (be aware they may expect tips), and the general buzz, especially stunning at night or during the New Year's Eve ball drop"
    ],
    "cn_tower": [
        "Ascending via high-speed glass-fronted elevators to the main observation levels: the LookOut Level (with floor-to-ceiling panoramic window walls) and the famous Glass Floor (for a thrilling view 342m/1,122ft straight down)",
        "Going even higher to the SkyPod, one of the highest observation platforms in the world (an additional 33 storeys above the main observation level, requires separate ticket)",
        "Experiencing the EdgeWalk (seasonal, weather permitting), the world’s highest full-circle, hands-free walk on a 1.5m (5ft) wide ledge encircling the top of the Tower’s main pod, 356m/1,168ft above the ground (for thrill-seekers, book well in advance)",
        "Dining at the award-winning 360 Restaurant, a revolving restaurant that completes a full rotation approximately every 72 minutes, offering stunning views and a cellar in the sky",
        "Photographing the panoramic cityscape of Toronto, Lake Ontario, and on clear days, even Niagara Falls or New York State",
        "Learning about the tower's construction, engineering, and its role as a communications tower through exhibits and informational displays"
    ],
    "chichen_itza": [
        "Exploring the magnificent El Castillo (Pyramid of Kukulcán), the iconic step-pyramid that dominates the site, and learning about its Mayan astronomical alignments (e.g., the serpent shadow effect during the spring and autumn equinoxes)",
        "Visiting other significant structures within the ancient Mayan city, such as the Great Ball Court (the largest in Mesoamerica), the Temple of the Warriors (with its Chac Mool figure and colonnades), the Group of a Thousand Columns, and the Observatory (El Caracol)",
        "Observing the intricate stone carvings, Mayan hieroglyphs, and depictions of deities like Kukulcán (the feathered serpent) and Chaac (the rain god) on the buildings",
        "Swimming in a nearby sacred cenote (natural sinkhole), such as Ik Kil or Yokdzonot, for a refreshing break after exploring the hot ruins (often combined with Chichen Itza tours)",
        "Taking a guided tour (highly recommended) to understand the rich history, culture, rituals, and astronomical knowledge of the Mayan civilization that flourished at Chichen Itza (a UNESCO World Heritage site)",
        "Photographing the impressive pyramids, ancient temples, and unique architectural features of this important archaeological site"
    ],
    "niagara_falls": [
        "Taking the 'Maid of the Mist' (USA side) or 'Hornblower Niagara Cruises/Niagara City Cruises' (Canada side) boat tour for an up-close and powerful experience of the mist and roar of the Horseshoe Falls, American Falls, and Bridal Veil Falls",
        "Experiencing the 'Cave of the Winds' (USA side), where you walk on wooden walkways near the base of the Bridal Veil Falls and get drenched by the spray",
        "Walking along the Niagara Parkway (Canada) or within Niagara Falls State Park (USA) for various viewpoints of the falls, including Terrapin Point (USA) and Table Rock Centre (Canada)",
        "Viewing the falls illuminated in vibrant colors at night, and enjoying seasonal fireworks displays over the falls",
        "Visiting Journey Behind the Falls (Canada) to descend and walk through tunnels behind the Horseshoe Falls",
        "Exploring other attractions in the Niagara Parks area, such as Queen Victoria Park, Clifton Hill (Canada side for entertainment), or taking a helicopter tour for an aerial view"
    ],
    "central_park": [
        "Strolling or biking through the vast network of paths, exploring iconic areas like The Mall and Literary Walk, Bethesda Terrace and Fountain, Strawberry Fields (John Lennon memorial), and Conservatory Water (for model sailboat racing)",
        "Having a picnic on the Great Lawn or Sheep Meadow with views of the surrounding Manhattan skyline",
        "Renting a rowboat or taking a gondola ride on The Lake, passing under Bow Bridge",
        "Visiting Central Park Zoo, the Central Park Carousel, or Wollman Rink (ice skating in winter, other activities in summer)",
        "Exploring the more rugged northern section with the North Woods and Harlem Meer, or the formal Conservatory Garden",
        "Attending free events like Shakespeare in the Park (summer), concerts by the New York Philharmonic, or SummerStage performances"
    ],
    "las_vegas_strip": [
        "Walking along Las Vegas Boulevard South to see the extravagant themed mega-resorts and casinos, such as Bellagio (with its famous fountains), Venetian (with canals and gondola rides), Caesars Palace, Luxor (pyramid and sphinx), and Paris Las Vegas (Eiffel Tower replica)",
        "Watching free street-side shows like the Fountains of Bellagio water ballet or the Mirage Volcano eruption",
        "Trying your luck at casino games (responsibly, if you choose to gamble)",
        "Attending a world-class show, concert, or residency featuring top entertainers",
        "Shopping at high-end retail promenades like The Forum Shops at Caesars, Grand Canal Shoppes at The Venetian, or Crystals at CityCenter",
        "Dining at celebrity chef restaurants or enjoying diverse culinary experiences, and experiencing the vibrant nightlife with clubs and bars"
    ],
    "yellowstone_national_park": [
        "Watching the eruption of Old Faithful geyser and exploring the Upper Geyser Basin, home to the world's largest concentration of geysers",
        "Admiring the vibrant colors of Grand Prismatic Spring in the Midway Geyser Basin and other geothermal features like mudpots and fumaroles at Mammoth Hot Springs' travertine terraces",
        "Wildlife viewing: looking for bison, elk, grizzly bears, wolves, pronghorn, and bighorn sheep in areas like Lamar Valley (often called 'America's Serengeti') and Hayden Valley",
        "Visiting the Grand Canyon of the Yellowstone to see the impressive Upper and Lower Falls of the Yellowstone River from viewpoints like Artist Point",
        "Driving the scenic Grand Loop Road to access various park attractions and enjoying hikes on numerous trails of varying difficulty",
        "Boating, fishing, or simply enjoying the views at Yellowstone Lake, North America's largest high-elevation lake"
    ],
    "banff_national_park_lake_louise": [ # and Moraine Lake
        "Photographing the stunning turquoise waters of Lake Louise with the Victoria Glacier and the iconic Fairmont Chateau Lake Louise in the background; also visit the equally breathtaking Moraine Lake with its vivid blue water framed by the Valley of the Ten Peaks (access to Moraine Lake is restricted, plan ahead with shuttle or tour)",
        "Canoeing or kayaking on Lake Louise or Moraine Lake for an unforgettable experience amidst the majestic mountain scenery",
        "Hiking popular trails such as the Lake Agnes Teahouse Trail, Plain of Six Glaciers Trail (from Lake Louise), or the Rockpile Trail and Lakeshore Trail (at Moraine Lake)",
        "Taking the Lake Louise Gondola (or Banff Gondola near Banff town) for panoramic views of the Canadian Rockies, glaciers, and valleys",
        "Wildlife viewing for animals like elk, deer, bighorn sheep, and occasionally bears (maintain a safe distance)",
        "Exploring the charming town of Banff, Johnston Canyon, or driving the scenic Icefields Parkway towards Jasper National Park"
    ],
    "space_needle_seattle": [
        "Taking a 41-second elevator ride to the 520-foot high observation deck for 360-degree panoramic views of Seattle's skyline, Elliott Bay, Puget Sound, the Olympic Mountains, Cascade Mountains, and Mount Rainier on clear days",
        "Stepping out onto 'The Loupe,' the world's first and only revolving glass floor, offering thrilling downward views of the city and the Needle's structure",
        "Enjoying a meal or drinks at 'The Loupe Lounge' (revolving glass floor level) or other dining options within the Space Needle",
        "Photographing the iconic futuristic structure, a symbol of Seattle and the 1962 World's Fair, from various angles and at different times of day (especially sunset or when illuminated at night)",
        "Learning about the history of the Space Needle and Seattle through interactive exhibits on the observation deck",
        "Visiting nearby attractions at Seattle Center, such as the Chihuly Garden and Glass, Museum of Pop Culture (MoPOP), or Pacific Science Center"
    ],

    # 南美地標 (South America)
    "machu_picchu": [
        "Exploring the remarkably preserved ancient Inca citadel, including the Temple of the Sun, Intihuatana stone, Royal Tomb, residential areas, and agricultural terraces",
        "Hiking Huayna Picchu or Machu Picchu Mountain for breathtaking panoramic views of the ruins and surrounding Andes (requires separate, pre-booked tickets well in advance as permits are limited)",
        "Taking a guided tour (highly recommended) to learn about Inca history, cosmology, architecture, and the possible purposes of this enigmatic 'Lost City of the Incas' (a UNESCO World Heritage site)",
        "Photographing the stunning landscape of stone ruins nestled dramatically in the cloud-forested mountains, especially during sunrise or late afternoon for optimal lighting",
        "Watching the sunrise over Machu Picchu (if staying in Aguas Calientes or taking an early bus/train to arrive before dawn)",
        "Visiting the Sun Gate (Inti Punku) for a classic first view of Machu Picchu if hiking a portion of the Inca Trail, or the Inca Bridge for a glimpse of Inca engineering"
    ],
    "christ_the_redeemer": [
        "Taking the historic cog train (Trem do Corcovado) or an official van up Corcovado Mountain through Tijuca National Park to reach the statue",
        "Admiring the colossal Art Deco statue of Jesus Christ with outstretched arms, a global icon of Rio de Janeiro and Brazil, and one of the New Seven Wonders of the World",
        "Enjoying breathtaking 360-degree panoramic views of Rio de Janeiro, including Sugarloaf Mountain, Copacabana and Ipanema beaches, Guanabara Bay, and the Rodrigo de Freitas Lagoon",
        "Photographing the statue from its base and the spectacular surrounding cityscape and landscape",
        "Visiting the small chapel located at the base of the statue, dedicated to Our Lady of Aparecida, the patron saint of Brazil",
        "Learning about the history, construction, and significance of the monument through informational plaques or guided tours"
    ],
    "iguazu_falls": [
        "Exploring the extensive network of walkways and viewpoints on both the Argentinian side (Parque Nacional Iguazú) and the Brazilian side (Parque Nacional do Iguaçu) for different perspectives of the hundreds of waterfalls",
        "Experiencing the immense power and roar of the Devil's Throat (Garganta del Diablo / Garganta do Diabo), the largest and most dramatic cataract, from viewing platforms that extend over the falls",
        "Taking a thrilling boat tour (e.g., Gran Aventura on the Argentinian side or Macuco Safari on the Brazilian side) that goes close to and even under some of the falls (expect to get soaked!)",
        "Walking the Upper Circuit and Lower Circuit trails on the Argentinian side for varied views and close encounters with different sections of the falls",
        "Photographing the spectacular waterfalls, lush subtropical rainforest, and frequent rainbows that form in the mist",
        "Wildlife spotting in the surrounding national parks, looking for coatis, toucans, monkeys, and colorful butterflies"
    ],
    "galapagos_islands": [
        "Taking a multi-day cruise on a small ship or yacht to visit various islands, as each island offers unique landscapes and wildlife viewing opportunities (the most common way to explore)",
        "Snorkeling or scuba diving in the rich marine environment to see sea lions, marine iguanas, Galapagos penguins, sea turtles, sharks (like hammerheads and Galapagos sharks), rays, and colorful fish",
        "Observing unique and fearless wildlife up close (maintaining a respectful distance as per park rules), such as giant tortoises in their natural habitat (e.g., at El Chato Tortoise Reserve on Santa Cruz Island), blue-footed boobies, frigatebirds with inflated red pouches, and land iguanas",
        "Hiking on volcanic landscapes, lava fields, and pristine beaches, learning about the geology and evolutionary history of the islands that inspired Charles Darwin",
        "Visiting the Charles Darwin Research Station on Santa Cruz Island to learn about conservation efforts and the tortoise breeding program",
        "Kayaking, birdwatching, and photographing the extraordinary biodiversity and unique natural beauty of this UNESCO World Heritage site"
    ],
    "torres_del_paine_national_park": [
        "Hiking famous multi-day treks like the 'W Trek' (typically 4-5 days) or the 'O Circuit' (full loop, 8-10 days) for stunning views of the granite Paine Towers, Cuernos del Paine (Horns), glaciers, lakes, and Patagonian landscapes",
        "Undertaking day hikes to iconic viewpoints, such as the Base of the Towers (Mirador Las Torres), Mirador Cuernos, or viewpoints overlooking Grey Glacier and Lake Pehoé",
        "Photographing the dramatic granite peaks, turquoise glacial lakes (like Pehoé, Nordenskjöld, Sarmiento), vast ice fields (Southern Patagonian Ice Field), and unique flora and fauna",
        "Wildlife viewing for animals such as guanacos, condors, foxes, and occasionally pumas (though elusive)",
        "Taking a boat trip on Lake Grey to get close to the face of Grey Glacier, or a catamaran across Lake Pehoé",
        "Experiencing the wild and unpredictable Patagonian weather, and enjoying the pristine, rugged beauty of this renowned national park (a UNESCO Biosphere Reserve)"
    ],
    "angel_falls": [
        "Taking a multi-day expedition (typically 3 days/2 nights) by motorized dugout canoe (curiara) up the Carrao and Churún rivers through Canaima National Park to reach the base of Angel Falls (Salto Ángel), the world's tallest uninterrupted waterfall",
        "Hiking through the rainforest from the river camp to a viewpoint near the base of the falls (Mirador Laime) to witness the spectacular cascade plunging from the summit of Auyán-Tepui",
        "Swimming in pools at the base of the falls or in the river (depending on conditions and guide advice)",
        "Spending nights in hammocks at rustic jungle camps, experiencing the sounds and atmosphere of the remote wilderness",
        "Taking a scenic overflight by small plane from Canaima or Ciudad Bolívar for a breathtaking aerial view of Angel Falls and the vast expanse of Auyán-Tepui (often done as part of a tour or separately)",
        "Learning about the indigenous Pemón culture and the unique geology of the tepuis (table-top mountains) in Canaima National Park (a UNESCO World Heritage site)"
    ],
    "salar_de_uyuni": [
        "Taking a multi-day 4x4 jeep tour (typically 1 to 4 days) across the vast Salar de Uyuni, the world's largest salt flat, and the surrounding Altiplano region",
        "During the dry season (May-November): Experiencing the seemingly endless white expanse of hexagonal salt patterns, visiting Incahuasi Island (Fish Island) with its giant ancient cacti and panoramic views, and taking creative forced-perspective photographs",
        "During the rainy season (December-April): Witnessing the breathtaking 'world's largest mirror' effect when a thin layer of water covers the salt flat, creating stunning reflections of the sky and clouds (access can be limited)",
        "Visiting the 'train cemetery' (Cementerio de Trenes) just outside Uyuni town, with its rusted antique steam locomotives",
        "Seeing the salt harvesting mounds and the original salt hotel (Palacio de Sal, now a museum), and learning about salt extraction",
        "Exploring the surrounding Altiplano highlights on longer tours, such as colorful lagoons (Laguna Colorada with flamingos, Laguna Verde), geysers (Sol de Mañana), hot springs (Termas de Polques), and dramatic volcanic landscapes"
    ],

    # 中東/非洲地標 (Middle East / Africa)
    "pyramids_of_giza": [
        "Exploring the exteriors of the three main pyramids: the Great Pyramid of Giza (Pyramid of Khufu), the Pyramid of Khafre, and the Pyramid of Menkaure",
        "Entering one of the pyramids (requires separate ticket, can be claustrophobic, limited access) to see the internal chambers",
        "Visiting the Great Sphinx of Giza, the colossal limestone statue with the body of a lion and the head of a human, and the adjacent Valley Temple of Khafre",
        "Taking a camel or horse ride across the Giza plateau for different perspectives and classic photo opportunities of the pyramids against the desert backdrop",
        "Visiting the Solar Boat Museum (Khufu Ship Museum) next to the Great Pyramid to see a remarkably preserved ancient Egyptian funerary boat",
        "Attending the Sound and Light Show in the evening, where the pyramids and Sphinx are illuminated with accompanying narration about ancient Egyptian history (optional)"
    ],
    "burj_khalifa": [
        "Ascending to one of the world's highest observation decks: 'At the Top' on levels 124 and 125, or the more exclusive 'At the Top, SKY' on level 148, for breathtaking panoramic views of Dubai's futuristic skyline, the desert, and the Arabian Gulf",
        "Photographing the stunning views, especially during sunset or at night when the city lights up",
        "Dining at At.mosphere, the restaurant and lounge on level 122, one of the world's highest restaurants",
        "Watching the spectacular Dubai Fountain show, a choreographed water and light display at the base of the Burj Khalifa on Burj Lake (best viewed from the Waterfront Promenade of The Dubai Mall or a nearby restaurant)",
        "Learning about the engineering, architecture, and construction of the world's tallest building through interactive exhibits at the observation deck levels",
        "Shopping at The Dubai Mall, one of the world's largest shopping malls, which is connected to the Burj Khalifa and also houses the Dubai Aquarium & Underwater Zoo"
    ],
    "petra_jordan": [ # (Was "petra")
        "Walking through the Siq, a narrow, winding sandstone gorge that serves as the dramatic main entrance to the ancient Nabataean city of Petra",
        "Emerging from the Siq to behold the iconic Al-Khazneh (The Treasury), an elaborately carved temple facade, one of the most famous archaeological sites in the world",
        "Exploring the vast archaeological site, including the Street of Facades, the Theatre, the Royal Tombs, the Colonnaded Street, and the Great Temple",
        "Hiking up the 800+ steps to Ad Deir (The Monastery), another magnificent rock-cut facade offering stunning views (a significant but rewarding climb)",
        "Photographing the 'Rose City' with its unique rock-cut architecture and dramatic desert landscape, noting how the sandstone changes color throughout the day",
        "Learning about the history of the Nabataean civilization, their ingenuity in water management, and their role as a trading hub, through a local guide or the Petra Museum"
    ],
    "table_mountain": [
        "Taking the rotating aerial cableway (Table Mountain Aerial Cableway) to the summit for spectacular 360-degree views of Cape Town, Table Bay, Robben Island, the Cape Peninsula, and the Atlantic Ocean",
        "Hiking up one of the many trails to the summit, such as Platteklip Gorge (the most direct but steep), Skeleton Gorge (more scenic, starting from Kirstenbosch Gardens), or India Venster (more challenging, for experienced hikers)",
        "Walking along the well-maintained pathways on the flat-topped summit, exploring different viewpoints, and enjoying the unique fynbos vegetation (part of the Cape Floral Kingdom, a UNESCO World Heritage site)",
        "Photographing the panoramic vistas, the city nestled below, and the dramatic cloud formations known as the 'tablecloth' that sometimes cover the mountain",
        "Abseiling from the top of Table Mountain (for adventure seekers, with registered operators)",
        "Enjoying refreshments or a meal at the café or restaurant on the summit while taking in the views"
    ],
    "sheikh_zayed_grand_mosque": [
        "Taking a guided tour (free, offered regularly) to learn about the mosque's stunning architecture, Islamic art, cultural significance, and design elements",
        "Admiring the pristine white Macedonian marble exterior, the 82 domes of various sizes, the four towering minarets, and the intricate floral designs inlaid with semi-precious stones",
        "Exploring the vast main prayer hall, which houses the world's largest hand-knotted carpet and one of the world's largest Swarovski crystal chandeliers",
        "Walking through the serene courtyards (sahan) with their reflective pools that beautifully mirror the mosque's columns and arches",
        "Photographing the breathtaking beauty of the mosque, both during the day (when the white marble gleams) and at night (when it is illuminated by a unique lunar lighting system that changes with the phases of the moon)",
        "Respecting the dress code (modest attire required for all visitors; abayas provided for women if needed) and behaving reverently within the sacred space"
    ],
    "masai_mara_national_reserve": [
        "Going on multiple game drives (early morning and late afternoon are often best) in a 4x4 safari vehicle with an experienced guide to spot the 'Big Five' (lion, leopard, elephant, rhino, buffalo) and other wildlife like cheetahs, giraffes, zebras, wildebeest, hyenas, and numerous bird species",
        "Witnessing the Great Migration (typically July to October), where millions of wildebeest and zebras cross the Mara River from Tanzania's Serengeti, facing crocodiles and predators (a dramatic natural spectacle)",
        "Taking a hot air balloon safari at sunrise over the Masai Mara plains for a breathtaking panoramic view of the landscape and wildlife, followed by a champagne breakfast in the bush",
        "Visiting a traditional Maasai village (manyatta) to learn about Maasai culture, customs, traditions, and their way of life (usually arranged through lodges or tour operators)",
        "Enjoying bush breakfasts, lunches, or sundowner drinks in scenic spots within the reserve",
        "Nature walks with armed rangers (where permitted) for a different perspective on the flora and fauna, or birdwatching (the Mara has a rich avian diversity)"
    ],
    "victoria_falls": [
        "Viewing the spectacular curtain of falling water from various viewpoints on both the Zambian side (e.g., Knife-Edge Bridge, Boiling Pot trail) and the Zimbabwean side (e.g., Main Falls, Rainbow Falls, Danger Point) of the Zambezi River",
        "Feeling the immense spray (the 'Smoke that Thunders' or Mosi-oa-Tunya) and hearing the roar of one of the world's largest waterfalls by combined width and height",
        "Taking a guided tour of the rainforest trails on the Zimbabwean side, which offer numerous perspectives of the falls",
        "Experiencing thrilling adventure activities such as white-water rafting or kayaking on the powerful Zambezi River below the falls (seasonal), bungee jumping from the Victoria Falls Bridge, or gorge swinging",
        "Taking a helicopter or microlight flight ('Flight of Angels') over Victoria Falls for breathtaking aerial views",
        "Enjoying a sunset cruise on the upper Zambezi River, spotting wildlife like hippos, crocodiles, and birds, or visiting Livingstone Island and swimming in Devil's Pool or Angel's Pool at the edge of the falls (seasonal, Zambian side only, requires guided tour)"
    ],
    "kilimanjaro": [
        "Attempting to climb Mount Kilimanjaro, Africa's highest peak and the world's tallest freestanding mountain, via one of several routes (e.g., Machame, Lemosho, Marangu, Rongai) with a licensed guide and porters (a challenging multi-day trek requiring good physical condition and acclimatization)",
        "Experiencing the diverse ecological zones on the ascent, from lush rainforest and moorland to alpine desert and the arctic summit zone with its glaciers and snowfields",
        "Reaching Uhuru Peak (5,895m / 19,341ft) on Kibo, the highest of Kilimanjaro's three volcanic cones, typically at sunrise for spectacular views",
        "Photographing the stunning mountain scenery, unique flora (like giant groundsels and lobelias), and the iconic snow-capped summit",
        "Learning about the geology of the dormant volcano and the local Chagga culture from guides and porters",
        "For those not climbing: Enjoying views of Kilimanjaro from nearby towns like Moshi or Arusha (Tanzania), or Amboseli National Park (Kenya) on clear days, or taking a scenic flight around the mountain"
    ],
    "dead_sea": [
        "Effortlessly floating in the hyper-saline waters of the Dead Sea, an experience unique due to the water's high salt and mineral concentration (avoid getting water in your eyes)",
        "Applying the mineral-rich black mud from the Dead Sea shores onto your skin, known for its therapeutic and cosmetic benefits, then rinsing off in the sea or showers",
        "Relaxing at one of the resorts or public beaches along the shores in Jordan or Israel, often equipped with facilities like showers, pools, and spas",
        "Photographing the unique landscape of the Dead Sea (the lowest point on Earth's land surface), with its calm, dense turquoise waters reflecting the arid surrounding mountains and desert, and distinctive salt formations",
        "Learning about the history, geology, and unique ecosystem of the Dead Sea, and the challenges it faces (e.g., receding water levels)",
        "Visiting nearby historical and natural sites, such as Masada and Ein Gedi (Israel side) or Mujib Biosphere Reserve and Ma'in Hot Springs (Jordan side)"
    ],
    "dome_of_the_rock": [
        "Admiring the iconic, gleaming golden dome of the Dome of the Rock (Qubbat as-Sakhrah), an Islamic shrine located on the Temple Mount (Haram al-Sharif) in the Old City of Jerusalem (access for non-Muslims to the Temple Mount plaza is restricted to specific hours and days, and entry into the Dome of the Rock itself is generally not permitted for non-Muslims)",
        "Observing the octagonal structure of the Dome of the Rock, adorned with intricate blue and turquoise ceramic tilework, Quranic calligraphy, and mosaics, from the Temple Mount plaza",
        "Photographing this significant religious landmark, a site of immense importance in Islam, Judaism (as the location of the First and Second Temples), and Christianity, from various viewpoints within and outside the Old City (e.g., from the Mount of Olives)",
        "Exploring the wider Temple Mount/Haram al-Sharif plaza, also home to Al-Aqsa Mosque (entry for non-Muslims generally not permitted into the mosque building) and other smaller Islamic structures, while respecting the site's sanctity and rules",
        "Learning about the history and religious significance of the Foundation Stone (Even HaShetiya in Hebrew, As-Sakhrah in Arabic) which the Dome of the Rock enshrines, believed to be the site where Abraham prepared to sacrifice his son, and from where Prophet Muhammad is said to have ascended to heaven",
        "Visiting other holy sites within the Old City of Jerusalem, such as the Western Wall, the Church of the Holy Sepulchre, and walking the Via Dolorosa"
    ],

    # 大洋洲地標 (Oceania)
    "sydney_opera_house": [
        "Taking a guided architectural tour to learn about its history, innovative design by Jørn Utzon, construction challenges, and its various performance venues (e.g., Concert Hall, Joan Sutherland Theatre)",
        "Attending a world-class performance, such as opera, ballet, classical music concert, theatre, or contemporary music show, inside one of its iconic halls (book tickets in advance)",
        "Photographing the iconic white sail-like shells from various angles and vantage points, such as from a ferry on Sydney Harbour, Mrs Macquarie's Chair, Circular Quay, or while walking across the Sydney Harbour Bridge",
        "Dining at one of the many restaurants or enjoying drinks at bars with stunning harbour views located at the Opera House, like Opera Bar or Portside Sydney",
        "Walking around the exterior promenade, admiring the building's unique form up close, and enjoying the bustling atmosphere of Circular Quay",
        "Visiting the gift shop for souvenirs, or attending free outdoor events and festivals that sometimes take place on the forecourt"
    ],
    "uluru": [
        "Walking the full Uluru base walk (approximately 10.6 km / 6.6 miles loop, takes about 3-4 hours) to experience the immense scale and diverse features of the monolith up close, including waterholes, rock art sites, and unique geological formations (best done early morning or late afternoon to avoid heat)",
        "Watching and photographing the spectacular color changes of Uluru at sunrise and sunset from designated viewing areas (Talinguru Nyakunytjaku for sunrise, Uluru sunset viewing area for sunset)",
        "Taking a guided tour led by an Anangu (local Aboriginal people) guide to learn about the cultural significance, Dreamtime stories (Tjukurpa), and traditional laws associated with Uluru (highly recommended for a deeper understanding)",
        "Visiting the Uluru-Kata Tjuta Cultural Centre to gain insights into Anangu culture, art, and land management practices",
        "Exploring the nearby Kata Tjuta (The Olgas) rock formations, particularly the Walpa Gorge walk or the Valley of the Winds walk, for different but equally stunning desert landscapes",
        "Stargazing in the clear desert night sky (Uluru is in a remote area with minimal light pollution), or participating in an Sounds of Silence or Field of Light Uluru experience (optional, requires booking)"
    ],
    "great_barrier_reef": [
        "Snorkeling or scuba diving from a boat tour or island resort to explore the vibrant coral reefs, encounter colorful tropical fish, sea turtles, manta rays, reef sharks, and other diverse marine life (many tour options cater to different skill levels)",
        "Taking a glass-bottom boat tour or a semi-submersible tour to view the coral and marine life without getting wet, suitable for all ages",
        "Going on a scenic helicopter or seaplane flight for a breathtaking aerial perspective of the reef's intricate patterns, turquoise waters, islands, and cays (e.g., seeing Heart Reef from the air)",
        "Visiting a pontoon stationed on the outer reef, which often includes facilities like underwater observatories, snorkeling platforms, and introductory dive options",
        "Learning about marine conservation, the ecology of the Great Barrier Reef (a UNESCO World Heritage site), and the threats it faces (e.g., climate change, coral bleaching) through onboard marine biologist talks or visitor centers",
        "Sailing or taking a catamaran cruise through the Whitsunday Islands or other reef areas, often including stops for snorkeling, swimming, and relaxing on pristine beaches like Whitehaven Beach"
    ],
    "hobbiton_movie_set": [
        "Taking a fully guided walking tour through the 12-acre movie set, visiting numerous Hobbit Holes with their distinctive round doors and charming details, as seen in 'The Lord of the Rings' and 'The Hobbit' trilogies",
        "Photographing iconic locations such as Bag End (Bilbo and Frodo Baggins' home), the Party Tree, the Mill, and the double-arched stone bridge",
        "Enjoying a complimentary, specially brewed beverage (ale, cider, or non-alcoholic ginger beer) at The Green Dragon Inn, a faithfully reconstructed pub from the movies",
        "Listening to fascinating behind-the-scenes stories and filmmaking secrets from your guide about how the Hobbiton set was created and used in the films",
        "Shopping for Middle-earth themed souvenirs, Weta Workshop collectibles, and other memorabilia at the Hobbiton Shire Store",
        "Immersing oneself in the picturesque, meticulously maintained landscape of The Shire, with its rolling green hills, gardens, and charming atmosphere, located on a working sheep farm"
    ],
    "sydney_harbour_bridge": [
        "Undertaking the BridgeClimb Sydney experience, climbing the steel arches of the bridge for breathtaking 360-degree panoramic views of Sydney Harbour, the Opera House, the city skyline, and beyond (various climb options available, book in advance)",
        "Walking or cycling across the bridge on the dedicated pedestrian walkway (eastern side) or cycleway (western side) for free, enjoying fantastic views and photo opportunities",
        "Visiting the Pylon Lookout on the southeastern pylon for historical exhibits about the bridge's construction and impressive views from its observation deck",
        "Photographing the iconic 'Coathanger' from various vantage points, such as Mrs Macquarie's Chair, Luna Park, McMahon's Point, or from a ferry on the harbour",
        "Taking a ferry ride under the bridge for a different perspective of its massive scale and engineering",
        "Learning about the history, design, and construction of this engineering marvel, which opened in 1932"
    ],
    "fiordland_national_park_milford_sound": [ # and Doubtful Sound
        "Taking a scenic boat cruise (day cruise or overnight cruise) on Milford Sound to experience its dramatic fiord scenery, with sheer cliffs (like Mitre Peak), cascading waterfalls (e.g., Stirling Falls, Bowen Falls), lush rainforest, and often, mist-shrouded peaks",
        "Kayaking on Milford Sound or Doubtful Sound for a more intimate and peaceful experience of the fiords, getting closer to the shoreline and wildlife",
        "Spotting wildlife such as New Zealand fur seals basking on rocks, dolphins, Fiordland crested penguins (seasonal), and various seabirds",
        "Exploring Doubtful Sound, a larger and more remote fiord, often via a tour involving a cruise across Lake Manapouri and a bus trip over Wilmot Pass",
        "Hiking parts of famous multi-day tracks like the Milford Track (requires advance booking months or even years ahead) or Kepler Track, or shorter day walks accessible from the Milford Road",
        "Photographing the awe-inspiring, moody, and pristine landscapes of Fiordland National Park (a UNESCO World Heritage site), known for its high rainfall and dramatic beauty"
    ],
    "bondi_beach": [
        "Swimming, surfing, or sunbathing on the famous crescent-shaped golden sands of Bondi Beach, one of Australia's most iconic beaches",
        "Learning to surf at one of the surf schools located at Bondi Beach",
        "Walking or jogging the scenic Bondi to Coogee Coastal Walk (approx. 6km), offering stunning ocean views, dramatic cliffs, and access to other beaches like Tamarama and Bronte",
        "Visiting the Bondi Icebergs Club, a historic swimming club with its famous ocean pool that waves often crash into, and enjoying a meal or drink at its restaurant with panoramic beach views",
        "Exploring the vibrant Campbell Parade, the street fronting the beach, lined with cafes, restaurants, surf shops, and fashion boutiques",
        "People-watching and soaking in the lively beach culture, or visiting the Bondi Markets (on weekends) for local crafts, fashion, and food"
    ],
    "aoraki_mount_cook_national_park": [
        "Hiking popular trails such as the Hooker Valley Track (relatively easy, 3-hour return) for spectacular views of Aoraki/Mount Cook, Mueller Glacier, Hooker Lake (with icebergs), and suspension bridges",
        "Admiring New Zealand's highest peak, Aoraki/Mount Cook, and the surrounding majestic Southern Alps, glaciers, and turquoise glacial lakes like Lake Pukaki and Lake Tekapo (often framed by colorful lupins in summer - Nov/Dec)",
        "Stargazing in the Aoraki Mackenzie International Dark Sky Reserve, one of the best places in the world for viewing the night sky due to its clear, dark conditions (guided stargazing tours available)",
        "Taking a scenic helicopter or ski plane flight for breathtaking aerial views of the mountains and glaciers, possibly including a snow landing on Tasman Glacier",
        "Visiting the Sir Edmund Hillary Alpine Centre at The Hermitage Hotel to learn about the region's mountaineering history, flora, fauna, and geology",
        "Experienced mountaineering or glacier hiking with qualified guides (for advanced adventurers)"
    ],
    "twelve_apostles_great_ocean_road": [
        "Viewing and photographing the iconic limestone stacks known as The Twelve Apostles (though fewer than twelve now remain due to erosion) rising dramatically from the Southern Ocean, from the designated clifftop viewing platforms",
        "Visiting at sunrise or sunset for the most spectacular lighting conditions, as the stacks change color and long shadows are cast (be prepared for crowds during these times)",
        "Walking along the boardwalks and pathways to different lookout points offering various perspectives of The Twelve Apostles and the rugged coastline",
        "Exploring other nearby dramatic rock formations and coastal features along this section of the Great Ocean Road, such as Loch Ard Gorge (site of a famous shipwreck), Gibson Steps (descend to the beach for a different view of some stacks), The Arch, London Bridge (now London Arch), and The Grotto",
        "Taking a scenic helicopter flight over The Twelve Apostles and the Shipwreck Coast for a breathtaking aerial view (optional)",
        "Learning about the geology of the limestone formations, the power of coastal erosion, and the maritime history of the area at the Twelve Apostles Visitor Centre"
    ],
    "easter_island_moai": [
        "Exploring Rano Raraku, the volcanic crater quarry where hundreds of moai were carved and many still stand in various stages of completion, some half-buried",
        "Visiting Ahu Tongariki to witness the impressive sight of fifteen restored moai standing on the largest ceremonial platform on the island, especially spectacular at sunrise",
        "Relaxing or swimming at Anakena Beach, a picturesque white coral sand beach with palm trees, featuring Ahu Nau Nau with its well-preserved moai (some with topknots) and Ahu Ature Huki",
        "Hiking to the rim of the Rano Kau volcano to see its stunning crater lake and visiting the nearby ceremonial village of Orongo, known for its Birdman cult petroglyphs and dramatic cliffside location",
        "Discovering Ahu Akivi, a unique inland platform with seven moai that face the ocean, aligned with the equinox",
        "Learning about Rapa Nui culture, history, and the mysteries of the moai at the Museo Antropológico Sebastián Englert in Hanga Roa",
        "Photographing the diverse moai sites across the island, such as Ahu Tahai (near Hanga Roa, good for sunset), Puna Pau (quarry for the red scoria pukao topknots), and Te Pito Kura (site of a large fallen moai and a magnetic stone)",
        "Attending a traditional Rapa Nui cultural performance (dance and music) or experiencing a Curanto (traditional underground feast)",
        "Stargazing in the exceptionally clear night skies due to the island's remote location and lack of light pollution",
        "Renting a car, scooter, or bicycle to independently explore the island's numerous archaeological sites and natural beauty, or taking a guided tour for deeper insights"
    ]
}

Writing landmark_activities.py


In [ ]:
# %%writefile clip_model_manager.py

import torch
import clip
import numpy as np
import logging
import traceback
from typing import List, Dict, Tuple, Optional, Union, Any
from PIL import Image

class CLIPModelManager:
    """
    專門管理 CLIP 模型相關的操作，包括模型載入、設備管理、圖像和文本的特徵編碼等核心功能
    """

    def __init__(self, model_name: str = "ViT-B/16", device: str = None):
        """
        初始化 CLIP 模型管理器

        Args:
            model_name: CLIP模型名稱，默認為"ViT-B/16"
            device: 運行設備，None則自動選擇
        """
        self.logger = logging.getLogger(__name__)
        self.model_name = model_name

        # 設置運行設備
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        self.model = None
        self.preprocess = None

        self._initialize_model()

    def _initialize_model(self):
        """
        初始化CLIP模型
        """
        try:
            self.logger.info(f"Initializing CLIP model ({self.model_name}) on {self.device}")
            self.model, self.preprocess = clip.load(self.model_name, device=self.device)
            self.logger.info("Successfully loaded CLIP model")
        except Exception as e:
            self.logger.error(f"Error loading CLIP model: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def encode_image(self, image_input: torch.Tensor) -> torch.Tensor:
        """
        編碼圖像特徵

        Args:
            image_input: 預處理後的圖像張量

        Returns:
            torch.Tensor: 標準化後的圖像特徵
        """
        try:
            with torch.no_grad():
                image_features = self.model.encode_image(image_input)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)
                return image_features
        except Exception as e:
            self.logger.error(f"Error encoding image features: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def encode_text_batch(self, text_prompts: List[str], batch_size: int = 128) -> torch.Tensor:
        """
        批量編碼文本特徵，避免CUDA內存問題

        Args:
            text_prompts: 文本提示列表
            batch_size: 批處理大小

        Returns:
            torch.Tensor: 標準化後的文本特徵
        """
        if not text_prompts:
            return None

        try:
            with torch.no_grad():
                features_list = []

                for i in range(0, len(text_prompts), batch_size):
                    batch_prompts = text_prompts[i:i+batch_size]
                    text_tokens = clip.tokenize(batch_prompts).to(self.device)
                    batch_features = self.model.encode_text(text_tokens)
                    batch_features = batch_features / batch_features.norm(dim=-1, keepdim=True)
                    features_list.append(batch_features)

                # 連接所有批次
                if len(features_list) > 1:
                    text_features = torch.cat(features_list, dim=0)
                else:
                    text_features = features_list[0]

                return text_features

        except Exception as e:
            self.logger.error(f"Error encoding text features: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def encode_single_text(self, text_prompts: List[str]) -> torch.Tensor:
        """
        編碼單個文本批次的特徵

        Args:
            text_prompts: 文本提示列表

        Returns:
            torch.Tensor: 標準化後的文本特徵
        """
        try:
            with torch.no_grad():
                text_tokens = clip.tokenize(text_prompts).to(self.device)
                text_features = self.model.encode_text(text_tokens)
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)
                return text_features
        except Exception as e:
            self.logger.error(f"Error encoding single text batch: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def calculate_similarity(self, image_features: torch.Tensor, text_features: torch.Tensor) -> np.ndarray:
        """
        計算圖像和文本特徵之間的相似度

        Args:
            image_features: 圖像特徵張量
            text_features: 文本特徵張量

        Returns:
            np.ndarray: 相似度分數數組
        """
        try:
            with torch.no_grad():
                similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
                similarity = similarity.cpu().numpy() if self.device == "cuda" else similarity.numpy()
                return similarity
        except Exception as e:
            self.logger.error(f"Error calculating similarity: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def preprocess_image(self, image: Union[Image.Image, np.ndarray]) -> torch.Tensor:
        """
        預處理圖像以供CLIP模型使用

        Args:
            image: PIL圖像或numpy數組

        Returns:
            torch.Tensor: 預處理後的圖像張量
        """
        try:
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            image_input = self.preprocess(image).unsqueeze(0).to(self.device)
            return image_input

        except Exception as e:
            self.logger.error(f"Error preprocessing image: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def process_image_region(self, image: Union[Image.Image, np.ndarray], box: List[float]) -> torch.Tensor:
        """
        處理圖像的特定區域

        Args:
            image: 原始圖像
            box: 邊界框 [x1, y1, x2, y2]

        Returns:
            torch.Tensor: 區域圖像的特徵
        """
        try:
            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 裁剪區域
            x1, y1, x2, y2 = map(int, box)
            cropped_image = image.crop((x1, y1, x2, y2))

            # 預處理並編碼
            image_input = self.preprocess_image(cropped_image)
            image_features = self.encode_image(image_input)

            return image_features

        except Exception as e:
            self.logger.error(f"Error processing image region: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def batch_process_regions(self, image: Union[Image.Image, np.ndarray],
                             boxes: List[List[float]]) -> torch.Tensor:
        """
        批量處理多個圖像區域

        Args:
            image: 原始圖像
            boxes: 邊界框列表

        Returns:
            torch.Tensor: 所有區域的圖像特徵
        """
        try:
            # ensure PIL format
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            if not boxes:
                return torch.empty(0)

            # 裁剪並預處理所有區域
            cropped_inputs = []
            for box in boxes:
                x1, y1, x2, y2 = map(int, box)
                cropped_image = image.crop((x1, y1, x2, y2))
                processed_image = self.preprocess(cropped_image).unsqueeze(0)
                cropped_inputs.append(processed_image)

            # 批量處理
            batch_tensor = torch.cat(cropped_inputs).to(self.device)
            image_features = self.encode_image(batch_tensor)

            return image_features

        except Exception as e:
            self.logger.error(f"Error batch processing regions: {e}")
            self.logger.error(traceback.format_exc())
            raise

    def is_model_loaded(self) -> bool:
        """
        檢查模型是否已成功載入

        Returns:
            bool: 模型載入狀態
        """
        return self.model is not None and self.preprocess is not None

    def get_device(self) -> str:
        """
        獲取當前設備

        Returns:
            str: 設備名稱
        """
        return self.device

    def get_model_name(self) -> str:
        """
        獲取模型名稱

        Returns:
            str: 模型名稱
        """
        return self.model_name

Writing clip_model_manager.py


In [ ]:
# %%writefile landmark_data_manager.py

import logging
import traceback
from typing import List, Dict, Tuple, Optional, Union, Any

# from landmark_data import ALL_LANDMARKS, get_all_landmark_prompts
# from landmark_activities import LANDMARK_ACTIVITIES

class LandmarkDataManager:
    """
    專門處理地標數據的載入、管理和查詢功能，包括地標信息、提示詞和活動建議
    """

    def __init__(self):
        """
        initialize landmark related
        """
        self.logger = logging.getLogger(__name__)
        self.landmark_data = {}
        self.landmark_prompts = []
        self.landmark_id_to_index = {}
        self.is_enabled = False

        self._load_landmark_data()

    def _load_landmark_data(self):
        """
        載入地標數據和相關資訊
        """
        try:
            self.landmark_data = ALL_LANDMARKS
            self.landmark_prompts = get_all_landmark_prompts()
            self.logger.info(f"Loaded {len(self.landmark_prompts)} landmark prompts for classification")

            # 創建地標ID到索引的映射，可快速查找
            self.landmark_id_to_index = {landmark_id: i for i, landmark_id in enumerate(ALL_LANDMARKS.keys())}

            self.is_enabled = True
            self.logger.info(f"Successfully loaded landmark data with {len(self.landmark_data)} landmarks")

        except ImportError:
            self.logger.warning("landmark_data.py not found. Landmark classification will be limited")
            self.landmark_data = {}
            self.landmark_prompts = []
            self.landmark_id_to_index = {}
            self.is_enabled = False
        except Exception as e:
            self.logger.error(f"Error loading landmark data: {e}")
            self.logger.error(traceback.format_exc())
            self.landmark_data = {}
            self.landmark_prompts = []
            self.landmark_id_to_index = {}
            self.is_enabled = False

    def get_landmark_prompts(self) -> List[str]:
        """
        獲取所有地標提示詞

        Returns:
            List[str]: 地標提示詞列表
        """
        return self.landmark_prompts

    def get_landmark_by_id(self, landmark_id: str) -> Dict[str, Any]:
        """
        根據地標ID獲取地標信息

        Args:
            landmark_id: Landmark ID

        Returns:
            Dict[str, Any]: 地標詳細信息
        """
        return self.landmark_data.get(landmark_id, {})

    def get_landmark_by_index(self, index: int) -> Tuple[str, Dict[str, Any]]:
        """
        根據索引獲取地標信息

        Args:
            index: 地標在列表中的索引

        Returns:
            Tuple[str, Dict[str, Any]]: (地標ID, 地標info)
        """
        try:
            landmark_ids = list(self.landmark_data.keys())
            if 0 <= index < len(landmark_ids):
                landmark_id = landmark_ids[index]
                return landmark_id, self.landmark_data[landmark_id]
            else:
                self.logger.warning(f"Index {index} out of range for landmark data")
                return None, {}
        except Exception as e:
            self.logger.error(f"Error getting landmark by index {index}: {e}")
            self.logger.error(traceback.format_exc())
            return None, {}

    def get_landmark_index(self, landmark_id: str) -> Optional[int]:
        """
        獲取地標ID對應的index

        Args:
            landmark_id: 地標ID

        Returns:
            Optional[int]: 索引，如果不存在則返回None
        """
        return self.landmark_id_to_index.get(landmark_id)

    def determine_landmark_type(self, landmark_id: str) -> str:
        """
        自動判斷地標類型，基於地標數據和命名

        Args:
            landmark_id: 地標ID

        Returns:
            str: 地標類型，用於調整閾值
        """
        if not landmark_id:
            return "building"  # 預設類型

        try:
            # 獲取地標詳細數據
            landmark_info = self.landmark_data.get(landmark_id, {})

            # 獲取地標相關文本
            landmark_id_lower = landmark_id.lower()
            landmark_name = landmark_info.get("name", "").lower()
            landmark_location = landmark_info.get("location", "").lower()
            landmark_aliases = [alias.lower() for alias in landmark_info.get("aliases", [])]

            # 合併所有文本數據用於特徵判斷
            combined_text = " ".join([landmark_id_lower, landmark_name] + landmark_aliases)

            # 地標類型的特色特徵
            type_features = {
                "skyscraper": ["skyscraper", "tall", "tower", "高樓", "摩天", "大厦", "タワー"],
                "tower": ["tower", "bell", "clock", "塔", "鐘樓", "タワー", "campanile"],
                "monument": ["monument", "memorial", "statue", "紀念", "雕像", "像", "memorial"],
                "natural": ["mountain", "lake", "canyon", "falls", "beach", "山", "湖", "峽谷", "瀑布", "海灘"],
                "temple": ["temple", "shrine", "寺", "神社", "廟"],
                "palace": ["palace", "castle", "宮", "城", "皇宮", "宫殿"],
                "distinctive": ["unique", "leaning", "slanted", "傾斜", "斜", "獨特", "傾く"]
            }

            # 檢查是否位於亞洲地區
            asian_regions = ["china", "japan", "korea", "taiwan", "singapore", "vietnam", "thailand",
                            "hong kong", "中國", "日本", "韓國", "台灣", "新加坡", "越南", "泰國", "香港"]
            is_asian = any(region in landmark_location for region in asian_regions)

            # 判斷地標類型
            best_type = None
            max_matches = 0

            for type_name, features in type_features.items():
                # 計算特徵詞匹配數量
                matches = sum(1 for feature in features if feature in combined_text)
                if matches > max_matches:
                    max_matches = matches
                    best_type = type_name

            # 處理亞洲地區特例
            if is_asian and best_type == "tower":
                best_type = "skyscraper"  # 亞洲地區的塔型建築閾值較低

            # 特例處理：檢測傾斜建築
            if any(term in combined_text for term in ["leaning", "slanted", "tilt", "inclined", "斜", "傾斜"]):
                return "distinctive"  # 傾斜建築需要特殊處理

            return best_type if best_type and max_matches > 0 else "building"  # 預設為一般建築

        except Exception as e:
            self.logger.error(f"Error determining landmark type for {landmark_id}: {e}")
            self.logger.error(traceback.format_exc())
            return "building"

    def extract_landmark_specific_info(self, landmark_id: str) -> Dict[str, Any]:
        """
        提取特定地標的詳細信息，包括特色模板和活動建議

        Args:
            landmark_id: 地標ID

        Returns:
            Dict[str, Any]: 地標特定信息
        """
        if not landmark_id or landmark_id == "unknown":
            return {"has_specific_activities": False}

        specific_info = {"has_specific_activities": False}

        try:
            # 從 landmark_data 中提取基本信息
            landmark_data_source = self.landmark_data.get(landmark_id)

            # 處理地標基本數據
            if landmark_data_source:
                # 提取正確的地標名稱
                if "name" in landmark_data_source:
                    specific_info["landmark_name"] = landmark_data_source["name"]

                # 提取所有可用的 prompts 作為特色模板
                if "prompts" in landmark_data_source:
                    specific_info["feature_templates"] = landmark_data_source["prompts"][:5]
                    specific_info["primary_template"] = landmark_data_source["prompts"][0]

                # 提取別名info
                if "aliases" in landmark_data_source:
                    specific_info["aliases"] = landmark_data_source["aliases"]

                # 提取位置信息
                if "location" in landmark_data_source:
                    specific_info["location"] = landmark_data_source["location"]

                # 提取其他相關信息
                for key in ["year_built", "architectural_style", "significance", "description"]:
                    if key in landmark_data_source:
                        specific_info[key] = landmark_data_source[key]

            # 嘗試從 LANDMARK_ACTIVITIES 中提取活動建議
            try:
                if landmark_id in LANDMARK_ACTIVITIES:
                    activities = LANDMARK_ACTIVITIES[landmark_id]
                    specific_info["landmark_specific_activities"] = activities
                    specific_info["has_specific_activities"] = True
                    self.logger.info(f"Found {len(activities)} specific activities for landmark {landmark_id}")
                else:
                    self.logger.info(f"No specific activities found for landmark {landmark_id} in LANDMARK_ACTIVITIES")
                    specific_info["has_specific_activities"] = False
            except ImportError:
                self.logger.warning("Could not import LANDMARK_ACTIVITIES from landmark_activities")
                specific_info["has_specific_activities"] = False
            except Exception as e:
                self.logger.error(f"Error loading landmark activities for {landmark_id}: {e}")
                self.logger.error(traceback.format_exc())
                specific_info["has_specific_activities"] = False

        except Exception as e:
            self.logger.error(f"Error extracting landmark specific info for {landmark_id}: {e}")
            self.logger.error(traceback.format_exc())

        return specific_info

    def get_landmark_count(self) -> int:
        """
        獲取地標總數

        Returns:
            int: 地標數量
        """
        return len(self.landmark_data)

    def is_landmark_enabled(self) -> bool:
        """
        檢查地標功能是否啟用

        Returns:
            bool: 地標功能狀態
        """
        return self.is_enabled

    def get_all_landmark_ids(self) -> List[str]:
        """
        獲取所有地標ID列表

        Returns:
            List[str]: 地標ID列表
        """
        return list(self.landmark_data.keys())

    def validate_landmark_id(self, landmark_id: str) -> bool:
        """
        驗證地標ID是否有效

        Args:
            landmark_id: 要驗證的地標ID

        Returns:
            bool: ID是否有效
        """
        return landmark_id in self.landmark_data

Writing landmark_data_manager.py


In [ ]:
# %%writefile image_analyzer.py

import numpy as np
import logging
import traceback
from typing import List, Dict, Tuple, Optional, Union, Any
from PIL import Image

class ImageAnalyzer:
    """
    專注於圖像分析和預處理，包括多尺度金字塔分析、視角分析、建築特徵識別和圖像增強等功能
    """

    def __init__(self):
        """
        初始化圖像分析器
        """
        self.logger = logging.getLogger(__name__)

    def get_image_hash(self, image: Union[Image.Image, np.ndarray]) -> int:
        """
        為圖像生成簡單的 hash 值用於快取

        Args:
            image: PIL Image 或 numpy 數組

        Returns:
            int: 圖像的 hash 值
        """
        try:
            if isinstance(image, np.ndarray):
                # 對於 numpy 數組，降採樣並計算簡單 hash
                small_img = image[::10, ::10] if image.ndim == 3 else image
                return hash(small_img.tobytes())
            else:
                # 對於 PIL 圖像，調整大小後轉換為 bytes
                small_img = image.resize((32, 32))
                return hash(small_img.tobytes())
        except Exception as e:
            self.logger.error(f"Error generating image hash: {e}")
            self.logger.error(traceback.format_exc())
            return 0

    def enhance_features(self, image: Union[Image.Image, np.ndarray]) -> Image.Image:
        """
        增強圖像特徵以改善地標檢測

        Args:
            image: 輸入圖像

        Returns:
            PIL.Image: 增強後的圖像
        """
        try:
            # ensure PIL format
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 轉換為numpy進行處理
            img_array = np.array(image)

            # 跳過灰度圖像的處理
            if len(img_array.shape) < 3:
                return image

            # 應用自適應對比度增強
            try:
                from skimage import color, exposure

                # 轉換到LAB色彩空間
                if img_array.shape[2] == 4:  # 處理RGBA
                    img_array = img_array[:,:,:3]

                lab = color.rgb2lab(img_array[:,:,:3] / 255.0)
                l_channel = lab[:,:,0]

                # 增強L通道的對比度
                p2, p98 = np.percentile(l_channel, (2, 98))
                l_channel_enhanced = exposure.rescale_intensity(l_channel, in_range=(p2, p98))

                # 替換L通道並轉換回RGB
                lab[:,:,0] = l_channel_enhanced
                enhanced_img = color.lab2rgb(lab) * 255.0
                enhanced_img = enhanced_img.astype(np.uint8)

                return Image.fromarray(enhanced_img)

            except ImportError:
                self.logger.warning("skimage not available for feature enhancement")
                return image

        except Exception as e:
            self.logger.error(f"Error in feature enhancement: {e}")
            self.logger.error(traceback.format_exc())
            return image

    def analyze_viewpoint(self, image: Union[Image.Image, np.ndarray],
                         clip_model_manager) -> Dict[str, Any]:
        """
        分析圖像視角以調整檢測參數

        Args:
            image: 輸入圖像
            clip_model_manager: CLIP模型管理器實例

        Returns:
            Dict: 視角分析結果
        """
        try:
            viewpoint_prompts = {
                "aerial_view": "an aerial view from above looking down",
                "street_level": "a street level view looking up at a tall structure",
                "eye_level": "an eye-level horizontal view of a landmark",
                "distant": "a distant view of a landmark on the horizon",
                "close_up": "a close-up detailed view of architectural features",
                "interior": "an interior view inside a structure",
                "angled_view": "an angled view of a structure",
                "low_angle": "a low angle view looking up at a building"
            }

            # 計算相似度分數
            viewpoint_scores = self.calculate_similarity_scores(image, viewpoint_prompts, clip_model_manager)

            # 找到主要視角
            dominant_viewpoint = max(viewpoint_scores.items(), key=lambda x: x[1])

            return {
                "viewpoint_scores": viewpoint_scores,
                "dominant_viewpoint": dominant_viewpoint[0],
                "confidence": dominant_viewpoint[1]
            }

        except Exception as e:
            self.logger.error(f"Error in viewpoint analysis: {e}")
            self.logger.error(traceback.format_exc())
            return {
                "viewpoint_scores": {},
                "dominant_viewpoint": "eye_level",
                "confidence": 0.0
            }

    def calculate_similarity_scores(self, image: Union[Image.Image, np.ndarray],
                                  prompts: Dict[str, str],
                                  clip_model_manager) -> Dict[str, float]:
        """
        計算圖像與一組特定提示之間的相似度分數

        Args:
            image: 輸入圖像
            prompts: 提示詞字典 {名稱: 提示文本}
            clip_model_manager: CLIP模型管理器實例

        Returns:
            Dict[str, float]: 每個提示的相似度分數
        """
        try:
            # ensure PIL format
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # preprocess image
            image_input = clip_model_manager.preprocess_image(image)

            # get image features
            image_features = clip_model_manager.encode_image(image_input)

            # 計算與每個提示的similarity
            scores = {}
            prompt_texts = list(prompts.values())
            prompt_features = clip_model_manager.encode_single_text(prompt_texts)

            # 計算相似度
            similarity = clip_model_manager.calculate_similarity(image_features, prompt_features)

            # result
            for i, (name, _) in enumerate(prompts.items()):
                scores[name] = float(similarity[0][i])

            return scores

        except Exception as e:
            self.logger.error(f"Error calculating similarity scores: {e}")
            self.logger.error(traceback.format_exc())
            return {}

    def analyze_architectural_features(self, image: Union[Image.Image, np.ndarray],
                                     clip_model_manager) -> Dict[str, Any]:
        """
        分析圖像中結構的建築特徵，不硬編碼特定地標

        Args:
            image: 輸入圖像
            clip_model_manager: CLIP模型管理器實例

        Returns:
            Dict: 建築特徵分析結果
        """
        try:
            # 定義通用建築特徵提示，適用於所有類型的地標
            architecture_prompts = {
                "tall_structure": "a tall vertical structure standing alone",
                "tiered_building": "a building with multiple stacked tiers or segments",
                "historical_structure": "a building with historical architectural elements",
                "modern_design": "a modern structure with contemporary architectural design",
                "segmented_exterior": "a structure with visible segmented or sectioned exterior",
                "viewing_platform": "a tall structure with observation area at the top",
                "time_display": "a structure with timepiece features",
                "glass_facade": "a building with prominent glass exterior surfaces",
                "memorial_structure": "a monument or memorial structure",
                "ancient_construction": "ancient constructed elements or archaeological features",
                "natural_landmark": "a natural geographic formation or landmark",
                "slanted_design": "a structure with non-vertical or leaning profile"
            }

            # 計算與通用建築模式的相似度分數
            context_scores = self.calculate_similarity_scores(image, architecture_prompts, clip_model_manager)

            # 確定最相關的建築特徵
            top_features = sorted(context_scores.items(), key=lambda x: x[1], reverse=True)[:3]

            # 計算特徵置信度
            context_confidence = sum(score for _, score in top_features) / 3

            # 根據頂級特徵確定主要建築類別
            architectural_categories = {
                "tower": ["tall_structure", "viewing_platform", "time_display"],
                "skyscraper": ["tall_structure", "modern_design", "glass_facade"],
                "historical": ["historical_structure", "ancient_construction", "memorial_structure"],
                "natural": ["natural_landmark"],
                "distinctive": ["tiered_building", "segmented_exterior", "slanted_design"]
            }

            # 根據頂級特徵為每個類別評分
            category_scores = {}
            for category, features in architectural_categories.items():
                category_score = 0
                for feature, score in context_scores.items():
                    if feature in features:
                        category_score += score
                category_scores[category] = category_score

            primary_category = max(category_scores.items(), key=lambda x: x[1])[0]

            return {
                "architectural_features": top_features,
                "context_confidence": context_confidence,
                "primary_category": primary_category,
                "category_scores": category_scores
            }

        except Exception as e:
            self.logger.error(f"Error in architectural feature analysis: {e}")
            self.logger.error(traceback.format_exc())
            return {
                "architectural_features": [],
                "context_confidence": 0.0,
                "primary_category": "building",
                "category_scores": {}
            }

    def perform_pyramid_analysis(self, image: Union[Image.Image, np.ndarray],
                               clip_model_manager, landmark_data_manager,
                               levels: int = 4, base_threshold: float = 0.25,
                               aspect_ratios: List[float] = [1.0, 0.75, 1.5]) -> Dict[str, Any]:
        """
        對圖像執行多尺度金字塔分析以改善地標檢測

        Args:
            image: 輸入圖像
            clip_model_manager: CLIP模型管理器實例
            landmark_data_manager: 地標數據管理器實例
            levels: 金字塔層級數
            base_threshold: 基礎置信度閾值
            aspect_ratios: 不同縱橫比列表

        Returns:
            Dict: 金字塔分析結果
        """
        try:
            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            width, height = image.size
            pyramid_results = []

            # 獲取預計算的地標文本特徵
            landmark_prompts = landmark_data_manager.get_landmark_prompts()
            if not landmark_prompts:
                return {
                    "is_landmark": False,
                    "results": [],
                    "best_result": None
                }

            landmark_text_features = clip_model_manager.encode_text_batch(landmark_prompts)

            # 對每個縮放和縱橫比組合進行處理
            for level in range(levels):
                # 計算縮放因子
                scale_factor = 1.0 - (level * 0.2)

                for aspect_ratio in aspect_ratios:
                    # 計算新尺寸，保持面積近似不變
                    if aspect_ratio != 1.0:
                        # 保持面積近似不變的情況下調整縱橫比
                        new_width = int(width * scale_factor * (1/aspect_ratio)**0.5)
                        new_height = int(height * scale_factor * aspect_ratio**0.5)
                    else:
                        new_width = int(width * scale_factor)
                        new_height = int(height * scale_factor)

                    # 調整圖像大小
                    scaled_image = image.resize((new_width, new_height), Image.LANCZOS)

                    # 預處理圖像
                    image_input = clip_model_manager.preprocess_image(scaled_image)

                    # 獲取圖像特徵
                    image_features = clip_model_manager.encode_image(image_input)

                    # 計算相似度
                    similarity = clip_model_manager.calculate_similarity(image_features, landmark_text_features)

                    # 找到最佳匹配
                    best_idx = similarity[0].argmax().item()
                    best_score = similarity[0][best_idx]

                    if best_score >= base_threshold:
                        landmark_id, landmark_info = landmark_data_manager.get_landmark_by_index(best_idx)
                        if landmark_id:
                            pyramid_results.append({
                                "landmark_id": landmark_id,
                                "landmark_name": landmark_info.get("name", "Unknown"),
                                "confidence": float(best_score),
                                "scale_factor": scale_factor,
                                "aspect_ratio": aspect_ratio,
                                "location": landmark_info.get("location", "Unknown Location")
                            })

            # 按置信度排序
            pyramid_results.sort(key=lambda x: x["confidence"], reverse=True)

            return {
                "is_landmark": len(pyramid_results) > 0,
                "results": pyramid_results,
                "best_result": pyramid_results[0] if pyramid_results else None
            }

        except Exception as e:
            self.logger.error(f"Error in pyramid analysis: {e}")
            self.logger.error(traceback.format_exc())
            return {
                "is_landmark": False,
                "results": [],
                "best_result": None
            }

Writing image_analyzer.py


In [ ]:
# %%writefile confidence_manager.py

import logging
import traceback
from typing import Dict, Any, Optional

class ConfidenceManager:
    """
    專門管理信心度相關邏輯，包括動態閾值調整、信心度乘數管理和地標類型特定的閾值處理
    """

    def __init__(self):
        """
        初始化置信度管理器
        """
        self.logger = logging.getLogger(__name__)

        # 初始化批處理參數
        self.batch_size = 16  # 默認批處理大小

        # 置信度閾值乘數配置
        self.confidence_threshold_multipliers = {
            "close_up": 0.9,     # 近景標準閾值
            "partial": 0.6,      # 部分可見降低閾值要求
            "distant": 0.5,      # 遠景更低閾值要求
            "full_image": 0.7    # 整張圖像需要更高閾值
        }

        # 地標類型閾值配置
        self.landmark_type_thresholds = {
            "tower": 0.5,         # 塔型建築需要更高閾值
            "skyscraper": 0.4,    # 摩天大樓使用較低閾值
            "building": 0.55,     # 一般的建築物閾值略微降低
            "monument": 0.5,      # 紀念碑閾值
            "natural": 0.6        # 自然景觀可以使用較低閾值
        }

    def set_batch_size(self, batch_size: int):
        """
        設置批處理大小

        Args:
            batch_size: 新的批處理大小
        """
        self.batch_size = max(1, batch_size)
        self.logger.info(f"Batch size set to {self.batch_size}")

    def adjust_confidence_threshold(self, detection_type: str, multiplier: float):
        """
        調整特定檢測類型的信心度的threshold

        Args:
            detection_type: 檢測類型 ('close_up', 'partial', 'distant', 'full_image')
            multiplier: 置信度閾值乘數
        """
        if detection_type in self.confidence_threshold_multipliers:
            self.confidence_threshold_multipliers[detection_type] = max(0.1, min(1.5, multiplier))
            self.logger.info(f"Adjusted confidence threshold multiplier for {detection_type} to {multiplier}")
        else:
            self.logger.warning(f"Unknown detection type: {detection_type}")

    def get_detection_type_multiplier(self, detection_type: str) -> float:
        """
        獲取檢測類型的置信度乘數

        Args:
            detection_type: 檢測類型

        Returns:
            float: 置信度乘數
        """
        return self.confidence_threshold_multipliers.get(detection_type, 1.0)

    def get_landmark_type_threshold(self, landmark_type: str) -> float:
        """
        獲取地標類型的閾值

        Args:
            landmark_type: 地標類型

        Returns:
            float: 地標類型閾值
        """
        return self.landmark_type_thresholds.get(landmark_type, 0.5)

    def calculate_adjusted_threshold(self, base_threshold: float, detection_type: str) -> float:
        """
        根據檢測類型計算調整後的閾值

        Args:
            base_threshold: 基礎閾值
            detection_type: 檢測type

        Returns:
            float: 調整後的閾值
        """
        try:
            base_multiplier = self.get_detection_type_multiplier(detection_type)
            adjusted_threshold = base_threshold * base_multiplier
            return adjusted_threshold
        except Exception as e:
            self.logger.error(f"Error calculating adjusted threshold: {e}")
            self.logger.error(traceback.format_exc())
            return base_threshold

    def calculate_final_threshold(self, base_threshold: float, detection_type: str,
                                landmark_type: str) -> float:
        """
        計算最終閾值，結合檢測類型和地標類型

        Args:
            base_threshold: 基礎閾值
            detection_type: 檢測type
            landmark_type: 地標type

        Returns:
            float: 最終閾值
        """
        try:
            # 根據檢測類型調整
            adjusted_threshold = self.calculate_adjusted_threshold(base_threshold, detection_type)

            # 根據地標類型進一步調整
            if landmark_type == "distinctive":
                # 特殊建築的閾值降低25%
                type_multiplier = 0.75
            else:
                # 使用已有的類型閾值
                type_multiplier = self.get_landmark_type_threshold(landmark_type) / 0.5

            final_threshold = adjusted_threshold * type_multiplier
            return final_threshold

        except Exception as e:
            self.logger.error(f"Error calculating final threshold: {e}")
            self.logger.error(traceback.format_exc())
            return base_threshold

    def evaluate_confidence(self, confidence: float, threshold: float) -> bool:
        """
        評估置信度是否達到閾值

        Args:
            confidence: 信心度score
            threshold: 閾值

        Returns:
            bool: 是否達到閾值
        """
        return confidence >= threshold

    def apply_architectural_boost(self, confidence: float, architectural_analysis: Dict[str, Any],
                                landmark_id: str) -> float:
        """
        根據建築特徵分析調整信心度

        Args:
            confidence: 原始置信度
            architectural_analysis: 建築特徵分析結果
            landmark_id: 地標ID

        Returns:
            float: 調整後的信心度
        """
        try:
            confidence_boost = 0
            landmark_id_lower = landmark_id.lower()

            top_features = architectural_analysis.get("architectural_features", [])
            primary_category = architectural_analysis.get("primary_category", "")

            # 使用主要建築類別來調整置信度，使用通用條件而非特定地標名稱
            if primary_category == "tower" and any(term in landmark_id_lower for term in ["tower", "spire", "needle"]):
                confidence_boost += 0.05
            elif primary_category == "skyscraper" and any(term in landmark_id_lower for term in ["building", "skyscraper", "tall"]):
                confidence_boost += 0.05
            elif primary_category == "historical" and any(term in landmark_id_lower for term in ["monument", "castle", "palace", "temple"]):
                confidence_boost += 0.05
            elif primary_category == "distinctive" and any(term in landmark_id_lower for term in ["unusual", "unique", "special", "famous"]):
                confidence_boost += 0.05

            # 根據特定特徵進一步微調，使用通用特徵描述而非特定地標
            for feature, score in top_features:
                if feature == "time_display" and "clock" in landmark_id_lower:
                    confidence_boost += 0.03
                elif feature == "segmented_exterior" and "segmented" in landmark_id_lower:
                    confidence_boost += 0.03
                elif feature == "slanted_design" and "leaning" in landmark_id_lower:
                    confidence_boost += 0.03

            # 應用信心度調整
            if confidence_boost > 0:
                adjusted_confidence = confidence + confidence_boost
                self.logger.info(f"Boosted confidence by {confidence_boost:.2f} based on architectural features ({primary_category})")
                return adjusted_confidence

            return confidence

        except Exception as e:
            self.logger.error(f"Error applying architectural boost: {e}")
            self.logger.error(traceback.format_exc())
            return confidence

    def determine_detection_type_from_region(self, region_width: int, region_height: int,
                                           image_width: int, image_height: int) -> str:
        """
        根據區域大小自動判斷檢測類型

        Args:
            region_width: 區域寬度
            region_height: 區域高度
            image_width: 圖像寬度
            image_height: 圖像高度

        Returns:
            str: 檢測類型
        """
        try:
            region_area_ratio = (region_width * region_height) / (image_width * image_height)

            if region_area_ratio > 0.5:
                return "close_up"
            elif region_area_ratio > 0.2:
                return "partial"
            else:
                return "distant"

        except Exception as e:
            self.logger.error(f"Error determining detection type from region: {e}")
            self.logger.error(traceback.format_exc())
            return "partial"

    def adjust_detection_type_by_viewpoint(self, detection_type: str, dominant_viewpoint: str) -> str:
        """
        根據視角調整檢測類型

        Args:
            detection_type: 原始檢測類型
            dominant_viewpoint: 主要視角

        Returns:
            str: 調整後的檢測類型
        """
        try:
            if dominant_viewpoint == "close_up" and detection_type != "close_up":
                return "close_up"
            elif dominant_viewpoint == "distant" and detection_type != "distant":
                return "distant"
            elif dominant_viewpoint == "angled_view":
                return "partial"  # 角度視圖可能是部分可見
            else:
                return detection_type

        except Exception as e:
            self.logger.error(f"Error adjusting detection type by viewpoint: {e}")
            self.logger.error(traceback.format_exc())
            return detection_type

    def get_batch_size(self) -> int:
        """
        獲取當前批處理大小

        Returns:
            int: 批處理大小
        """
        return self.batch_size

    def get_all_threshold_multipliers(self) -> Dict[str, float]:
        """
        獲取所有置信度閾值乘數

        Returns:
            Dict[str, float]: 閾值乘數字典
        """
        return self.confidence_threshold_multipliers.copy()

    def get_all_landmark_type_thresholds(self) -> Dict[str, float]:
        """
        獲取所有地標類型閾值

        Returns:
            Dict[str, float]: 地標類型閾值字典
        """
        return self.landmark_type_thresholds.copy()

Overwriting confidence_manager.py


In [ ]:
# %%writefile result_cache_manager.py

import logging
import traceback
from typing import Dict, Any, Tuple, Optional, Union
from PIL import Image
import numpy as np

class ResultCacheManager:
    """
    專門處理結果快取和性能優化，包括快取策略管理、快取大小控制和快取命中率優化
    """

    def __init__(self, cache_max_size: int = 100):
        """
        初始化結果快取管理器

        Args:
            cache_max_size: 最大快取項目數
        """
        self.logger = logging.getLogger(__name__)

        # 初始化結果快取
        self.results_cache = {}  # 使用圖像hash作為鍵
        self.cache_max_size = cache_max_size  # 最大快取項目數

    def generate_cache_key(self, image_hash: int, additional_params: Tuple) -> Tuple:
        """
        生成快取鍵

        Args:
            image_hash
            additional_params: 附加參數元組

        Returns:
            Tuple: 快取鍵
        """
        try:
            return (image_hash, additional_params)
        except Exception as e:
            self.logger.error(f"Error generating cache key: {e}")
            self.logger.error(traceback.format_exc())
            return (0, additional_params)

    def get_region_cache_key(self, image_hash: int, box: Tuple[float, ...],
                           detection_type: str) -> Tuple:
        """
        生成區域分析的快取鍵

        Args:
            image_hash
            box: 邊界框
            detection_type: 檢測類型

        Returns:
            Tuple: 區域快取鍵
        """
        try:
            return self.generate_cache_key(image_hash, (tuple(box), detection_type))
        except Exception as e:
            self.logger.error(f"Error generating region cache key: {e}")
            self.logger.error(traceback.format_exc())
            return (0, (tuple(box), detection_type))

    def get_image_cache_key(self, image_hash: int, analysis_type: str,
                          detailed_analysis: bool = False) -> Tuple:
        """
        生成整張圖像分析的快取鍵

        Args:
            image_hash: 圖像哈希值
            analysis_type: 分析類型
            detailed_analysis: 是否詳細分析

        Returns:
            Tuple: 圖像快取鍵
        """
        try:
            return self.generate_cache_key(image_hash, (analysis_type, detailed_analysis))
        except Exception as e:
            self.logger.error(f"Error generating image cache key: {e}")
            self.logger.error(traceback.format_exc())
            return (0, (analysis_type, detailed_analysis))

    def get_cached_result(self, cache_key: Tuple) -> Optional[Dict[str, Any]]:
        """
        獲取快取結果

        Args:
            cache_key: 快取鍵

        Returns:
            Optional[Dict[str, Any]]: 快取結果，如果不存在則返回None
        """
        try:
            return self.results_cache.get(cache_key)
        except Exception as e:
            self.logger.error(f"Error getting cached result: {e}")
            self.logger.error(traceback.format_exc())
            return None

    def set_cached_result(self, cache_key: Tuple, result: Dict[str, Any]):
        """
        設置快取結果

        Args:
            cache_key: 快取鍵
            result: 要快取的結果
        """
        try:
            self.results_cache[cache_key] = result
            self.manage_cache_size()
        except Exception as e:
            self.logger.error(f"Error setting cached result: {e}")
            self.logger.error(traceback.format_exc())

    def manage_cache_size(self):
        """
        管理結果快取大小
        """
        try:
            if len(self.results_cache) > self.cache_max_size:
                oldest_key = next(iter(self.results_cache))
                del self.results_cache[oldest_key]
        except Exception as e:
            self.logger.error(f"Error managing cache size: {e}")
            self.logger.error(traceback.format_exc())

    def clear_cache(self):
        """
        清空快取
        """
        try:
            self.results_cache.clear()
            self.logger.info("Cache cleared successfully")
        except Exception as e:
            self.logger.error(f"Error clearing cache: {e}")
            self.logger.error(traceback.format_exc())

    def get_cache_stats(self) -> Dict[str, Any]:
        """
        獲取快取統計信息

        Returns:
            Dict[str, Any]: 快取統計信息
        """
        try:
            return {
                "cache_size": len(self.results_cache),
                "max_cache_size": self.cache_max_size,
                "cache_usage_ratio": len(self.results_cache) / self.cache_max_size if self.cache_max_size > 0 else 0
            }
        except Exception as e:
            self.logger.error(f"Error getting cache stats: {e}")
            self.logger.error(traceback.format_exc())
            return {
                "cache_size": 0,
                "max_cache_size": self.cache_max_size,
                "cache_usage_ratio": 0
            }

    def set_max_cache_size(self, max_size: int):
        """
        設置最大快取大小

        Args:
            max_size: 新的最大快取大小
        """
        try:
            self.cache_max_size = max(1, max_size)
            self.manage_cache_size()
            self.logger.info(f"Max cache size set to {self.cache_max_size}")
        except Exception as e:
            self.logger.error(f"Error setting max cache size: {e}")
            self.logger.error(traceback.format_exc())

    def remove_cached_result(self, cache_key: Tuple) -> bool:
        """
        移除特定的快取結果

        Args:
            cache_key: 快取鍵

        Returns:
            bool: 是否成功移除
        """
        try:
            if cache_key in self.results_cache:
                del self.results_cache[cache_key]
                return True
            return False
        except Exception as e:
            self.logger.error(f"Error removing cached result: {e}")
            self.logger.error(traceback.format_exc())
            return False

    def is_cache_enabled(self) -> bool:
        """
        檢查快取是否啟用

        Returns:
            bool: 快取啟用狀態
        """
        return self.cache_max_size > 0

    def get_cache_keys(self) -> list:
        """
        獲取所有快取鍵

        Returns:
            list: 快取鍵列表
        """
        try:
            return list(self.results_cache.keys())
        except Exception as e:
            self.logger.error(f"Error getting cache keys: {e}")
            self.logger.error(traceback.format_exc())
            return []

    def has_cached_result(self, cache_key: Tuple) -> bool:
        """
        檢查是否存在快取結果

        Args:
            cache_key: 快取鍵

        Returns:
            bool: 是否存在快取結果
        """
        try:
            return cache_key in self.results_cache
        except Exception as e:
            self.logger.error(f"Error checking cached result: {e}")
            self.logger.error(traceback.format_exc())
            return False

Writing result_cache_manager.py


In [ ]:
# %%writefile clip_zero_shot_classifier.py

import torch
import clip
from PIL import Image
import numpy as np
import logging
import traceback
from typing import List, Dict, Tuple, Optional, Union, Any

# from clip_model_manager import CLIPModelManager
# from landmark_data_manager import LandmarkDataManager
# from image_analyzer import ImageAnalyzer
# from confidence_manager import ConfidenceManager
# from result_cache_manager import ResultCacheManager

class CLIPZeroShotClassifier:
    """
    使用CLIP模型進行zero shot，專注於辨識世界知名地標。
    作為YOLO的補充，處理YOLO無法辨識到的地標。

    這是一個總窗口class，協調各個組件的工作以提供統一的對外接口。
    """

    def __init__(self, model_name: str = "ViT-B/16", device: str = None):
        """
        初始化CLIP零樣本分類器

        Args:
            model_name: CLIP模型名稱，默認為"ViT-B/16"
            device: 運行設備，None則自動選擇
        """
        self.logger = logging.getLogger(__name__)

        # 初始化各個組件
        self.clip_model_manager = CLIPModelManager(model_name, device)
        self.landmark_data_manager = LandmarkDataManager()
        self.image_analyzer = ImageAnalyzer()
        self.confidence_manager = ConfidenceManager()
        self.cache_manager = ResultCacheManager()

        # 預計算地標文本特徵
        self.landmark_text_features = None
        self._precompute_landmark_features()

        self.logger.info(f"Initializing CLIP Zero-Shot Landmark Classifier ({model_name}) on {self.clip_model_manager.get_device()}")

    def _precompute_landmark_features(self):
        """
        預計算地標文本特徵，提高批處理效率
        """
        try:
            if self.landmark_data_manager.is_landmark_enabled():
                landmark_prompts = self.landmark_data_manager.get_landmark_prompts()
                if landmark_prompts:
                    self.landmark_text_features = self.clip_model_manager.encode_text_batch(landmark_prompts)
                    self.logger.info(f"Precomputed text features for {len(landmark_prompts)} landmark prompts")
                else:
                    self.logger.warning("No landmark prompts available for precomputation")
            else:
                self.logger.warning("Landmark data not enabled, skipping feature precomputation")
        except Exception as e:
            self.logger.error(f"Error precomputing landmark features: {e}")
            self.logger.error(traceback.format_exc())

    def set_batch_size(self, batch_size: int):
        """
        設置批處理大小

        Args:
            batch_size: 新的批處理大小
        """
        self.confidence_manager.set_batch_size(batch_size)

    def adjust_confidence_threshold(self, detection_type: str, multiplier: float):
        """
        調整特定檢測類型的置信度閾值乘數

        Args
            detection_type: 檢測類型 ('close_up', 'partial', 'distant', 'full_image')
            multiplier: 置信度閾值乘數
        """
        self.confidence_manager.adjust_confidence_threshold(detection_type, multiplier)

    def classify_image_region(self,
                            image: Union[Image.Image, np.ndarray],
                            box: List[float],
                            threshold: float = 0.25,
                            detection_type: str = "close_up") -> Dict[str, Any]:
        """
        對圖像的特定區域進行地標分類，具有增強的多尺度和部分識別能力

        Args:
            image: 原始圖像 (PIL Image 或 numpy數組)
            box: 邊界框 [x1, y1, x2, y2]
            threshold: 基礎分類置信度閾值
            detection_type: 檢測類型，影響置信度調整

        Returns:
            Dict: 地標分類結果
        """
        try:
            if not self.landmark_data_manager.is_landmark_enabled():
                return {"is_landmark": False, "confidence": 0.0}

            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 生成圖像區域的hash用於快取
            image_hash = self.image_analyzer.get_image_hash(image)
            region_key = self.cache_manager.get_region_cache_key(image_hash, tuple(box), detection_type)

            # 檢查快取
            cached_result = self.cache_manager.get_cached_result(region_key)
            if cached_result is not None:
                return cached_result

            # 裁剪區域
            x1, y1, x2, y2 = map(int, box)
            cropped_image = image.crop((x1, y1, x2, y2))
            enhanced_image = self.image_analyzer.enhance_features(cropped_image)

            # 分析視角信息
            viewpoint_info = self.image_analyzer.analyze_viewpoint(enhanced_image, self.clip_model_manager)
            dominant_viewpoint = viewpoint_info["dominant_viewpoint"]

            # 計算區域信息
            region_width = x2 - x1
            region_height = y2 - y1
            image_width, image_height = image.size

            # 根據區域大小判斷可能的檢測類型
            if detection_type == "auto":
                detection_type = self.confidence_manager.determine_detection_type_from_region(
                    region_width, region_height, image_width, image_height
                )

            # 根據視角調整檢測類型
            detection_type = self.confidence_manager.adjust_detection_type_by_viewpoint(detection_type, dominant_viewpoint)

            # 調整置信度閾值
            adjusted_threshold = self.confidence_manager.calculate_adjusted_threshold(threshold, detection_type)

            # 準備多尺度和縱橫比分析
            scales = [1.0]
            if detection_type in ["partial", "distant"]:
                scales = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

            if dominant_viewpoint in ["angled_view", "low_angle"]:
                scales = [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]

            aspect_ratios = [1.0, 0.8, 1.2]
            if dominant_viewpoint in ["angled_view", "unique_feature"]:
                aspect_ratios = [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.5]

            best_result = {
                "landmark_id": None,
                "landmark_name": None,
                "confidence": 0.0,
                "is_landmark": False
            }

            # 多尺度和縱橫比分析
            for scale in scales:
                for aspect_ratio in aspect_ratios:
                    try:
                        # 縮放裁剪區域
                        current_width, current_height = cropped_image.size

                        if aspect_ratio != 1.0:
                            new_width = int(current_width * scale * (1/aspect_ratio)**0.5)
                            new_height = int(current_height * scale * aspect_ratio**0.5)
                        else:
                            new_width = int(current_width * scale)
                            new_height = int(current_height * scale)

                        new_width = max(1, new_width)
                        new_height = max(1, new_height)

                        scaled_image = cropped_image.resize((new_width, new_height), Image.LANCZOS)

                        # 預處理並獲取特徵
                        image_input = self.clip_model_manager.preprocess_image(scaled_image)
                        image_features = self.clip_model_manager.encode_image(image_input)

                        # 計算相似度
                        similarity = self.clip_model_manager.calculate_similarity(image_features, self.landmark_text_features)

                        # 找到最佳匹配
                        best_idx = similarity[0].argmax().item()
                        best_score = similarity[0][best_idx]

                        # 如果當前尺度結果更好，則更新
                        if best_score > best_result["confidence"]:
                            landmark_id, landmark_info = self.landmark_data_manager.get_landmark_by_index(best_idx)

                            if landmark_id:
                                # 先從 LandmarkDataManager 拿 location
                                loc = landmark_info.get("location", "")
                                # 如果 loc 為空，就從全域 ALL_LANDMARKS 補上
                                if not loc and landmark_id in ALL_LANDMARKS:
                                    loc = ALL_LANDMARKS[landmark_id].get("location", "")
                                best_result = {
                                    "landmark_id": landmark_id,
                                    "landmark_name": landmark_info.get("name", "Unknown"),
                                    "location": loc or "Unknown Location",
                                    "confidence": float(best_score),
                                    "is_landmark": best_score >= adjusted_threshold,
                                    "scale_used": scale,
                                    "aspect_ratio_used": aspect_ratio,
                                    "viewpoint": dominant_viewpoint
                                }

                                # 添加額外可用信息
                                for key in ["year_built", "architectural_style", "significance"]:
                                    if key in landmark_info:
                                        best_result[key] = landmark_info[key]

                    except Exception as e:
                        self.logger.error(f"Error in scale analysis: {e}")
                        continue

            # 應用地標類型閾值調整
            if best_result["landmark_id"]:
                landmark_type = self.landmark_data_manager.determine_landmark_type(best_result["landmark_id"])
                final_threshold = self.confidence_manager.calculate_final_threshold(adjusted_threshold, detection_type, landmark_type)

                best_result["is_landmark"] = self.confidence_manager.evaluate_confidence(best_result["confidence"], final_threshold)
                best_result["landmark_type"] = landmark_type
                best_result["threshold_applied"] = final_threshold

            # 快取結果
            self.cache_manager.set_cached_result(region_key, best_result)

            return best_result

        except Exception as e:
            self.logger.error(f"Error in classify_image_region: {e}")
            self.logger.error(traceback.format_exc())
            return {"is_landmark": False, "confidence": 0.0}


    def classify_batch_regions(self,
                              image: Union[Image.Image, np.ndarray],
                              boxes: List[List[float]],
                              threshold: float = 0.28) -> List[Dict[str, Any]]:
        """
        批量處理多個圖像區域，提高效率

        Args:
            image: 原始圖像
            boxes: 邊界框列表
            threshold: 置信度閾值

        Returns:
            List[Dict]: 分類結果列表
        """
        try:
            if not self.landmark_data_manager.is_landmark_enabled() or self.landmark_text_features is None:
                return [{"is_landmark": False, "confidence": 0.0} for _ in boxes]

            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            if not boxes:
                return []

            # 批量處理所有區域
            batch_features = self.clip_model_manager.batch_process_regions(image, boxes)

            # 計算相似度
            similarity = self.clip_model_manager.calculate_similarity(batch_features, self.landmark_text_features)

            # 處理每個區域的結果
            results = []
            for i, sim in enumerate(similarity):
                best_idx = sim.argmax().item()
                best_score = sim[best_idx]

                if best_score >= threshold:
                    landmark_id, landmark_info = self.landmark_data_manager.get_landmark_by_index(best_idx)

                    if landmark_id:
                        # 如果landmark_info["location"] 為空，則從 ALL_LANDMARKS 補
                        loc = landmark_info.get("location", "")
                        if not loc and landmark_id in ALL_LANDMARKS:
                            loc = ALL_LANDMARKS[landmark_id].get("location", "")
                        results.append({
                            "landmark_id": landmark_id,
                            "landmark_name": landmark_info.get("name", "Unknown"),
                            "location": loc or "Unknown Location",
                            "confidence": float(best_score),
                            "is_landmark": True,
                            "box": boxes[i]
                        })
                    else:
                        results.append({
                            "landmark_id": None,
                            "landmark_name": None,
                            "confidence": float(best_score),
                            "is_landmark": False,
                            "box": boxes[i]
                        })
                else:
                    results.append({
                        "landmark_id": None,
                        "landmark_name": None,
                        "confidence": float(best_score),
                        "is_landmark": False,
                        "box": boxes[i]
                    })

            return results

        except Exception as e:
            self.logger.error(f"Error in classify_batch_regions: {e}")
            self.logger.error(traceback.format_exc())
            return [{"is_landmark": False, "confidence": 0.0} for _ in boxes]

    def search_entire_image(self,
                           image: Union[Image.Image, np.ndarray],
                           threshold: float = 0.35,
                           detailed_analysis: bool = False) -> Dict[str, Any]:
        """
        檢查整張圖像是否包含地標，具有增強的分析能力

        Args:
            image: 原始圖像
            threshold: 置信度閾值
            detailed_analysis: 是否進行詳細分析，包括多區域檢測

        Returns:
            Dict: 地標分類結果
        """
        try:
            if not self.landmark_data_manager.is_landmark_enabled() or self.landmark_text_features is None:
                return {"is_landmark": False, "confidence": 0.0}

            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 檢查cache
            image_hash = self.image_analyzer.get_image_hash(image)
            image_key = self.cache_manager.get_image_cache_key(image_hash, "entire_image", detailed_analysis)

            cached_result = self.cache_manager.get_cached_result(image_key)
            if cached_result is not None:
                return cached_result

            # 調整閾值
            adjusted_threshold = self.confidence_manager.calculate_adjusted_threshold(threshold, "full_image")

            # 預處理並獲取特徵
            image_input = self.clip_model_manager.preprocess_image(image)
            image_features = self.clip_model_manager.encode_image(image_input)

            # calculate相似度
            similarity = self.clip_model_manager.calculate_similarity(image_features, self.landmark_text_features)

            # 找到最佳匹配
            best_idx = similarity[0].argmax().item()
            best_score = similarity[0][best_idx]

            # 獲取top3地標
            top_indices = similarity[0].argsort()[-3:][::-1]
            top_landmarks = []

            for idx in top_indices:
                score = similarity[0][idx]
                landmark_id, landmark_info = self.landmark_data_manager.get_landmark_by_index(idx)

                if landmark_id:
                    # 補 location
                    loc_top = landmark_info.get("location", "")
                    if not loc_top and landmark_id in ALL_LANDMARKS:
                        loc_top = ALL_LANDMARKS[landmark_id].get("location", "")
                    landmark_result = {
                        "landmark_id": landmark_id,
                        "landmark_name": landmark_info.get("name", "Unknown"),
                        "location": loc_top or "Unknown Location",
                        "confidence": float(score)
                    }

                    # 加額外可用信息
                    for key in ["year_built", "architectural_style", "significance"]:
                        if key in landmark_info:
                            landmark_result[key] = landmark_info[key]

                    top_landmarks.append(landmark_result)

            # main result
            result = {}
            if best_score >= adjusted_threshold:
                landmark_id, landmark_info = self.landmark_data_manager.get_landmark_by_index(best_idx)

                if landmark_id:
                    # 應用地標類型特定閾值
                    landmark_type = self.landmark_data_manager.determine_landmark_type(landmark_id)
                    final_threshold = self.confidence_manager.calculate_final_threshold(adjusted_threshold, "full_image", landmark_type)

                    if self.confidence_manager.evaluate_confidence(best_score, final_threshold):
                        # 補 location
                        loc_main = landmark_info.get("location", "")
                        if not loc_main and landmark_id in ALL_LANDMARKS:
                            loc_main = ALL_LANDMARKS[landmark_id].get("location", "")
                        result = {
                            "landmark_id": landmark_id,
                            "landmark_name": landmark_info.get("name", "Unknown"),
                            "location": loc_main or "Unknown Location",
                            "confidence": float(best_score),
                            "is_landmark": True,
                            "landmark_type": landmark_type,
                            "top_landmarks": top_landmarks
                        }

                        # 添加額外可用信息
                        for key in ["year_built", "architectural_style", "significance"]:
                            if key in landmark_info:
                                result[key] = landmark_info[key]
                    else:
                        result = {
                            "landmark_id": None,
                            "landmark_name": None,
                            "confidence": float(best_score),
                            "is_landmark": False,
                            "top_landmarks": top_landmarks
                        }
            else:
                result = {
                    "landmark_id": None,
                    "landmark_name": None,
                    "confidence": float(best_score),
                    "is_landmark": False,
                    "top_landmarks": top_landmarks
                }

            # 詳細分析
            if detailed_analysis and result.get("is_landmark", False):
                width, height = image.size
                regions = [
                    [width * 0.25, height * 0.25, width * 0.75, height * 0.75],
                    [0, 0, width * 0.5, height],
                    [width * 0.5, 0, width, height],
                    [0, 0, width, height * 0.5],
                    [0, height * 0.5, width, height]
                ]

                region_results = []
                for i, box in enumerate(regions):
                    region_result = self.classify_image_region(
                        image,
                        box,
                        threshold=threshold * 0.9,
                        detection_type="partial"
                    )
                    if region_result["is_landmark"]:
                        region_result["region_name"] = ["center", "left", "right", "top", "bottom"][i]
                        region_results.append(region_result)

                if region_results:
                    result["region_analyses"] = region_results

            # 快取結果
            self.cache_manager.set_cached_result(image_key, result)

            return result

        except Exception as e:
            self.logger.error(f"Error in search_entire_image: {e}")
            self.logger.error(traceback.format_exc())
            return {"is_landmark": False, "confidence": 0.0}


    def intelligent_landmark_search(self,
                                  image: Union[Image.Image, np.ndarray],
                                  yolo_boxes: Optional[List[List[float]]] = None,
                                  base_threshold: float = 0.25) -> Dict[str, Any]:
        """
        對圖像進行地標搜索，綜合整張圖像分析和區域分析

        Args:
            image: 原始圖像
            yolo_boxes: YOLO檢測到的邊界框 (可選)
            base_threshold: 基礎置信度閾值

        Returns:
            Dict: 包含所有檢測結果的綜合分析
        """
        try:
            if not self.landmark_data_manager.is_landmark_enabled():
                return {
                    "full_image_analysis": {},
                    "is_landmark_scene": False,
                    "detected_landmarks": []
                }

            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 調整閾值
            actual_threshold = base_threshold * 0.85 if yolo_boxes is None or len(yolo_boxes) == 0 else base_threshold

            # 首先對整張圖像進行分析
            full_image_result = self.search_entire_image(
                image,
                threshold=actual_threshold,
                detailed_analysis=True
            )

            # 如果沒有YOLO框且全圖分析未發現地標，進行金字塔分析
            if (yolo_boxes is None or len(yolo_boxes) == 0) and (not full_image_result or not full_image_result.get("is_landmark", False)):
                self.logger.info("No YOLO boxes provided, attempting multi-scale pyramid analysis")
                pyramid_results = self.image_analyzer.perform_pyramid_analysis(
                    image,
                    self.clip_model_manager,
                    self.landmark_data_manager,
                    levels=4,
                    base_threshold=actual_threshold,
                    aspect_ratios=[1.0, 0.75, 1.5, 0.5, 2.0]
                )

                if pyramid_results and pyramid_results.get("is_landmark", False) and pyramid_results.get("best_result", {}).get("confidence", 0) > actual_threshold:
                    if not full_image_result or not full_image_result.get("is_landmark", False):
                        full_image_result = {
                            "is_landmark": True,
                            "landmark_id": pyramid_results["best_result"]["landmark_id"],
                            "landmark_name": pyramid_results["best_result"]["landmark_name"],
                            "confidence": pyramid_results["best_result"]["confidence"],
                            "location": pyramid_results["best_result"].get("location", "Unknown Location")
                        }
                        self.logger.info(f"Pyramid analysis detected landmark: {pyramid_results['best_result']['landmark_name']} with confidence {pyramid_results['best_result']['confidence']:.3f}")

            # 初始化結果dict
            result = {
                "full_image_analysis": full_image_result if full_image_result else {},
                "is_landmark_scene": False,
                "detected_landmarks": []
            }

            # 處理上下文感知比較
            if full_image_result and "top_landmarks" in full_image_result and len(full_image_result["top_landmarks"]) >= 2:
                top_landmarks = full_image_result["top_landmarks"]

                if len(top_landmarks) >= 2 and abs(top_landmarks[0]["confidence"] - top_landmarks[1]["confidence"]) < 0.1:
                    architectural_analysis = self.image_analyzer.analyze_architectural_features(image, self.clip_model_manager)

                    for i, landmark in enumerate(top_landmarks[:2]):
                        if i >= len(top_landmarks):
                            continue

                        adjusted_confidence = self.confidence_manager.apply_architectural_boost(
                            landmark["confidence"],
                            architectural_analysis,
                            landmark.get("landmark_id", "")
                        )

                        if adjusted_confidence != landmark["confidence"]:
                            top_landmarks[i]["confidence"] = adjusted_confidence

                    # 重新排序
                    top_landmarks.sort(key=lambda x: x["confidence"], reverse=True)
                    full_image_result["top_landmarks"] = top_landmarks
                    if top_landmarks:
                        full_image_result["landmark_id"] = top_landmarks[0]["landmark_id"]
                        full_image_result["landmark_name"] = top_landmarks[0]["landmark_name"]
                        full_image_result["confidence"] = top_landmarks[0]["confidence"]
                        full_image_result["location"] = top_landmarks[0].get("location", "Unknown Location")

            # 處理全圖結果
            if full_image_result and full_image_result.get("is_landmark", False):
                result["is_landmark_scene"] = True
                landmark_id = full_image_result.get("landmark_id", "unknown")

                landmark_specific_info = self.landmark_data_manager.extract_landmark_specific_info(landmark_id)

                landmark_info = {
                    "landmark_id": landmark_id,
                    "landmark_name": full_image_result.get("landmark_name", "Unknown Landmark"),
                    "confidence": full_image_result.get("confidence", 0.0),
                    "location": full_image_result.get("location", "Unknown Location"),
                    "region_type": "full_image",
                    "box": [0, 0, getattr(image, 'width', 0), getattr(image, 'height', 0)]
                }

                landmark_info.update(landmark_specific_info)

                if landmark_specific_info.get("landmark_name"):
                    landmark_info["landmark_name"] = landmark_specific_info["landmark_name"]

                result["detected_landmarks"].append(landmark_info)

                if landmark_specific_info.get("has_specific_activities", False):
                    result["primary_landmark_activities"] = landmark_specific_info.get("landmark_specific_activities", [])
                    self.logger.info(f"Set primary landmark activities: {len(result['primary_landmark_activities'])} activities for {landmark_info['landmark_name']}")

            # 處理YOLO邊界框
            if yolo_boxes and len(yolo_boxes) > 0:
                for box in yolo_boxes:
                    try:
                        box_result = self.classify_image_region(
                            image,
                            box,
                            threshold=base_threshold,
                            detection_type="auto"
                        )

                        if box_result and box_result.get("is_landmark", False):
                            is_duplicate = False
                            for existing in result["detected_landmarks"]:
                                if existing.get("landmark_id") == box_result.get("landmark_id"):
                                    if box_result.get("confidence", 0) > existing.get("confidence", 0):
                                        existing.update({
                                            "confidence": box_result.get("confidence", 0),
                                            "region_type": "yolo_box",
                                            "box": box
                                        })
                                    is_duplicate = True
                                    break

                            if not is_duplicate:
                                result["detected_landmarks"].append({
                                    "landmark_id": box_result.get("landmark_id", "unknown"),
                                    "landmark_name": box_result.get("landmark_name", "Unknown Landmark"),
                                    "confidence": box_result.get("confidence", 0.0),
                                    "location": box_result.get("location", "Unknown Location"),
                                    "region_type": "yolo_box",
                                    "box": box
                                })
                    except Exception as e:
                        self.logger.error(f"Error in analyzing YOLO box: {e}")
                        continue

            # 網格搜索（如果需要）
            should_do_grid_search = (
                len(result["detected_landmarks"]) == 0 or
                max([landmark.get("confidence", 0) for landmark in result["detected_landmarks"]], default=0) < 0.5
            )

            if should_do_grid_search:
                try:
                    width, height = getattr(image, 'size', (getattr(image, 'width', 0), getattr(image, 'height', 0)))
                    if not isinstance(width, (int, float)) or width <= 0:
                        width = getattr(image, 'width', 0)
                    if not isinstance(height, (int, float)) or height <= 0:
                        height = getattr(image, 'height', 0)

                    if width > 0 and height > 0:
                        grid_boxes = []
                        for i in range(5):
                            for j in range(5):
                                grid_boxes.append([
                                    width * (j/5), height * (i/5),
                                    width * ((j+1)/5), height * ((i+1)/5)
                                ])

                        for box in grid_boxes:
                            try:
                                grid_result = self.classify_image_region(
                                    image,
                                    box,
                                    threshold=base_threshold * 0.9,
                                    detection_type="partial"
                                )

                                if grid_result and grid_result.get("is_landmark", False):
                                    is_duplicate = False
                                    for existing in result["detected_landmarks"]:
                                        if existing.get("landmark_id") == grid_result.get("landmark_id"):
                                            is_duplicate = True
                                            break

                                    if not is_duplicate:
                                        result["detected_landmarks"].append({
                                            "landmark_id": grid_result.get("landmark_id", "unknown"),
                                            "landmark_name": grid_result.get("landmark_name", "Unknown Landmark"),
                                            "confidence": grid_result.get("confidence", 0.0),
                                            "location": grid_result.get("location", "Unknown Location"),
                                            "region_type": "grid",
                                            "box": box
                                        })
                            except Exception as e:
                                self.logger.error(f"Error in analyzing grid region: {e}")
                                continue
                except Exception as e:
                    self.logger.error(f"Error in grid search: {e}")
                    self.logger.error(traceback.format_exc())

            # 按置信度排序檢測結果
            result["detected_landmarks"].sort(key=lambda x: x.get("confidence", 0), reverse=True)

            # 更新整體場景類型判斷
            if len(result["detected_landmarks"]) > 0:
                result["is_landmark_scene"] = True
                result["primary_landmark"] = result["detected_landmarks"][0]

                if full_image_result and "clip_analysis" in full_image_result:
                    result["clip_analysis_on_full_image"] = full_image_result["clip_analysis"]

            return result

        except Exception as e:
            self.logger.error(f"Error in intelligent_landmark_search: {e}")
            self.logger.error(traceback.format_exc())
            return {
                "full_image_analysis": {},
                "is_landmark_scene": False,
                "detected_landmarks": []
            }

    def enhanced_landmark_detection(self,
                                  image: Union[Image.Image, np.ndarray],
                                  threshold: float = 0.3) -> Dict[str, Any]:
        """
        使用多種分析技術進行增強地標檢測

        Args:
            image: 輸入圖像
            threshold: 基礎置信度閾值

        Returns:
            Dict: 綜合地標檢測結果
        """
        try:
            if not self.landmark_data_manager.is_landmark_enabled():
                return {"is_landmark_scene": False, "detected_landmarks": []}

            # 確保圖像是PIL格式
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    raise ValueError("Unsupported image format. Expected PIL Image or numpy array.")

            # 1: 分析視角以調整檢測參數
            viewpoint_info = self.image_analyzer.analyze_viewpoint(image, self.clip_model_manager)
            viewpoint = viewpoint_info["dominant_viewpoint"]

            # 根據視角調整閾值
            if viewpoint == "distant":
                adjusted_threshold = threshold * 0.7
            elif viewpoint == "close_up":
                adjusted_threshold = threshold * 1.1
            else:
                adjusted_threshold = threshold

            # 2: 執行多尺度金字塔分析
            pyramid_results = self.image_analyzer.perform_pyramid_analysis(
                image,
                self.clip_model_manager,
                self.landmark_data_manager,
                levels=3,
                base_threshold=adjusted_threshold
            )

            # 3: 執行基於網格的區域分析
            grid_results = []
            width, height = image.size

            # 根據視角創建自適應網格
            if viewpoint == "distant":
                grid_size = 3
            elif viewpoint == "close_up":
                grid_size = 5
            else:
                grid_size = 4

            # 生成網格區域
            for i in range(grid_size):
                for j in range(grid_size):
                    box = [
                        width * (j/grid_size),
                        height * (i/grid_size),
                        width * ((j+1)/grid_size),
                        height * ((i+1)/grid_size)
                    ]

                    region_result = self.classify_image_region(
                        image,
                        box,
                        threshold=adjusted_threshold,
                        detection_type="auto"
                    )

                    if region_result["is_landmark"]:
                        region_result["grid_position"] = (i, j)
                        grid_results.append(region_result)

            # 4: 交叉驗證並合併結果
            all_detections = []

            # 添加金字塔結果
            if pyramid_results["is_landmark"] and pyramid_results["best_result"]:
                all_detections.append({
                    "source": "pyramid",
                    "landmark_id": pyramid_results["best_result"]["landmark_id"],
                    "landmark_name": pyramid_results["best_result"]["landmark_name"],
                    "confidence": pyramid_results["best_result"]["confidence"],
                    "scale_factor": pyramid_results["best_result"].get("scale_factor", 1.0)
                })

            # 添加網格結果
            for result in grid_results:
                all_detections.append({
                    "source": "grid",
                    "landmark_id": result["landmark_id"],
                    "landmark_name": result["landmark_name"],
                    "confidence": result["confidence"],
                    "grid_position": result.get("grid_position", (0, 0))
                })

            # 搜索整張圖像
            full_image_result = self.search_entire_image(image, threshold=adjusted_threshold)
            if full_image_result and full_image_result.get("is_landmark", False):
                all_detections.append({
                    "source": "full_image",
                    "landmark_id": full_image_result["landmark_id"],
                    "landmark_name": full_image_result["landmark_name"],
                    "confidence": full_image_result["confidence"]
                })

            # 按地標ID分組並計算總體置信度
            landmark_groups = {}
            for detection in all_detections:
                landmark_id = detection["landmark_id"]
                if landmark_id not in landmark_groups:
                    landmark_groups[landmark_id] = {
                        "landmark_id": landmark_id,
                        "landmark_name": detection["landmark_name"],
                        "detections": [],
                        "sources": set()
                    }

                landmark_groups[landmark_id]["detections"].append(detection)
                landmark_groups[landmark_id]["sources"].add(detection["source"])

            # 計算每個地標的總體置信度
            for landmark_id, group in landmark_groups.items():
                detections = group["detections"]

                # 基礎置信度是任何來源的最大置信度
                max_confidence = max(d["confidence"] for d in detections)

                # 多來源檢測獎勵
                source_count = len(group["sources"])
                source_bonus = min(0.15, (source_count - 1) * 0.05)

                # 一致性獎勵
                detection_count = len(detections)
                consistency_bonus = min(0.1, (detection_count - 1) * 0.02)

                # 計算最終置信度
                aggregate_confidence = min(1.0, max_confidence + source_bonus + consistency_bonus)

                group["confidence"] = aggregate_confidence
                group["detection_count"] = detection_count
                group["source_count"] = source_count

            # 照信心度排序地標
            sorted_landmarks = sorted(
                landmark_groups.values(),
                key=lambda x: x["confidence"],
                reverse=True
            )

            return {
                "is_landmark_scene": len(sorted_landmarks) > 0,
                "detected_landmarks": sorted_landmarks,
                "viewpoint_info": viewpoint_info,
                "primary_landmark": sorted_landmarks[0] if sorted_landmarks else None
            }

        except Exception as e:
            self.logger.error(f"Error in enhanced_landmark_detection: {e}")
            self.logger.error(traceback.format_exc())
            return {"is_landmark_scene": False, "detected_landmarks": []}

Writing clip_zero_shot_classifier.py


In [ ]:
# %%writefile component_initializer.py
import os
import traceback
import logging
from typing import Dict, Optional, Any, Tuple

# from spatial_analyzer import SpatialAnalyzer
# from scene_description import SceneDescriptor
# from enhance_scene_describer import EnhancedSceneDescriber
# from clip_analyzer import CLIPAnalyzer
# from clip_zero_shot_classifier import CLIPZeroShotClassifier
# from llm_enhancer import LLMEnhancer
# from landmark_activities import LANDMARK_ACTIVITIES
# from scene_type import SCENE_TYPES
# from object_categories import OBJECT_CATEGORIES


class ComponentInitializer:
    """
    負責初始化和管理 SceneAnalyzer 的所有子組件。
    處理組件初始化失敗的情況並提供優雅的降級機制。
    """

    def __init__(self, class_names: Dict[int, str] = None, use_llm: bool = True,
                 use_clip: bool = True, enable_landmark: bool = True,
                 llm_model_path: str = None):
        """
        初始化組件管理器。

        Args:
            class_names: YOLO 類別 ID 到名稱的映射字典
            use_llm: 是否啟用 LLM 增強功能
            use_clip: 是否啟用 CLIP 分析功能
            enable_landmark: 是否啟用地標檢測功能
            llm_model_path: LLM 模型路徑（可選）
        """
        self.logger = logging.getLogger(__name__)

        # 存儲初始化參數
        self.class_names = class_names
        self.use_llm = use_llm
        self.use_clip = use_clip
        self.enable_landmark = enable_landmark
        self.llm_model_path = llm_model_path

        # 初始化組件容器
        self.components = {}
        self.data_structures = {}
        self.initialization_status = {}

        # 初始化所有組件
        self._initialize_all_components()

    def _initialize_all_components(self):
        """初始化所有必要的組件和數據結構。"""
        try:
            # 1. 首先載入數據
            self._load_data_structures()

            # 2. 初始化核心分析組件
            self._initialize_core_analyzers()

            # 3. 初始化 CLIP 相關內容
            if self.use_clip:
                self._initialize_clip_components()

            # 4. 初始化 LLM 組件
            if self.use_llm:
                self._initialize_llm_components()

            self.logger.info("All components initialized successfully")

        except Exception as e:
            self.logger.error(f"Error during component initialization: {e}")
            traceback.print_exc()
            raise

    def _load_data_structures(self):
        """載入必要的數據結構。"""
        data_loaders = {
            'LANDMARK_ACTIVITIES': self._load_landmark_activities,
            'SCENE_TYPES': self._load_scene_types,
            'OBJECT_CATEGORIES': self._load_object_categories
        }

        for data_name, loader_func in data_loaders.items():
            try:
                self.data_structures[data_name] = loader_func()
                self.initialization_status[data_name] = True
                self.logger.info(f"Loaded {data_name} successfully")
            except Exception as e:
                self.logger.warning(f"Failed to load {data_name}: {e}")
                self.data_structures[data_name] = {}
                self.initialization_status[data_name] = False

    def _load_landmark_activities(self) -> Dict:
        """載入地標活動數據。"""
        try:
            return LANDMARK_ACTIVITIES
        except ImportError as e:
            self.logger.warning(f"Could not import LANDMARK_ACTIVITIES: {e}")
            return {}

    def _load_scene_types(self) -> Dict:
        """載入場景類型數據。"""
        try:
            return SCENE_TYPES
        except ImportError as e:
            self.logger.warning(f"Could not import SCENE_TYPES: {e}")
            return {}

    def _load_object_categories(self) -> Dict:
        """載入物體類別數據。"""
        try:
            return OBJECT_CATEGORIES
        except ImportError as e:
            self.logger.warning(f"Could not import OBJECT_CATEGORIES: {e}")
            return {}

    def _initialize_core_analyzers(self):
        """初始化核心分析組件。"""
        # 初始化 SpatialAnalyzer
        try:
            self.components['spatial_analyzer'] = SpatialAnalyzer(
                class_names=self.class_names,
                object_categories=self.data_structures.get('OBJECT_CATEGORIES', {})
            )
            self.initialization_status['spatial_analyzer'] = True
            self.logger.info("Initialized SpatialAnalyzer successfully")
        except Exception as e:
            self.logger.error(f"Error initializing SpatialAnalyzer: {e}")
            traceback.print_exc()
            self.initialization_status['spatial_analyzer'] = False
            self.components['spatial_analyzer'] = None

        # 初始化 SceneDescriptor
        try:
            self.components['descriptor'] = SceneDescriptor(
                scene_types=self.data_structures.get('SCENE_TYPES', {}),
                object_categories=self.data_structures.get('OBJECT_CATEGORIES', {})
            )
            self.initialization_status['descriptor'] = True
            self.logger.info("Initialized SceneDescriptor successfully")
        except Exception as e:
            self.logger.error(f"Error initializing SceneDescriptor: {e}")
            traceback.print_exc()
            self.initialization_status['descriptor'] = False
            self.components['descriptor'] = None

        # 初始化 EnhancedSceneDescriber
        try:
            if self.components.get('spatial_analyzer'):
                self.components['scene_describer'] = EnhancedSceneDescriber(
                    scene_types=self.data_structures.get('SCENE_TYPES', {}),
                    spatial_analyzer_instance=self.components['spatial_analyzer']
                )
                self.initialization_status['scene_describer'] = True
                self.logger.info("Initialized EnhancedSceneDescriber successfully")
            else:
                self.logger.warning("Cannot initialize EnhancedSceneDescriber without SpatialAnalyzer")
                self.initialization_status['scene_describer'] = False
                self.components['scene_describer'] = None
        except Exception as e:
            self.logger.error(f"Error initializing EnhancedSceneDescriber: {e}")
            traceback.print_exc()
            self.initialization_status['scene_describer'] = False
            self.components['scene_describer'] = None

    def _initialize_clip_components(self):
        """初始化 CLIP 相關組件。"""
        # 初始化 CLIPAnalyzer
        try:
            self.components['clip_analyzer'] = CLIPAnalyzer()
            self.initialization_status['clip_analyzer'] = True
            self.logger.info("Initialized CLIPAnalyzer successfully")

            # 如果啟用地標檢測，初始化 CLIPZeroShotClassifier
            if self.enable_landmark:
                self._initialize_landmark_classifier()

        except Exception as e:
            self.logger.warning(f"Could not initialize CLIP analyzer: {e}")
            self.logger.info("Scene analysis will proceed without CLIP. Install CLIP with 'pip install clip' for enhanced scene understanding.")
            self.use_clip = False
            self.initialization_status['clip_analyzer'] = False
            self.components['clip_analyzer'] = None

    def _initialize_landmark_classifier(self):
        """初始化地標分類器。"""
        try:
            # 嘗試使用已載入的 CLIP 模型實例
            if (self.components.get('clip_analyzer') and
                hasattr(self.components['clip_analyzer'], 'get_clip_instance')):
                model, preprocess, device = self.components['clip_analyzer'].get_clip_instance()
                self.components['landmark_classifier'] = CLIPZeroShotClassifier(device=device)
                self.logger.info("Initialized landmark classifier with shared CLIP model")
            else:
                self.components['landmark_classifier'] = CLIPZeroShotClassifier()
                self.logger.info("Initialized landmark classifier with independent CLIP model")

            # 配置地標檢測器參數
            self._configure_landmark_classifier()
            self.initialization_status['landmark_classifier'] = True

        except (ImportError, Exception) as e:
            self.logger.warning(f"Could not initialize landmark classifier: {e}")
            self.initialization_status['landmark_classifier'] = False
            self.components['landmark_classifier'] = None
            # 不完全禁用地標檢測，允許運行時重新嘗試

    def _configure_landmark_classifier(self):
        """配置地標分類器的參數。"""
        if self.components.get('landmark_classifier'):
            try:
                classifier = self.components['landmark_classifier']
                classifier.set_batch_size(8)
                classifier.adjust_confidence_threshold("full_image", 0.8)
                classifier.adjust_confidence_threshold("distant", 0.65)
                self.logger.info("Landmark detection enabled with optimized settings")
            except Exception as e:
                self.logger.warning(f"Error configuring landmark classifier: {e}")

    def _initialize_llm_components(self):
        """初始化 LLM 組件。"""
        try:
            self.components['llm_enhancer'] = LLMEnhancer(model_path=self.llm_model_path)
            self.initialization_status['llm_enhancer'] = True
            self.logger.info("LLM enhancer initialized successfully")
        except Exception as e:
            self.logger.warning(f"Could not initialize LLM enhancer: {e}")
            self.logger.info("Scene analysis will proceed without LLM. Make sure required packages are installed.")
            self.use_llm = False
            self.initialization_status['llm_enhancer'] = False
            self.components['llm_enhancer'] = None

    def get_component(self, component_name: str) -> Optional[Any]:
        """
        獲取指定的組件實例。

        Args:
            component_name: 組件名稱

        Returns:
            組件實例或 None（如果未初始化成功）
        """
        return self.components.get(component_name)

    def get_data_structure(self, data_name: str) -> Dict:
        """
        獲取指定的數據結構。

        Args:
            data_name: 數據結構名稱

        Returns:
            數據結構字典
        """
        return self.data_structures.get(data_name, {})

    def is_component_available(self, component_name: str) -> bool:
        """
        檢查指定組件是否可用。

        Args:
            component_name: 組件名稱

        Returns:
            組件是否可用
        """
        return self.initialization_status.get(component_name, False)

    def get_initialization_summary(self) -> Dict[str, bool]:
        """
        獲取所有組件的初始化狀態摘要。

        Returns:
            組件名稱到初始化狀態的映射
        """
        return self.initialization_status.copy()

    def reinitialize_component(self, component_name: str) -> bool:
        """
        重新初始化指定的組件。

        Args:
            component_name: 要重新初始化的組件名稱

        Returns:
            重新初始化是否成功
        """
        try:
            if component_name == 'landmark_classifier' and self.use_clip and self.enable_landmark:
                self._initialize_landmark_classifier()
                return self.initialization_status.get('landmark_classifier', False)
            else:
                self.logger.warning(f"Reinitializing {component_name} is not supported")
                return False
        except Exception as e:
            self.logger.error(f"Error reinitializing {component_name}: {e}")
            return False

    def update_landmark_enable_status(self, enable_landmark: bool):
        """
        更新地標檢測的啟用狀態。

        Args:
            enable_landmark: 是否啟用地標檢測
        """
        self.enable_landmark = enable_landmark

        # 如果啟用地標檢測但分類器不可用，嘗試重新初始化
        if enable_landmark and not self.is_component_available('landmark_classifier'):
            if self.use_clip:
                self.reinitialize_component('landmark_classifier')

        # 更新相關組件的狀態
        for component_name in ['scene_describer', 'clip_analyzer', 'landmark_classifier']:
            component = self.get_component(component_name)
            if component and hasattr(component, 'enable_landmark'):
                component.enable_landmark = enable_landmark

Writing component_initializer.py


In [ ]:
# %%writefile scene_scoring_engine.py
import logging
import traceback
from typing import Dict, List, Tuple, Optional, Any

# from scene_type import SCENE_TYPES

class SceneScoringEngine:
    """
    負責場景評分相關的所有計算邏輯，包括基於 YOLO 檢測的場景評分、
    多種場景分數融合，以及最終場景類型的確定。
    這邊會有YOLO, CLIP, Places365混合運用的分數計算
    """

    # 日常場景，用於特殊評分
    EVERYDAY_SCENE_TYPE_KEYS = [
        "general_indoor_space", "generic_street_view",
        "desk_area_workspace", "outdoor_gathering_spot",
        "kitchen_counter_or_utility_area"
    ]

    def __init__(self, scene_types: Dict[str, Any], enable_landmark: bool = True):
        """
        初始化場景評分引擎。

        Args:
            scene_types: 場景類型定義字典
            enable_landmark: 是否啟用地標檢測功能
        """
        self.logger = logging.getLogger(__name__)
        self.scene_types = scene_types
        self.enable_landmark = enable_landmark

    def compute_scene_scores(self, detected_objects: List[Dict],
                           spatial_analysis_results: Optional[Dict] = None) -> Dict[str, float]:
        """
        基於檢測到的物體計算各場景類型的置信度分數。
        增強了對日常場景的評分能力，並考慮物體豐富度和空間聚合性。

        Args:
            detected_objects: 檢測到的物體列表，包含物體詳細資訊
            spatial_analysis_results: 空間分析器的輸出結果，特別是 'objects_by_region' 部分

        Returns:
            場景類型到置信度分數的映射字典
        """
        scene_scores = {}
        if not detected_objects:
            for scene_type_key in self.scene_types:
                scene_scores[scene_type_key] = 0.0
            return scene_scores

        # 準備檢測物體的數據
        detected_class_ids_all = [obj["class_id"] for obj in detected_objects]
        detected_classes_set_all = set(detected_class_ids_all)
        class_counts_all = {}
        for obj in detected_objects:
            class_id = obj["class_id"]
            class_counts_all[class_id] = class_counts_all.get(class_id, 0) + 1

        # 評估 scene_types 中定義的每個場景類型
        for scene_type, scene_def in self.scene_types.items():
            required_obj_ids_defined = set(scene_def.get("required_objects", []))
            optional_obj_ids_defined = set(scene_def.get("optional_objects", []))
            min_required_matches_needed = scene_def.get("minimum_required", 0)

            # 確定哪些實際檢測到的物體與此場景類型相關
            # 這些列表將存儲實際檢測到的物體字典，而不僅僅是 class_ids
            actual_required_objects_found_list = []
            for req_id in required_obj_ids_defined:
                if req_id in detected_classes_set_all:
                    # 找到此必需物體的第一個實例添加到列表中（用於後續的聚合性檢查）
                    for dobj in detected_objects:
                        if dobj['class_id'] == req_id:
                            actual_required_objects_found_list.append(dobj)
                            break

            num_required_matches_found = len(actual_required_objects_found_list)

            actual_optional_objects_found_list = []
            for opt_id in optional_obj_ids_defined:
                if opt_id in detected_classes_set_all:
                    for dobj in detected_objects:
                        if dobj['class_id'] == opt_id:
                            actual_optional_objects_found_list.append(dobj)
                            break

            num_optional_matches_found = len(actual_optional_objects_found_list)

            # 初始分數計算權重
            # 基礎分數：55% 來自必需物體，25% 來自可選物體，10% 豐富度，10% 聚合性（最大值）
            required_weight = 0.55
            optional_weight = 0.25
            richness_bonus_max = 0.10
            cohesion_bonus_max = 0.10  # _get_object_spatial_cohesion_score 的最大獎勵是 0.1

            current_scene_score = 0.0
            objects_to_check_for_cohesion = []  # 用於空間聚合性評分

            # 檢查 minimum_required 條件並計算基礎分數
            if num_required_matches_found >= min_required_matches_needed:
                if len(required_obj_ids_defined) > 0:
                    required_ratio = num_required_matches_found / len(required_obj_ids_defined)
                else:  # 沒有定義必需物體，但 min_required_matches_needed 可能為 0
                    required_ratio = 1.0 if min_required_matches_needed == 0 else 0.0

                current_scene_score = required_ratio * required_weight
                objects_to_check_for_cohesion.extend(actual_required_objects_found_list)

                # 從可選物體添加分數
                if len(optional_obj_ids_defined) > 0:
                    optional_ratio = num_optional_matches_found / len(optional_obj_ids_defined)
                    current_scene_score += optional_ratio * optional_weight
                objects_to_check_for_cohesion.extend(actual_optional_objects_found_list)

            # 日常場景的靈活處理，如果嚴格的 minimum_required（基於 'required_objects'）未滿足
            elif scene_type in self.EVERYDAY_SCENE_TYPE_KEYS:
                # 如果日常場景有許多可選項目，它仍可能是一個弱候選
                # 檢查是否存在相當比例的 'optional_objects'
                if (len(optional_obj_ids_defined) > 0 and
                    (num_optional_matches_found / len(optional_obj_ids_defined)) >= 0.25):  # 例如，至少 25% 的典型可選項目
                    # 對這些類型的基礎分數更多地基於可選物體的滿足度
                    current_scene_score = (num_optional_matches_found / len(optional_obj_ids_defined)) * (required_weight + optional_weight * 0.5)  # 給予一些基礎分數
                    objects_to_check_for_cohesion.extend(actual_optional_objects_found_list)
                else:
                    scene_scores[scene_type] = 0.0
                    continue  # 跳過此場景類型
            else:  # 對於非日常場景，如果未滿足 minimum_required，分數為 0
                scene_scores[scene_type] = 0.0
                continue

            # 物體豐富度/多樣性的獎勵
            # 考慮找到的與場景定義相關的唯一物體類別
            relevant_defined_class_ids = required_obj_ids_defined.union(optional_obj_ids_defined)
            unique_relevant_detected_classes = relevant_defined_class_ids.intersection(detected_classes_set_all)

            object_richness_score = 0.0
            if len(relevant_defined_class_ids) > 0:
                richness_ratio = len(unique_relevant_detected_classes) / len(relevant_defined_class_ids)
                object_richness_score = min(richness_bonus_max, richness_ratio * 0.15)  # 豐富度最大 10% 獎勵
            current_scene_score += object_richness_score

            # 空間聚合性的獎勵（如果提供了 spatial_analysis_results）
            spatial_cohesion_bonus = 0.0
            if spatial_analysis_results and objects_to_check_for_cohesion:
                spatial_cohesion_bonus = self._get_object_spatial_cohesion_score(
                    objects_to_check_for_cohesion,  # 傳遞實際檢測到的物體字典列表
                    spatial_analysis_results
                )
            current_scene_score += spatial_cohesion_bonus  # 此獎勵最大 0.1

            # 關鍵物體多個實例的獎勵（原始邏輯的精煉版）
            multiple_instance_bonus = 0.0
            # 對於多實例獎勵，專注於場景定義中心的物體
            key_objects_for_multi_instance_check = required_obj_ids_defined
            if scene_type in self.EVERYDAY_SCENE_TYPE_KEYS and len(optional_obj_ids_defined) > 0:
                # 對於日常場景，如果某些可選物體多次出現，也可以是關鍵的
                # 例如，"general_indoor_space" 中的多把椅子
                key_objects_for_multi_instance_check = key_objects_for_multi_instance_check.union(
                    set(list(optional_obj_ids_defined)[:max(1, len(optional_obj_ids_defined)//2)])  # 考慮前半部分的可選物體
                )

            for class_id_check in key_objects_for_multi_instance_check:
                if class_id_check in detected_classes_set_all and class_counts_all.get(class_id_check, 0) > 1:
                    multiple_instance_bonus += 0.025  # 每種類型稍微小一點的獎勵
            current_scene_score += min(0.075, multiple_instance_bonus)  # 最大 7.5% 獎勵

            # 應用 SCENE_TYPES 中定義的場景特定優先級
            if "priority" in scene_def:
                current_scene_score *= scene_def["priority"]

            scene_scores[scene_type] = min(1.0, max(0.0, current_scene_score))

        # 如果通過實例屬性 self.enable_landmark 禁用地標檢測，
        # 確保地標特定場景類型的分數被歸零。
        if not self.enable_landmark:
            landmark_scene_types = ["tourist_landmark", "natural_landmark", "historical_monument"]
            for lm_scene_type in landmark_scene_types:
                if lm_scene_type in scene_scores:
                    scene_scores[lm_scene_type] = 0.0

        return scene_scores

    def _get_object_spatial_cohesion_score(self, objects_for_scene: List[Dict],
                                         spatial_analysis_results: Optional[Dict]) -> float:
        """
        基於場景關鍵物體的空間聚合程度計算分數。
        較高的分數意味著物體在較少的區域中更加集中。
        這是一個啟發式方法，可以進一步精煉。

        Args:
            objects_for_scene: 與當前評估場景類型相關的檢測物體列表（至少包含 'class_id' 的字典）
            spatial_analysis_results: SpatialAnalyzer._analyze_regions 的輸出
                                    預期格式：{'objects_by_region': {'region_name': [{'class_id': id, ...}, ...]}}

        Returns:
            float: 聚合性分數，通常是小額獎勵（例如，0.0 到 0.1）
        """
        if (not objects_for_scene or not spatial_analysis_results or
            "objects_by_region" not in spatial_analysis_results or
            not spatial_analysis_results["objects_by_region"]):
            return 0.0

        # 獲取定義當前場景類型的關鍵物體的 class_ids 集合
        key_object_class_ids = {obj.get('class_id') for obj in objects_for_scene if obj.get('class_id') is not None}
        if not key_object_class_ids:
            return 0.0

        # 找出這些關鍵物體出現在哪些區域
        regions_containing_key_objects = set()
        # 計算找到的關鍵物體實例數量
        # 這有助於區分 1 個區域中的 1 把椅子與分佈在 5 個區域中的 5 把椅子
        total_key_object_instances_found = 0

        for region_name, objects_in_region_list in spatial_analysis_results["objects_by_region"].items():
            region_has_key_object = False
            for obj_in_region in objects_in_region_list:
                if obj_in_region.get('class_id') in key_object_class_ids:
                    region_has_key_object = True
                    total_key_object_instances_found += 1  # 計算每個實例
            if region_has_key_object:
                regions_containing_key_objects.add(region_name)

        num_distinct_key_objects_in_scene = len(key_object_class_ids)  # 關鍵物體的類型數量
        num_instances_of_key_objects_passed = len(objects_for_scene)  # 傳遞的實例數量

        if not regions_containing_key_objects or num_instances_of_key_objects_passed == 0:
            return 0.0

        # 簡單的啟發式方法：
        if (len(regions_containing_key_objects) == 1 and
            total_key_object_instances_found >= num_instances_of_key_objects_passed * 0.75):
            return 0.10  # 最強聚合性：大部分/所有關鍵物體實例在單個區域中
        elif (len(regions_containing_key_objects) <= 2 and
              total_key_object_instances_found >= num_instances_of_key_objects_passed * 0.60):
            return 0.05  # 中等聚合性：大部分/所有關鍵物體實例在最多兩個區域中
        elif (len(regions_containing_key_objects) <= 3 and
              total_key_object_instances_found >= num_instances_of_key_objects_passed * 0.50):
            return 0.02  # 較弱聚合性

        return 0.0

    def determine_scene_type(self, scene_scores: Dict[str, float]) -> Tuple[str, float]:
        """
        基於分數確定最可能的場景類型。如果偵測到地標分數夠高，則優先回傳 "tourist_landmark"。

        Args:
            scene_scores: 場景類型到置信度分數的映射字典

        Returns:
            (最佳場景類型, 置信度) 的元組
        """
        if not scene_scores:
            return "unknown", 0.0

        # 檢查地標相關分數是否達到門檻，如果是，直接回傳 "tourist_landmark"
        # 假設場景分數 dictionary 中，"tourist_landmark"、"historical_monument"、"natural_landmark" 三個 key
        # 分別代表不同類型地標。將它們加總，若總分超過 0.3，就認定為地標場景。
        landmark_score = (
            scene_scores.get("tourist_landmark", 0.0) +
            scene_scores.get("historical_monument", 0.0) +
            scene_scores.get("natural_landmark", 0.0)
        )
        if landmark_score >= 0.3:
            # 回傳地標場景類型，以及該分數總和
            return "tourist_landmark", float(landmark_score)

        # 找分數最高的那個場景
        best_scene = max(scene_scores, key=scene_scores.get)
        best_score = scene_scores[best_scene]
        return best_scene, float(best_score)

    def fuse_scene_scores(self, yolo_scene_scores: Dict[str, float],
                         clip_scene_scores: Dict[str, float],
                         num_yolo_detections: int = 0,
                         avg_yolo_confidence: float = 0.0,
                         lighting_info: Optional[Dict] = None,
                         places365_info: Optional[Dict] = None) -> Dict[str, float]:
        """
        融合來自 YOLO 物體檢測、CLIP 分析和 Places365 場景分類的場景分數。
        根據場景類型、YOLO 檢測的豐富度、照明資訊和 Places365 置信度調整權重。

        Args:
            yolo_scene_scores: 基於 YOLO 物體檢測的場景分數
            clip_scene_scores: 基於 CLIP 分析的場景分數
            num_yolo_detections: YOLO 檢測到的置信度足夠的非地標物體總數
            avg_yolo_confidence: YOLO 檢測到的非地標物體的平均置信度
            lighting_info: 可選的照明條件分析結果，預期包含 'is_indoor' (bool) 和 'confidence' (float)
            places365_info: 可選的 Places365 場景分類結果，預期包含 'mapped_scene_type'、'confidence' 和 'is_indoor'

        Returns:
            Dict: 融合了所有三個分析來源的場景分數
        """
        # 處理其中一個分數字典可能為空或所有分數實際上為零的情況
        # 提取和處理 Places365 場景分數
        places365_scene_scores_map = {}  # 修改變數名稱以避免與傳入的字典衝突
        if places365_info and places365_info.get('confidence', 0) > 0.1:
            mapped_scene_type = places365_info.get('mapped_scene_type', 'unknown')
            places365_confidence = places365_info.get('confidence', 0.0)

            if mapped_scene_type in self.scene_types.keys():
                places365_scene_scores_map[mapped_scene_type] = places365_confidence  # 使用新的字典
                self.logger.info(f"Places365 contributing: {mapped_scene_type} with confidence {places365_confidence:.3f}")

        # 檢查各個數據來源是否具有有意義的分數
        yolo_has_meaningful_scores = bool(yolo_scene_scores and any(s > 1e-5 for s in yolo_scene_scores.values()))  # 確保是布林值
        clip_has_meaningful_scores = bool(clip_scene_scores and any(s > 1e-5 for s in clip_scene_scores.values()))  # 確保是布林值
        places365_has_meaningful_scores = bool(places365_scene_scores_map and any(s > 1e-5 for s in places365_scene_scores_map.values()))

        # 計算有意義的數據來源數量
        meaningful_sources_count = sum([
            yolo_has_meaningful_scores,
            clip_has_meaningful_scores,
            places365_has_meaningful_scores
        ])

        # 處理特殊情況：無有效數據源或僅有單一數據源
        if meaningful_sources_count == 0:
            return {st: 0.0 for st in self.scene_types.keys()}
        elif meaningful_sources_count == 1:
            if yolo_has_meaningful_scores:
                return {st: yolo_scene_scores.get(st, 0.0) for st in self.scene_types.keys()}
            elif clip_has_meaningful_scores:
                return {st: clip_scene_scores.get(st, 0.0) for st in self.scene_types.keys()}
            elif places365_has_meaningful_scores:
                return {st: places365_scene_scores_map.get(st, 0.0) for st in self.scene_types.keys()}

        # 初始化融合分數結果字典
        fused_scores = {}
        all_relevant_scene_types = set(self.scene_types.keys())
        all_possible_scene_types = all_relevant_scene_types.union(
            set(yolo_scene_scores.keys()),
            set(clip_scene_scores.keys()),
            set(places365_scene_scores_map.keys())
        )

        # 基礎權重 - 調整以適應三個來源
        default_yolo_weight = 0.5
        default_clip_weight = 0.3
        default_places365_weight = 0.2

        is_lighting_indoor = None
        lighting_analysis_confidence = 0.0
        if lighting_info and isinstance(lighting_info, dict):
            is_lighting_indoor = lighting_info.get("is_indoor")
            lighting_analysis_confidence = lighting_info.get("confidence", 0.0)

        for scene_type in all_possible_scene_types:
            yolo_score = yolo_scene_scores.get(scene_type, 0.0)
            clip_score = clip_scene_scores.get(scene_type, 0.0)
            places365_score = places365_scene_scores_map.get(scene_type, 0.0)

            current_yolo_weight = default_yolo_weight
            current_clip_weight = default_clip_weight
            current_places365_weight = default_places365_weight

            scene_definition = self.scene_types.get(scene_type, {})

            # 基於場景類型性質和 YOLO 豐富度的權重調整
            if scene_type in self.EVERYDAY_SCENE_TYPE_KEYS:
                # Places365 在日常場景分類方面表現出色
                if num_yolo_detections >= 5 and avg_yolo_confidence >= 0.45:  # 豐富的 YOLO 用於日常場景
                    current_yolo_weight = 0.60
                    current_clip_weight = 0.15
                    current_places365_weight = 0.25
                elif num_yolo_detections >= 3:  # 中等 YOLO 用於日常場景
                    current_yolo_weight = 0.50
                    current_clip_weight = 0.20
                    current_places365_weight = 0.30
                else:  # 降低 YOLO 用於日常場景，更多依賴 Places365
                    current_yolo_weight = 0.35
                    current_clip_weight = 0.25
                    current_places365_weight = 0.40

            # 對於 CLIP 的全域理解或特定訓練通常更有價值的場景
            elif any(keyword in scene_type.lower() for keyword in ["asian", "cultural", "aerial", "landmark", "monument", "tourist", "natural_landmark", "historical_monument"]):
                current_yolo_weight = 0.25
                current_clip_weight = 0.65
                current_places365_weight = 0.10  # 地標場景的較低權重

            # 對於特定室內常見場景（非地標），物體檢測是關鍵，但 Places365 提供強大的場景上下文
            elif any(keyword in scene_type.lower() for keyword in
                    ["room", "kitchen", "office", "bedroom", "desk_area", "indoor_space",
                     "professional_kitchen", "cafe", "library", "gym", "retail_store",
                     "supermarket", "classroom", "conference_room", "medical_facility",
                     "educational_setting", "dining_area"]):
                current_yolo_weight = 0.55
                current_clip_weight = 0.20
                current_places365_weight = 0.25

            # 對於特定室外常見場景（非地標），物體仍然重要
            elif any(keyword in scene_type.lower() for keyword in
                    ["parking_lot", "park_area", "beach", "harbor", "playground", "sports_field", "bus_stop", "train_station", "airport"]):
                current_yolo_weight = 0.50
                current_clip_weight = 0.25
                current_places365_weight = 0.25

            # 如果為此次運行全域禁用地標檢測
            if not self.enable_landmark:
                if any(keyword in scene_type.lower() for keyword in ["landmark", "monument", "tourist"]):
                    yolo_score = 0.0  # 應該已經從 compute_scene_scores 中為 0
                    clip_score *= 0.05  # 重度懲罰
                    places365_score *= 0.8 if scene_type not in self.EVERYDAY_SCENE_TYPE_KEYS else 1.0  # 地標場景的輕微懲罰
                elif (scene_type not in self.EVERYDAY_SCENE_TYPE_KEYS and
                      not any(keyword in scene_type.lower() for keyword in ["asian", "cultural", "aerial"])):
                    # 將權重從 CLIP 重新分配給 YOLO 和 Places365
                    weight_boost = 0.05
                    current_yolo_weight = min(0.9, current_yolo_weight + weight_boost)
                    current_places365_weight = min(0.9, current_places365_weight + weight_boost)
                    current_clip_weight = max(0.1, current_clip_weight - weight_boost * 2)

            # 如果 Places365 對此特定場景類型有高置信度，則提升其權重
            if places365_score > 0.0 and places365_info:  # 這裡的 places365_score 已經是從 map 中獲取
                places365_original_confidence = places365_info.get('confidence', 0.0)  # 獲取原始的 Places365 信心度
                if places365_original_confidence > 0.7:
                    boost_factor = min(0.2, (places365_original_confidence - 0.7) * 0.4)
                    current_places365_weight += boost_factor
                    total_other_weight = current_yolo_weight + current_clip_weight
                    if total_other_weight > 0:
                        reduction_factor = boost_factor / total_other_weight
                        current_yolo_weight *= (1 - reduction_factor)
                        current_clip_weight *= (1 - reduction_factor)

            # 權重標準化處理
            total_weight = current_yolo_weight + current_clip_weight + current_places365_weight
            if total_weight > 0:  # 避免除以零
                current_yolo_weight /= total_weight
                current_clip_weight /= total_weight
                current_places365_weight /= total_weight
            else:
                current_yolo_weight = 1/3
                current_clip_weight = 1/3
                current_places365_weight = 1/3

             # 計算融合score
            fused_score = (yolo_score * current_yolo_weight) + (clip_score * current_clip_weight) + (places365_score * current_places365_weight)

            # 處理室內外判斷的衝突分析
            places365_is_indoor = None
            places365_confidence_for_indoor = 0.0
            effective_is_indoor = is_lighting_indoor
            effective_confidence = lighting_analysis_confidence

            if places365_info and isinstance(places365_info, dict):
                places365_is_indoor = places365_info.get('is_indoor')
                places365_confidence_for_indoor = places365_info.get('confidence', 0.0)

                # Places365 在置信度高時覆蓋照明分析
                if places365_confidence_for_indoor >= 0.8 and places365_is_indoor is not None:
                    effective_is_indoor = places365_is_indoor
                    effective_confidence = places365_confidence_for_indoor

                    # 只在特定場景類型首次處理時輸出調試資訊
                    if (scene_type == "intersection" or
                        (scene_type in ["urban_intersection", "street_view"] and
                         scene_type == sorted(all_possible_scene_types)[0])):
                        self.logger.debug(f"Using Places365 indoor/outdoor decision: {places365_is_indoor} (confidence: {places365_confidence_for_indoor:.3f}) over lighting analysis")

            if effective_is_indoor is not None and effective_confidence >= 0.65:
                # 基於其定義確定場景類型本質上是室內還是室外
                is_defined_as_indoor = ("indoor" in scene_definition.get("description", "").lower() or
                                       any(kw in scene_type.lower() for kw in ["room", "kitchen", "office", "indoor", "library", "cafe", "gym"]))
                is_defined_as_outdoor = ("outdoor" in scene_definition.get("description", "").lower() or
                                        any(kw in scene_type.lower() for kw in ["street", "park", "aerial", "beach", "harbor", "intersection", "crosswalk"]))

                lighting_adjustment_strength = 0.20  # 最大調整因子（例如，20%）
                # 根據分析在閾值以上的置信度來縮放調整
                adjustment_scale = (effective_confidence - 0.65) / (1.0 - 0.65)  # 從 0 到 1 縮放
                adjustment = lighting_adjustment_strength * adjustment_scale
                adjustment = min(lighting_adjustment_strength, max(0, adjustment))  # 限制調整

                if effective_is_indoor and is_defined_as_outdoor:
                    fused_score *= (1.0 - adjustment)
                elif not effective_is_indoor and is_defined_as_indoor:
                    fused_score *= (1.0 - adjustment)
                elif effective_is_indoor and is_defined_as_indoor:
                    fused_score = min(1.0, fused_score * (1.0 + adjustment * 0.5))
                elif not effective_is_indoor and is_defined_as_outdoor:
                    fused_score = min(1.0, fused_score * (1.0 + adjustment * 0.5))

            fused_scores[scene_type] = min(1.0, max(0.0, fused_score))

        return fused_scores

    def update_enable_landmark_status(self, enable_landmark: bool):
        """
        更新地標檢測的啟用狀態。

        Args:
            enable_landmark: 是否啟用地標檢測
        """
        self.enable_landmark = enable_landmark

Overwriting scene_scoring_engine.py


In [ ]:
# %%writefile landmark_processing_manager.py
import re
import logging
import traceback
import numpy as np
from typing import Dict, List, Tuple, Optional, Any
from PIL import Image

# from clip_zero_shot_classifier import CLIPZeroShotClassifier
# from landmark_activities import LANDMARK_ACTIVITIES
# from landmark_data import ALL_LANDMARKS


class LandmarkProcessingManager:
    """
    負責處理所有地標相關的檢測和處理邏輯，包括未知物體的地標識別、
    地標物體的創建和驗證，以及地標引用的清理。
    """

    def __init__(self, enable_landmark: bool = True, use_clip: bool = True):
        """
        初始化地標處理管理器。

        Args:
            enable_landmark: 是否啟用地標檢測功能
            use_clip: 是否啟用 CLIP 分析功能
        """
        self.logger = logging.getLogger(__name__)
        self.enable_landmark = enable_landmark
        self.use_clip = use_clip

        # 載入地標相關數據
        self.landmark_activities = {}
        self.all_landmarks = {}
        self._load_landmark_data()

        # 地標分類器將按需初始化
        self.landmark_classifier = None

    def _load_landmark_data(self):
        """載入地標相關的數據結構。"""
        try:
            self.landmark_activities = LANDMARK_ACTIVITIES
            self.logger.info("Loaded LANDMARK_ACTIVITIES successfully")
        except ImportError as e:
            self.logger.warning(f"Failed to load LANDMARK_ACTIVITIES: {e}")
            self.landmark_activities = {}

        try:
            self.all_landmarks = ALL_LANDMARKS
            self.logger.info("Loaded ALL_LANDMARKS successfully")
        except ImportError as e:
            self.logger.warning(f"Failed to load ALL_LANDMARKS: {e}")
            self.all_landmarks = {}

    def set_landmark_classifier(self, landmark_classifier):
        """
        設置地標分類器實例。

        Args:
            landmark_classifier: CLIPZeroShotClassifier 實例
        """
        self.landmark_classifier = landmark_classifier

    def process_unknown_objects(self, detection_result, detected_objects, clip_analyzer=None):
        """
        對 YOLO 未能識別或信心度低的物體進行地標檢測。

        Args:
            detection_result: YOLO 檢測結果
            detected_objects: 已識別的物體列表
            clip_analyzer: CLIP 分析器實例（用於按需初始化地標分類器）

        Returns:
            tuple: (更新後的物體列表, 地標物體列表)
        """
        if (not self.enable_landmark or not self.use_clip or
            not hasattr(self, 'use_landmark_detection') or not self.use_landmark_detection):
            # 未啟用地標識別時，確保返回的物體列表中不包含任何地標物體
            cleaned_objects = [obj for obj in detected_objects if not obj.get("is_landmark", False)]
            return cleaned_objects, []

        try:
            # 獲取原始圖像
            original_image = None
            if detection_result is not None and hasattr(detection_result, 'orig_img'):
                original_image = detection_result.orig_img

            # 檢查原始圖像是否存在
            if original_image is None:
                self.logger.warning("Original image not available for landmark detection")
                return detected_objects, []

            # 確保原始圖像為 PIL 格式或可轉換為 PIL 格式
            if not isinstance(original_image, Image.Image):
                if isinstance(original_image, np.ndarray):
                    try:
                        if original_image.ndim == 3 and original_image.shape[2] == 4:  # RGBA
                            original_image = original_image[:, :, :3]  # 轉換為 RGB
                        if original_image.ndim == 2:  # 灰度圖
                            original_image = Image.fromarray(original_image).convert("RGB")
                        else:  # 假設為 RGB 或 BGR
                            original_image = Image.fromarray(original_image)

                        if hasattr(original_image, 'mode') and original_image.mode == 'BGR':  # 從 OpenCV 明確將 BGR 轉換為 RGB
                            original_image = original_image.convert('RGB')
                    except Exception as e:
                        self.logger.warning(f"Error converting image for landmark detection: {e}")
                        return detected_objects, []
                else:
                    self.logger.warning(f"Cannot process image of type {type(original_image)}")
                    return detected_objects, []

            # 獲取圖像維度
            if isinstance(original_image, np.ndarray):
                h, w = original_image.shape[:2]
            elif isinstance(original_image, Image.Image):
                w, h = original_image.size
            else:
                self.logger.warning(f"Unable to determine image dimensions for type {type(original_image)}")
                return detected_objects, []

            # 收集可能含有地標的區域
            candidate_boxes = []
            low_conf_boxes = []

            # 即使沒有 YOLO 檢測到的物體，也嘗試進行更詳細的地標分析
            if len(detected_objects) == 0:
                # 創建一個包含整個圖像的框
                full_image_box = [0, 0, w, h]
                low_conf_boxes.append(full_image_box)
                candidate_boxes.append((full_image_box, "full_image"))

                # 加入網格分析以增加檢測成功率
                grid_size = 2  # 2x2 網格
                for i in range(grid_size):
                    for j in range(grid_size):
                        # 創建網格框
                        grid_box = [
                            j * w / grid_size,
                            i * h / grid_size,
                            (j + 1) * w / grid_size,
                            (i + 1) * h / grid_size
                        ]
                        low_conf_boxes.append(grid_box)
                        candidate_boxes.append((grid_box, "grid"))

                # 創建更大的中心框（覆蓋中心 70% 區域）
                center_box = [
                    w * 0.15, h * 0.15,
                    w * 0.85, h * 0.85
                ]
                low_conf_boxes.append(center_box)
                candidate_boxes.append((center_box, "center"))

                self.logger.info("No YOLO detections, attempting detailed landmark analysis with multiple regions")
            else:
                try:
                    # 獲取原始 YOLO 檢測結果中的低置信度物體
                    if (hasattr(detection_result, 'boxes') and
                        hasattr(detection_result.boxes, 'xyxy') and
                        hasattr(detection_result.boxes, 'conf') and
                        hasattr(detection_result.boxes, 'cls')):
                        all_boxes = (detection_result.boxes.xyxy.cpu().numpy()
                                   if hasattr(detection_result.boxes.xyxy, 'cpu')
                                   else detection_result.boxes.xyxy)
                        all_confs = (detection_result.boxes.conf.cpu().numpy()
                                   if hasattr(detection_result.boxes.conf, 'cpu')
                                   else detection_result.boxes.conf)
                        all_cls = (detection_result.boxes.cls.cpu().numpy()
                                 if hasattr(detection_result.boxes.cls, 'cpu')
                                 else detection_result.boxes.cls)

                        # 收集低置信度區域和可能含有地標的區域（如建築物）
                        for i, (box, conf, cls) in enumerate(zip(all_boxes, all_confs, all_cls)):
                            is_low_conf = conf < 0.4 and conf > 0.1

                            # 根據物體類別 ID 識別建築物 - 使用通用分類
                            common_building_classes = [11, 12, 13, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]  # 常見建築類別 ID
                            is_building = int(cls) in common_building_classes

                            # 計算相對面積 - 大物體
                            is_large_object = (box[2] - box[0]) * (box[3] - box[1]) > (0.1 * w * h)

                            if is_low_conf or is_building:
                                # 確保 box 是一個有效的數組或列表
                                if isinstance(box, (list, tuple, np.ndarray)) and len(box) >= 4:
                                    low_conf_boxes.append(box)
                                    if is_large_object:
                                        candidate_boxes.append((box, "building" if is_building else "low_conf"))
                except Exception as e:
                    self.logger.error(f"Error processing YOLO detections: {e}")
                    traceback.print_exc()

            # 按需初始化地標分類器
            if not self.landmark_classifier:
                if clip_analyzer and hasattr(clip_analyzer, 'get_clip_instance'):
                    try:
                        self.logger.info("Initializing landmark classifier for process_unknown_objects")
                        model, preprocess, device = clip_analyzer.get_clip_instance()
                        self.landmark_classifier = CLIPZeroShotClassifier(device=device)
                    except Exception as e:
                        self.logger.error(f"Error initializing landmark classifier: {e}")
                        return detected_objects, []
                else:
                    self.logger.warning("landmark_classifier not available and cannot be initialized")
                    return detected_objects, []

            # 使用智能地標搜索
            landmark_results = None
            try:
                # 確保有有效的框
                if not low_conf_boxes:
                    # 如果沒有低置信度框，添加全圖
                    low_conf_boxes.append([0, 0, w, h])

                landmark_results = self.landmark_classifier.intelligent_landmark_search(
                    original_image,
                    yolo_boxes=low_conf_boxes,
                    base_threshold=0.25
                )
            except Exception as e:
                self.logger.error(f"Error in intelligent_landmark_search: {e}")
                traceback.print_exc()
                return detected_objects, []

            # 處理識別結果
            landmark_objects = []

            # 如果有效的地標結果
            if landmark_results and landmark_results.get("is_landmark_scene", False):
                for landmark_info in landmark_results.get("detected_landmarks", []):
                    try:
                        # 使用 landmark_classifier 的閾值判斷
                        base_threshold = 0.25  # 基礎閾值

                        # 獲取地標類型並設定閾值
                        landmark_type = "architectural"  # 預設類型
                        type_threshold = 0.5  # 預設閾值

                        # 優先使用 landmark_classifier
                        if (hasattr(self.landmark_classifier, '_determine_landmark_type') and
                            landmark_info.get("landmark_id")):
                            landmark_type = self.landmark_classifier._determine_landmark_type(landmark_info.get("landmark_id"))
                            type_threshold = getattr(self.landmark_classifier, 'landmark_type_thresholds', {}).get(landmark_type, 0.5)
                        # 否則使用本地方法
                        elif hasattr(self, '_determine_landmark_type'):
                            landmark_type = self._determine_landmark_type(landmark_info.get("landmark_id", ""))
                            # 依據地標類型調整閾值
                            if landmark_type == "skyscraper":
                                type_threshold = 0.4
                            elif landmark_type == "natural":
                                type_threshold = 0.6
                        # 或者直接從地標 ID 推斷
                        else:
                            landmark_id = landmark_info.get("landmark_id", "").lower()
                            if any(term in landmark_id for term in ["mountain", "canyon", "waterfall", "lake", "river", "natural"]):
                                landmark_type = "natural"
                                type_threshold = 0.6
                            elif any(term in landmark_id for term in ["skyscraper", "building", "tower", "tall"]):
                                landmark_type = "skyscraper"
                                type_threshold = 0.4
                            elif any(term in landmark_id for term in ["monument", "memorial", "statue", "historical"]):
                                landmark_type = "monument"
                                type_threshold = 0.5

                        effective_threshold = base_threshold * (type_threshold / 0.5)

                        # 如果置信度足夠高
                        if landmark_info.get("confidence", 0) > effective_threshold:
                            # 獲取邊界框
                            if "box" in landmark_info:
                                box = landmark_info["box"]
                            else:
                                # 如果沒有邊界框，使用整個圖像的 90% 區域
                                margin_x, margin_y = w * 0.05, h * 0.05
                                box = [margin_x, margin_y, w - margin_x, h - margin_y]

                            # 計算中心點和其他必要信息
                            center_x = (box[0] + box[2]) / 2
                            center_y = (box[1] + box[3]) / 2
                            norm_center_x = center_x / w if w > 0 else 0.5
                            norm_center_y = center_y / h if h > 0 else 0.5

                            # 獲取區域位置（需要 spatial_analyzer 的支持）
                            region = "center"  # 預設

                            # 創建地標物體
                            landmark_obj = {
                                "class_id": (landmark_info.get("landmark_id", "")[:15]
                                           if isinstance(landmark_info.get("landmark_id", ""), str)
                                           else "-100"),  # 截斷過長的 ID
                                "class_name": landmark_info.get("landmark_name", "Unknown Landmark"),
                                "confidence": landmark_info.get("confidence", 0.0),
                                "box": box,
                                "center": (center_x, center_y),
                                "normalized_center": (norm_center_x, norm_center_y),
                                "size": (box[2] - box[0], box[3] - box[1]),
                                "normalized_size": (
                                    (box[2] - box[0]) / w if w > 0 else 0,
                                    (box[3] - box[1]) / h if h > 0 else 0
                                ),
                                "area": (box[2] - box[0]) * (box[3] - box[1]),
                                "normalized_area": (
                                    (box[2] - box[0]) * (box[3] - box[1]) / (w * h) if w * h > 0 else 0
                                ),
                                "region": region,
                                "is_landmark": True,
                                "landmark_id": landmark_info.get("landmark_id", ""),
                                "location": landmark_info.get("location", "Unknown Location")
                            }

                            # 添加額外信息
                            for key in ["year_built", "architectural_style", "significance"]:
                                if key in landmark_info:
                                    landmark_obj[key] = landmark_info[key]

                            # 添加地標類型
                            landmark_obj["landmark_type"] = landmark_type

                            # 添加到檢測物體列表
                            detected_objects.append(landmark_obj)
                            landmark_objects.append(landmark_obj)
                            self.logger.info(f"Detected landmark: {landmark_info.get('landmark_name', 'Unknown')} with confidence {landmark_info.get('confidence', 0.0):.2f}")
                    except Exception as e:
                        self.logger.error(f"Error processing landmark: {e}")
                        continue

                return detected_objects, landmark_objects

            return detected_objects, []

        except Exception as e:
            self.logger.error(f"Error in landmark detection: {e}")
            traceback.print_exc()
            return detected_objects, []

    def remove_landmark_references(self, text):
        """
        從文本中移除所有地標引用。

        Args:
            text: 輸入文本

        Returns:
            str: 清除地標引用後的文本
        """
        if not text:
            return text

        try:
            # 動態收集所有地標名稱和位置
            landmark_names = []
            locations = []

            for landmark_id, info in self.all_landmarks.items():
                # 收集地標名稱及其別名
                landmark_names.append(info["name"])
                landmark_names.extend(info.get("aliases", []))

                # 收集地理位置
                if "location" in info:
                    location = info["location"]
                    locations.append(location)

                    # 處理分離的城市和國家名稱
                    parts = location.split(",")
                    if len(parts) >= 1:
                        locations.append(parts[0].strip())
                    if len(parts) >= 2:
                        locations.append(parts[1].strip())

            # 使用正則表達式動態替換所有地標名稱
            for name in landmark_names:
                if name and len(name) > 2:  # 避免過短的名稱
                    text = re.sub(r'\b' + re.escape(name) + r'\b', "tall structure", text, flags=re.IGNORECASE)

            # 動態替換所有位置引用
            for location in locations:
                if location and len(location) > 2:
                    # 替換常見位置表述模式
                    text = re.sub(r'in ' + re.escape(location), "in the urban area", text, flags=re.IGNORECASE)
                    text = re.sub(r'of ' + re.escape(location), "of the urban area", text, flags=re.IGNORECASE)
                    text = re.sub(r'\b' + re.escape(location) + r'\b', "the urban area", text, flags=re.IGNORECASE)

        except Exception as e:
            self.logger.warning(f"Error in dynamic landmark reference removal, using generic patterns: {e}")
            # 通用地標描述模式
            landmark_patterns = [
                # 地標地點模式
                (r'an iconic structure in ([A-Z][a-zA-Z\s,]+)', r'an urban structure'),
                (r'a famous (monument|tower|landmark) in ([A-Z][a-zA-Z\s,]+)', r'an urban structure'),
                (r'(the [A-Z][a-zA-Z\s]+ Tower)', r'the tower'),
                (r'(the [A-Z][a-zA-Z\s]+ Building)', r'the building'),
                (r'(the CN Tower)', r'the tower'),
                (r'([A-Z][a-zA-Z\s]+) Tower', r'tall structure'),

                # 地標位置關係模式
                (r'(centered|built|located|positioned) around the ([A-Z][a-zA-Z\s]+? (Tower|Monument|Landmark))', r'located in this area'),

                # 地標活動模式
                (r'(sightseeing|guided tours|cultural tourism) (at|around|near) (this landmark|the [A-Z][a-zA-Z\s]+)', r'\1 in this area'),

                # 一般性地標形容模式
                (r'this (famous|iconic|historic|well-known) (landmark|monument|tower|structure)', r'this urban structure'),
                (r'landmark scene', r'urban scene'),
                (r'tourist destination', r'urban area'),
                (r'tourist attraction', r'urban area')
            ]

            for pattern, replacement in landmark_patterns:
                text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

        return text

    def get_alternative_scene_type(self, landmark_scene_type, detected_objects, scene_scores):
        """
        為地標場景類型選擇適合的替代類型。

        Args:
            landmark_scene_type: 原始地標場景類型
            detected_objects: 檢測到的物體列表
            scene_scores: 所有場景類型的分數

        Returns:
            str: 適合的替代場景類型
        """
        # 1. 嘗試從現有場景分數中找出第二高的非地標場景
        landmark_types = {"tourist_landmark", "natural_landmark", "historical_monument"}
        alternative_scores = {k: v for k, v in scene_scores.items() if k not in landmark_types and v > 0.2}

        if alternative_scores:
            # 返回分數最高的非地標場景類型
            return max(alternative_scores.items(), key=lambda x: x[1])[0]

        # 2. 基於物體組合推斷場景類型
        object_counts = {}
        for obj in detected_objects:
            class_name = obj.get("class_name", "")
            if class_name not in object_counts:
                object_counts[class_name] = 0
            object_counts[class_name] += 1

        # 根據物體組合決定場景類型
        if "car" in object_counts or "truck" in object_counts or "bus" in object_counts:
            # 有車輛，可能是街道或交叉路口
            if "traffic light" in object_counts or "stop sign" in object_counts:
                return "intersection"
            else:
                return "city_street"

        if "building" in object_counts and object_counts.get("person", 0) > 0:
            # 有建築物和人，可能是商業區
            return "commercial_district"

        if object_counts.get("person", 0) > 3:
            # 多個行人，可能是行人區
            return "pedestrian_area"

        if "bench" in object_counts or "potted plant" in object_counts:
            # 有長椅或盆栽，可能是公園區域
            return "park_area"

        # 3. 根據原始地標場景類型選擇合適的替代場景
        if landmark_scene_type == "natural_landmark":
            return "outdoor_natural_area"
        elif landmark_scene_type == "historical_monument":
            return "urban_architecture"

        # 默認回退到城市街道
        return "city_street"

    def extract_landmark_specific_activities(self, landmark_objects):
        """
        從識別的地標中提取特定活動。

        Args:
            landmark_objects: 地標物體列表

        Returns:
            List[str]: 地標特定活動列表
        """
        landmark_specific_activities = []

        # 優先收集來自識別地標的特定活動
        for lm_obj in landmark_objects:
            lm_id = lm_obj.get("landmark_id")
            if lm_id and lm_id in self.landmark_activities:
                landmark_specific_activities.extend(self.landmark_activities[lm_id])

        if landmark_specific_activities:
            landmark_names = [lm.get('landmark_name', 'unknown') for lm in landmark_objects if lm.get('is_landmark', False)]
            self.logger.info(f"Added {len(landmark_specific_activities)} landmark-specific activities for {', '.join(landmark_names)}")

        return landmark_specific_activities

    def update_enable_landmark_status(self, enable_landmark: bool):
        """
        更新地標檢測的啟用狀態。

        Args:
            enable_landmark: 是否啟用地標檢測
        """
        self.enable_landmark = enable_landmark

    def update_use_landmark_detection_status(self, use_landmark_detection: bool):
        """
        更新地標檢測使用狀態。

        Args:
            use_landmark_detection: 是否使用地標檢測
        """
        self.use_landmark_detection = use_landmark_detection

Writing landmark_processing_manager.py


In [ ]:
# %%writefile scene_analysis_coordinator.py
import logging
import traceback
import numpy as np
from typing import Dict, List, Tuple, Optional, Any
from PIL import Image

class SceneAnalysisCoordinator:
    """
    負責整個場景分析流程的協調和控制邏輯，包含主要的分析流程、
    處理無檢測結果的回退邏輯，以及多源分析結果的整合。
    """

    def __init__(self, component_initializer, scene_scoring_engine, landmark_processing_manager,
                 scene_confidence_threshold: float = 0.6):
        """
        初始化場景分析協調器。

        Args:
            component_initializer: 組件初始化器實例
            scene_scoring_engine: 場景評分引擎實例
            landmark_processing_manager: 地標處理管理器實例
            scene_confidence_threshold: 場景置信度閾值
        """
        self.logger = logging.getLogger(__name__)
        self.component_initializer = component_initializer
        self.scene_scoring_engine = scene_scoring_engine
        self.landmark_processing_manager = landmark_processing_manager
        self.scene_confidence_threshold = scene_confidence_threshold

        # 獲取必要的組件和數據
        self.spatial_analyzer = component_initializer.get_component('spatial_analyzer')
        self.descriptor = component_initializer.get_component('descriptor')
        self.scene_describer = component_initializer.get_component('scene_describer')
        self.clip_analyzer = component_initializer.get_component('clip_analyzer')
        self.llm_enhancer = component_initializer.get_component('llm_enhancer')

        self.scene_types = component_initializer.get_data_structure('SCENE_TYPES')

        # 從組件初始化器獲取功能開關狀態
        self.use_clip = component_initializer.use_clip
        self.use_llm = component_initializer.use_llm
        self.enable_landmark = component_initializer.enable_landmark

    def analyze(self, detection_result: Any, lighting_info: Optional[Dict] = None,
                class_confidence_threshold: float = 0.25, scene_confidence_threshold: float = 0.6,
                enable_landmark: bool = True, places365_info: Optional[Dict] = None) -> Dict:
        """
        分析檢測結果以確定場景類型並提供理解。

        Args:
            detection_result: 來自 YOLOv8 或類似系統的檢測結果
            lighting_info: 可選的照明條件分析結果
            class_confidence_threshold: 考慮物體的最小置信度
            scene_confidence_threshold: 確定場景的最小置信度
            enable_landmark: 是否為此次運行啟用地標檢測和識別
            places365_info: 可選的 Places365 場景分類結果

        Returns:
            包含場景分析結果的字典
        """
        current_run_enable_landmark = enable_landmark
        self.logger.info(f"DIAGNOSTIC (SceneAnalyzer.analyze): Called with current_run_enable_landmark={current_run_enable_landmark}")
        self.logger.debug(f"SceneAnalyzer received lighting_info type: {type(lighting_info)}")
        self.logger.debug(f"SceneAnalyzer lighting_info source: {lighting_info.get('source', 'unknown') if isinstance(lighting_info, dict) else 'not_dict'}")

        # 記錄 Places365 資訊
        if places365_info:
            self.logger.info(f"DIAGNOSTIC: Places365 info received - scene: {places365_info.get('scene_label', 'unknown')}, "
                           f"mapped: {places365_info.get('mapped_scene_type', 'unknown')}, "
                           f"confidence: {places365_info.get('confidence', 0.0):.3f}")

        # 同步 enable_landmark 狀態到子組件（為此次分析運行）
        self._sync_landmark_status_to_components(current_run_enable_landmark)

        # 提取和處理原始圖像
        original_image_pil, image_dims_val = self._extract_image_info(detection_result)

        # 處理無 YOLO 檢測結果的情況
        no_yolo_detections = self._check_no_yolo_detections(detection_result)

        if no_yolo_detections:
            return self._handle_no_yolo_detections(
                original_image_pil, image_dims_val, current_run_enable_landmark,
                lighting_info, places365_info
            )

        # 主處理流程（有 YOLO 檢測結果）
        return self._handle_main_analysis_flow(
            detection_result, original_image_pil, image_dims_val,
            class_confidence_threshold, scene_confidence_threshold,
            current_run_enable_landmark, lighting_info, places365_info
        )

    def _sync_landmark_status_to_components(self, current_run_enable_landmark: bool):
        """同步地標狀態到所有相關組件。"""
        # 更新場景評分引擎
        self.scene_scoring_engine.update_enable_landmark_status(current_run_enable_landmark)

        # 更新地標處理管理器
        self.landmark_processing_manager.update_enable_landmark_status(current_run_enable_landmark)

        # 更新其他組件的地標狀態
        for component_name in ['scene_describer', 'clip_analyzer', 'landmark_classifier']:
            component = self.component_initializer.get_component(component_name)
            if component and hasattr(component, 'enable_landmark'):
                component.enable_landmark = current_run_enable_landmark

        # 更新實例狀態
        self.enable_landmark = current_run_enable_landmark

    def _extract_image_info(self, detection_result) -> Tuple[Optional[Image.Image], Optional[Tuple[int, int]]]:
        """從檢測結果中提取圖像信息。"""
        original_image_pil = None
        image_dims_val = None  # 將是 (width, height)

        if (detection_result is not None and hasattr(detection_result, 'orig_img') and
            detection_result.orig_img is not None):
            if isinstance(detection_result.orig_img, np.ndarray):
                try:
                    img_array = detection_result.orig_img
                    if img_array.ndim == 3 and img_array.shape[2] == 4:  # RGBA
                        img_array = img_array[:, :, :3]  # 轉換為 RGB
                    if img_array.ndim == 2:  # 灰度
                        original_image_pil = Image.fromarray(img_array).convert("RGB")
                    else:  # 假設 RGB 或 BGR（如果源是 cv2 BGR，PIL 在 fromarray 時會處理 BGR->RGB，但明確處理更好）
                        original_image_pil = Image.fromarray(img_array)

                    if hasattr(original_image_pil, 'mode') and original_image_pil.mode == 'BGR':  # 明確將 OpenCV 的 BGR 轉換為 PIL 的 RGB
                        original_image_pil = original_image_pil.convert('RGB')

                    image_dims_val = (original_image_pil.width, original_image_pil.height)
                except Exception as e:
                    self.logger.warning(f"Error converting NumPy orig_img to PIL: {e}")
            elif hasattr(detection_result.orig_img, 'size') and callable(getattr(detection_result.orig_img, 'convert', None)):
                original_image_pil = detection_result.orig_img.copy().convert("RGB")  # 確保 RGB
                image_dims_val = original_image_pil.size
            else:
                self.logger.warning(f"detection_result.orig_img (type: {type(detection_result.orig_img)}) is not a recognized NumPy array or PIL Image.")
        else:
            self.logger.warning("detection_result.orig_img not available. Image-based analysis will be limited.")

        return original_image_pil, image_dims_val

    def _check_no_yolo_detections(self, detection_result) -> bool:
        """檢查是否沒有 YOLO 檢測結果。"""
        return (detection_result is None or
                not hasattr(detection_result, 'boxes') or
                not hasattr(detection_result.boxes, 'xyxy') or
                len(detection_result.boxes.xyxy) == 0)

    def _handle_no_yolo_detections(self, original_image_pil, image_dims_val,
                                 current_run_enable_landmark, lighting_info, places365_info) -> Dict:
        """處理無 YOLO 檢測結果的情況。"""
        tried_landmark_detection = False
        landmark_detection_result = None

        # 嘗試地標檢測
        if original_image_pil and self.use_clip and current_run_enable_landmark:
            landmark_detection_result = self._attempt_landmark_detection_no_yolo(
                original_image_pil, image_dims_val, lighting_info
            )
            tried_landmark_detection = True

            if landmark_detection_result:
                return landmark_detection_result

        # 如果地標檢測失敗或未嘗試，使用 CLIP 進行一般場景分析
        if not landmark_detection_result and self.use_clip and original_image_pil:
            clip_fallback_result = self._attempt_clip_fallback_analysis(
                original_image_pil, image_dims_val, current_run_enable_landmark, lighting_info
            )
            if clip_fallback_result:
                return clip_fallback_result

        # 最終回退邏輯
        return self._get_final_fallback_result(places365_info, lighting_info)

    def _attempt_landmark_detection_no_yolo(self, original_image_pil, image_dims_val, lighting_info) -> Optional[Dict]:
        """在無 YOLO 檢測的情況下嘗試地標檢測。"""
        try:
            # 初始化地標分類器（如果需要）
            landmark_classifier = self.component_initializer.get_component('landmark_classifier')
            if not landmark_classifier and self.clip_analyzer:
                if hasattr(self.clip_analyzer, 'get_clip_instance'):
                    try:
                        model, preprocess, device = self.clip_analyzer.get_clip_instance()
                        landmark_classifier = CLIPZeroShotClassifier(device=device)
                        self.landmark_processing_manager.set_landmark_classifier(landmark_classifier)
                        self.logger.info("Initialized landmark classifier with shared CLIP model")
                    except Exception as e:
                        self.logger.warning(f"Could not initialize landmark classifier: {e}")
                        return None

            if landmark_classifier:
                self.logger.info("Attempting landmark detection with no YOLO boxes")
                landmark_results_no_yolo = landmark_classifier.intelligent_landmark_search(
                    original_image_pil, yolo_boxes=None, base_threshold=0.2  # 略微降低閾值，提高靈敏度
                )

                # 確保在無地標場景時返回有效結果
                if landmark_results_no_yolo is None:
                    landmark_results_no_yolo = {"is_landmark_scene": False, "detected_landmarks": []}

                if (landmark_results_no_yolo and landmark_results_no_yolo.get("is_landmark_scene", False)):
                    return self._process_landmark_detection_result(
                        landmark_results_no_yolo, image_dims_val, lighting_info
                    )
        except Exception as e:
            self.logger.error(f"Error in landmark-only detection path (analyze method): {e}")
            traceback.print_exc()

        return None

    def _process_landmark_detection_result(self, landmark_results, image_dims_val, lighting_info) -> Dict:
        """處理地標檢測結果並生成最終輸出。"""
        primary_landmark = landmark_results.get("primary_landmark")

        # 放寬閾值條件，以便捕獲更多潛在地標
        if not primary_landmark or primary_landmark.get("confidence", 0) <= 0.25:
            return None

        detected_objects_from_landmarks_list = []
        w_img, h_img = image_dims_val if image_dims_val else (1, 1)

        for lm_info_item in landmark_results.get("detected_landmarks", []):
            if lm_info_item.get("confidence", 0) > 0.25:  # 降低閾值與上面保持一致
                # 安全獲取 box 值，避免索引錯誤
                box = lm_info_item.get("box", [0, 0, w_img, h_img])
                if len(box) < 4:
                    box = [0, 0, w_img, h_img]

                # 計算中心點和標準化坐標
                center_x, center_y = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2
                norm_cx = center_x / w_img if w_img > 0 else 0.5
                norm_cy = center_y / h_img if h_img > 0 else 0.5

                # 決定地標類型
                landmark_type = "architectural"  # 預設類型
                landmark_id = lm_info_item.get("landmark_id", "")

                landmark_classifier = self.component_initializer.get_component('landmark_classifier')
                if (landmark_classifier and hasattr(landmark_classifier, '_determine_landmark_type') and landmark_id):
                    try:
                        landmark_type = landmark_classifier._determine_landmark_type(landmark_id)
                    except Exception as e:
                        self.logger.error(f"Error determining landmark type: {e}")
                else:
                    # 使用簡單的基於 ID 的啟發式方法推斷類型
                    landmark_id_lower = landmark_id.lower() if isinstance(landmark_id, str) else ""
                    if "natural" in landmark_id_lower or any(term in landmark_id_lower for term in ["mountain", "waterfall", "canyon", "lake"]):
                        landmark_type = "natural"
                    elif "monument" in landmark_id_lower or "memorial" in landmark_id_lower or "historical" in landmark_id_lower:
                        landmark_type = "monument"

                # 決定區域位置
                region = "center"  # 預設值
                if self.spatial_analyzer and hasattr(self.spatial_analyzer, '_determine_region'):
                    try:
                        region = self.spatial_analyzer._determine_region(norm_cx, norm_cy)
                    except Exception as e:
                        self.logger.error(f"Error determining region: {e}")

                # 取得並補 location
                loc_lm = lm_info_item.get("location", "")
                if not loc_lm and landmark_id in ALL_LANDMARKS:
                    loc_lm = ALL_LANDMARKS[landmark_id].get("location", "")

                # 創建地標物體
                landmark_obj = {
                    "class_id": lm_info_item.get("landmark_id", f"LM_{lm_info_item.get('landmark_name','unk')}")[:15],
                    "class_name": lm_info_item.get("landmark_name", "Unknown Landmark"),
                    "confidence": lm_info_item.get("confidence", 0.0),
                    "box": box,
                    "center": (center_x, center_y),
                    "normalized_center": (norm_cx, norm_cy),
                    "size": (box[2] - box[0], box[3] - box[1]),
                    "normalized_size": (
                        (box[2] - box[0])/(w_img if w_img>0 else 1),
                        (box[3] - box[1])/(h_img if h_img>0 else 1)
                    ),
                    "area": (box[2] - box[0]) * (box[3] - box[1]),
                    "normalized_area": (
                        (box[2] - box[0]) * (box[3] - box[1])
                    ) / ((w_img*h_img) if w_img*h_img >0 else 1),
                    "is_landmark": True,
                    "landmark_id": landmark_id,
                    "location": loc_lm or "Unknown Location",
                    "region": region,
                    "year_built": lm_info_item.get("year_built", ""),
                    "architectural_style": lm_info_item.get("architectural_style", ""),
                    "significance": lm_info_item.get("significance", ""),
                    "landmark_type": landmark_type
                }
                detected_objects_from_landmarks_list.append(landmark_obj)

        if not detected_objects_from_landmarks_list:
            return None

        # 設定場景類型
        best_scene_val = "tourist_landmark"  # 預設
        if primary_landmark:
            try:
                lm_type = primary_landmark.get("landmark_type", "architectural")
                if lm_type and "natural" in lm_type.lower():
                    best_scene_val = "natural_landmark"
                elif lm_type and ("historical" in lm_type.lower() or "monument" in lm_type.lower()):
                    best_scene_val = "historical_monument"
            except Exception as e:
                self.logger.error(f"Error determining scene type from landmark type: {e}")

        # 確保場景類型有效
        if best_scene_val not in self.scene_types:
            best_scene_val = "tourist_landmark"  # 預設場景類型

        # 設定置信度
        scene_confidence = primary_landmark.get("confidence", 0.0) if primary_landmark else 0.0

        # 生成其他必要的分析結果
        region_analysis = self._generate_region_analysis(detected_objects_from_landmarks_list)

        functional_zones = self._generate_functional_zones(
            detected_objects_from_landmarks_list,
            best_scene_val
        )

        scene_description = self._generate_scene_description(
            best_scene_val, detected_objects_from_landmarks_list, scene_confidence,
            lighting_info, functional_zones, image_dims_val
        )

        enhanced_description = self._enhance_description_with_llm(
            scene_description, best_scene_val, detected_objects_from_landmarks_list,
            scene_confidence, lighting_info, functional_zones, landmark_results, image_dims_val
        )
        possible_activities = self._extract_possible_activities(detected_objects_from_landmarks_list, landmark_results)

        # 準備最終結果
        return {
            "scene_type": best_scene_val,
            "scene_name": self.scene_types.get(best_scene_val, {}).get("name", "Landmark"),
            "confidence": round(float(scene_confidence), 4),
            "description": scene_description,
            "enhanced_description": enhanced_description,
            "objects_present": detected_objects_from_landmarks_list,
            "object_count": len(detected_objects_from_landmarks_list),
            "regions": region_analysis,
            "possible_activities": possible_activities,
            "functional_zones": functional_zones,
            "detected_landmarks": [lm for lm in detected_objects_from_landmarks_list if lm.get("is_landmark", False)],
            "primary_landmark": primary_landmark,
            "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
        }


    def _attempt_clip_fallback_analysis(self, original_image_pil, image_dims_val,
                                      current_run_enable_landmark, lighting_info) -> Optional[Dict]:
        """嘗試使用 CLIP 進行一般場景分析。"""
        try:
            clip_analysis_val = None
            if self.clip_analyzer and hasattr(self.clip_analyzer, 'analyze_image'):
                try:
                    clip_analysis_val = self.clip_analyzer.analyze_image(
                        original_image_pil,
                        enable_landmark=current_run_enable_landmark
                    )
                except Exception as e:
                    self.logger.error(f"Error in CLIP analysis: {e}")

            scene_type_llm = "llm_inferred_no_yolo"
            confidence_llm = 0.0

            if clip_analysis_val and isinstance(clip_analysis_val, dict):
                top_scene = clip_analysis_val.get("top_scene")
                if top_scene and isinstance(top_scene, tuple) and len(top_scene) >= 2:
                    confidence_llm = top_scene[1]
                    if isinstance(top_scene[0], str):
                        scene_type_llm = top_scene[0]

            desc_llm = "Primary object detection did not yield results. This description is based on overall image context."

            w_llm, h_llm = image_dims_val if image_dims_val else (1, 1)
            enhanced_desc_llm = self._enhance_no_detection_description(
                desc_llm, scene_type_llm, confidence_llm, lighting_info,
                clip_analysis_val, current_run_enable_landmark, w_llm, h_llm
            )

            # 安全類型轉換
            try:
                confidence_float = float(confidence_llm)
            except (ValueError, TypeError):
                confidence_float = 0.0

            # 確保增強描述不為空
            if not enhanced_desc_llm or not isinstance(enhanced_desc_llm, str):
                enhanced_desc_llm = desc_llm

            # 返回結果
            return {
                "scene_type": scene_type_llm,
                "confidence": round(confidence_float, 4),
                "description": desc_llm,
                "enhanced_description": enhanced_desc_llm,
                "objects_present": [],
                "object_count": 0,
                "regions": {},
                "possible_activities": [],
                "safety_concerns": [],
                "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
            }
        except Exception as e:
            self.logger.error(f"Error in CLIP no-detection fallback (analyze method): {e}")
            traceback.print_exc()
            return None

    def _get_final_fallback_result(self, places365_info, lighting_info) -> Dict:
        """獲取最終的回退結果。"""
        # 檢查 Places365 是否提供有用的場景信息（即使沒有 YOLO 檢測）
        fallback_scene_type = "unknown"
        fallback_confidence = 0.0
        fallback_description = "No objects were detected in the image, and contextual analysis could not be performed or failed."

        if places365_info and places365_info.get('confidence', 0) > 0.3:
            fallback_scene_type = places365_info.get('mapped_scene_type', 'unknown')
            fallback_confidence = places365_info.get('confidence', 0.0)
            fallback_description = f"Scene appears to be {places365_info.get('scene_label', 'an unidentified location')} based on overall visual context."

        return {
            "scene_type": fallback_scene_type,
            "confidence": fallback_confidence,
            "description": fallback_description,
            "enhanced_description": "The image analysis system could not detect any recognizable objects or landmarks in this image.",
            "objects_present": [],
            "object_count": 0,
            "regions": {},
            "possible_activities": [],
            "safety_concerns": [],
            "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
        }

    def _handle_main_analysis_flow(self, detection_result, original_image_pil, image_dims_val,
                                 class_confidence_threshold, scene_confidence_threshold,
                                 current_run_enable_landmark, lighting_info, places365_info) -> Dict:
        """處理主要的分析流程（有 YOLO 檢測結果）。"""
        # 更新類別名稱映射
        if hasattr(detection_result, 'names'):
            if hasattr(self.spatial_analyzer, 'class_names'):
                self.spatial_analyzer.class_names = detection_result.names

        # 提取檢測到的物體
        detected_objects_main = self.spatial_analyzer._extract_detected_objects(
            detection_result,
            confidence_threshold=class_confidence_threshold
        )

        if not detected_objects_main:
            return {
                "scene_type": "unknown", "confidence": 0.0,
                "description": "No objects detected with sufficient confidence by the primary vision system.",
                "objects_present": [], "object_count": 0, "regions": {}, "possible_activities": [],
                "safety_concerns": [], "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
            }

        # 空間分析
        region_analysis_val = self.spatial_analyzer._analyze_regions(detected_objects_main)

        # 地標處理和整合
        landmark_objects_identified = []
        landmark_specific_activities = []
        final_landmark_info = {}

        if self.use_clip and current_run_enable_landmark:
            detected_objects_main, landmark_objects_identified = self.landmark_processing_manager.process_unknown_objects(
                detection_result, detected_objects_main, self.clip_analyzer
            )

            if landmark_objects_identified:
                landmark_specific_activities = self.landmark_processing_manager.extract_landmark_specific_activities(
                    landmark_objects_identified
                )
                final_landmark_info = {
                    "detected_landmarks": landmark_objects_identified,
                    "primary_landmark": max(landmark_objects_identified, key=lambda x: x.get("confidence", 0.0), default=None),
                    "detailed_landmarks": landmark_objects_identified
                }

        # 如果當前運行禁用地標檢測，清理地標物體
        if not current_run_enable_landmark:
            detected_objects_main = [obj for obj in detected_objects_main if not obj.get("is_landmark", False)]
            final_landmark_info = {}

        # 計算場景分數並進行融合
        yolo_scene_scores = self.scene_scoring_engine.compute_scene_scores(
            detected_objects_main, spatial_analysis_results=region_analysis_val
        )

        clip_scene_scores = {}
        clip_analysis_results = None
        if self.use_clip and original_image_pil is not None:
            clip_analysis_results, clip_scene_scores = self._perform_clip_analysis(
                original_image_pil, current_run_enable_landmark, lighting_info
            )

        # 融合場景分數
        yolo_only_objects = [obj for obj in detected_objects_main if not obj.get("is_landmark")]
        num_yolo_detections = len(yolo_only_objects)
        avg_yolo_confidence = (sum(obj.get('confidence', 0.0) for obj in yolo_only_objects) / num_yolo_detections
                              if num_yolo_detections > 0 else 0.0)

        scene_scores_fused = self.scene_scoring_engine.fuse_scene_scores(
            yolo_scene_scores, clip_scene_scores,
            num_yolo_detections=num_yolo_detections,
            avg_yolo_confidence=avg_yolo_confidence,
            lighting_info=lighting_info,
            places365_info=places365_info
        )

        # 確定最終場景類型
        final_best_scene, final_scene_confidence = self.scene_scoring_engine.determine_scene_type(scene_scores_fused)

        # 處理禁用地標檢測時的替代場景類型
        if (not current_run_enable_landmark and
            final_best_scene in ["tourist_landmark", "natural_landmark", "historical_monument"]):
            alt_scene_type = self.landmark_processing_manager.get_alternative_scene_type(
                final_best_scene, detected_objects_main, scene_scores_fused
            )
            final_best_scene = alt_scene_type
            final_scene_confidence = scene_scores_fused.get(alt_scene_type, 0.6)

        # 生成最終的描述性內容
        final_result = self._generate_final_result(
            final_best_scene, final_scene_confidence, detected_objects_main,
            landmark_specific_activities, landmark_objects_identified, final_landmark_info,
            region_analysis_val, lighting_info, scene_scores_fused, current_run_enable_landmark,
            clip_analysis_results, image_dims_val, scene_confidence_threshold
        )

        return final_result

    def _perform_clip_analysis(self, original_image_pil, current_run_enable_landmark, lighting_info) -> Tuple[Optional[Dict], Dict]:
        """執行 CLIP 分析。"""
        clip_analysis_results = None
        clip_scene_scores = {}

        try:
            clip_analysis_results = self.clip_analyzer.analyze_image(
                original_image_pil,
                enable_landmark=current_run_enable_landmark,
                exclude_categories=["landmark", "tourist", "monument", "tower", "attraction", "scenic", "historical", "famous"] if not current_run_enable_landmark else None
            )

            if isinstance(clip_analysis_results, dict):
                clip_scene_scores = clip_analysis_results.get("scene_scores", {})

                # 如果禁用地標檢測，再次過濾
                if not current_run_enable_landmark:
                    clip_scene_scores = {k: v for k, v in clip_scene_scores.items()
                                       if not any(kw in k.lower() for kw in ["landmark", "monument", "tourist"])}
                    if "cultural_analysis" in clip_analysis_results:
                        del clip_analysis_results["cultural_analysis"]
                    if ("top_scene" in clip_analysis_results and
                        any(term in clip_analysis_results.get("top_scene", ["unknown", 0.0])[0].lower()
                            for term in ["landmark", "monument", "tourist"])):
                        non_lm_cs = sorted([item for item in clip_scene_scores.items() if item[1] > 0],
                                         key=lambda x: x[1], reverse=True)
                        clip_analysis_results["top_scene"] = non_lm_cs[0] if non_lm_cs else ("unknown", 0.0)

                # 處理照明信息回退
                if (not lighting_info and "lighting_condition" in clip_analysis_results):
                    lt, lc = clip_analysis_results.get("lighting_condition", ("unknown", 0.0))
                    lighting_info = {"time_of_day": lt, "confidence": lc, "source": "CLIP_fallback"}
        except Exception as e:
            self.logger.error(f"Error in main CLIP analysis for YOLO path (analyze method): {e}")

        return clip_analysis_results, clip_scene_scores

    def _generate_final_result(self, final_best_scene, final_scene_confidence, detected_objects_main,
                             landmark_specific_activities, landmark_objects_identified, final_landmark_info,
                             region_analysis_val, lighting_info, scene_scores_fused, current_run_enable_landmark,
                             clip_analysis_results, image_dims_val, scene_confidence_threshold) -> Dict:
        """生成最終的分析結果。"""
        # 生成最終的描述性內容（活動、安全、區域）
        final_activities = []

        # 通用活動推斷
        generic_activities = []
        if self.descriptor and hasattr(self.descriptor, '_infer_possible_activities'):
            generic_activities = self.descriptor._infer_possible_activities(
                final_best_scene, detected_objects_main,
                enable_landmark=current_run_enable_landmark, scene_scores=scene_scores_fused
            )

        # 優先處理策略：使用特定地標活動，不足時才從通用活動補充
        if landmark_specific_activities:
            # 如果有特定活動，優先保留，去除與特定活動重複的通用活動
            unique_generic_activities = [act for act in generic_activities if act not in landmark_specific_activities]

            # 如果特定活動少於3個，從通用活動中補充
            if len(landmark_specific_activities) < 3:
                # 補充通用活動但總數不超過7個
                supplement_count = min(3 - len(landmark_specific_activities), len(unique_generic_activities))
                if supplement_count > 0:
                    final_activities.extend(unique_generic_activities[:supplement_count])
        else:
            # 若無特定活動，則使用所有通用活動
            final_activities.extend(generic_activities)

        # 去重並排序，但確保特定地標活動保持在前面
        final_activities_set = set(final_activities)
        final_activities = []

        # 先加入特定地標活動（按原順序）
        for activity in landmark_specific_activities:
            if activity in final_activities_set:
                final_activities.append(activity)
                final_activities_set.remove(activity)

        # 再加入通用活動（按字母排序）
        final_activities.extend(sorted(list(final_activities_set)))

        # 安全問題識別
        final_safety_concerns = []
        if self.descriptor and hasattr(self.descriptor, '_identify_safety_concerns'):
            final_safety_concerns = self.descriptor._identify_safety_concerns(detected_objects_main, final_best_scene)

        # 功能區域識別
        final_functional_zones = {}
        if self.spatial_analyzer and hasattr(self.spatial_analyzer, '_identify_functional_zones'):
            general_zones = self.spatial_analyzer._identify_functional_zones(detected_objects_main, final_best_scene)
            final_functional_zones.update(general_zones)

        # 地標相關的功能區域
        if landmark_objects_identified and self.spatial_analyzer and hasattr(self.spatial_analyzer, '_identify_landmark_zones'):
            landmark_zones = self.spatial_analyzer._identify_landmark_zones(landmark_objects_identified)
            final_functional_zones.update(landmark_zones)

        # 如果當前運行禁用地標檢測，過濾相關內容
        if not current_run_enable_landmark:
            final_functional_zones = {
                        str(k): v
                        for k, v in final_functional_zones.items()
                        if (not str(k).isdigit())
                        and (not any(kw in str(k).lower() for kw in ["landmark", "monument", "viewing", "tourist"]))
                    }


            current_activities_temp = [act for act in final_activities
                                     if not any(kw in act.lower() for kw in ["sightsee", "photograph", "tour", "histor", "landmark", "monument", "cultur"])]
            final_activities = current_activities_temp
            if not final_activities and self.descriptor and hasattr(self.descriptor, '_infer_possible_activities'):
                final_activities = self.descriptor._infer_possible_activities("generic_street_view", detected_objects_main, enable_landmark=False)

        # 創建淨化的光線資訊，避免不合理的時間描述
        lighting_info_clean = None
        if lighting_info:
            lighting_info_clean = {
                "is_indoor": lighting_info.get("is_indoor"),
                "confidence": lighting_info.get("confidence", 0.0),
                "time_of_day": lighting_info.get("time_of_day", "unknown")
            }

        # 生成場景描述
        base_scene_description = self._generate_scene_description(
            final_best_scene, detected_objects_main, final_scene_confidence,
            lighting_info_clean, final_functional_zones, image_dims_val
        )

        # 清理地標引用（如果禁用地標檢測）
        if not current_run_enable_landmark:
            base_scene_description = self.landmark_processing_manager.remove_landmark_references(base_scene_description)

        # LLM 增強
        enhanced_final_description = self._enhance_final_description(
            base_scene_description, final_best_scene, final_scene_confidence, detected_objects_main,
            final_functional_zones, final_activities, final_safety_concerns, lighting_info,
            clip_analysis_results, current_run_enable_landmark, image_dims_val, final_landmark_info
        )

        # 清理增強描述的地標引用
        if not current_run_enable_landmark:
            enhanced_final_description = self.landmark_processing_manager.remove_landmark_references(enhanced_final_description)

        # 構建最終輸出字典
        output_result = {
            "scene_type": final_best_scene if final_scene_confidence >= scene_confidence_threshold else "unknown",
            "scene_name": (self.scene_types.get(final_best_scene, {}).get("name", "Unknown Scene")
                          if final_scene_confidence >= scene_confidence_threshold else "Unknown Scene"),
            "confidence": round(float(final_scene_confidence), 4),
            "description": base_scene_description,
            "enhanced_description": enhanced_final_description,
            "objects_present": [{"class_id": obj.get("class_id", -1),
                               "class_name": obj.get("class_name", "unknown"),
                               "confidence": round(float(obj.get("confidence", 0.0)), 4)}
                              for obj in detected_objects_main],
            "object_count": len(detected_objects_main),
            "regions": region_analysis_val,
            "possible_activities": final_activities,
            "safety_concerns": final_safety_concerns,
            "functional_zones": final_functional_zones,
            "lighting_conditions": lighting_info if lighting_info else {"time_of_day": "unknown", "confidence": 0.0, "source": "default"}
        }

        # 添加替代場景
        if self.descriptor and hasattr(self.descriptor, '_get_alternative_scenes'):
            output_result["alternative_scenes"] = self.descriptor._get_alternative_scenes(
                scene_scores_fused, scene_confidence_threshold, top_k=2
            )

        # 添加地標相關信息
        if current_run_enable_landmark and final_landmark_info and final_landmark_info.get("detected_landmarks"):
            output_result.update(final_landmark_info)
            if final_best_scene in ["tourist_landmark", "natural_landmark", "historical_monument"]:
                output_result["scene_source"] = "landmark_detection"
        elif not current_run_enable_landmark:
            for key_rm in ["detected_landmarks", "primary_landmark", "detailed_landmarks", "scene_source"]:
                if key_rm in output_result:
                    del output_result[key_rm]

        # 添加 CLIP 分析結果
        if clip_analysis_results and isinstance(clip_analysis_results, dict) and "error" not in clip_analysis_results:
            top_scene_clip = clip_analysis_results.get("top_scene", ("unknown", 0.0))
            output_result["clip_analysis"] = {
                "top_scene": (top_scene_clip[0], round(float(top_scene_clip[1]), 4)),
                "cultural_analysis": clip_analysis_results.get("cultural_analysis", {}) if current_run_enable_landmark else {}
            }

        return output_result

    # 輔助方法
    def _generate_region_analysis(self, detected_objects):
        """生成區域分析結果。"""
        if self.spatial_analyzer and hasattr(self.spatial_analyzer, '_analyze_regions'):
            try:
                return self.spatial_analyzer._analyze_regions(detected_objects)
            except Exception as e:
                self.logger.error(f"Error analyzing regions: {e}")
        return {}

    def _generate_functional_zones(self, detected_objects, scene_type):
        """
        生成功能區域。
        由於原本直接呼叫 _identify_landmark_zones，導致非地標場景必定回 {}。
        這裡改為呼叫 _identify_functional_zones，並帶入 scene_type。
        """
        try:
            # 如果 spatial_analyzer 可以識別 functional zones，就調用它
            if self.spatial_analyzer and hasattr(self.spatial_analyzer, '_identify_functional_zones'):
                return self.spatial_analyzer._identify_functional_zones(detected_objects, scene_type)
        except Exception as e:
            self.logger.error(f"Error identifying functional zones: {e}")
            self.logger.error(traceback.format_exc())
        return {}


    def _generate_scene_description(self, scene_type, detected_objects, confidence,
                                  lighting_info, functional_zones, image_dims):
        """生成場景描述。"""
        if self.scene_describer and hasattr(self.scene_describer, 'generate_description'):
            try:
                for obj in detected_objects:
                    if obj.get("is_landmark"):
                        loc_obj = obj.get("location", "")
                        lm_id_obj = obj.get("landmark_id")
                        if (not loc_obj) and lm_id_obj and lm_id_obj in ALL_LANDMARKS:
                            obj["location"] = ALL_LANDMARKS[lm_id_obj].get("location", "")

                return self.scene_describer.generate_description(
                    scene_type=scene_type,
                    detected_objects=detected_objects,
                    confidence=confidence,
                    lighting_info=lighting_info,
                    functional_zones=list(functional_zones.keys()) if functional_zones else [],
                    enable_landmark=self.enable_landmark,
                    scene_scores={scene_type: confidence},
                    spatial_analysis={},
                    image_dimensions=image_dims
                )
            except Exception as e:
                self.logger.error(f"Error generating scene description: {e}")
        return f"A {scene_type} scene."

    def _enhance_description_with_llm(self, scene_description, scene_type, detected_objects,
                                    confidence, lighting_info, functional_zones, landmark_results, image_dims):
        """使用 LLM 增強描述。"""
        if not self.use_llm or not self.llm_enhancer:
            return scene_description

        try:
            prominent_objects_detail = ""
            if self.scene_describer and hasattr(self.scene_describer, 'format_object_list_for_description'):
                try:
                    prominent_objects_detail = self.scene_describer.format_object_list_for_description(
                        detected_objects[:min(1, len(detected_objects))]
                    )
                except Exception as e:
                    self.logger.error(f"Error formatting object list: {e}")

            w_img, h_img = image_dims if image_dims else (1, 1)
            scene_data_llm = {
                "original_description": scene_description,
                "scene_type": scene_type,
                "scene_name": self.scene_types.get(scene_type, {}).get("name", "Landmark"),
                "detected_objects": detected_objects,
                "object_list": "landmark",
                "confidence": confidence,
                "lighting_info": lighting_info,
                "functional_zones": functional_zones,
                "clip_analysis": landmark_results.get("clip_analysis_on_full_image", {}),
                "enable_landmark": True,
                "image_width": w_img,
                "image_height": h_img,
                "prominent_objects_detail": prominent_objects_detail
            }

            return self.llm_enhancer.enhance_description(scene_data_llm)
        except Exception as e:
            self.logger.error(f"Error enhancing description with LLM: {e}")
            traceback.print_exc()
            return scene_description

    def _enhance_no_detection_description(self, desc, scene_type, confidence, lighting_info,
                                        clip_analysis, enable_landmark, width, height):
        """增強無檢測結果的描述。"""
        if not self.use_llm or not self.llm_enhancer:
            return desc

        try:
            clip_analysis_safe = {}
            if isinstance(clip_analysis, dict):
                clip_analysis_safe = clip_analysis

            scene_data_llm = {
                "original_description": desc,
                "scene_type": scene_type,
                "scene_name": "Contextually Inferred (No Detections)",
                "detected_objects": [],
                "object_list": "general ambiance",
                "confidence": confidence,
                "lighting_info": lighting_info or {"time_of_day": "unknown", "confidence": 0.0},
                "clip_analysis": clip_analysis_safe,
                "enable_landmark": enable_landmark,
                "image_width": width,
                "image_height": height,
                "prominent_objects_detail": "the overall visual context"
            }

            if hasattr(self.llm_enhancer, 'enhance_description'):
                try:
                    enhanced = self.llm_enhancer.enhance_description(scene_data_llm)
                    if enhanced and len(enhanced.strip()) >= 20:
                        return enhanced
                except Exception as e:
                    self.logger.error(f"Error in enhance_description: {e}")

            if hasattr(self.llm_enhancer, 'handle_no_detection'):
                try:
                    return self.llm_enhancer.handle_no_detection(clip_analysis_safe)
                except Exception as e:
                    self.logger.error(f"Error in handle_no_detection: {e}")
        except Exception as e:
            self.logger.error(f"Error preparing data for LLM enhancement: {e}")
            traceback.print_exc()

        return desc

    def _extract_possible_activities(self, detected_objects, landmark_results):
        """提取可能的活動。"""
        possible_activities = ["Sightseeing"]

        # 檢查是否有主要地標活動從 CLIP 分析結果中獲取
        primary_landmark_activities = landmark_results.get("primary_landmark_activities", [])

        if primary_landmark_activities:
            self.logger.info(f"Using {len(primary_landmark_activities)} landmark-specific activities")
            possible_activities = primary_landmark_activities
        else:
            # 從檢測到的地標中提取特定活動
            landmark_specific_activities = self.landmark_processing_manager.extract_landmark_specific_activities(detected_objects)

            if landmark_specific_activities:
                possible_activities = list(set(landmark_specific_activities))  # 去重
                self.logger.info(f"Extracted {len(possible_activities)} activities from landmark data")
            else:
                # 回退到通用活動推斷
                if self.descriptor and hasattr(self.descriptor, '_infer_possible_activities'):
                    try:
                        possible_activities = self.descriptor._infer_possible_activities(
                            "tourist_landmark",
                            detected_objects,
                            enable_landmark=True,
                            scene_scores={"tourist_landmark": 0.8}
                        )
                    except Exception as e:
                        self.logger.error(f"Error inferring possible activities: {e}")

        return possible_activities

    def _enhance_final_description(self, base_description, scene_type, scene_confidence, detected_objects,
                                 functional_zones, activities, safety_concerns, lighting_info,
                                 clip_analysis_results, enable_landmark, image_dims, landmark_info):
        """增強最終描述。"""
        if not self.use_llm or not self.llm_enhancer:
            return base_description

        try:
            obj_list_for_llm = ", ".join(sorted(list(set(
                obj["class_name"] for obj in detected_objects
                if obj.get("confidence", 0) > 0.4 and not obj.get("is_landmark")
            ))))

            if not obj_list_for_llm and enable_landmark and landmark_info.get("primary_landmark"):
                obj_list_for_llm = landmark_info["primary_landmark"].get("class_name", "a prominent feature")
            elif not obj_list_for_llm:
                obj_list_for_llm = "various visual elements"

            # 生成物體統計信息
            object_statistics = {}
            for obj in detected_objects:
                class_name = obj.get("class_name", "unknown")
                if class_name not in object_statistics:
                    object_statistics[class_name] = {
                        "count": 0,
                        "avg_confidence": 0.0,
                        "max_confidence": 0.0,
                        "instances": []
                    }

                stats = object_statistics[class_name]
                stats["count"] += 1
                stats["instances"].append(obj)
                stats["max_confidence"] = max(stats["max_confidence"], obj.get("confidence", 0.0))

            # 計算平均信心度
            for class_name, stats in object_statistics.items():
                if stats["count"] > 0:
                    total_conf = sum(inst.get("confidence", 0.0) for inst in stats["instances"])
                    stats["avg_confidence"] = total_conf / stats["count"]

            llm_scene_data = {
                "original_description": base_description,
                "scene_type": scene_type,
                "scene_name": self.scene_types.get(scene_type, {}).get("name", "Unknown Scene"),
                "detected_objects": detected_objects,
                "object_list": obj_list_for_llm,
                "object_statistics": object_statistics,
                "confidence": scene_confidence,
                "lighting_info": lighting_info,
                "functional_zones": functional_zones,
                "activities": activities,
                "safety_concerns": safety_concerns,
                "clip_analysis": clip_analysis_results if isinstance(clip_analysis_results, dict) else None,
                "enable_landmark": enable_landmark,
                "image_width": image_dims[0] if image_dims else None,
                "image_height": image_dims[1] if image_dims else None,
                "prominent_objects_detail": ""
            }

            # 添加顯著物體詳細信息
            if self.scene_describer and hasattr(self.scene_describer, 'get_prominent_objects') and hasattr(self.scene_describer, 'format_object_list_for_description'):
                try:
                    prominent_objects = self.scene_describer.get_prominent_objects(
                        detected_objects, min_prominence_score=0.1, max_categories_to_return=3, max_total_objects=7
                    )
                    llm_scene_data["prominent_objects_detail"] = self.scene_describer.format_object_list_for_description(prominent_objects)
                except Exception as e:
                    self.logger.error(f"Error getting prominent objects: {e}")

            if enable_landmark and landmark_info.get("primary_landmark"):
                llm_scene_data["primary_landmark_info"] = landmark_info["primary_landmark"]

            return self.llm_enhancer.enhance_description(llm_scene_data)
        except Exception as e:
            self.logger.error(f"Error in LLM Enhancement in main flow (analyze method): {e}")
            return base_description

Writing scene_analysis_coordinator.py


In [ ]:
# %%writefile scene_analyzer.py

import os
import numpy as np
import logging
import traceback
from typing import Dict, List, Tuple, Any, Optional
from PIL import Image

# from component_initializer import ComponentInitializer
# from scene_scoring_engine import SceneScoringEngine
# from landmark_processing_manager import LandmarkProcessingManager
# from scene_analysis_coordinator import SceneAnalysisCoordinator

class SceneAnalyzer:
    """
    Core class for scene analysis and understanding based on object detection results.
    Analyzes detected objects, their relationships, and infers the scene type.
    此class為場景理解的總窗口

    This is the main Facade class that coordinates all scene analysis components
    while maintaining the original public interface for backward compatibility.
    """

    EVERYDAY_SCENE_TYPE_KEYS = [
        "general_indoor_space", "generic_street_view",
        "desk_area_workspace", "outdoor_gathering_spot",
        "kitchen_counter_or_utility_area"
    ]

    def __init__(self, class_names: Dict[int, str] = None, use_llm: bool = True,
                 use_clip: bool = True, enable_landmark: bool = True,
                 llm_model_path: str = None):
        """
        Initialize the scene analyzer with optional class name mappings.

        Args:
            class_names: Dictionary mapping class IDs to class names (optional)
            use_llm: Whether to enable LLM enhancement functionality
            use_clip: Whether to enable CLIP analysis functionality
            enable_landmark: Whether to enable landmark detection functionality
            llm_model_path: Path to LLM model (optional)
        """
        self.logger = logging.getLogger(__name__)

        try:
            # Initialize all components through the component initializer
            self.component_initializer = ComponentInitializer(
                class_names=class_names,
                use_llm=use_llm,
                use_clip=use_clip,
                enable_landmark=enable_landmark,
                llm_model_path=llm_model_path
            )

            # Get data structures for easy access
            self.SCENE_TYPES = self.component_initializer.get_data_structure('SCENE_TYPES')
            self.OBJECT_CATEGORIES = self.component_initializer.get_data_structure('OBJECT_CATEGORIES')
            self.LANDMARK_ACTIVITIES = self.component_initializer.get_data_structure('LANDMARK_ACTIVITIES')

            # Initialize specialized engines
            self.scene_scoring_engine = SceneScoringEngine(
                scene_types=self.SCENE_TYPES,
                enable_landmark=enable_landmark
            )

            self.landmark_processing_manager = LandmarkProcessingManager(
                enable_landmark=enable_landmark,
                use_clip=use_clip
            )

            # Initialize the main coordinator
            self.scene_analysis_coordinator = SceneAnalysisCoordinator(
                component_initializer=self.component_initializer,
                scene_scoring_engine=self.scene_scoring_engine,
                landmark_processing_manager=self.landmark_processing_manager
            )

            # Store configuration for backward compatibility
            self.class_names = class_names
            self.use_clip = use_clip
            self.use_llm = use_llm
            self.enable_landmark = enable_landmark
            self.use_landmark_detection = enable_landmark

            # Get component references for backward compatibility
            self.spatial_analyzer = self.component_initializer.get_component('spatial_analyzer')
            self.descriptor = self.component_initializer.get_component('descriptor')
            self.scene_describer = self.component_initializer.get_component('scene_describer')
            self.clip_analyzer = self.component_initializer.get_component('clip_analyzer')
            self.llm_enhancer = self.component_initializer.get_component('llm_enhancer')
            self.landmark_classifier = self.component_initializer.get_component('landmark_classifier')

            # Set landmark classifier in the processing manager
            if self.landmark_classifier:
                self.landmark_processing_manager.set_landmark_classifier(self.landmark_classifier)

            self.logger.info("SceneAnalyzer initialized successfully with all components")

        except Exception as e:
            self.logger.error(f"Critical error during SceneAnalyzer initialization: {e}")
            traceback.print_exc()
            raise

    def analyze(self, detection_result: Any, lighting_info: Optional[Dict] = None,
                class_confidence_threshold: float = 0.25, scene_confidence_threshold: float = 0.6,
                enable_landmark: bool = True, places365_info: Optional[Dict] = None) -> Dict:
        """
        Analyze detection results to determine scene type and provide understanding.

        Args:
            detection_result: Detection result from YOLOv8 or similar
            lighting_info: Optional lighting condition analysis results
            class_confidence_threshold: Minimum confidence to consider an object
            scene_confidence_threshold: Minimum confidence to determine a scene
            enable_landmark: Whether to enable landmark detection and recognition for this run
            places365_info: Optional Places365 scene classification results

        Returns:
            Dictionary with scene analysis results
        """
        try:
            return self.scene_analysis_coordinator.analyze(
                detection_result=detection_result,
                lighting_info=lighting_info,
                class_confidence_threshold=class_confidence_threshold,
                scene_confidence_threshold=scene_confidence_threshold,
                enable_landmark=enable_landmark,
                places365_info=places365_info
            )
        except Exception as e:
            self.logger.error(f"Error in scene analysis: {e}")
            traceback.print_exc()
            # Return a safe fallback result
            return {
                "scene_type": "unknown",
                "confidence": 0.0,
                "description": "Scene analysis failed due to an internal error.",
                "enhanced_description": "An error occurred during scene analysis. Please check the system logs for details.",
                "objects_present": [],
                "object_count": 0,
                "regions": {},
                "possible_activities": [],
                "safety_concerns": [],
                "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
            }

    def generate_scene_description(self, scene_type: str, detected_objects: List[Dict],
                                 confidence: float, lighting_info: Optional[Dict] = None,
                                 functional_zones: Optional[Dict] = None,
                                 enable_landmark: bool = True,
                                 scene_scores: Optional[Dict] = None,
                                 spatial_analysis: Optional[Dict] = None,
                                 image_dimensions: Optional[Tuple[int, int]] = None) -> str:
        """
        Generate scene description and pass all necessary context to the underlying describer.

        Args:
            scene_type: Identified scene type
            detected_objects: List of detected objects
            confidence: Scene classification confidence
            lighting_info: Lighting condition information (optional)
            functional_zones: Functional zone information (optional)
            enable_landmark: Whether to enable landmark description (optional)
            scene_scores: Scene scores (optional)
            spatial_analysis: Spatial analysis results (optional)
            image_dimensions: Image dimensions (width, height) (optional)

        Returns:
            str: Generated scene description
        """
        try:
            # Convert functional_zones from Dict to List[str] and filter technical terms
            functional_zones_list = []
            if functional_zones and isinstance(functional_zones, dict):
                # Filter out technical terms, keep only meaningful descriptions
                filtered_zones = {k: v for k, v in functional_zones.items()
                                if not k.endswith('_zone') or k in ['dining_zone', 'seating_zone', 'work_zone']}
                functional_zones_list = [v.get('description', k) for k, v in filtered_zones.items()
                                       if isinstance(v, dict) and v.get('description')]
            elif functional_zones and isinstance(functional_zones, list):
                # Filter technical terms from list
                functional_zones_list = [zone for zone in functional_zones
                                       if not zone.endswith('_zone') or 'area' in zone]

            # Generate detailed object statistics
            object_statistics = {}
            for obj in detected_objects:
                class_name = obj.get("class_name", "unknown")
                if class_name not in object_statistics:
                    object_statistics[class_name] = {
                        "count": 0,
                        "avg_confidence": 0.0,
                        "max_confidence": 0.0,
                        "instances": []
                    }

                stats = object_statistics[class_name]
                stats["count"] += 1
                stats["instances"].append(obj)
                stats["max_confidence"] = max(stats["max_confidence"], obj.get("confidence", 0.0))

            # Calculate average confidence
            for class_name, stats in object_statistics.items():
                if stats["count"] > 0:
                    total_conf = sum(inst.get("confidence", 0.0) for inst in stats["instances"])
                    stats["avg_confidence"] = total_conf / stats["count"]

            if self.scene_describer:
                return self.scene_describer.generate_description(
                    scene_type=scene_type,
                    detected_objects=detected_objects,
                    confidence=confidence,
                    lighting_info=lighting_info,
                    functional_zones=functional_zones_list,
                    enable_landmark=enable_landmark,
                    scene_scores=scene_scores,
                    spatial_analysis=spatial_analysis,
                    image_dimensions=image_dimensions,
                    object_statistics=object_statistics
                )
            else:
                return f"A {scene_type} scene with {len(detected_objects)} detected objects."

        except Exception as e:
            self.logger.error(f"Error generating scene description: {e}")
            return f"A {scene_type} scene."

    def process_unknown_objects(self, detection_result, detected_objects):
        """
        Process objects that YOLO failed to identify or have low confidence for landmark detection.

        Args:
            detection_result: YOLO detection results
            detected_objects: List of identified objects

        Returns:
            tuple: (updated object list, landmark object list)
        """
        try:
            return self.landmark_processing_manager.process_unknown_objects(
                detection_result, detected_objects, self.clip_analyzer
            )
        except Exception as e:
            self.logger.error(f"Error processing unknown objects: {e}")
            traceback.print_exc()
            return detected_objects, []

    def _compute_scene_scores(self, detected_objects: List[Dict],
                            spatial_analysis_results: Optional[Dict] = None) -> Dict[str, float]:
        """
        Compute confidence scores for each scene type based on detected objects.

        Args:
            detected_objects: List of detected objects with their details
            spatial_analysis_results: Optional output from SpatialAnalyzer

        Returns:
            Dictionary mapping scene types to confidence scores
        """
        return self.scene_scoring_engine.compute_scene_scores(
            detected_objects, spatial_analysis_results
        )

    def _determine_scene_type(self, scene_scores: Dict[str, float]) -> Tuple[str, float]:
        """
        Determine the most likely scene type based on scores.

        Args:
            scene_scores: Dictionary mapping scene types to confidence scores

        Returns:
            Tuple of (best_scene_type, confidence)
        """
        return self.scene_scoring_engine.determine_scene_type(scene_scores)

    def _fuse_scene_scores(self, yolo_scene_scores: Dict[str, float],
                          clip_scene_scores: Dict[str, float],
                          num_yolo_detections: int = 0,
                          avg_yolo_confidence: float = 0.0,
                          lighting_info: Optional[Dict] = None,
                          places365_info: Optional[Dict] = None) -> Dict[str, float]:
        """
        Fuse scene scores from YOLO-based object detection, CLIP-based analysis, and Places365.

        Args:
            yolo_scene_scores: Scene scores based on YOLO object detection
            clip_scene_scores: Scene scores based on CLIP analysis
            num_yolo_detections: Total number of non-landmark objects detected by YOLO
            avg_yolo_confidence: Average confidence of non-landmark objects detected by YOLO
            lighting_info: Optional lighting condition analysis results
            places365_info: Optional Places365 scene classification results

        Returns:
            Dict: Fused scene scores incorporating all analysis sources
        """
        return self.scene_scoring_engine.fuse_scene_scores(
            yolo_scene_scores, clip_scene_scores, num_yolo_detections,
            avg_yolo_confidence, lighting_info, places365_info
        )

    def _get_alternative_scene_type(self, landmark_scene_type, detected_objects, scene_scores):
        """
        Select appropriate alternative type for landmark scene types.

        Args:
            landmark_scene_type: Original landmark scene type
            detected_objects: List of detected objects
            scene_scores: All scene type scores

        Returns:
            str: Appropriate alternative scene type
        """
        return self.landmark_processing_manager.get_alternative_scene_type(
            landmark_scene_type, detected_objects, scene_scores
        )

    def _remove_landmark_references(self, text):
        """
        Remove all landmark references from text.

        Args:
            text: Input text

        Returns:
            str: Text with landmark references removed
        """
        return self.landmark_processing_manager.remove_landmark_references(text)

    def _define_image_regions(self):
        """Define regions of the image for spatial analysis (3x3 grid)."""
        self.regions = {
            "top_left": (0, 0, 1/3, 1/3),
            "top_center": (1/3, 0, 2/3, 1/3),
            "top_right": (2/3, 0, 1, 1/3),
            "middle_left": (0, 1/3, 1/3, 2/3),
            "middle_center": (1/3, 1/3, 2/3, 2/3),
            "middle_right": (2/3, 1/3, 1, 2/3),
            "bottom_left": (0, 2/3, 1/3, 1),
            "bottom_center": (1/3, 2/3, 2/3, 1),
            "bottom_right": (2/3, 2/3, 1, 1)
        }

    def get_component_status(self) -> Dict[str, bool]:
        """
        Get the initialization status of all components.

        Returns:
            Dictionary mapping component names to their initialization status
        """
        return self.component_initializer.get_initialization_summary()

    def is_component_available(self, component_name: str) -> bool:
        """
        Check if a specific component is available and properly initialized.

        Args:
            component_name: Name of the component to check

        Returns:
            bool: Whether the component is available
        """
        return self.component_initializer.is_component_available(component_name)

    def update_landmark_enable_status(self, enable_landmark: bool):
        """
        Update the landmark detection enable status across all components.

        Args:
            enable_landmark: Whether to enable landmark detection
        """
        self.enable_landmark = enable_landmark
        self.use_landmark_detection = enable_landmark

        # Update all related components
        self.component_initializer.update_landmark_enable_status(enable_landmark)
        self.scene_scoring_engine.update_enable_landmark_status(enable_landmark)
        self.landmark_processing_manager.update_enable_landmark_status(enable_landmark)

        # Update the coordinator's enable_landmark status
        if hasattr(self.scene_analysis_coordinator, 'enable_landmark'):
            self.scene_analysis_coordinator.enable_landmark = enable_landmark

Writing scene_analyzer.py


In [ ]:
# %%writefile image_processor.py
import os
import numpy as np
import torch
import cv2
from PIL import Image
import tempfile
import uuid
from typing import Dict, List, Any, Optional, Tuple

# from detection_model import DetectionModel
# from color_mapper import ColorMapper
# from visualization_helper import VisualizationHelper
# from evaluation_metrics import EvaluationMetrics
# from lighting_analyzer import LightingAnalyzer
# from scene_analyzer import SceneAnalyzer
# from places365_model import Places365Model

class ImageProcessor:
    """
    Class for handling image processing and object detection operations
    Separates processing logic from UI components
    """

    def __init__(self, use_llm=True, llm_model_path=None, enable_places365=True, places365_model_name='resnet50_places365'):
        """Initialize the image processor with required components"""
        print(f"Initializing ImageProcessor with use_llm={use_llm}, enable_places365={enable_places365}")

        try:
            # Initialize basic components first
            self.use_llm = use_llm
            self.llm_model_path = llm_model_path
            self.enable_places365 = enable_places365
            self.model_instances = {}

            self.coco_class_names = {
                0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
                5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
                10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
                14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
                20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
                25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
                30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat',
                35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket',
                39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife',
                44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich',
                49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza',
                54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant',
                59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop',
                64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone',
                68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator',
                73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear',
                78: 'hair drier', 79: 'toothbrush'
            }

            # Initialize ColorMapper
            self.color_mapper = ColorMapper()
            print("ColorMapper initialized successfully")

            # Initialize LightingAnalyzer
            self.lighting_analyzer = LightingAnalyzer()
            print("LightingAnalyzer initialized successfully")

            # Initialize Places365 model if enabled
            self.places365_model = None
            if self.enable_places365:
                try:
                    self.places365_model = Places365Model(
                        model_name=places365_model_name,
                        device=None
                    )
                    print(f"Places365 model initialized successfully with {places365_model_name}")
                except Exception as e:
                    print(f"Warning: Failed to initialize Places365 model: {e}")
                    print("Continuing without Places365 analysis")
                    self.enable_places365 = False
                    self.places365_model = None

            # Initialize SceneAnalyzer with error handling
            self.scene_analyzer = None
            self.class_names = self.coco_class_names

            try:
                # Initialize SceneAnalyzer without class_names (will be set later)
                self.scene_analyzer = SceneAnalyzer(
                    class_names=self.coco_class_names,
                    use_llm=self.use_llm,
                    use_clip=True,
                    enable_landmark=True,
                    llm_model_path=self.llm_model_path
                )
                print("SceneAnalyzer initialized successfully")

                # Verify critical components
                if self.scene_analyzer is not None:
                    print(f"SceneAnalyzer status - spatial_analyzer: {hasattr(self.scene_analyzer, 'spatial_analyzer')}, "
                        f"descriptor: {hasattr(self.scene_analyzer, 'descriptor')}, "
                        f"scene_describer: {hasattr(self.scene_analyzer, 'scene_describer')}")
                else:
                    print("WARNING: scene_analyzer is None after initialization")

            except Exception as e:
                print(f"Error initializing SceneAnalyzer: {e}")
                import traceback
                traceback.print_exc()
                self.scene_analyzer = None

            print("ImageProcessor initialization completed successfully")

        except Exception as e:
            print(f"Critical error during ImageProcessor initialization: {e}")
            import traceback
            traceback.print_exc()
            raise RuntimeError(f"Failed to initialize ImageProcessor: {str(e)}")

    def get_model_instance(self, model_name: str, confidence: float = 0.25, iou: float = 0.25) -> DetectionModel:
        """
        Get or create a model instance based on model name

        Args:
            model_name: Name of the model to use
            confidence: Confidence threshold for detection
            iou: IoU threshold for non-maximum suppression

        Returns:
            DetectionModel instance
        """
        if model_name not in self.model_instances:
            print(f"Creating new model instance for {model_name}")
            self.model_instances[model_name] = DetectionModel(
                model_name=model_name,
                confidence=confidence,
                iou=iou
            )
        else:
            print(f"Using existing model instance for {model_name}")
            self.model_instances[model_name].confidence = confidence

        return self.model_instances[model_name]

    def analyze_scene(self, detection_result: Any, lighting_info: Optional[Dict] = None, enable_landmark=True, places365_info=None) -> Dict:
        """
        Perform scene analysis on detection results

        Args:
            detection_result: Object detection result from YOLOv8
            lighting_info: Lighting condition analysis results (optional)
            enable_landmark: Whether to enable landmark detection
            places365_info: Places365 analysis results (optional)

        Returns:
            Dictionary containing scene analysis results
        """
        print(f"DEBUG: analyze_scene received enable_landmark={enable_landmark}")
        try:
            # Check if detection_result has valid names
            class_names = getattr(detection_result, 'names', None) if detection_result else None

            # Initialize or reinitialize scene analyzer if needed
            if self.scene_analyzer is None:
                print("Scene analyzer not initialized, creating new instance")
                self.scene_analyzer = SceneAnalyzer(
                    class_names=class_names,
                    use_llm=self.use_llm,
                    use_clip=True,
                    enable_landmark=enable_landmark,
                    llm_model_path=self.llm_model_path
                )

                if self.scene_analyzer is None:
                    raise ValueError("Failed to create SceneAnalyzer instance")
            else:
                # Update existing scene analyzer settings
                self.scene_analyzer.enable_landmark = enable_landmark

                # Update class names if available and different
                if class_names and self.scene_analyzer.class_names != class_names:
                    self.scene_analyzer.class_names = class_names
                    if hasattr(self.scene_analyzer, 'spatial_analyzer') and self.scene_analyzer.spatial_analyzer:
                        self.scene_analyzer.spatial_analyzer.class_names = class_names

                # Update landmark detection settings in child components
                if hasattr(self.scene_analyzer, 'spatial_analyzer') and self.scene_analyzer.spatial_analyzer:
                    self.scene_analyzer.spatial_analyzer.enable_landmark = enable_landmark

            # Perform scene analysis with lighting info and Places365 context
            scene_analysis = self.scene_analyzer.analyze(
                detection_result=detection_result,
                lighting_info=lighting_info,
                class_confidence_threshold=0.35,
                scene_confidence_threshold=0.6,
                enable_landmark=enable_landmark,
                places365_info=places365_info
            )

            return scene_analysis

        except Exception as e:
            print(f"Error in scene analysis: {str(e)}")
            import traceback
            traceback.print_exc()

            # Return a valid default result
            return {
                "scene_type": "unknown",
                "confidence": 0.0,
                "description": f"Error during scene analysis: {str(e)}",
                "enhanced_description": "Scene analysis could not be completed due to an error.",
                "objects_present": [],
                "object_count": 0,
                "regions": {},
                "possible_activities": [],
                "safety_concerns": [],
                "lighting_conditions": lighting_info or {"time_of_day": "unknown", "confidence": 0.0}
            }

    def analyze_lighting_conditions(self, image, places365_info: Optional[Dict] = None):
        """
        分析光照條件並考慮 Places365 場景資訊。

        Args:
            image: 輸入圖像
            places365_info: Places365 場景分析結果，用於覆蓋邏輯

        Returns:
            Dict: 光照分析結果
        """
        return self.lighting_analyzer.analyze(image, places365_info=places365_info)

    def analyze_places365_scene(self, image):
        """
        Analyze scene using Places365 model.

        Args:
            image: Input image (PIL Image)

        Returns:
            Dict: Places365 analysis results or None if disabled/failed
        """
        if not self.enable_places365 or self.places365_model is None:
            return None

        try:
            if not isinstance(image, Image.Image):
                if isinstance(image, np.ndarray):
                    image = Image.fromarray(image)
                else:
                    print(f"Warning: Cannot process image of type {type(image)} for Places365")
                    return None

            places365_result = self.places365_model.predict(image)

            if places365_result and places365_result.get('confidence', 0) > 0.1:
                print(f"Places365 detected: {places365_result['scene_label']} "
                    f"(mapped: {places365_result['mapped_scene_type']}) "
                    f"confidence: {places365_result['confidence']:.3f}")
                return places365_result
            else:
                print("Places365 analysis failed or low confidence")
                return None

        except Exception as e:
            print(f"Error in Places365 analysis: {str(e)}")
            return None

    def process_image(self, image: Any, model_name: str, confidence_threshold: float, filter_classes: Optional[List[int]] = None,  enable_landmark: bool = True) -> Tuple[Any, str, Dict]:
        """
        Process an image for object detection and scene analysis.
        Args:
            image: Input image (numpy array or PIL Image).
            model_name: Name of the model to use.
            confidence_threshold: Confidence threshold for detection.
            filter_classes: Optional list of classes to filter results.
            enable_landmark: Whether to enable landmark detection for this run.
        Returns:
            Tuple of (result_image_pil, result_text, stats_data_with_scene_analysis).
        """
        model_instance = self.get_model_instance(model_name, confidence_threshold)
        if model_instance is None:
            return None, f"Failed to load model: {model_name}. Please check model configuration.", {}

        result = None
        stats_data = {}
        temp_path = None
        pil_image_for_processing = None # Use this to store the consistently processed PIL image

        try:
            if isinstance(image, np.ndarray):
                if image.ndim == 3 and image.shape[2] == 3: # RGB or BGR
                    # Assuming BGR from OpenCV, convert to RGB for PIL standard
                    image_rgb_np = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    pil_image_for_processing = Image.fromarray(image_rgb_np)
                elif image.ndim == 3 and image.shape[2] == 4: # RGBA or BGRA
                    image_rgba_np = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA) # Ensure RGBA
                    pil_image_for_processing = Image.fromarray(image_rgba_np).convert("RGB") # Convert to RGB
                elif image.ndim == 2: # Grayscale
                    pil_image_for_processing = Image.fromarray(image).convert("RGB")
                else:
                    pil_image_for_processing = Image.fromarray(image) # Hope for the best
            elif isinstance(image, Image.Image):
                pil_image_for_processing = image.copy() # Use a copy
            elif image is None:
                return None, "No image provided. Please upload an image.", {}
            else:
                return None, f"Unsupported image type: {type(image)}. Please provide a NumPy array or PIL Image.", {}

            if pil_image_for_processing.mode != "RGB": # Ensure final image is RGB
                pil_image_for_processing = pil_image_for_processing.convert("RGB")

            # Add Places365 scene analysis parallel to lighting analysis
            places365_info = self.analyze_places365_scene(pil_image_for_processing)

            lighting_info = self.analyze_lighting_conditions(pil_image_for_processing, places365_info=places365_info)

            temp_dir = tempfile.gettempdir()
            temp_filename = f"temp_{uuid.uuid4().hex}.jpg"
            temp_path = os.path.join(temp_dir, temp_filename)
            pil_image_for_processing.save(temp_path, format="JPEG")

            result = model_instance.detect(temp_path)

            if result is None or not hasattr(result, 'boxes'):
                scene_analysis_no_yolo = self.analyze_scene(result, lighting_info, enable_landmark=enable_landmark, places365_info=places365_info)
                desc_no_yolo = scene_analysis_no_yolo.get("enhanced_description", scene_analysis_no_yolo.get("description", "Detection failed, scene context analysis attempted."))
                stats_data["scene_analysis"] = scene_analysis_no_yolo
                if places365_info:
                    stats_data["places365_analysis"] = places365_info
                return pil_image_for_processing, desc_no_yolo, stats_data

            # 統計資訊
            stats_data = EvaluationMetrics.calculate_basic_stats(result)
            spatial_metrics = EvaluationMetrics.calculate_distance_metrics(result)
            stats_data["spatial_metrics"] = spatial_metrics
            stats_data["lighting_conditions"] = lighting_info
            if places365_info:
                stats_data["places365_analysis"] = places365_info

            if filter_classes and len(filter_classes) > 0:
                classes = result.boxes.cls.cpu().numpy().astype(int)
                confs = result.boxes.conf.cpu().numpy()
                mask = np.isin(classes, filter_classes)
                filtered_stats_data = {
                    "total_objects": int(np.sum(mask)), "class_statistics": {},
                    "average_confidence": float(np.mean(confs[mask])) if np.any(mask) else 0.0,
                    "spatial_metrics": stats_data.get("spatial_metrics",{}),
                    "lighting_conditions": lighting_info
                }
                if places365_info:
                    filtered_stats_data["places365_analysis"] = places365_info
                names = result.names
                class_conf_sums = {}
                for cls_id_int, conf_val in zip(classes[mask], confs[mask]):
                    cls_name = names[cls_id_int]
                    if cls_name not in filtered_stats_data["class_statistics"]:
                        filtered_stats_data["class_statistics"][cls_name] = {"count": 0}
                        class_conf_sums[cls_name] = 0.0
                    filtered_stats_data["class_statistics"][cls_name]["count"] += 1 # 累計統計資訊
                    class_conf_sums[cls_name] += conf_val
                for cls_name_stat, data_stat in filtered_stats_data["class_statistics"].items():
                    data_stat["average_confidence"] = round(class_conf_sums[cls_name_stat] / data_stat["count"] if data_stat["count"] > 0 else 0.0, 4)
                stats_data = filtered_stats_data

            viz_data = EvaluationMetrics.generate_visualization_data(result, self.color_mapper.get_all_colors())

            result_image_pil = VisualizationHelper.visualize_detection(
                temp_path, result, color_mapper=self.color_mapper,
                figsize=(12, 12), return_pil=True, filter_classes=filter_classes
            )

            result_text_summary = EvaluationMetrics.format_detection_summary(viz_data)

            #  Pass the enable_landmark parameter from function signature
            # Initialize or update scene analyzer if needed
            if self.scene_analyzer is None:
                print("Creating SceneAnalyzer in process_image")
                self.scene_analyzer = SceneAnalyzer(
                    class_names=result.names if result else None,
                    use_llm=self.use_llm,
                    use_clip=True,
                    enable_landmark=enable_landmark,
                    llm_model_path=self.llm_model_path
                )

                if self.scene_analyzer is None:
                    print("ERROR: Failed to create SceneAnalyzer in process_image")
            else:
                # Update existing scene analyzer with current settings
                if result and hasattr(result, 'names'):
                    # 使用檢測結果的類別名稱或回退到預定義映射
                    current_class_names = result.names if result.names else self.coco_class_names

                    self.scene_analyzer.class_names = current_class_names
                    if hasattr(self.scene_analyzer, 'spatial_analyzer') and self.scene_analyzer.spatial_analyzer:
                        self.scene_analyzer.spatial_analyzer.update_class_names(current_class_names)

                    logger.info(f"Updated class names in scene analyzer: {list(current_class_names.keys())}")

                self.scene_analyzer.enable_landmark = enable_landmark
                if hasattr(self.scene_analyzer, 'spatial_analyzer') and self.scene_analyzer.spatial_analyzer:
                    self.scene_analyzer.spatial_analyzer.enable_landmark = enable_landmark

            # Perform scene analysis using the existing analyze_scene method
            scene_analysis_result = self.analyze_scene(
                detection_result=result,
                lighting_info=lighting_info,
                enable_landmark=enable_landmark,
                places365_info=places365_info
            )

            stats_data["scene_analysis"] = scene_analysis_result

            final_result_text = result_text_summary

            # Use enable_landmark parameter for landmark block
            if enable_landmark and "detected_landmarks" in scene_analysis_result:
                landmarks_detected = scene_analysis_result.get("detected_landmarks", [])
                if not landmarks_detected and scene_analysis_result.get("primary_landmark"):
                    primary_lm = scene_analysis_result.get("primary_landmark")
                    if isinstance(primary_lm, dict): landmarks_detected = [primary_lm]

                if landmarks_detected:
                    final_result_text += "\n\n--- Detected Landmarks ---\n"
                    # Ensure drawing on the correct PIL image
                    img_to_draw_on = result_image_pil.copy() # Draw on a copy
                    img_for_drawing_cv2 = cv2.cvtColor(np.array(img_to_draw_on), cv2.COLOR_RGB2BGR)

                    for landmark_item in landmarks_detected:
                        if not isinstance(landmark_item, dict): continue

                        # Use .get() for all potentially missing keys 比較保險
                        landmark_name_disp = landmark_item.get("class_name", landmark_item.get("name", "N/A"))
                        landmark_loc_disp = landmark_item.get("location", "N/A")
                        landmark_conf_disp = landmark_item.get("confidence", 0.0)

                        final_result_text += f"• {landmark_name_disp} ({landmark_loc_disp}, confidence: {landmark_conf_disp:.2f})\n"

                        if "box" in landmark_item:
                            box = landmark_item["box"]
                            pt1 = (int(box[0]), int(box[1])); pt2 = (int(box[2]), int(box[3]))
                            color_lm = (255, 0, 255); thickness_lm = 3 # Magenta BGR
                            cv2.rectangle(img_for_drawing_cv2, pt1, pt2, color_lm, thickness_lm)

                            label_lm = f"{landmark_name_disp} ({landmark_conf_disp:.2f})"
                            font_scale_lm = 0.6; font_thickness_lm = 1
                            (w_text, h_text), baseline = cv2.getTextSize(label_lm, cv2.FONT_HERSHEY_SIMPLEX, font_scale_lm, font_thickness_lm)

                            # Label position logic (simplified from your extensive one for brevity)
                            label_y_pos = pt1[1] - baseline - 3
                            if label_y_pos < h_text : # If label goes above image, put it below box
                                label_y_pos = pt2[1] + h_text + baseline + 3

                            label_bg_pt1 = (pt1[0], label_y_pos - h_text - baseline)
                            label_bg_pt2 = (pt1[0] + w_text, label_y_pos + baseline)

                            cv2.rectangle(img_for_drawing_cv2, label_bg_pt1, label_bg_pt2, color_lm, -1)
                            cv2.putText(img_for_drawing_cv2, label_lm, (pt1[0], label_y_pos),
                                        cv2.FONT_HERSHEY_SIMPLEX, font_scale_lm, (255,255,255), font_thickness_lm, cv2.LINE_AA)

                    result_image_pil = Image.fromarray(cv2.cvtColor(img_for_drawing_cv2, cv2.COLOR_BGR2RGB))

            return result_image_pil, final_result_text, stats_data

        except Exception as e:
            error_message = f"Error in ImageProcessor.process_image: {str(e)}"
            import traceback
            traceback.print_exc()
            return pil_image_for_processing if pil_image_for_processing else None, error_message, {}
        finally:
            if temp_path and os.path.exists(temp_path):
                try: os.remove(temp_path)
                except Exception as e: print(f"Warning: Cannot delete temp file {temp_path}: {str(e)}")

    def format_result_text(self, stats: Dict) -> str:
        """
        Format detection statistics into readable text with improved spacing

        Args:
            stats: Dictionary containing detection statistics

        Returns:
            Formatted text summary
        """
        if not stats or "total_objects" not in stats:
            return "No objects detected."

        # 減少不必要的空行
        lines = [
            f"Detected {stats['total_objects']} objects.",
            f"Average confidence: {stats.get('average_confidence', 0):.2f}",
            "Objects by class:"
        ]

        if "class_statistics" in stats and stats["class_statistics"]:
            # 按計數排序類別
            sorted_classes = sorted(
                stats["class_statistics"].items(),
                key=lambda x: x[1]["count"],
                reverse=True
            )

            for cls_name, cls_stats in sorted_classes:
                count = cls_stats["count"]
                conf = cls_stats.get("average_confidence", 0)

                item_text = "item" if count == 1 else "items"
                lines.append(f"• {cls_name}: {count} {item_text} (avg conf: {conf:.2f})")
        else:
            lines.append("No class information available.")

        # 添加空間資訊
        if "spatial_metrics" in stats and "spatial_distribution" in stats["spatial_metrics"]:
            lines.append("Object Distribution:")

            dist = stats["spatial_metrics"]["spatial_distribution"]
            x_mean = dist.get("x_mean", 0)
            y_mean = dist.get("y_mean", 0)

            # 描述物體的大致位置
            if x_mean < 0.33:
                h_pos = "on the left side"
            elif x_mean < 0.67:
                h_pos = "in the center"
            else:
                h_pos = "on the right side"

            if y_mean < 0.33:
                v_pos = "in the upper part"
            elif y_mean < 0.67:
                v_pos = "in the middle"
            else:
                v_pos = "in the lower part"

            lines.append(f"• Most objects appear {h_pos} {v_pos} of the image")

        return "\n".join(lines)

    def format_json_for_display(self, stats: Dict) -> Dict:
        """
        Format statistics JSON for better display

        Args:
            stats: Raw statistics dictionary

        Returns:
            Formatted statistics structure for display
        """
        # Create a cleaner copy of the stats for display
        display_stats = {}

        # Add summary section
        display_stats["summary"] = {
            "total_objects": stats.get("total_objects", 0),
            "average_confidence": round(stats.get("average_confidence", 0), 3)
        }

        # Add class statistics in a more organized way
        if "class_statistics" in stats and stats["class_statistics"]:
            # Sort classes by count (descending)
            sorted_classes = sorted(
                stats["class_statistics"].items(),
                key=lambda x: x[1].get("count", 0),
                reverse=True
            )

            class_stats = {}
            for cls_name, cls_data in sorted_classes:
                class_stats[cls_name] = {
                    "count": cls_data.get("count", 0),
                    "average_confidence": round(cls_data.get("average_confidence", 0), 3)
                }

            display_stats["detected_objects"] = class_stats

        # Simplify spatial metrics
        if "spatial_metrics" in stats:
            spatial = stats["spatial_metrics"]

            # Simplify spatial distribution
            if "spatial_distribution" in spatial:
                dist = spatial["spatial_distribution"]
                display_stats["spatial"] = {
                    "distribution": {
                        "x_mean": round(dist.get("x_mean", 0), 3),
                        "y_mean": round(dist.get("y_mean", 0), 3),
                        "x_std": round(dist.get("x_std", 0), 3),
                        "y_std": round(dist.get("y_std", 0), 3)
                    }
                }

            # Add simplified size information
            if "size_distribution" in spatial:
                size = spatial["size_distribution"]
                display_stats["spatial"]["size"] = {
                    "mean_area": round(size.get("mean_area", 0), 3),
                    "min_area": round(size.get("min_area", 0), 3),
                    "max_area": round(size.get("max_area", 0), 3)
                }

        return display_stats

    def prepare_visualization_data(self, stats: Dict, available_classes: Dict[int, str]) -> Dict:
        """
        Prepare data for visualization based on detection statistics

        Args:
            stats: Detection statistics
            available_classes: Dictionary of available class IDs and names

        Returns:
            Visualization data dictionary
        """
        if not stats or "class_statistics" not in stats or not stats["class_statistics"]:
            return {"error": "No detection data available"}

        # Prepare visualization data
        viz_data = {
            "total_objects": stats.get("total_objects", 0),
            "average_confidence": stats.get("average_confidence", 0),
            "class_data": []
        }

        # Class data
        for cls_name, cls_stats in stats.get("class_statistics", {}).items():
            # Search class ID
            class_id = -1
            for id, name in available_classes.items():
                if name == cls_name:
                    class_id = id
                    break

            cls_data = {
                "name": cls_name,
                "class_id": class_id,
                "count": cls_stats.get("count", 0),
                "average_confidence": cls_stats.get("average_confidence", 0),
                "color": self.color_mapper.get_color(class_id if class_id >= 0 else cls_name)
            }

            viz_data["class_data"].append(cls_data)

        # Descending order
        viz_data["class_data"].sort(key=lambda x: x["count"], reverse=True)

        return viz_data

Writing image_processor.py


In [ ]:
# %%writefile video_processor.py
import cv2
import os
import tempfile
import uuid
from PIL import Image
import numpy as np
from typing import Dict, List, Tuple, Any, Optional
import time
from collections import defaultdict

# from image_processor import ImageProcessor
# from evaluation_metrics import EvaluationMetrics
# from scene_analyzer import SceneAnalyzer
# from detection_model import DetectionModel

class VideoProcessor:
    """
    Handles the processing of video files, including object detection
    and scene analysis on selected frames.
    """
    def __init__(self, image_processor: ImageProcessor):
        """
        Initializes the VideoProcessor.

        Args:
            image_processor (ImageProcessor): An initialized ImageProcessor instance.
        """
        self.image_processor = image_processor

    def process_video_file(self,
                           video_path: str,
                           model_name: str,
                           confidence_threshold: float,
                           process_interval: int = 5,
                           scene_desc_interval_sec: int = 3) -> Tuple[Optional[str], str, Dict]:
        """
        Processes an uploaded video file, performs detection and periodic scene analysis,
        and returns the path to the annotated output video file along with a summary.

        Args:
            video_path (str): Path to the input video file.
            model_name (str): Name of the YOLO model to use.
            confidence_threshold (float): Confidence threshold for object detection.
            process_interval (int): Process every Nth frame. Defaults to 5.
            scene_desc_interval_sec (int): Update scene description every N seconds. Defaults to 3.

        Returns:
            Tuple[Optional[str], str, Dict]: (Path to output video or None, Summary text, Statistics dictionary)
        """
        if not video_path or not os.path.exists(video_path):
            print(f"Error: Video file not found at {video_path}")
            return None, "Error: Video file not found.", {}

        print(f"Starting video processing for: {video_path}")
        start_time = time.time()

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            return None, "Error opening video file.", {}

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0: # Handle case where fps is not available or invalid
             fps = 30 # Assume a default fps
             print(f"Warning: Could not get valid FPS for video. Assuming {fps} FPS.")
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Video properties: {width}x{height} @ {fps:.2f} FPS, Total Frames: {total_frames_video}")

        # Calculate description update interval in frames
        description_update_interval_frames = int(fps * scene_desc_interval_sec)
        if description_update_interval_frames < 1:
            description_update_interval_frames = int(fps) # Update at least once per second if interval is too short

        object_trackers = {}  # 儲存ID與物體的映射
        last_detected_objects = {}  # 儲存上一次檢測到的物體資訊
        next_object_id = 0  # 下一個可用的物體ID
        tracking_threshold = 0.6  # 相同物體的IoU
        object_colors = {}  # 每個被追蹤的物體分配固定顏色

        # Setup Output Video
        output_filename = f"processed_{uuid.uuid4().hex}_{os.path.basename(video_path)}"
        temp_dir = tempfile.gettempdir() # Use system's temp directory
        output_path = os.path.join(temp_dir, output_filename)
        # Ensure the output path has a compatible extension (like .mp4)
        if not output_path.lower().endswith(('.mp4', '.avi', '.mov')):
            output_path += ".mp4"

        # Use 'mp4v' for MP4, common and well-supported
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        if not out.isOpened():
            print(f"Error: Could not open VideoWriter for path: {output_path}")
            cap.release()
            return None, f"Error creating output video file at {output_path}.", {}
        print(f"Output video will be saved to: {output_path}")

        frame_count = 0
        processed_frame_count = 0
        all_stats = [] # Store stats for each processed frame
        summary_lines = []
        last_description = "Analyzing scene..." # Initial description
        frame_since_last_desc = description_update_interval_frames # Trigger analysis on first processed frame

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break # End of video

                frame_count += 1
                frame_since_last_desc += 1
                current_frame_annotated = False # Flag if this frame was processed and annotated

                # Process frame based on interval
                if frame_count % process_interval == 0:
                    processed_frame_count += 1
                    print(f"Processing frame {frame_count}...")
                    current_frame_annotated = True

                    # Use ImageProcessor for single-frame tasks
                    # 1. Convert frame format BGR -> RGB -> PIL
                    try:
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        pil_image = Image.fromarray(frame_rgb)
                    except Exception as e:
                        print(f"Error converting frame {frame_count}: {e}")
                        continue # Skip this frame

                    # 2. Get appropriate model instance
                    # Confidence is passed from UI, model_name too
                    model_instance = self.image_processor.get_model_instance(model_name, confidence_threshold)
                    if not model_instance or not model_instance.is_model_loaded:
                         print(f"Error: Model {model_name} not loaded. Skipping frame {frame_count}.")
                         # Draw basic frame without annotation
                         cv2.putText(frame, f"Scene: {last_description[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3, cv2.LINE_AA)
                         cv2.putText(frame, f"Scene: {last_description[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
                         out.write(frame)
                         continue


                    # 3. Perform detection
                    detection_result = model_instance.detect(pil_image) # Use PIL image

                    current_description_for_frame = last_description # Default to last known description
                    scene_analysis_result = None
                    stats = {}

                    if detection_result and hasattr(detection_result, 'boxes') and len(detection_result.boxes) > 0:
                        # Ensure SceneAnalyzer is ready within ImageProcessor
                        if not hasattr(self.image_processor, 'scene_analyzer') or self.image_processor.scene_analyzer is None:
                             print("Initializing SceneAnalyzer...")
                             # Pass class names from the current detection result
                             self.image_processor.scene_analyzer = SceneAnalyzer(class_names=detection_result.names)
                        elif self.image_processor.scene_analyzer.class_names is None:
                             # Update class names if they were missing
                             self.image_processor.scene_analyzer.class_names = detection_result.names
                             if hasattr(self.image_processor.scene_analyzer, 'spatial_analyzer'):
                                 self.image_processor.scene_analyzer.spatial_analyzer.class_names = detection_result.names


                        # 4. Perform Scene Analysis (periodically)
                        if frame_since_last_desc >= description_update_interval_frames:
                            print(f"Analyzing scene at frame {frame_count} (threshold: {description_update_interval_frames} frames)...")
                            # Pass lighting_info=None for now, as it's disabled for performance
                            scene_analysis_result = self.image_processor.analyze_scene(detection_result, lighting_info=None)
                            current_description_for_frame = scene_analysis_result.get("description", last_description)
                            last_description = current_description_for_frame # Cache the new description
                            frame_since_last_desc = 0 # Reset counter

                        # 5. Calculate Statistics for this frame
                        stats = EvaluationMetrics.calculate_basic_stats(detection_result)
                        stats['frame_number'] = frame_count # Add frame number to stats
                        all_stats.append(stats)

                        # 6. Draw annotations
                        names = detection_result.names
                        boxes = detection_result.boxes.xyxy.cpu().numpy()
                        classes = detection_result.boxes.cls.cpu().numpy().astype(int)
                        confs = detection_result.boxes.conf.cpu().numpy()

                        def calculate_iou(box1, box2):
                            """Calculate Intersection IOU value"""
                            x1_1, y1_1, x2_1, y2_1 = box1
                            x1_2, y1_2, x2_2, y2_2 = box2

                            xi1 = max(x1_1, x1_2)
                            yi1 = max(y1_1, y1_2)
                            xi2 = min(x2_1, x2_2)
                            yi2 = min(y2_1, y2_2)

                            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
                            box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
                            box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

                            union_area = box1_area + box2_area - inter_area

                            return inter_area / union_area if union_area > 0 else 0

                        # 處理當前幀中的所有檢測
                        current_detected_objects = {}

                        for box, cls_id, conf in zip(boxes, classes, confs):
                            x1, y1, x2, y2 = map(int, box)

                            # 查找最匹配的已追蹤物體
                            best_match_id = None
                            best_match_iou = 0

                            for obj_id, (old_box, old_cls_id, _) in last_detected_objects.items():
                                if old_cls_id == cls_id:  # 同一類別才比較
                                    iou = calculate_iou(box, old_box)
                                    if iou > tracking_threshold and iou > best_match_iou:
                                        best_match_id = obj_id
                                        best_match_iou = iou

                            # 如果找到匹配，使用現有ID；否則分配新ID
                            if best_match_id is not None:
                                obj_id = best_match_id
                            else:
                                obj_id = next_object_id
                                next_object_id += 1

                                # 使用更明顯的顏色
                                bright_colors = [
                                    (0, 0, 255),    # red
                                    (0, 255, 0),    # green
                                    (255, 0, 0),    # blue
                                    (0, 255, 255),  # yellow
                                    (255, 0, 255),  # purple
                                    (255, 128, 0),  # orange
                                    (128, 0, 255)   # purple
                                ]
                                object_colors[obj_id] = bright_colors[obj_id % len(bright_colors)]

                            # update tracking info
                            current_detected_objects[obj_id] = (box, cls_id, conf)

                            color = object_colors.get(obj_id, (0, 255, 0))  # default is green
                            label = f"{names.get(cls_id, 'Unknown')}-{obj_id}: {conf:.2f}"

                            # 平滑化邊界框：如果是已知物體，與上一幀位置平均
                            if obj_id in last_detected_objects:
                                old_box, _, _ = last_detected_objects[obj_id]
                                old_x1, old_y1, old_x2, old_y2 = map(int, old_box)
                                # 平滑係數
                                alpha = 0.7  # current weight
                                beta = 0.3   # history weight

                                x1 = int(alpha * x1 + beta * old_x1)
                                y1 = int(alpha * y1 + beta * old_y1)
                                x2 = int(alpha * x2 + beta * old_x2)
                                y2 = int(alpha * y2 + beta * old_y2)

                            # draw box and label
                            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                            # add text
                            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                            cv2.rectangle(frame, (x1, y1 - h - 10), (x1 + w, y1 - 10), color, -1)
                            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

                        # update tracking info
                        last_detected_objects = current_detected_objects.copy()


                    # Draw the current scene description on the frame
                    cv2.putText(frame, f"Scene: {current_description_for_frame[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3, cv2.LINE_AA) # Black outline
                    cv2.putText(frame, f"Scene: {current_description_for_frame[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA) # White text

                # Write the frame (annotated or original) to the output video
                # Draw last known description if this frame wasn't processed
                if not current_frame_annotated:
                    cv2.putText(frame, f"Scene: {last_description[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 3, cv2.LINE_AA)
                    cv2.putText(frame, f"Scene: {last_description[:80]}...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

                out.write(frame) # Write frame to output file

        except Exception as e:
            print(f"Error during video processing loop for {video_path}: {e}")
            import traceback
            traceback.print_exc()
            summary_lines.append(f"An error occurred during processing: {e}")
        finally:
            # Release resources
            cap.release()
            out.release()
            print(f"Video processing finished. Resources released. Output path: {output_path}")
            if not os.path.exists(output_path) or os.path.getsize(output_path) == 0:
                print(f"Error: Output video file was not created or is empty at {output_path}")
                summary_lines.append("Error: Failed to create output video.")
                output_path = None

        end_time = time.time()
        processing_time = end_time - start_time
        summary_lines.insert(0, f"Finished processing in {processing_time:.2f} seconds.")
        summary_lines.insert(1, f"Processed {processed_frame_count} frames out of {frame_count} (interval: {process_interval} frames).")
        summary_lines.insert(2, f"Scene description updated approximately every {scene_desc_interval_sec} seconds.")

        # Generate Aggregate Statistics
        aggregated_stats = {
            "total_frames_read": frame_count,
            "total_frames_processed": processed_frame_count,
            "avg_objects_per_processed_frame": 0, # Calculate below
            "cumulative_detections": {}, # Total times each class was detected
            "max_concurrent_detections": {} # Max count of each class in a single processed frame
            }
        object_cumulative_counts = {}
        object_max_concurrent_counts = {} # Store the max count found for each object type
        total_detected_in_processed = 0

        # Iterate through stats collected from each processed frame
        for frame_stats in all_stats:
            total_objects_in_frame = frame_stats.get("total_objects", 0)
            total_detected_in_processed += total_objects_in_frame

            # Iterate through object classes detected in this frame
            for obj_name, obj_data in frame_stats.get("class_statistics", {}).items():
                count_in_frame = obj_data.get("count", 0)

                # Cumulative count
                if obj_name not in object_cumulative_counts:
                    object_cumulative_counts[obj_name] = 0
                object_cumulative_counts[obj_name] += count_in_frame

                # Max concurrent count
                if obj_name not in object_max_concurrent_counts:
                    object_max_concurrent_counts[obj_name] = 0
                # Update the max count if the current frame's count is higher
                object_max_concurrent_counts[obj_name] = max(object_max_concurrent_counts[obj_name], count_in_frame)

        # Add sorted results to the final dictionary
        aggregated_stats["cumulative_detections"] = dict(sorted(object_cumulative_counts.items(), key=lambda item: item[1], reverse=True))
        aggregated_stats["max_concurrent_detections"] = dict(sorted(object_max_concurrent_counts.items(), key=lambda item: item[1], reverse=True))

        # Calculate average objects per processed frame
        if processed_frame_count > 0:
             aggregated_stats["avg_objects_per_processed_frame"] = round(total_detected_in_processed / processed_frame_count, 2)

        summary_text = "\n".join(summary_lines)
        print("Generated Summary:\n", summary_text)
        print("Aggregated Stats (Revised):\n", aggregated_stats) # Print the revised stats

        # Return the potentially updated output_path
        return output_path, summary_text, aggregated_stats

Writing video_processor.py


In [ ]:
# %%writefile model_manager.py
import os
import torch
import logging
from typing import Dict, Optional, Any
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

class ModelLoadingError(Exception):
    """Custom exception for model loading failures"""
    pass


class ModelGenerationError(Exception):
    """Custom exception for model generation failures"""
    pass


class ModelManager:
    """
    負責LLM模型的載入、設備管理和文本生成。
    管理模型、記憶體優化和設備配置。
    """

    def __init__(self,
                 model_path: Optional[str] = None,
                 tokenizer_path: Optional[str] = None,
                 device: Optional[str] = None,
                 max_length: int = 2048,
                 temperature: float = 0.3,
                 top_p: float = 0.85):
        """
        初始化模型管理器

        Args:
            model_path: LLM模型的路徑或HuggingFace模型名稱，默認使用Llama 3.2
            tokenizer_path: tokenizer的路徑，通常與model_path相同
            device: 運行設備 ('cpu'或'cuda')，None時自動檢測
            max_length: 輸入文本的最大長度
            temperature: 生成文本的溫度參數
            top_p: 生成文本時的核心採樣機率閾值
        """
        # 設置專屬logger
        self.logger = logging.getLogger(self.__class__.__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

        # 模型配置
        self.model_path = model_path or "meta-llama/Llama-3.2-3B-Instruct"
        self.tokenizer_path = tokenizer_path or self.model_path

        # 設備管理
        self.device = self._detect_device(device)
        self.logger.info(f"Device selected: {self.device}")

        # 生成參數
        self.max_length = max_length
        self.temperature = temperature
        self.top_p = top_p

        # 模型狀態
        self.model = None
        self.tokenizer = None
        self._model_loaded = False
        self.call_count = 0

        # HuggingFace認證
        self.hf_token = self._setup_huggingface_auth()

    def _detect_device(self, device: Optional[str]) -> str:
        """
        檢測並設置運行設備

        Args:
            device: 用戶指定的設備，None時自動檢測

        Returns:
            str: ('cuda' or 'cpu')
        """
        if device:
            if device == 'cuda' and not torch.cuda.is_available():
                self.logger.warning("CUDA requested but not available, falling back to CPU")
                return 'cpu'
            return device

        detected_device = 'cuda' if torch.cuda.is_available() else 'cpu'

        if detected_device == 'cuda':
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            self.logger.info(f"CUDA detected with {gpu_memory:.2f} GB GPU memory")

        return detected_device

    def _setup_huggingface_auth(self) -> Optional[str]:
        """
        設置HuggingFace認證

        Returns:
            Optional[str]: HuggingFace token，如果可用
        """
        hf_token = os.environ.get("HF_TOKEN")

        if hf_token:
            try:
                login(token=hf_token)
                self.logger.info("Successfully authenticated with HuggingFace")
                return hf_token
            except Exception as e:
                self.logger.error(f"HuggingFace authentication failed: {e}")
                return None
        else:
            self.logger.warning("HF_TOKEN not found. Access to gated models may be limited")
            return None

    def _load_model(self):
        """
        載入LLM模型和tokenizer，使用8位量化以節省記憶體

        Raises:
            ModelLoadingError: 當模型載入失敗時
        """
        if self._model_loaded:
            return

        try:
            self.logger.info(f"Loading model from {self.model_path} with 8-bit quantization")

            # 清理GPU記憶體
            self._clear_gpu_cache()

            # 設置8位量化配置
            quantization_config = BitsAndBytesConfig(
                load_in_8bit=True,
                llm_int8_enable_fp32_cpu_offload=True
            )

            # 載入tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.tokenizer_path,
                padding_side="left",
                use_fast=False,
                token=self.hf_token
            )

            # 設置特殊標記
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # 載入模型
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                quantization_config=quantization_config,
                device_map="auto",
                low_cpu_mem_usage=True,
                token=self.hf_token
            )

            self._model_loaded = True
            self.logger.info("Model loaded successfully")

        except Exception as e:
            error_msg = f"Failed to load model: {str(e)}"
            self.logger.error(error_msg)
            raise ModelLoadingError(error_msg) from e

    def _clear_gpu_cache(self):
        """清理GPU記憶體緩存"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            self.logger.debug("GPU cache cleared")

    def generate_response(self, prompt: str, **generation_kwargs) -> str:
        """
        生成LLM回應

        Args:
            prompt: 輸入提示詞
            **generation_kwargs: 額外的生成參數，可覆蓋預設值

        Returns:
            str: 生成的回應文本

        Raises:
            ModelGenerationError: 當生成失敗時
        """
        # 確保模型已載入
        if not self._model_loaded:
            self._load_model()

        try:
            self.call_count += 1
            self.logger.info(f"Generating response (call #{self.call_count})")

            # clean GPU
            self._clear_gpu_cache()

            # 設置固定種子以提高一致性
            torch.manual_seed(42)

            # prepare input
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=self.max_length
            ).to(self.device)

            # 準備生成參數
            generation_params = self._prepare_generation_params(**generation_kwargs)
            generation_params.update({
                "pad_token_id": self.tokenizer.eos_token_id,
                "attention_mask": inputs.attention_mask,
                "use_cache": True,
            })

            # resposne
            with torch.no_grad():
                outputs = self.model.generate(inputs.input_ids, **generation_params)

            # 解碼回應
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = self._extract_generated_response(full_response, prompt)

            if not response or len(response.strip()) < 10:
                raise ModelGenerationError("Generated response is too short or empty")

            self.logger.info(f"Response generated successfully ({len(response)} characters)")
            return response

        except Exception as e:
            error_msg = f"Text generation failed: {str(e)}"
            self.logger.error(error_msg)
            raise ModelGenerationError(error_msg) from e

    def _prepare_generation_params(self, **kwargs) -> Dict[str, Any]:
        """
        準備生成參數，支援模型特定的優化

        Args:
            **kwargs: 用戶提供的生成參數

        Returns:
            Dict[str, Any]: 完整的生成參數配置
        """
        # basic parameters
        params = {
            "max_new_tokens": 120,
            "temperature": self.temperature,
            "top_p": self.top_p,
            "do_sample": True,
        }

        # 針對Llama模型的特殊優化
        if "llama" in self.model_path.lower():
            params.update({
                "max_new_tokens": 600,
                "temperature": 0.35, # not too big
                "top_p": 0.75,
                "repetition_penalty": 1.5,
                "num_beams": 5,
                "length_penalty": 1,
                "no_repeat_ngram_size": 3
            })
        else:
            params.update({
                "max_new_tokens": 300,
                "temperature": 0.6,
                "top_p": 0.9,
                "num_beams": 1,
                "repetition_penalty": 1.05
            })

        # 用戶參數覆蓋預設值
        params.update(kwargs)

        return params

    def _extract_generated_response(self, full_response: str, prompt: str) -> str:
        """
        從完整回應中提取生成的部分

        Args:
            full_response: 模型的完整輸出
            prompt: 原始提示詞

        Returns:
            str: 提取的生成回應
        """
        # 尋找assistant標記
        assistant_tag = "<|assistant|>"
        if assistant_tag in full_response:
            response = full_response.split(assistant_tag)[-1].strip()

            # 檢查是否有未閉合的user標記
            user_tag = "<|user|>"
            if user_tag in response:
                response = response.split(user_tag)[0].strip()

            return response

        # 移除輸入提示詞
        if full_response.startswith(prompt):
            return full_response[len(prompt):].strip()

        return full_response.strip()

    def reset_context(self):
        """重置模型上下文，清理GPU緩存"""
        if self._model_loaded:
            self._clear_gpu_cache()
            self.logger.info("Model context reset")
        else:
            self.logger.info("Model not loaded, no context to reset")

    def get_current_device(self) -> str:
        """
        獲取當前運行設備

        Returns:
            str: 當前設備名稱
        """
        return self.device

    def is_model_loaded(self) -> bool:
        """
        檢查模型是否已載入

        Returns:
            bool: 模型載入狀態
        """
        return self._model_loaded

    def get_call_count(self) -> int:
        """
        獲取模型調用次數

        Returns:
            int: 調用次數
        """
        return self.call_count

    def get_model_info(self) -> Dict[str, Any]:
        """
        獲取模型信息

        Returns:
            Dict[str, Any]: 包含模型路徑、設備、載入狀態等信息
        """
        return {
            "model_path": self.model_path,
            "device": self.device,
            "is_loaded": self._model_loaded,
            "call_count": self.call_count,
            "has_hf_token": self.hf_token is not None
        }

Writing model_manager.py


In [ ]:
# %%writefile prompt_template_manager.py
import logging
import traceback
from typing import Dict, List, Any, Optional

class PromptTemplateError(Exception):
    """提示模板相關錯誤的自定義異常"""
    pass


class PromptTemplateManager:
    """
    負責管理和格式化各種LLM提示模板。
    包含場景描述增強、錯誤檢測、無檢測處理等不同場景的模板。
    """

    def __init__(self):
        """初始化提示模板管理器"""
        # set the logger
        self.logger = logging.getLogger(self.__class__.__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

        # initialize all templates
        self._initialize_templates()
        self.logger.info("PromptTemplateManager initialized successfully")

    def _initialize_templates(self):
        """初始化所有提示模板"""
        try:
            self._setup_enhancement_template()
            self._setup_verification_template()
            self._setup_no_detection_template()
            self.logger.info("All prompt templates initialized")
        except Exception as e:
            self.logger.error(f"Failed to initialize templates: {str(e)}")
            self.logger.error(traceback.format_exc())
            raise PromptTemplateError(f"Template initialization failed: {str(e)}") from e


    def format_enhancement_prompt_with_landmark(self, scene_data: Dict[str, Any], object_list: str, original_description: str) -> str:
        try:
            # 確保場景類型被正確清理
            scene_type = scene_data.get("scene_type", "unknown scene")
            cleaned_scene_type = self._clean_scene_type(scene_type)

            # 通用文本格式清理：處理底線和格式化問題
            cleaned_description = self._clean_text_formatting(original_description)

            # 額外清理場景類型底線格式
            cleaned_description = self._clean_scene_type_underscores(cleaned_description)

            # 強化輸入清理
            cleaned_description = self._enhance_input_cleaning(cleaned_description)

            # 在原始描述中替換未清理的場景類型
            if scene_type != cleaned_scene_type:
                cleaned_description = cleaned_description.replace(scene_type, cleaned_scene_type)

            # 檢查是否有地標資訊
            landmark_info = scene_data.get("landmark_location_info")
            is_fallback = scene_data.get("is_fallback", False)

            # 準備額外的地標指導內容
            additional_guidance = ""
            if landmark_info:
                landmark_name = landmark_info.get("name", "")
                landmark_location = landmark_info.get("location", "")
                additional_guidance = f"""
            LANDMARK LOCATION REQUIREMENT: This scene features {landmark_name} located in {landmark_location}.
            16. MANDATORY: Include the specific location "{landmark_location}" when first mentioning {landmark_name}. Use natural phrasing such as "Located in {landmark_location}, the {landmark_name}..." or "The {landmark_name} in {landmark_location}..." or "Standing majestically in {landmark_location}, {landmark_name}...".
            17. Avoid mechanical openings like "The tourist landmark is centered around" or "The scene is centered around". Instead, begin with the landmark itself as the subject.
            18. NEVER use terms with underscores like "tourist_landmark" or "historical_site" in your response. Use natural language: "tourist landmark", "historical site", "cultural attraction" etc.
            19. The geographical reference must appear naturally in the opening sentence, integrated as essential context rather than supplementary information."""
            elif is_fallback:
                additional_guidance = """
            FALLBACK MODE: The previous enhancement was insufficient. Provide a more detailed description focusing on key visual elements, human activities, atmospheric details, and architectural features."""

            # 建構完整的模板內容
            if additional_guidance:
                # 在CRITICAL RULES後添加地標相關指導
                enhanced_template = self.enhance_description_template.replace(
                    "15. When describing quantities or arrangements, use only information explicitly confirmed by the object detection system.",
                    f"15. When describing quantities or arrangements, use only information explicitly confirmed by the object detection system.{additional_guidance}"
                )
            else:
                enhanced_template = self.enhance_description_template

            formatted_prompt = enhanced_template.format(
                original_description=cleaned_description,
                object_list=object_list
            )

            return formatted_prompt

        except Exception as e:
            self.logger.error(f"Failed to format enhancement prompt: {str(e)}")
            self.logger.error(traceback.format_exc())
            raise PromptTemplateError(f"Prompt formatting failed: {e}") from e

    def _clean_text_formatting(self, text: str) -> str:
        """
        通用文本格式清理方法，處理底線、格式化等問題

        Args:
            text: 需要清理的原始文本

        Returns:
            str: 清理後的文本
        """
        if not text:
            return text

        try:
            import re

            # 替換常見的技術性詞彙
            replacements = {
                'tourist_landmark': 'tourist landmark',
                'historical_site': 'historical site',
                'religious_building': 'religious building',
                'cultural_landmark': 'cultural landmark',
                'architectural_site': 'architectural site',
                'natural_landmark': 'natural landmark'
            }

            cleaned = text
            for old_term, new_term in replacements.items():
                cleaned = cleaned.replace(old_term, new_term)

            # 處理其他底線情況
            cleaned = re.sub(r'(\w+)_(\w+)', lambda m: f"{m.group(1)} {m.group(2)}", cleaned)

            # 處理多個連續底線
            cleaned = re.sub(r'_+', ' ', cleaned)

            # 清理多餘空格
            cleaned = re.sub(r'\s+', ' ', cleaned)

            return cleaned.strip()

        except Exception as e:
            self.logger.warning(f"Error in text formatting cleanup: {str(e)}")
            return text

    def _clean_scene_type_underscores(self, text: str) -> str:
        """
        專門清理場景類型中的底線格式

        Args:
            text: 需要清理的文本

        Returns:
            str: 清理後的文本
        """
        if not text:
            return text

        try:
            import re

            # 專門處理場景類型的底線格式
            scene_type_patterns = [
                'urban_intersection', 'city_street', 'downtown_area', 'business_district',
                'residential_area', 'commercial_zone', 'industrial_area', 'shopping_center',
                'traffic_intersection', 'pedestrian_crossing', 'public_square'
            ]

            for pattern in scene_type_patterns:
                if pattern in text:
                    replacement = pattern.replace('_', ' ')
                    text = text.replace(pattern, replacement)

            # 處理任何剩餘的場景類型底線模式
            text = re.sub(r'\b([a-z]+)_([a-z]+)(?=\s+(?:features|shows|displays|contains|is|area|zone|scene))',
                        r'\1 \2', text, flags=re.IGNORECASE)

            return text

        except Exception as e:
            self.logger.warning(f"Error in scene type underscore cleanup: {str(e)}")
            return text

    def _enhance_input_cleaning(self, description: str) -> str:
        """
        增強輸入描述的清理功能

        Args:
            description: 待清理的描述

        Returns:
            str: 清理後的描述
        """
        if not description:
            return description

        try:
            import re

            # 預防性清理底線格式
            description = re.sub(r'\b(\w+)_(\w+)\b', r'\1 \2', description)

            # 清理可能導致語法問題的模式
            problematic_patterns = [
                (r'\s+,\s+', ', '),  # 修正空格-逗號問題
                (r'\bIn\s*,', 'In the area,'),  # 預防性修正
                (r'\s+\.', '.'),  # 修正句號前空格
            ]

            for pattern, replacement in problematic_patterns:
                description = re.sub(pattern, replacement, description)

            return description.strip()

        except Exception as e:
            self.logger.warning(f"Error in enhanced input cleaning: {str(e)}")
            return description

    def _setup_enhancement_template(self):
        """設置場景描述增強模板"""
        self.enhance_description_template = """
            <|system|>
            You are an expert visual analyst. Your task is to improve the readability and fluency of scene descriptions using STRICT factual accuracy.
            Your **top priority is to avoid hallucination** or fabrication. You are working in a computer vision pipeline using object detection (YOLO) and image embeddings. You MUST treat the input object list as a whitelist. Do not speculate beyond this list.
            </|system|>
            <|user|>
            Rewrite the following scene description to be fluent and clear. DO NOT add any objects, events, or spatial relationships that are not explicitly present in the original or object list.
            ORIGINAL:
            {original_description}
            CRITICAL RULES:
            1. CRITICAL ADHERENCE TO INPUT: Strictly adhere to the information explicitly provided in the ORIGINAL description and the {object_list}.
               a. NEVER assume or infer room types, object functions, scene purposes, or abstract conceptual zones (e.g., 'personal items zone', 'activity area') unless such concepts, along with their specific constituent objects and locations, are explicitly detailed in the ORIGINAL description or clearly supported by multiple items in the {object_list}.
               b. Your role is to rephrase and enhance the provided factual data, not to introduce new conceptual layers or interpretations not directly supported by the input.
            2. OBJECT WHITELIST & DETAIL ACCURACY:
               a. The provided {object_list} is an exhaustive list of objects confirmed by the vision system. Mention ONLY objects from this list or objects explicitly detailed in the ORIGINAL description.
               b. DO NOT invent additional objects or infer the presence of 'various scattered objects' if only a single specific item (e.g., one 'handbag') is mentioned in relation to a category or area. Describe only what is explicitly listed.
            3. NEVER speculate on object quantity. If the description says "10 people" , DO NOT say "dozens" or "many". Maintain the original quantity unless specified.
            4. SPATIAL ACCURACY - STRICTLY FROM ORIGINAL:
               a. Base ALL descriptions of object locations (e.g., 'foreground', 'background', 'middle center') and spatial relationships STRICTLY on the information explicitly provided in the ORIGINAL description.
               b. If the ORIGINAL description states an object is 'in the background,' use that exact term. If it specifies 'in the foreground,' use that. If it describes an object as being 'carried by a person', reflect this precise relationship.
               c. If the ORIGINAL description is less specific about an object's location (e.g., 'a car is present'), then use general, non-committal terms like 'visible in the scene' or 'present in the image.'
               d. DO NOT re-interpret object positions from any perceived understanding of the raw image; your sole source for spatial information is the ORIGINAL description. Do not relocate objects (e.g., moving a carried handbag from the person to 'the background').
            5. You MAY describe confirmed materials, colors, and composition style if visually obvious and non-speculative, AND if such details are hinted at or present in the ORIGINAL description or {object_list}.
            6. Write 2–4 complete, well-structured sentences with punctuation.
            7. Final output MUST be a single fluent paragraph of 60–200 words (not longer). Within this concise format, every sentence should aim to introduce new information or build upon previous statements without significant overlap.
            8. Begin your response directly with the scene description. Do NOT include any introductory phrases, explanations, or formatting indicators.
            9. Ensure grammatical completeness in all sentences. Each sentence must have a complete subject and predicate structure.
               a. NEVER use underscore formatting (e.g., tourist_landmark, urban_intersection). Always use natural spacing (tourist landmark, urban intersection).
               b. NEVER begin sentences with incomplete phrases like "In ," or "Overall," without proper subjects. Always ensure complete sentence structure.
               c. AVOID redundant or circular phrasing such as "with lights turned illuminating" or "atmosphere of is one of."
               d. If you encounter incomplete spatial descriptions like "visible in ," or "positioned in the middle of.", complete them naturally by adding appropriate context such as "visible in the scene" or "positioned in the middle of the frame", ensuring these completions are consistent with the ORIGINAL description. Always ensure spatial descriptions have complete prepositional phrases.
               e. GRAMMAR AND FLUENCY CHECK: Ensure all sentences are grammatically flawless and flow naturally. Avoid awkward phrasing or dangling prepositions (e.g., 'glow over ,'). Mentally re-read your generated description to catch and correct such minor errors before finalizing.
            10. Vary sentence structures naturally while maintaining grammatical accuracy.
            11. CRITICAL: Avoid repeating the mention of specific objects, groups of objects, or their spatial arrangements. Once an object or layout aspect is described, only refer to it again if providing genuinely NEW and DISTINCT information or a significantly different perspective that adds substantial value. Strive for conciseness and information density.
            12. Create natural spatial flow by connecting object descriptions organically rather than listing positions mechanically.
            13. Use transitional phrases to connect ideas smoothly, varying expression patterns throughout the description.
            14. For the concluding sentence, focus on the overall atmosphere, style, perceived activity, or overarching impression of the scene. DO NOT simply restate the primary objects or their layout as a summary or 'backdrop' if they have already been clearly described earlier in the paragraph. The conclusion should offer a higher-level takeaway.
            15. When describing quantities or arrangements, use only information explicitly confirmed by the object detection system or ORIGINAL description.
            </|user|>
            <|assistant|>
            """

    def _setup_verification_template(self):
        """設置檢測結果驗證模板"""
        self.verify_detection_template = """
            Task: You are an advanced vision system that verifies computer vision detections for accuracy.
            Analyze the following detection results and identify any potential errors or inconsistencies:
            SCENE TYPE: {scene_type}
            SCENE NAME: {scene_name}
            CONFIDENCE: {confidence:.2f}
            DETECTED OBJECTS: {detected_objects}
            CLIP ANALYSIS RESULTS:
            {clip_analysis}
            Possible Errors to Check:
            1. Objects misidentified (e.g., architectural elements labeled as vehicles)
            2. Cultural elements misunderstood (e.g., Asian temple structures labeled as boats)
            3. Objects that seem out of place for this type of scene
            4. Inconsistencies between different detection systems
            If you find potential errors, list them clearly with explanations. If the detections seem reasonable, state that they appear accurate.
            Verification Results:
            """

    def _setup_no_detection_template(self):
        """設置無檢測結果處理模板"""
        self.no_detection_template = """
            Task: You are an advanced scene understanding system analyzing an image where standard object detection failed to identify specific objects.
            Based on advanced image embeddings (CLIP analysis), we have the following information:
            MOST LIKELY SCENE: {top_scene} (confidence: {top_confidence:.2f})
            VIEWPOINT: {viewpoint}
            LIGHTING: {lighting_condition}
            CULTURAL ANALYSIS: {cultural_analysis}
            Create a detailed description of what might be in this scene, considering:
            1. The most likely type of location or setting
            2. Possible architectural or natural elements present
            3. The lighting and atmosphere
            4. Potential cultural or regional characteristics
            Your description should be natural, flowing, and offer insights into what the image likely contains despite the lack of specific object detection.
            Scene Description:
            """

    def format_enhancement_prompt(self, scene_data: Dict[str, Any], object_list: str, original_description: str) -> str:
        try:
            # 確保場景類型被正確清理
            scene_type = scene_data.get("scene_type", "unknown scene")
            cleaned_scene_type = self._clean_scene_type(scene_type)

            # 在原始描述中替換未清理的場景類型
            if scene_type != cleaned_scene_type:
                original_description = original_description.replace(scene_type, cleaned_scene_type)

            formatted_prompt = self.enhance_description_template.format(
                original_description=original_description,
                object_list=object_list
            )

            return formatted_prompt

        except Exception as e:
            self.logger.error(f"Failed to format enhancement prompt: {str(e)}")
            self.logger.error(traceback.format_exc())
            raise PromptTemplateError(f"Prompt formatting failed: {e}") from e


    def format_verification_prompt(self,
                                 detected_objects: List[Dict],
                                 clip_analysis: Dict[str, Any],
                                 scene_type: str,
                                 scene_name: str,
                                 confidence: float) -> str:
        """
        格式化檢測結果驗證提示

        Args:
            detected_objects: 檢測到的物件列表
            clip_analysis: CLIP分析結果
            scene_type: 場景類型
            scene_name: 場景名稱
            confidence: 場景分類信心度

        Returns:
            str: 格式化後的驗證提示字符串

        Raises:
            PromptTemplateError: 當模板格式化失敗時
        """
        try:
            self.logger.debug("Formatting verification prompt")

            # 格式化物件列表和CLIP分析結果
            objects_str = self._format_objects_for_prompt(detected_objects)
            clip_str = self._format_clip_results(clip_analysis)

            # 格式化提示
            formatted_prompt = self.verify_detection_template.format(
                scene_type=scene_type,
                scene_name=scene_name,
                confidence=confidence,
                detected_objects=objects_str,
                clip_analysis=clip_str
            )

            self.logger.debug(f"Verification prompt formatted successfully (length: {len(formatted_prompt)})")
            return formatted_prompt

        except Exception as e:
            error_msg = f"Failed to format verification prompt: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise PromptTemplateError(error_msg) from e

    def format_no_detection_prompt(self, clip_analysis: Dict[str, Any]) -> str:
        """
        格式化無檢測結果處理提示

        Args:
            clip_analysis: CLIP分析結果字典

        Returns:
            str: 格式化後的無檢測處理提示字符串

        Raises:
            PromptTemplateError: 當模板格式化失敗時
        """
        try:
            self.logger.debug("Formatting no-detection prompt")

            # 提取CLIP分析結果
            top_scene, top_confidence = clip_analysis.get("top_scene", ("unknown", 0))
            viewpoint = clip_analysis.get("viewpoint", ("standard", 0))[0]
            lighting = clip_analysis.get("lighting_condition", ("unknown", 0))[0]

            # 格式化文化分析
            cultural_str = self._format_cultural_analysis(clip_analysis.get("cultural_analysis", {}))

            # 格式化提示
            formatted_prompt = self.no_detection_template.format(
                top_scene=top_scene,
                top_confidence=top_confidence,
                viewpoint=viewpoint,
                lighting_condition=lighting,
                cultural_analysis=cultural_str
            )

            self.logger.debug(f"No-detection prompt formatted successfully (length: {len(formatted_prompt)})")
            return formatted_prompt

        except Exception as e:
            error_msg = f"Failed to format no-detection prompt: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise PromptTemplateError(error_msg) from e

    def _clean_scene_type(self, scene_type: str) -> str:
        """
        清理場景類型，使其更適合用於提示詞

        Args:
            scene_type: 原始場景類型

        Returns:
            str: 清理後的場景類型
        """
        if not scene_type:
            return "scene"

        # 將底線替換為空格並首字母大寫
        if '_' in scene_type:
            return ' '.join(word.capitalize() for word in scene_type.split('_'))

        return scene_type

    def _format_objects_for_prompt(self, objects: List[Dict]) -> str:
        """
        格式化物件列表以用於提示

        Args:
            objects: 檢測到的物件列表

        Returns:
            str: 格式化後的物件字符串
        """
        if not objects:
            return "No objects detected"

        try:
            formatted = []
            for obj in objects:
                class_name = obj.get("class_name", "unknown")
                confidence = obj.get("confidence", 0)
                formatted.append(f"{class_name} (confidence: {confidence:.2f})")

            return "\n- " + "\n- ".join(formatted)

        except Exception as e:
            self.logger.warning(f"Error formatting objects: {str(e)}")
            return "Object formatting error"

    def _format_clip_results(self, clip_analysis: Dict) -> str:
        """
        格式化CLIP分析結果以用於提示

        Args:
            clip_analysis: CLIP分析結果字典

        Returns:
            str: 格式化後的CLIP分析字符串
        """
        if not clip_analysis or "error" in clip_analysis:
            return "No CLIP analysis available"

        try:
            parts = ["CLIP Analysis Results:"]

            # 添加頂級場景
            top_scene, confidence = clip_analysis.get("top_scene", ("unknown", 0))
            parts.append(f"- Most likely scene: {top_scene} (confidence: {confidence:.2f})")

            # 添加視角
            viewpoint, vp_conf = clip_analysis.get("viewpoint", ("standard", 0))
            parts.append(f"- Camera viewpoint: {viewpoint} (confidence: {vp_conf:.2f})")

            # 添加物件組合
            if "object_combinations" in clip_analysis:
                combos = []
                for combo, score in clip_analysis["object_combinations"][:3]:
                    combos.append(f"{combo} ({score:.2f})")
                parts.append(f"- Object combinations: {', '.join(combos)}")

            # 添加文化分析
            if "cultural_analysis" in clip_analysis:
                parts.append("- Cultural analysis:")
                for culture_type, data in clip_analysis["cultural_analysis"].items():
                    best_desc = data.get("best_description", "")
                    desc_conf = data.get("confidence", 0)
                    parts.append(f"  * {culture_type}: {best_desc} ({desc_conf:.2f})")

            return "\n".join(parts)

        except Exception as e:
            self.logger.warning(f"Error formatting CLIP results: {str(e)}")
            return "CLIP analysis formatting error"

    def _format_cultural_analysis(self, cultural_analysis: Dict) -> str:
        """
        格式化文化分析結果

        Args:
            cultural_analysis: 文化分析結果字典

        Returns:
            str: 格式化後的文化分析字符串
        """
        if not cultural_analysis:
            return "No specific cultural elements detected"

        try:
            parts = []
            for culture_type, data in cultural_analysis.items():
                best_desc = data.get("best_description", "")
                desc_conf = data.get("confidence", 0)
                parts.append(f"{culture_type}: {best_desc} (confidence: {desc_conf:.2f})")

            return "\n".join(parts)

        except Exception as e:
            self.logger.warning(f"Error formatting cultural analysis: {str(e)}")
            return "Cultural analysis formatting error"

    def get_template_info(self) -> Dict[str, Any]:
        """
        獲取模板管理器的信息

        Returns:
            Dict[str, Any]: 包含模板數量和狀態的信息
        """
        return {
            "templates_count": 3,
            "available_templates": [
                "enhance_description_template",
                "verify_detection_template",
                "no_detection_template"
            ],
            "initialization_status": "success"
        }

Writing prompt_template_manager.py


In [ ]:
# %%writefile response_processor.py
import re
import logging
import traceback
from typing import Dict, List, Any, Optional, Set


class ResponseProcessingError(Exception):
    """回應處理相關錯誤的自定義異常"""
    pass


class ResponseProcessor:
    """
    負責處理和清理LLM模型輸出的回應。
    包含格式清理、重複內容檢測、語法完整性確保等功能。
    """

    def __init__(self):
        """初始化回應處理器"""
        # set the logger
        self.logger = logging.getLogger(self.__class__.__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

        # 初始化清理規則和替換字典
        self._initialize_cleaning_rules()
        self.logger.info("ResponseProcessor initialized successfully")


    def _initialize_cleaning_rules(self):
        """初始化各種清理規則和替換字典，把常見有問題情況優化"""
        try:
            # 設置重複詞彙的替換字典
            self.replacement_alternatives = {
                'visible': ['present', 'evident', 'apparent', 'observable'],
                'positioned': ['arranged', 'placed', 'set', 'organized'],
                'located': ['found', 'placed', 'situated', 'established'],
                'situated': ['placed', 'positioned', 'arranged', 'set'],
                'appears': ['seems', 'looks', 'presents', 'exhibits'],
                'features': ['includes', 'contains', 'displays', 'showcases'],
                'shows': ['reveals', 'presents', 'exhibits', 'demonstrates'],
                'displays': ['presents', 'exhibits', 'shows', 'reveals']
            }

            # 設置需要移除的前綴短語
            self.prefixes_to_remove = [
                "Here's the enhanced description:",
                "Enhanced description:",
                "Here is the enhanced scene description:",
                "I've enhanced the description while preserving all factual details:",
                "Enhanced Description:",
                "Scene Description:",
                "Description:",
                "Here is the enhanced description:",
                "Here's the enhanced description:",
                "Here is a rewritten scene description that adheres to the provided critical rules:",
                "Here is the rewritten scene description:",
                "Here's a rewritten scene description:",
                "The rewritten scene description is as follows:"
            ]

            # 設置需要移除的後綴短語
            self.suffixes_to_remove = [
                "I've maintained all the key factual elements",
                "I've preserved all the factual details",
                "All factual elements have been maintained"
            ]

            # 設置重複檢測模式
            self.repetitive_patterns = [
                (r'\b(visible)\b.*?\b(visible)\b', 'Multiple uses of "visible" detected'),
                (r'\b(positioned)\b.*?\b(positioned)\b', 'Multiple uses of "positioned" detected'),
                (r'\b(located)\b.*?\b(located)\b', 'Multiple uses of "located" detected'),
                (r'\b(situated)\b.*?\b(situated)\b', 'Multiple uses of "situated" detected'),
                (r'\b(appears)\b.*?\b(appears)\b', 'Multiple uses of "appears" detected'),
                (r'\b(features)\b.*?\b(features)\b', 'Multiple uses of "features" detected'),
                (r'\bThis\s+(\w+)\s+.*?\bThis\s+\1\b', 'Repetitive sentence structure detected')
            ]

            # 斜線組合的形容詞替換字典(有時會有斜線格式問題)
            self.slash_replacements = {
                'sunrise/sunset': 'warm lighting',
                'sunset/sunrise': 'warm lighting',
                'day/night': 'ambient lighting',
                'night/day': 'ambient lighting',
                'morning/evening': 'soft lighting',
                'evening/morning': 'soft lighting',
                'dawn/dusk': 'gentle lighting',
                'dusk/dawn': 'gentle lighting',
                'sunny/cloudy': 'natural lighting',
                'cloudy/sunny': 'natural lighting',
                'bright/dark': 'varied lighting',
                'dark/bright': 'varied lighting',
                'light/shadow': 'contrasting illumination',
                'shadow/light': 'contrasting illumination',
                'indoor/outdoor': 'mixed environment',
                'outdoor/indoor': 'mixed environment',
                'inside/outside': 'transitional space',
                'outside/inside': 'transitional space',
                'urban/rural': 'diverse landscape',
                'rural/urban': 'diverse landscape',
                'modern/traditional': 'architectural blend',
                'traditional/modern': 'architectural blend',
                'old/new': 'varied architecture',
                'new/old': 'varied architecture',
                'busy/quiet': 'dynamic atmosphere',
                'quiet/busy': 'dynamic atmosphere',
                'crowded/empty': 'varying occupancy',
                'empty/crowded': 'varying occupancy',
                'hot/cold': 'comfortable temperature',
                'cold/hot': 'comfortable temperature',
                'wet/dry': 'mixed conditions',
                'dry/wet': 'mixed conditions',
                'summer/winter': 'seasonal atmosphere',
                'winter/summer': 'seasonal atmosphere',
                'spring/autumn': 'transitional season',
                'autumn/spring': 'transitional season',
                'left/right': 'balanced composition',
                'right/left': 'balanced composition',
                'near/far': 'layered perspective',
                'far/near': 'layered perspective',
                'high/low': 'varied elevation',
                'low/high': 'varied elevation',
                'big/small': 'diverse scale',
                'small/big': 'diverse scale',
                'wide/narrow': 'varied width',
                'narrow/wide': 'varied width',
                'open/closed': 'flexible space',
                'closed/open': 'flexible space',
                'public/private': 'community space',
                'private/public': 'community space',
                'formal/informal': 'relaxed setting',
                'informal/formal': 'relaxed setting',
                'commercial/residential': 'mixed-use area',
                'residential/commercial': 'mixed-use area'
            }

            # 新增：擴展的底線替換字典
            self.underscore_replacements = {
                'urban_intersection': 'urban intersection',
                'tourist_landmark': 'tourist landmark',
                'historical_site': 'historical site',
                'religious_building': 'religious building',
                'natural_landmark': 'natural landmark',
                'commercial_area': 'commercial area',
                'residential_area': 'residential area',
                'public_space': 'public space',
                'outdoor_scene': 'outdoor scene',
                'indoor_scene': 'indoor scene',
                'street_scene': 'street scene',
                'city_center': 'city center',
                'shopping_district': 'shopping district',
                'business_district': 'business district',
                'traffic_light': 'traffic light',
                'street_lamp': 'street lamp',
                'parking_meter': 'parking meter',
                'fire_hydrant': 'fire hydrant',
                'bus_stop': 'bus stop',
                'train_station': 'train station',
                'police_car': 'police car',
                'fire_truck': 'fire truck',
                'school_bus': 'school bus',
                'time_of_day': 'time of day',
                'weather_condition': 'weather condition',
                'lighting_condition': 'lighting condition',
                'atmospheric_condition': 'atmospheric condition',
                'human_activity': 'human activity',
                'pedestrian_traffic': 'pedestrian traffic',
                'vehicle_traffic': 'vehicle traffic',
                'social_gathering': 'social gathering',
                'object_detection': 'object detection',
                'scene_analysis': 'scene analysis',
                'image_classification': 'image classification',
                'computer_vision': 'computer vision'
            }

            self.logger.info("Cleaning rules initialized successfully")

        except Exception as e:
            error_msg = f"Failed to initialize cleaning rules: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise ResponseProcessingError(error_msg) from e

    def clean_response(self, response: str, model_type: str = "general") -> str:
        """
        清理LLM回應

        Args:
            response: 原始LLM回應
            model_type: 模型類型（用於特定清理規則）

        Returns:
            str: 清理後的回應

        Raises:
            ResponseProcessingError: 當回應處理失敗時
        """
        if not response:
            raise ResponseProcessingError("Empty response provided for cleaning")

        try:
            self.logger.debug(f"Starting response cleaning (original length: {len(response)})")

            # 保存原始回應作為備份
            original_response = response

            # 根據模型類型選擇清理策略
            if "llama" in model_type.lower():
                cleaned_response = self._clean_llama_response(response)
            else:
                cleaned_response = self._clean_general_response(response)

            # 如果清理後內容過短，嘗試從原始回應中恢復
            if len(cleaned_response.strip()) < 40:
                self.logger.warning("Cleaned response too short, attempting recovery")
                cleaned_response = self._recover_from_overcleaning(original_response)

            # 最終驗證
            self._validate_cleaned_response(cleaned_response)

            self.logger.debug(f"Response cleaning completed (final length: {len(cleaned_response)})")
            return cleaned_response

        except Exception as e:
            error_msg = f"Response cleaning failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise ResponseProcessingError(error_msg) from e

    def _clean_llama_response(self, response: str) -> str:
        """
        專門處理Llama模型的回應清理

        Args:
            response: 原始Llama回應

        Returns:
            str: 清理後的回應
        """
        # 首先應用通用清理
        response = self._clean_general_response(response)

        # Llama特有的前綴清理
        llama_prefixes = [
            "Here's the enhanced description:",
            "Enhanced description:",
            "Here is the enhanced scene description:",
            "I've enhanced the description while preserving all factual details:"
        ]

        for prefix in llama_prefixes:
            if response.lower().startswith(prefix.lower()):
                response = response[len(prefix):].strip()

        # Llama特有的後綴清理
        llama_suffixes = [
            "I've maintained all the key factual elements",
            "I've preserved all the factual details",
            "All factual elements have been maintained"
        ]

        for suffix in llama_suffixes:
            if response.lower().endswith(suffix.lower()):
                response = response[:response.rfind(suffix)].strip()

        return response

    def _clean_general_response(self, response: str) -> str:
        """
        通用回應清理方法

        Args:
            response: 原始回應

        Returns:
            str: 清理後的回應
        """
        response = self._critical_format_preprocess(response)

        # 1. 移除系統remark
        response = self._remove_system_markers(response)

        # 2. 移除介紹性prefix
        response = self._remove_introduction_prefixes(response)

        # 3. 移除格式標記和上下文標籤
        response = self._remove_format_markers(response)

        # 4. 清理場景類型引用
        response = self._clean_scene_type_references(response)

        # 5. 標準化標點符號
        response = self._normalize_punctuation(response)

        # 6. 移除重複句子
        response = self._remove_duplicate_sentences(response)

        # 7. 處理重複詞彙
        response = self._handle_repetitive_vocabulary(response)

        # 8. ensure completement
        response = self._ensure_grammatical_completeness(response)

        # 9. 控制字數長度
        response = self._control_word_length(response)

        # 10. 最終格式化
        response = self._final_formatting(response)

        return response


    def _critical_format_preprocess(self, response: str) -> str:
        """
        關鍵格式預處理，處理最常見的格式問題

        Args:
            response: 原始回應

        Returns:
            str: 預處理後的回應
        """
        if not response:
            return response

        try:
            import re

            # 第一優先級：處理斜線問題
            # 首先處理已知的斜線組合，使用形容詞替換
            for slash_combo, replacement in self.slash_replacements.items():
                if slash_combo.lower() in response.lower():
                    # 保持原始大小寫格式
                    if slash_combo.upper() in response:
                        replacement_formatted = replacement.upper()
                    elif slash_combo.title() in response:
                        replacement_formatted = replacement.title()
                    else:
                        replacement_formatted = replacement

                    # 執行替換（不區分大小寫）
                    response = re.sub(re.escape(slash_combo), replacement_formatted, response, flags=re.IGNORECASE)
                    self.logger.debug(f"Replaced slash pattern '{slash_combo}' with '{replacement_formatted}'")

            # 處理其他未預定義的斜線模式
            # 標準斜線模式：word/word
            slash_pattern = r'\b([a-zA-Z]+)/([a-zA-Z]+)\b'
            matches = list(re.finditer(slash_pattern, response))
            for match in reversed(matches):  # 從後往前處理避免位置偏移
                word1, word2 = match.groups()
                # 選擇較短或更常見的詞作為替換
                if len(word1) <= len(word2):
                    replacement = word1
                else:
                    replacement = word2
                response = response[:match.start()] + replacement + response[match.end():]
                self.logger.debug(f"Replaced general slash pattern '{match.group(0)}' with '{replacement}'")

            # 第二優先級：處理底線格式
            # 首先處理已知的底線組合
            for underscore_combo, replacement in self.underscore_replacements.items():
                if underscore_combo in response:
                    response = response.replace(underscore_combo, replacement)
                    self.logger.debug(f"Replaced underscore pattern '{underscore_combo}' with '{replacement}'")

            # 處理三個詞的底線組合：word_word_word → word word word
            response = re.sub(r'\b([a-z]+)_([a-z]+)_([a-z]+)\b', r'\1 \2 \3', response)

            # 處理任何剩餘的底線模式：word_word → word word
            response = re.sub(r'\b([a-zA-Z]+)_([a-zA-Z]+)\b', r'\1 \2', response)

            # 第三優先級：修正不完整句子
            incomplete_sentence_fixes = [
                (r'\bIn\s*,\s*', 'Throughout the area, '),
                (r'\bOverall,\s+exudes\b', 'Overall, the scene exudes'),
                (r'\bThe overall atmosphere of\s+is\b', 'The overall atmosphere'),
                (r'\bwith its lights turned illuminating\b', 'with its lights illuminating'),
                (r'\bwhere it stands as\b', 'where it stands as'),
            ]

            for pattern, replacement in incomplete_sentence_fixes:
                response = re.sub(pattern, replacement, response, flags=re.IGNORECASE)

            # 第四優先級：語法修正處理(像是person and people)
            grammar_fixes = [
                (r'\b(\d+)\s+persons\b', r'\1 people'),
                (r'\bone\s+persons\b', 'one person'),
                (r'\btwo\s+persons\b', 'two people'),
                (r'\bthree\s+persons\b', 'three people'),
                (r'\bfour\s+persons\b', 'four people'),
                (r'\bfive\s+persons\b', 'five people'),
                (r'\bsix\s+persons\b', 'six people'),
                (r'\bseven\s+persons\b', 'seven people'),
                (r'\beight\s+persons\b', 'eight people'),
                (r'\bnine\s+persons\b', 'nine people'),
                (r'\bten\s+persons\b', 'ten people'),
                (r'\bmultiple\s+persons\b', 'multiple people'),
                (r'\bseveral\s+persons\b', 'several people'),
                (r'\bmany\s+persons\b', 'many people'),
                (r'\ba\s+few\s+persons\b', 'a few people'),
                (r'\bsome\s+persons\b', 'some people')
            ]

            for pattern, replacement in grammar_fixes:
                response = re.sub(pattern, replacement, response, flags=re.IGNORECASE)

            return response

        except Exception as e:
            self.logger.warning(f"Error in critical format preprocessing: {str(e)}")
            return response

    def _remove_system_markers(self, response: str) -> str:
        """移除系統樣式標記"""
        # 移除對話remark
        response = re.sub(r'<\|.*?\|>', '', response)

        # 移除輸出remark
        output_start = response.find("[OUTPUT_START]")
        output_end = response.find("[OUTPUT_END]")
        if output_start != -1 and output_end != -1 and output_end > output_start:
            response = response[output_start + len("[OUTPUT_START]"):output_end].strip()

        # 移除其他remark
        section_markers = [
            r'\[.*?\]',
            r'OUTPUT_START\s*:|OUTPUT_END\s*:',
            r'ENHANCED DESCRIPTION\s*:',
            r'Scene Type\s*:.*?(?=\n|$)',
            r'Original Description\s*:.*?(?=\n|$)',
            r'GOOD\s*:|BAD\s*:',
            r'PROBLEM\s*:.*?(?=\n|$)',
            r'</?\|(?:assistant|system|user)\|>',
            r'\(Note:.*?\)',
            r'\(.*?I\'ve.*?\)',
            r'\(.*?as per your request.*?\)'
        ]

        for marker in section_markers:
            response = re.sub(marker, '', response, flags=re.IGNORECASE)

        return response

    def _remove_introduction_prefixes(self, response: str) -> str:
        """移除介紹性前綴"""
        # 處理 "Here is..." 類型的prefix
        intro_prefixes = [
            r'^Here\s+is\s+(?:a\s+|the\s+)?(?:rewritten\s+|enhanced\s+)?scene\s+description.*?:\s*',
            r'^The\s+(?:rewritten\s+|enhanced\s+)?(?:scene\s+)?description\s+is.*?:\s*',
            r'^Here\'s\s+(?:a\s+|the\s+)?(?:rewritten\s+|enhanced\s+)?description.*?:\s*'
        ]

        for prefix_pattern in intro_prefixes:
            response = re.sub(prefix_pattern, '', response, flags=re.IGNORECASE)

        # 處理固定prefix
        for prefix in self.prefixes_to_remove:
            if response.lower().startswith(prefix.lower()):
                response = response[len(prefix):].strip()

        return response

    def _remove_format_markers(self, response: str) -> str:
        """移除格式標記和上下文標籤（保留括號內的地理與細節資訊）"""
        # 移除上下文相關remark
        response = re.sub(r'<\s*Context:.*?>', '', response)
        response = re.sub(r'Context:.*?(?=\n|$)', '', response)
        response = re.sub(r'Note:.*?(?=\n|$)', '', response, flags=re.IGNORECASE)

        # 移除Markdown格式
        response = re.sub(r'\*\*|\*|__|\|', '', response)

        # 移除任何剩餘的特殊標記 (避開括號內容，以免剔除地理位置等有用資訊)
        response = re.sub(r'</?\|.*?\|>', '', response)
        # ※ 以下移除「刪除整個括號及其內文」的方式已註解，以保留地理位置資訊
        # response = re.sub(r'\(.*?\)', '', response)

        return response


    def _clean_scene_type_references(self, response: str) -> str:
        """清理不當的場景類型引用"""
        scene_type_pattern = r'This ([a-zA-Z_]+) (features|shows|displays|contains)'
        match = re.search(scene_type_pattern, response)
        if match and '_' in match.group(1):
            fixed_text = f"This scene {match.group(2)}"
            response = re.sub(scene_type_pattern, fixed_text, response)

        return response

    def _normalize_punctuation(self, response: str) -> str:
        """標準化標點符號"""
        # 減少破折號使用
        response = re.sub(r'—', ', ', response)
        response = re.sub(r' - ', ', ', response)

        # 處理連續標點符號
        response = re.sub(r'([.,;:!?])\1+', r'\1', response)

        # 修復不完整句子的標點
        response = re.sub(r',\s*$', '.', response)

        # 修復句號後缺少空格的問題
        response = re.sub(r'([.!?])([A-Z])', r'\1 \2', response)

        # 清理多餘空格和換行
        response = response.replace('\r', ' ')
        response = re.sub(r'\n+', ' ', response)
        response = re.sub(r'\s{2,}', ' ', response)

        return response


    def _remove_duplicate_sentences(self, response: str, similarity_threshold: float = 0.85) -> str:
        """
        移除重複或高度相似的句子，使用 Jaccard 相似度進行比較。
        Args:
            response: 原始回應文本。
            similarity_threshold: 認定句子重複的相似度閾值 (0.0 到 1.0)。
                                  較高的閾值表示句子需要非常相似才會被移除。
        Returns:
            str: 移除重複句子後的文本。
        """
        try:
            if not response or not response.strip():
                return ""

            # (?<=[.!?]) 會保留分隔符在句尾, \s+ 會消耗句尾的空格
            # 這樣用 ' ' join 回去時, 標點和下個句子間剛好一個空格
            sentences = re.split(r'(?<=[.!?])\s+', response.strip())

            unique_sentences_data = [] # Store tuples of (original_sentence, simplified_word_set)

            min_sentence_len_for_check = 8 # 簡化後詞彙數少於此值，除非完全相同否則不輕易判斷為重複

            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue

                # 創建簡化版本用於比較 (小寫，移除標點，分割為詞彙集合)
                # 保留數字，因為數字可能是關鍵資訊
                simplified_text = re.sub(r'[^\w\s\d]', '', sentence.lower())
                current_sentence_words = set(simplified_text.split())

                if not current_sentence_words: # 如果處理後是空集合，跳過
                    continue

                is_duplicate = False
                # 與已保留的唯一句子比較
                for i, (kept_sentence_text, kept_sentence_words) in enumerate(unique_sentences_data):
                    # Jaccard Index
                    intersection_len = len(current_sentence_words.intersection(kept_sentence_words))
                    union_len = len(current_sentence_words.union(kept_sentence_words))

                    if union_len == 0: # 兩個都是空集合，代表相同句子
                        jaccard_similarity = 1.0
                    else:
                        jaccard_similarity = intersection_len / union_len

                    # 用Jaccard 相似度超過閾值，不是兩個都非常短的句子 (避免 "Yes." 和 "No." 被錯誤合併)
                    # 新句子完全被舊句子包含 (且舊句子更長)
                    # 舊句子完全被新句子包含 (且新句子更長) -> 這種情況就需要替換
                    if jaccard_similarity >= similarity_threshold:
                        # 如果當前句子比已保留的句子短，且高度相似，則認為是重複
                        if len(current_sentence_words) < len(kept_sentence_words):
                            is_duplicate = True
                            self.logger.debug(f"Sentence \"{sentence[:30]}...\" marked duplicate (shorter, similar to \"{kept_sentence_text[:30]}...\") Jaccard: {jaccard_similarity:.2f}")
                            break
                        # 如果當前句子比已保留的句子長，且高度相似，則替換掉已保留的
                        elif len(current_sentence_words) > len(kept_sentence_words):
                            self.logger.debug(f"Sentence \"{kept_sentence_text[:30]}...\" replaced by longer similar sentence \"{sentence[:30]}...\" Jaccard: {jaccard_similarity:.2f}")
                            unique_sentences_data.pop(i) # 移除舊的、較短的句子

                        # 如果長度差不多，但相似度高，保留第一個出現的
                        elif current_sentence_words != kept_sentence_words : # 避免完全相同的句子被錯誤地跳過替換邏輯
                             is_duplicate = True # 保留先出現的
                             self.logger.debug(f"Sentence \"{sentence[:30]}...\" marked duplicate (similar length, similar to \"{kept_sentence_text[:30]}...\") Jaccard: {jaccard_similarity:.2f}")
                             break

                if not is_duplicate:
                    unique_sentences_data.append((sentence, current_sentence_words))

            # 重組唯一句子
            final_sentences = [s_data[0] for s_data in unique_sentences_data]

            # 確保每個句子以標點結尾 (因為 split 可能會產生沒有標點的最後一個片段)
            reconstructed_response = ""
            for i, s in enumerate(final_sentences):
                s = s.strip()
                if not s: continue
                if not s[-1] in ".!?":
                    s += "."
                reconstructed_response += s
                if i < len(final_sentences) - 1:
                     reconstructed_response += " " # 在句子間添加空格

            return reconstructed_response.strip()

        except Exception as e:
            self.logger.error(f"Error in _remove_duplicate_sentences: {str(e)}")
            self.logger.error(traceback.format_exc())
            return response # 發生錯誤時返回原始回應

    def _handle_repetitive_vocabulary(self, response: str) -> str:
        """處理重複詞彙，使用 re.sub 和可呼叫的替換函數以提高效率和準確性。"""
        try:
            # 檢測重複模式 (僅警告)
            if hasattr(self, 'repetitive_patterns'):
                for pattern, issue in self.repetitive_patterns:
                    if re.search(pattern, response, re.IGNORECASE | re.DOTALL):
                        self.logger.warning(f"Text quality issue detected: {issue} in response: \"{response[:100]}...\"")

            if not hasattr(self, 'replacement_alternatives') or not self.replacement_alternatives:
                return response

            processed_response = response

            for word_to_replace, alternatives in self.replacement_alternatives.items():
                if not alternatives:  # 如果沒有可用的替代詞，則跳過
                    continue

                # 為每個詞創建一個獨立的計數器和替代索引
                # 使用閉包或一個小類來封裝狀態
                class WordReplacer:
                    def __init__(self, alternatives_list):
                        self.count = 0
                        self.alternative_idx = 0
                        self.alternatives_list = alternatives_list

                    def __call__(self, match_obj):
                        self.count += 1
                        original_word = match_obj.group(0)
                        if self.count > 1:  # 從第二次出現開始替換
                            replacement = self.alternatives_list[self.alternative_idx % len(self.alternatives_list)]
                            self.alternative_idx += 1
                            # 保持原始大小寫格式
                            if original_word.isupper():
                                return replacement.upper()
                            elif original_word.istitle():
                                return replacement.capitalize()
                            return replacement
                        return original_word # 因為第一次出現, 就不用替換

                replacer_instance = WordReplacer(alternatives)
                # 使用 \b 確保匹配的是整個單詞
                pattern = re.compile(r'\b' + re.escape(word_to_replace) + r'\b', re.IGNORECASE)
                processed_response = pattern.sub(replacer_instance, processed_response)

            return processed_response

        except Exception as e:
            self.logger.error(f"Error in _handle_repetitive_vocabulary: {str(e)}")
            self.logger.error(traceback.format_exc())
            return response # 發生錯誤時返回原始回應

    def _ensure_grammatical_completeness(self, response: str) -> str:
        """
        確保語法完整性，處理不完整句子和格式問題

        Args:
            response: 待檢查的回應文本

        Returns:
            str: 語法完整的回應文本
        """
        try:
            if not response or not response.strip():
                return response

            # 第一階段：檢查並修正不完整的句子模式
            incomplete_patterns = [
                # 介詞後直接結束的問題（針對 "over ." 等情況）
                (r'\b(over|under|through|across|along|beneath|beyond|throughout)\s*\.', 'incomplete_preposition'),
                (r'\b(with|without|against|towards|beside|between|among)\s*\.', 'incomplete_preposition'),
                (r'\b(into|onto|upon|within|behind|below|above)\s*\.', 'incomplete_preposition'),

                # 處理 "In ," 這類缺失詞彙的問題
                (r'\bIn\s*,', 'incomplete_location'),
                (r'\bAt\s*,', 'incomplete_location'),
                (r'\bOn\s*,', 'incomplete_location'),
                (r'\bWith\s*,', 'incomplete_context'),

                # 不完整的描述模式
                (r'\b(fine|the)\s+(the\s+)?(?:urban|area|scene)\b(?!\s+\w)', 'incomplete_description'),

                # 連詞或介詞後直接標點的問題
                (r'\b(and|or|but|with|from|in|at|on|by|for|to)\s*[.!?]', 'incomplete_conjunction'),

                # 重複詞彙
                (r'\b(\w+)\s+\1\b', 'word_repetition'),

                # 不完整的場景類型引用（如 "urban_intersection" 格式問題）
                (r'\b(\w+)_(\w+)\b', 'underscore_format'),

                # 地標場景特有問題
                (r'\btourist_landmark\b', 'underscore_format'),
                (r'\burban_intersection\b', 'underscore_format'),
                (r'\bIn\s*,\s*(?=\w)', 'incomplete_prepositional'),
                (r'\bOverall,\s+(?=exudes|shows|displays)(?!\s+(?:the|this|it))', 'missing_subject'),
                (r'\batmosphere of\s+is one of\b', 'redundant_structure'),
                (r'\bwith.*?turned\s+illuminating\b', 'redundant_participle')
            ]

            for pattern, issue_type in incomplete_patterns:
                try:
                    matches = list(re.finditer(pattern, response, re.IGNORECASE))

                    for match in matches:
                        if issue_type == 'incomplete_preposition':
                            # 處理介詞後直接結束的情況
                            response = self._fix_incomplete_preposition(response, match)

                        elif issue_type == 'underscore_format':
                            # 將下劃線格式轉換為空格分隔
                            original = match.group(0)
                            replacement = original.replace('_', ' ')
                            response = response.replace(original, replacement)

                        elif issue_type == 'word_repetition':
                            # 移除重複的詞彙
                            repeated_word = match.group(1)
                            response = response.replace(f"{repeated_word} {repeated_word}", repeated_word)

                        elif issue_type == 'incomplete_location' or issue_type == 'incomplete_context':
                            # 移除不完整的位置或上下文引用
                            response = response.replace(match.group(0), '')

                        elif issue_type == 'incomplete_prepositional':
                            # 處理不完整的介詞短語
                            response = re.sub(r'\bIn\s*,\s*', 'Throughout the scene, ', response)

                        elif issue_type == 'missing_subject':
                            # 為Overall句子添加主語
                            response = re.sub(r'\bOverall,\s+(?=exudes)', 'Overall, the scene ', response)

                        elif issue_type == 'redundant_structure':
                            # 簡化冗餘結構
                            response = re.sub(r'\batmosphere of\s+is one of\b', 'atmosphere is one of', response)

                        elif issue_type == 'redundant_participle':
                            # 清理冗餘分詞
                            response = re.sub(r'turned\s+illuminating', 'illuminating', response)

                        else:
                            # 對於其他不完整模式，直接移除
                            response = response.replace(match.group(0), '')

                    # 清理多餘空格
                    response = re.sub(r'\s{2,}', ' ', response).strip()

                except re.error as e:
                    self.logger.warning(f"Regular expression pattern error for {issue_type}: {pattern} - {str(e)}")
                    continue

            # 第二階段：處理物件類別格式問題
            response = self._clean_object_class_references(response)

            # 第三階段：確保句子正確結束
            response = self._ensure_proper_sentence_ending(response)

            # 第四階段：最終語法檢查
            response = self._final_grammar_check(response)

            return response.strip()

        except Exception as e:
            self.logger.error(f"Error in _ensure_grammatical_completeness: {str(e)}")
            return response

    def _fix_incomplete_preposition(self, response: str, match) -> str:
        """
        修正不完整的介詞短語

        Args:
            response: 回應文本
            match: 正則匹配對象

        Returns:
            str: 修正後的回應
        """
        preposition = match.group(1)
        match_start = match.start()

        # 找到句子的開始位置
        sentence_start = response.rfind('.', 0, match_start)
        sentence_start = sentence_start + 1 if sentence_start != -1 else 0

        # 提取句子片段
        sentence_fragment = response[sentence_start:match_start].strip()

        # 如果句子片段有意義，嘗試移除不完整的介詞部分
        if len(sentence_fragment) > 10:
            # 移除介詞及其後的內容，添加適當的句號
            response = response[:match_start].rstrip() + '.'
        else:
            # 如果句子片段太短，移除整個不完整的句子
            response = response[:sentence_start] + response[match.end():]

        return response

    def _clean_object_class_references(self, response: str) -> str:
        """
        清理物件類別引用中的格式問題

        Args:
            response: 回應文本

        Returns:
            str: 清理後的回應
        """
        # 移除類別ID引用（如 "unknown-class 2", "Class 0" 等）
        class_id_patterns = [
            r'\bunknown[- ]?class\s*\d+\s*objects?',
            r'\bclass[- ]?\d+\s*objects?',
            r'\b[Cc]lass\s*\d+\s*objects?',
            r'\bunknown[- ][Cc]lass\s*\d+\s*objects?'
        ]

        for pattern in class_id_patterns:
            try:
                # 替換為更自然的描述
                response = re.sub(pattern, 'objects', response, flags=re.IGNORECASE)
            except re.error as e:
                self.logger.warning(f"Error cleaning class reference pattern {pattern}: {str(e)}")
                continue

        # 處理數量描述中的問題
        response = re.sub(r'\b(\w+)\s+unknown[- ]?\w*\s*objects?', r'\1 objects', response, flags=re.IGNORECASE)

        return response

    def _ensure_proper_sentence_ending(self, response: str) -> str:
        """
        確保句子有適當的結尾

        Args:
            response: 回應文本

        Returns:
            str: 具有適當結尾的回應
        """
        if not response or not response.strip():
            return response

        response = response.strip()

        # 檢查是否以標點符號結尾
        if response and response[-1] not in ['.', '!', '?']:

            # 常見介詞和連詞列表
            problematic_endings = [
                "into", "onto", "about", "above", "across", "after", "along", "around",
                "at", "before", "behind", "below", "beneath", "beside", "between",
                "beyond", "by", "down", "during", "except", "for", "from", "in",
                "inside", "near", "of", "off", "on", "over", "through", "to",
                "toward", "under", "up", "upon", "with", "within", "and", "or", "but"
            ]

            words = response.split()
            if words:
                last_word = words[-1].lower().rstrip('.,!?')

                if last_word in problematic_endings:
                    # 找到最後完整的句子
                    last_period_pos = max(
                        response.rfind('.'),
                        response.rfind('!'),
                        response.rfind('?')
                    )

                    if last_period_pos > len(response) // 2:  # 如果有較近的完整句子
                        response = response[:last_period_pos + 1]
                    else:
                        # 移除問題詞彙並添加句號
                        if len(words) > 1:
                            response = " ".join(words[:-1]) + "."
                        else:
                            response = "The scene displays various elements."
                else:
                    # 正常情況下添加句號
                    response += "."

        return response

    def _final_grammar_check(self, response: str) -> str:
        """
        最終語法檢查和清理

        Args:
            response: 回應文本

        Returns:
            str: 最終清理後的回應
        """
        if not response:
            return response

        # 修正連續標點符號
        response = re.sub(r'([.!?]){2,}', r'\1', response)

        # 修正句號前的空格
        response = re.sub(r'\s+([.!?])', r'\1', response)

        # 修正句號後缺少空格的問題
        response = re.sub(r'([.!?])([A-Z])', r'\1 \2', response)

        # 確保首字母大寫
        if response and response[0].islower():
            response = response[0].upper() + response[1:]

        # 移除多餘的空格
        response = re.sub(r'\s{2,}', ' ', response)

        # 處理空句子或過短的回應
        if len(response.strip()) < 20:
            return "The scene contains various visual elements."

        return response.strip()

    def _control_word_length(self, response: str) -> str:
        """控制文字長度在合理範圍內"""
        words = response.split()
        if len(words) > 200:
            # 找到接近字數限制的句子結束處
            truncated = ' '.join(words[:200])
            last_period = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))

            if last_period > 0:
                response = truncated[:last_period+1]
            else:
                response = truncated + "."

        return response

    def _final_formatting(self, response: str) -> str:
        """最終格式化處理"""
        # 確保首字母大寫
        if response and response[0].islower():
            response = response[0].upper() + response[1:]

        # 統一格式為單一段落
        response = re.sub(r'\s*\n\s*', ' ', response)
        response = ' '.join(response.split())

        return response.strip()

    def _recover_from_overcleaning(self, original_response: str) -> str:
        """從過度清理中恢復內容"""
        try:
            # 嘗試從原始回應中找到最佳段落
            paragraphs = [p for p in original_response.split('\n\n') if p.strip()]
            if paragraphs:
                # 選擇最長的段落作為主要描述
                best_para = max(paragraphs, key=len)
                # 使用基本清理規則
                best_para = re.sub(r'\[.*?\]', '', best_para)
                best_para = re.sub(r'\s{2,}', ' ', best_para).strip()

                if len(best_para) >= 40:
                    return best_para

            return "Unable to generate a valid enhanced description."

        except Exception as e:
            self.logger.error(f"Recovery from overcleaning failed: {str(e)}")
            return "Description generation error."

    def _validate_cleaned_response(self, response: str):
        """驗證清理後的回應"""
        if not response:
            raise ResponseProcessingError("Response is empty after cleaning")

        if len(response.strip()) < 20:
            raise ResponseProcessingError("Response is too short after cleaning")

        # 檢查是否包含基本的句子結構
        if not re.search(r'[.!?]', response):
            raise ResponseProcessingError("Response lacks proper sentence structure")

    def remove_explanatory_notes(self, response: str) -> str:
        """
        移除解釋性注釋和說明

        Args:
            response: 包含可能注釋的回應

        Returns:
            str: 移除注釋後的回應
        """
        try:
            # 識別常見的注釋和解釋模式
            note_patterns = [
                r'(?:^|\n)Note:.*?(?:\n|$)',
                r'(?:^|\n)I have (?:followed|adhered to|ensured).*?(?:\n|$)',
                r'(?:^|\n)This description (?:follows|adheres to|maintains).*?(?:\n|$)',
                r'(?:^|\n)The enhanced description (?:maintains|preserves).*?(?:\n|$)'
            ]

            # 尋找段落
            paragraphs = [p.strip() for p in response.split('\n\n') if p.strip()]

            # 如果只有一個段落，檢查並清理它
            if len(paragraphs) == 1:
                for pattern in note_patterns:
                    paragraphs[0] = re.sub(pattern, '', paragraphs[0], flags=re.IGNORECASE)
                return paragraphs[0].strip()

            # 如果有多個段落，移除注釋段落
            content_paragraphs = []
            for paragraph in paragraphs:
                is_note = False
                for pattern in note_patterns:
                    if re.search(pattern, paragraph, flags=re.IGNORECASE):
                        is_note = True
                        break

                # 檢查段落是否以常見的注釋詞開頭
                if paragraph.lower().startswith(('note:', 'please note:', 'remember:')):
                    is_note = True

                if not is_note:
                    content_paragraphs.append(paragraph)

            return '\n\n'.join(content_paragraphs).strip()

        except Exception as e:
            self.logger.error(f"Failed to remove explanatory notes: {str(e)}")
            return response

    def get_processor_info(self) -> Dict[str, Any]:
        """
        獲取處理器信息

        Returns:
            Dict[str, Any]: 包含處理器狀態和配置的信息
        """
        return {
            "replacement_alternatives_count": len(self.replacement_alternatives),
            "prefixes_to_remove_count": len(self.prefixes_to_remove),
            "suffixes_to_remove_count": len(self.suffixes_to_remove),
            "repetitive_patterns_count": len(self.repetitive_patterns),
            "initialization_status": "success"
        }

Writing response_processor.py


In [ ]:
# %%writefile text_quality_validator.py
import re
import logging
import traceback
from typing import Dict, List, Any, Optional, Set, Tuple


class TextQualityValidator:
    """
    負責驗證和確保生成文本的品質和事實準確性。
    包含事實檢查、視角一致性、場景類型一致性等驗證功能。
    """

    def __init__(self):
        """初始化文本品質驗證器"""
        # 設置專屬logger
        self.logger = logging.getLogger(self.__class__.__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

        # 初始化驗證規則
        self._initialize_validation_rules()
        self.logger.info("TextQualityValidator initialized successfully")

    def _initialize_validation_rules(self):
        """初始化各種驗證規則和詞彙庫"""
        try:
            # 地點和文化詞彙列表
            self.location_terms = ["plaza", "square", "market", "mall", "avenue", "boulevard"]
            self.cultural_terms = ["european", "asian", "american", "african", "western", "eastern"]

            # 視角詞彙對應表
            self.perspective_terms = {
                "aerial": ["aerial", "bird's-eye", "overhead", "top-down", "above", "looking down"],
                "ground": ["street-level", "ground level", "eye-level", "standing"],
                "indoor": ["inside", "interior", "indoor", "within"],
                "close-up": ["close-up", "detailed view", "close shot"]
            }

            # 視角前綴對應表
            self.perspective_prefixes = {
                "aerial": "From an aerial perspective, ",
                "ground": "From street level, ",
                "indoor": "In this indoor setting, ",
                "close-up": "In this close-up view, "
            }

            # 數值檢測模式
            self.number_patterns = [
                (r'(\d+)\s+(people|person|pedestrians|individuals)', r'\1', r'\2'),
                (r'(\d+)\s+(cars|vehicles|automobiles)', r'\1', r'\2'),
                (r'(\d+)\s+(buildings|structures)', r'\1', r'\2'),
                (r'(\d+)\s+(plants|potted plants|flowers)', r'\1', r'\2'),
                (r'(\d+)\s+(beds|furniture|tables|chairs)', r'\1', r'\2')
            ]

            # 禁用場景詞列表
            self.prohibited_scene_words = ["plaza", "square", "european", "asian", "american"]

            self.logger.info("Validation rules initialized successfully")

        except Exception as e:
            error_msg = f"Failed to initialize validation rules: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise Exception(error_msg) from e

    def verify_factual_accuracy(self,
                               original_desc: str,
                               generated_desc: str,
                               object_list: str) -> str:
        """
        驗證生成描述的事實準確性

        Args:
            original_desc: 原始場景描述
            generated_desc: 生成的描述
            object_list: 檢測到的物件列表

        Returns:
            str: 驗證並可能修正後的描述
        """
        try:
            self.logger.debug("Starting factual accuracy verification")

            # 將原始描述和物體列表合併為授權詞彙源
            authorized_content = original_desc.lower() + " " + object_list.lower()

            # 檢查和替換未授權的地點和文化詞彙
            verified_desc = self._check_unauthorized_terms(generated_desc, authorized_content)

            # 檢查重複用詞問題
            verified_desc = self._detect_repetitive_patterns(verified_desc)

            self.logger.debug("Factual accuracy verification completed")
            return verified_desc

        except Exception as e:
            error_msg = f"Factual accuracy verification failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return generated_desc  # 發生錯誤時返回原始生成描述

    def _check_unauthorized_terms(self, generated_desc: str, authorized_content: str) -> str:
        """檢查並替換未授權的詞彙"""
        # 檢查生成文本中的每個詞
        for term in self.location_terms + self.cultural_terms:
            # 僅當該詞出現在生成文本但不在授權內容中時進行替換
            if term in generated_desc.lower() and term not in authorized_content:
                # 根據詞語類型選擇適當的替換詞
                if term in self.location_terms:
                    replacement = "area"
                else:
                    replacement = "scene"

                # 使用正則表達式進行完整詞匹配替換
                pattern = re.compile(r'\b' + term + r'\b', re.IGNORECASE)
                generated_desc = pattern.sub(replacement, generated_desc)

        return generated_desc

    def _detect_repetitive_patterns(self, generated_desc: str) -> str:
        """檢測並處理重複用詞問題"""
        repetitive_patterns = [
            (r'\b(visible)\b.*?\b(visible)\b', 'Multiple uses of "visible" detected'),
            (r'\b(positioned)\b.*?\b(positioned)\b', 'Multiple uses of "positioned" detected'),
            (r'\b(located)\b.*?\b(located)\b', 'Multiple uses of "located" detected'),
            (r'\b(situated)\b.*?\b(situated)\b', 'Multiple uses of "situated" detected'),
            (r'\b(appears)\b.*?\b(appears)\b', 'Multiple uses of "appears" detected'),
            (r'\b(features)\b.*?\b(features)\b', 'Multiple uses of "features" detected'),
            (r'\bThis\s+(\w+)\s+.*?\bThis\s+\1\b', 'Repetitive sentence structure detected')
        ]

        # 替換詞典
        replacement_dict = {
            'visible': ['present', 'evident', 'apparent', 'observable'],
            'positioned': ['arranged', 'placed', 'set', 'organized'],
            'located': ['found', 'placed', 'situated', 'established'],
            'situated': ['placed', 'positioned', 'arranged', 'set'],
            'appears': ['seems', 'looks', 'presents', 'exhibits'],
            'features': ['includes', 'contains', 'displays', 'showcases']
        }

        for pattern, issue in repetitive_patterns:
            matches = list(re.finditer(pattern, generated_desc, re.IGNORECASE | re.DOTALL))
            if matches:
                self.logger.warning(f"Text quality issue detected: {issue}")

                # 針對特定重複詞彙進行替換
                for word in replacement_dict.keys():
                    if word in issue.lower():
                        word_pattern = re.compile(r'\b' + word + r'\b', re.IGNORECASE)
                        word_matches = list(word_pattern.finditer(generated_desc))

                        # 保留第一次出現，替換後續出現
                        for i, match in enumerate(word_matches[1:], 1):
                            if i <= len(replacement_dict[word]):
                                replacement = replacement_dict[word][(i-1) % len(replacement_dict[word])]

                                # 保持原始大小寫格式
                                if match.group().isupper():
                                    replacement = replacement.upper()
                                elif match.group().istitle():
                                    replacement = replacement.capitalize()

                                # 執行替換
                                generated_desc = generated_desc[:match.start()] + replacement + generated_desc[match.end():]
                                # 重新計算後續匹配位置
                                word_matches = list(word_pattern.finditer(generated_desc))
                        break

        return generated_desc

    def fact_check_description(self,
                             original_desc: str,
                             enhanced_desc: str,
                             scene_type: str,
                             detected_objects: List[str]) -> str:
        """
        對增強後的描述進行全面的事實檢查

        Args:
            original_desc: 原始場景描述
            enhanced_desc: 增強後的描述
            scene_type: 場景類型
            detected_objects: 檢測到的物體名稱列表

        Returns:
            str: 經過事實檢查的描述
        """
        try:
            self.logger.debug("Starting comprehensive fact checking")

            # 如果增強描述為空或太短，返回原始描述
            if not enhanced_desc or len(enhanced_desc) < 30:
                return original_desc

            # 1. 檢查數值一致性
            enhanced_desc = self._check_numerical_consistency(original_desc, enhanced_desc)

            # 2. 檢查視角一致性
            enhanced_desc = self._check_perspective_consistency(original_desc, enhanced_desc)

            # 3. 檢查場景類型一致性
            enhanced_desc = self._check_scene_type_consistency(enhanced_desc, scene_type)

            # 4. 確保文字長度適當
            enhanced_desc = self._ensure_appropriate_length(enhanced_desc)

            self.logger.debug("Comprehensive fact checking completed")
            return enhanced_desc

        except Exception as e:
            error_msg = f"Fact checking failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return enhanced_desc  # 發生錯誤時返回增強描述

    def _check_numerical_consistency(self, original_desc: str, enhanced_desc: str) -> str:
        """檢查數值一致性"""
        # 檢查原始描述中的每個數字
        for pattern, num_group, word_group in self.number_patterns:
            original_matches = re.finditer(pattern, original_desc, re.IGNORECASE)
            for match in original_matches:
                number = match.group(1)
                noun = match.group(2)

                # 檢查增強描述中是否保留了這個數字
                enhanced_pattern = r'(\d+)\s+(' + re.escape(noun) + r'|' + re.escape(noun.rstrip('s')) + r'|' + re.escape(noun + 's') + r')'
                enhanced_matches = list(re.finditer(enhanced_pattern, enhanced_desc, re.IGNORECASE))

                if not enhanced_matches:
                    # 數字+名詞未在增強描述中找到
                    plural_form = noun if noun.endswith('s') or number == '1' else noun + 's'
                    if enhanced_desc.startswith("This") or enhanced_desc.startswith("The"):
                        enhanced_desc = enhanced_desc.replace("This ", f"This scene with {number} {plural_form} ", 1)
                        enhanced_desc = enhanced_desc.replace("The ", f"The scene with {number} {plural_form} ", 1)
                    else:
                        enhanced_desc = f"The scene includes {number} {plural_form}. " + enhanced_desc
                elif enhanced_matches and enhanced_matches[0].group(1) != number:
                    # 存在但數字不一致，需要更正數字
                    for ematch in enhanced_matches:
                        wrong_number = ematch.group(1)
                        enhanced_desc = enhanced_desc.replace(f"{wrong_number} {ematch.group(2)}", f"{number} {ematch.group(2)}")

        return enhanced_desc

    def _check_perspective_consistency(self, original_desc: str, enhanced_desc: str) -> str:
        """檢查視角一致性"""
        # 確定原始視角
        original_perspective = None
        for persp, terms in self.perspective_terms.items():
            if any(term in original_desc.lower() for term in terms):
                original_perspective = persp
                break

        # 檢查是否保留了視角
        if original_perspective:
            enhanced_has_perspective = any(term in enhanced_desc.lower() for term in self.perspective_terms[original_perspective])

            if not enhanced_has_perspective:
                # 添加缺失的視角
                prefix = self.perspective_prefixes.get(original_perspective, "")
                if prefix:
                    if enhanced_desc[0].isupper():
                        enhanced_desc = prefix + enhanced_desc[0].lower() + enhanced_desc[1:]
                    else:
                        enhanced_desc = prefix + enhanced_desc

        return enhanced_desc

    def _check_scene_type_consistency(self, enhanced_desc: str, scene_type: str) -> str:
        """檢查場景類型一致性"""
        if scene_type and scene_type.lower() != "unknown" and scene_type.lower() not in enhanced_desc.lower():
            # 添加場景類型
            if enhanced_desc.startswith("This ") or enhanced_desc.startswith("The "):
                # 避免產生重複
                if "scene" in enhanced_desc[:15].lower():
                    fixed_type = scene_type.lower()
                    enhanced_desc = enhanced_desc.replace("scene", fixed_type, 1)
                else:
                    enhanced_desc = enhanced_desc.replace("This ", f"This {scene_type} ", 1)
                    enhanced_desc = enhanced_desc.replace("The ", f"The {scene_type} ", 1)
            else:
                enhanced_desc = f"This {scene_type} " + enhanced_desc

        return enhanced_desc

    def _ensure_appropriate_length(self, enhanced_desc: str) -> str:
        """確保文字長度適當"""
        words = enhanced_desc.split()
        if len(words) > 200:
            # 找尋接近字數限制的句子結束處
            truncated = ' '.join(words[:200])
            last_period = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))

            if last_period > 0:
                enhanced_desc = truncated[:last_period+1]
            else:
                enhanced_desc = truncated + '.'

        return enhanced_desc

    def ensure_scene_type_consistency(self,
                                    description: str,
                                    scene_type: str,
                                    original_desc: str) -> str:
        """
        確保描述中的場景類型與指定的場景類型一致

        Args:
            description: 待檢查的描述
            scene_type: 指定的場景類型
            original_desc: 原始描述（用於參考）

        Returns:
            str: 場景類型一致的描述
        """
        try:
            self.logger.debug("Ensuring scene type consistency")
            scene_type = scene_type.replace('_', ' ')
            # 檢查是否包含禁止的場景詞
            for word in self.prohibited_scene_words:
                if word in description.lower() and word not in original_desc.lower() and word not in scene_type.lower():
                    # 替換錯誤場景詞為正確場景類型
                    pattern = re.compile(r'\b' + word + r'\b', re.IGNORECASE)
                    description = pattern.sub(scene_type, description)

            # 確保場景類型在描述中被提及
            if scene_type.lower() not in description.lower():
                # 尋找通用場景詞並替換
                for general_term in ["scene", "area", "place", "location"]:
                    if general_term in description.lower():
                        pattern = re.compile(r'\b' + general_term + r'\b', re.IGNORECASE)
                        description = pattern.sub(scene_type, description, count=1)
                        break
                else:
                    # 如果沒有找到通用詞，在開頭添加場景類型
                    if description.startswith("The "):
                        description = description.replace("The ", f"The {scene_type} ", 1)
                    elif description.startswith("This "):
                        description = description.replace("This ", f"This {scene_type} ", 1)
                    else:
                        description = f"This {scene_type} " + description

            self.logger.debug("Scene type consistency ensured")
            return description

        except Exception as e:
            error_msg = f"Scene type consistency check failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return description

    def extract_perspective_from_description(self, description: str) -> str:
        """
        從原始描述中提取視角信息

        Args:
            description: 原始場景描述

        Returns:
            str: 提取到的視角描述，如果沒有則返回空字符串
        """
        try:
            for persp_type, terms in self.perspective_terms.items():
                for term in terms:
                    if term.lower() in description.lower():
                        self.logger.debug(f"Perspective detected: {term}")
                        return term

            return ""

        except Exception as e:
            self.logger.error(f"Perspective extraction failed: {str(e)}")
            return ""

    def extract_objects_from_description(self, description: str) -> List[str]:
        """
        從原始描述中提取物件提及

        Args:
            description: 原始場景描述

        Returns:
            List[str]: 提取到的物件列表
        """
        try:
            extracted_objects = []

            for pattern in self.number_patterns:
                matches = re.finditer(pattern[0], description, re.IGNORECASE)
                for match in matches:
                    number = match.group(1)
                    object_type = match.group(2)
                    extracted_objects.append(f"{number} {object_type}")

            self.logger.debug(f"Extracted {len(extracted_objects)} objects from description")
            return extracted_objects

        except Exception as e:
            self.logger.error(f"Object extraction failed: {str(e)}")
            return []

    def validate_response_completeness(self, response: str) -> Tuple[bool, str]:
        """
        驗證回應的完整性

        Args:
            response: 待驗證的回應

        Returns:
            Tuple[bool, str]: (是否完整, 問題描述)
        """
        try:
            # 檢查回應長度
            if len(response) < 100:
                return False, "Response too short"

            # 檢查句子結尾
            if len(response) < 200 and "." not in response[-30:]:
                return False, "No proper sentence ending"

            # 檢查不完整短語
            incomplete_phrases = ["in the", "with the", "and the"]
            if any(response.endswith(phrase) for phrase in incomplete_phrases):
                return False, "Ends with incomplete phrase"

            return True, "Response is complete"

        except Exception as e:
            self.logger.error(f"Response completeness validation failed: {str(e)}")
            return False, "Validation error"

    def get_validator_info(self) -> Dict[str, Any]:
        """
        獲取驗證器信息

        Returns:
            Dict[str, Any]: 包含驗證器狀態和配置的信息
        """
        return {
            "location_terms_count": len(self.location_terms),
            "cultural_terms_count": len(self.cultural_terms),
            "perspective_types_count": len(self.perspective_terms),
            "number_patterns_count": len(self.number_patterns),
            "prohibited_words_count": len(self.prohibited_scene_words),
            "initialization_status": "success"
        }

Writing text_quality_validator.py


In [ ]:
# %%writefile llm_enhancer.py
import logging
import traceback
from typing import Dict, List, Any, Optional

# from model_manager import ModelManager
# from prompt_template_manager import PromptTemplateManager
# from response_processor import ResponseProcessor
# from text_quality_validator import TextQualityValidator
# from landmark_data import ALL_LANDMARKS

class LLMEnhancer:
    """
    LLM增強器的主要窗口，協調模型管理、提示模板、回應處理和品質驗證等組件。
    提供統一的接口來處理場景描述增強、檢測結果驗證和無檢測情況處理。
    """

    def __init__(self,
                 model_path: Optional[str] = None,
                 tokenizer_path: Optional[str] = None,
                 device: Optional[str] = None,
                 max_length: int = 2048,
                 temperature: float = 0.3,
                 top_p: float = 0.85):
        """
        初始化LLM增強器門面

        Args:
            model_path: LLM模型的路徑或HuggingFace模型名稱，預設使用Llama 3.2
            tokenizer_path: tokenizer的路徑，通常與model_path相同
            device: 運行設備 ('cpu'或'cuda')，None時自動檢測
            max_length: 輸入文本的最大長度
            temperature: 生成文本的溫度參數
            top_p: 生成文本時的核心採樣機率閾值
        """
        # 設置專屬logger
        self.logger = logging.getLogger(self.__class__.__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

        try:
            # 初始化四個核心組件
            self.model_manager = ModelManager(
                model_path=model_path,
                tokenizer_path=tokenizer_path,
                device=device,
                max_length=max_length,
                temperature=temperature,
                top_p=top_p
            )

            self.prompt_manager = PromptTemplateManager()
            self.response_processor = ResponseProcessor()
            self.quality_validator = TextQualityValidator()

            # 保存模型路徑以供後續使用
            self.model_path = model_path or "meta-llama/Llama-3.2-3B-Instruct"

            self.logger.info("LLMEnhancer facade initialized successfully")

        except Exception as e:
            error_msg = f"Failed to initialize LLMEnhancer facade: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            raise Exception(error_msg) from e

    def enhance_description(self, scene_data: Dict[str, Any]) -> str:
        """
        場景描述增強器主要入口方法，整合所有組件來處理場景描述增強

        Args:
            scene_data: 包含場景資訊的字典，包括原始描述、檢測物件 (含 is_landmark)、
                        場景類型、時間/光線資訊等

        Returns:
            str: 增強後的場景描述
        """
        try:
            self.logger.info("Starting scene description enhancement")

            # 1. 重置模型上下文
            self.model_manager.reset_context()

            # 2. 取出原始描述
            original_desc = scene_data.get("original_description", "")
            if not original_desc:
                self.logger.warning("No original description provided")
                return "No original description provided."

            # 3. 準備物件統計資訊
            object_list = self._prepare_object_statistics(scene_data)
            if not object_list:
                object_keywords = self.quality_validator.extract_objects_from_description(original_desc)
                object_list = ", ".join(object_keywords) if object_keywords else "objects visible in the scene"

            # 4. 檢測地標並準備地標資訊
            landmark_info = self._extract_landmark_info(scene_data)

            # 5. 將地標資訊加入scene_data
            enhanced_scene_data = scene_data.copy()
            if landmark_info:
                enhanced_scene_data["landmark_location_info"] = landmark_info

            # 6. 生成 prompt
            prompt = self.prompt_manager.format_enhancement_prompt_with_landmark(
                scene_data=enhanced_scene_data,
                object_list=object_list,
                original_description=original_desc
            )

            # 7. 生成 LLM 回應
            self.logger.info("Generating LLM response")
            response = self.model_manager.generate_response(prompt)

            # 8. 處理不完整回應（重試機制）
            response = self._handle_incomplete_response(response, prompt, original_desc)

            # 9. 清理 LLM 回應
            model_type = self.model_path
            raw_cleaned = self.response_processor.clean_response(response, model_type)

            # 10. 移除解釋性注釋
            cleaned_response = self.response_processor.remove_explanatory_notes(raw_cleaned)

            # 11. 事實準確性驗證
            try:
                cleaned_response = self.quality_validator.verify_factual_accuracy(
                    original_desc, cleaned_response, object_list
                )
            except Exception:
                self.logger.warning("Fact verification failed; using response without verification")

            # 12. 場景類型一致性確保
            scene_type = scene_data.get("scene_type", "unknown scene")
            word_count = len(cleaned_response.split())
            if word_count >= 5 and scene_type.lower() not in cleaned_response.lower():
                cleaned_response = self.quality_validator.ensure_scene_type_consistency(
                    cleaned_response, scene_type, original_desc
                )

            # 13. 視角一致性處理
            perspective = self.quality_validator.extract_perspective_from_description(original_desc)
            if perspective and perspective.lower() not in cleaned_response.lower():
                cleaned_response = f"{perspective}, {cleaned_response[0].lower()}{cleaned_response[1:]}"

            # 14. 最終驗證：如果結果過短，嘗試fallback
            final_result = cleaned_response.strip()
            if not final_result or len(final_result) < 20:
                self.logger.warning("Enhanced description too short; attempting fallback")

                # Fallback prompt
                fallback_scene_data = enhanced_scene_data.copy()
                fallback_scene_data["is_fallback"] = True
                fallback_prompt = self.prompt_manager.format_enhancement_prompt_with_landmark(
                    scene_data=fallback_scene_data,
                    object_list=object_list,
                    original_description=original_desc
                )

                fallback_resp = self.model_manager.generate_response(fallback_prompt)
                fallback_cleaned = self.response_processor.clean_response(fallback_resp, model_type)
                fallback_cleaned = self.response_processor.remove_explanatory_notes(fallback_cleaned)

                final_result = fallback_cleaned.strip()
                if not final_result or len(final_result) < 20:
                    self.logger.warning("Fallback also insufficient; returning original")
                    return original_desc

            # 15. display enhanced description
            self.logger.info(f"Scene description enhancement completed successfully ({len(final_result)} chars)")
            return final_result

        except Exception as e:
            error_msg = f"Enhancement failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return scene_data.get("original_description", "Unable to enhance description")

    def _extract_landmark_info(self, scene_data: Dict[str, Any]) -> Optional[Dict[str, str]]:
        """
        提取地標資訊，但不構建prompt內容

        Args:
            scene_data: 場景資料字典

        Returns:
            Optional[Dict[str, str]]: 地標資訊字典，包含name和location，如果沒有地標則返回None
        """
        try:
            # 檢查是否有地標
            lm_id_in_data = scene_data.get("landmark_id")
            if not lm_id_in_data:
                # 從檢測物件中尋找地標
                for obj in scene_data.get("detected_objects", []):
                    if obj.get("is_landmark") and obj.get("landmark_id"):
                        lm_id_in_data = obj["landmark_id"]
                        break

            # 如果沒有檢測到地標，返回None
            if not lm_id_in_data:
                return None

            # 從landmark_data.py提取地標資訊
            if lm_id_in_data in ALL_LANDMARKS:
                lm_info = ALL_LANDMARKS[lm_id_in_data]
                landmark_name = scene_data.get("scene_name", lm_info.get("name", lm_id_in_data))
                landmark_location = lm_info.get("location", "")

                if landmark_location:
                    return {
                        "name": landmark_name,
                        "location": landmark_location,
                        "landmark_id": lm_id_in_data
                    }

            return None

        except Exception as e:
            self.logger.error(f"Error extracting landmark info: {str(e)}")
            return None


    def _prepare_object_statistics(self, scene_data: Dict[str, Any]) -> str:
        """
        準備物件統計資訊用於提示詞生成

        Args:
            scene_data: 場景資料字典

        Returns:
            str: 格式化的物件統計資訊
        """
        try:
            # 高信心度閾值
            high_confidence_threshold = 0.65

            # 優先使用預計算的統計資訊
            object_statistics = scene_data.get("object_statistics", {})
            object_counts = {}

            if object_statistics:
                for class_name, stats in object_statistics.items():
                    if stats.get("count", 0) > 0 and stats.get("avg_confidence", 0) >= high_confidence_threshold:
                        object_counts[class_name] = stats["count"]
            else:
                # 回退到原有的計算方式
                detected_objects = scene_data.get("detected_objects", [])
                filtered_objects = []

                for obj in detected_objects:
                    confidence = obj.get("confidence", 0)
                    class_name = obj.get("class_name", "")

                    # 為特殊類別設置更高閾值
                    special_classes = ["airplane", "helicopter", "boat"]
                    if class_name in special_classes:
                        if confidence < 0.75:
                            continue

                    if confidence >= high_confidence_threshold:
                        filtered_objects.append(obj)

                for obj in filtered_objects:
                    class_name = obj.get("class_name", "")
                    if class_name not in object_counts:
                        object_counts[class_name] = 0
                    object_counts[class_name] += 1

            # 格式化物件描述
            return ", ".join([
                f"{count} {obj}{'s' if count > 1 else ''}"
                for obj, count in object_counts.items()
            ])

        except Exception as e:
            self.logger.error(f"Object statistics preparation failed: {str(e)}")
            return "objects visible in the scene"

    def _handle_incomplete_response(self, response: str, prompt: str, original_desc: str) -> str:
        """
        處理不完整的回應，必要時重新生成

        Args:
            response: 原始回應
            prompt: 使用的提示詞
            original_desc: 原始描述

        Returns:
            str: 處理後的回應
        """
        try:
            # 檢查回應完整性
            is_complete, issue = self.quality_validator.validate_response_completeness(response)

            max_retries = 3
            attempts = 0

            while not is_complete and attempts < max_retries:
                self.logger.warning(f"Incomplete response detected ({issue}), retrying... Attempt {attempts+1}/{max_retries}")

                # 重新生成
                response = self.model_manager.generate_response(prompt)
                is_complete, issue = self.quality_validator.validate_response_completeness(response)
                attempts += 1

            if not response or len(response.strip()) < 10:
                self.logger.warning("Generated response was empty or too short, returning original description")
                return original_desc

            return response

        except Exception as e:
            self.logger.error(f"Incomplete response handling failed: {str(e)}")
            return response  # 返回原始回應

    def verify_detection(self,
                        detected_objects: List[Dict],
                        clip_analysis: Dict[str, Any],
                        scene_type: str,
                        scene_name: str,
                        confidence: float) -> Dict[str, Any]:
        """
        驗證並可能修正YOLO的檢測結果

        Args:
            detected_objects: YOLO檢測到的物體列表
            clip_analysis: CLIP分析結果
            scene_type: 識別的場景類型
            scene_name: 場景名稱
            confidence: 場景分類的信心度

        Returns:
            Dict: 包含驗證結果和建議的字典
        """
        try:
            self.logger.info("Starting detection verification")

            # 格式化驗證提示
            prompt = self.prompt_manager.format_verification_prompt(
                detected_objects=detected_objects,
                clip_analysis=clip_analysis,
                scene_type=scene_type,
                scene_name=scene_name,
                confidence=confidence
            )

            # 調用LLM進行驗證
            verification_result = self.model_manager.generate_response(prompt)

            # 清理回應
            cleaned_result = self.response_processor.clean_response(verification_result, self.model_path)

            # 解析驗證結果
            result = {
                "verification_text": cleaned_result,
                "has_errors": "appear accurate" not in cleaned_result.lower(),
                "corrected_objects": None
            }

            self.logger.info("Detection verification completed")
            return result

        except Exception as e:
            error_msg = f"Detection verification failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return {
                "verification_text": "Verification failed",
                "has_errors": False,
                "corrected_objects": None
            }

    def handle_no_detection(self, clip_analysis: Dict[str, Any]) -> str:
        """
        處理YOLO未檢測到物體的情況

        Args:
            clip_analysis: CLIP分析結果

        Returns:
            str: 生成的場景描述
        """
        try:
            self.logger.info("Handling no detection scenario")

            # 格式化無檢測提示
            prompt = self.prompt_manager.format_no_detection_prompt(clip_analysis)

            # 調用LLM生成描述
            description = self.model_manager.generate_response(prompt)

            # 清理回應
            cleaned_description = self.response_processor.clean_response(description, self.model_path)

            self.logger.info("No detection handling completed")
            return cleaned_description

        except Exception as e:
            error_msg = f"No detection handling failed: {str(e)}"
            self.logger.error(error_msg)
            self.logger.error(traceback.format_exc())
            return "Unable to generate scene description"

    def reset_context(self):
        """重置LLM模型上下文"""
        try:
            self.model_manager.reset_context()
            self.logger.info("LLM context reset completed")
        except Exception as e:
            self.logger.error(f"Context reset failed: {str(e)}")

    def get_call_count(self) -> int:
        """
        獲取模型調用次數

        Returns:
            int: 調用次數
        """
        return self.model_manager.get_call_count()

    def get_model_info(self) -> Dict[str, Any]:
        """
        獲取模型和組件資訊

        Returns:
            Dict[str, Any]: 包含所有組件狀態的綜合資訊
        """
        try:
            return {
                "model_manager": self.model_manager.get_model_info(),
                "prompt_manager": self.prompt_manager.get_template_info(),
                "response_processor": self.response_processor.get_processor_info(),
                "quality_validator": self.quality_validator.get_validator_info(),
                "facade_status": "initialized"
            }
        except Exception as e:
            self.logger.error(f"Failed to get component info: {str(e)}")
            return {"facade_status": "error", "error_message": str(e)}

    def is_model_loaded(self) -> bool:
        """
        檢查模型是否已載入

        Returns:
            bool: 模型載入狀態
        """
        return self.model_manager.is_model_loaded()

    def get_current_device(self) -> str:
        """
        獲取當前運行設備

        Returns:
            str: 當前設備名稱
        """
        return self.model_manager.get_current_device()

    def _detect_scene_type(self, detected_objects: List[Dict]) -> str:
        """
        基於物件分佈和模式檢測場景類型

        Args:
            detected_objects: 檢測到的物件列表

        Returns:
            str: 檢測到的場景類型
        """
        try:
            # 預設場景類型
            scene_type = "intersection"

            # 計算物件數量
            object_counts = {}
            for obj in detected_objects:
                class_name = obj.get("class_name", "")
                if class_name not in object_counts:
                    object_counts[class_name] = 0
                object_counts[class_name] += 1

            # 人數統計
            people_count = object_counts.get("person", 0)

            # 交通工具統計
            car_count = object_counts.get("car", 0)
            bus_count = object_counts.get("bus", 0)
            truck_count = object_counts.get("truck", 0)
            total_vehicles = car_count + bus_count + truck_count

            # 簡單的場景類型檢測邏輯
            if people_count > 8 and total_vehicles < 2:
                scene_type = "pedestrian_crossing"
            elif people_count > 5 and total_vehicles > 2:
                scene_type = "busy_intersection"
            elif people_count < 3 and total_vehicles > 3:
                scene_type = "traffic_junction"

            return scene_type

        except Exception as e:
            self.logger.error(f"Scene type detection failed: {str(e)}")
            return "intersection"

Writing llm_enhancer.py


In [ ]:
# %%writefile app.py
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from typing import Dict, List, Any, Optional, Tuple
import cv2
from PIL import Image
import tempfile
import uuid
# import spaces

# from detection_model import DetectionModel
# from color_mapper import ColorMapper
# from evaluation_metrics import EvaluationMetrics
# from style import Style
# from image_processor import ImageProcessor
# from video_processor import VideoProcessor
# from llm_enhancer import LLMEnhancer

# Initialize Processors with LLM support
image_processor = None
video_processor = None

def initialize_processors():
    global image_processor, video_processor

    try:
        print("Attempting to initialize ImageProcessor with LLM support...")
        image_processor = ImageProcessor(use_llm=True, llm_model_path="meta-llama/Llama-3.2-3B-Instruct")
        print("ImageProcessor initialized successfully with LLM")

        # 添加診斷檢查
        if hasattr(image_processor, 'scene_analyzer'):
            if image_processor.scene_analyzer is not None:
                print(f"scene_analyzer initialized: {type(image_processor.scene_analyzer)}")
                if hasattr(image_processor.scene_analyzer, 'use_llm'):
                    print(f"scene_analyzer.use_llm available: {image_processor.scene_analyzer.use_llm}")
            else:
                print("WARNING: scene_analyzer is None after initialization")
        else:
            print("WARNING: scene_analyzer attribute not found in image_processor")

        video_processor = VideoProcessor(image_processor)
        print("VideoProcessor initialized successfully")
        return True

    except Exception as e:
        print(f"Error initializing processors with LLM: {e}")
        import traceback
        traceback.print_exc()

        # Create fallback processor without LLM
        try:
            print("Attempting fallback initialization without LLM...")
            image_processor = ImageProcessor(use_llm=False, enable_places365=False)
            video_processor = VideoProcessor(image_processor)
            print("Fallback processors initialized successfully without LLM and Places365")
            return True

        except Exception as fallback_error:
            print(f"Fatal error: Cannot initialize processors: {fallback_error}")
            import traceback
            traceback.print_exc()
            image_processor = None
            video_processor = None
            return False

# Initialize processors
initialization_success = initialize_processors()
if not initialization_success:
    print("WARNING: Failed to initialize processors. Application may not function correctly.")

# Helper Function
def get_all_classes():
    """Gets all available COCO classes."""
    # Try to get from a loaded model first
    if image_processor and image_processor.model_instances:
         for model_instance in image_processor.model_instances.values():
              if model_instance and model_instance.is_model_loaded:
                   try:
                        # Ensure class_names is a dict {id: name}
                        if isinstance(model_instance.class_names, dict):
                             return sorted([(int(idx), name) for idx, name in model_instance.class_names.items()])
                   except Exception as e:
                        print(f"Error getting class names from model: {e}")

    # Fallback to standard COCO (ensure keys are ints)
    default_classes = {
        0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus',
        6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant',
        11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat',
        16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear',
        22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag',
        27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard',
        32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove',
        36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle',
        40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
        46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
        51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair',
        57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet',
        62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard',
        67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink',
        72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors',
        77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'
    }
    return sorted(default_classes.items())

# @spaces.GPU(duration=180)
def handle_image_upload(image, model_name, confidence_threshold, filter_classes=None, use_llm=True, enable_landmark=True):
    """Processes a single uploaded image."""
    # Enhanced safety check for image_processor
    if image_processor is None:
        error_msg = "Image processor is not initialized. Please restart the application or check system dependencies."
        print(f"ERROR: {error_msg}")

        # Create error plot
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.text(0.5, 0.5, "Initialization Error\nProcessor Not Available",
                color="red", ha="center", va="center", fontsize=14, fontweight="bold")
        ax.axis('off')

        return (None, error_msg, {}, fig, f"<div style='color: red; font-weight: bold;'>Error: {error_msg}</div>",
                "<div style='color: red;'>Error: System not initialized</div>",
                [["System Error"]], [["System Error"]], {}, {"time_of_day": "error", "confidence": 0})

    # Additional safety check for processor attributes
    if not hasattr(image_processor, 'use_llm'):
        error_msg = "Image processor is corrupted. Missing required attributes."
        print(f"ERROR: {error_msg}")

        fig, ax = plt.subplots(figsize=(8, 6))
        ax.text(0.5, 0.5, "Processor Error\nCorrupted State",
                color="red", ha="center", va="center", fontsize=14, fontweight="bold")
        ax.axis('off')

        return (None, error_msg, {}, fig, f"<div style='color: red; font-weight: bold;'>Error: {error_msg}</div>",
                "<div style='color: red;'>Error: Processor corrupted</div>",
                [["Processor Error"]], [["Processor Error"]], {}, {"time_of_day": "error", "confidence": 0})

    print(f"DIAGNOSTIC: Image upload handled with enable_landmark={enable_landmark}, use_llm={use_llm}")
    print(f"Processing image with model: {model_name}, confidence: {confidence_threshold}, use_llm: {use_llm}, enable_landmark: {enable_landmark}")
    try:
        image_processor.use_llm = use_llm

        # 確保 scene_analyzer 不是 None
        if hasattr(image_processor, 'scene_analyzer') and image_processor.scene_analyzer is not None:
            if hasattr(image_processor.scene_analyzer, 'use_llm'):
                image_processor.scene_analyzer.use_llm = use_llm
                print(f"Updated existing scene_analyzer use_llm setting to: {use_llm}")

            # 檢查並設置 landmark detection
            if hasattr(image_processor.scene_analyzer, 'use_landmark_detection'):
                # 設置所有相關標記
                image_processor.scene_analyzer.use_landmark_detection = enable_landmark
                image_processor.scene_analyzer.enable_landmark = enable_landmark

                # 確保處理器也設置了這選項
                image_processor.enable_landmark = enable_landmark

                # 檢查並設置更深層次的組件
                if hasattr(image_processor.scene_analyzer, 'scene_describer') and image_processor.scene_analyzer.scene_describer is not None:
                    image_processor.scene_analyzer.scene_describer.enable_landmark = enable_landmark

                # 檢查並設置CLIP分析器上的標記
                if hasattr(image_processor.scene_analyzer, 'clip_analyzer') and image_processor.scene_analyzer.clip_analyzer is not None:
                    if hasattr(image_processor.scene_analyzer.clip_analyzer, 'enable_landmark'):
                        image_processor.scene_analyzer.clip_analyzer.enable_landmark = enable_landmark

                # 檢查並設置LLM增強器
                if hasattr(image_processor.scene_analyzer, 'llm_enhancer') and image_processor.scene_analyzer.llm_enhancer is not None:
                    if hasattr(image_processor.scene_analyzer.llm_enhancer, 'enable_landmark'):
                        image_processor.scene_analyzer.llm_enhancer.enable_landmark = enable_landmark
                        print(f"Updated LLM enhancer enable_landmark to: {enable_landmark}")

                print(f"Updated all landmark detection settings to: {enable_landmark}")
        else:
            print("WARNING: scene_analyzer is None or not available")
            if hasattr(image_processor, 'enable_landmark'):
                image_processor.enable_landmark = enable_landmark

                # 設置更深層次的組別
                if hasattr(image_processor.scene_analyzer, 'scene_describer'):
                    image_processor.scene_analyzer.scene_describer.enable_landmark = enable_landmark

                # 設置CLIP分析器上的標記
                if hasattr(image_processor.scene_analyzer, 'clip_analyzer'):
                    if hasattr(image_processor.scene_analyzer.clip_analyzer, 'enable_landmark'):
                        image_processor.scene_analyzer.clip_analyzer.enable_landmark = enable_landmark

                # 如果有LLM增強器，也設置它
                if hasattr(image_processor.scene_analyzer, 'llm_enhancer'):
                    image_processor.scene_analyzer.llm_enhancer.enable_landmark = enable_landmark
                    print(f"Updated LLM enhancer enable_landmark to: {enable_landmark}")

                print(f"Updated all landmark detection settings to: {enable_landmark}")

        class_ids_to_filter = None
        if filter_classes:
            class_ids_to_filter = []
            available_classes_dict = dict(get_all_classes())
            name_to_id = {name: id for id, name in available_classes_dict.items()}
            for class_str in filter_classes:
                class_name_or_id = class_str.split(":")[0].strip()
                class_id = -1
                try:
                    class_id = int(class_name_or_id)
                    if class_id not in available_classes_dict:
                        class_id = -1
                except ValueError:
                    if class_name_or_id in name_to_id:
                        class_id = name_to_id[class_name_or_id]
                    elif class_str in name_to_id: # Check full string "id: name"
                        class_id = name_to_id[class_str]

                if class_id != -1:
                    class_ids_to_filter.append(class_id)
                else:
                    print(f"Warning: Could not parse class filter: {class_str}")
            print(f"Filtering image results for class IDs: {class_ids_to_filter}")

        # Call the existing image processing logic
        print(f"DEBUG: app.py 傳遞 enable_landmark={enable_landmark} 到 process_image")
        result_image, result_text, stats = image_processor.process_image(
            image,
            model_name,
            confidence_threshold,
            class_ids_to_filter,
            enable_landmark
        )

        # Format stats for JSON display
        formatted_stats = image_processor.format_json_for_display(stats)

        # Prepare visualization data for the plot
        plot_figure = None
        if stats and "class_statistics" in stats and stats["class_statistics"]:
            available_classes_dict = dict(get_all_classes())
            viz_data = image_processor.prepare_visualization_data(stats, available_classes_dict)
            if "error" not in viz_data:
                 plot_figure = EvaluationMetrics.create_enhanced_stats_plot(viz_data)
            else:
                 fig, ax = plt.subplots(figsize=(8, 6))
                 ax.text(0.5, 0.5, viz_data["error"], ha='center', va='center', fontsize=12)
                 ax.axis('off')
                 plot_figure = fig
        else:
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.text(0.5, 0.5, "No detection data for plot", ha='center', va='center', fontsize=12)
            ax.axis('off')
            plot_figure = fig

        # Extract scene analysis info
        scene_analysis = stats.get("scene_analysis", {})
        scene_desc = scene_analysis.get("description", "Scene analysis requires detected objects.")
        # Ensure scene_desc is a string before adding HTML
        if not isinstance(scene_desc, str):
            scene_desc = str(scene_desc)

        def clean_description(desc):
            if not desc:
                return ""

            # 先過濾問答格式
            if "Questions:" in desc:
                desc = desc.split("Questions:")[0].strip()
            if "Answers:" in desc:
                desc = desc.split("Answers:")[0].strip()

            # 然後按行過濾代碼和其他非敘述內容
            lines = desc.split('\n')
            clean_lines = []
            skip_block = False

            for line in lines:
                # 檢測問題格式
                if re.match(r'^\d+\.\s+(What|How|Why|When|Where|Who|The)', line):
                    continue

                # 檢查需要跳過的行
                if line.strip().startswith(':param') or line.strip().startswith('"""'):
                    continue
                if line.strip().startswith("Exercise") or "class SceneDescriptionSystem" in line:
                    skip_block = True
                    continue
                if ('def generate_scene_description' in line or
                    'def enhance_scene_descriptions' in line or
                    'def __init__' in line):
                    skip_block = True
                    continue
                if line.strip().startswith('#TEST'):
                    skip_block = True
                    continue

                if skip_block and line.strip() == "":
                    skip_block = False

                # 如果不需要跳過
                if not skip_block:
                    clean_lines.append(line)

            cleaned_text = '\n'.join(clean_lines)

            # 如果清理後為空，返回原始描述的第一段作為保險
            if not cleaned_text.strip():
                paragraphs = [p.strip() for p in desc.split('\n\n') if p.strip()]
                if paragraphs:
                    return paragraphs[0]
                return desc

            return cleaned_text

        # 獲取和處理場景描述
        scene_analysis = stats.get("scene_analysis", {})
        print("Processing scene_analysis:", scene_analysis.keys())

        # 獲取原始描述
        scene_desc = scene_analysis.get("description", "Scene analysis requires detected objects.")
        if not isinstance(scene_desc, str):
            scene_desc = str(scene_desc)

        print(f"Original scene description (first 50 chars): {scene_desc[:50]}...")

        # determine original description
        clean_scene_desc = clean_description(scene_desc)
        print(f"Cleaned scene description (first 50 chars): {clean_scene_desc[:50]}...")

        if not clean_scene_desc.strip():
            clean_scene_desc = scene_desc

        scene_desc_html = f"<div>{clean_scene_desc}</div>"

        # 獲取LLM增強描述並且確保設置默認值為空字符串而非 None，不然會有None type Error
        enhanced_description = scene_analysis.get("enhanced_description", "")
        if enhanced_description is None:
            enhanced_description = ""

        if not enhanced_description or not enhanced_description.strip():
            print("WARNING: LLM enhanced description is empty!")

        # bedge & label
        llm_badge = ""
        description_to_show = ""

        # 在 Original Scene Analysis 折疊區顯示原始的描述
        if use_llm and enhanced_description:
            llm_badge = '<span style="display:inline-block; margin-left:8px; padding:3px 10px; border-radius:12px; background: linear-gradient(90deg, #38b2ac, #4299e1); color:white; font-size:0.7rem; font-weight:bold; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); border: 1px solid rgba(255, 255, 255, 0.2);">LLM Enhanced</span>'
            description_to_show = enhanced_description

        else:
            llm_badge = '<span style="display:inline-block; margin-left:8px; padding:3px 10px; border-radius:12px; background-color:#718096; color:white; font-size:0.7rem; font-weight:bold;">Basic</span>'
            description_to_show = clean_scene_desc

        # 使用LLM敘述時會有徽章標籤在標題上
        scene_description_html = f'''
        <div>
            <div class="section-heading" style="font-size:1.2rem; margin-top:15px;">Scene Description {llm_badge}
                <span style="font-size:0.8rem; color:#666; font-weight:normal; display:block; margin-top:2px;">
                    {('(Enhanced by AI language model)' if use_llm and enhanced_description else '(Based on object detection)')}
                </span>
            </div>
            <div style="padding:15px; background-color:#ffffff; border-radius:8px; border:1px solid #e2e8f0; margin-bottom:20px; box-shadow:0 1px 3px rgba(0,0,0,0.05);">
                {description_to_show}
            </div>
        </div>
        '''

        # 原始描述只在使用 LLM 且有增強描述時在折疊區顯示
        original_desc_visibility = "block" if use_llm and enhanced_description else "none"
        original_desc_html = f'''
        <div id="original_scene_analysis_accordion" style="display: {original_desc_visibility};">
            <div style="padding:15px; background-color:#f0f0f0; border-radius:8px; border:1px solid #e2e8f0;">
                {clean_scene_desc}
            </div>
        </div>
        '''

        # Prepare activities list
        activities_list = scene_analysis.get("possible_activities", [])
        if not activities_list:
            activities_list_data = [["No specific activities inferred"]] # Data for Dataframe
        else:
            activities_list_data = [[activity] for activity in activities_list]

        # Prepare safety concerns list
        safety_concerns_list = scene_analysis.get("safety_concerns", [])
        if not safety_concerns_list:
            safety_data = [["No safety concerns detected"]] # Data for Dataframe
        else:
            safety_data = [[concern] for concern in safety_concerns_list]

        zones = scene_analysis.get("functional_zones", {})
        lighting = scene_analysis.get("lighting_conditions", {"time_of_day": "unknown", "confidence": 0})

        # 如果描述為空，記錄警告
        if not clean_scene_desc.strip():
            print("WARNING: Scene description is empty after cleaning!")
        if not enhanced_description.strip():
            print("WARNING: LLM enhanced description is empty!")

        return (result_image, result_text, formatted_stats, plot_figure,
            scene_description_html, original_desc_html,
            activities_list_data, safety_data, zones, lighting)

    except Exception as e:
        print(f"Error in handle_image_upload: {e}")
        import traceback
        error_msg = f"Error processing image: {str(e)}\n{traceback.format_exc()}"
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, "Processing Error", color="red", ha="center", va="center")
        ax.axis('off')
        # Ensure return structure matches outputs even on error
        return (None, error_msg, {}, fig, f"<div>Error: {str(e)}</div>", "Error",
            [["Error"]], [["Error"]], {}, {"time_of_day": "error", "confidence": 0})

def download_video_from_url(video_url, max_duration_minutes=10):
    """
    Downloads a video from a YouTube URL and returns the local path to the downloaded file.

    Args:
        video_url (str): URL of the YouTube video to download
        max_duration_minutes (int): Maximum allowed video duration in minutes

    Returns:
        tuple: (Path to the downloaded video file or None, Error message or None)
    """
    try:
        # Create a temporary directory to store the video
        temp_dir = tempfile.gettempdir()
        output_filename = f"downloaded_{uuid.uuid4().hex}.mp4"
        output_path = os.path.join(temp_dir, output_filename)

        # Check if it's a YouTube URL
        if "youtube.com" in video_url or "youtu.be" in video_url:
            # Import yt-dlp here to avoid dependency if not needed
            import yt_dlp

            # Setup yt-dlp options
            ydl_opts = {
                'format': 'best[ext=mp4]/best',  # Best quality MP4 or best available format
                'outtmpl': output_path,
                'noplaylist': True,
                'quiet': False,  # Set to True to reduce output
                'no_warnings': False,
            }

            # First extract info to check duration
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                print(f"Extracting info from YouTube URL: {video_url}")
                info_dict = ydl.extract_info(video_url, download=False)

                # Check if video exists
                if not info_dict:
                    return None, "Could not retrieve video information. Please check the URL."

                video_title = info_dict.get('title', 'Unknown Title')
                duration = info_dict.get('duration', 0)

                print(f"Video title: {video_title}")
                print(f"Video duration: {duration} seconds")

                # Check video duration
                if duration > max_duration_minutes * 60:
                    return None, f"Video is too long ({duration} seconds). Maximum duration is {max_duration_minutes} minutes."

                # Download the video
                print(f"Downloading YouTube video: {video_title}")
                ydl.download([video_url])

            # Verify the file exists and has content
            if not os.path.exists(output_path) or os.path.getsize(output_path) == 0:
                return None, "Download failed: Empty or missing file."

            print(f"Successfully downloaded video to: {output_path}")
            return output_path, None
        else:
            return None, "Only YouTube URLs are supported at this time. Please enter a valid YouTube URL."

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        print(f"Error downloading video: {e}\n{error_details}")
        return None, f"Error downloading video: {str(e)}"


# @spaces.GPU
def handle_video_upload(video_input, video_url, input_type, model_name, confidence_threshold, process_interval):
    """Handles video upload or URL input and calls the VideoProcessor."""

    print(f"Received video request: input_type={input_type}")
    video_path = None

    # Handle based on input type
    if input_type == "upload" and video_input:
        print(f"Processing uploaded video file")
        video_path = video_input
    elif input_type == "url" and video_url:
        print(f"Processing video from URL: {video_url}")
        # Download video from URL
        video_path, error_message = download_video_from_url(video_url)
        if error_message:
            error_html = f"<div class='video-summary-content-wrapper'><pre>{error_message}</pre></div>"
            return None, error_html, {"error": error_message}
    else:
        print("No valid video input provided.")
        return None, "<div class='video-summary-content-wrapper'><pre>Please upload a video file or provide a valid video URL.</pre></div>", {}

    print(f"Starting video processing with: model={model_name}, confidence={confidence_threshold}, interval={process_interval}")
    try:
        # Call the VideoProcessor method
        output_video_path, summary_text, stats_dict = video_processor.process_video_file(
            video_path=video_path,
            model_name=model_name,
            confidence_threshold=confidence_threshold,
            process_interval=int(process_interval) # Ensure interval is int
        )
        print(f"Video processing function returned: path={output_video_path}, summary length={len(summary_text)}")

        # Wrap processing summary in HTML tags for consistent styling with scene understanding page
        summary_html = f"<div class='video-summary-content-wrapper'><pre>{summary_text}</pre></div>"

        # Format statistics for better display
        formatted_stats = {}
        if stats_dict and isinstance(stats_dict, dict):
            formatted_stats = stats_dict

        return output_video_path, summary_html, formatted_stats

    except Exception as e:
        print(f"Error in handle_video_upload: {e}")
        import traceback
        error_msg = f"Error processing video: {str(e)}\n{traceback.format_exc()}"
        error_html = f"<div class='video-summary-content-wrapper'><pre>{error_msg}</pre></div>"
        return None, error_html, {"error": str(e)}


# Create Gradio Interface
def create_interface():
    """Creates the Gradio interface with Tabs."""
    css = Style.get_css()
    available_models = DetectionModel.get_available_models()
    model_choices = [model["model_file"] for model in available_models]
    class_choices_formatted = [f"{id}: {name}" for id, name in get_all_classes()] # Use formatted choices

    with gr.Blocks(css=css, theme=gr.themes.Soft(primary_hue="teal", secondary_hue="blue")) as demo:

        # Header
        with gr.Group(elem_classes="app-header"):
              gr.HTML("""
                    <div style="text-align: center; width: 100%; padding: 2rem 0 3rem 0; background: linear-gradient(135deg, #f0f9ff, #e1f5fe);">
                        <h1 style="font-size: 3.5rem; margin-bottom: 0.5rem; background: linear-gradient(90deg, #38b2ac, #4299e1); -webkit-background-clip: text; -webkit-text-fill-color: transparent; font-weight: bold; font-family: 'Arial', sans-serif;">VisionScout</h1>
                        <h2 style="color: #4A5568; font-size: 1.2rem; font-weight: 400; margin-top: 0.5rem; margin-bottom: 1.5rem; font-family: 'Arial', sans-serif;">Object Detection and Scene Understanding</h2>
                        <div style="display: flex; justify-content: center; gap: 10px; margin: 0.5rem 0;"><div style="height: 3px; width: 80px; background: linear-gradient(90deg, #38b2ac, #4299e1);"></div></div>
                        <div style="display: flex; justify-content: center; gap: 25px; margin-top: 1.5rem;">
                            <div style="padding: 8px 15px; border-radius: 20px; background: rgba(66, 153, 225, 0.15); color: #2b6cb0; font-weight: 500; font-size: 0.9rem;"><span style="margin-right: 6px;">🖼️</span> Image Analysis</div>
                            <div style="padding: 8px 15px; border-radius: 20px; background: rgba(56, 178, 172, 0.15); color: #2b6cb0; font-weight: 500; font-size: 0.9rem;"><span style="margin-right: 6px;">🎬</span> Video Analysis</div>
                        </div>
                         <div style="margin-top: 20px; padding: 10px 15px; background-color: rgba(255, 248, 230, 0.9); border-left: 3px solid #f6ad55; border-radius: 6px; max-width: 600px; margin-left: auto; margin-right: auto; text-align: left;">
                             <p style="margin: 0; font-size: 0.9rem; color: #805ad5; font-weight: 500;">
                                 <span style="margin-right: 5px;">📱</span> iPhone users: HEIC images may not be supported.
                                 <a href="https://cloudconvert.com/heic-to-jpg" target="_blank" style="color: #3182ce; text-decoration: underline;">Convert HEIC to JPG</a> before uploading if needed.
                             </p>
                         </div>
                    </div>
                """)

        # Main Content with Tabs
        with gr.Tabs(elem_classes="tabs"):

            # Tab 1: Image Processing
            with gr.Tab("Image Processing"):
                current_image_model = gr.State("yolov8m.pt") # State for image model selection
                with gr.Row(equal_height=False): # Allow columns to have different heights
                    # Left Column: Image Input & Controls
                    with gr.Column(scale=4, elem_classes="input-panel"):
                        with gr.Group():
                            gr.HTML('<div class="section-heading">Upload Image</div>')
                            image_input = gr.Image(type="pil", label="Upload an image", elem_classes="upload-box")

                            with gr.Accordion("Image Analysis Settings", open=False):
                                image_model_dropdown = gr.Dropdown(
                                    choices=model_choices,
                                    value="yolov8m.pt", # Default for images
                                    label="Select Model",
                                    info="Choose speed vs. accuracy (n=fast, m=balanced, x=accurate)"
                                )
                                # Display model info
                                image_model_info = gr.Markdown(DetectionModel.get_model_description("yolov8m.pt"))

                                image_confidence = gr.Slider(
                                    minimum=0.1, maximum=0.9, value=0.25, step=0.05,
                                    label="Confidence Threshold",
                                    info="Minimum confidence for displaying a detected object"
                                )

                                use_llm = gr.Checkbox(
                                    label="Use LLM for enhanced scene descriptions",
                                    value=True,
                                    info="Provides more detailed and natural language descriptions (may increase processing time)"
                                )

                                use_landmark_detection = gr.Checkbox(
                                    label="Use CLIP for Landmark Detection",
                                    value=False,
                                    info="Detect famous landmarks, monuments, and tourist attractions that standard object detection cannot recognize (increases processing time)"
                                )

                                with gr.Accordion("Filter Classes", open=False):
                                     gr.HTML('<div class="section-heading" style="font-size: 1rem;">Common Categories</div>')
                                     with gr.Row():
                                         people_btn = gr.Button("People", size="sm")
                                         vehicles_btn = gr.Button("Vehicles", size="sm")
                                         animals_btn = gr.Button("Animals", size="sm")
                                         objects_btn = gr.Button("Common Objects", size="sm")
                                     image_class_filter = gr.Dropdown(
                                         choices=class_choices_formatted, # Use formatted choices
                                         multiselect=True,
                                         label="Select Classes to Display",
                                         info="Leave empty to show all detected objects"
                                     )

                        image_detect_btn = gr.Button("Analyze Image", variant="primary", elem_classes="detect-btn")

                        with gr.Group(elem_classes="how-to-use"):
                             gr.HTML('<div class="section-heading">How to Use (Image)</div>')
                             gr.Markdown("""
                                1. Upload an image or use the camera
                                2. *(Optional)* Adjust settings like confidence threshold or model size (n, m = balanced, x = accurate)
                                3. In **Analysis Settings**, you can:
                                    * Uncheck **Use LLM** to skip enhanced descriptions (faster)
                                    * Check **Use CLIP for Landmark Detection** to identify famous landmarks like museums, monuments, and tourist attractions *(may take longer)*
                                    * Filter object classes to focus on specific types of objects *(optional)*
                                4. Click **Analyze Image** button

                                **💡 Tip:** For landmark recognition (e.g. Louvre Museum), make sure to enable **CLIP for Landmark Detection** in the settings above.
                                """)


                        # Image Examples
                        gr.Examples(
                            examples=[
                                "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/room_01.jpg",
                                "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/street_04.jpg",
                                "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/street_05.jpg",
                                "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/landmark_Louvre_01.jpg",
                                ],
                            inputs=image_input,
                            label="Example Images"
                         )

                        gr.HTML("""
                            <div style="text-align: center; margin-top: 8px; padding: 6px; background-color: #f8f9fa; border-radius: 4px; border: 1px solid #e2e8f0;">
                                <p style="font-size: 12px; color: #718096; margin: 0;">
                                    📷 Sample images sourced from <a href="https://unsplash.com" target="_blank" style="color: #3182ce; text-decoration: underline;">Unsplash</a>
                                </p>
                            </div>
                        """)

                    # Right Column: Image Results
                    with gr.Column(scale=6, elem_classes="output-panel"):
                        with gr.Tabs(elem_classes="tabs"):
                            with gr.Tab("Detection Result"):
                                image_result_image = gr.Image(type="pil", label="Detection Result")
                                gr.HTML('<div class="section-heading">Detection Details</div>')
                                image_result_text = gr.Textbox(label=None, lines=10, elem_id="detection-details", container=False)

                            with gr.Tab("Scene Understanding"):
                                gr.HTML('<div class="section-heading">Scene Analysis</div>')
                                gr.HTML("""
                                    <details class="info-details" style="margin: 5px 0 15px 0;">
                                        <summary style="padding: 8px; background-color: #f0f7ff; border-radius: 6px; border-left: 3px solid #4299e1; font-weight: bold; cursor: pointer; color: #2b6cb0;">
                                            🔍 The AI Vision Scout Report: Click for important notes about this analysis
                                        </summary>
                                        <div style="margin-top: 8px; padding: 10px; background-color: #f8f9fa; border-radius: 6px; border: 1px solid #e2e8f0;">
                                            <p style="font-size: 13px; color: #718096; margin: 0;">
                                                <b>About this analysis:</b> This analysis is the model's best guess based on visible objects.
                                                Like human scouts, it sometimes gets lost or sees things that aren't there (but don't we all?).
                                                Consider this an educated opinion rather than absolute truth. For critical applications, always verify with human eyes! 🧐
                                            </p>
                                        </div>
                                    </details>
                                """)

                                gr.HTML('''
                                    <div style="margin-top: 5px; padding: 6px 10px; background-color: #f0f9ff; border-radius: 4px; border-left: 3px solid #63b3ed; font-size: 12px; margin-bottom: 10px;">
                                        <p style="margin: 0; color: #4a5568;">
                                            <b>Note:</b> AI descriptions may vary slightly with each generation, reflecting the creative nature of AI. This is similar to how a person might use different words each time they describe the same image. Processing time may be longer during first use or when analyzing complex scenes, as the LLM enhancement requires additional computational resources.
                                        </p>
                                    </div>
                                    ''')
                                image_scene_description_html = gr.HTML(label=None, elem_id="scene_analysis_description_text")

                                # 使用LLM增強敘述時也會顯示原本敘述內容
                                with gr.Accordion("Original Scene Analysis", open=False, elem_id="original_scene_analysis_accordion"):
                                    image_llm_description = gr.HTML(label=None, elem_id="original_scene_description_text")

                                with gr.Row():
                                     with gr.Column(scale=1):
                                         gr.HTML('<div class="section-heading" style="font-size:1rem; text-align:left;">Possible Activities</div>')
                                         image_activities_list = gr.Dataframe(headers=["Activity"], datatype=["str"], row_count=5, col_count=1, wrap=True)

                                     with gr.Column(scale=1):
                                         gr.HTML('<div class="section-heading" style="font-size:1rem; text-align:left;">Safety Concerns</div>')
                                         image_safety_list = gr.Dataframe(headers=["Concern"], datatype=["str"], row_count=5, col_count=1, wrap=True)

                                gr.HTML('<div class="section-heading">Functional Zones</div>')
                                image_zones_json = gr.JSON(label=None, elem_classes="json-box")

                                gr.HTML('<div class="section-heading">Lighting Conditions</div>')
                                image_lighting_info = gr.JSON(label=None, elem_classes="json-box")

                            with gr.Tab("Statistics"):
                                with gr.Row():
                                    with gr.Column(scale=3, elem_classes="plot-column"):
                                        gr.HTML('<div class="section-heading">Object Distribution</div>')
                                        image_plot_output = gr.Plot(label=None, elem_classes="large-plot-container")
                                    with gr.Column(scale=2, elem_classes="stats-column"):
                                        gr.HTML('<div class="section-heading">Detection Statistics</div>')
                                        image_stats_json = gr.JSON(label=None, elem_classes="enhanced-json-display")

            # Tab 2: Video Processing
            with gr.Tab("Video Processing"):
                with gr.Row(equal_height=False):
                    # Left Column: Video Input & Controls
                    with gr.Column(scale=4, elem_classes="input-panel"):
                        with gr.Group():
                            gr.HTML('<div class="section-heading">Video Input</div>')

                            # Add input type selection
                            video_input_type = gr.Radio(
                                ["upload", "url"],
                                label="Input Method",
                                value="upload",
                                info="Choose how to provide the video"
                            )

                            # File upload (will be shown/hidden based on selection)
                            with gr.Group(elem_id="upload-video-group"):
                                video_input = gr.Video(
                                    label="Upload a video file (MP4, AVI, MOV)",
                                    sources=["upload"],
                                    visible=True
                                )

                            # URL input (will be shown/hidden based on selection)
                            with gr.Group(elem_id="url-video-group"):
                                video_url_input = gr.Textbox(
                                    label="Enter video URL (YouTube or direct video link)",
                                    placeholder="https://www.youtube.com/watch?v=...",
                                    visible=False,
                                    elem_classes="custom-video-url-input"
                                )
                                gr.HTML("""
                                    <div style="padding: 8px; margin-top: 5px; background-color: #fff8f8; border-radius: 4px; border-left: 3px solid #f87171; font-size: 12px;">
                                        <p style="margin: 0; color: #4b5563;">
                                            Note: Currently only YouTube URLs are supported. Maximum video duration is 10 minutes. Due to YouTube's anti-bot protection, some videos may not be downloadable. For protected videos, please upload a local video file instead.
                                        </p>
                                    </div>
                                """)

                            with gr.Accordion("Video Analysis Settings", open=True):
                                video_model_dropdown = gr.Dropdown(
                                    choices=model_choices,
                                    value="yolov8n.pt", # Default 'n' for video
                                    label="Select Model (Video)",
                                    info="Faster models (like 'n') are recommended"
                                )
                                video_confidence = gr.Slider(
                                    minimum=0.1, maximum=0.9, value=0.4, step=0.05,
                                    label="Confidence Threshold (Video)"
                                )
                                video_process_interval = gr.Slider(
                                    minimum=1, maximum=60, value=10, step=1, # Allow up to 60 frame interval
                                    label="Processing Interval (Frames)",
                                    info="Analyze every Nth frame (higher value = faster)"
                                )
                        video_process_btn = gr.Button("Process Video", variant="primary", elem_classes="detect-btn")

                        with gr.Group(elem_classes="how-to-use"):
                            gr.HTML('<div class="section-heading">How to Use (Video)</div>')
                            gr.Markdown("""
                            1. Choose your input method: Upload a file or enter a URL.
                            2. Adjust settings if needed (using a faster model and larger interval is recommended for longer videos).
                            3. Click "Process Video". **Processing can take a significant amount of time.**
                            4. The annotated video and summary will appear on the right when finished.
                            """)

                        # Add video examples
                        gr.HTML('<div class="section-heading">Example Videos</div>')
                        gr.HTML("""
                            <div style="padding: 10px; background-color: #f0f7ff; border-radius: 6px; margin-bottom: 15px;">
                                <p style="font-size: 14px; color: #4A5568; margin: 0;">
                                    Upload any video containing objects that YOLO can detect. For testing, find sample videos
                                    <a href="https://www.pexels.com/search/videos/street/" target="_blank" style="color: #3182ce; text-decoration: underline;">here</a>.
                                </p>
                            </div>
                        """)

                    # Right Column: Video Results
                    with gr.Column(scale=6, elem_classes="output-panel video-result-panel"):
                        gr.HTML("""
                            <div class="section-heading">Video Result</div>
                            <details class="info-details" style="margin: 5px 0 15px 0;">
                                <summary style="padding: 8px; background-color: #f0f7ff; border-radius: 6px; border-left: 3px solid #4299e1; font-weight: bold; cursor: pointer; color: #2b6cb0;">
                                    🎬 Video Processing Notes
                                </summary>
                                <div style="margin-top: 8px; padding: 10px; background-color: #f8f9fa; border-radius: 6px; border: 1px solid #e2e8f0;">
                                    <p style="font-size: 13px; color: #718096; margin: 0;">
                                        The processed video includes bounding boxes around detected objects. For longer videos,
                                        consider using a faster model (like YOLOv8n) and a higher frame interval to reduce processing time.
                                    </p>
                                </div>
                            </details>
                        """)
                        video_output = gr.Video(label="Processed Video", elem_classes="video-output-container") # Output for the processed video file

                        gr.HTML('<div class="section-heading">Processing Summary</div>')
                        # 使用HTML顯示影片的摘要
                        video_summary_text = gr.HTML(
                            label=None,
                            elem_id="video-summary-html-output"
                        )

                        gr.HTML('<div class="section-heading">Aggregated Statistics</div>')
                        video_stats_json = gr.JSON(label=None, elem_classes="video-stats-display") # Display statistics

        # Event Listeners
        # Image Model Change Handler
        image_model_dropdown.change(
            fn=lambda model: (model, DetectionModel.get_model_description(model)),
            inputs=[image_model_dropdown],
            outputs=[current_image_model, image_model_info] # Update state and description
        )

        # Image Filter Buttons
        available_classes_list = get_all_classes() # Get list of (id, name)
        people_classes_ids = [0]
        vehicles_classes_ids = [1, 2, 3, 4, 5, 6, 7, 8]
        animals_classes_ids = list(range(14, 24))
        common_objects_ids = [39, 41, 42, 43, 44, 45, 56, 57, 60, 62, 63, 67, 73] # Bottle, cup, fork, knife, spoon, bowl, chair, couch, table, tv, laptop, phone, book

        people_btn.click(lambda: [f"{id}: {name}" for id, name in available_classes_list if id in people_classes_ids], outputs=image_class_filter)
        vehicles_btn.click(lambda: [f"{id}: {name}" for id, name in available_classes_list if id in vehicles_classes_ids], outputs=image_class_filter)
        animals_btn.click(lambda: [f"{id}: {name}" for id, name in available_classes_list if id in animals_classes_ids], outputs=image_class_filter)
        objects_btn.click(lambda: [f"{id}: {name}" for id, name in available_classes_list if id in common_objects_ids], outputs=image_class_filter)

        video_input_type.change(
            fn=lambda input_type: [
                # Show/hide file upload
                gr.update(visible=(input_type == "upload")),
                # Show/hide URL input
                gr.update(visible=(input_type == "url"))
            ],
            inputs=[video_input_type],
            outputs=[video_input, video_url_input]
        )

        image_detect_btn.click(
            fn=handle_image_upload,
            inputs=[image_input, image_model_dropdown, image_confidence, image_class_filter, use_llm, use_landmark_detection ],
            outputs=[
                image_result_image, image_result_text, image_stats_json, image_plot_output,
                image_scene_description_html, image_llm_description, image_activities_list, image_safety_list, image_zones_json,
                image_lighting_info
            ]
        )

        video_process_btn.click(
            fn=handle_video_upload,
            inputs=[
                video_input,
                video_url_input,
                video_input_type,
                video_model_dropdown,
                video_confidence,
                video_process_interval
            ],
            outputs=[video_output, video_summary_text, video_stats_json]
        )

        # Footer
        gr.HTML("""
            <div class="footer" style="padding: 25px 0; text-align: center; background: linear-gradient(to right, #f5f9fc, #e1f5fe); border-top: 1px solid #e2e8f0; margin-top: 30px;">
                <div style="margin-bottom: 15px;">
                    <p style="font-size: 14px; color: #4A5568; margin: 5px 0;">Powered by YOLOv8, CLIP, Places365, Meta Llama3.2 and Ultralytics • Created with Gradio</p>
                </div>
                <div style="display: flex; align-items: center; justify-content: center; gap: 20px; margin-top: 15px;">
                    <p style="font-family: 'Arial', sans-serif; font-size: 14px; font-weight: 500; letter-spacing: 2px; background: linear-gradient(90deg, #38b2ac, #4299e1); -webkit-background-clip: text; -webkit-text-fill-color: transparent; margin: 0; text-transform: uppercase; display: inline-block;">EXPLORE THE CODE →</p>
                    <a href="https://github.com/Eric-Chung-0511/Learning-Record/tree/main/Data%20Science%20Projects/VisionScout" target="_blank" style="text-decoration: none;">
                        <img src="https://img.shields.io/badge/GitHub-VisionScout-4299e1?logo=github&style=for-the-badge">
                    </a>
                </div>
            </div>
        """)

    return demo


if __name__ == "__main__":
    demo_interface = create_interface()

    demo_interface.launch(debug=True)

Writing app.py


In [ ]:
# %%writefile requirements.txt
# torch>=2.0.0
# torchvision>=0.15.0
# ultralytics>=8.0.0
# opencv-python>=4.7.0
# pillow>=9.4.0
# numpy>=1.23.5
# matplotlib>=3.7.0
# gradio>=3.32.0
# git+https://github.com/openai/CLIP.git
# yt-dlp>=2023.3.4
# requests>=2.28.1
# transformers
# accelerate
# bitsandbytes
# sentencepiece
# huggingface_hub>=0.19.0
# urllib3>=1.26.0

Writing requirements.txt


In [ ]:
from google.colab import files

# files.download('detection_model.py')
# files.download('color_mapper.py')
# files.download('visualization_helper.py')
# files.download('evaluation_metrics.py')
# files.download('style.py')
# files.download('scene_type.py')
# files.download('confidence_templates.py')
# files.download('scene_detail_templates.py')
# files.download('object_template_fillers.py')
# files.download('safety_templates.py')
# files.download('activity_templates.py')
# files.download('object_categories.py')
# files.download('lighting_conditions.py')
# files.download('viewpoint_templates.py')
# files.download('cultural_templates.py')
# files.download('region_analyzer.py')
# files.download('object_extractor.py')
# files.download('scene_viewpoint_analyzer.py')
# files.download('zone_evaluator.py')
# files.download('scene_zone_identifier.py')
# files.download('functional_zone_identifier.py')
# files.download('spatial_analyzer.py')
# files.download('places365_model.py')
# files.download('viewpoint_detector.py')
# files.download('template_manager.py')
# files.download('object_description_generator.py')
# files.download('cultural_context_analyzer.py')
# files.download('text_formatter.py')
# files.download('enhanced_scene_describer.py')
# files.download('configuration_manager.py')
# files.download('feature_extractor.py')
# files.download('indoor_outdoor_classifier.py')
# files.download('lighting_condition_analyzer.py')
# files.download('lighting_analyzer.py')
# files.download('scene_description.py')
# files.download('clip_prompts.py')
# files.download('clip_analyzer.py')
# files.download('landmark_data.py')
# files.download('landmark_activities.py')
# files.download('clip_model_manager.py')
# files.download('landmark_data_manager.py')
# files.download('image_analyzer.py')
# files.download('confidence_manager.py')
# files.download('result_cache_manager.py')
# files.download('clip_zero_shot_classifier.py')
# files.download('component_initializer.py')
# files.download('scene_scoring_engine.py')
# files.download('landmark_processing_manager.py')
# files.download('scene_analysis_coordinator.py')
# files.download('scene_analyzer.py')
# files.download('image_processor.py')
# files.download('video_processor.py')
# files.download('model_manager.py')
# files.download('prompt_template_manager.py')
# files.download('response_processor.py')
# files.download('text_quality_validator.py')
# files.download('llm_enhancer.py')
# files.download('app.py')

# files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#    examples=[
#                                 "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/room_01.jpg",
#                                 "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/street_04.jpg",
#                                 "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/street_05.jpg",
#                                 "/content/drive/Othercomputers/我的 MacBook Pro/Learning/VisionScout/test_images/landmark_Louvre_01.jpg",
#                                 ],